A cute little demo showing the simplest usage of minGPT. Configured to run fine on Macbook Air in like a minute.

In [ ]:
classdef CrossCurrencySwapRFRHelper < H_RootObject
    % FixedRateLeg
    % properties: fixedSchedule 
    
    properties(SetAccess = public)
        
        rateType
        
        rate
        rateTenor
        
        gearing
        spread
        
%         domesticIndex
        exchangeRate

        domesticFixedLegTenor
        domesticFixedLegConvention
        domesticFixedLegDayCounter
        domesticFixedLegCalendar
        
        
        foreignFloatingLegTenor
        foreignFloatingLegConvention
        foreignFloatingLegDayCounter
        foreignFloatingLegCalendar
        
        foreignSpread
        
        spotLag
        payLag

        payLagNotionalMaturityFlag
        
        foreignIndex
        
        evaluationDate
        
        crossCurrencySwap
%         domesticDCurve
        discountCurve
        foreignDCurve
        
        earliestDate
        latestDate
        
    end
    
    methods
        % constructor
        function crossCurrencySwapHelper = CrossCurrencySwapRFRHelper(params)
                if nargin > 0
                    crossCurrencySwapHelper.rateType = 'CrossCurrencySwap';
                    
                    crossCurrencySwapHelper.rate = params('rate');
                    crossCurrencySwapHelper.rateTenor = params('rateTenor');
                    
                    crossCurrencySwapHelper.exchangeRate = params('exchangeRate');
%                     crossCurrencySwapHelper.domesticIndex = params('domesticIndex');

                    crossCurrencySwapHelper.domesticFixedLegTenor = params('domesticFixedLegTenor');
                    crossCurrencySwapHelper.domesticFixedLegConvention = params('domesticFixedLegConvention');
                    crossCurrencySwapHelper.domesticFixedLegDayCounter = params('domesticFixedLegDayCounter');
                    crossCurrencySwapHelper.domesticFixedLegCalendar = params('domesticFixedLegCalendar');
                    
                    
                    crossCurrencySwapHelper.foreignFloatingLegTenor = params('foreignFloatingLegTenor');
                    crossCurrencySwapHelper.foreignFloatingLegConvention = params('foreignFloatingLegConvention');
                    crossCurrencySwapHelper.foreignFloatingLegDayCounter = params('foreignFloatingLegDayCounter');
                    crossCurrencySwapHelper.foreignFloatingLegCalendar = params('foreignFloatingLegCalendar');
                    
                    crossCurrencySwapHelper.foreignSpread = params('foreignSpread');
                    
                    crossCurrencySwapHelper.spotLag = params('spotLag');
                    crossCurrencySwapHelper.payLag = params('payLag');
%                     crossCurrencySwapHelper.payLagNotionalMaturity = params('payLagNotionalMaturity');
                    
                    if isKey(params,'payLagNotionalMaturityFlag')
                        crossCurrencySwapHelper.payLagNotionalMaturityFlag = params('payLagNotionalMaturityFlag');
                    else
                        crossCurrencySwapHelper.payLagNotionalMaturityFlag = true;
                    end
                    
                    
                    crossCurrencySwapHelper.foreignIndex = params('foreignIndex');
                    
                    crossCurrencySwapHelper.evaluationDate = crossCurrencySwapHelper.foreignIndex.forwardCurve.asOfDate;
                    
%                     crossCurrencySwapHelper.domesticDCurve = params('domesticDCurve');
                    crossCurrencySwapHelper.discountCurve = params('discountCurve');
                    
                    % can be generalized later for dual curve input
                    
%                     crossCurrencySwapHelper.foreignDCurve = crossCurrencySwapHelper.foreignIndex.forwardCurve;
                    crossCurrencySwapHelper.foreignDCurve = params('foreignDCurve');
                    
                    if crossCurrencySwapHelper.discountCurve.asOfDate ~= crossCurrencySwapHelper.foreignIndex.forwardCurve.asOfDate
                        error('forwardCurve asOfDate is different from discountCurve asOfDate')
                    end
                    
                    crossCurrencySwapHelper.crossCurrencySwap = MakeCrossCurrencySwapRFR(crossCurrencySwapHelper);
                
                    out = InitializeDates(crossCurrencySwapHelper);

                    crossCurrencySwapHelper.earliestDate = out.earliestDate;
                    crossCurrencySwapHelper.latestDate = out.latestDate;
                end
        end
        
        function out = MakeCrossCurrencySwapRFR(crossCurrencySwapHelper)
%             domesticIndex = crossCurrencySwapHelper.domesticIndex;
            foreignIndex = crossCurrencySwapHelper.foreignIndex;
            
            domesticCalendar = crossCurrencySwapHelper.domesticFixedLegCalendar;
            foreignCalendar = crossCurrencySwapHelper.foreignFloatingLegCalendar;
            
            calcCalendar = JointCalendar({domesticCalendar,foreignCalendar});
            
%             fixingDays = max(domesticIndex.fixingDays,foreignIndex.fixingDays);
%             fixingDays = foreignIndex.fixingDays;
            
            referenceDate = foreignIndex.forwardCurve.asOfDate;
            
            startDate = calcCalendar.Advance(referenceDate,'Following',...
                                                crossCurrencySwapHelper.spotLag,'day',false);
             useEndOfMonth = false;
             if IsEndOfMonth(startDate)
                 useEndOfMonth = true;
             end
             
            endDate = AddTenor(startDate,crossCurrencySwapHelper.rateTenor);
            % fixed & floating schedule
            domesticFixedLegTenor = crossCurrencySwapHelper.domesticFixedLegTenor;
            domesticFixedLegConvention = crossCurrencySwapHelper.domesticFixedLegConvention;
            
            % fixed & floating schedule
             domesticSchedule = ScheduleWTermConvention(startDate,endDate,domesticFixedLegTenor,calcCalendar,...
                                        domesticFixedLegConvention,domesticFixedLegConvention, 'Backward',useEndOfMonth);
                                    
             
             foreignFloatingLegTenor = crossCurrencySwapHelper.foreignFloatingLegTenor;
             foreignFloatingLegConvention = crossCurrencySwapHelper.foreignFloatingLegConvention;

             foreignSchedule = ScheduleWTermConvention(startDate,endDate,foreignFloatingLegTenor,calcCalendar,...
                                        foreignFloatingLegConvention,foreignFloatingLegConvention, 'Backward',useEndOfMonth);

             type = 'Payer';
             domesticNominal = crossCurrencySwapHelper.exchangeRate;
%              domesticFixedRate = crossCurrencySwapHelper.rate;
             domesticFixedRate = 0.0;
             domesticDayCounter = crossCurrencySwapHelper.domesticFixedLegDayCounter;

             foreignNominal = 1.0;
             foreignGearing = 1.0;
        %      foreignSpread = 0.0;
             foreignSpread = crossCurrencySwapHelper.foreignSpread;

             foreignDayCounter = crossCurrencySwapHelper.foreignFloatingLegDayCounter;

             notionalExchangeType = 'Both';
             
             spotLag = crossCurrencySwapHelper.spotLag;
             payLag = crossCurrencySwapHelper.payLag;
             payLagNotionalMaturityFlag = crossCurrencySwapHelper.payLagNotionalMaturityFlag;
             
             crossCurrencySwapRFR = CrossCurrencySwapRFR(type,domesticNominal,domesticSchedule,domesticFixedRate,domesticDayCounter, ...
                                               foreignNominal,foreignSchedule,foreignGearing,foreignSpread,foreignDayCounter,...
                                               foreignIndex,...
                                               notionalExchangeType,spotLag,payLag,payLagNotionalMaturityFlag);
                                  
             out = crossCurrencySwapRFR;
        end
        
        function out = InitializeDates(crossCurrencySwapHelper)
            
            %earliestDate
            earliestDate = crossCurrencySwapHelper.crossCurrencySwap.StartDate();
            latestDate = crossCurrencySwapHelper.crossCurrencySwap.EndDate();
            
            % need to be implemented ??
            % check later !!
            
%             floatEndValueDate = crossCurrencySwapHelper.vanillaSwap.floatingLeg.couponRates{end}.fixingEndDate; 
%             % if the last floating Index fixingEndDate is after
%             % latestDate(last accrualEndDate then we change
%             % the lastestDate into floatEndValueDate
%             
%             if DateDiff(floatEndValueDate,latestDate) > 0
%                 latestDate = floatEndValueDate;
%             end
            
            out.earliestDate = earliestDate;
            out.latestDate = latestDate;
           
        end
        
        function out = ImpliedQuote(crossCurrencySwapHelper)
            outInfo = crossCurrencySwapHelper.crossCurrencySwap.Calculate(crossCurrencySwapHelper.discountCurve,crossCurrencySwapHelper.foreignDCurve);
            out = outInfo.fairRate;
            
        end
        
        % bootstrapping
        function out = QuoteError(crossCurrencySwapHelper)
            out = crossCurrencySwapHelper.rate - crossCurrencySwapHelper.ImpliedQuote();
        end
        
        function out = BootStrapError(crossCurrencySwapHelper,guess,idx)
            crossCurrencySwapHelper.Update(idx,guess);
            out = crossCurrencySwapHelper.QuoteError();
        end
        
        function Update(crossCurrencySwapHelper,idx,guess)
             % forwardCurve update
%             forwardRawData = crossCurrencySwapHelper.iborIndex.forwardCurve.params('rawData');
%             forwardRawData(idx,2) = guess;
%             crossCurrencySwapHelper.iborIndex.forwardCurve.params('rawData')= forwardRawData;
            
            % for singleCurve framework this is redundant
            
            % discountCurve update
            discountRawData = crossCurrencySwapHelper.discountCurve.params('rawData');
            discountRawData(idx,2) = guess;
            crossCurrencySwapHelper.discountCurve.params('rawData')= discountRawData;
            
        end
        
        
    end
    
end



In [ ]:
classdef CrossCurrencySwapRFR < H_RootObject
    % FixedRateLeg
    % properties: fixedSchedule 
    
    properties(SetAccess = public)
        type
        domesticNominal
        domesticSchedule
        domesticFixedRate
        domesticDayCounter
        
        foreignNominal
        foreignSchedule
        foreignGearing
        foreignSpread
        foreignDayCounter
        foreignIndex
        notionalExchangeType
        
        domesticLeg
        foreignLeg
        
        spotLag
        payLag
        payLagNotionalMaturityFlag
        
        NPV;
        fairRate;
       
    end
    
    methods
        % constructor
        function crossCurrencySwap = CrossCurrencySwapRFR(type,domesticNominal,domesticSchedule,domesticFixedRate,domesticDayCounter,...
                                      foreignNominal,foreignSchedule,foreignGearing,foreignSpread,foreignDayCounter,...
                                      foreignIndex,...
                                      notionalExchangeType,spotLag,payLag,payLagNotionalMaturityFlag)
            
            crossCurrencySwap.type = type;
            crossCurrencySwap.domesticNominal = domesticNominal;
            crossCurrencySwap.domesticSchedule = domesticSchedule;
            crossCurrencySwap.domesticFixedRate = domesticFixedRate;
            crossCurrencySwap.domesticDayCounter = domesticDayCounter;
            
            crossCurrencySwap.foreignNominal = foreignNominal;
            crossCurrencySwap.foreignSchedule = foreignSchedule;
            crossCurrencySwap.foreignGearing = foreignGearing;
            crossCurrencySwap.foreignSpread = foreignSpread;
            crossCurrencySwap.foreignDayCounter = foreignDayCounter;
            crossCurrencySwap.foreignIndex = foreignIndex;
            
            crossCurrencySwap.notionalExchangeType = notionalExchangeType;
            
            crossCurrencySwap.spotLag = spotLag;
            crossCurrencySwap.payLag = payLag;
            crossCurrencySwap.payLagNotionalMaturityFlag = payLagNotionalMaturityFlag;
            
%             crossCurrencySwap.domesticLeg = FixedRateLeg(domesticSchedule,domesticNominal,domesticFixedRate,domesticDayCounter,domesticSchedule.convention);
            % payLag = 0 for CrossCurrencySwap
            
            crossCurrencySwap.domesticLeg = FixedRateLeg(domesticSchedule,domesticNominal,domesticFixedRate,domesticDayCounter,domesticSchedule.convention,payLag);
                                    
            if strcmp(notionalExchangeType,'Both') || strcmp(notionalExchangeType,'Initial')
                couponRates = crossCurrencySwap.domesticLeg.couponRates;
                tempRates = cell(length(couponRates)+1,1);
                tempRates{1} = SimpleCashflow(-domesticNominal,domesticSchedule.startDate);
                for i=2:length(tempRates)
                    tempRates{i} = couponRates{i-1};
                end
                crossCurrencySwap.domesticLeg.couponRates = tempRates;
            end
            
%             if strcmp(notionalExchangeType,'Both') || strcmp(notionalExchangeType,'Final')
%                 crossCurrencySwap.domesticLeg.couponRates{end+1} = SimpleCashflow(domesticNominal,domesticSchedule.endDate);
%             end
            
             if strcmp(notionalExchangeType,'Both') || strcmp(notionalExchangeType,'Final')
                couponRates = crossCurrencySwap.domesticLeg.couponRates;
                if  crossCurrencySwap.payLagNotionalMaturityFlag 
                    crossCurrencySwap.domesticLeg.couponRates{end+1} = SimpleCashflow(domesticNominal,domesticSchedule.endDate);
                else
                    lastCouponPaymentDate = couponRates{end}.paymentDate;
                    crossCurrencySwap.domesticLeg.couponRates{end+1} = SimpleCashflow(domesticNominal,lastCouponPaymentDate);
                
                end
                
            end
            
%             crossCurrencySwap.foreignLeg = FloatingRateLeg(foreignSchedule,foreignIndex,foreignGearing,foreignSpread,foreignNominal,...
%                                       foreignDayCounter,foreignSchedule.convention);
             % payLag = 0 for CrossCurrencySwap
             
%             crossCurrencySwap.foreignLeg = FloatingRateLeg(foreignSchedule,foreignIndex,foreignGearing,foreignSpread,foreignNominal,...
%                                       foreignDayCounter,foreignSchedule.convention,0);
                                  
%             crossCurrencySwap.foreignLeg = FloatingRateLeg(foreignSchedule,foreignIndex,foreignGearing,foreignSpread,foreignNominal,...
%                                       foreignDayCounter,foreignSchedule.convention,0);
                                  
            crossCurrencySwap.foreignLeg = OvernightLeg(foreignSchedule,foreignIndex,foreignGearing,foreignSpread,foreignNominal,...
                                        foreignDayCounter,foreignIndex.convention,payLag);
                                  
            if strcmp(notionalExchangeType,'Both') || strcmp(notionalExchangeType,'Initial')
                couponRates = crossCurrencySwap.foreignLeg.couponRates;
                tempRates = cell(length(couponRates)+1,1);
                tempRates{1} = SimpleCashflow(-foreignNominal,foreignSchedule.startDate);
                for i=2:length(tempRates)
                    tempRates{i} = couponRates{i-1};
                end
                crossCurrencySwap.foreignLeg.couponRates = tempRates;
            end
            
%             if strcmp(notionalExchangeType,'Both') || strcmp(notionalExchangeType,'Final')
%                 crossCurrencySwap.foreignLeg.couponRates{end+1} = SimpleCashflow(foreignNominal,foreignSchedule.endDate);
%             end
            
            if strcmp(notionalExchangeType,'Both') || strcmp(notionalExchangeType,'Final')
                couponRates = crossCurrencySwap.foreignLeg.couponRates;
                if  crossCurrencySwap.payLagNotionalMaturityFlag 
                    crossCurrencySwap.foreignLeg.couponRates{end+1} = SimpleCashflow(foreignNominal,foreignSchedule.endDate);
                else
                    lastCouponPaymentDate = couponRates{end}.paymentDate;
                    crossCurrencySwap.foreignLeg.couponRates{end+1} = SimpleCashflow(foreignNominal,lastCouponPaymentDate);
                end
                
            end
            
        end
        
        function out = StartDate(crossCurrencySwap)
            d1 = crossCurrencySwap.domesticLeg.StartDate();
            d2 = crossCurrencySwap.foreignLeg.StartDate();
            
            if DateDiff(d1,d2) > 0
                out = d2;
            else
                out = d1;
            end
        end
        
        function out = EndDate(crossCurrencySwap)
            d1 = crossCurrencySwap.domesticLeg.EndDate();
            d2 = crossCurrencySwap.foreignLeg.EndDate();
            
            if DateDiff(d1,d2) > 0
                out = d1;
            else
                out = d2;
            end
        end
        
        function out = Calculate(crossCurrencySwap,domesticDCurve,foreignDCurve)
            % to be updated
            % very weird samsung crossCurrencySwap pricing logic :(
            % talk to CSY in the future
            temp_ccyRate= crossCurrencySwap.domesticNominal;
            
            %notional include
            domesticFixedNPV = crossCurrencySwap.domesticLeg.NPV(domesticDCurve);
            
            % for simple cashflow BPS is 0
            domesticFixedBPS = crossCurrencySwap.domesticLeg.BPS(domesticDCurve);
            
            foreignFloatingNPV = crossCurrencySwap.foreignLeg.NPV(foreignDCurve);
            foreignFloatingBPS = crossCurrencySwap.foreignLeg.BPS(foreignDCurve);
            foreignSpreadNPV = crossCurrencySwap.foreignSpread*foreignFloatingBPS; 
            
            if strcmp(crossCurrencySwap.type,'Payer')
                out.NPV = -domesticFixedNPV + temp_ccyRate*(foreignSpreadNPV + foreignFloatingNPV);
%                 out.fairRate =  temp_ccyRate*(foreignSpreadNPV + foreignFloatingNPV)/domesticFixedBPS;
                out.fairRate =  1.0*(out.NPV)/domesticFixedBPS;
            else
                out.NPV = -1.0*(-domesticFixedNPV + temp_ccyRate*(foreignSpreadNPV + foreignFloatingNPV));
%                 out.fairRate =  temp_ccyRate*(foreignSpreadNPV + foreignFloatingNPV)/domesticFixedBPS;
                out.fairRate =  -1.0*(out.NPV)/domesticFixedBPS;
            end
            
        end
    end
    
end



In [ ]:
classdef CrossCurrencySwapHelper < H_RootObject
    % FixedRateLeg
    % properties: fixedSchedule 
    
    properties(SetAccess = public)
        
        rateType
        
        rate
        rateTenor
        
        gearing
        spread
        
%         domesticIndex
        exchangeRate

        domesticFixedLegTenor
        domesticFixedLegConvention
        domesticFixedLegDayCounter
        domesticFixedLegCalendar
        
        spotLag
        
        foreignIndex
        
        evaluationDate
        
        crossCurrencySwap
%         domesticDCurve
        discountCurve
        foreignDCurve
        
        earliestDate
        latestDate
        
    end
    
    methods
        % constructor
        function crossCurrencySwapHelper = CrossCurrencySwapHelper(params)
                if nargin > 0
                    crossCurrencySwapHelper.rateType = 'CrossCurrencySwap';
                    
                    crossCurrencySwapHelper.rate = params('rate');
                    crossCurrencySwapHelper.rateTenor = params('rateTenor');
                    
                    crossCurrencySwapHelper.exchangeRate = params('exchangeRate');
%                     crossCurrencySwapHelper.domesticIndex = params('domesticIndex');

                    crossCurrencySwapHelper.domesticFixedLegTenor = params('domesticFixedLegTenor');
                    crossCurrencySwapHelper.domesticFixedLegConvention = params('domesticFixedLegConvention');
                    crossCurrencySwapHelper.domesticFixedLegDayCounter = params('domesticFixedLegDayCounter');
                    crossCurrencySwapHelper.domesticFixedLegCalendar = params('domesticFixedLegCalendar');
                    
                    crossCurrencySwapHelper.spotLag = params('spotLag');
                    
                    crossCurrencySwapHelper.foreignIndex = params('foreignIndex');
                    
                    crossCurrencySwapHelper.evaluationDate = crossCurrencySwapHelper.foreignIndex.forwardCurve.asOfDate;
                    
%                     crossCurrencySwapHelper.domesticDCurve = params('domesticDCurve');
                    crossCurrencySwapHelper.discountCurve = params('discountCurve');
                    
                    % can be generalized later for dual curve input
                    
%                     crossCurrencySwapHelper.foreignDCurve = crossCurrencySwapHelper.foreignIndex.forwardCurve;
                    crossCurrencySwapHelper.foreignDCurve = params('foreignDCurve');
                    
                    if crossCurrencySwapHelper.discountCurve.asOfDate ~= crossCurrencySwapHelper.foreignIndex.forwardCurve.asOfDate
                        error('forwardCurve asOfDate is different from discountCurve asOfDate')
                    end
                    
                    crossCurrencySwapHelper.crossCurrencySwap = MakeCrossCurrencySwap(crossCurrencySwapHelper);
                
                    out = InitializeDates(crossCurrencySwapHelper);

                    crossCurrencySwapHelper.earliestDate = out.earliestDate;
                    crossCurrencySwapHelper.latestDate = out.latestDate;
                end
        end
        
        function out = MakeCrossCurrencySwap(crossCurrencySwapHelper)
%             domesticIndex = crossCurrencySwapHelper.domesticIndex;
            foreignIndex = crossCurrencySwapHelper.foreignIndex;
            
            domesticCalendar = crossCurrencySwapHelper.domesticFixedLegCalendar;
            foreignCalendar = foreignIndex.fixingCalendar;
            
            jointCalendar = JointCalendar({domesticCalendar,foreignCalendar});
            
%             fixingDays = max(domesticIndex.fixingDays,foreignIndex.fixingDays);
%             fixingDays = foreignIndex.fixingDays;
            
            referenceDate = foreignIndex.forwardCurve.asOfDate;
            
            startDate = jointCalendar.Advance(referenceDate,'Following',...
                                                crossCurrencySwapHelper.spotLag,'day',false);
            
            endDate = AddTenor(startDate,crossCurrencySwapHelper.rateTenor);
            % fixed & floating schedule
            domesticFixedLegTenor = crossCurrencySwapHelper.domesticFixedLegTenor;
            domesticFixedLegCovention = crossCurrencySwapHelper.domesticFixedLegConvention;
            
            useEndOfMonth = false;
            if IsEndOfMonth(startDate)
               useEndOfMonth = true;
            end
            
%             domesticSchedule = Schedule(startDate,endDate,domesticFixedLegTenor ,jointCalendar,...
%                                     domesticFixedLegCovention, 'Backward',false); 
%             
%             foreignSchedule = Schedule(startDate,endDate,foreignIndex.tenor,jointCalendar,...
%                                     foreignIndex.convention, 'Backward',false);
            
            domesticSchedule = ScheduleWTermConvention(startDate,endDate,domesticFixedLegTenor ,domesticCalendar,...
                                    domesticFixedLegCovention,domesticFixedLegCovention, 'Backward',useEndOfMonth); 
            
            foreignSchedule = ScheduleWTermConvention(startDate,endDate,foreignIndex.tenor,domesticCalendar,...
                                    domesticFixedLegCovention,domesticFixedLegCovention, 'Backward',useEndOfMonth);
                                
             type = 'Payer';

             domesticNominal = crossCurrencySwapHelper.exchangeRate;
%              domesticGearing = crossCurrencySwapHelper.gearing;
%              domesticSpread = 0.0;
            domesticFixedRate = 0.0;
            domesticDayCounter = crossCurrencySwapHelper.domesticFixedLegDayCounter;
             
             foreignNominal = 1.0;
             foreignGearing = 1.0;
             foreignSpread = 0.0;
             foreignDayCounter = foreignIndex.dayCounter;
             
             notionalExchangeType = 'Both';
             
             crossCurrencySwap = CrossCurrencySwap(type,domesticNominal,domesticSchedule,domesticFixedRate,domesticDayCounter, ...
                                   foreignNominal,foreignSchedule,foreignGearing,foreignSpread,foreignDayCounter,...
                                   foreignIndex,...
                                   notionalExchangeType);
                                  
             out = crossCurrencySwap;
        end
        
        function out = InitializeDates(crossCurrencySwapHelper)
            
            %earliestDate
            earliestDate = crossCurrencySwapHelper.crossCurrencySwap.StartDate();
            latestDate = crossCurrencySwapHelper.crossCurrencySwap.EndDate();
            
            % need to be implemented ??
            % check later !!
            
%             floatEndValueDate = crossCurrencySwapHelper.vanillaSwap.floatingLeg.couponRates{end}.fixingEndDate; 
%             % if the last floating Index fixingEndDate is after
%             % latestDate(last accrualEndDate then we change
%             % the lastestDate into floatEndValueDate
%             
%             if DateDiff(floatEndValueDate,latestDate) > 0
%                 latestDate = floatEndValueDate;
%             end
            
            out.earliestDate = earliestDate;
            out.latestDate = latestDate;
           
        end
        
        function out = ImpliedQuote(crossCurrencySwapHelper)
            outInfo = crossCurrencySwapHelper.crossCurrencySwap.Calculate(crossCurrencySwapHelper.discountCurve,crossCurrencySwapHelper.foreignDCurve);
            out = outInfo.fairRate;
            
        end
        
        % bootstrapping
        function out = QuoteError(crossCurrencySwapHelper)
            out = crossCurrencySwapHelper.rate - crossCurrencySwapHelper.ImpliedQuote();
        end
        
        function out = BootStrapError(crossCurrencySwapHelper,guess,idx)
            crossCurrencySwapHelper.Update(idx,guess);
            out = crossCurrencySwapHelper.QuoteError();
        end
        
        function Update(crossCurrencySwapHelper,idx,guess)
             % forwardCurve update
%             forwardRawData = crossCurrencySwapHelper.iborIndex.forwardCurve.params('rawData');
%             forwardRawData(idx,2) = guess;
%             crossCurrencySwapHelper.iborIndex.forwardCurve.params('rawData')= forwardRawData;
            
            % for singleCurve framework this is redundant
            
            % discountCurve update
            discountRawData = crossCurrencySwapHelper.discountCurve.params('rawData');
            discountRawData(idx,2) = guess;
            crossCurrencySwapHelper.discountCurve.params('rawData')= discountRawData;
            
        end
        
        
    end
    
end



In [ ]:
classdef CrossCurrencySwapRFRHelper < H_RootObject
    % FixedRateLeg
    % properties: fixedSchedule 
    
    properties(SetAccess = public)
        
        rateType
        
        rate
        rateTenor
        
        gearing
        spread
        
%         domesticIndex
        exchangeRate

        domesticFixedLegTenor
        domesticFixedLegConvention
        domesticFixedLegDayCounter
        domesticFixedLegCalendar
        
        
        foreignFloatingLegTenor
        foreignFloatingLegConvention
        foreignFloatingLegDayCounter
        foreignFloatingLegCalendar
        
        foreignSpread
        
        spotLag
        payLag

        payLagNotionalMaturityFlag
        
        foreignIndex
        
        evaluationDate
        
        crossCurrencySwap
%         domesticDCurve
        discountCurve
        foreignDCurve
        
        earliestDate
        latestDate
        
    end
    
    methods
        % constructor
        function crossCurrencySwapHelper = CrossCurrencySwapRFRHelper(params)
                if nargin > 0
                    crossCurrencySwapHelper.rateType = 'CrossCurrencySwap';
                    
                    crossCurrencySwapHelper.rate = params('rate');
                    crossCurrencySwapHelper.rateTenor = params('rateTenor');
                    
                    crossCurrencySwapHelper.exchangeRate = params('exchangeRate');
%                     crossCurrencySwapHelper.domesticIndex = params('domesticIndex');

                    crossCurrencySwapHelper.domesticFixedLegTenor = params('domesticFixedLegTenor');
                    crossCurrencySwapHelper.domesticFixedLegConvention = params('domesticFixedLegConvention');
                    crossCurrencySwapHelper.domesticFixedLegDayCounter = params('domesticFixedLegDayCounter');
                    crossCurrencySwapHelper.domesticFixedLegCalendar = params('domesticFixedLegCalendar');
                    
                    
                    crossCurrencySwapHelper.foreignFloatingLegTenor = params('foreignFloatingLegTenor');
                    crossCurrencySwapHelper.foreignFloatingLegConvention = params('foreignFloatingLegConvention');
                    crossCurrencySwapHelper.foreignFloatingLegDayCounter = params('foreignFloatingLegDayCounter');
                    crossCurrencySwapHelper.foreignFloatingLegCalendar = params('foreignFloatingLegCalendar');
                    
                    crossCurrencySwapHelper.foreignSpread = params('foreignSpread');
                    
                    crossCurrencySwapHelper.spotLag = params('spotLag');
                    crossCurrencySwapHelper.payLag = params('payLag');
%                     crossCurrencySwapHelper.payLagNotionalMaturity = params('payLagNotionalMaturity');
                    
                    if isKey(params,'payLagNotionalMaturityFlag')
                        crossCurrencySwapHelper.payLagNotionalMaturityFlag = params('payLagNotionalMaturityFlag');
                    else
                        crossCurrencySwapHelper.payLagNotionalMaturityFlag = true;
                    end
                    
                    
                    crossCurrencySwapHelper.foreignIndex = params('foreignIndex');
                    
                    crossCurrencySwapHelper.evaluationDate = crossCurrencySwapHelper.foreignIndex.forwardCurve.asOfDate;
                    
%                     crossCurrencySwapHelper.domesticDCurve = params('domesticDCurve');
                    crossCurrencySwapHelper.discountCurve = params('discountCurve');
                    
                    % can be generalized later for dual curve input
                    
%                     crossCurrencySwapHelper.foreignDCurve = crossCurrencySwapHelper.foreignIndex.forwardCurve;
                    crossCurrencySwapHelper.foreignDCurve = params('foreignDCurve');
                    
                    if crossCurrencySwapHelper.discountCurve.asOfDate ~= crossCurrencySwapHelper.foreignIndex.forwardCurve.asOfDate
                        error('forwardCurve asOfDate is different from discountCurve asOfDate')
                    end
                    
                    crossCurrencySwapHelper.crossCurrencySwap = MakeCrossCurrencySwapRFR(crossCurrencySwapHelper);
                
                    out = InitializeDates(crossCurrencySwapHelper);

                    crossCurrencySwapHelper.earliestDate = out.earliestDate;
                    crossCurrencySwapHelper.latestDate = out.latestDate;
                end
        end
        
        function out = MakeCrossCurrencySwapRFR(crossCurrencySwapHelper)
%             domesticIndex = crossCurrencySwapHelper.domesticIndex;
            foreignIndex = crossCurrencySwapHelper.foreignIndex;
            
            domesticCalendar = crossCurrencySwapHelper.domesticFixedLegCalendar;
            foreignCalendar = crossCurrencySwapHelper.foreignFloatingLegCalendar;
            
            calcCalendar = JointCalendar({domesticCalendar,foreignCalendar});
            
%             fixingDays = max(domesticIndex.fixingDays,foreignIndex.fixingDays);
%             fixingDays = foreignIndex.fixingDays;
            
            referenceDate = foreignIndex.forwardCurve.asOfDate;
            
            startDate = calcCalendar.Advance(referenceDate,'Following',...
                                                crossCurrencySwapHelper.spotLag,'day',false);
             useEndOfMonth = false;
             if IsEndOfMonth(startDate)
                 useEndOfMonth = true;
             end
             
            endDate = AddTenor(startDate,crossCurrencySwapHelper.rateTenor);
            % fixed & floating schedule
            domesticFixedLegTenor = crossCurrencySwapHelper.domesticFixedLegTenor;
            domesticFixedLegConvention = crossCurrencySwapHelper.domesticFixedLegConvention;
            
            % fixed & floating schedule
             domesticSchedule = ScheduleWTermConvention(startDate,endDate,domesticFixedLegTenor,calcCalendar,...
                                        domesticFixedLegConvention,domesticFixedLegConvention, 'Backward',useEndOfMonth);
                                    
             
             foreignFloatingLegTenor = crossCurrencySwapHelper.foreignFloatingLegTenor;
             foreignFloatingLegConvention = crossCurrencySwapHelper.foreignFloatingLegConvention;

             foreignSchedule = ScheduleWTermConvention(startDate,endDate,foreignFloatingLegTenor,calcCalendar,...
                                        foreignFloatingLegConvention,foreignFloatingLegConvention, 'Backward',useEndOfMonth);

             type = 'Payer';
             domesticNominal = crossCurrencySwapHelper.exchangeRate;
%              domesticFixedRate = crossCurrencySwapHelper.rate;
             domesticFixedRate = 0.0;
             domesticDayCounter = crossCurrencySwapHelper.domesticFixedLegDayCounter;

             foreignNominal = 1.0;
             foreignGearing = 1.0;
        %      foreignSpread = 0.0;
             foreignSpread = crossCurrencySwapHelper.foreignSpread;

             foreignDayCounter = crossCurrencySwapHelper.foreignFloatingLegDayCounter;

             notionalExchangeType = 'Both';
             
             spotLag = crossCurrencySwapHelper.spotLag;
             payLag = crossCurrencySwapHelper.payLag;
             payLagNotionalMaturityFlag = crossCurrencySwapHelper.payLagNotionalMaturityFlag;
             
             crossCurrencySwapRFR = CrossCurrencySwapRFR(type,domesticNominal,domesticSchedule,domesticFixedRate,domesticDayCounter, ...
                                               foreignNominal,foreignSchedule,foreignGearing,foreignSpread,foreignDayCounter,...
                                               foreignIndex,...
                                               notionalExchangeType,spotLag,payLag,payLagNotionalMaturityFlag);
                                  
             out = crossCurrencySwapRFR;
        end
        
        function out = InitializeDates(crossCurrencySwapHelper)
            
            %earliestDate
            earliestDate = crossCurrencySwapHelper.crossCurrencySwap.StartDate();
            latestDate = crossCurrencySwapHelper.crossCurrencySwap.EndDate();
            
            % need to be implemented ??
            % check later !!
            
%             floatEndValueDate = crossCurrencySwapHelper.vanillaSwap.floatingLeg.couponRates{end}.fixingEndDate; 
%             % if the last floating Index fixingEndDate is after
%             % latestDate(last accrualEndDate then we change
%             % the lastestDate into floatEndValueDate
%             
%             if DateDiff(floatEndValueDate,latestDate) > 0
%                 latestDate = floatEndValueDate;
%             end
            
            out.earliestDate = earliestDate;
            out.latestDate = latestDate;
           
        end
        
        function out = ImpliedQuote(crossCurrencySwapHelper)
            outInfo = crossCurrencySwapHelper.crossCurrencySwap.Calculate(crossCurrencySwapHelper.discountCurve,crossCurrencySwapHelper.foreignDCurve);
            out = outInfo.fairRate;
            
        end
        
        % bootstrapping
        function out = QuoteError(crossCurrencySwapHelper)
            out = crossCurrencySwapHelper.rate - crossCurrencySwapHelper.ImpliedQuote();
        end
        
        function out = BootStrapError(crossCurrencySwapHelper,guess,idx)
            crossCurrencySwapHelper.Update(idx,guess);
            out = crossCurrencySwapHelper.QuoteError();
        end
        
        function Update(crossCurrencySwapHelper,idx,guess)
             % forwardCurve update
%             forwardRawData = crossCurrencySwapHelper.iborIndex.forwardCurve.params('rawData');
%             forwardRawData(idx,2) = guess;
%             crossCurrencySwapHelper.iborIndex.forwardCurve.params('rawData')= forwardRawData;
            
            % for singleCurve framework this is redundant
            
            % discountCurve update
            discountRawData = crossCurrencySwapHelper.discountCurve.params('rawData');
            discountRawData(idx,2) = guess;
            crossCurrencySwapHelper.discountCurve.params('rawData')= discountRawData;
            
        end
        
        
    end
    
end



In [ ]:
classdef CrossCurrencySwap < H_RootObject
    % FixedRateLeg
    % properties: fixedSchedule 
    
    properties(SetAccess = public)
        type
        domesticNominal
        domesticSchedule
        domesticFixedRate
        domesticDayCounter
        
        foreignNominal
        foreignSchedule
        foreignGearing
        foreignSpread
        foreignDayCounter
        foreignIndex
        notionalExchangeType
        
        domesticLeg
        foreignLeg
        
        NPV;
        fairRate;
       
    end
    
    methods
        % constructor
        function crossCurrencySwap = CrossCurrencySwap(type,domesticNominal,domesticSchedule,domesticFixedRate,domesticDayCounter,...
                                      foreignNominal,foreignSchedule,foreignGearing,foreignSpread,foreignDayCounter,...
                                      foreignIndex,...
                                      notionalExchangeType)
            
            crossCurrencySwap.type = type;
            crossCurrencySwap.domesticNominal = domesticNominal;
            crossCurrencySwap.domesticSchedule = domesticSchedule;
            crossCurrencySwap.domesticFixedRate = domesticFixedRate;
            crossCurrencySwap.domesticDayCounter = domesticDayCounter;
            
            crossCurrencySwap.foreignNominal = foreignNominal;
            crossCurrencySwap.foreignSchedule = foreignSchedule;
            crossCurrencySwap.foreignGearing = foreignGearing;
            crossCurrencySwap.foreignSpread = foreignSpread;
            crossCurrencySwap.foreignDayCounter = foreignDayCounter;
            crossCurrencySwap.foreignIndex = foreignIndex;
            
            crossCurrencySwap.notionalExchangeType = notionalExchangeType;
            
%             crossCurrencySwap.domesticLeg = FixedRateLeg(domesticSchedule,domesticNominal,domesticFixedRate,domesticDayCounter,domesticSchedule.convention);
            % payLag = 0 for CrossCurrencySwap
            
            crossCurrencySwap.domesticLeg = FixedRateLeg(domesticSchedule,domesticNominal,domesticFixedRate,domesticDayCounter,domesticSchedule.convention,0);
                                    
            if strcmp(notionalExchangeType,'Both') || strcmp(notionalExchangeType,'Initial')
                couponRates = crossCurrencySwap.domesticLeg.couponRates;
                tempRates = cell(length(couponRates)+1,1);
                tempRates{1} = SimpleCashflow(-domesticNominal,domesticSchedule.startDate);
                for i=2:length(tempRates)
                    tempRates{i} = couponRates{i-1};
                end
                crossCurrencySwap.domesticLeg.couponRates = tempRates;
            end
            
            if strcmp(notionalExchangeType,'Both') || strcmp(notionalExchangeType,'Final')
                crossCurrencySwap.domesticLeg.couponRates{end+1} = SimpleCashflow(domesticNominal,domesticSchedule.endDate);
            end
            
            
%             crossCurrencySwap.foreignLeg = FloatingRateLeg(foreignSchedule,foreignIndex,foreignGearing,foreignSpread,foreignNominal,...
%                                       foreignDayCounter,foreignSchedule.convention);
             % payLag = 0 for CrossCurrencySwap
             
            crossCurrencySwap.foreignLeg = FloatingRateLeg(foreignSchedule,foreignIndex,foreignGearing,foreignSpread,foreignNominal,...
                                      foreignDayCounter,foreignSchedule.convention,0);
                                  
            if strcmp(notionalExchangeType,'Both') || strcmp(notionalExchangeType,'Initial')
                couponRates = crossCurrencySwap.foreignLeg.couponRates;
                tempRates = cell(length(couponRates)+1,1);
                tempRates{1} = SimpleCashflow(-foreignNominal,foreignSchedule.startDate);
                for i=2:length(tempRates)
                    tempRates{i} = couponRates{i-1};
                end
                crossCurrencySwap.foreignLeg.couponRates = tempRates;
            end
            
            if strcmp(notionalExchangeType,'Both') || strcmp(notionalExchangeType,'Final')
                crossCurrencySwap.foreignLeg.couponRates{end+1} = SimpleCashflow(foreignNominal,foreignSchedule.endDate);
            end
            
        end
        
        function out = StartDate(crossCurrencySwap)
            d1 = crossCurrencySwap.domesticLeg.StartDate();
            d2 = crossCurrencySwap.foreignLeg.StartDate();
            
            if DateDiff(d1,d2) > 0
                out = d2;
            else
                out = d1;
            end
        end
        
        function out = EndDate(crossCurrencySwap)
            d1 = crossCurrencySwap.domesticLeg.EndDate();
            d2 = crossCurrencySwap.foreignLeg.EndDate();
            
            if DateDiff(d1,d2) > 0
                out = d1;
            else
                out = d2;
            end
        end
        
        function out = Calculate(crossCurrencySwap,domesticDCurve,foreignDCurve)
            % to be updated
            % very weird samsung crossCurrencySwap pricing logic :(
            % talk to CSY in the future
            temp_ccyRate= crossCurrencySwap.domesticNominal;
            
            %notional include
            domesticFixedNPV = crossCurrencySwap.domesticLeg.NPV(domesticDCurve);
            
            % for simple cashflow BPS is 0
            domesticFixedBPS = crossCurrencySwap.domesticLeg.BPS(domesticDCurve);
            
            foreignFloatingNPV = crossCurrencySwap.foreignLeg.NPV(foreignDCurve);
            foreignFloatingBPS = crossCurrencySwap.foreignLeg.BPS(foreignDCurve);
            foreignSpreadNPV = crossCurrencySwap.foreignSpread*foreignFloatingBPS; 
            
            if strcmp(crossCurrencySwap.type,'Payer')
                out.NPV = -domesticFixedNPV + temp_ccyRate*(foreignSpreadNPV + foreignFloatingNPV);
                out.fairRate =  1.0*(out.NPV)/domesticFixedBPS;
            else
                out.NPV = -1.0*(-domesticFixedNPV + temp_ccyRate*(foreignSpreadNPV + foreignFloatingNPV));
                out.fairRate =  -1.0*(out.NPV)/domesticFixedBPS;
            end
            
        end
    end
    
end



In [ ]:
classdef CrossCurrencyBasisSwapRFR < H_RootObject
    % FixedRateLeg
    % properties: fixedSchedule 
    
    properties(SetAccess = public)
        type
        domesticNominal
        domesticIndex
        domesticSchedule
        domesticGearing
        domesticSpread
        domesticDayCounter
        
        foreignNominal
        foreignIndex
        foreignSchedule
        foreignGearing
        foreignSpread
        foreignDayCounter
        
        notionalExchangeType
        payLag
        
        payLagNotionalMaturityFlag
        
        domesticLeg
        foreignLeg
        
        NPV;
        fairRate;
       
    end
    
    methods
        % constructor
        function crossCurrencyBasisSwap = CrossCurrencyBasisSwapRFR(type,domesticNominal,domesticIndex,domesticSchedule,domesticGearing,domesticSpread,domesticDayCounter,...
                                      foreignNominal,foreignIndex,foreignSchedule,foreignGearing,foreignSpread,foreignDayCounter,...
                                      notionalExchangeType,payLag,payLagNotionalMaturityFlag)
            
            crossCurrencyBasisSwap.type = type;
            crossCurrencyBasisSwap.domesticIndex = domesticIndex;
            crossCurrencyBasisSwap.domesticNominal = domesticNominal;
            crossCurrencyBasisSwap.domesticSchedule = domesticSchedule;
            crossCurrencyBasisSwap.domesticGearing = domesticGearing;
            crossCurrencyBasisSwap.domesticSpread = domesticSpread;
            crossCurrencyBasisSwap.domesticDayCounter = domesticDayCounter;
            
            crossCurrencyBasisSwap.foreignNominal = foreignNominal;
            crossCurrencyBasisSwap.foreignIndex = foreignIndex;
            crossCurrencyBasisSwap.foreignSchedule = foreignSchedule;
            crossCurrencyBasisSwap.foreignGearing = foreignGearing;
            crossCurrencyBasisSwap.foreignSpread = foreignSpread;
            crossCurrencyBasisSwap.foreignDayCounter = foreignDayCounter;
            
            
            crossCurrencyBasisSwap.notionalExchangeType = notionalExchangeType;
            crossCurrencyBasisSwap.payLag = payLag;
            crossCurrencyBasisSwap.payLagNotionalMaturityFlag = payLagNotionalMaturityFlag;
            
%             crossCurrencySwap.domesticLeg = FixedRateLeg(domesticSchedule,domesticNominal,domesticFixedRate,domesticDayCounter,domesticSchedule.convention);
            % payLag = 0 for CrossCurrencySwap
            
%             crossCurrencyBasisSwap.domesticLeg = FixedRateLeg(domesticSchedule,domesticNominal,domesticFixedRate,domesticDayCounter,domesticSchedule.convention,0);
            
%             crossCurrencyBasisSwap.domesticLeg = OvernightLeg(domesticSchedule,domesticIndex,domesticGearing,domesticSpread,domesticNominal,...
%                                       domesticDayCounter,domesticSchedule.convention,payLag);
                                  
            crossCurrencyBasisSwap.domesticLeg = FloatingRateLeg(domesticSchedule,domesticIndex,domesticGearing,domesticSpread,domesticNominal,...
                                      domesticDayCounter,domesticSchedule.convention,payLag);
                                  
            if strcmp(notionalExchangeType,'Both') || strcmp(notionalExchangeType,'Initial')
                couponRates = crossCurrencyBasisSwap.domesticLeg.couponRates;
                tempRates = cell(length(couponRates)+1,1);
                tempRates{1} = SimpleCashflow(-domesticNominal,domesticSchedule.startDate);
                for i=2:length(tempRates)
                    tempRates{i} = couponRates{i-1};
                end
                crossCurrencyBasisSwap.domesticLeg.couponRates = tempRates;
            end
            
            if strcmp(notionalExchangeType,'Both') || strcmp(notionalExchangeType,'Final')
                couponRates = crossCurrencyBasisSwap.domesticLeg.couponRates;
                if  crossCurrencyBasisSwap.payLagNotionalMaturityFlag 
                    crossCurrencyBasisSwap.domesticLeg.couponRates{end+1} = SimpleCashflow(domesticNominal,domesticSchedule.endDate);
                else
                    lastCouponPaymentDate = couponRates{end}.paymentDate;
                    crossCurrencyBasisSwap.domesticLeg.couponRates{end+1} = SimpleCashflow(domesticNominal,lastCouponPaymentDate);
                
                end
                
            end
            
            
%             crossCurrencySwap.foreignLeg = FloatingRateLeg(foreignSchedule,foreignIndex,foreignGearing,foreignSpread,foreignNominal,...
%                                       foreignDayCounter,foreignSchedule.convention);
             % payLag = 0 for CrossCurrencySwap
             
            crossCurrencyBasisSwap.foreignLeg = OvernightLeg(foreignSchedule,foreignIndex,foreignGearing,foreignSpread,foreignNominal,...
                                      foreignDayCounter,foreignSchedule.convention,payLag);
                                  
            if strcmp(notionalExchangeType,'Both') || strcmp(notionalExchangeType,'Initial')
                couponRates = crossCurrencyBasisSwap.foreignLeg.couponRates;
                tempRates = cell(length(couponRates)+1,1);
                tempRates{1} = SimpleCashflow(-foreignNominal,foreignSchedule.startDate);
                for i=2:length(tempRates)
                    tempRates{i} = couponRates{i-1};
                end
                crossCurrencyBasisSwap.foreignLeg.couponRates = tempRates;
            end
            
            if strcmp(notionalExchangeType,'Both') || strcmp(notionalExchangeType,'Final')
                couponRates = crossCurrencyBasisSwap.foreignLeg.couponRates;
                if  crossCurrencyBasisSwap.payLagNotionalMaturityFlag 
                    crossCurrencyBasisSwap.foreignLeg.couponRates{end+1} = SimpleCashflow(foreignNominal,foreignSchedule.endDate);
                else
                    lastCouponPaymentDate = couponRates{end}.paymentDate;
                    crossCurrencyBasisSwap.foreignLeg.couponRates{end+1} = SimpleCashflow(foreignNominal,lastCouponPaymentDate);
                end
                
            end
            
        end
        
        function out = StartDate(crossCurrencyBasisSwap)
%             d1 = crossCurrencyBasisSwap.domesticLeg.StartDate();
%             d2 = crossCurrencyBasisSwap.foreignLeg.StartDate();
            
            d1 = crossCurrencyBasisSwap.domesticLeg.schedule.startDate;
            d2 = crossCurrencyBasisSwap.foreignLeg.schedule.startDate;
            
            if DateDiff(d1,d2) > 0
                out = d2;
            else
                out = d1;
            end
        end
        
        function out = EndDate(crossCurrencyBasisSwap)
%             d1 = crossCurrencyBasisSwap.domesticLeg.EndDate();
%             d2 = crossCurrencyBasisSwap.foreignLeg.EndDate();
            
            d1 = crossCurrencyBasisSwap.domesticLeg.schedule.endDate;
            d2 = crossCurrencyBasisSwap.foreignLeg.schedule.endDate;
            
            if DateDiff(d1,d2) > 0
                out = d1;
            else
                out = d2;
            end
        end
        
        function out = Calculate(crossCurrencyBasisSwap,domesticDCurve,foreignDCurve)
            % to be updated
            % very weird samsung crossCurrencySwap pricing logic :(
            % talk to CSY in the future
            temp_ccyRate= crossCurrencyBasisSwap.domesticNominal;
            
            %notional include
            domesticNPV = crossCurrencyBasisSwap.domesticLeg.NPV(domesticDCurve);
            
            % for simple cashflow BPS is 0
            domesticBPS = crossCurrencyBasisSwap.domesticLeg.BPS(domesticDCurve);
            domesticSpreadNPV = crossCurrencyBasisSwap.domesticSpread*domesticBPS; 
            
            foreignNPV = crossCurrencyBasisSwap.foreignLeg.NPV(foreignDCurve);
            foreignBPS = crossCurrencyBasisSwap.foreignLeg.BPS(foreignDCurve);
%             foreignSpreadNPV = crossCurrencyBasisSwap.foreignSpread*foreignFloatingBPS; 
            
            if strcmp(crossCurrencyBasisSwap.type,'Payer')
                out.NPV = domesticNPV + domesticSpreadNPV - temp_ccyRate*foreignNPV;
                out.fairDomesticSpread =  (temp_ccyRate*foreignNPV - domesticNPV)/domesticBPS;
            else
                out.NPV = -1.0*(domesticNPV + domesticSpreadNPV - temp_ccyRate*foreignNPV);
                out.fairDomesticSpread =  -1.0*(temp_ccyRate*foreignNPV - domesticNPV)/domesticBPS;
            end
            
        end
    end
    
end



In [ ]:
classdef CrossCurrencyBasisSwapHelper < H_RootObject
    % FixedRateLeg
    % properties: fixedSchedule 
    
    properties(SetAccess = public)
        
        rateType
        
        rate
        rateTenor
        
%         gearing
%         spread
        
        exchangeRate

        domesticIndex
        
        domesticFloatingLegTenor
        domesticFloatingLegConvention
        domesticFloatingLegDayCounter
        domesticFloatingLegCalendar
                
        foreignIndex

        foreignFloatingLegTenor
        foreignFloatingLegConvention
        foreignFloatingLegDayCounter
        foreignFloatingLegCalendar
        
        spotLag
        payLag
        
        payLagNotionalMaturityFlag
        
        evaluationDate
        
        crossCurrencyBasisSwap
        discountCurve
        foreignDCurve
        
        earliestDate
        latestDate
        
    end
    
    methods
        % constructor
        function crossCurrencyBasisSwapHelper = CrossCurrencyBasisSwapHelper(params)
                if nargin > 0
                    crossCurrencyBasisSwapHelper.rateType = 'CrossCurrencyBasisSwap';
                    
                    crossCurrencyBasisSwapHelper.rate = params('rate');
                    crossCurrencyBasisSwapHelper.rateTenor = params('rateTenor');
                    
                    crossCurrencyBasisSwapHelper.exchangeRate = params('exchangeRate');
                    
                    crossCurrencyBasisSwapHelper.domesticIndex = params('domesticIndex');

                    crossCurrencyBasisSwapHelper.domesticFloatingLegTenor = params('domesticFloatingLegTenor');
                    crossCurrencyBasisSwapHelper.domesticFloatingLegConvention = params('domesticFloatingLegConvention');
                    crossCurrencyBasisSwapHelper.domesticFloatingLegDayCounter = params('domesticFloatingLegDayCounter');
                    crossCurrencyBasisSwapHelper.domesticFloatingLegCalendar = params('domesticFloatingLegCalendar');
                    
                    crossCurrencyBasisSwapHelper.foreignIndex = params('foreignIndex');
                    
                    crossCurrencyBasisSwapHelper.foreignFloatingLegTenor = params('foreignFloatingLegTenor');
                    crossCurrencyBasisSwapHelper.foreignFloatingLegConvention = params('foreignFloatingLegConvention');
                    crossCurrencyBasisSwapHelper.foreignFloatingLegDayCounter = params('foreignFloatingLegDayCounter');
                    crossCurrencyBasisSwapHelper.foreignFloatingLegCalendar = params('foreignFloatingLegCalendar');
                    
                    
                    crossCurrencyBasisSwapHelper.spotLag = params('spotLag');
                    
                    crossCurrencyBasisSwapHelper.payLag = params('payLag');
                    
                    if isKey(params,'payLagNotionalMaturityFlag')
                        crossCurrencyBasisSwapHelper.payLagNotionalMaturityFlag = params('payLagNotionalMaturityFlag');
                    else
                        crossCurrencyBasisSwapHelper.payLagNotionalMaturityFlag = true;
                    end
                    
                    
                    crossCurrencyBasisSwapHelper.evaluationDate = crossCurrencyBasisSwapHelper.foreignIndex.forwardCurve.asOfDate;
                    
                    crossCurrencyBasisSwapHelper.discountCurve = params('discountCurve');
                    
                    % can be generalized later for dual curve input
                    
                    crossCurrencyBasisSwapHelper.foreignDCurve = params('foreignDCurve');
                    
                    if crossCurrencyBasisSwapHelper.discountCurve.asOfDate ~= crossCurrencyBasisSwapHelper.foreignIndex.forwardCurve.asOfDate
                        error('forwardCurve asOfDate is different from discountCurve asOfDate')
                    end
                    
                    crossCurrencyBasisSwapHelper.crossCurrencyBasisSwap = MakeCrossCurrencyBasisSwap(crossCurrencyBasisSwapHelper);
                
                    out = InitializeDates(crossCurrencyBasisSwapHelper);

                    crossCurrencyBasisSwapHelper.earliestDate = out.earliestDate;
                    crossCurrencyBasisSwapHelper.latestDate = out.latestDate;
                end
        end
        
        function out = MakeCrossCurrencyBasisSwap(crossCurrencyBasisSwapHelper)
            
            domesticIndex = crossCurrencyBasisSwapHelper.domesticIndex;
            foreignIndex = crossCurrencyBasisSwapHelper.foreignIndex;
            
            domesticCalendar = crossCurrencyBasisSwapHelper.domesticFloatingLegCalendar;
            foreignCalendar = crossCurrencyBasisSwapHelper.foreignFloatingLegCalendar;
            
            jointCalendar = JointCalendar({domesticCalendar,foreignCalendar});
            
%             fixingDays = max(domesticIndex.fixingDays,foreignIndex.fixingDays);
%             fixingDays = foreignIndex.fixingDays;
            
            referenceDate = foreignIndex.forwardCurve.asOfDate;
            
            startDate = jointCalendar.Advance(referenceDate,'Following',...
                                                crossCurrencyBasisSwapHelper.spotLag,'day',false);
            
            endDate = AddTenor(startDate,crossCurrencyBasisSwapHelper.rateTenor);
            % fixed & floating schedule
            domesticFloatingLegTenor = crossCurrencyBasisSwapHelper.domesticFloatingLegTenor;
            domesticFloatingLegConvention = crossCurrencyBasisSwapHelper.domesticFloatingLegConvention;
            
            foreignFloatingLegTenor = crossCurrencyBasisSwapHelper.foreignFloatingLegTenor;
            foreignFloatingLegConvention = crossCurrencyBasisSwapHelper.foreignFloatingLegConvention;
            
            useEndOfMonth = false;
            if IsEndOfMonth(startDate)
               useEndOfMonth = true;
            end
            
%             domesticSchedule = Schedule(startDate,endDate,domesticFixedLegTenor ,jointCalendar,...
%                                     domesticFixedLegCovention, 'Backward',false); 
%             
%             foreignSchedule = Schedule(startDate,endDate,foreignIndex.tenor,jointCalendar,...
%                                     foreignIndex.convention, 'Backward',false);
            
            domesticSchedule = ScheduleWTermConvention(startDate,endDate,domesticFloatingLegTenor ,domesticCalendar,...
                                    domesticFloatingLegConvention,domesticFloatingLegConvention, 'Backward',useEndOfMonth); 
            
            foreignSchedule = ScheduleWTermConvention(startDate,endDate,foreignFloatingLegTenor,foreignCalendar,...
                                    foreignFloatingLegConvention,foreignFloatingLegConvention, 'Backward',useEndOfMonth);
                                
             type = 'Payer';

             domesticNominal = crossCurrencyBasisSwapHelper.exchangeRate;
             domesticGearing = 1.0;
             domesticSpread = 0.0;
             domesticFixedRate = 0.0;
             domesticDayCounter = crossCurrencyBasisSwapHelper.domesticFloatingLegDayCounter;
             
             foreignNominal = 1.0;
             foreignGearing = 1.0;
             foreignSpread = 0.0;
             foreignDayCounter = crossCurrencyBasisSwapHelper.foreignFloatingLegDayCounter;
             
             notionalExchangeType = 'Both';
             payLag = crossCurrencyBasisSwapHelper.payLag;
             
             payLagNotionalMaturityFlag = crossCurrencyBasisSwapHelper.payLagNotionalMaturityFlag;
             
             crossCurrencyBasisSwap = CrossCurrencyBasisSwap(type,domesticNominal,domesticIndex,domesticSchedule,domesticGearing,domesticSpread,domesticDayCounter, ...
                                   foreignNominal,foreignIndex,foreignSchedule,foreignGearing,foreignSpread,foreignDayCounter,...
                                   notionalExchangeType,payLag,payLagNotionalMaturityFlag);
             
%              crossCurrencySwap = CrossCurrencySwap(type,domesticNominal,domesticSchedule,domesticFixedRate,domesticDayCounter, ...
%                                    foreignNominal,foreignSchedule,foreignGearing,foreignSpread,foreignDayCounter,...
%                                    foreignIndex,...
%                                    notionalExchangeType);
%                                   
             out = crossCurrencyBasisSwap;
        end
        
        function out = InitializeDates(crossCurrencyBasisSwapHelper)
            
            %earliestDate
            earliestDate = crossCurrencyBasisSwapHelper.crossCurrencyBasisSwap.StartDate();
            latestDate = crossCurrencyBasisSwapHelper.crossCurrencyBasisSwap.EndDate();
            
            
            % use PillarDate As last paydate only when
            % useLastEndDateAsPillarDate is false
            if ~crossCurrencyBasisSwapHelper.domesticIndex.useLastEndDateAsPillarDate
            
                latestReleventDate = crossCurrencyBasisSwapHelper.crossCurrencyBasisSwap.domesticLeg.couponRates{end}.paymentDate; 
                % if the last floating Index fixingEndDate is after
                % latestDate(last accrualEndDate then we change
                % the lastestDate into floatEndValueDate

                if DateDiff(latestReleventDate,latestDate) > 0
                    latestDate = latestReleventDate;
                end
            end
            
            % need to be implemented ??
            % check later !!
            
%             floatEndValueDate = crossCurrencySwapHelper.vanillaSwap.floatingLeg.couponRates{end}.fixingEndDate; 
%             % if the last floating Index fixingEndDate is after
%             % latestDate(last accrualEndDate then we change
%             % the lastestDate into floatEndValueDate
%             
%             if DateDiff(floatEndValueDate,latestDate) > 0
%                 latestDate = floatEndValueDate;
%             end
            
            out.earliestDate = earliestDate;
            out.latestDate = latestDate;
           
        end
        
        function out = ImpliedQuote(crossCurrencyBasisSwapHelper)
            outInfo = crossCurrencyBasisSwapHelper.crossCurrencyBasisSwap.Calculate(crossCurrencyBasisSwapHelper.discountCurve,crossCurrencyBasisSwapHelper.foreignDCurve);
            out = outInfo.fairDomesticSpread;
            
        end
        
        % bootstrapping
        function out = QuoteError(crossCurrencyBasisSwapHelper)
            out = crossCurrencyBasisSwapHelper.rate - crossCurrencyBasisSwapHelper.ImpliedQuote();
        end
        
        function out = BootStrapError(crossCurrencyBasisSwapHelper,guess,idx)
            crossCurrencyBasisSwapHelper.Update(idx,guess);
            out = crossCurrencyBasisSwapHelper.QuoteError();
        end
        
        function Update(crossCurrencyBasisSwapHelper,idx,guess)
             % forwardCurve update
%             forwardRawData = crossCurrencySwapHelper.iborIndex.forwardCurve.params('rawData');
%             forwardRawData(idx,2) = guess;
%             crossCurrencySwapHelper.iborIndex.forwardCurve.params('rawData')= forwardRawData;
            
            % for singleCurve framework this is redundant
            
            % discountCurve update
            discountRawData = crossCurrencyBasisSwapHelper.discountCurve.params('rawData');
            discountRawData(idx,2) = guess;
            crossCurrencyBasisSwapHelper.discountCurve.params('rawData')= discountRawData;
            
        end
        
        
    end
    
end



In [ ]:
classdef CrossCurrencyBasisSwap < H_RootObject
    % FixedRateLeg
    % properties: fixedSchedule 
    
    properties(SetAccess = public)
        type
        domesticNominal
        domesticIndex
        domesticSchedule
        domesticGearing
        domesticSpread
        domesticDayCounter
        
        foreignNominal
        foreignIndex
        foreignSchedule
        foreignGearing
        foreignSpread
        foreignDayCounter
        
        notionalExchangeType
        payLag
        
        payLagNotionalMaturityFlag
        
        domesticLeg
        foreignLeg
        
        NPV;
        fairRate;
       
    end
    
    methods
        % constructor
        function crossCurrencyBasisSwap = CrossCurrencyBasisSwap(type,domesticNominal,domesticIndex,domesticSchedule,domesticGearing,domesticSpread,domesticDayCounter,...
                                      foreignNominal,foreignIndex,foreignSchedule,foreignGearing,foreignSpread,foreignDayCounter,...
                                      notionalExchangeType,payLag,payLagNotionalMaturityFlag)
            
            crossCurrencyBasisSwap.type = type;
            crossCurrencyBasisSwap.domesticIndex = domesticIndex;
            crossCurrencyBasisSwap.domesticNominal = domesticNominal;
            crossCurrencyBasisSwap.domesticSchedule = domesticSchedule;
            crossCurrencyBasisSwap.domesticGearing = domesticGearing;
            crossCurrencyBasisSwap.domesticSpread = domesticSpread;
            crossCurrencyBasisSwap.domesticDayCounter = domesticDayCounter;
            
            crossCurrencyBasisSwap.foreignNominal = foreignNominal;
            crossCurrencyBasisSwap.foreignIndex = foreignIndex;
            crossCurrencyBasisSwap.foreignSchedule = foreignSchedule;
            crossCurrencyBasisSwap.foreignGearing = foreignGearing;
            crossCurrencyBasisSwap.foreignSpread = foreignSpread;
            crossCurrencyBasisSwap.foreignDayCounter = foreignDayCounter;
            
            
            crossCurrencyBasisSwap.notionalExchangeType = notionalExchangeType;
            crossCurrencyBasisSwap.payLag = payLag;
            crossCurrencyBasisSwap.payLagNotionalMaturityFlag = payLagNotionalMaturityFlag;
            
%             crossCurrencySwap.domesticLeg = FixedRateLeg(domesticSchedule,domesticNominal,domesticFixedRate,domesticDayCounter,domesticSchedule.convention);
            % payLag = 0 for CrossCurrencySwap
            
%             crossCurrencyBasisSwap.domesticLeg = FixedRateLeg(domesticSchedule,domesticNominal,domesticFixedRate,domesticDayCounter,domesticSchedule.convention,0);
            
            crossCurrencyBasisSwap.domesticLeg = OvernightLeg(domesticSchedule,domesticIndex,domesticGearing,domesticSpread,domesticNominal,...
                                      domesticDayCounter,domesticSchedule.convention,payLag);
                                  
            if strcmp(notionalExchangeType,'Both') || strcmp(notionalExchangeType,'Initial')
                couponRates = crossCurrencyBasisSwap.domesticLeg.couponRates;
                tempRates = cell(length(couponRates)+1,1);
                tempRates{1} = SimpleCashflow(-domesticNominal,domesticSchedule.startDate);
                for i=2:length(tempRates)
                    tempRates{i} = couponRates{i-1};
                end
                crossCurrencyBasisSwap.domesticLeg.couponRates = tempRates;
            end
            
            if strcmp(notionalExchangeType,'Both') || strcmp(notionalExchangeType,'Final')
                couponRates = crossCurrencyBasisSwap.domesticLeg.couponRates;
                if  crossCurrencyBasisSwap.payLagNotionalMaturityFlag 
                    crossCurrencyBasisSwap.domesticLeg.couponRates{end+1} = SimpleCashflow(domesticNominal,domesticSchedule.endDate);
                else
                    lastCouponPaymentDate = couponRates{end}.paymentDate;
                    crossCurrencyBasisSwap.domesticLeg.couponRates{end+1} = SimpleCashflow(domesticNominal,lastCouponPaymentDate);
                
                end
                
            end
            
            
%             crossCurrencySwap.foreignLeg = FloatingRateLeg(foreignSchedule,foreignIndex,foreignGearing,foreignSpread,foreignNominal,...
%                                       foreignDayCounter,foreignSchedule.convention);
             % payLag = 0 for CrossCurrencySwap
             
            crossCurrencyBasisSwap.foreignLeg = OvernightLeg(foreignSchedule,foreignIndex,foreignGearing,foreignSpread,foreignNominal,...
                                      foreignDayCounter,foreignSchedule.convention,payLag);
                                  
            if strcmp(notionalExchangeType,'Both') || strcmp(notionalExchangeType,'Initial')
                couponRates = crossCurrencyBasisSwap.foreignLeg.couponRates;
                tempRates = cell(length(couponRates)+1,1);
                tempRates{1} = SimpleCashflow(-foreignNominal,foreignSchedule.startDate);
                for i=2:length(tempRates)
                    tempRates{i} = couponRates{i-1};
                end
                crossCurrencyBasisSwap.foreignLeg.couponRates = tempRates;
            end
            
            if strcmp(notionalExchangeType,'Both') || strcmp(notionalExchangeType,'Final')
                couponRates = crossCurrencyBasisSwap.foreignLeg.couponRates;
                if  crossCurrencyBasisSwap.payLagNotionalMaturityFlag 
                    crossCurrencyBasisSwap.foreignLeg.couponRates{end+1} = SimpleCashflow(foreignNominal,foreignSchedule.endDate);
                else
                    lastCouponPaymentDate = couponRates{end}.paymentDate;
                    crossCurrencyBasisSwap.foreignLeg.couponRates{end+1} = SimpleCashflow(foreignNominal,lastCouponPaymentDate);
                end
                
            end
            
        end
        
        function out = StartDate(crossCurrencyBasisSwap)
%             d1 = crossCurrencyBasisSwap.domesticLeg.StartDate();
%             d2 = crossCurrencyBasisSwap.foreignLeg.StartDate();
            
            d1 = crossCurrencyBasisSwap.domesticLeg.schedule.startDate;
            d2 = crossCurrencyBasisSwap.foreignLeg.schedule.startDate;
            
            if DateDiff(d1,d2) > 0
                out = d2;
            else
                out = d1;
            end
        end
        
        function out = EndDate(crossCurrencyBasisSwap)
%             d1 = crossCurrencyBasisSwap.domesticLeg.EndDate();
%             d2 = crossCurrencyBasisSwap.foreignLeg.EndDate();
            
            d1 = crossCurrencyBasisSwap.domesticLeg.schedule.endDate;
            d2 = crossCurrencyBasisSwap.foreignLeg.schedule.endDate;
            
            if DateDiff(d1,d2) > 0
                out = d1;
            else
                out = d2;
            end
        end
        
        function out = Calculate(crossCurrencyBasisSwap,domesticDCurve,foreignDCurve)
            % to be updated
            % very weird samsung crossCurrencySwap pricing logic :(
            % talk to CSY in the future
            temp_ccyRate= crossCurrencyBasisSwap.domesticNominal;
            
            %notional include
            domesticNPV = crossCurrencyBasisSwap.domesticLeg.NPV(domesticDCurve);
            
            % for simple cashflow BPS is 0
            domesticBPS = crossCurrencyBasisSwap.domesticLeg.BPS(domesticDCurve);
            domesticSpreadNPV = crossCurrencyBasisSwap.domesticSpread*domesticBPS; 
            
            foreignNPV = crossCurrencyBasisSwap.foreignLeg.NPV(foreignDCurve);
            foreignBPS = crossCurrencyBasisSwap.foreignLeg.BPS(foreignDCurve);
%             foreignSpreadNPV = crossCurrencyBasisSwap.foreignSpread*foreignFloatingBPS; 
            
            if strcmp(crossCurrencyBasisSwap.type,'Payer')
                out.NPV = domesticNPV + domesticSpreadNPV - temp_ccyRate*foreignNPV;
                out.fairDomesticSpread =  (temp_ccyRate*foreignNPV - domesticNPV)/domesticBPS;
            else
                out.NPV = -1.0*(domesticNPV + domesticSpreadNPV - temp_ccyRate*foreignNPV);
                out.fairDomesticSpread =  -1.0*(temp_ccyRate*foreignNPV - domesticNPV)/domesticBPS;
            end
            
        end
    end
    
end



In [ ]:
classdef CreditDefaultSwap < H_RootObject
    % FixedRateLeg
    % properties: fixedSchedule 
    
    properties(SetAccess = public)
        type
        engineType
        nominal
        fixedSchedule
        spread
        fixedDayCounter
        fixedConvention
        
        settlesAccrual
        paysAtDefaultTime
        
        protectionStartDate
        
        fixedRateLeg
        upfrontPayment
        
        NPV;
        fairRate;
       
    end
    
    methods
        % constructor
        function creditDefaultSwap = CreditDefaultSwap(type,engineType,nominal,fixedSchedule,spread,fixedDayCounter, ...
                                      fixedConvention,settlesAccrual,paysAtDefaultTime,protectionStartDate)
            
            creditDefaultSwap.type = type;
            creditDefaultSwap.engineType = engineType;
            creditDefaultSwap.nominal = nominal;
            creditDefaultSwap.fixedSchedule = fixedSchedule;
            creditDefaultSwap.spread = spread;
            creditDefaultSwap.fixedDayCounter = fixedDayCounter;
            creditDefaultSwap.fixedConvention = fixedConvention;
            
            creditDefaultSwap.settlesAccrual = settlesAccrual;
            creditDefaultSwap.paysAtDefaultTime = paysAtDefaultTime;
            
            creditDefaultSwap.protectionStartDate = protectionStartDate;
            
            % bug fixed 20180913
            % when yield curve building, fixedLeg's payment date convention
            % is given as floatingLeg's payment date convention in quantlib
            % !!!!
            
%             vanillaSwap.fixedLeg = FixedRateLeg(fixedSchedule,nominal,fixedRate,fixedDayCounter,fixedSchedule.convention);
            creditDefaultSwap.fixedRateLeg = FixedRateLeg(fixedSchedule,nominal,spread,fixedDayCounter,fixedConvention);
            
            creditDefaultSwap.upfrontPayment = SimpleCashflow(0.0,fixedSchedule.dates{1});
            
            
            
        end
        
        function out = StartDate(creditDefaultSwap)
            d1 = creditDefaultSwap.fixedRateLeg.StartDate();
            out = d1;
%             d2 = vanillaSwap.floatingLeg.StartDate();
            
%             if DateDiff(d1,d2) > 0
%                 out = d2;
%             else
%                 out = d1;
%             end

        end
        
        function out = EndDate(creditDefaultSwap)
            d1 = creditDefaultSwap.fixedRateLeg.EndDate();
            out = d1;
%             d2 = vanillaSwap.floatingLeg.EndDate();
            
%             if DateDiff(d1,d2) > 0
%                 out = d1;
%             else
%                 out = d2;
%             end

        end
        
        function out = Calculate(creditDefaultSwap,recoveryRate,survProbCurve,discountCurve)
                
                if strcmp(creditDefaultSwap.engineType,'Integrate')
                    
                    out = Calculate_Integral(creditDefaultSwap,recoveryRate,survProbCurve,discountCurve);
                else
                    out = Calculate_Mid(creditDefaultSwap,recoveryRate,survProbCurve,discountCurve);
                end
        end
        
        function out = Calculate_Mid(creditDefaultSwap,recoveryRate,survProbCurve,discountCurve)
            % MidPointCDS Engine !
            
            % consistency check !
            if DateDiff(survProbCurve.asOfDate,discountCurve.asOfDate) ~= 0
                error('inconsistency ! survProbCurve and discountCurve asOfDate should be the same !')
            end
            
            evaluationDate = discountCurve.asOfDate;
            upfPV01 = 0.0;
            upfrontNPV = 0.0;
            %upfront flow npv has not occured yet  (on-the-run)
            if DateDiff(creditDefaultSwap.upfrontPayment.paymentDate,evaluationDate) > 0
                effectiveUpFrontDate = creditDefaultSwap.protectionStartDate;
                upfPV01 = survProbCurve.SurvProbFromDate(effectiveUpFrontDate) * ...
                          discountCurve.DFFromDate(creditDefaultSwap.upfrontPayment.paymentDate);
                
                upfrontNPV = upfPV01 * creditDefaultSwap.upfrontPayment.amount;
            end
            
            couponLegNPV = 0.0;
            defaultLegNPV = 0.0;
            
            legSize = length(creditDefaultSwap.fixedRateLeg.couponRates);
            for i=1:legSize
                fixedRateCoupon = creditDefaultSwap.fixedRateLeg.couponRates{i};
                if DateDiff(fixedRateCoupon.paymentDate,evaluationDate) <= 0
                    continue;
                end
                
                paymentDate = fixedRateCoupon.paymentDate;
                startDate = fixedRateCoupon.accrualStartDate;
                endDate = fixedRateCoupon.accrualEndDate;
                
                if i==1
                    startDate = creditDefaultSwap.protectionStartDate;
                end
                
                if DateDiff(startDate,evaluationDate) <= 0 && DateDiff(evaluationDate,endDate) <= 0
                    effectiveStartDate = evaluationDate;
                else
                    effectiveStartDate = startDate;
                end
                
                defaultDate = effectiveStartDate.AddDate(floor(DateDiff(endDate,effectiveStartDate)/2),'day');
                
                S = survProbCurve.SurvProbFromDate(paymentDate);
                P = survProbCurve.DefaultProbFromDate(effectiveStartDate,endDate);
                
                % fixedRate payment in case of survival
                couponLegNPV = couponLegNPV + S * fixedRateCoupon.nominal * fixedRateCoupon.Amount() * ...
                                                  discountCurve.DFFromDate(paymentDate);
                
                % accrual cashflow in case of default
                if creditDefaultSwap.settlesAccrual
                    if creditDefaultSwap.paysAtDefaultTime
                        couponLegNPV = couponLegNPV + P * fixedRateCoupon.nominal * fixedRateCoupon.AccruedAmount(defaultDate)* ...
                                       discountCurve.DFFromDate(defaultDate);
                    else
                        couponLegNPV = couponLegNPV + P * fixedRateCoupon.nominal * fixedRateCoupon.Amount()* ...
                                       discountCurve.DFFromDate(paymentDate);
                    end
                end
                
                % at default cashflow
                claim = creditDefaultSwap.nominal * (1.0 - recoveryRate);
                    
                if creditDefaultSwap.paysAtDefaultTime
                    defaultLegNPV = defaultLegNPV + P * claim * discountCurve.DFFromDate(defaultDate);
                else
                    defaultLegNPV = defaultLegNPV + P * claim * discountCurve.DFFromDate(paymentDate);
                end
            end

            upfrontSign = 1.0;
            if strcmp(creditDefaultSwap.type,'ProtectionBuyer')
                couponLegNPV = couponLegNPV * -1.0;
                upfrontNPV = upfrontNPV * -1.0;
            else
                defaultLegNPV = defaultLegNPV * -1.0;
            end
            
            out.NPV = defaultLegNPV + couponLegNPV + upfrontNPV;
            out.fairSpread =  -1.0 * defaultLegNPV * creditDefaultSwap.spread/couponLegNPV;
            
            
        end
        
        function out = Calculate_Integral(creditDefaultSwap,recoveryRate,survProbCurve,discountCurve)
            % MidPointCDS Engine !
            
            % consistency check !
            if DateDiff(survProbCurve.asOfDate,discountCurve.asOfDate) ~= 0
                error('inconsistency ! survProbCurve and discountCurve asOfDate should be the same !')
            end
            
            evaluationDate = discountCurve.asOfDate;
            upfPV01 = 0.0;
            upfrontNPV = 0.0;
            %upfront flow npv has not occured yet  (on-the-run)
            if DateDiff(creditDefaultSwap.upfrontPayment.paymentDate,evaluationDate) > 0
                effectiveUpFrontDate = creditDefaultSwap.protectionStartDate;
                upfPV01 = survProbCurve.SurvProbFromDate(effectiveUpFrontDate) * ...
                          discountCurve.DFFromDate(creditDefaultSwap.upfrontPayment.paymentDate);
                
                upfrontNPV = upfPV01 * creditDefaultSwap.upfrontPayment.amount;
            end
            
            couponLegNPV = 0.0;
            defaultLegNPV = 0.0;
            
            legSize = length(creditDefaultSwap.fixedRateLeg.couponRates);
            for i=1:legSize
                fixedRateCoupon = creditDefaultSwap.fixedRateLeg.couponRates{i};
                if DateDiff(fixedRateCoupon.paymentDate,evaluationDate) <= 0
                    continue;
                end
                
                paymentDate = fixedRateCoupon.paymentDate;
                startDate = fixedRateCoupon.accrualStartDate;
                endDate = fixedRateCoupon.accrualEndDate;
                
                if i==1
                    startDate = creditDefaultSwap.protectionStartDate;
                end
                
                if DateDiff(startDate,evaluationDate) <= 0 && DateDiff(evaluationDate,endDate) <= 0
                    effectiveStartDate = evaluationDate;
                else
                    effectiveStartDate = startDate;
                end
                
%                 defaultDate = effectiveStartDate.AddDate(floor(DateDiff(endDate,effectiveStartDate)/2),'day');
                
                S = survProbCurve.SurvProbFromDate(paymentDate);
%                 P = survProbCurve.DefaultProbFromDate(effectiveStartDate,endDate);
                
                % fixedRate payment in case of survival
                couponLegNPV = couponLegNPV + S * fixedRateCoupon.nominal * fixedRateCoupon.Amount() * ...
                                                  discountCurve.DFFromDate(paymentDate);
                
                date0 = effectiveStartDate;
                stepSize = 7;
                date1 = date0.AddDate(stepSize,'day');
                
                %date1 should be earlier than endDate
                % forward generation of the schedule is dangerous!!
                % check for the theta in the future
                
                if DateDiff(date1,endDate) > 0
                    date1 = endDate;
                end
                    
                P0 = survProbCurve.DefaultProbFromDate(date0);
                endDiscount = discountCurve.DFFromDate(paymentDate);
                
                while DateDiff(date0,endDate) < 0
                    if creditDefaultSwap.paysAtDefaultTime
                        B =  discountCurve.DFFromDate(date1);
                    else
                        B = endDiscount;
                    end
                    
                    P1 = survProbCurve.DefaultProbFromDate(date1);
                    dP = P1 - P0;
                    
                    % accrual cashflow in case of default
                    if creditDefaultSwap.settlesAccrual
                        if creditDefaultSwap.paysAtDefaultTime
                            couponLegNPV = couponLegNPV + dP * fixedRateCoupon.nominal * fixedRateCoupon.AccruedAmount(date1)* ...
                                           B;
                        else
                            couponLegNPV = couponLegNPV + dP * fixedRateCoupon.nominal * fixedRateCoupon.Amount()* ...
                                           B;
                        end
                    end
                    
                    % at default cashflow
                    claim = creditDefaultSwap.nominal * (1.0 - recoveryRate);

                    if creditDefaultSwap.paysAtDefaultTime
                        defaultLegNPV = defaultLegNPV + dP * claim * B;
                    else
                        defaultLegNPV = defaultLegNPV + dP * claim * B;
                    end
                    
                    % setup for next time around the loop
                    P0 = P1;
                    date0 = date1;
                    date1 = date0.AddDate(stepSize,'day');
                    if DateDiff(date1,endDate) > 0
                        date1 = endDate;
                    end
                    
                end
                
                
                
            end

            upfrontSign = 1.0;
            if strcmp(creditDefaultSwap.type,'ProtectionBuyer')
                couponLegNPV = couponLegNPV * -1.0;
                upfrontNPV = upfrontNPV * -1.0;
            else
                defaultLegNPV = defaultLegNPV * -1.0;
            end
            
            out.NPV = defaultLegNPV + couponLegNPV + upfrontNPV;
            out.fairSpread =  -1.0 * defaultLegNPV * creditDefaultSwap.spread/couponLegNPV;
            
            
        end
        
        
    end
    
end



In [ ]:
function  [c, ceq]  = ConstraintFunctionLGM2F_CK1F(tvar)

    ceq = 0;
    c = tvar(1)*tvar(1) + tvar(2)*tvar(2) -1.0;   

end





In [ ]:
function  [c, ceq]  = ConstraintFunctionELSHedgeATMVegaZeroOTMLongVega(tvar,optParams)
%     K_swaption = optParams.params.diagSwaption.strike;
%     black_swaption = optParams.params.diagSwaption.blackPrice;
%     swaption_expiry_use_yn = optParams.params.swaption_expiry_use_yn;
%     dt= 1.0/(optParams.params.freq);

%     c = [];
    
    ELS = optParams.ELS;
    targetOptions = optParams.targetOptions;
    sizeOfOptions = optParams.sizeOfOptions;
    smoothingWeight = optParams.smoothingWeight;
    
    midIndex = optParams.midIndex;
    lowerIndex = optParams.midIndex - optParams.lowerNum;
    upperIndex = optParams.midIndex + optParams.upperNum;
    
    OTMLongVegaIndex = optParams.OTMLongVegaIndex;
    
    optionWeight = tvar;
    
    idx =1;
    
    c = zeros(optParams.targetGreeksSize,1);
    ceq = zeros(optParams.targetGreeksSize,1);
    if strcmp(optParams.targetFunctionType,'RelativeError')
        % cost function 
        for i=1:optParams.targetGreeksSize
            targetGreeks = optParams.targetGreeks{i,1};
            ELSGreeksColumn = ELS{:,{targetGreeks}};
            optionGreeksColumn = zeros(length(ELSGreeksColumn),sizeOfOptions);

            for idxhong=1:sizeOfOptions
                targetOptionSingle = targetOptions{idxhong,1};
                optionGreeksColumn(:,idxhong) = targetOptionSingle{:,{targetGreeks}};
            end

            for j= lowerIndex:upperIndex
                if j == midIndex
                    if abs(ELSGreeksColumn(j)) < 1e-4
                        ceq(i) = -1.0*ELSGreeksColumn(j);
                        for k= 1 :sizeOfOptions
    %                         out(idx) = out(idx) + smoothingWeight(i)*optionWeight(k)*(optionGreeksColumn(j,k));
                            ceq(i) = ceq(i) +smoothingWeight(i)*optionWeight(k)*(optionGreeksColumn(j,k));
                        end
                    else
    %                     tempOut =  -1.0*ELSGreeksColumn(j);
                        ceq(i) =  -1.0;

                        for k= 1 :sizeOfOptions
    %                         out(idx) = out(idx) + smoothingWeight(i)*optionWeight(k)*(optionGreeksColumn(j,k)/ELSGreeksColumn(j));
                            ceq(i) = ceq(i) +smoothingWeight(i)*optionWeight(k)*(optionGreeksColumn(j,k))/ELSGreeksColumn(j);
                        end
                    end
                elseif j == OTMLongVegaIndex
                    c(i) =  -1.0;

                    for k= 1 :sizeOfOptions
%                         out(idx) = out(idx) + smoothingWeight(i)*optionWeight(k)*(optionGreeksColumn(j,k)/ELSGreeksColumn(j));
                        c(i) = c(i) +smoothingWeight(i)*optionWeight(k)*(optionGreeksColumn(j,k))/ELSGreeksColumn(j);
                    end
                end
%                 out = out + tempOut*tempOut;
                idx = idx + 1;
            end
        end
    else
        % cost function 
        for i=1:optParams.targetGreeksSize
            targetGreeks = optParams.targetGreeks{i,1};
            ELSGreeksColumn = ELS{:,{targetGreeks}};
            optionGreeksColumn = zeros(length(ELSGreeksColumn),sizeOfOptions);

            for idxhong=1:sizeOfOptions
                targetOptionSingle = targetOptions{idxhong,1};
                optionGreeksColumn(:,idxhong) = targetOptionSingle{:,{targetGreeks}};
            end

            for j= lowerIndex:upperIndex
                if j == midIndex
%                     out(idx) = -1.0*ELSGreeksColumn(j);
%                     tempOut =  -1.0*ELSGreeksColumn(j);
                    ceq(i) =  -1.0*ELSGreeksColumn(j);
                    for k= 1 :sizeOfOptions
%                         out(idx) = out(idx) + 100.0*smoothingWeight(i)*optionWeight(k)*(optionGreeksColumn(j,k));
                        ceq(i) = ceq(i) +smoothingWeight(i)*optionWeight(k)*(optionGreeksColumn(j,k));
                    end
                elseif j== OTMLongVegaIndex
%                     out(idx) = -1.0*ELSGreeksColumn(j);
%                     tempOut =  -1.0*ELSGreeksColumn(j);
                    c(i) =  -1.0*ELSGreeksColumn(j);
                    for k= 1 :sizeOfOptions
%                         out(idx) = out(idx) + smoothingWeight(i)*optionWeight(k)*(optionGreeksColumn(j,k));
                        c(i) = c(i) +smoothingWeight(i)*optionWeight(k)*(optionGreeksColumn(j,k));
                    end
                    c(i) = c(i) * -1.0;
                end
%                 out = out + tempOut*tempOut;
                idx = idx + 1;
            end
        end
        
    end
    
    aaa= 1.0;
end





In [ ]:
function  [c, ceq]  = ConstraintFunctionELSHedgeATMVegaZero(tvar,optParams)
%     K_swaption = optParams.params.diagSwaption.strike;
%     black_swaption = optParams.params.diagSwaption.blackPrice;
%     swaption_expiry_use_yn = optParams.params.swaption_expiry_use_yn;
%     dt= 1.0/(optParams.params.freq);

    c = [];
    
    ELS = optParams.ELS;
    targetOptions = optParams.targetOptions;
    sizeOfOptions = optParams.sizeOfOptions;
    smoothingWeight = optParams.smoothingWeight;
    
    midIndex = optParams.midIndex;
    lowerIndex = optParams.midIndex - optParams.lowerNum;
    upperIndex = optParams.midIndex + optParams.upperNum;
    
    optionWeight = tvar;
    
    idx =1;
    
    ceq = zeros(optParams.targetGreeksSize,1);
    if strcmp(optParams.targetFunctionType,'RelativeError')
        % cost function 
        for i=1:optParams.targetGreeksSize
            targetGreeks = optParams.targetGreeks{i,1};
            ELSGreeksColumn = ELS{:,{targetGreeks}};
            optionGreeksColumn = zeros(length(ELSGreeksColumn),sizeOfOptions);

            for idxhong=1:sizeOfOptions
                targetOptionSingle = targetOptions{idxhong,1};
                optionGreeksColumn(:,idxhong) = targetOptionSingle{:,{targetGreeks}};
            end

            for j= midIndex:midIndex
                if abs(ELSGreeksColumn(j)) < 1e-4
                    ceq(i) = -1.0*ELSGreeksColumn(j);
                    for k= 1 :sizeOfOptions
%                         out(idx) = out(idx) + smoothingWeight(i)*optionWeight(k)*(optionGreeksColumn(j,k));
                        ceq(i) = ceq(i) +smoothingWeight(i)*optionWeight(k)*(optionGreeksColumn(j,k));
                    end
                else
%                     tempOut =  -1.0*ELSGreeksColumn(j);
                    ceq(i) =  -1.0;
                    
                    for k= 1 :sizeOfOptions
%                         out(idx) = out(idx) + smoothingWeight(i)*optionWeight(k)*(optionGreeksColumn(j,k)/ELSGreeksColumn(j));
                        ceq(i) = ceq(i) +smoothingWeight(i)*optionWeight(k)*(optionGreeksColumn(j,k))/ELSGreeksColumn(j);
                    end
                end
%                 out = out + tempOut*tempOut;
                idx = idx + 1;
            end
        end
    else
        % cost function 
        for i=1:optParams.targetGreeksSize
            targetGreeks = optParams.targetGreeks{i,1};
            ELSGreeksColumn = ELS{:,{targetGreeks}};
            optionGreeksColumn = zeros(length(ELSGreeksColumn),sizeOfOptions);

            for idxhong=1:sizeOfOptions
                targetOptionSingle = targetOptions{idxhong,1};
                optionGreeksColumn(:,idxhong) = targetOptionSingle{:,{targetGreeks}};
            end

            for j= midIndex:midIndex
                if j == midIndex
%                     out(idx) = -1.0*ELSGreeksColumn(j);
%                     tempOut =  -1.0*ELSGreeksColumn(j);
                    ceq(i) =  -1.0*ELSGreeksColumn(j);
                    for k= 1 :sizeOfOptions
%                         out(idx) = out(idx) + 100.0*smoothingWeight(i)*optionWeight(k)*(optionGreeksColumn(j,k));
                        ceq(i) = ceq(i) +smoothingWeight(i)*optionWeight(k)*(optionGreeksColumn(j,k));
                    end
                else
%                     out(idx) = -1.0*ELSGreeksColumn(j);
%                     tempOut =  -1.0*ELSGreeksColumn(j);
                    ceq(i) =  -1.0*ELSGreeksColumn(j);
                    for k= 1 :sizeOfOptions
%                         out(idx) = out(idx) + smoothingWeight(i)*optionWeight(k)*(optionGreeksColumn(j,k));
                        ceq(i) = ceq(i) +smoothingWeight(i)*optionWeight(k)*(optionGreeksColumn(j,k));
                    end
                end
%                 out = out + tempOut*tempOut;
                idx = idx + 1;
            end
        end
        
    end
    
    aaa= 1.0;
end





In [ ]:
classdef CMSSpreadRangeAccrualSwap < Security
    %RangeAccrual is the SuperClass of all IRModel
    % properties: zeroCurve 
    % methods : DF, FWD,PV01,FSR
    properties(SetAccess = public)
        nominal
        coupon
        couponFreq
        tenorL
        tenorS
        lower
        upper
        rangeDays
        callYN
        schedule
        couponSchedule
        callSchedule
        manualScheduleYN
        
        %floatingLeg
        floatingSchedule
        floatingSpread
        floatingIndex
        
        allFixings
        
    end
    
    methods
        % constructor
        function rangeAccrual = CMSSpreadRangeAccrualSwap(params,swapParams)
                rangeAccrual.payCcy = params.payCcy;
                rangeAccrual.startDate = params.startDate;
                rangeAccrual.maturity = params.maturity;
                
                %rangeAccrual.maturityDate = params.startDate.AddDate(params.maturity,'year');
                rangeAccrual.nominal = params.params('nominal');
                rangeAccrual.coupon = params.params('coupon');
                rangeAccrual.couponFreq= params.params('couponFreq');
                rangeAccrual.tenorL = params.params('tenorL');
                rangeAccrual.tenorS = params.params('tenorS');
                rangeAccrual.lower = params.params('lower');
                rangeAccrual.upper = params.params('upper');
                rangeAccrual.rangeDays = params.params('rangeDays');
                rangeAccrual.callYN = params.params('callYN');
                rangeAccrual.manualScheduleYN = params.params('manualScheduleYN');
                if rangeAccrual.manualScheduleYN
                    rangeAccrual.couponSchedule = rangeAccrual.ManualScheduleGenerate(params.params('couponScheduleInt'));
                    rangeAccrual.callSchedule = rangeAccrual.ManualScheduleGenerate(params.params('callScheduleInt'));
                    rangeAccrual.floatingSchedule = rangeAccrual.ManualScheduleGenerate(swapParams.params('floatingScheduleInt'));
                
                else
                    maturityDate = params.startDate.AddDate(params.maturity,'year');
                    rangeAccrual.schedule = rangeAccrual.ScheduleGenerate(rangeAccrual.startDate,maturityDate,rangeAccrual.couponFreq);
                
                end
                
                 %floatingLeg
                rangeAccrual.floatingSpread = swapParams.params('floatingSpread');
                rangeAccrual.floatingIndex = swapParams.params('floatingIndex');
                
                rangeAccrual.allFixings = swapParams.params('allFixings'); 
        end
        
        function out = ManualScheduleGenerate(rangeAccrual,scheduleInt)
            scheduleSize = size(scheduleInt,1);
            out.numOfPeriod = scheduleSize;
            for i=1:scheduleSize
                out.resetDates(i) = H_Date(scheduleInt(i,1));
                out.startDates(i) = H_Date(scheduleInt(i,2));
                out.endDates(i) = H_Date(scheduleInt(i,3));
                out.payDates(i) = H_Date(scheduleInt(i,4));
                out.dayCount(i) = out.endDates(i).DateDiff(out.startDates(i));
                out.dayCountFraction(i) = out.dayCount(i)/ 365.0;
            end
            
            
        end
        
        function out = ScheduleGenerate(rangeAccrual,initialDate,maturityDate,freq)
            schedule = ScheduleGenerate(initialDate,maturityDate,freq);
            out.startDates= schedule.startDates;
            out.resetDates = schedule.resetDates;
            out.endDates = schedule.endDates;
            out.payDates = schedule.payDates;
            out.dayCount = schedule.dayCount;
            out.dayCountFraction = schedule.dayCountFraction;
            out.numOfPeriod = schedule.numOfPeriod;
        end
        
        
    end
    
end



In [ ]:
classdef CMSSpreadAndEquityDualRAPutAtMat < Security
    %RangeAccrual is the SuperClass of all IRModel
    % properties: zeroCurve 
    % methods : DF, FWD,PV01,FSR
    properties(SetAccess = public)
        nominal
        coupon
        couponFreq
        tenorL
        tenorS
        lowerS
        upperS
        lowerEQ
        upperEQ
        putAtMatStrike
        rangeDays
        callYN
        schedule
        couponSchedule
        callSchedule
        manualScheduleYN
        hedgeBuffer
    end
    
    methods
        % constructor
        function rangeAccrual = CMSSpreadAndEquityDualRAPutAtMat(params)
                rangeAccrual.payCcy = params.payCcy;
                rangeAccrual.startDate = params.startDate;
                rangeAccrual.maturity = params.maturity;
                
                %rangeAccrual.maturityDate = params.startDate.AddDate(params.maturity,'year');
                rangeAccrual.nominal = params.params('nominal');
                rangeAccrual.coupon = params.params('coupon');
                rangeAccrual.couponFreq= params.params('couponFreq');
                rangeAccrual.tenorL = params.params('tenorL');
                rangeAccrual.tenorS = params.params('tenorS');
                rangeAccrual.lowerS = params.params('lowerS');
                rangeAccrual.upperS = params.params('upperS');
                rangeAccrual.lowerEQ = params.params('lowerEQ');
                rangeAccrual.upperEQ = params.params('upperEQ');
                rangeAccrual.putAtMatStrike = params.params('putAtMatStrike');
                rangeAccrual.rangeDays = params.params('rangeDays');
                rangeAccrual.callYN = params.params('callYN');
                rangeAccrual.manualScheduleYN = params.params('manualScheduleYN');
                if rangeAccrual.manualScheduleYN
                    rangeAccrual.couponSchedule = rangeAccrual.ManualScheduleGenerate(params.params('couponScheduleInt'));
                    rangeAccrual.callSchedule = rangeAccrual.ManualScheduleGenerate(params.params('callScheduleInt'));
                    
                
                else
                    maturityDate = params.startDate.AddDate(params.maturity,'year');
                    rangeAccrual.schedule = rangeAccrual.ScheduleGenerate(rangeAccrual.startDate,maturityDate,rangeAccrual.couponFreq);
                
                end
                
                if isKey(params.params,'hedgeBuffer')
                    rangeAccrual.hedgeBuffer = params.params('hedgeBuffer');
                else
                    rangeAccrual.hedgeBuffer = 0.0;
                end
                 
        end
        
        function out = ManualScheduleGenerate(rangeAccrual,scheduleInt)
            scheduleSize = size(scheduleInt,1);
            out.numOfPeriod = scheduleSize;
            for i=1:scheduleSize
                out.resetDates(i) = H_Date(scheduleInt(i,1));
                out.startDates(i) = H_Date(scheduleInt(i,2));
                out.endDates(i) = H_Date(scheduleInt(i,3));
                out.payDates(i) = H_Date(scheduleInt(i,4));
                out.dayCount(i) = out.endDates(i).DateDiff(out.startDates(i));
                out.dayCountFraction(i) = out.dayCount(i)/ 365.0;
            end
            
            
        end
        
        function out = ScheduleGenerate(rangeAccrual,initialDate,maturityDate,freq)
            schedule = ScheduleGenerate(initialDate,maturityDate,freq);
            out.startDates= schedule.startDates;
            out.resetDates = schedule.resetDates;
            out.endDates = schedule.endDates;
            out.payDates = schedule.payDates;
            out.dayCount = schedule.dayCount;
            out.dayCountFraction = schedule.dayCountFraction;
            out.numOfPeriod = schedule.numOfPeriod;
        end
        
        function out = numMethodTimeStepsGenerate(rangeAccrual,valueDate,schedule,mcOneTimeStep)
            
            periodCount = schedule.numOfPeriod;
            out.timeStepsPerPeriod =  cell(periodCount,1);
            
            out.startTimeIdx = zeros(periodCount,1);
            out.endM1TimeIdx = zeros(periodCount,1);
            
            out.numOfTimeSteps = zeros(periodCount,1);
            out.totalTimeSteps = [];
            out.totalTimeStepsSize = 0;
            numOfAlivePeriod = 0;
            
            for i=1:periodCount
                timeSteps = [];
                if DateDiff(valueDate,schedule.payDates(i)) >= 0
                    out.timeStepsPerPeriod(i) = timeSteps;
                    continue;
                end
                
                numOfAlivePeriod = numOfAlivePeriod + 1;
                if DateDiff(valueDate,schedule.startDates(i)) >= 0
                    startDate = valueDate;
                    endDate = schedule.endDates(i);
                    % we exclude end date from the 
                    endDate = endDate.AddDate(-1,'day');
                    
                    timeSteps = [];
                    % startDate should be included
                    startTime = startDate.DateDiff(valueDate);
                    timeSteps = [timeSteps; startTime];
                    
                    while 1
                        endTime = endDate.DateDiff(valueDate);
                        timeSteps = [timeSteps; endTime];
                        endDate = endDate.AddDate(-1*mcOneTimeStep,'day');
                        if endDate.DateDiff(startDate) < 0 break;end
                    end
                    timeSteps = sort(unique(timeSteps),'ascend');
                    %out.timeStepsPerPeriod {i,1}.timeSteps = [];
                    out.timeStepsPerPeriod{i,1}.timeSteps = timeSteps;
                    out.numOfTimeSteps(i) = length(timeSteps);
                    out.totalTimeSteps = [out.totalTimeSteps; timeSteps];
                    
                    % 1st period
                    out.startTimeIdx(i) = 1;
                    out.endM1TimeIdx(i) = length(timeSteps);
                    
                else
                    startDate = schedule.startDates(i);
                    endDate = schedule.endDates(i);
                    % we exclude end date from the 
                    endDate = endDate.AddDate(-1,'day');
                    
                    timeSteps = [];
                    % startDate should be included
                    startTime = startDate.DateDiff(valueDate);
                    timeSteps = [timeSteps; startTime];
                    
                    while 1
                        endTime = endDate.DateDiff(valueDate);
                        timeSteps = [timeSteps; endTime];
                        
                        endDate = endDate.AddDate(-1*mcOneTimeStep,'day');
                        if endDate.DateDiff(startDate) < 0 break;end
                    end
                    timeSteps = sort(unique(timeSteps),'ascend');
                    out.timeStepsPerPeriod{i,1}.timeSteps = timeSteps;
                    out.numOfTimeSteps(i) = length(timeSteps);
                    out.totalTimeSteps = [out.totalTimeSteps; timeSteps];
                    
                    % 1st period
                    out.startTimeIdx(i) = out.endM1TimeIdx(i-1) + 1;
                    out.endM1TimeIdx(i) = out.endM1TimeIdx(i-1) + length(timeSteps);
                    
                end
            end
            
            % endDate of the last period (= Security's maturityDate should
            % be included
            
            endTime = DateDiff(schedule.endDates(periodCount),valueDate);
            out.totalTimeSteps = [out.totalTimeSteps;endTime];
            out.totalTimeSteps = sort(unique(out.totalTimeSteps),'ascend');
            out.totalTimeStepsSize = length(out.totalTimeSteps);
            
            out.periodCount = periodCount;
            out.numOfAlivePeriod = numOfAlivePeriod;
        end
        
        function out = generateModelStates(rangeAccrual,valueDate,schedule,numMethodInfo,model)
            
            pathSize = numMethodInfo.pathSize;
            numOfFactors = numMethodInfo.numOfFactors;
            totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
            totalTimeSteps = numMethodInfo.totalTimeSteps;
            Z = numMethodInfo.Z;
            modelStates = zeros(numOfFactors*totalTimeStepsSize,pathSize);
            
            currentStep = totalTimeSteps(1);
            for i=1:totalTimeStepsSize-1
                nextStep = totalTimeSteps(i+1);
                sqrtDt = sqrt((nextStep-currentStep)/365.0);
                stateLocalVariance = model.stateLocalVariance(currentStep,nextStep);
                L = chol(stateLocalVariance);
                prevModelStates = modelStates(numOfFactors*(i-1)+1:numOfFactors*(i-1)+2,:);
                dW = Z(numOfFactors*i+1:numOfFactors*i+2,:);
                modelStates(numOfFactors*i+1:numOfFactors*i+2,:) = prevModelStates + L*dW; 
                currentStep = nextStep;
            end
            
            out = modelStates;
        end
        
        function out = computePriceMCForward(rangeAccrual,valueDate,model)
            schedule = rangeAccrual.schedule;
            
            % numMethodTimeSteps Generate
            mcOneTimeStep = 1;
            outInfo = rangeAccrual.numMethodTimeStepsGenerate(valueDate,schedule,mcOneTimeStep);
            
            numMethodInfo.timeStepsInfo = outInfo;
            
            % mc random number init
            %pathSize = 2^16;
            pathSize = 5000;
            
            %pathSize = 2^10;
            
            rng('default');
            rng(0);
            
            numOfFactors = 2;
            
            Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
            Z=[Z,-Z];
            
            numMethodInfo.pathSize = pathSize;
            numMethodInfo.numOfFactors = numOfFactors;
            numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
            numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
            numMethodInfo.Z = Z;
            
            % numeraire info
            numMatTime = numMethodInfo.totalTimeSteps(end);
            dfNumMat = model.DF(numMatTime/365.0);
            
            %generate model states for the given timeSteps
            modelStates = rangeAccrual.generateModelStates(valueDate,schedule,numMethodInfo,model);
            
            % induct backward and calculate cashflow
            
            
            periodCount = schedule.numOfPeriod;
            
            nonCall.npv = 0;
            
            nonCall.cashflow_e = zeros(periodCount + 1,1);
            nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
            nonCall.rangeProb_e = zeros(periodCount + 1,1);
            
            nonCall.df_e = zeros(periodCount + 1,1);
            nonCall.df_c = zeros(periodCount + 1,1);
            
            % sum of cashflow_npv
            nonCall.payoff = zeros(1,pathSize);
            
            nonCall.payoffStateVectors_cashflow = zeros(periodCount+1,pathSize);
            nonCall.payoffStateVectors_cashflow_npv = zeros(periodCount+1,pathSize);
            nonCall.payoffStateVector_df = zeros(periodCount + 1,pathSize);
            
            payoffStateVectors_cashflow = zeros(1, pathSize);
            payoffStateVectors_cashflow_npv = zeros(1, pathSize);
            payoffStateVectors_df = zeros(1,pathSize);
            
            % we calculate cashflow backward from maturityDate to valueDate
            % rangeAccrual coupons are added at the startDate of the Period
            % At maturity
            
            nominal = rangeAccrual.nominal;
            coupon = rangeAccrual.coupon;
            
            currentTime = numMethodInfo.totalTimeSteps(end);
            totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
            currentTimeIdx = totalTimeStepsSize;
            
            modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
            payoffStateVectors_cashflow = nominal*ones(1,pathSize);
            nonCall.payoffStateVectors_cashflow(end,:) = payoffStateVectors_cashflow;
            
            payoffStateVectors_cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,payoffStateVectors_cashflow);
            
            nonCall.payoffStateVectors_cashflow_npv(end,:) = payoffStateVectors_cashflow_npv;
            
            nonCall.cashflow_npv_e(end) = mean(payoffStateVectors_cashflow_npv);
            
            nonCall.payoff = nonCall.payoff + payoffStateVectors_cashflow_npv;
            
            payoffStateVectors_df = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,1.0*ones(1,pathSize));
            
            nonCall.payoffStateVector_df(end,:) = payoffStateVectors_df;
            nonCall.df_e(end) = mean(nonCall.payoffStateVector_df(end,:));
            nonCall.df_c(end) = model.DF(currentTime/365.0);
%             
%             nonCall.cashflow_npv_df(end,:) =  nonCall.npv/nominal;
%             nonCall.cashflow_npv_df_e(end) = mean(nonCall.cashflow_npv_df(end,:));
            
            lastTimeIdx = currentTimeIdx;
            totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
            
            for i=periodCount:-1:1
                % past schedule neglect
                if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                endM1TimeIdx = numMethodInfo.timeStepsInfo.endM1TimeIdx(i);
                
                %% FromToInduction start
                
                %modelsStates induct backward(lastTimeIdx to endM1TimeIdx)
                while endM1TimeIdx < lastTimeIdx
                    lastTimeIdx = lastTimeIdx -1;
                    if lastTimeIdx == endM1TimeIdx break;end
                end
                
                currentTimeIdx = lastTimeIdx;
                currentTime = totalTimeSteps(currentTimeIdx);
                % now we are at endM1TimeIdx
                % variables for coupon valuation initialized
                ralower = rangeAccrual.lower;
                raupper = rangeAccrual.upper;
                
                ralowerS = rangeAccrual.lowerS;
                raupperS = rangeAccrual.upperS;
                
                tenorCMS = rangeAccrual.tenor;
                
                tenorLong = rangeAccrual.tenorL;
                tenorShort = rangeAccrual.tenorS;
                
                cD = 0; % cumulated day count fraction
                cP = 0; 
                dI = 0; % dummy idx for loop
                dcfC = 0;
                dcfN = 0;
                dD = 0;
                idxCMS = zeros(1,pathSize);
                
                idxL = zeros(1,pathSize);
                idxS = zeros(1,pathSize);
                
                idxC = zeros(1,pathSize);
                idxN = zeros(1,pathSize);
                
                spreadC = zeros(1,pathSize);
                spreadN = zeros(1,pathSize);
                probSpread = zeros(1,pathSize);
                simpleYN = true;
                numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                
                for j=numOfTimeSteps:-1:1
                    % evaluate dealdescription
                    % last step is omitted
                    currentTime = totalTimeSteps(currentTimeIdx);
                    dcfC = currentTime/365.0;
                    if j~= numOfTimeSteps
                        dD = (dcfN-dcfC);
                    end
                    
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
                    idxCMS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorCMS,modelStateT);
                    idxC = idxCMS;
                    
                    idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                    idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                    spreadC = idxL - idxS;
                     
%                     probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                    probSpread = model.DRPr(idxC,idxN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS,simpleYN,pathSize)*dD;
                    
                    cD = cD + dD;
                    cP = cP + probSpread;
  
                    dcfN = dcfC;
                    
                    idxN = idxC;
                    spreadN = spreadC;
                    
                    %induct backward to the next time step
                    % one time backward induction
                    lastTimeIdx = currentTimeIdx;
                    
                    if startTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        currentTimeIdx = lastTimeIdx;
                    end
                    
                    
                end
                
                %% FromToInduction end
                %% AtDate operation
                % Adding a new cashflow to the payoff
                
                dcf = schedule.dayCountFraction(i);
                payoffStateVectors_cashflow = nominal * coupon * dcf * cP/cD;
                
                % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                % i.e. currentTimeIdx has not changed
                % no need to upodate modelStateT
                % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                % since cashflow's are added at the startDate(i) we need to
                % multiply cashflow with the df(startDate(i), payDate(i))
                payTime = DateDiff(schedule.payDates(i),valueDate);
                df_se = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);
                
                for k=1:pathSize
                    payoffStateVectors_cashflow(k) = payoffStateVectors_cashflow(k) + df_se(k);
                end
                
                nonCall.payoffStateVectors_cashflow(i,:) = payoffStateVectors_cashflow;
                
                nonCall.cashflow_e(i) = mean(payoffStateVectors_cashflow);
                nonCall.rangeProb_e(i) = mean(nonCall.cashflow_e(i)/(nominal * coupon * dcf));
                
                payoffStateVectors_cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,payoffStateVectors_cashflow);
            
                nonCall.cashflow_npv_e(i) = mean(payoffStateVectors_cashflow_npv);
                
                nonCall.payoffStateVectors_cashflow_npv(i,:) = payoffStateVectors_cashflow_npv;
                nonCall.payoff = nonCall.payoff + payoffStateVectors_cashflow_npv;
                
                payoffStateVectors_df = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,1.0*ones(1,pathSize));
                
                nonCall.payoffStateVector_df(i,:) = payoffStateVectors_df;
                nonCall.df_e(i) = mean(nonCall.payoffStateVector_df(i,:));
                nonCall.df_c(i) = model.DF(currentTime/365.0);
                
            end
            
            nonCall.npv = mean(nonCall.payoff);
  
            out.cashflow= nonCall.cashflow_e;
            out.cashflowNpv = nonCall.cashflow_npv_e;
            out.df_e = nonCall.df_e;
            out.df_c = nonCall.df_c;
            out.daycountFraction = schedule.dayCountFraction(i);
            out.rangeProb = nonCall.rangeProb_e;
            out.npv = nonCall.npv;
           
        end
        
        function out = computePrice(rangeAccrual,valueDate,model)
            schedule = rangeAccrual.schedule;
            periodCount = schedule.numOfPeriod;
            rangeProb = zeros(periodCount,1);
            discountFactor = zeros(periodCount,1);
            daycountFraction = zeros(periodCount,1);
            cashflow = zeros(periodCount,1);
            cashflowNpv = zeros(periodCount,1);
            rangeDays = rangeAccrual.rangeDays;
            npv = 0;
            for i=1:periodCount
                if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end;
                
                observeDate = schedule.startDates(i);
                if DateDiff(valueDate,schedule.startDates(i)) >= 0
                    observeDate = valueDate;
                    observeDate = AddDate(observeDate,1,'day');
                else
                    observeDate = schedule.startDates(i);
                end
                
                observeDays = DateDiff(schedule.endDates(i),schedule.startDates(i));
                days = DateDiff(schedule.endDates(i), observeDate);
                discountFactor(i) = model.DF( DateDiff(schedule.payDates(i),valueDate)/365.0);
                daycountFraction(i) = schedule.dayCountFraction(i);
                for j=1:days
                    toDate = observeDate;
                    tpDate = schedule.payDates(i);
                    tenor1 = rangeAccrual.tenorL;
                    tenor2 = rangeAccrual.tenorS;
                    tau =0.25;
                    lower = rangeAccrual.lower;
                    upper = rangeAccrual.upper;
                    rangeValue = model.CMSSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,lower,upper);
%                     rangeValue = 1.0;
                    rangeProb(i) = rangeProb(i) + rangeValue;
                    observeDate = AddDate(observeDate,1,'day');
                end
                cashflow(i) = rangeAccrual.nominal * rangeAccrual.coupon * (rangeProb(i) + rangeDays(i))/observeDays * daycountFraction(i);
                cashflowNpv(i) = cashflow(i) * discountFactor(i);
                npv = npv + cashflowNpv(i);
                % bug fixed observeDays -> days
                rangeProb(i) = rangeProb(i)/days;
                
                if i == periodCount
                    npv = npv + rangeAccrual.nominal * discountFactor(i);
                end
            end
            out.cashflow= cashflow;
            out.cashflowNpv = cashflowNpv;
            out.discountFactor = discountFactor;
            out.daycountFraction = daycountFraction;
            out.rangeProb = rangeProb;
            out.npv = npv;
        end
        
%         function out = discountPayoff(rangeAccrual,eventTime,numMatTime ...
%                 ,model,modelStatesT,cashflow)
%             
%             dfNumMat = model.DF(numMatTime/365.0);
%             % for discountPayoff T=numMatTime
%             % P(0,TnumMat)/P(Te,Tnummat)*Payoff
%             %
%             discountT = model.discountFactor(eventTime, numMatTime,numMatTime,modelStatesT);
%             stateSize = size(modelStatesT,2);
%             payoff = zeros(1,stateSize);
%             for i=1:stateSize
%                 payoff(i)= dfNumMat/discountT(i)*cashflow(i);
%             end    
%             out = payoff;
%         end
        
        
        
    end
    
end



In [ ]:
classdef CMSRangeAccrual < Security
    %RangeAccrual is the SuperClass of all IRModel
    % properties: zeroCurve 
    % methods : DF, FWD,PV01,FSR
    properties(SetAccess = public)
        nominal
        coupon
        couponFreq
        tenorL
        lower
        upper
        rangeDays
        callYN
        schedule
        couponSchedule
        callSchedule
        manualScheduleYN
        
    end
    
    methods
        % constructor
        function rangeAccrual = CMSRangeAccrual(params)
                rangeAccrual.payCcy = params.payCcy;
                rangeAccrual.startDate = params.startDate;
                rangeAccrual.maturity = params.maturity;
                
                %rangeAccrual.maturityDate = params.startDate.AddDate(params.maturity,'year');
                rangeAccrual.nominal = params.params('nominal');
                rangeAccrual.coupon = params.params('coupon');
                rangeAccrual.couponFreq= params.params('couponFreq');
                rangeAccrual.tenorL = params.params('tenorL');
                rangeAccrual.lower = params.params('lower');
                rangeAccrual.upper = params.params('upper');
                rangeAccrual.rangeDays = params.params('rangeDays');
                rangeAccrual.callYN = params.params('callYN');
                rangeAccrual.manualScheduleYN = params.params('manualScheduleYN');
                if rangeAccrual.manualScheduleYN
                    rangeAccrual.couponSchedule = rangeAccrual.ManualScheduleGenerate(params.params('couponScheduleInt'));
                    rangeAccrual.callSchedule = rangeAccrual.ManualScheduleGenerate(params.params('callScheduleInt'));
                    
                
                else
                    maturityDate = params.startDate.AddDate(params.maturity,'year');
                    rangeAccrual.schedule = rangeAccrual.ScheduleGenerate(rangeAccrual.startDate,maturityDate,rangeAccrual.couponFreq);
                
                end
                 
        end
        
        function out = ManualScheduleGenerate(rangeAccrual,scheduleInt)
            scheduleSize = size(scheduleInt,1);
            out.numOfPeriod = scheduleSize;
            for i=1:scheduleSize
                out.resetDates(i) = H_Date(scheduleInt(i,1));
                out.startDates(i) = H_Date(scheduleInt(i,2));
                out.endDates(i) = H_Date(scheduleInt(i,3));
                out.payDates(i) = H_Date(scheduleInt(i,4));
                out.dayCount(i) = out.endDates(i).DateDiff(out.startDates(i));
                out.dayCountFraction(i) = out.dayCount(i)/ 365.0;
            end
            
            
        end
        
        function out = ScheduleGenerate(rangeAccrual,initialDate,maturityDate,freq)
            schedule = ScheduleGenerate(initialDate,maturityDate,freq);
            out.startDates= schedule.startDates;
            out.resetDates = schedule.resetDates;
            out.endDates = schedule.endDates;
            out.payDates = schedule.payDates;
            out.dayCount = schedule.dayCount;
            out.dayCountFraction = schedule.dayCountFraction;
            out.numOfPeriod = schedule.numOfPeriod;
        end
        
    end    
    
end



In [ ]:
classdef CMSRA_DailyWeeklyRiskyPricer < H_RootObject
    %Pricer(Security,Model,NumMethod)
    %Prcer is constructed from Security, Model, NumMethod 
    
    properties
        m_security
        m_model
        m_numMethod
        overHedgeYN
    end
    
    methods
        function pricer = CMSRA_DailyWeeklyRiskyPricer(security,model,numMethod,overHedgeYN)
                pricer.m_security = security;
                pricer.m_model = model;
                pricer.m_numMethod = numMethod;
                pricer.overHedgeYN = overHedgeYN;
        end
        
        function out = computePriceMCCallable(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                callSchedule = security.callSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;

                % generate random numbers and modelstates forward
                rng('default');
                rng(0);

                numOfFactors = 2;

                %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
                Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
                Z=[Z;-Z];
                Z=Z';
                
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;

                % we check with the c code, so we use generated random numbers
%                 mc_random = importdata('mc_random.xlsx');
%                 mc_random = importdata('D:\mc_random_number_new_report.txt');
%                 Z= mc_random;
                
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                numMethodInfo.Z = Z;



                %generate model states for the given timeSteps
                modelStates = numMethod.generateModelStates(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                days = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                numMatTime = max(numMatTime, days);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                numeraire.numMatTime = numMatTime;
                numeraire.dfNumMat = dfNumMat;
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % pricer single info initialize
                
                nonCall = PricerInfo(periodCount +1, pathSize);
                callable = PricerInfo(periodCount +1, pathSize);
                dfCol = PricerInfo(periodCount +1, pathSize);
                 
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                
            
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
                modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                
                payTime = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                %process payoff at maturity
                nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                %df column
                payoffStateA.cashflow = ones(1,pathSize);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                  

                lastTimeIdx = currentTimeIdx;
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                
                % flag for inclusive schedule
%                 simpleYN = false;
%                 simpleYN = true;
                if pricer.overHedgeYN == true 
                    simpleYN = 2;
                else
                    simpleYN = 1;
                end
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                 %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    ralower = security.lower;
                    raupper = security.upper;
                    tenorLong = security.tenorL;
%                     tenorShort = security.tenorS;

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                        if pricer.overHedgeYN == true 
                            simpleYN = 2;
                        end
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
                        idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                        spreadC = idxL;
%                         idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                         spreadC = idxL - idxS;
                        
                        probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
        % if the current period is inclusive, then the start date is not evaluated in the above for loop.
		% instead the fixing will be treated separately below.
		% however current_time_idx is updated at the end of the for loop.
		% so we need to update model_states accordingly.
		% if model_states were updated previously then the following additionaly step doesn't affect the result anyway.
                    currentTime = totalTimeSteps(currentTimeIdx);
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
                    
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays(i) + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
                    df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);

                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                    callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    %df column
                    payoffStateA.cashflow = ones(1,pathSize);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
                    if callYN(i) && (DateDiff(valueDate,callSchedule.resetDates(i)) < 0)
                        % 1.undiscount callable.payoff
                        % 2.build the state variables matrix
                        % 3.regress 1's with the given state variables
                        % (model states 1,2)
                        % 4. exercise per each path in
                        % discount the new payoff and assign into callable
                        % column
                        
                        %1 undiscount callable.payoff and get the
                        %contiValue
                       
                        %exercise value
                        exerciseValue = security.nominal*ones(1,pathSize);
                        payTime = DateDiff(callSchedule.payDates(i),valueDate);
                        %event is at startDate so we need to discount the
                        %cashflow first from paydate to startdate
                        df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                        for k=1:pathSize
                            exerciseValue(k) = exerciseValue(k) * df_ep(k);
                        end
                        
                        contiValue = zeros(1,pathSize);
                        onePayoff = ones(1,pathSize);
                        onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,onePayoff);
                        contiValue = callable.payoff./onePayoffD;
                        
                        % build the state variabls matrix
                        decisionVar1 = modelStateT(1,:);
                        decisionVar2 = modelStateT(2,:);
                        % we use upto 2nd order polynomial as regression
                        % equatioin
                        dimensionLS = 6;
                        regMatrix = ones(pathSize,dimensionLS);
                        
                        for idx_r=1:pathSize
                            regMatrix(idx_r,2) = decisionVar1(idx_r);
                            regMatrix(idx_r,3) = decisionVar2(idx_r);
                            regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r);
                            regMatrix(idx_r,5) = decisionVar1(idx_r)*decisionVar2(idx_r);
                            regMatrix(idx_r,6) = decisionVar2(idx_r)*decisionVar2(idx_r);
                        end
                        
                        % simple regression <- for path dependent option
                        % we should change it
                        % beta  : regression coefficients
                        beta =zeros(dimensionLS,1);
                        beta = regress(contiValue',regMatrix);
                        regContiValue = regMatrix*beta;
                        
                        exercisedPayoff = regContiValue;
                        % if regContiValue is bigger than exerciseValue
                        % then issue exercise call option
                        for idx_r=1:pathSize
                            if regContiValue(idx_r) >= exerciseValue(idx_r) 
                                exercisedPayoff(idx_r) = exerciseValue(idx_r);
                            end
                        end
                        
                        % we discount the new payoff and 
                        % assign it into the existing callable payoff
                        callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,exercisedPayoff);
                        
                    end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.nonCall = nonCall;
                callable.npv = mean(callable.payoff);
                out.callable = callable;
                out.dfCol = dfCol;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        function out = computePriceClosedTRZ(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;
                
                pathSize = 1;
                
%                 % generate random numbers and modelstates forward
%                 rng('default');
%                 rng(0);
% 
                numOfFactors = 2;
% 
%                 %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
%                 Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
%                 Z=[Z;-Z];
%                 Z=Z';
%                 
%                 
%                 %TimeInversion
%                 sizeZ = size(Z,1);
%                 A = zeros(sizeZ,sizeZ);
%                 for i=1:sizeZ
%                     A(i,sizeZ + 1 -i) = 1;
%                 end
%                 Z = A*Z;
% 
%                 % we check with the c, so we use generated random numbers
% %                 mc_random = importdata('mc_random.xlsx');
% %                 mc_random = importdata('D:\mc_random_number_new_report.txt');
% %                 Z= mc_random;
%                 
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
%                 numMethodInfo.Z = Z;
% 
% 
% 
%                 %generate model states for the given timeSteps
%                 modelStates = numMethod.generateModelStates(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                dfNumMat = model.DF(numMatTime/365.0);
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % cashflow columns initialize
                % 1st nonCall
                nonCall.npv = 0;
                nonCall.cashflow_e = zeros(periodCount + 1,1);
                nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
                nonCall.rangeProb_e = zeros(periodCount + 1,1);
                nonCall.df_e = zeros(periodCount + 1,1);
                nonCall.df_c = zeros(periodCount + 1,1);

                % debug information, cashflow & npv per path
                nonCall.payoff = zeros(1,pathSize);
                nonCall.payoffStates.cashflow = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.cashflow_npv = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.df = zeros(periodCount + 1,pathSize);
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                payoffStateA.df = zeros(1,pathSize);
                
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                totalTimeStepsSize = numMethodInfo.timeStepsInfo.totalTimeStepsSize;
                if totalTimeStepsSize == 0 || (totalTimeStepsSize == 1 && totalTimeSteps(1) == 0) 
                    out.nonCall = nonCall;
                    return;
                end
                
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
%                 modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                %process payoff at maturity
                nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                
%                 payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
%                 
                payoffStateA.cashflow_npv = model.DFRisky(currentTime/365.0)*payoffStateA.cashflow;
                
                nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv ;
                
%                 payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                payoffStateA.df = model.DFRisky(currentTime/365.0) *ones(1,pathSize);
                    

                nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                
                nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);
                nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                
%                 % callable at maturity is the same as noncallable
%                 callable.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:);
%                 callable.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 callable.payoff = callable.payoff + callable.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 
%                 callable.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx);
%                 callable.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx);
                  
                lastTimeIdx = currentTimeIdx;
                
                
                % flag for inclusive schedule
%                 simpleYN = false;
                simpleYN = 1;
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                     %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    ralower = security.lower;
                    raupper = security.upper;
                    tenorLong = security.tenorL;
%                     tenorShort = security.tenorS;

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        toDate = AddDate(valueDate,currentTime,'day');
                        tpDate = schedule.payDates(i);
%                         spreadC = model.CMSSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorLong,tenorShort,0.25,ralower,raupper);
                        spreadC = model.CMSDigitalRangeDate(valueDate,toDate,tpDate,tenorLong,0.25,ralower,raupper);
                        
                        %modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
%                         idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
%                         idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                         spreadC = idxL - idxS;
%                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD; 

                        if simpleYN
                            probSpread = spreadC*dD;
                        else
                            probSpread = spreadC*dD;
%                             probSpread = 0.5*(spreadC + spreadN)*dD;
                            
                        end

                        
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays(i) + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
%                     df_sp = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);

%                     for k=1:pathSize
%                         payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_sp(k);
%                     end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    % 1st nonCall column
                    nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                    payoffStateA.cashflow_npv = payoffStateA.cashflow * model.DFRisky(payTime/365.0);
                    
%                     payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
                    nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv;
                    payoffStateA.df = model.DFRisky(payTime/365.0);
%                     payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                    nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                    
                    nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                    
                    nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                    nonCall.rangeProb_e(currentNodeIdx) = mean(nonCall.cashflow_e(currentNodeIdx)/(nominal * coupon * dcf));
                    nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);

                    nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                    nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                    
%                     %2nd callable column
%                     callable.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:);
%                     callable.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:);
%                     callable.payoff = callable.payoff + callable.payoffStates.cashflow_npv(currentNodeIdx,:);
%                     
%                     callable.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx);
%                     callable.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx);

                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
%                     if callYN(i) && (DateDiff(valueDate,schedule.startDates(i)) < 0)
%                         % 1.undiscount callable.payoff
%                         % 2.build the state variables matrix
%                         % 3.regress 1's with the given state variables
%                         % (model states 1,2)
%                         % 4. exercise per each path in
%                         % discount the new payoff and assign into callable
%                         % column
%                         
%                         %1 undiscount callable.payoff and get the
%                         %contiValue
%                        
%                         %exercise value
%                         exerciseValue = security.nominal*ones(1,pathSize);
%                         contiValue = zeros(1,pathSize);
%                         onePayoff = ones(1,pathSize);
%                         onePayoffD = model.discountPayoff(currentTime,numMatTime ...
%                         ,modelStateT,onePayoff);
%                         contiValue = callable.payoff./onePayoffD;
%                         
%                         % build the state variabls matrix
%                         decisionVar1 = modelStateT(1,:);
%                         decisionVar2 = modelStateT(2,:);
%                         % we use upto 2nd order polynomial as regression
%                         % equatioin
%                         dimensionLS = 6;
%                         regMatrix = ones(pathSize,dimensionLS);
%                         
%                         for idx_r=1:pathSize
%                             regMatrix(idx_r,2) = decisionVar1(idx_r);
%                             regMatrix(idx_r,3) = decisionVar2(idx_r);
%                             regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r);
%                             regMatrix(idx_r,5) = decisionVar1(idx_r)*decisionVar2(idx_r);
%                             regMatrix(idx_r,6) = decisionVar2(idx_r)*decisionVar2(idx_r);
%                         end
%                         
%                         % simple regression <- for path dependent option
%                         % we should change it
%                         % beta  : regression coefficients
%                         beta =zeros(dimensionLS,1);
%                         beta = regress(contiValue',regMatrix);
%                         regContiValue = regMatrix*beta;
%                         
%                         exercisedPayoff = regContiValue;
%                         % if regContiValue is bigger than exerciseValue
%                         % then issue exercise call option
%                         for idx_r=1:pathSize
%                             if regContiValue(idx_r) >= exerciseValue(idx_r) 
%                                 exercisedPayoff(idx_r) = exerciseValue(idx_r);
%                             end
%                         end
%                         
%                         % we discount the new payoff and 
%                         % assign it into the existing callable payoff
%                         callable.payoff = model.discountPayoff(currentTime,numMatTime ...
%                         ,modelStateT,exercisedPayoff);
%                         
%                     end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.pricerInfo.nonCall = nonCall;
                
%                 callable.npv = mean(callable.payoff);
%                 out.pricerInfo.callable = callable;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        
        
        
    end
    
end



In [ ]:
classdef CMSCMSSpreadDualRangeAccrualOld < Security
    %RangeAccrual is the SuperClass of all IRModel
    % properties: zeroCurve 
    % methods : DF, FWD,PV01,FSR
    properties(SetAccess = public)
        nominal
        coupon
        couponFreq
        tenor
        tenorL
        tenorS
        lower
        upper
        lowerS
        upperS
        rangeDays
        callYN
        schedule
        couponSchedule
        callSchedule
        manualScheduleYN
        
    end
    
    methods
        % constructor
        function rangeAccrual = CMSCMSSpreadDualRangeAccrualOld(params)
                security = Security(params);
                rangeAccrual.payCcy = security.payCcy;
                rangeAccrual.startDate = security.startDate;
                rangeAccrual.endDate = security.endDate;
                rangeAccrual.maturity = security.maturity;
                
                %rangeAccrual.maturityDate = params.startDate.AddDate(params.maturity,'year');
                rangeAccrual.nominal = params.params('nominal');
                rangeAccrual.coupon = params.params('coupon');
                rangeAccrual.couponFreq= params.params('couponFreq');
                rangeAccrual.tenor = params.params('tenor');
                rangeAccrual.tenorL = params.params('tenorL');
                rangeAccrual.tenorS = params.params('tenorS');
                rangeAccrual.lower = params.params('lower');
                rangeAccrual.upper = params.params('upper');
                rangeAccrual.lowerS = params.params('lowerS');
                rangeAccrual.upperS = params.params('upperS');
                rangeAccrual.rangeDays = params.params('rangeDays');
                rangeAccrual.callYN = params.params('callYN');
                rangeAccrual.manualScheduleYN = params.params('manualScheduleYN');
                if rangeAccrual.manualScheduleYN
                    rangeAccrual.couponSchedule = rangeAccrual.ManualScheduleGenerate(params.params('couponScheduleInt'));
                    rangeAccrual.callSchedule = rangeAccrual.ManualScheduleGenerate(params.params('callScheduleInt'));
%                     rangeAccrual.callSchedule = rangeAccrual.ManualCallScheduleGenerate(params.params('callScheduleInt'));
                    
                
                else
                    maturityDate = params.startDate.AddDate(params.maturity,'year');
                    rangeAccrual.schedule = rangeAccrual.ScheduleGenerate(rangeAccrual.startDate,maturityDate,rangeAccrual.couponFreq);
                
                end
                 
        end
        
        function out = ManualScheduleGenerate(rangeAccrual,scheduleInt)
            scheduleSize = size(scheduleInt,1);
            out.numOfPeriod = scheduleSize;
            for i=1:scheduleSize
                out.resetDates(i) = H_Date(scheduleInt(i,1));
                out.startDates(i) = H_Date(scheduleInt(i,2));
                out.endDates(i) = H_Date(scheduleInt(i,3));
                out.payDates(i) = H_Date(scheduleInt(i,4));
                out.dayCount(i) = out.endDates(i).DateDiff(out.startDates(i));
                out.dayCountFraction(i) = out.dayCount(i)/ 365.0;
            end
            
            
        end
        
        function out = ManualCallScheduleGenerate(rangeAccrual,scheduleInt)
            scheduleSize = size(scheduleInt,1);
            out.numOfPeriod = scheduleSize;
            for i=1:scheduleSize
                out.resetDates(i) = H_Date(scheduleInt(i,1));
                out.payDates(i) = H_Date(scheduleInt(i,2));
            end
            
            
        end
        
        function out = ScheduleGenerate(rangeAccrual,initialDate,maturityDate,freq)
            schedule = ScheduleGenerate(initialDate,maturityDate,freq);
            out.startDates= schedule.startDates;
            out.resetDates = schedule.resetDates;
            out.endDates = schedule.endDates;
            out.payDates = schedule.payDates;
            out.dayCount = schedule.dayCount;
            out.dayCountFraction = schedule.dayCountFraction;
            out.numOfPeriod = schedule.numOfPeriod;
        end
        

        
        
    end
    
end



In [ ]:
classdef CMSCMSSpreadDualRangeAccrualGeneric < Security
    %RangeAccrual is the SuperClass of all IRModel
    % properties: zeroCurve 
    % methods : DF, FWD,PV01,FSR
    properties(SetAccess = public)
        nominal
        coupon
        couponFreq
        tenor
        tenorL
        tenorS
        lower
        upper
        lowerS
        upperS
        rangeDays
        callYN
        schedule
        couponSchedule
        callSchedule
        manualScheduleYN
        
        tenorAssetName
        tenorLAssetName
        tenorSAssetName
        
        tenorRateType
        tenorLRateType
        tenorSRateType
    end
    
    methods
        % constructor
        function rangeAccrual = CMSCMSSpreadDualRangeAccrualGeneric(params)
                security = Security(params);
                rangeAccrual.payCcy = security.payCcy;
                rangeAccrual.startDate = security.startDate;
                rangeAccrual.endDate = security.endDate;
                rangeAccrual.maturity = security.maturity;
                
                %rangeAccrual.maturityDate = params.startDate.AddDate(params.maturity,'year');
                rangeAccrual.nominal = params.params('nominal');
                rangeAccrual.coupon = params.params('coupon');
                rangeAccrual.couponFreq= params.params('couponFreq');
                rangeAccrual.tenor = params.params('tenor');
                rangeAccrual.tenorL = params.params('tenorL');
                rangeAccrual.tenorS = params.params('tenorS');
                
                rangeAccrual.tenorAssetName = params.params('tenorAssetName');
                rangeAccrual.tenorLAssetName = params.params('tenorLAssetName');
                rangeAccrual.tenorSAssetName = params.params('tenorSAssetName');
                
                rangeAccrual.tenorRateType = params.params('tenorRateType');
                rangeAccrual.tenorLRateType = params.params('tenorLRateType');
                rangeAccrual.tenorSRateType = params.params('tenorSRateType');
                
                
                rangeAccrual.lower = params.params('lower');
                rangeAccrual.upper = params.params('upper');
                rangeAccrual.lowerS = params.params('lowerS');
                rangeAccrual.upperS = params.params('upperS');
                rangeAccrual.rangeDays = params.params('rangeDays');
                rangeAccrual.callYN = params.params('callYN');
                rangeAccrual.manualScheduleYN = params.params('manualScheduleYN');
                if rangeAccrual.manualScheduleYN
                    rangeAccrual.couponSchedule = rangeAccrual.ManualScheduleGenerate(params.params('couponScheduleInt'));
%                     rangeAccrual.callSchedule = rangeAccrual.ManualScheduleGenerate(params.params('callScheduleInt'));
                    rangeAccrual.callSchedule = rangeAccrual.ManualCallScheduleGenerate(params.params('callScheduleInt'));
                    
                
                else
                    maturityDate = params.startDate.AddDate(params.maturity,'year');
                    rangeAccrual.schedule = rangeAccrual.ScheduleGenerate(rangeAccrual.startDate,maturityDate,rangeAccrual.couponFreq);
                
                end
                 
        end
        
        function out = ManualScheduleGenerate(rangeAccrual,scheduleInt)
            scheduleSize = size(scheduleInt,1);
            out.numOfPeriod = scheduleSize;
            for i=1:scheduleSize
                out.resetDates(i) = H_Date(scheduleInt(i,1));
                out.startDates(i) = H_Date(scheduleInt(i,2));
                out.endDates(i) = H_Date(scheduleInt(i,3));
                out.payDates(i) = H_Date(scheduleInt(i,4));
                out.dayCount(i) = out.endDates(i).DateDiff(out.startDates(i));
                out.dayCountFraction(i) = out.dayCount(i)/ 365.0;
            end
            
            
        end
        
        function out = ManualCallScheduleGenerate(rangeAccrual,scheduleInt)
            scheduleSize = size(scheduleInt,1);
            out.numOfPeriod = scheduleSize;
            for i=1:scheduleSize
                out.resetDates(i) = H_Date(scheduleInt(i,1));
                out.payDates(i) = H_Date(scheduleInt(i,2));
            end
            
            
        end
        
        function out = ScheduleGenerate(rangeAccrual,initialDate,maturityDate,freq)
            schedule = ScheduleGenerate(initialDate,maturityDate,freq);
            out.startDates= schedule.startDates;
            out.resetDates = schedule.resetDates;
            out.endDates = schedule.endDates;
            out.payDates = schedule.payDates;
            out.dayCount = schedule.dayCount;
            out.dayCountFraction = schedule.dayCountFraction;
            out.numOfPeriod = schedule.numOfPeriod;
        end
        

        
        
    end
    
end



In [ ]:
classdef CMSCMSSpreadDualRangeAccrual < Security
    %RangeAccrual is the SuperClass of all IRModel
    % properties: zeroCurve 
    % methods : DF, FWD,PV01,FSR
    properties(SetAccess = public)
        nominal
        coupon
        couponFreq
        tenor
        tenorL
        tenorS
        lower
        upper
        lowerS
        upperS
        rangeDays
        callYN
        schedule
        couponSchedule
        callSchedule
        manualScheduleYN
        
    end
    
    methods
        % constructor
        function rangeAccrual = CMSCMSSpreadDualRangeAccrual(params)
                security = Security(params);
                rangeAccrual.payCcy = security.payCcy;
                rangeAccrual.startDate = security.startDate;
                rangeAccrual.endDate = security.endDate;
                rangeAccrual.maturity = security.maturity;
                
                %rangeAccrual.maturityDate = params.startDate.AddDate(params.maturity,'year');
                rangeAccrual.nominal = params.params('nominal');
                rangeAccrual.coupon = params.params('coupon');
                rangeAccrual.couponFreq= params.params('couponFreq');
                rangeAccrual.tenor = params.params('tenor');
                rangeAccrual.tenorL = params.params('tenorL');
                rangeAccrual.tenorS = params.params('tenorS');
                rangeAccrual.lower = params.params('lower');
                rangeAccrual.upper = params.params('upper');
                rangeAccrual.lowerS = params.params('lowerS');
                rangeAccrual.upperS = params.params('upperS');
                rangeAccrual.rangeDays = params.params('rangeDays');
                rangeAccrual.callYN = params.params('callYN');
                rangeAccrual.manualScheduleYN = params.params('manualScheduleYN');
                if rangeAccrual.manualScheduleYN
                    rangeAccrual.couponSchedule = rangeAccrual.ManualScheduleGenerate(params.params('couponScheduleInt'));
%                     rangeAccrual.callSchedule = rangeAccrual.ManualScheduleGenerate(params.params('callScheduleInt'));
                    rangeAccrual.callSchedule = rangeAccrual.ManualCallScheduleGenerate(params.params('callScheduleInt'));
                    
                
                else
                    maturityDate = params.startDate.AddDate(params.maturity,'year');
                    rangeAccrual.schedule = rangeAccrual.ScheduleGenerate(rangeAccrual.startDate,maturityDate,rangeAccrual.couponFreq);
                
                end
                 
        end
        
        function out = ManualScheduleGenerate(rangeAccrual,scheduleInt)
            scheduleSize = size(scheduleInt,1);
            out.numOfPeriod = scheduleSize;
            for i=1:scheduleSize
                out.resetDates(i) = H_Date(scheduleInt(i,1));
                out.startDates(i) = H_Date(scheduleInt(i,2));
                out.endDates(i) = H_Date(scheduleInt(i,3));
                out.payDates(i) = H_Date(scheduleInt(i,4));
                out.dayCount(i) = out.endDates(i).DateDiff(out.startDates(i));
                out.dayCountFraction(i) = out.dayCount(i)/ 365.0;
            end
            
            
        end
        
        function out = ManualCallScheduleGenerate(rangeAccrual,scheduleInt)
            scheduleSize = size(scheduleInt,1);
            out.numOfPeriod = scheduleSize;
            for i=1:scheduleSize
                out.resetDates(i) = H_Date(scheduleInt(i,1));
                out.payDates(i) = H_Date(scheduleInt(i,2));
            end
            
            
        end
        
        function out = ScheduleGenerate(rangeAccrual,initialDate,maturityDate,freq)
            schedule = ScheduleGenerate(initialDate,maturityDate,freq);
            out.startDates= schedule.startDates;
            out.resetDates = schedule.resetDates;
            out.endDates = schedule.endDates;
            out.payDates = schedule.payDates;
            out.dayCount = schedule.dayCount;
            out.dayCountFraction = schedule.dayCountFraction;
            out.numOfPeriod = schedule.numOfPeriod;
        end
        

        
        
    end
    
end



In [ ]:
classdef CMSAVGSteepner < Security
    %RangeAccrual is the SuperClass of all IRModel
    % properties: zeroCurve 
    % methods : DF, FWD,PV01,FSR
    properties(SetAccess = public)
        nominal
%         coupon
%         couponFreq
        localFloor
        localCap
        multiplier
        spread
        tenorL
        tenorS
%         lowerS
%         upperS
        rangeDays
        callYN
        schedule
        couponSchedule
        callSchedule
        manualScheduleYN
        isExercised
    end
    
    methods
        % constructor
        function rangeAccrual = CMSAVGSteepner(params)
                rangeAccrual.payCcy = params.payCcy;
                rangeAccrual.startDate = params.startDate;
                rangeAccrual.maturity = params.maturity;
                
                %rangeAccrual.maturityDate = params.startDate.AddDate(params.maturity,'year');
                rangeAccrual.nominal = params.params('nominal');
%                 rangeAccrual.coupon = params.params('coupon');
%                 rangeAccrual.couponFreq= params.params('couponFreq');

                rangeAccrual.localCap = params.params('localCap');
                rangeAccrual.localFloor = params.params('localFloor');
                rangeAccrual.multiplier = params.params('multiplier');
                rangeAccrual.spread = params.params('spread');
                
                rangeAccrual.tenorL = params.params('tenorL');
                rangeAccrual.tenorS = params.params('tenorS');
%                 rangeAccrual.lowerS = params.params('lowerS');
%                 rangeAccrual.upperS = params.params('upperS');
                rangeAccrual.rangeDays = params.params('rangeDays');
                rangeAccrual.callYN = params.params('callYN');
                rangeAccrual.manualScheduleYN = params.params('manualScheduleYN');
                if rangeAccrual.manualScheduleYN
                    rangeAccrual.couponSchedule = rangeAccrual.ManualScheduleGenerate(params.params('couponScheduleInt'));
                    rangeAccrual.callSchedule = rangeAccrual.ManualScheduleGenerate(params.params('callScheduleInt'));
                    
                
                else
                    maturityDate = params.startDate.AddDate(params.maturity,'year');
                    rangeAccrual.schedule = rangeAccrual.ScheduleGenerate(rangeAccrual.startDate,maturityDate,rangeAccrual.couponFreq);
                
                end
                
                if isfield(params, 'isExercised')
                    rangeAccrual.isExercised = params.isExercised;
                else
                    rangeAccrual.isExercised = false;  % default value is false
                end
        end
        
        function out = ManualScheduleGenerate(rangeAccrual,scheduleInt)
            scheduleSize = size(scheduleInt,1);
            out.numOfPeriod = scheduleSize;
            for i=1:scheduleSize
                out.resetDates(i) = H_Date(scheduleInt(i,1));
                out.startDates(i) = H_Date(scheduleInt(i,2));
                out.endDates(i) = H_Date(scheduleInt(i,3));
                out.payDates(i) = H_Date(scheduleInt(i,4));
                out.dayCount(i) = out.endDates(i).DateDiff(out.startDates(i));
                out.dayCountFraction(i) = out.dayCount(i)/ 365.0;
            end
            
            
        end
        
        function out = ScheduleGenerate(rangeAccrual,initialDate,maturityDate,freq)
            schedule = ScheduleGenerate(initialDate,maturityDate,freq);
            out.startDates= schedule.startDates;
            out.resetDates = schedule.resetDates;
            out.endDates = schedule.endDates;
            out.payDates = schedule.payDates;
            out.dayCount = schedule.dayCount;
            out.dayCountFraction = schedule.dayCountFraction;
            out.numOfPeriod = schedule.numOfPeriod;
        end
        
        function out = numMethodTimeStepsGenerate(rangeAccrual,valueDate,schedule,mcOneTimeStep)
            
            periodCount = schedule.numOfPeriod;
            out.timeStepsPerPeriod =  cell(periodCount,1);
            
            out.startTimeIdx = zeros(periodCount,1);
            out.endM1TimeIdx = zeros(periodCount,1);
            
            out.numOfTimeSteps = zeros(periodCount,1);
            out.totalTimeSteps = [];
            out.totalTimeStepsSize = 0;
            numOfAlivePeriod = 0;
            
            for i=1:periodCount
                timeSteps = [];
                if DateDiff(valueDate,schedule.payDates(i)) >= 0
                    out.timeStepsPerPeriod(i) = timeSteps;
                    continue;
                end
                
                numOfAlivePeriod = numOfAlivePeriod + 1;
                if DateDiff(valueDate,schedule.startDates(i)) >= 0
                    startDate = valueDate;
                    endDate = schedule.endDates(i);
                    % we exclude end date from the 
                    endDate = endDate.AddDate(-1,'day');
                    
                    timeSteps = [];
                    % startDate should be included
                    startTime = startDate.DateDiff(valueDate);
                    timeSteps = [timeSteps; startTime];
                    
                    while 1
                        endTime = endDate.DateDiff(valueDate);
                        timeSteps = [timeSteps; endTime];
                        endDate = endDate.AddDate(-1*mcOneTimeStep,'day');
                        if endDate.DateDiff(startDate) < 0 break;end
                    end
                    timeSteps = sort(unique(timeSteps),'ascend');
                    %out.timeStepsPerPeriod {i,1}.timeSteps = [];
                    out.timeStepsPerPeriod{i,1}.timeSteps = timeSteps;
                    out.numOfTimeSteps(i) = length(timeSteps);
                    out.totalTimeSteps = [out.totalTimeSteps; timeSteps];
                    
                    % 1st period
                    out.startTimeIdx(i) = 1;
                    out.endM1TimeIdx(i) = length(timeSteps);
                    
                else
                    startDate = schedule.startDates(i);
                    endDate = schedule.endDates(i);
                    % we exclude end date from the 
                    endDate = endDate.AddDate(-1,'day');
                    
                    timeSteps = [];
                    % startDate should be included
                    startTime = startDate.DateDiff(valueDate);
                    timeSteps = [timeSteps; startTime];
                    
                    while 1
                        endTime = endDate.DateDiff(valueDate);
                        timeSteps = [timeSteps; endTime];
                        
                        endDate = endDate.AddDate(-1*mcOneTimeStep,'day');
                        if endDate.DateDiff(startDate) < 0 break;end
                    end
                    timeSteps = sort(unique(timeSteps),'ascend');
                    out.timeStepsPerPeriod{i,1}.timeSteps = timeSteps;
                    out.numOfTimeSteps(i) = length(timeSteps);
                    out.totalTimeSteps = [out.totalTimeSteps; timeSteps];
                    
                    % 1st period
                    out.startTimeIdx(i) = out.endM1TimeIdx(i-1) + 1;
                    out.endM1TimeIdx(i) = out.endM1TimeIdx(i-1) + length(timeSteps);
                    
                end
            end
            
            % endDate of the last period (= Security's maturityDate should
            % be included
            
            endTime = DateDiff(schedule.endDates(periodCount),valueDate);
            out.totalTimeSteps = [out.totalTimeSteps;endTime];
            out.totalTimeSteps = sort(unique(out.totalTimeSteps),'ascend');
            out.totalTimeStepsSize = length(out.totalTimeSteps);
            
            out.periodCount = periodCount;
            out.numOfAlivePeriod = numOfAlivePeriod;
        end
        
        function out = generateModelStates(rangeAccrual,valueDate,schedule,numMethodInfo,model)
            
            pathSize = numMethodInfo.pathSize;
            numOfFactors = numMethodInfo.numOfFactors;
            totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
            totalTimeSteps = numMethodInfo.totalTimeSteps;
            Z = numMethodInfo.Z;
            modelStates = zeros(numOfFactors*totalTimeStepsSize,pathSize);
            
            currentStep = totalTimeSteps(1);
            for i=1:totalTimeStepsSize-1
                nextStep = totalTimeSteps(i+1);
                sqrtDt = sqrt((nextStep-currentStep)/365.0);
                stateLocalVariance = model.stateLocalVariance(currentStep,nextStep);
                L = chol(stateLocalVariance);
                prevModelStates = modelStates(numOfFactors*(i-1)+1:numOfFactors*(i-1)+2,:);
                dW = Z(numOfFactors*i+1:numOfFactors*i+2,:);
                modelStates(numOfFactors*i+1:numOfFactors*i+2,:) = prevModelStates + L*dW; 
                currentStep = nextStep;
            end
            
            out = modelStates;
        end
        
        function out = computePriceMCForward(rangeAccrual,valueDate,model)
            schedule = rangeAccrual.schedule;
            
            % numMethodTimeSteps Generate
            mcOneTimeStep = 1;
            outInfo = rangeAccrual.numMethodTimeStepsGenerate(valueDate,schedule,mcOneTimeStep);
            
            numMethodInfo.timeStepsInfo = outInfo;
            
            % mc random number init
            %pathSize = 2^16;
            pathSize = 5000;
            
            %pathSize = 2^10;
            
            rng('default');
            rng(0);
            
            numOfFactors = 2;
            
            Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
            Z=[Z,-Z];
            
            numMethodInfo.pathSize = pathSize;
            numMethodInfo.numOfFactors = numOfFactors;
            numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
            numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
            numMethodInfo.Z = Z;
            
            % numeraire info
            numMatTime = numMethodInfo.totalTimeSteps(end);
            dfNumMat = model.DF(numMatTime/365.0);
            
            %generate model states for the given timeSteps
            modelStates = rangeAccrual.generateModelStates(valueDate,schedule,numMethodInfo,model);
            
            % induct backward and calculate cashflow
            
            
            periodCount = schedule.numOfPeriod;
            
            nonCall.npv = 0;
            
            nonCall.cashflow_e = zeros(periodCount + 1,1);
            nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
            nonCall.rangeProb_e = zeros(periodCount + 1,1);
            
            nonCall.df_e = zeros(periodCount + 1,1);
            nonCall.df_c = zeros(periodCount + 1,1);
            
            % sum of cashflow_npv
            nonCall.payoff = zeros(1,pathSize);
            
            nonCall.payoffStateVectors_cashflow = zeros(periodCount+1,pathSize);
            nonCall.payoffStateVectors_cashflow_npv = zeros(periodCount+1,pathSize);
            nonCall.payoffStateVector_df = zeros(periodCount + 1,pathSize);
            
            payoffStateVectors_cashflow = zeros(1, pathSize);
            payoffStateVectors_cashflow_npv = zeros(1, pathSize);
            payoffStateVectors_df = zeros(1,pathSize);
            
            % we calculate cashflow backward from maturityDate to valueDate
            % rangeAccrual coupons are added at the startDate of the Period
            % At maturity
            
            nominal = rangeAccrual.nominal;
            coupon = rangeAccrual.coupon;
            
            currentTime = numMethodInfo.totalTimeSteps(end);
            totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
            currentTimeIdx = totalTimeStepsSize;
            
            modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
            payoffStateVectors_cashflow = nominal*ones(1,pathSize);
            nonCall.payoffStateVectors_cashflow(end,:) = payoffStateVectors_cashflow;
            
            payoffStateVectors_cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,payoffStateVectors_cashflow);
            
            nonCall.payoffStateVectors_cashflow_npv(end,:) = payoffStateVectors_cashflow_npv;
            
            nonCall.cashflow_npv_e(end) = mean(payoffStateVectors_cashflow_npv);
            
            nonCall.payoff = nonCall.payoff + payoffStateVectors_cashflow_npv;
            
            payoffStateVectors_df = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,1.0*ones(1,pathSize));
            
            nonCall.payoffStateVector_df(end,:) = payoffStateVectors_df;
            nonCall.df_e(end) = mean(nonCall.payoffStateVector_df(end,:));
            nonCall.df_c(end) = model.DF(currentTime/365.0);
%             
%             nonCall.cashflow_npv_df(end,:) =  nonCall.npv/nominal;
%             nonCall.cashflow_npv_df_e(end) = mean(nonCall.cashflow_npv_df(end,:));
            
            lastTimeIdx = currentTimeIdx;
            totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
            
            for i=periodCount:-1:1
                % past schedule neglect
                if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                endM1TimeIdx = numMethodInfo.timeStepsInfo.endM1TimeIdx(i);
                
                %% FromToInduction start
                
                %modelsStates induct backward(lastTimeIdx to endM1TimeIdx)
                while endM1TimeIdx < lastTimeIdx
                    lastTimeIdx = lastTimeIdx -1;
                    if lastTimeIdx == endM1TimeIdx break;end
                end
                
                currentTimeIdx = lastTimeIdx;
                currentTime = totalTimeSteps(currentTimeIdx);
                % now we are at endM1TimeIdx
                % variables for coupon valuation initialized
                ralower = rangeAccrual.lower;
                raupper = rangeAccrual.upper;
                tenorLong = rangeAccrual.tenorL;
                tenorShort = rangeAccrual.tenorS;
                
                cD = 0; % cumulated day count fraction
                cP = 0; 
                dI = 0; % dummy idx for loop
                dcfC = 0;
                dcfN = 0;
                dD = 0;
                idxL = zeros(1,pathSize);
                idxS = zeros(1,pathSize);
                spreadC = zeros(1,pathSize);
                spreadN = zeros(1,pathSize);
                probSpread = zeros(1,pathSize);
                simpleYN = true;
                numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                
                for j=numOfTimeSteps:-1:1
                    % evaluate dealdescription
                    % last step is omitted
                    currentTime = totalTimeSteps(currentTimeIdx);
                    dcfC = currentTime/365.0;
                    if j~= numOfTimeSteps
                        dD = (dcfN-dcfC);
                    end
                    
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
                    idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                    idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                    spreadC = idxL - idxS;
                     
                    probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                    cD = cD + dD;
                    cP = cP + probSpread;
  
                    dcfN = dcfC;
                    spreadN = spreadC;
                    
                    %induct backward to the next time step
                    % one time backward induction
                    lastTimeIdx = currentTimeIdx;
                    
                    if startTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        currentTimeIdx = lastTimeIdx;
                    end
                    
                    
                end
                
                %% FromToInduction end
                %% AtDate operation
                % Adding a new cashflow to the payoff
                
                dcf = schedule.dayCountFraction(i);
                payoffStateVectors_cashflow = nominal * coupon * dcf * cP/cD;
                
                % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                % i.e. currentTimeIdx has not changed
                % no need to upodate modelStateT
                % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                % since cashflow's are added at the startDate(i) we need to
                % multiply cashflow with the df(startDate(i), payDate(i))
                payTime = DateDiff(schedule.payDates(i),valueDate);
                df_se = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);
                
                for k=1:pathSize
                    payoffStateVectors_cashflow(k) = payoffStateVectors_cashflow(k) + df_se(k);
                end
                
                nonCall.payoffStateVectors_cashflow(i,:) = payoffStateVectors_cashflow;
                
                nonCall.cashflow_e(i) = mean(payoffStateVectors_cashflow);
                nonCall.rangeProb_e(i) = mean(nonCall.cashflow_e(i)/(nominal * coupon * dcf));
                
                payoffStateVectors_cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,payoffStateVectors_cashflow);
            
                nonCall.cashflow_npv_e(i) = mean(payoffStateVectors_cashflow_npv);
                
                nonCall.payoffStateVectors_cashflow_npv(i,:) = payoffStateVectors_cashflow_npv;
                nonCall.payoff = nonCall.payoff + payoffStateVectors_cashflow_npv;
                
                payoffStateVectors_df = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,1.0*ones(1,pathSize));
                
                nonCall.payoffStateVector_df(i,:) = payoffStateVectors_df;
                nonCall.df_e(i) = mean(nonCall.payoffStateVector_df(i,:));
                nonCall.df_c(i) = model.DF(currentTime/365.0);
                
            end
            
            nonCall.npv = mean(nonCall.payoff);
  
            out.cashflow= nonCall.cashflow_e;
            out.cashflowNpv = nonCall.cashflow_npv_e;
            out.df_e = nonCall.df_e;
            out.df_c = nonCall.df_c;
            out.daycountFraction = schedule.dayCountFraction(i);
            out.rangeProb = nonCall.rangeProb_e;
            out.npv = nonCall.npv;
           
        end
        
        function out = computePrice(rangeAccrual,valueDate,model)
            schedule = rangeAccrual.schedule;
            periodCount = schedule.numOfPeriod;
            rangeProb = zeros(periodCount,1);
            discountFactor = zeros(periodCount,1);
            daycountFraction = zeros(periodCount,1);
            cashflow = zeros(periodCount,1);
            cashflowNpv = zeros(periodCount,1);
            rangeDays = rangeAccrual.rangeDays;
            npv = 0;
            for i=1:periodCount
                if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end;
                
                observeDate = schedule.startDates(i);
                if DateDiff(valueDate,schedule.startDates(i)) >= 0
                    observeDate = valueDate;
                    observeDate = AddDate(observeDate,1,'day');
                else
                    observeDate = schedule.startDates(i);
                end
                
                observeDays = DateDiff(schedule.endDates(i),schedule.startDates(i));
                days = DateDiff(schedule.endDates(i), observeDate);
                discountFactor(i) = model.DF( DateDiff(schedule.payDates(i),valueDate)/365.0);
                daycountFraction(i) = schedule.dayCountFraction(i);
                for j=1:days
                    toDate = observeDate;
                    tpDate = schedule.payDates(i);
                    tenor1 = rangeAccrual.tenorL;
                    tenor2 = rangeAccrual.tenorS;
                    tau =0.25;
                    lower = rangeAccrual.lower;
                    upper = rangeAccrual.upper;
                    rangeValue = model.CMSSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,lower,upper);
%                     rangeValue = 1.0;
                    rangeProb(i) = rangeProb(i) + rangeValue;
                    observeDate = AddDate(observeDate,1,'day');
                end
                cashflow(i) = rangeAccrual.nominal * rangeAccrual.coupon * (rangeProb(i) + rangeDays(i))/observeDays * daycountFraction(i);
                cashflowNpv(i) = cashflow(i) * discountFactor(i);
                npv = npv + cashflowNpv(i);
                % bug fixed observeDays -> days
                rangeProb(i) = rangeProb(i)/days;
                
                if i == periodCount
                    npv = npv + rangeAccrual.nominal * discountFactor(i);
                end
            end
            out.cashflow= cashflow;
            out.cashflowNpv = cashflowNpv;
            out.discountFactor = discountFactor;
            out.daycountFraction = daycountFraction;
            out.rangeProb = rangeProb;
            out.npv = npv;
        end
        
%         function out = discountPayoff(rangeAccrual,eventTime,numMatTime ...
%                 ,model,modelStatesT,cashflow)
%             
%             dfNumMat = model.DF(numMatTime/365.0);
%             % for discountPayoff T=numMatTime
%             % P(0,TnumMat)/P(Te,Tnummat)*Payoff
%             %
%             discountT = model.discountFactor(eventTime, numMatTime,numMatTime,modelStatesT);
%             stateSize = size(modelStatesT,2);
%             payoff = zeros(1,stateSize);
%             for i=1:stateSize
%                 payoff(i)= dfNumMat/discountT(i)*cashflow(i);
%             end    
%             out = payoff;
%         end
        
        
        
    end
    
end



In [ ]:
classdef CK2FRisky < IRModelRisky
    %UNTITLED Summary of this class goes here
    %   Detailed explanation goes here
    
    properties
        % vol functor
        vol_r
        vol_s
        meanR_r
        meanR_s
        corr_rs
    end
    
    methods
        function ck2F = CK2FRisky(params)
            if nargin > 0
                ck2F.zeroCurve = params.zeroCurve;
                ck2F.discountCurve = params.discountCurve;
                ck2F.mktData = params.mktData;
                ck2F.modelParams = params.modelParams;
                
                ck2F.vol_r = params.modelParams('vol_r');
                ck2F.vol_s = params.modelParams('vol_s');
                ck2F.meanR_r =  params.modelParams('meanR_r');
                ck2F.meanR_s =  params.modelParams('meanR_s');
                ck2F.corr_rs =  params.modelParams('corr_rs');
            end
        end
        
        function out = SimpleSRPrOverHedge(lgm2F,spreadC,spreadN,ralower,raupper)
            buffer = 0.001;
            outVec = 0.0;
            if spreadC < ralower - buffer
                outVec = 0.0;
            elseif spreadC >= ralower - buffer && spreadC < ralower
                outVec = 1.0/buffer * (spreadC - (ralower - buffer));
            elseif spreadC >= ralower && spreadC < raupper
                outVec = 1.0;
            elseif spreadC >= raupper && spreadC < raupper + buffer
                outVec = -1.0/buffer *(spreadC - (raupper + buffer));
            else
                outVec = 0.0;
            end
            
            out = outVec;
        end
        
        function out = SimpleSRPr(ck2F,spreadC,spreadN,ralower,raupper)
            outVec = 0.0;
            if spreadC >= ralower && spreadC <= raupper
                outVec = 1.0;
            end
            out = outVec;
        end
        
        
        function out = TRZ_SRPr(ck2F,curr,next,L,U)
            length = 1.0;
            remain = 1.0;
            if (curr >= L && curr <= U && next >= L && next <= U)
                length = remain;
            elseif ( curr >= L && curr <= U && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr >= L && curr <= U && next < L ) 
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr > U && next >= L && next <= U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next < L )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next >= L && next <= U )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next > U )
                length = 0.0;
            elseif ( curr < L && next < L)
                length = 0.0;
            else
                disp('error')
            end
            
            out = length;
        end
        
        function out = SRPr(ck2F,spreadC,spreadN,ralower,raupper,simpleYN,pathSize)
            
            outVec = zeros(1,pathSize);
            
            if simpleYN == 1
                for i =1:pathSize
                    outVec(i) = ck2F.SimpleSRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            elseif simpleYN == 2
                for i =1:pathSize
                    outVec(i) = ck2F.SimpleSRPrOverHedge(spreadC(i),spreadN(i),ralower,raupper);
                end
            else
                for i =1:pathSize
                    outVec(i) = ck2F.TRZ_SRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            end
            
            out = outVec;
        end
        
        function out =  CMS_PSA_SimpleDate(ck2F,valueDate,observeTime,numMatTime, tenor,modelStatesT)
           %CMS
           tau = 0.25;
           to      = observeTime/365.0;
           toDate = valueDate.AddDate(observeTime,'day');
           tNDate = toDate.AddDate(tenor,'year');
           tN   =  tNDate.DateDiff(valueDate)/365.0;
           
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           p_nN = ck2F.PV01Date(valueDate,toDate,tNDate,tau);
           
           df_Tn = ck2F.DF(to);
           df_TN = ck2F.DF(tN);
           y_nN = (df_Tn - df_TN)/p_nN;
           
           c_nN = ck2F.C_nNDate(valueDate,toDate,tNDate,tau,p_nN,y_nN);
           g_nN = ck2F.G_nNDate(valueDate,toDate,tNDate,tau,p_nN);
           h_nTerm = ck2F.H_nTermDate(valueDate,toDate,numMatDate);
           varCovar = ck2F.LocalVariance(0,to);
           
           stateSize = size(modelStatesT,2);
           cms_PCA_SimpleOut = zeros(1,stateSize);
           
           driftTerm = 0;
           for i=1:2
               for j=1:2
                   driftTerm = driftTerm + (c_nN(i) * h_nTerm(j) + c_nN(i) * g_nN(j))*varCovar(i,j);
               end
           end
           
           for i=1:stateSize
               cms_PCA_SimpleOut(i) = y_nN + driftTerm + c_nN(1)*modelStatesT(1,i) + c_nN(2) * modelStatesT(2,i); 
           end
           
           out = cms_PCA_SimpleOut;
           
            
        end
        
        function out = HWGeneralVariance(ck2F,from,to,T1,T2,MRS1,MRS2,sigmaTimes,sigmaValue1,sigmaValue2,corrValue)
            
            sigmaSize = length(sigmaValue1);
            
            for i=1:length(sigmaTimes)
                if from<= sigmaTimes(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:length(sigmaTimes)
                if to<= sigmaTimes(i)
                    break;
                end
            end
            endIdx = i;
            
            
            newExpScale = zeros(4,1);
            lastExpScale = zeros(4,1);
            variances = zeros(4,1);
            
            lastU  = from;
            lastExpScale(2) = exp(MRS1*lastU);
            lastExpScale(3) = exp(MRS2*lastU);
            lastExpScale(4) = exp((MRS1 + MRS2)*lastU);
            
            for i=startIdx:endIdx
                if (i< endIdx) U = sigmaTimes(i);else U = to;end;
                if (i< sigmaSize) sig1 = sigmaValue1(i);else sig1 = sigmaValue1(sigmaSize);end;
                if (i< sigmaSize) sig2 = sigmaValue2(i);else sig2 = sigmaValue2(sigmaSize);end;
                if (i< sigmaSize) corr12 = corrValue(i);else corr12 = corrValue(sigmaSize);end;
                
                variances(1) = variances(1) + corr12 * sig1 * sig2 * (U - lastU);
                
                newExpScale(2) = exp(MRS1*U);
                variances(2) = variances(2) + corr12 * sig1 * sig2 * (1.0/MRS1) * (newExpScale(2)-lastExpScale(2));
                lastExpScale(2) = newExpScale(2);
                
                newExpScale(3) = exp(MRS2*U);
                variances(3) = variances(3) + corr12 * sig1 * sig2 * (1.0/MRS2) * (newExpScale(3)-lastExpScale(3));
                lastExpScale(3) = newExpScale(3);
                
                newExpScale(4) = exp((MRS1+MRS2)*U);
                variances(4) = variances(4) + corr12 * sig1 * sig2 * (1.0/(MRS1+MRS2)) * (newExpScale(4)-lastExpScale(4));
                lastExpScale(4) = newExpScale(4);
                
                lastU = U;
                
            end
            
            value = (1.0/ MRS1)*(1.0/MRS2)*variances(1);
            value = value + -1.0*exp(-1.0*MRS1*T1)*(1.0/ MRS1)*(1.0/MRS2)*variances(2);
            value = value + -1.0*exp(-1.0*MRS2*T2)*(1.0/ MRS1)*(1.0/MRS2)*variances(3);
            value = value +  1.0*exp(-1.0*MRS1*T1)*exp(-1.0*MRS2*T2)*(1.0/ MRS1)*(1.0/MRS2)*variances(4);

            out  = value;
        end
        
        function out = fwdZcVariance(ck2F, from, to, T1, T2)
            
            MRS_r = ck2F.meanR_r;
            MRS_s = ck2F.meanR_s;
            sigma_r = ck2F.vol_r.quote;
            sigma_s = ck2F.vol_s.quote;
            correl_rs = ck2F.corr_rs.quote;
            correl_1 = ones(length(correl_rs),1);
            sigmaTimes = ck2F.vol_r.tenor;
            localVariance = zeros(6,1);
            
            localVariance(1) = ck2F.HWGeneralVariance(from,to,T1,T1,MRS_r,MRS_r,sigmaTimes,sigma_r,sigma_r,correl_1);
            localVariance(2) = ck2F.HWGeneralVariance(from,to,to,to,MRS_r,MRS_r,sigmaTimes,sigma_r,sigma_r,correl_1);
            
            localVariance(3) = ck2F.HWGeneralVariance(from,to,T1,T1,MRS_s,MRS_s,sigmaTimes,sigma_s,sigma_s,correl_1);
            localVariance(4) = ck2F.HWGeneralVariance(from,to,to,to,MRS_s,MRS_s,sigmaTimes,sigma_s,sigma_s,correl_1);
            
            localVariance(5) = ck2F.HWGeneralVariance(from,to,T1,T1,MRS_r,MRS_s,sigmaTimes,sigma_r,sigma_s,correl_rs);
            localVariance(6) = ck2F.HWGeneralVariance(from,to,to,to,MRS_r,MRS_s,sigmaTimes,sigma_r,sigma_s,correl_rs);
            
            value = localVariance(1) - localVariance(2);
            value = value + localVariance(3) - localVariance(4);
            value = value + 2.0*(localVariance(5) - localVariance(6));
            
            measureChange = zeros(8,1);
            
            measureChange(1) = ck2F.HWGeneralVariance(from,to,T1,T2,MRS_r,MRS_r,sigmaTimes,sigma_r,sigma_r,correl_1);
            measureChange(2) = ck2F.HWGeneralVariance(from,to,to,T2,MRS_r,MRS_r,sigmaTimes,sigma_r,sigma_r,correl_1);
            
            measureChange(3) = ck2F.HWGeneralVariance(from,to,T1,T2,MRS_r,MRS_s,sigmaTimes,sigma_r,sigma_s,correl_rs);
            measureChange(4) = ck2F.HWGeneralVariance(from,to,to,T2,MRS_r,MRS_s,sigmaTimes,sigma_r,sigma_s,correl_rs);
            
            measureChange(5) = ck2F.HWGeneralVariance(from,to,T1,T2,MRS_s,MRS_r,sigmaTimes,sigma_s,sigma_r,correl_rs);
            measureChange(6) = ck2F.HWGeneralVariance(from,to,to,T2,MRS_s,MRS_r,sigmaTimes,sigma_s,sigma_r,correl_rs);
            
            measureChange(7) = ck2F.HWGeneralVariance(from,to,T1,T2,MRS_s,MRS_s,sigmaTimes,sigma_s,sigma_s,correl_1);
            measureChange(8) = ck2F.HWGeneralVariance(from,to,to,T2,MRS_s,MRS_s,sigmaTimes,sigma_s,sigma_s,correl_1);
            
            measureChangeTerm =                     1.0 * (measureChange(1) - measureChange(2));
            measureChangeTerm = measureChangeTerm + 1.0 * (measureChange(3) - measureChange(4));
            measureChangeTerm = measureChangeTerm + 1.0 * (measureChange(5) - measureChange(6));
            measureChangeTerm = measureChangeTerm + 1.0 * (measureChange(7) - measureChange(8));
            
            value = value + -2.0*measureChangeTerm;
            
            out = value;
            
        end
        
        function out = discountFactorRisky(ck2F, startTime, endTime,numMatTime,modelStatesT)
           zcT = ck2F.DFRisky(endTime/365.0);
           zct = ck2F.DFRisky(startTime/365.0);
           beta = ck2F.Beta(startTime/365.0,endTime/365.0);
           zcDriftTerm = ck2F.fwdZcVariance(0,startTime/365.0,endTime/365.0,numMatTime/365.0);
           %numOfPath
           stateSize = size(modelStatesT,2);
           discountFactorOut = zeros(1,stateSize);
           
           for i=1:stateSize
               discountFactorOut(i) = zcT/zct * exp(-0.5*zcDriftTerm - beta(1)*modelStatesT(1,i) ...
                   - beta(2) * modelStatesT(2,i)); 
           end
           
           out = discountFactorOut;
        end
        
        function out = discountFactor(ck2F, startTime, endTime,numMatTime,modelStatesT)
           zcT = ck2F.DF(endTime/365.0);
           zct = ck2F.DF(startTime/365.0);
           beta = ck2F.Beta(startTime/365.0,endTime/365.0);
           zcDriftTerm = ck2F.fwdZcVariance(0,startTime/365.0,endTime/365.0,numMatTime/365.0);
           %numOfPath
           stateSize = size(modelStatesT,2);
           discountFactorOut = zeros(1,stateSize);
           
           for i=1:stateSize
               discountFactorOut(i) = zcT/zct * exp(-0.5*zcDriftTerm - beta(1)*modelStatesT(1,i) ...
                   - beta(2) * modelStatesT(2,i)); 
           end
           
           out = discountFactorOut;
        end
        
        function out = discountPayoffRisky(ck2F,eventTime,numMatTime ...
                ,modelStatesT,cashflow)
            
            dfNumMat = ck2F.DFRisky(numMatTime/365.0);
            % for discountPayoff T=numMatTime
            % P(0,TnumMat)/P(Te,Tnummat)*Payoff
            %
            discountT = ck2F.discountFactorRisky(eventTime, numMatTime,numMatTime,modelStatesT);
            stateSize = size(modelStatesT,2);
            payoff = zeros(1,stateSize);
            for i=1:stateSize
                payoff(i)= dfNumMat/discountT(i)*cashflow(i);
            end    
            out = payoff;
        end
        
        function out = discountPayoff(ck2F,eventTime,numMatTime ...
                ,modelStatesT,cashflow)
            
            dfNumMat = ck2F.DF(numMatTime/365.0);
            % for discountPayoff T=numMatTime
            % P(0,TnumMat)/P(Te,Tnummat)*Payoff
            %
            discountT = ck2F.discountFactor(eventTime, numMatTime,numMatTime,modelStatesT);
            stateSize = size(modelStatesT,2);
            payoff = zeros(1,stateSize);
            for i=1:stateSize
                payoff(i)= dfNumMat/discountT(i)*cashflow(i);
            end    
            out = payoff;
        end
        
        function out = stateLocalDrift(ck2F,fromTime, toTime)
            drift = zeros(2,2);
            drift(1,1) =  exp(-1.0*ck2F.meanR_r*(toTime-fromTime)/365.0);
            drift(2,2) =  exp(-1.0*ck2F.meanR_s*(toTime-fromTime)/365.0);
            out = drift;
        end
        
        function out = stateLocalVariance(ck2F,fromTime, toTime)
            out = ck2F.LocalVariance(fromTime/365.0,toTime/365.0);
        end
        
        function variances = LocalVariance(ck2F,from,to)
%             sig =[];
            sig(1) = ck2F.vol_r;
            sig(2) = ck2F.vol_s;
%             corr =[];
            corr_rs = ck2F.corr_rs;
            corr(1,1) = 1.0;
            corr(2,1) = corr_rs.quote(1);
            corr(1,2) = corr(2,1);
            corr(2,2) = 1.0;
            
%             meanR = [];
            meanR(1) = ck2F.meanR_r;
            meanR(2) = ck2F.meanR_s;
            
            alphaSize = length(ck2F.vol_r.quote);
            
            for i=1:length(sig(1).tenor)
                if from<= sig(1).tenor(i)
                    break;
                end
            end
            startIdx = i;
            
%             endIdx = size(sig(1).tenor,1); 
            for i=1:length(sig(1).tenor)
                if to<= sig(1).tenor(i)
                    break;
                end
            end
            endIdx = i;
            
%             tiny = {};
%             lastExpScale={};
%             newExpScale = {};
            for i=1:2
                for j=1:2
                    if(abs(meanR(i)+meanR(j)) < 1.0E-12) tiny(i,j) = 1; else tiny(i,j) = 0; end;
                    lastExpScale(i,j) =  exp((meanR(i)+meanR(j))*from);
                end
            end
            
            lastU  = from;
            
%             variances=[];
%             vol =[];
%             correl ={};
           variances = zeros(2,2);
           for i=startIdx:endIdx
                if (i< endIdx) U = sig(1).tenor(i);else U = to;end;
                if (i< alphaSize) vol(1) = sig(1).quote(i);else vol(1) = sig(1).quote(alphaSize);end;
                if (i< alphaSize) vol(2) = sig(2).quote(i);else vol(2) = sig(2).quote(alphaSize);end;
                correl(1,1) =1.0;
                correl(2,2) =1.0;
                if (i< alphaSize) correl(2,1) = corr_rs.quote(i);else correl(2,1) = corr_rs.quote(alphaSize);end;
                correl(1,2) = correl(2,1);
                for j=1:2
                    for k=1:2
                        newExpScale(j,k) =  exp((meanR(j)+meanR(k))*U);
                        if(tiny(j,k)~=1)
                            scale =newExpScale(j,k)-lastExpScale(j,k);
                        else
                            scale = U - lastU;
                        end
                        variances(j,k) = variances(j,k) + vol(j)*vol(k)*correl(j,k)...
                                       * scale;
                        lastExpScale(j,k) = newExpScale(j,k);
                    end
                end
                lastU = U;
           end
           
            for j=1:2
                for k=1:2
                    if(tiny(j,k)~= 1)
                        variances(j,k) = variances(j,k)/(meanR(j)+meanR(k))*exp(-1.0*(meanR(j)+meanR(k))*to);
                    end
                end
            end
        end
        
        function out = sigma_Test(ck2F,t,sigma)
            
            sum = 0.0;
            prev_t = 0.0;
            inRangeYN = false;
            for i=1:length(sigma.tenor)
                alpha = sigma.quote(i);
                if t<= sigma.tenor(i)
                    sum = alpha;
                    inRangeYN = true;
                    break;
                end
            end
            
            if ~inRangeYN 
                sum = alpha;
            end
            out = sum;
        end

        function beta = Beta(ck2F,t,T)
            a_r = ck2F.meanR_r;
            a_s = ck2F.meanR_s;
            beta =[];
            if(abs(a_r) > 1.0E-12)
                beta(1) = (1.0-exp(-a_r*(T-t)))/a_r;
            else
                beta(1) = (T-t);
            end
            
            if(abs(a_s) > 1.0E-12)
                beta(2) = (1.0-exp(-a_s*(T-t)))/a_s;
            else
                beta(2) = (T-t);
            end
        end
        
        function beta = BetaDate(ck2F,valueDate,tDate,TDate)
            t = tDate.DateDiff(valueDate)/365.0;
            T = TDate.DateDiff(valueDate)/365.0;
            
            a_r = ck2F.meanR_r;
            a_s = ck2F.meanR_s;
            beta =[];
            
            if(abs(a_r) > 1.0E-12)
                beta(1) = (1.0-exp(-a_r*(T-t)))/a_r;
            else
                beta(1) = (T-t);
            end
            
            if(abs(a_s) > 1.0E-12)
                beta(2) = (1.0-exp(-a_s*(T-t)))/a_s;
            else
                beta(2) = (T-t);
            end
            
        end
        
        function c_nN = C_nN(ck2F,tn,tN,tau,pnN,ynN)
            p = @ck2F.DF;
            c_nN=[0.0 0.0];
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = ck2F.Beta(tn,tp);
                df = p(tp);
                c_nN(1) = c_nN(1) +tau * beta(1) * df;
                c_nN(2) = c_nN(2) +tau * beta(2) * df;
            end
            
            c_nN(1) = c_nN(1)* ynN;
            c_nN(2) = c_nN(2)* ynN;
            
            c_nN(1) = c_nN(1) + beta(1)*df;
            c_nN(2) = c_nN(2) + beta(2)*df;
            
            c_nN(1) = c_nN(1)/pnN;
            c_nN(2) = c_nN(2)/pnN;
        end
        
        function c_nN = C_nNDate(ck2F,valueDate,tnDate,tNDate,tau,pnN,ynN)
            tn = tnDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            
            addMonthUnit = round(12*tau);
            cnt = round((tN-tn)/tau);
            
            p = @ck2F.DF;
            c_nN=[0.0 0.0];
            
            for i=1:cnt
                tpDate = tnDate.AddDate(i*addMonthUnit,'month');
                tp = tpDate.DateDiff(valueDate)/365.0;
                beta = ck2F.BetaDate(valueDate,tnDate,tpDate);
                df = p(tp);
                c_nN(1) = c_nN(1) +tau * beta(1) * df;
                c_nN(2) = c_nN(2) +tau * beta(2) * df;
            end
            
            c_nN(1) = c_nN(1)* ynN;
            c_nN(2) = c_nN(2)* ynN;
            
            c_nN(1) = c_nN(1) + beta(1)*df;
            c_nN(2) = c_nN(2) + beta(2)*df;
            
            c_nN(1) = c_nN(1)/pnN;
            c_nN(2) = c_nN(2)/pnN;
        end
        
         function g_nN = G_nN(ck2F,tn,tN,tau,pnN)
            p = @ck2F.DF;
            g_nN=[0.0 0.0];
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = ck2F.Beta(tn,tp);
                df = p(tp);
                g_nN(1) = g_nN(1) +tau * beta(1) * df;
                g_nN(2) = g_nN(2) +tau * beta(2) * df;
            end
            
            g_nN(1) = g_nN(1)/pnN;
            g_nN(2) = g_nN(2)/pnN;
         end
        
        function g_nN = G_nNDate(ck2F,valueDate,tnDate,tNDate,tau,pnN)
            tn = tnDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            addMonthUnit = round(12*tau);
            cnt = round((tN-tn)/tau);
            
            p = @ck2F.DF;
            g_nN=[0.0 0.0];
            
            for i=1:cnt
                tpDate = tnDate.AddDate(i*addMonthUnit,'month');
                tp = tpDate.DateDiff(valueDate)/365.0;
                beta = ck2F.BetaDate(valueDate,tnDate,tpDate);
                df = p(tp);
                g_nN(1) = g_nN(1) +tau * beta(1) * df;
                g_nN(2) = g_nN(2) +tau * beta(2) * df;
            end
            
            g_nN(1) = g_nN(1)/pnN;
            g_nN(2) = g_nN(2)/pnN;
        end 
        
        function h_nTerm = H_nTerm(ck2F,tn,tTerm)
           h_nTerm =[];
           beta = ck2F.Beta(tn,tTerm);
           h_nTerm(1) = -1.0*beta(1);
           h_nTerm(2) = -1.0*beta(2);
        end
        
        function h_nTerm = H_nTermDate(ck2F,valueDate,tnDate,tTermDate)
           h_nTerm =[];
           beta = ck2F.BetaDate(valueDate,tnDate,tTermDate);
           h_nTerm(1) = -1.0*beta(1);
           h_nTerm(2) = -1.0*beta(2);
        end
        
        
        function out = CMSDigitalArg(ck2F,to,tp,tenor1,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           sizeN = 2;
           p_nN1 = ck2F.PV01(to,to+tenor1,tau);
           df_Tn1 = ck2F.DF(to);
           df_TN1 = ck2F.DF(to + tenor1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           c_nN1 = ck2F.C_nN(to,to + tenor1,tau,p_nN1,y_nN1);
           g_nN1 = ck2F.G_nN(to,to + tenor1,tau,p_nN1);
           h_nTerm1 = ck2F.H_nTerm(to,tp);
           
           varCovar = ck2F.LocalVariance(0,to);
           
           driftTerm1=0.0;
           for i=1:sizeN
               for j=1:sizeN
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance1 = 0.0;
           for i=1:sizeN
               for j=1:sizeN
                    spreadVariance1 = spreadVariance1 + c_nN1(i) * c_nN1(j) * varCovar(i,j);
               end
           end
           
           out.fwdSpread = y_nN1;
           out.mu = y_nN1 + driftTerm1;
           out.sigma = sqrt(spreadVariance1);
           out.c_nN1 = c_nN1;
           
        end
        
        function out = CMSDigitalArgDate(ck2F,valueDate,toDate,tpDate,tenor1,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           to      = toDate.DateDiff(valueDate)/365.0;
           tN1Date = toDate.AddDate(tenor1,'year');
           tN1   =  tN1Date.DateDiff(valueDate)/365.0;
           
           p_nN1 = ck2F.PV01Date(valueDate,toDate,tN1Date,tau);
           
           df_Tn1 = ck2F.DF(to);
           df_TN1 = ck2F.DF(tN1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           
           c_nN1 = ck2F.C_nNDate(valueDate,toDate,tN1Date,tau,p_nN1,y_nN1);
           g_nN1 = ck2F.G_nNDate(valueDate,toDate,tN1Date,tau,p_nN1);
           h_nTerm1 = ck2F.H_nTermDate(valueDate,toDate,tpDate);
           
           varCovar = ck2F.LocalVariance(0,to);
           
           driftTerm1=0.0;
           driftTerm2=0.0;
           for i=1:2
               for j=1:2
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
               end
           end
           
           variance1 = 0.0;
           for i=1:2
               for j=1:2
                    variance1 = variance1 + (c_nN1(i)) * (c_nN1(j)) * varCovar(i,j);
               end
           end
           
           out.fwdSpread = y_nN1;
           out.mu = (y_nN1 + driftTerm1);
           out.sigma = sqrt(variance1);
           out.c_nN1 = c_nN1;
           out.mu1 = y_nN1 + driftTerm1;
           out.sig1 = sqrt(variance1/to);
           
        end
        
        function out = CMSDigitalDate(ck2F,valueDate,toDate,tpDate,tenor1,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsDigitalArg = ck2F.CMSDigitalArgDate(valueDate,toDate,tpDate,tenor1,tau);
           mu = cmsDigitalArg.mu;
           sigma = cmsDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSDigital(ck2F,to,tp,tenor1,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsDigitalArg = ck2F.CMSDigitalArg(to,tp,tenor1,tau);
           mu = cmsDigitalArg.mu;
           sigma = cmsDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSDigitalRangeDate(ck2F,valueDate,toDate,tpDate,tenor1,tau,lower,upper)
            lower_value = ck2F.CMSDigitalDate(valueDate,toDate,tpDate,tenor1,tau,lower);
            upper_value = ck2F.CMSDigitalDate(valueDate,toDate,tpDate,tenor1,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CMSDigitalRange(ck2F,to,tp,tenor1,tau,lower,upper)
            lower_value = ck2F.CMSDigital(to,tp,tenor1,tau,lower);
            upper_value = ck2F.CMSDigital(to,tp,tenor1,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CMSSpreadDigitalArg(ck2F,to,tp,tenor1,tenor2,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           p_nN1 = ck2F.PV01(to,to+tenor1,tau);
           df_Tn1 = ck2F.DF(to);
           df_TN1 = ck2F.DF(to + tenor1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           c_nN1 = ck2F.C_nN(to,to + tenor1,tau,p_nN1,y_nN1);
           g_nN1 = ck2F.G_nN(to,to + tenor1,tau,p_nN1);
           h_nTerm1 = ck2F.H_nTerm(to,tp);
           
           %CMS2
           p_nN2 = ck2F.PV01(to,to+tenor2,tau);
           df_Tn2 = ck2F.DF(to);
           df_TN2 = ck2F.DF(to + tenor2);
           y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
           c_nN2 = ck2F.C_nN(to,to + tenor2,tau,p_nN2,y_nN2);
           g_nN2 = ck2F.G_nN(to,to + tenor2,tau,p_nN2);
           h_nTerm2 = ck2F.H_nTerm(to,tp);
           
           varCovar = ck2F.LocalVariance(0,to);
           
           driftTerm1=0.0;
           driftTerm2=0.0;
           for i=1:2
               for j=1:2
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
                    driftTerm2 = driftTerm2 + c_nN2(i) * (h_nTerm2(j) + g_nN2(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           variance1 = 0.0;
           variance2 = 0.0;
           coVariance = 0.0;
           for i=1:2
               for j=1:2
                    spreadVariance = spreadVariance + (c_nN1(i)-c_nN2(i)) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
                    variance1 = variance1 + (c_nN1(i)) * (c_nN1(j)) * varCovar(i,j);
                    variance2 = variance2 + (c_nN2(i)) * (c_nN2(j)) * varCovar(i,j);
                    coVariance = coVariance + (c_nN1(i)) * (c_nN2(j)) * varCovar(i,j);
               end
           end
           
           out.fwdSpread = y_nN1 - y_nN2;
           out.mu = (y_nN1 + driftTerm1) - (y_nN2 + driftTerm2);
           out.sigma = sqrt(spreadVariance);
           out.c_nN1 = c_nN1;
           out.c_nN2 = c_nN2;
           out.varCovar = varCovar;
           out.mu1 = y_nN1 + driftTerm1;
           out.mu2 = y_nN2 + driftTerm2;
           out.corr = coVariance/sqrt(variance1*variance2);
           out.sig1 = sqrt(variance1/to);
           out.sig2 = sqrt(variance2/to);
           
        end
        
        function out = CMSSpreadDigitalArgDate(ck2F,valueDate,toDate,tpDate,tenor1,tenor2,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           to      = toDate.DateDiff(valueDate)/365.0;
           tN1Date = toDate.AddDate(tenor1,'year');
           tN1   =  tN1Date.DateDiff(valueDate)/365.0;
           
           p_nN1 = ck2F.PV01Date(valueDate,toDate,tN1Date,tau);
           
           df_Tn1 = ck2F.DF(to);
           df_TN1 = ck2F.DF(tN1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           
           c_nN1 = ck2F.C_nNDate(valueDate,toDate,tN1Date,tau,p_nN1,y_nN1);
           g_nN1 = ck2F.G_nNDate(valueDate,toDate,tN1Date,tau,p_nN1);
           h_nTerm1 = ck2F.H_nTermDate(valueDate,toDate,tpDate);
           
           %CMS2
           tN2Date = toDate.AddDate(tenor2,'year');
           tN2   =  tN2Date.DateDiff(valueDate)/365.0;
           
           p_nN2 = ck2F.PV01Date(valueDate,toDate,tN2Date,tau);
           
           df_Tn2 = ck2F.DF(to);
           df_TN2 = ck2F.DF(tN2);
           y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
           
           c_nN2 = ck2F.C_nNDate(valueDate,toDate,tN2Date,tau,p_nN2,y_nN2);
           g_nN2 = ck2F.G_nNDate(valueDate,toDate,tN2Date,tau,p_nN2);
           h_nTerm2 = ck2F.H_nTermDate(valueDate,toDate,tpDate);
           
           varCovar = ck2F.LocalVariance(0,to);
           
           driftTerm1=0.0;
           driftTerm2=0.0;
           for i=1:2
               for j=1:2
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
                    driftTerm2 = driftTerm2 + c_nN2(i) * (h_nTerm2(j) + g_nN2(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           variance1 = 0.0;
           variance2 = 0.0;
           coVariance = 0.0;
           for i=1:2
               for j=1:2
                    spreadVariance = spreadVariance + (c_nN1(i)-c_nN2(i)) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
                    variance1 = variance1 + (c_nN1(i)) * (c_nN1(j)) * varCovar(i,j);
                    variance2 = variance2 + (c_nN2(i)) * (c_nN2(j)) * varCovar(i,j);
                    coVariance = coVariance + (c_nN1(i)) * (c_nN2(j)) * varCovar(i,j);
               end
           end
           
           out.fwdSpread = y_nN1 - y_nN2;
           out.mu = (y_nN1 + driftTerm1) - (y_nN2 + driftTerm2);
           out.sigma = sqrt(spreadVariance);
           out.c_nN1 = c_nN1;
           out.c_nN2 = c_nN2;
           out.varCovar = varCovar;
           out.mu1 = y_nN1 + driftTerm1;
           out.mu2 = y_nN2 + driftTerm2;
           out.corr = coVariance/sqrt(variance1*variance2);
           out.sig1 = sqrt(variance1/to);
           out.sig2 = sqrt(variance2/to);
           
        end
        
        function out = CMSSpreadDigital(ck2F,to,tp,tenor1,tenor2,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsSpreadDigitalArg = ck2F.CMSSpreadDigitalArg(to,tp,tenor1,tenor2,tau);
           mu = cmsSpreadDigitalArg.mu;
           sigma = cmsSpreadDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSSpreadDigitalDate(ck2F,valueDate,toDate,tpDate,tenor1,tenor2,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsSpreadDigitalArg = ck2F.CMSSpreadDigitalArgDate(valueDate,toDate,tpDate,tenor1,tenor2,tau);
           mu = cmsSpreadDigitalArg.mu;
           sigma = cmsSpreadDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSSpreadDigitalRange(ck2F,to,tp,tenor1,tenor2,tau,lower,upper)
            lower_value = ck2F.CMSSpreadDigital(to,tp,tenor1,tenor2,tau,lower);
            upper_value = ck2F.CMSSpreadDigital(to,tp,tenor1,tenor2,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CMSSpreadDigitalRangeDate(ck2F,valueDate,toDate,tpDate,tenor1,tenor2,tau,lower,upper)
            lower_value = ck2F.CMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,lower);
            upper_value = ck2F.CMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,upper);
            out = lower_value - upper_value;
        end
        
        function atmSwaption = ATMSwaption(ck2F,tx,tnr)
            tau = 0.25;
            pv01 = ck2F.PV01(tx,tx+tnr,tau);
            fsr  = ck2F.Fsr(tx,tx+tnr,tau);
            c_nN = ck2F.C_nN(tx,tx+tnr,tau,pv01,fsr);
            variances = ck2F.LocalVariance(0,tx);
            totalVar = 0.0;
            for i=1:2
                for j=1:2
                    totalVar = totalVar + c_nN(i)*c_nN(j)*variances(i,j);
                end
            end
            atmSwaption =  pv01 * sqrt(totalVar)/sqrt(2.0*pi);
        end
        
        function spreadInfo = SpreadInfo(ck2F,tx,tnr1,tnr2)
            tau = 0.25;
            cmsSpreadDigitalArg =  ck2F.CMSSpreadDigitalArg(tx,tx+1.0,tnr1,tnr2,tau);
            spreadInfo = cmsSpreadDigitalArg;
            spreadInfo.normSigma = cmsSpreadDigitalArg.sigma/sqrt(tx);
        end
        
        function impCorrel = ImpCorrel(ck2F,tx,tnr1,tnr2)
            
            tau = 0.25;
            pv01_1 = ck2F.PV01(tx,tx+tnr1,tau);
            fsr_1  = ck2F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = ck2F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = ck2F.PV01(tx,tx+tnr2,tau);
            fsr_2  = ck2F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = ck2F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            variances = ck2F.LocalVariance(0,tx);
            var1 = 0;
            var2 = 0;
            coVar = 0;
            for j=1:2
                for k=1:2
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                    var2 = var2 + c_nN_2(j)*c_nN_2(k)*variances(j,k);
                    coVar = coVar + c_nN_1(j)*c_nN_2(k)*variances(j,k);
                end
            end
            impCorrel = coVar/sqrt(var1)/sqrt(var2);
        end
        
        function impCovar = ImpCovar(ck2F,tx,tnr1,tnr2)
            tau = 0.25;
            pv01_1 = ck2F.PV01(tx,tx+tnr1,tau);
            fsr_1  = ck2F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = ck2F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = ck2F.PV01(tx,tx+tnr2,tau);
            fsr_2  = ck2F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = ck2F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            variances = ck2F.LocalVariance(0,tx);
            var1 = 0;
            var2 = 0;
            coVar = 0;
            for j=1:2
                for k=1:2
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                    var2 = var2 + c_nN_2(j)*c_nN_2(k)*variances(j,k);
                    coVar = coVar + c_nN_1(j)*c_nN_2(k)*variances(j,k);
                end
            end
            impCovar = coVar;
        end
        
        
        function instFwdCorr = InsFwdCorr(ck2F,tx,tnr1,tnr2)
            tau = 0.25;
            tenor = ck2F.vol_r.tenor;
            tIdx = min(find(tx<=tenor));
            meanR_r = ck2F.meanR_r;
            meanR_s = ck2F.meanR_s;
            vol_r = ck2F.vol_r;
            vol_s = ck2F.vol_s;
            corr_rs = ck2F.corr_rs;
            covar = 0.0;
            var1 = 0;
            var2 = 0;
            
            var1 = exp(-2*meanR_r*tnr1)*vol_r.quote(tIdx)*vol_r.quote(tIdx) + exp(-2*meanR_s*tnr1)*vol_s.quote(tIdx)*vol_s.quote(tIdx) + 2*exp(-(meanR_r+meanR_s)*tnr1)*vol_r.quote(tIdx)*vol_s.quote(tIdx)*corr_rs.quote(tIdx);
            var2 = exp(-2*meanR_r*tnr2)*vol_r.quote(tIdx)*vol_r.quote(tIdx) + exp(-2*meanR_s*tnr2)*vol_s.quote(tIdx)*vol_s.quote(tIdx) + 2*exp(-(meanR_r+meanR_s)*tnr2)*vol_r.quote(tIdx)*vol_s.quote(tIdx)*corr_rs.quote(tIdx);
            covar = exp(-meanR_r*(tnr1+tnr2))*vol_r.quote(tIdx)*vol_r.quote(tIdx) + exp(-meanR_s*(tnr1+tnr2))*vol_s.quote(tIdx)*vol_s.quote(tIdx) ...
                + exp(-(meanR_r*tnr1+meanR_s*tnr2))*vol_r.quote(tIdx)*vol_s.quote(tIdx)*corr_rs.quote(tIdx) + exp(-(meanR_r*tnr2+meanR_s*tnr1))*vol_r.quote(tIdx)*vol_s.quote(tIdx)*corr_rs.quote(tIdx);
            
            instFwdCorr = covar/sqrt(var1*var2);
        end
        
        
        
        function instFwdCorrTest = InsFwdCorrTest(ck2F,tx,tnr1,tnr2)
            tau = 0.25;
            sigma_r = ck2F.sigma_Test(tx,ck2F.vol_r);
            sigma_s = ck2F.sigma_Test(tx,ck2F.vol_s);
            a_r = ck2F.meanR_r;
            a_s = ck2F.meanR_s;
            corr = ck2F.corr_rs.quote(1);
            
            covar = 0.0;
            var1 = 0;
            var2 = 0;
            var1 =  exp(-2.0*a_r*tnr1)*sigma_r*sigma_r + exp(-2.0*a_s*tnr1)*sigma_s*sigma_s + exp(-1.0*(a_r+a_s)*tnr1)*2.0*corr*sigma_r*sigma_s;
            var2 =  exp(-2.0*a_r*tnr2)*sigma_r*sigma_r + exp(-2.0*a_s*tnr2)*sigma_s*sigma_s + exp(-1.0*(a_r+a_s)*tnr2)*2.0*corr*sigma_r*sigma_s;
            covar = exp(-1.0*a_r*(tnr1+ tnr2))*sigma_r*sigma_r + exp(-1.0*a_s*(tnr1+tnr2))*sigma_s*sigma_s + exp(-1.0*a_r*tnr1)*exp(-1.0*a_s*tnr2)*corr*sigma_r*sigma_s + exp(-1.0*a_r*tnr2)*exp(-1.0*a_s*tnr1)*corr*sigma_r*sigma_s;
            instFwdCorrTest = covar/sqrt(var1*var2);
        end
        
        
        function instFwdSwapCorrTest = InsSwapFwdCorrTest(ck2F,tx,tnr1,tnr2)
            tau = 0.25;
            C_tT1 = zeros(2,1);
            pv01_1 = ck2F.PV01(tx,tx+tnr1,tau);
            fsr_1  = ck2F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = ck2F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            C_tT1(1) = c_nN_1(1);
            C_tT1(2) = c_nN_1(2);
            
            C_tT2 = zeros(2,1);
            pv01_2 = ck2F.PV01(tx,tx+tnr2,tau);
            fsr_2  = ck2F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = ck2F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            C_tT2(1) = c_nN_2(1);
            C_tT2(2) = c_nN_2(2);
            
            dAlpha_t = zeros(2,1);
            dAlpha_t(1) = ck2F.sigma_Test(tx,ck2F.vol_r);
            dAlpha_t(2) = ck2F.sigma_Test(tx,ck2F.vol_s);
            corr = zeros(2,2);
            corr(1,1) = 1.0;
            corr(1,2) = ck2F.corr_rs.quote(1);
            corr(2,1) = corr(1,2);
            corr(2,2) = 1.0;
            
            covar = 0.0;
            var1 = 0.0;
            var2 = 0.0;
            for i=1:2
                for j=1:2
                    var1 = var1 + C_tT1(i)*C_tT1(j)*dAlpha_t(i)*dAlpha_t(j)*corr(i,j);
                    var2 = var2 + C_tT2(i)*C_tT2(j)*dAlpha_t(i)*dAlpha_t(j)*corr(i,j);
                    covar = covar + C_tT1(i)*C_tT2(j)*dAlpha_t(i)*dAlpha_t(j)*corr(i,j);
                end
            end
            instFwdSwapCorrTest = covar/sqrt(var1*var2);
        end
        
        function impVariance = ImpVariance(ck2F,tx,tnr1)
            tau = 0.25;
            pv01_1 = ck2F.PV01(tx,tx+tnr1,tau);
            fsr_1  = ck2F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = ck2F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            variances = ck2F.LocalVariance(0,tx);
            var1 = 0;
            for j=1:2
                for k=1:2
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                end
            end
            impVariance = var1;
        end

        function out = TargetFunctionATMSwaptionGlobal(ck2F,tvar,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            ck2FSwaptionPrice = zeros(1,size(tenor,2));
            
            for i=1:length(ck2F.vol_r.quote)
                ck2F.vol_r.quote(i) = tvar(i);
                ck2F.vol_s.quote(i) = tvar(i+length(ck2F.vol_r.quote));
            end
            out= 0;
            
            for j=1:length(expiry)
                for k=1:length(tenor)
                    ck2FSwaptionPrice(j,k) = ck2F.ATMSwaption(expiry(j),tenor(k));
                    out = out + (ck2FSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0)*(ck2FSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0)*10000;
                end
            end
        end   
        
        function out = TargetFunctionATMSwaption(ck2F,idxNow,tvar,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            ck2FSwaptionPrice = zeros(1,size(tenor,2));
            
            ck2F.vol_r.quote(idxNow) = tvar(1);
            ck2F.vol_s.quote(idxNow) = tvar(2);
            ck2F.corr_rs.quote(idxNow) = tvar(3);
            
            out= 0;
            for j=1:size(tenor,2)
                ck2FSwaptionPrice(j) = ck2F.ATMSwaption(expiry(idxNow),tenor(j));
                out = out + (ck2FSwaptionPrice(j)/blackSwaptionPrice(idxNow,j)-1.0)*(ck2FSwaptionPrice(j)/blackSwaptionPrice(idxNow,j)-1.0)*10000;
            end
            
        end        
        
        function out = CalibrateToATMSwaption(ck2F,black,calibrationFlag)
            out.vol_r = ck2F.vol_r;
            out.vol_s = ck2F.vol_s;
            out.corr_rs = ck2F.corr_rs;
          % Calculate Black ATM Swaption Prices
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
            
            expirySize = size(expiry,2);
            tenorSize = size(tenor,2);
            
            blackSwaptionPrice = zeros(expirySize,tenorSize);
            for k=1:expirySize
                for j=1:tenorSize
                    blackSwaptionPrice(k,j) = black.BlackATMSwaption(expiry(k),tenor(j));
                end
            end
            
            params.expiry = expiry;
            params.tenor = tenor;
            params.blackSwaptionPrice = blackSwaptionPrice;
            params.expirySize = expirySize;
            params.tenorSize =  tenorSize;
            
            out.blackSwaptionPrice = blackSwaptionPrice;
            out.CK2FSwaptionPrice = zeros(expirySize,tenorSize);
            out.relDiff = zeros(expirySize,tenorSize);
            out.rmseTotal=0.0;
            
            calibrationType = 'bootstrap';
            if nargin < 3
                calibrationType = 'bootstrap'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;
            switch calibrationType
                case 'bootstrap'
                    for k=1:expirySize
                        if k ~= 1
                            ck2F.vol_r.quote(k) = ck2F.vol_r.quote(k-1);
                            ck2F.vol_s.quote(k) = ck2F.vol_s.quote(k-1);
                            ck2F.corr_rs.quote(k) = ck2F.corr_rs.quote(k-1);
                        end    
                        tvar = zeros(1,3);
                        tvar(1) = ck2F.vol_r.quote(k);
                        tvar(2) = ck2F.vol_s.quote(k);
                        tvar(3) = ck2F.corr_rs.quote(k);
                        
                        lb(1) = 0.0001;
                        lb(2) = 0.0001;
                        ub(1) = 10.0;
                        ub(2) = 10.0;
                        
                        lb(3) = -0.99;
                        ub(3) = 0.0;
                        
                        targetfun = @(tvar) Targetck2FATMSwaption(ck2F,k,tvar,params);

                        options=optimset('Algorithm','interior-point');    
                        [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);

                        out.vol_r.quote(k) = x(1);
                        out.vol_s.quote(k) = x(2);
                        out.corr_rs.quote(k) = x(3);
                        
                        ck2F.vol_r.quote(k) = x(1);
                        ck2F.vol_s.quote(k) = x(2);
                        ck2F.corr_rs.quote(k) = x(3);
                        
                        for j=1:tenorSize
                            out.ck2FSwaptionPrice(k,j) = ck2F.ATMSwaption(expiry(k),tenor(j));
                            out.relDiff(k,j) = out.ck2FSwaptionPrice(k,j) /out.blackSwaptionPrice(k,j)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(k,j)*out.relDiff(k,j);
                            N=N+1;
                        end
                    end
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.vol_r = ck2F.vol_r;
                    out.vol_s = ck2F.vol_s;
                    out.corr_rs = ck2F.corr_rs;
                    out.ck2F = ck2F;
                case 'global'
                    for j=1:length(ck2F.vol_r.quote)
                        tvar(j) = ck2F.vol_r.quote(j);
                        tvar(length(ck2F.vol_r.quote)+j) = ck2F.vol_s.quote(j);
                        lb(j)= 0.0001;
                        lb(length(ck2F.vol_r.quote)+j)= 0.0001;
                        ub(j)= 10;
                        ub(length(ck2F.vol_r.quote)+j)= 10;
                    end
                    
                    outSize = params.expirySize*params.tenorSize; 
                    
                    if 1
                        x =  zeros(outSize,1);
                        options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                        tic
                        [ret, popt, info, covar]=levmar('TargetFunctionCK2FATMSwaptionSurfaceGlobal',tvar,x, 200, options,'unc',ck2F,params);
                        toc
                        for j=1:length(ck2F.vol_r.quote)
                            out.vol_r.quote(j) = popt(j);
                            out.vol_s.quote(j) = popt(j+length(ck2F.vol_r.quote));
                            ck2F.vol_r.quote(j) = popt(j);
                            ck2F.vol_s.quote(j) = popt(j+length(ck2F.vol_s.quote));
                        end
                    else
                        targetfun = @(tvar) TargetFunctionATMSwaptionGlobal(ck2F,tvar,params);
                        options=optimset('Algorithm','interior-point');
    %                     options=optimset('Algorithm','trust-region-reflective');
                        [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);

                        for j=1:length(ck2F.vol_r.quote)
                            out.vol_r.quote(j) = x(j);
                            out.vol_s.quote(j) = x(j+length(ck2F.vol_r.quote));
                            ck2F.vol_r.quote(j) = x(j);
                            ck2F.vol_s.quote(j) = x(j+length(ck2F.vol_s.quote));
                        end
                    end
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.CK2FSwaptionPrice(j,k) = ck2F.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.CK2FSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                otherwise
                    disp('unImplemented')
            end
        end    
    end
    
end




In [ ]:
classdef CK2F < IRModel
    %UNTITLED Summary of this class goes here
    %   Detailed explanation goes here
    
    properties
        % vol functor
        vol_r
        vol_s
        meanR_r
        meanR_s
        corr_rs
    end
    
    methods
        function ck2F = CK2F(params)
            if nargin > 0
                ck2F.zeroCurve = params.zeroCurve;
                ck2F.vol_r = params.modelParams('vol_r');
                ck2F.vol_s = params.modelParams('vol_s');
                ck2F.meanR_r =  params.modelParams('meanR_r');
                ck2F.meanR_s =  params.modelParams('meanR_s');
                ck2F.corr_rs =  params.modelParams('corr_rs');
            end
        end
        
        function out = SimpleSRPrOverHedge(ck2F,spreadC,spreadN,ralower,raupper)
            buffer = 0.001;
            outVec = 0.0;
            if spreadC < ralower - buffer
                outVec = 0.0;
            elseif spreadC >= ralower - buffer && spreadC < ralower
                outVec = 1.0/buffer * (spreadC - (ralower - buffer));
            elseif spreadC >= ralower && spreadC < raupper
                outVec = 1.0;
            elseif spreadC >= raupper && spreadC < raupper + buffer
                outVec = -1.0/buffer *(spreadC - (raupper + buffer));
            else
                outVec = 0.0;
            end
            
            out = outVec;
        end
        
        function out = SimpleSRPr(ck2F,spreadC,spreadN,ralower,raupper)
            outVec = 0.0;
            if spreadC >= ralower && spreadC <= raupper
                outVec = 1.0;
            end
            out = outVec;
        end
        
        
        function out = TRZ_SRPr(ck2F,curr,next,L,U)
            length = 1.0;
            remain = 1.0;
            if (curr >= L && curr <= U && next >= L && next <= U)
                length = remain;
            elseif ( curr >= L && curr <= U && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr >= L && curr <= U && next < L ) 
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr > U && next >= L && next <= U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next < L )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next >= L && next <= U )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next > U )
                length = 0.0;
            elseif ( curr < L && next < L)
                length = 0.0;
            else
                disp('error')
            end
            
            out = length;
        end
        
        function out = SRPr(ck2F,spreadC,spreadN,ralower,raupper,simpleYN,pathSize)
            
            outVec = zeros(1,pathSize);
            
            if simpleYN == 1
                for i =1:pathSize
                    outVec(i) = ck2F.SimpleSRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            elseif simpleYN == 2
                for i =1:pathSize
                    outVec(i) = ck2F.SimpleSRPrOverHedge(spreadC(i),spreadN(i),ralower,raupper);
                end
            else
                for i =1:pathSize
                    outVec(i) = ck2F.TRZ_SRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            end
            
            out = outVec;
        end
        
        function out =  CMS_PSA_SimpleDate(ck2F,valueDate,observeTime,numMatTime, tenor,modelStatesT)
           %CMS
           tau = 0.25;
           to      = observeTime/365.0;
           toDate = valueDate.AddDate(observeTime,'day');
           tNDate = toDate.AddDate(tenor,'year');
           tN   =  tNDate.DateDiff(valueDate)/365.0;
           
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           p_nN = ck2F.PV01Date(valueDate,toDate,tNDate,tau);
           
           df_Tn = ck2F.DF(to);
           df_TN = ck2F.DF(tN);
           y_nN = (df_Tn - df_TN)/p_nN;
           
           c_nN = ck2F.C_nNDate(valueDate,toDate,tNDate,tau,p_nN,y_nN);
           g_nN = ck2F.G_nNDate(valueDate,toDate,tNDate,tau,p_nN);
           h_nTerm = ck2F.H_nTermDate(valueDate,toDate,numMatDate);
           varCovar = ck2F.LocalVariance(0,to);
           
           stateSize = size(modelStatesT,2);
           cms_PCA_SimpleOut = zeros(1,stateSize);
           
           driftTerm = 0;
           for i=1:2
               for j=1:2
                   driftTerm = driftTerm + (c_nN(i) * h_nTerm(j) + c_nN(i) * g_nN(j))*varCovar(i,j);
               end
           end
           
           for i=1:stateSize
               cms_PCA_SimpleOut(i) = y_nN + driftTerm + c_nN(1)*modelStatesT(1,i) + c_nN(2) * modelStatesT(2,i); 
           end
           
           out = cms_PCA_SimpleOut;
           
            
        end
        
        function out = HWGeneralVariance(ck2F,from,to,T1,T2,MRS1,MRS2,sigmaTimes,sigmaValue1,sigmaValue2,corrValue)
            
            sigmaSize = length(sigmaValue1);
            
            for i=1:length(sigmaTimes)
                if from<= sigmaTimes(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:length(sigmaTimes)
                if to<= sigmaTimes(i)
                    break;
                end
            end
            endIdx = i;
            
            
            newExpScale = zeros(4,1);
            lastExpScale = zeros(4,1);
            variances = zeros(4,1);
            
            lastU  = from;
            lastExpScale(2) = exp(MRS1*lastU);
            lastExpScale(3) = exp(MRS2*lastU);
            lastExpScale(4) = exp((MRS1 + MRS2)*lastU);
            
            for i=startIdx:endIdx
                if (i< endIdx) U = sigmaTimes(i);else U = to;end;
                if (i< sigmaSize) sig1 = sigmaValue1(i);else sig1 = sigmaValue1(sigmaSize);end;
                if (i< sigmaSize) sig2 = sigmaValue2(i);else sig2 = sigmaValue2(sigmaSize);end;
                if (i< sigmaSize) corr12 = corrValue(i);else corr12 = corrValue(sigmaSize);end;
                
                variances(1) = variances(1) + corr12 * sig1 * sig2 * (U - lastU);
                
                newExpScale(2) = exp(MRS1*U);
                variances(2) = variances(2) + corr12 * sig1 * sig2 * (1.0/MRS1) * (newExpScale(2)-lastExpScale(2));
                lastExpScale(2) = newExpScale(2);
                
                newExpScale(3) = exp(MRS2*U);
                variances(3) = variances(3) + corr12 * sig1 * sig2 * (1.0/MRS2) * (newExpScale(3)-lastExpScale(3));
                lastExpScale(3) = newExpScale(3);
                
                newExpScale(4) = exp((MRS1+MRS2)*U);
                variances(4) = variances(4) + corr12 * sig1 * sig2 * (1.0/(MRS1+MRS2)) * (newExpScale(4)-lastExpScale(4));
                lastExpScale(4) = newExpScale(4);
                
                lastU = U;
                
            end
            
            value = (1.0/ MRS1)*(1.0/MRS2)*variances(1);
            value = value + -1.0*exp(-1.0*MRS1*T1)*(1.0/ MRS1)*(1.0/MRS2)*variances(2);
            value = value + -1.0*exp(-1.0*MRS2*T2)*(1.0/ MRS1)*(1.0/MRS2)*variances(3);
            value = value +  1.0*exp(-1.0*MRS1*T1)*exp(-1.0*MRS2*T2)*(1.0/ MRS1)*(1.0/MRS2)*variances(4);

            out  = value;
        end
        
        function out = fwdZcVariance(ck2F, from, to, T1, T2)
            
            MRS_r = ck2F.meanR_r;
            MRS_s = ck2F.meanR_s;
            sigma_r = ck2F.vol_r.quote;
            sigma_s = ck2F.vol_s.quote;
            correl_rs = ck2F.corr_rs.quote;
            correl_1 = ones(length(correl_rs),1);
            sigmaTimes = ck2F.vol_r.tenor;
            localVariance = zeros(6,1);
            
            localVariance(1) = ck2F.HWGeneralVariance(from,to,T1,T1,MRS_r,MRS_r,sigmaTimes,sigma_r,sigma_r,correl_1);
            localVariance(2) = ck2F.HWGeneralVariance(from,to,to,to,MRS_r,MRS_r,sigmaTimes,sigma_r,sigma_r,correl_1);
            
            localVariance(3) = ck2F.HWGeneralVariance(from,to,T1,T1,MRS_s,MRS_s,sigmaTimes,sigma_s,sigma_s,correl_1);
            localVariance(4) = ck2F.HWGeneralVariance(from,to,to,to,MRS_s,MRS_s,sigmaTimes,sigma_s,sigma_s,correl_1);
            
            localVariance(5) = ck2F.HWGeneralVariance(from,to,T1,T1,MRS_r,MRS_s,sigmaTimes,sigma_r,sigma_s,correl_rs);
            localVariance(6) = ck2F.HWGeneralVariance(from,to,to,to,MRS_r,MRS_s,sigmaTimes,sigma_r,sigma_s,correl_rs);
            
            value = localVariance(1) - localVariance(2);
            value = value + localVariance(3) - localVariance(4);
            value = value + 2.0*(localVariance(5) - localVariance(6));
            
            measureChange = zeros(8,1);
            
            measureChange(1) = ck2F.HWGeneralVariance(from,to,T1,T2,MRS_r,MRS_r,sigmaTimes,sigma_r,sigma_r,correl_1);
            measureChange(2) = ck2F.HWGeneralVariance(from,to,to,T2,MRS_r,MRS_r,sigmaTimes,sigma_r,sigma_r,correl_1);
            
            measureChange(3) = ck2F.HWGeneralVariance(from,to,T1,T2,MRS_r,MRS_s,sigmaTimes,sigma_r,sigma_s,correl_rs);
            measureChange(4) = ck2F.HWGeneralVariance(from,to,to,T2,MRS_r,MRS_s,sigmaTimes,sigma_r,sigma_s,correl_rs);
            
            measureChange(5) = ck2F.HWGeneralVariance(from,to,T1,T2,MRS_s,MRS_r,sigmaTimes,sigma_s,sigma_r,correl_rs);
            measureChange(6) = ck2F.HWGeneralVariance(from,to,to,T2,MRS_s,MRS_r,sigmaTimes,sigma_s,sigma_r,correl_rs);
            
            measureChange(7) = ck2F.HWGeneralVariance(from,to,T1,T2,MRS_s,MRS_s,sigmaTimes,sigma_s,sigma_s,correl_1);
            measureChange(8) = ck2F.HWGeneralVariance(from,to,to,T2,MRS_s,MRS_s,sigmaTimes,sigma_s,sigma_s,correl_1);
            
            measureChangeTerm =                     1.0 * (measureChange(1) - measureChange(2));
            measureChangeTerm = measureChangeTerm + 1.0 * (measureChange(3) - measureChange(4));
            measureChangeTerm = measureChangeTerm + 1.0 * (measureChange(5) - measureChange(6));
            measureChangeTerm = measureChangeTerm + 1.0 * (measureChange(7) - measureChange(8));
            
            value = value + -2.0*measureChangeTerm;
            
            out = value;
            
        end
        
        function out = discountFactor(ck2F, startTime, endTime,numMatTime,modelStatesT)
           zcT = ck2F.DF(endTime/365.0);
           zct = ck2F.DF(startTime/365.0);
           beta = ck2F.Beta(startTime/365.0,endTime/365.0);
           zcDriftTerm = ck2F.fwdZcVariance(0,startTime/365.0,endTime/365.0,numMatTime/365.0);
           %numOfPath
           stateSize = size(modelStatesT,2);
           discountFactorOut = zeros(1,stateSize);
           
           for i=1:stateSize
               discountFactorOut(i) = zcT/zct * exp(-0.5*zcDriftTerm - beta(1)*modelStatesT(1,i) ...
                   - beta(2) * modelStatesT(2,i)); 
           end
           
           out = discountFactorOut;
        end
        
        function out = discountPayoff(ck2F,eventTime,numMatTime ...
                ,modelStatesT,cashflow)
            
            dfNumMat = ck2F.DF(numMatTime/365.0);
            % for discountPayoff T=numMatTime
            % P(0,TnumMat)/P(Te,Tnummat)*Payoff
            %
            discountT = ck2F.discountFactor(eventTime, numMatTime,numMatTime,modelStatesT);
            stateSize = size(modelStatesT,2);
            payoff = zeros(1,stateSize);
            for i=1:stateSize
                payoff(i)= dfNumMat/discountT(i)*cashflow(i);
            end    
            out = payoff;
        end
        
        function out = stateLocalDrift(ck2F,fromTime, toTime)
            drift = zeros(2,2);
            drift(1,1) =  exp(-1.0*ck2F.meanR_r*(toTime-fromTime)/365.0);
            drift(2,2) =  exp(-1.0*ck2F.meanR_s*(toTime-fromTime)/365.0);
            out = drift;
        end
        
        function out = stateLocalVariance(ck2F,fromTime, toTime)
            out = ck2F.LocalVariance(fromTime/365.0,toTime/365.0);
        end
        
        function variances = LocalVariance(ck2F,from,to)
%             sig =[];
            sig(1) = ck2F.vol_r;
            sig(2) = ck2F.vol_s;
%             corr =[];
            corr_rs = ck2F.corr_rs;
            corr(1,1) = 1.0;
            corr(2,1) = corr_rs.quote(1);
            corr(1,2) = corr(2,1);
            corr(2,2) = 1.0;
            
%             meanR = [];
            meanR(1) = ck2F.meanR_r;
            meanR(2) = ck2F.meanR_s;
            
            alphaSize = length(ck2F.vol_r.quote);
            
            for i=1:length(sig(1).tenor)
                if from<= sig(1).tenor(i)
                    break;
                end
            end
            startIdx = i;
            
%             endIdx = size(sig(1).tenor,1); 
            for i=1:length(sig(1).tenor)
                if to<= sig(1).tenor(i)
                    break;
                end
            end
            endIdx = i;
            
%             tiny = {};
%             lastExpScale={};
%             newExpScale = {};
            for i=1:2
                for j=1:2
                    if(abs(meanR(i)+meanR(j)) < 1.0E-12) tiny(i,j) = 1; else tiny(i,j) = 0; end;
                    lastExpScale(i,j) =  exp((meanR(i)+meanR(j))*from);
                end
            end
            
            lastU  = from;
            
%             variances=[];
%             vol =[];
%             correl ={};
           variances = zeros(2,2);
           for i=startIdx:endIdx
                if (i< endIdx) U = sig(1).tenor(i);else U = to;end;
                if (i< alphaSize) vol(1) = sig(1).quote(i);else vol(1) = sig(1).quote(alphaSize);end;
                if (i< alphaSize) vol(2) = sig(2).quote(i);else vol(2) = sig(2).quote(alphaSize);end;
                correl(1,1) =1.0;
                correl(2,2) =1.0;
                if (i< alphaSize) correl(2,1) = corr_rs.quote(i);else correl(2,1) = corr_rs.quote(alphaSize);end;
                correl(1,2) = correl(2,1);
                for j=1:2
                    for k=1:2
                        newExpScale(j,k) =  exp((meanR(j)+meanR(k))*U);
                        if(tiny(j,k)~=1)
                            scale =newExpScale(j,k)-lastExpScale(j,k);
                        else
                            scale = U - lastU;
                        end
                        variances(j,k) = variances(j,k) + vol(j)*vol(k)*correl(j,k)...
                                       * scale;
                        lastExpScale(j,k) = newExpScale(j,k);
                    end
                end
                lastU = U;
           end
           
            for j=1:2
                for k=1:2
                    if(tiny(j,k)~= 1)
                        variances(j,k) = variances(j,k)/(meanR(j)+meanR(k))*exp(-1.0*(meanR(j)+meanR(k))*to);
                    end
                end
            end
        end
        
        function out = sigma_Test(ck2F,t,sigma)
            
            sum = 0.0;
            prev_t = 0.0;
            inRangeYN = false;
            for i=1:length(sigma.tenor)
                alpha = sigma.quote(i);
                if t<= sigma.tenor(i)
                    sum = alpha;
                    inRangeYN = true;
                    break;
                end
            end
            
            if ~inRangeYN 
                sum = alpha;
            end
            out = sum;
        end

        function beta = Beta(ck2F,t,T)
            a_r = ck2F.meanR_r;
            a_s = ck2F.meanR_s;
            beta =[];
            if(abs(a_r) > 1.0E-12)
                beta(1) = (1.0-exp(-a_r*(T-t)))/a_r;
            else
                beta(1) = (T-t);
            end
            
            if(abs(a_s) > 1.0E-12)
                beta(2) = (1.0-exp(-a_s*(T-t)))/a_s;
            else
                beta(2) = (T-t);
            end
        end
        
        function beta = BetaDate(ck2F,valueDate,tDate,TDate)
            t = tDate.DateDiff(valueDate)/365.0;
            T = TDate.DateDiff(valueDate)/365.0;
            
            a_r = ck2F.meanR_r;
            a_s = ck2F.meanR_s;
            beta =[];
            
            if(abs(a_r) > 1.0E-12)
                beta(1) = (1.0-exp(-a_r*(T-t)))/a_r;
            else
                beta(1) = (T-t);
            end
            
            if(abs(a_s) > 1.0E-12)
                beta(2) = (1.0-exp(-a_s*(T-t)))/a_s;
            else
                beta(2) = (T-t);
            end
            
        end
        
        function c_nN = C_nN(ck2F,tn,tN,tau,pnN,ynN)
            p = @ck2F.DF;
            c_nN=[0.0 0.0];
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = ck2F.Beta(tn,tp);
                df = p(tp);
                c_nN(1) = c_nN(1) +tau * beta(1) * df;
                c_nN(2) = c_nN(2) +tau * beta(2) * df;
            end
            
            c_nN(1) = c_nN(1)* ynN;
            c_nN(2) = c_nN(2)* ynN;
            
            c_nN(1) = c_nN(1) + beta(1)*df;
            c_nN(2) = c_nN(2) + beta(2)*df;
            
            c_nN(1) = c_nN(1)/pnN;
            c_nN(2) = c_nN(2)/pnN;
        end
        
        function c_nN = C_nNDate(ck2F,valueDate,tnDate,tNDate,tau,pnN,ynN)
            tn = tnDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            
            addMonthUnit = round(12*tau);
            cnt = round((tN-tn)/tau);
            
            p = @ck2F.DF;
            c_nN=[0.0 0.0];
            
            for i=1:cnt
                tpDate = tnDate.AddDate(i*addMonthUnit,'month');
                tp = tpDate.DateDiff(valueDate)/365.0;
                beta = ck2F.BetaDate(valueDate,tnDate,tpDate);
                df = p(tp);
                c_nN(1) = c_nN(1) +tau * beta(1) * df;
                c_nN(2) = c_nN(2) +tau * beta(2) * df;
            end
            
            c_nN(1) = c_nN(1)* ynN;
            c_nN(2) = c_nN(2)* ynN;
            
            c_nN(1) = c_nN(1) + beta(1)*df;
            c_nN(2) = c_nN(2) + beta(2)*df;
            
            c_nN(1) = c_nN(1)/pnN;
            c_nN(2) = c_nN(2)/pnN;
        end
        
         function g_nN = G_nN(ck2F,tn,tN,tau,pnN)
            p = @ck2F.DF;
            g_nN=[0.0 0.0];
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = ck2F.Beta(tn,tp);
                df = p(tp);
                g_nN(1) = g_nN(1) +tau * beta(1) * df;
                g_nN(2) = g_nN(2) +tau * beta(2) * df;
            end
            
            g_nN(1) = g_nN(1)/pnN;
            g_nN(2) = g_nN(2)/pnN;
         end
        
        function g_nN = G_nNDate(ck2F,valueDate,tnDate,tNDate,tau,pnN)
            tn = tnDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            addMonthUnit = round(12*tau);
            cnt = round((tN-tn)/tau);
            
            p = @ck2F.DF;
            g_nN=[0.0 0.0];
            
            for i=1:cnt
                tpDate = tnDate.AddDate(i*addMonthUnit,'month');
                tp = tpDate.DateDiff(valueDate)/365.0;
                beta = ck2F.BetaDate(valueDate,tnDate,tpDate);
                df = p(tp);
                g_nN(1) = g_nN(1) +tau * beta(1) * df;
                g_nN(2) = g_nN(2) +tau * beta(2) * df;
            end
            
            g_nN(1) = g_nN(1)/pnN;
            g_nN(2) = g_nN(2)/pnN;
        end 
        
        function h_nTerm = H_nTerm(ck2F,tn,tTerm)
           h_nTerm =[];
           beta = ck2F.Beta(tn,tTerm);
           h_nTerm(1) = -1.0*beta(1);
           h_nTerm(2) = -1.0*beta(2);
        end
        
        function h_nTerm = H_nTermDate(ck2F,valueDate,tnDate,tTermDate)
           h_nTerm =[];
           beta = ck2F.BetaDate(valueDate,tnDate,tTermDate);
           h_nTerm(1) = -1.0*beta(1);
           h_nTerm(2) = -1.0*beta(2);
        end
        
        
        function out = CMSDigitalArg(ck2F,to,tp,tenor1,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           sizeN = 2;
           p_nN1 = ck2F.PV01(to,to+tenor1,tau);
           df_Tn1 = ck2F.DF(to);
           df_TN1 = ck2F.DF(to + tenor1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           c_nN1 = ck2F.C_nN(to,to + tenor1,tau,p_nN1,y_nN1);
           g_nN1 = ck2F.G_nN(to,to + tenor1,tau,p_nN1);
           h_nTerm1 = ck2F.H_nTerm(to,tp);
           
           varCovar = ck2F.LocalVariance(0,to);
           
           driftTerm1=0.0;
           for i=1:sizeN
               for j=1:sizeN
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance1 = 0.0;
           for i=1:sizeN
               for j=1:sizeN
                    spreadVariance1 = spreadVariance1 + c_nN1(i) * c_nN1(j) * varCovar(i,j);
               end
           end
           
           out.fwdSpread = y_nN1;
           out.mu = y_nN1 + driftTerm1;
           out.sigma = sqrt(spreadVariance1);
           out.c_nN1 = c_nN1;
           
        end
        
        function out = CMSDigital(ck2F,to,tp,tenor1,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsDigitalArg = ck2F.CMSDigitalArg(to,tp,tenor1,tau);
           mu = cmsDigitalArg.mu;
           sigma = cmsDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSDigitalRange(ck2F,to,tp,tenor1,tau,lower,upper)
            lower_value = ck2F.CMSDigital(to,tp,tenor1,tau,lower);
            upper_value = ck2F.CMSDigital(to,tp,tenor1,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CMSSpreadDigitalArg(ck2F,to,tp,tenor1,tenor2,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           p_nN1 = ck2F.PV01(to,to+tenor1,tau);
           df_Tn1 = ck2F.DF(to);
           df_TN1 = ck2F.DF(to + tenor1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           c_nN1 = ck2F.C_nN(to,to + tenor1,tau,p_nN1,y_nN1);
           g_nN1 = ck2F.G_nN(to,to + tenor1,tau,p_nN1);
           h_nTerm1 = ck2F.H_nTerm(to,tp);
           
           %CMS2
           p_nN2 = ck2F.PV01(to,to+tenor2,tau);
           df_Tn2 = ck2F.DF(to);
           df_TN2 = ck2F.DF(to + tenor2);
           y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
           c_nN2 = ck2F.C_nN(to,to + tenor2,tau,p_nN2,y_nN2);
           g_nN2 = ck2F.G_nN(to,to + tenor2,tau,p_nN2);
           h_nTerm2 = ck2F.H_nTerm(to,tp);
           
           varCovar = ck2F.LocalVariance(0,to);
           
           driftTerm1=0.0;
           driftTerm2=0.0;
           for i=1:2
               for j=1:2
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
                    driftTerm2 = driftTerm2 + c_nN2(i) * (h_nTerm2(j) + g_nN2(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           variance1 = 0.0;
           variance2 = 0.0;
           coVariance = 0.0;
           for i=1:2
               for j=1:2
                    spreadVariance = spreadVariance + (c_nN1(i)-c_nN2(i)) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
                    variance1 = variance1 + (c_nN1(i)) * (c_nN1(j)) * varCovar(i,j);
                    variance2 = variance2 + (c_nN2(i)) * (c_nN2(j)) * varCovar(i,j);
                    coVariance = coVariance + (c_nN1(i)) * (c_nN2(j)) * varCovar(i,j);
               end
           end
           
           out.fwdSpread = y_nN1 - y_nN2;
           out.mu = (y_nN1 + driftTerm1) - (y_nN2 + driftTerm2);
           out.pmu = driftTerm1 - driftTerm2;
           out.sigma = sqrt(spreadVariance);
           out.c_nN1 = c_nN1;
           out.c_nN2 = c_nN2;
           out.varCovar = varCovar;
           out.mu1 = y_nN1 + driftTerm1;
           out.mu2 = y_nN2 + driftTerm2;
           out.pmu1 = driftTerm1;
           out.pmu2 = driftTerm2;
           
           out.y1 = y_nN1;
           out.y2 = y_nN2;
           
           out.corr = coVariance/sqrt(variance1*variance2);
           out.sig1 = sqrt(variance1);
           out.sig2 = sqrt(variance2);
           
        end
        
        function out = CMSSpreadDigitalArgDate(ck2F,valueDate,toDate,tpDate,tenor1,tenor2,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           to      = toDate.DateDiff(valueDate)/365.0;
           tN1Date = toDate.AddDate(tenor1,'year');
           tN1   =  tN1Date.DateDiff(valueDate)/365.0;
           
           p_nN1 = ck2F.PV01Date(valueDate,toDate,tN1Date,tau);
           
           df_Tn1 = ck2F.DF(to);
           df_TN1 = ck2F.DF(tN1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           
           c_nN1 = ck2F.C_nNDate(valueDate,toDate,tN1Date,tau,p_nN1,y_nN1);
           g_nN1 = ck2F.G_nNDate(valueDate,toDate,tN1Date,tau,p_nN1);
           h_nTerm1 = ck2F.H_nTermDate(valueDate,toDate,tpDate);
           
           %CMS2
           tN2Date = toDate.AddDate(tenor2,'year');
           tN2   =  tN2Date.DateDiff(valueDate)/365.0;
           
           p_nN2 = ck2F.PV01Date(valueDate,toDate,tN2Date,tau);
           
           df_Tn2 = ck2F.DF(to);
           df_TN2 = ck2F.DF(tN2);
           y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
           
           c_nN2 = ck2F.C_nNDate(valueDate,toDate,tN2Date,tau,p_nN2,y_nN2);
           g_nN2 = ck2F.G_nNDate(valueDate,toDate,tN2Date,tau,p_nN2);
           h_nTerm2 = ck2F.H_nTermDate(valueDate,toDate,tpDate);
           
           varCovar = ck2F.LocalVariance(0,to);
           
           driftTerm1=0.0;
           driftTerm2=0.0;
           for i=1:2
               for j=1:2
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
                    driftTerm2 = driftTerm2 + c_nN2(i) * (h_nTerm2(j) + g_nN2(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           variance1 = 0.0;
           variance2 = 0.0;
           coVariance = 0.0;
           for i=1:2
               for j=1:2
                    spreadVariance = spreadVariance + (c_nN1(i)-c_nN2(i)) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
                    variance1 = variance1 + (c_nN1(i)) * (c_nN1(j)) * varCovar(i,j);
                    variance2 = variance2 + (c_nN2(i)) * (c_nN2(j)) * varCovar(i,j);
                    coVariance = coVariance + (c_nN1(i)) * (c_nN2(j)) * varCovar(i,j);
               end
           end
           
           out.fwdSpread = y_nN1 - y_nN2;
           out.mu = (y_nN1 + driftTerm1) - (y_nN2 + driftTerm2);
           out.pmu = driftTerm1 - driftTerm2;
           out.sigma = sqrt(spreadVariance);
           out.c_nN1 = c_nN1;
           out.c_nN2 = c_nN2;
           out.varCovar = varCovar;
           out.mu1 = y_nN1 + driftTerm1;
           out.mu2 = y_nN2 + driftTerm2;
           
           out.pmu1 = driftTerm1;
           out.pmu2 = driftTerm2;
           
           out.corr = coVariance/sqrt(variance1*variance2);
           out.sig1 = sqrt(variance1);
           out.sig2 = sqrt(variance2);
           
        end
        
        function out = CMSSpreadDigital(ck2F,to,tp,tenor1,tenor2,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsSpreadDigitalArg = ck2F.CMSSpreadDigitalArg(to,tp,tenor1,tenor2,tau);
           mu = cmsSpreadDigitalArg.mu;
           sigma = cmsSpreadDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSSpreadDigitalDate(ck2F,valueDate,toDate,tpDate,tenor1,tenor2,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsSpreadDigitalArg = ck2F.CMSSpreadDigitalArgDate(valueDate,toDate,tpDate,tenor1,tenor2,tau);
           mu = cmsSpreadDigitalArg.mu;
           sigma = cmsSpreadDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSSpreadDigitalRange(ck2F,to,tp,tenor1,tenor2,tau,lower,upper)
            lower_value = ck2F.CMSSpreadDigital(to,tp,tenor1,tenor2,tau,lower);
            upper_value = ck2F.CMSSpreadDigital(to,tp,tenor1,tenor2,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CMSSpreadDigitalRangeDate(ck2F,valueDate,toDate,tpDate,tenor1,tenor2,tau,lower,upper)
            lower_value = ck2F.CMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,lower);
            upper_value = ck2F.CMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,upper);
            out = lower_value - upper_value;
        end
        
        function atmSwaption = ATMSwaption(ck2F,tx,tnr)
            tau = 0.25;
            pv01 = ck2F.PV01(tx,tx+tnr,tau);
            fsr  = ck2F.Fsr(tx,tx+tnr,tau);
            c_nN = ck2F.C_nN(tx,tx+tnr,tau,pv01,fsr);
            variances = ck2F.LocalVariance(0,tx);
            totalVar = 0.0;
            for i=1:2
                for j=1:2
                    totalVar = totalVar + c_nN(i)*c_nN(j)*variances(i,j);
                end
            end
            atmSwaption =  pv01 * sqrt(totalVar)/sqrt(2.0*pi);
        end
        
        function spreadInfo = SpreadInfo(ck2F,tx,tnr1,tnr2)
            tau = 0.25;
            cmsSpreadDigitalArg =  ck2F.CMSSpreadDigitalArg(tx,tx+1.0,tnr1,tnr2,tau);
            spreadInfo = cmsSpreadDigitalArg;
            spreadInfo.normSigma = cmsSpreadDigitalArg.sigma/sqrt(tx);
        end
        
        function impCorrel = ImpCorrel(ck2F,tx,tnr1,tnr2)
            
            tau = 0.25;
            pv01_1 = ck2F.PV01(tx,tx+tnr1,tau);
            fsr_1  = ck2F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = ck2F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = ck2F.PV01(tx,tx+tnr2,tau);
            fsr_2  = ck2F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = ck2F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            variances = ck2F.LocalVariance(0,tx);
            var1 = 0;
            var2 = 0;
            coVar = 0;
            for j=1:2
                for k=1:2
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                    var2 = var2 + c_nN_2(j)*c_nN_2(k)*variances(j,k);
                    coVar = coVar + c_nN_1(j)*c_nN_2(k)*variances(j,k);
                end
            end
            impCorrel = coVar/sqrt(var1)/sqrt(var2);
        end
        
        function impCovar = ImpCovar(ck2F,tx,tnr1,tnr2)
            tau = 0.25;
            pv01_1 = ck2F.PV01(tx,tx+tnr1,tau);
            fsr_1  = ck2F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = ck2F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = ck2F.PV01(tx,tx+tnr2,tau);
            fsr_2  = ck2F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = ck2F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            variances = ck2F.LocalVariance(0,tx);
            var1 = 0;
            var2 = 0;
            coVar = 0;
            for j=1:2
                for k=1:2
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                    var2 = var2 + c_nN_2(j)*c_nN_2(k)*variances(j,k);
                    coVar = coVar + c_nN_1(j)*c_nN_2(k)*variances(j,k);
                end
            end
            impCovar = coVar;
        end
        
        
        function instFwdCorr = InsFwdCorr(ck2F,tx,tnr1,tnr2)
            tau = 0.25;
            tenor = ck2F.vol_r.tenor;
            tIdx = min(find(tx<=tenor));
            meanR_r = ck2F.meanR_r;
            meanR_s = ck2F.meanR_s;
            vol_r = ck2F.vol_r;
            vol_s = ck2F.vol_s;
            corr_rs = ck2F.corr_rs;
            covar = 0.0;
            var1 = 0;
            var2 = 0;
            
            var1 = exp(-2*meanR_r*tnr1)*vol_r.quote(tIdx)*vol_r.quote(tIdx) + exp(-2*meanR_s*tnr1)*vol_s.quote(tIdx)*vol_s.quote(tIdx) + 2*exp(-(meanR_r+meanR_s)*tnr1)*vol_r.quote(tIdx)*vol_s.quote(tIdx)*corr_rs.quote(tIdx);
            var2 = exp(-2*meanR_r*tnr2)*vol_r.quote(tIdx)*vol_r.quote(tIdx) + exp(-2*meanR_s*tnr2)*vol_s.quote(tIdx)*vol_s.quote(tIdx) + 2*exp(-(meanR_r+meanR_s)*tnr2)*vol_r.quote(tIdx)*vol_s.quote(tIdx)*corr_rs.quote(tIdx);
            covar = exp(-meanR_r*(tnr1+tnr2))*vol_r.quote(tIdx)*vol_r.quote(tIdx) + exp(-meanR_s*(tnr1+tnr2))*vol_s.quote(tIdx)*vol_s.quote(tIdx) ...
                + exp(-(meanR_r*tnr1+meanR_s*tnr2))*vol_r.quote(tIdx)*vol_s.quote(tIdx)*corr_rs.quote(tIdx) + exp(-(meanR_r*tnr2+meanR_s*tnr1))*vol_r.quote(tIdx)*vol_s.quote(tIdx)*corr_rs.quote(tIdx);
            
            instFwdCorr = covar/sqrt(var1*var2);
        end
        
        
        
        function instFwdCorrTest = InsFwdCorrTest(ck2F,tx,tnr1,tnr2)
            tau = 0.25;
            sigma_r = ck2F.sigma_Test(tx,ck2F.vol_r);
            sigma_s = ck2F.sigma_Test(tx,ck2F.vol_s);
            a_r = ck2F.meanR_r;
            a_s = ck2F.meanR_s;
            corr = ck2F.corr_rs.quote(1);
            
            covar = 0.0;
            var1 = 0;
            var2 = 0;
            var1 =  exp(-2.0*a_r*tnr1)*sigma_r*sigma_r + exp(-2.0*a_s*tnr1)*sigma_s*sigma_s + exp(-1.0*(a_r+a_s)*tnr1)*2.0*corr*sigma_r*sigma_s;
            var2 =  exp(-2.0*a_r*tnr2)*sigma_r*sigma_r + exp(-2.0*a_s*tnr2)*sigma_s*sigma_s + exp(-1.0*(a_r+a_s)*tnr2)*2.0*corr*sigma_r*sigma_s;
            covar = exp(-1.0*a_r*(tnr1+ tnr2))*sigma_r*sigma_r + exp(-1.0*a_s*(tnr1+tnr2))*sigma_s*sigma_s + exp(-1.0*a_r*tnr1)*exp(-1.0*a_s*tnr2)*corr*sigma_r*sigma_s + exp(-1.0*a_r*tnr2)*exp(-1.0*a_s*tnr1)*corr*sigma_r*sigma_s;
            instFwdCorrTest = covar/sqrt(var1*var2);
        end
        
        
        function instFwdSwapCorrTest = InsSwapFwdCorrTest(ck2F,tx,tnr1,tnr2)
            tau = 0.25;
            C_tT1 = zeros(2,1);
            pv01_1 = ck2F.PV01(tx,tx+tnr1,tau);
            fsr_1  = ck2F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = ck2F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            C_tT1(1) = c_nN_1(1);
            C_tT1(2) = c_nN_1(2);
            
            C_tT2 = zeros(2,1);
            pv01_2 = ck2F.PV01(tx,tx+tnr2,tau);
            fsr_2  = ck2F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = ck2F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            C_tT2(1) = c_nN_2(1);
            C_tT2(2) = c_nN_2(2);
            
            dAlpha_t = zeros(2,1);
            dAlpha_t(1) = ck2F.sigma_Test(tx,ck2F.vol_r);
            dAlpha_t(2) = ck2F.sigma_Test(tx,ck2F.vol_s);
            corr = zeros(2,2);
            corr(1,1) = 1.0;
            corr(1,2) = ck2F.corr_rs.quote(1);
            corr(2,1) = corr(1,2);
            corr(2,2) = 1.0;
            
            covar = 0.0;
            var1 = 0.0;
            var2 = 0.0;
            for i=1:2
                for j=1:2
                    var1 = var1 + C_tT1(i)*C_tT1(j)*dAlpha_t(i)*dAlpha_t(j)*corr(i,j);
                    var2 = var2 + C_tT2(i)*C_tT2(j)*dAlpha_t(i)*dAlpha_t(j)*corr(i,j);
                    covar = covar + C_tT1(i)*C_tT2(j)*dAlpha_t(i)*dAlpha_t(j)*corr(i,j);
                end
            end
            instFwdSwapCorrTest = covar/sqrt(var1*var2);
        end
        
        function impVariance = ImpVariance(ck2F,tx,tnr1)
            tau = 0.25;
            pv01_1 = ck2F.PV01(tx,tx+tnr1,tau);
            fsr_1  = ck2F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = ck2F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            variances = ck2F.LocalVariance(0,tx);
            var1 = 0;
            for j=1:2
                for k=1:2
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                end
            end
            impVariance = var1;
        end

        function out = TargetFunctionATMSwaptionGlobal(ck2F,tvar,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            ck2FSwaptionPrice = zeros(1,size(tenor,2));
            
            for i=1:length(ck2F.vol_r.quote)
                ck2F.vol_r.quote(i) = tvar(i);
                ck2F.vol_s.quote(i) = tvar(i+length(ck2F.vol_r.quote));
            end
            out= 0;
            
            for j=1:length(expiry)
                for k=1:length(tenor)
                    ck2FSwaptionPrice(j,k) = ck2F.ATMSwaption(expiry(j),tenor(k));
                    out = out + (ck2FSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0)*(ck2FSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0)*10000;
                end
            end
        end   
        
        function out = TargetFunctionATMSwaption(ck2F,idxNow,tvar,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            ck2FSwaptionPrice = zeros(1,size(tenor,2));
            
            ck2F.vol_r.quote(idxNow) = tvar(1);
            ck2F.vol_s.quote(idxNow) = tvar(2);
            ck2F.corr_rs.quote(idxNow) = tvar(3);
            
            out= 0;
            for j=1:size(tenor,2)
                ck2FSwaptionPrice(j) = ck2F.ATMSwaption(expiry(idxNow),tenor(j));
                out = out + (ck2FSwaptionPrice(j)/blackSwaptionPrice(idxNow,j)-1.0)*(ck2FSwaptionPrice(j)/blackSwaptionPrice(idxNow,j)-1.0)*10000;
            end
            
        end        
        
        function out = CalibrateToATMSwaption(ck2F,black,calibrationFlag)
            out.vol_r = ck2F.vol_r;
            out.vol_s = ck2F.vol_s;
            out.corr_rs = ck2F.corr_rs;
          % Calculate Black ATM Swaption Prices
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
            
            expirySize = size(expiry,2);
            tenorSize = size(tenor,2);
            
            blackSwaptionPrice = zeros(expirySize,tenorSize);
            for k=1:expirySize
                for j=1:tenorSize
                    blackSwaptionPrice(k,j) = black.BlackATMSwaption(expiry(k),tenor(j));
                end
            end
            
            params.expiry = expiry;
            params.tenor = tenor;
            params.blackSwaptionPrice = blackSwaptionPrice;
            params.expirySize = expirySize;
            params.tenorSize =  tenorSize;
            
            out.blackSwaptionPrice = blackSwaptionPrice;
            out.CK2FSwaptionPrice = zeros(expirySize,tenorSize);
            out.relDiff = zeros(expirySize,tenorSize);
            out.rmseTotal=0.0;
            
            calibrationType = 'bootstrap';
            if nargin < 3
                calibrationType = 'bootstrap'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;
            switch calibrationType
                case 'bootstrap'
                    for k=1:expirySize
                        if k ~= 1
                            ck2F.vol_r.quote(k) = ck2F.vol_r.quote(k-1);
                            ck2F.vol_s.quote(k) = ck2F.vol_s.quote(k-1);
                            ck2F.corr_rs.quote(k) = ck2F.corr_rs.quote(k-1);
                        end    
                        tvar = zeros(1,3);
                        tvar(1) = ck2F.vol_r.quote(k);
                        tvar(2) = ck2F.vol_s.quote(k);
                        tvar(3) = ck2F.corr_rs.quote(k);
                        
                        lb(1) = 0.0001;
                        lb(2) = 0.0001;
                        ub(1) = 10.0;
                        ub(2) = 10.0;
                        
                        lb(3) = -0.99;
                        ub(3) = 0.0;
                        
                        targetfun = @(tvar) Targetck2FATMSwaption(ck2F,k,tvar,params);

                        options=optimset('Algorithm','interior-point');    
                        [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);

                        out.vol_r.quote(k) = x(1);
                        out.vol_s.quote(k) = x(2);
                        out.corr_rs.quote(k) = x(3);
                        
                        ck2F.vol_r.quote(k) = x(1);
                        ck2F.vol_s.quote(k) = x(2);
                        ck2F.corr_rs.quote(k) = x(3);
                        
                        for j=1:tenorSize
                            out.ck2FSwaptionPrice(k,j) = ck2F.ATMSwaption(expiry(k),tenor(j));
                            out.relDiff(k,j) = out.ck2FSwaptionPrice(k,j) /out.blackSwaptionPrice(k,j)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(k,j)*out.relDiff(k,j);
                            N=N+1;
                        end
                    end
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.vol_r = ck2F.vol_r;
                    out.vol_s = ck2F.vol_s;
                    out.corr_rs = ck2F.corr_rs;
                    out.ck2F = ck2F;
                case 'global'
                    for j=1:length(ck2F.vol_r.quote)
                        tvar(j) = ck2F.vol_r.quote(j);
                        tvar(length(ck2F.vol_r.quote)+j) = ck2F.vol_s.quote(j);
                        lb(j)= 0.0001;
                        lb(length(ck2F.vol_r.quote)+j)= 0.0001;
                        ub(j)= 10;
                        ub(length(ck2F.vol_r.quote)+j)= 10;
                    end
                    
                    outSize = params.expirySize*params.tenorSize; 
                    
                    if 1
                        x =  zeros(outSize,1);
                        options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                        tic
                        [ret, popt, info, covar]=levmar('TargetFunctionCK2FATMSwaptionSurfaceGlobal',tvar,x, 200, options,'unc',ck2F,params);
                        toc
                        for j=1:length(ck2F.vol_r.quote)
                            out.vol_r.quote(j) = popt(j);
                            out.vol_s.quote(j) = popt(j+length(ck2F.vol_r.quote));
                            ck2F.vol_r.quote(j) = popt(j);
                            ck2F.vol_s.quote(j) = popt(j+length(ck2F.vol_s.quote));
                        end
                    else
                        targetfun = @(tvar) TargetFunctionATMSwaptionGlobal(ck2F,tvar,params);
                        options=optimset('Algorithm','interior-point');
    %                     options=optimset('Algorithm','trust-region-reflective');
                        [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);

                        for j=1:length(ck2F.vol_r.quote)
                            out.vol_r.quote(j) = x(j);
                            out.vol_s.quote(j) = x(j+length(ck2F.vol_r.quote));
                            ck2F.vol_r.quote(j) = x(j);
                            ck2F.vol_s.quote(j) = x(j+length(ck2F.vol_s.quote));
                        end
                    end
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.CK2FSwaptionPrice(j,k) = ck2F.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.CK2FSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                otherwise
                    disp('unImplemented')
            end
        end    
    end
    
end



In [ ]:
classdef CK1FRisky < IRModelRisky
    %UNTITLED Summary of this class goes here
    %   Detailed explanation goes here
    
    properties
        % vol functor
        vol_r
        meanR_r
        matu
        joint_swap_tenor
        targetSize
        freq % 
        numOfFactors
        numOfFactor
        modelType
        
        instrumentStartDate
        instrumentEndDate
        calibrationTargetTenor
    end
    
    methods
        function ck1F = CK1FRisky(params)
            if nargin > 0
                
                ck1F.zeroCurve = params.zeroCurve;
                ck1F.discountCurve = params.discountCurve;
                
                ck1F.mktData = params.mktData;
                ck1F.modelParams = params.modelParams;
                
                ck1F.vol_r = params.modelParams('vol_r');
                ck1F.meanR_r =  params.modelParams('meanR_r');
%                 ck1F.matu =  params.modelParams('matu');
                ck1F.joint_swap_tenor =  params.modelParams('joint_swap_tenor');
%                 ck1F.targetSize = params.modelParams('targetSize');
                ck1F.freq  =params.modelParams('freq');
                
                if isKey(params.modelParams,'matu')
                    ck1F.matu = params.modelParams('matu');
                else
                    ck1F.matu = 15.0; % default
                end
                
                
                if isKey(params.modelParams,'targetSize')
                    ck1F.targetSize = params.modelParams('targetSize');
                else
                    ck1F.targetSize = 15; % default
                end
                
                % make instrument specific info as calibration input
                 
                if isKey(params.modelParams,'instrumentStartDate')
                    ck1F.instrumentStartDate = params.modelParams('instrumentStartDate');
                else
                    % default as same as evaluationDate
                    ck1F.instrumentStartDate = H_Date(ck1F.zeroCurve.asOfDate); % default
                end
                
                if isKey(params.modelParams,'instrumentEndDate')
                    ck1F.instrumentEndDate = params.modelParams('instrumentEndDate');
                else
                    % default as instrumentStartDate + matu
                    ck1F.instrumentEndDate = ck1F.instrumentStartDate.AddDate(round(ck1F.matu*365),'year'); % default
                end
                
                if isKey(params.modelParams,'calibrationTargetTenor')
                    ck1F.calibrationTargetTenor = params.modelParams('calibrationTargetTenor');
                else
                    % default as 1,2,3,...,14
%                     ck1F.calibrationTargetTenor = 1:1:14; % default
                    ck1F.calibrationTargetTenor = 1:1:19; % default
                end
                
                ck1F.numOfFactors = 1;
                ck1F.numOfFactor = 1;
                ck1F.modelType = 'CK1F';
                
            end
           
        end
        
        function out = SimpleSRPrOverHedge(ck1F,spreadC,spreadN,ralower,raupper)
            buffer = 0.001;
            outVec = 0.0;
            if spreadC < ralower - buffer
                outVec = 0.0;
            elseif spreadC >= ralower - buffer && spreadC < ralower
                outVec = 1.0/buffer * (spreadC - (ralower - buffer));
            elseif spreadC >= ralower && spreadC < raupper
                outVec = 1.0;
            elseif spreadC >= raupper && spreadC < raupper + buffer
                outVec = -1.0/buffer *(spreadC - (raupper + buffer));
            else
                outVec = 0.0;
            end
            
            out = outVec;
        end
        
        function out = SimpleSRPr(ck1F,spreadC,spreadN,ralower,raupper)
            outVec = 0.0;
            if spreadC >= ralower && spreadC <= raupper
                outVec = 1.0;
            end
            out = outVec;
        end
        
        
        function out = TRZ_SRPr(ck1F,curr,next,L,U)
            length = 1.0;
            remain = 1.0;
            if (curr >= L && curr <= U && next >= L && next <= U)
                length = remain;
            elseif ( curr >= L && curr <= U && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr >= L && curr <= U && next < L ) 
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr > U && next >= L && next <= U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next < L )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next >= L && next <= U )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next > U )
                length = 0.0;
            elseif ( curr < L && next < L)
                length = 0.0;
            else
                disp('error')
            end
            
            out = length;
        end
        
        function out = SRPr(ck1F,spreadC,spreadN,ralower,raupper,simpleYN,pathSize)
            
            outVec = zeros(1,pathSize);
            
            if simpleYN == 1
                for i =1:pathSize
                    outVec(i) = ck1F.SimpleSRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            elseif simpleYN == 2
                for i =1:pathSize
                    outVec(i) = ck1F.SimpleSRPrOverHedge(spreadC(i),spreadN(i),ralower,raupper);
                end
            else
                for i =1:pathSize
                    outVec(i) = ck1F.TRZ_SRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            end
            
            out = outVec;
        end
        
        function out =  Libor_SimpleDate(ck1F,valueDate,evalTime, fwdStartTime, numMatTime, tenor,modelStatesT)
           %CMS
           fwdStartDate = valueDate.AddDate(fwdStartTime,'day');
           fwdEndDate = fwdStartDate.AddDate(tenor,'year');
           fwdEndTime = fwdEndDate.DateDiff(valueDate);
           
           deltaT = (fwdEndTime - fwdStartTime)/365.0;
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           
           stateSize = size(modelStatesT,2);
           df_Tn = zeros(1,stateSize);
           df_TN = zeros(1,stateSize);
           
           df_Tn = ck1F.discountFactor(evalTime,fwdStartTime,numMatTime,modelStatesT);
           df_TN = ck1F.discountFactor(evalTime,fwdEndTime,numMatTime,modelStatesT);
           
           liborOut = zeros(1,stateSize);
           
           for i=1:stateSize
                liborOut(i) = (df_Tn(i)/df_TN(i) - 1.0)/deltaT;
           end
           
           out = liborOut;
            
        end
        
        function out =  CMS_SimpleDate(ck1F,valueDate,observeTime,numMatTime, tenor,modelStatesT)
           %CMS
           tau = 0.25;
           to      = observeTime/365.0;
           toDate = valueDate.AddDate(observeTime,'day');
           if tenor < 1
               tenorMonth = round(tenor*12);
               tNDate = toDate.AddDate(tenorMonth,'month');
           else
               tNDate = toDate.AddDate(tenor,'year');
           end
           
           tNTime = tNDate.DateDiff(valueDate); 
           tN     = tNTime/365.0;
            
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           addMonthUnit = round(12*tau);
           cnt = round((tN-to)/tau);
           
%            p_nN = ck1F.PV01Date(valueDate,toDate,tNDate,tau);
           stateSize = size(modelStatesT,2);
           
           annuity = zeros(1,stateSize);
           df_Tp = zeros(1,stateSize);
           
           df_Tn = zeros(1,stateSize);
           df_TN = zeros(1,stateSize);
           
           for i=1:cnt
                tpDate = toDate.AddDate(i*addMonthUnit,'month');
                tpTime = tpDate.DateDiff(valueDate);
                df_Tp = ck1F.discountFactor(observeTime,tpTime,numMatTime,modelStatesT);
                annuity = annuity + tau * df_Tp;
            end
           
           df_Tn = ck1F.discountFactor(observeTime,observeTime,numMatTime,modelStatesT);
           df_TN = ck1F.discountFactor(observeTime,tNTime,numMatTime,modelStatesT);
           
           cms_PCA_SimpleOut = zeros(1,stateSize);
           
           for i=1:stateSize
                cms_PCA_SimpleOut(i) = (df_Tn(i) - df_TN(i))/annuity(i);
           end
           
           out = cms_PCA_SimpleOut;
            
        end
        
        function out =  CMS_PSA_SimpleDate_FallbackSimple(ck1F,valueDate,observeTime,numMatTime, tenor,modelStatesT)
        
                cms = CMS_PSA_SimpleDate(ck1F,valueDate,observeTime,numMatTime, tenor,modelStatesT);
                stateSize = size(modelStatesT,2);
                fallbackOut = zeros(1,stateSize);
                
                fbs_3m = 0.0026161;
                
                
                for i=1:stateSize
                    if cms(i) >= -1.0
                        fallbackOut(i) = 365.25/360.0 *( (1.0 + 0.125 * fbs_3m) * cms(i) + fbs_3m);
                    else
                        fallbackOut(i) = 365.25/360.0 *(-2.0 + 0.5 * fbs_3m);
                    end
                end
                
                out = fallbackOut;
        end
        
        function out =  CMS_PSA_SimpleDate_Fallback(ck1F,valueDate,observeTime,numMatTime, tenor,modelStatesT)
        
                cms = CMS_PSA_SimpleDate(ck1F,valueDate,observeTime,numMatTime, tenor,modelStatesT);
                stateSize = size(modelStatesT,2);
                fallbackOut = zeros(1,stateSize);
                
                fbs_3m = 0.0026161;
                
                
                for i=1:stateSize
                    if cms(i) >= -1.0
                        fallbackOut(i) = 365.25/360.0 *( 2*(power(1.0 + cms(i),0.5)-1.0) + 0.5 * fbs_3m *(power(1.0 + cms(i),0.25) + 1.0) );
                    else
                        fallbackOut(i) = 365.25/360.0 *(-2.0 + 0.5 * fbs_3m);
                    end
                end
                
                out = fallbackOut;
        end
        
        function out =  CMS_PSA_SimpleDate(ck1F,valueDate,observeTime,numMatTime, tenor,modelStatesT)
           %CMS
%            tau = 0.25;
           tau = 1.0/ck1F.freq;
           % if tenor is 0.25(3M) then use tenor(0.25) instead
           % model(zeroCurve)'s freq
           
           if abs(tenor- 0.25) < 1e-8
                tau = tenor;
           end
           
           to      = observeTime/365.0;
           toDate = valueDate.AddDate(observeTime,'day');
           tNDate = toDate.AddDate(tenor,'year');
           tN   =  tNDate.DateDiff(valueDate)/365.0;
           
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           p_nN = ck1F.PV01Date(valueDate,toDate,tNDate,tau);
           
           df_Tn = ck1F.DF(to);
           df_TN = ck1F.DF(tN);
           y_nN = (df_Tn - df_TN)/p_nN;
           
           c_nN = ck1F.C_nNDate(valueDate,toDate,tNDate,tau,p_nN,y_nN);
           g_nN = ck1F.G_nNDate(valueDate,toDate,tNDate,tau,p_nN);
           h_nTerm = ck1F.H_nTermDate(valueDate,toDate,numMatDate);
           varCovar = ck1F.LocalVariance(0,to);
           
           stateSize = size(modelStatesT,2);
           cms_PCA_SimpleOut = zeros(1,stateSize);
           
           driftTerm = 0;
           for i=1:1
               for j=1:1
                   driftTerm = driftTerm + (c_nN(i) * h_nTerm(j) + c_nN(i) * g_nN(j))*varCovar(i,j);
               end
           end
           
           for i=1:stateSize
               cms_PCA_SimpleOut(i) = y_nN + driftTerm + c_nN(1)*modelStatesT(1,i); 
           end
           
           out = cms_PCA_SimpleOut;
           
            
        end
        
        function out = HWGeneralVariance(ck1F,from,to,T1,T2,MRS1,MRS2,sigmaTimes,sigmaValue1,sigmaValue2,corrValue)
            
            sigmaSize = length(sigmaValue1);
            
            for i=1:length(sigmaTimes)
                if from<= sigmaTimes(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:length(sigmaTimes)
                if to<= sigmaTimes(i)
                    break;
                end
            end
            endIdx = i;
            
            
            newExpScale = zeros(4,1);
            lastExpScale = zeros(4,1);
            variances = zeros(4,1);
            
            lastU  = from;
            lastExpScale(2) = exp(MRS1*lastU);
            lastExpScale(3) = exp(MRS2*lastU);
            lastExpScale(4) = exp((MRS1 + MRS2)*lastU);
            
            for i=startIdx:endIdx
                if (i< endIdx) U = sigmaTimes(i);else U = to;end;
                if (i< sigmaSize) sig1 = sigmaValue1(i);else sig1 = sigmaValue1(sigmaSize);end;
                if (i< sigmaSize) sig2 = sigmaValue2(i);else sig2 = sigmaValue2(sigmaSize);end;
                if (i< sigmaSize) corr12 = corrValue(i);else corr12 = corrValue(sigmaSize);end;
                
                variances(1) = variances(1) + corr12 * sig1 * sig2 * (U - lastU);
                
                newExpScale(2) = exp(MRS1*U);
                variances(2) = variances(2) + corr12 * sig1 * sig2 * (1.0/MRS1) * (newExpScale(2)-lastExpScale(2));
                lastExpScale(2) = newExpScale(2);
                
                newExpScale(3) = exp(MRS2*U);
                variances(3) = variances(3) + corr12 * sig1 * sig2 * (1.0/MRS2) * (newExpScale(3)-lastExpScale(3));
                lastExpScale(3) = newExpScale(3);
                
                newExpScale(4) = exp((MRS1+MRS2)*U);
                variances(4) = variances(4) + corr12 * sig1 * sig2 * (1.0/(MRS1+MRS2)) * (newExpScale(4)-lastExpScale(4));
                lastExpScale(4) = newExpScale(4);
                
                lastU = U;
                
            end
            
            value = (1.0/ MRS1)*(1.0/MRS2)*variances(1);
            value = value + -1.0*exp(-1.0*MRS1*T1)*(1.0/ MRS1)*(1.0/MRS2)*variances(2);
            value = value + -1.0*exp(-1.0*MRS2*T2)*(1.0/ MRS1)*(1.0/MRS2)*variances(3);
            value = value +  1.0*exp(-1.0*MRS1*T1)*exp(-1.0*MRS2*T2)*(1.0/ MRS1)*(1.0/MRS2)*variances(4);

            out  = value;
        end
        
        function out = fwdZcVariance(ck1F, from, to, T1, T2)
            
            MRS_r = ck1F.meanR_r;
            sigma_r = ck1F.vol_r.quote;
            correl_1 = ones(length(sigma_r),1);
            sigmaTimes = ck1F.vol_r.tenor;
            localVariance = zeros(6,1);
            
            localVariance(1) = ck1F.HWGeneralVariance(from,to,T1,T1,MRS_r,MRS_r,sigmaTimes,sigma_r,sigma_r,correl_1);
            localVariance(2) = ck1F.HWGeneralVariance(from,to,to,to,MRS_r,MRS_r,sigmaTimes,sigma_r,sigma_r,correl_1);
            
            value = localVariance(1) - localVariance(2);
            
            measureChange = zeros(8,1);
            
            measureChange(1) = ck1F.HWGeneralVariance(from,to,T1,T2,MRS_r,MRS_r,sigmaTimes,sigma_r,sigma_r,correl_1);
            measureChange(2) = ck1F.HWGeneralVariance(from,to,to,T2,MRS_r,MRS_r,sigmaTimes,sigma_r,sigma_r,correl_1);
            
            measureChangeTerm =                     1.0 * (measureChange(1) - measureChange(2));
            
            value = value + -2.0*measureChangeTerm;
            
            out = value;
            
        end
        
        function out = discountFactorRisky(ck1F, startTime, endTime,numMatTime,modelStatesT)
           zcT = ck1F.DFRisky(endTime/365.0);
           zct = ck1F.DFRisky(startTime/365.0);
           beta = ck1F.Beta(startTime/365.0,endTime/365.0);
           zcDriftTerm = ck1F.fwdZcVariance(0,startTime/365.0,endTime/365.0,numMatTime/365.0);
           %numOfPath
           stateSize = size(modelStatesT,2);
           discountFactorOut = zeros(1,stateSize);
           
           for i=1:stateSize
               discountFactorOut(i) = zcT/zct * exp(-0.5*zcDriftTerm - beta(1)*modelStatesT(1,i));
           end
           
           out = discountFactorOut;
        end
        
        function out = discountFactor(ck1F, startTime, endTime,numMatTime,modelStatesT)
           zcT = ck1F.DF(endTime/365.0);
           zct = ck1F.DF(startTime/365.0);
           beta = ck1F.Beta(startTime/365.0,endTime/365.0);
           zcDriftTerm = ck1F.fwdZcVariance(0,startTime/365.0,endTime/365.0,numMatTime/365.0);
           %numOfPath
           stateSize = size(modelStatesT,2);
           discountFactorOut = zeros(1,stateSize);
           
           for i=1:stateSize
               discountFactorOut(i) = zcT/zct * exp(-0.5*zcDriftTerm - beta(1)*modelStatesT(1,i)); 
           end
           
           out = discountFactorOut;
        end
        
        function out = discountPayoffRisky(ck1F,eventTime,numMatTime ...
                ,modelStatesT,cashflow)
            
            dfNumMat = ck1F.DFRisky(numMatTime/365.0);
            % for discountPayoff T=numMatTime
            % P(0,TnumMat)/P(Te,Tnummat)*Payoff
            %
            discountT = ck1F.discountFactorRisky(eventTime, numMatTime,numMatTime,modelStatesT);
            stateSize = size(modelStatesT,2);
            payoff = zeros(1,stateSize);
            for i=1:stateSize
                payoff(i)= dfNumMat/discountT(i)*cashflow(i);
            end    
            out = payoff;
        end
        
        function out = discountPayoff(ck1F,eventTime,numMatTime ...
                ,modelStatesT,cashflow)
            
            dfNumMat = ck1F.DF(numMatTime/365.0);
            % for discountPayoff T=numMatTime
            % P(0,TnumMat)/P(Te,Tnummat)*Payoff
            %
            discountT = ck1F.discountFactor(eventTime, numMatTime,numMatTime,modelStatesT);
            stateSize = size(modelStatesT,2);
            payoff = zeros(1,stateSize);
            for i=1:stateSize
                payoff(i)= dfNumMat/discountT(i)*cashflow(i);
            end    
            out = payoff;
        end
        
        function out = stateLocalDrift(ck1F,fromTime, toTime)
            drift = zeros(1,1);
            drift(1,1) =  exp(-1.0*ck1F.meanR_r*(toTime-fromTime)/365.0);
            out = drift;
        end
        
        function out = stateLocalVariance(ck1F,fromTime, toTime)
            out = ck1F.LocalVariance(fromTime/365.0,toTime/365.0);
        end
        
        function variances = LocalVariance(ck1F,from,to)
%             sig =[];
            sig(1) = ck1F.vol_r;
%             corr =[];
            corr(1,1) = 1.0;
            
%             meanR = [];
            meanR(1) = ck1F.meanR_r;
            
            alphaSize = length(ck1F.vol_r.quote);
            
            for i=1:length(sig(1).tenor)
                if from<= sig(1).tenor(i)
                    break;
                end
            end
            startIdx = i;
            
%             endIdx = size(sig(1).tenor,1); 
            for i=1:length(sig(1).tenor)
                if to<= sig(1).tenor(i)
                    break;
                end
            end
            endIdx = i;
            
%             tiny = {};
%             lastExpScale={};
%             newExpScale = {};
            for i=1:1
                for j=1:1
                    if(abs(meanR(i)+meanR(j)) < 1.0E-12) tiny(i,j) = 1; else tiny(i,j) = 0; end;
                    lastExpScale(i,j) =  exp((meanR(i)+meanR(j))*from);
                end
            end
            
            lastU  = from;
            
%             variances=[];
%             vol =[];
%             correl ={};
           variances = zeros(1,1);
           for i=startIdx:endIdx
                if (i< endIdx) U = sig(1).tenor(i);else U = to;end;
                if (i< alphaSize) vol(1) = sig(1).quote(i);else vol(1) = sig(1).quote(alphaSize);end;
                correl(1,1) =1.0;
                for j=1:1
                    for k=1:1
                        newExpScale(j,k) =  exp((meanR(j)+meanR(k))*U);
                        if(tiny(j,k)~=1)
                            scale =newExpScale(j,k)-lastExpScale(j,k);
                        else
                            scale = U - lastU;
                        end
                        variances(j,k) = variances(j,k) + vol(j)*vol(k)*correl(j,k)...
                                       * scale;
                        lastExpScale(j,k) = newExpScale(j,k);
                    end
                end
                lastU = U;
           end
           
            for j=1:1
                for k=1:1
                    if(tiny(j,k)~= 1)
                        variances(j,k) = variances(j,k)/(meanR(j)+meanR(k))*exp(-1.0*(meanR(j)+meanR(k))*to);
                    end
                end
            end
        end
        
        function out = sigma_Test(ck1F,t,sigma)
            
            sum = 0.0;
            prev_t = 0.0;
            inRangeYN = false;
            for i=1:length(sigma.tenor)
                alpha = sigma.quote(i);
                if t<= sigma.tenor(i)
                    sum = alpha;
                    inRangeYN = true;
                    break;
                end
            end
            
            if ~inRangeYN 
                sum = alpha;
            end
            out = sum;
        end

        function beta = Beta(ck1F,t,T)
            a_r = ck1F.meanR_r;
            beta =[];
            if(abs(a_r) > 1.0E-12)
                beta(1) = (1.0-exp(-a_r*(T-t)))/a_r;
            else
                beta(1) = (T-t);
            end
        end
        
        function beta = BetaDate(ck1F,valueDate,tDate,TDate)
            t = tDate.DateDiff(valueDate)/365.0;
            T = TDate.DateDiff(valueDate)/365.0;
            
            a_r = ck1F.meanR_r;
            beta =[];
            
            if(abs(a_r) > 1.0E-12)
                beta(1) = (1.0-exp(-a_r*(T-t)))/a_r;
            else
                beta(1) = (T-t);
            end
            
        end
        
        function c_nN = C_nN(ck1F,tn,tN,tau,pnN,ynN)
            p = @ck1F.DF;
            c_nN=[0.0];
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = ck1F.Beta(tn,tp);
                df = p(tp);
                c_nN(1) = c_nN(1) +tau * beta(1) * df;
            end
            
            c_nN(1) = c_nN(1)* ynN;
            
            c_nN(1) = c_nN(1) + beta(1)*df;
            
            c_nN(1) = c_nN(1)/pnN;
            
        end
        
        function c_nN = C_nNDate(ck1F,valueDate,tnDate,tNDate,tau,pnN,ynN)
            tn = tnDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            
            addMonthUnit = round(12*tau);
            cnt = round((tN-tn)/tau);
            
            p = @ck1F.DF;
            c_nN=[0.0];
            
            prevTime = tn;
            deltaT = 0.0;
            
            for i=1:cnt
                tpDate = tnDate.AddDate(i*addMonthUnit,'month');
                tp = tpDate.DateDiff(valueDate)/365.0;
                deltaT = tp - prevTime;
                beta = ck1F.BetaDate(valueDate,tnDate,tpDate);
                df = p(tp);
%                 c_nN(1) = c_nN(1) +tau * beta(1) * df;
                c_nN(1) = c_nN(1) +deltaT * beta(1) * df;
                prevTime = tp;
            end
            
            c_nN(1) = c_nN(1)* ynN;
            
            c_nN(1) = c_nN(1) + beta(1)*df;
            
            c_nN(1) = c_nN(1)/pnN;
            
        end
        
         function g_nN = G_nN(ck1F,tn,tN,tau,pnN)
            p = @ck1F.DF;
            g_nN=[0.0];
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = ck1F.Beta(tn,tp);
                df = p(tp);
                g_nN(1) = g_nN(1) +tau * beta(1) * df;
            end
            
            g_nN(1) = g_nN(1)/pnN;
            
         end
        
        function g_nN = G_nNDate(ck1F,valueDate,tnDate,tNDate,tau,pnN)
            tn = tnDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            addMonthUnit = round(12*tau);
            cnt = round((tN-tn)/tau);
            
            p = @ck1F.DF;
            g_nN=[0.0];
            
            prevTime = tn;
            deltaT = 0.0;
            
            for i=1:cnt
                tpDate = tnDate.AddDate(i*addMonthUnit,'month');
                tp = tpDate.DateDiff(valueDate)/365.0;
                deltaT = tp - prevTime;
                beta = ck1F.BetaDate(valueDate,tnDate,tpDate);
                df = p(tp);
                g_nN(1) = g_nN(1) +tau * beta(1) * df;
                g_nN(1) = g_nN(1) + deltaT * beta(1) * df;
                prevTime = tp;
                
            end
            
            g_nN(1) = g_nN(1)/pnN;
            
        end 
        
        function h_nTerm = H_nTerm(ck1F,tn,tTerm)
           h_nTerm =[];
           beta = ck1F.Beta(tn,tTerm);
           h_nTerm(1) = -1.0*beta(1);
           
        end
        
        function h_nTerm = H_nTermDate(ck1F,valueDate,tnDate,tTermDate)
           h_nTerm =[];
           beta = ck1F.BetaDate(valueDate,tnDate,tTermDate);
           h_nTerm(1) = -1.0*beta(1);
           
        end
        
        function out = inductMCForwardNew(ck1F,fromTime,toTime,lastX,dZ)
        
            stateLocalVariance = ck1F.stateLocalVariance(fromTime,toTime);
            stateLocalDrift = ck1F.stateLocalDrift(fromTime,toTime);
            L = chol(stateLocalVariance,'lower');
            nextX = stateLocalDrift * lastX + L*dZ;
            out.nextX = nextX;
            

        end
        
        function out = CDDigitalArgDate(ck1F,valueDate,toDate,tpDate,tenor1,tau)
           % probability that CMS rate > strike
            %CMS 1
           toTime =  toDate.DateDiff(valueDate);
           to      = toTime/365.0;
           if tenor1 < 1 && tenor1 >= 1.0/12.0
               tenorMonth = round(12*tenor1);
                tN1Date = toDate.AddDate(tenorMonth,'month');
           elseif tenor1 >= 1
               tN1Date = toDate.AddDate(tenor1,'year');
           else
               disp('unimplemented')
           end
           
           tN1Time = tN1Date.DateDiff(valueDate);
           tN1   =  tN1Time/365.0;
           
           deltaT = (tN1Time - toTime)/365.0;
           
           df_Tn1 = ck1F.DF(to);
           df_TN1 = ck1F.DF(tN1);
           
           c_nN1 = ck1F.BetaDate(valueDate,toDate,tN1Date);
           c_nTerm = ck1F.BetaDate(valueDate,toDate,tpDate);
           
           varCovar = ck1F.LocalVariance(0,to);
           
           driftTerm=0.0;
           for i=1:1
               for j=1:1
                    driftTerm = driftTerm + c_nN1(i) * c_nTerm(j) * varCovar(i,j);
               end
           end
           
           variance = 0.0;
           for i=1:1
               for j=1:1
                    variance = variance + c_nN1(i) * c_nN1(j) * varCovar(i,j);
               end
           end
           
%            out.mu = 0.5*variance  -1.0*log(df_TN1/df_Tn1) - driftTerm;
           out.mu = -0.5*variance  + 1.0*log(df_TN1/df_Tn1) + driftTerm;
           
           out.sigma = sqrt(variance);
           out.c_nN = c_nN1;
           out.varCovar = varCovar;
           out.deltaT = deltaT;
        end
        
        function out = CMSDigitalArg(ck1F,to,tp,tenor1,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           sizeN = 1;
           p_nN1 = ck1F.PV01(to,to+tenor1,tau);
           df_Tn1 = ck1F.DF(to);
           df_TN1 = ck1F.DF(to + tenor1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           c_nN1 = ck1F.C_nN(to,to + tenor1,tau,p_nN1,y_nN1);
           g_nN1 = ck1F.G_nN(to,to + tenor1,tau,p_nN1);
           h_nTerm1 = ck1F.H_nTerm(to,tp);
           
           varCovar = ck1F.LocalVariance(0,to);
           
           driftTerm1=0.0;
           for i=1:sizeN
               for j=1:sizeN
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance1 = 0.0;
           for i=1:sizeN
               for j=1:sizeN
                    spreadVariance1 = spreadVariance1 + c_nN1(i) * c_nN1(j) * varCovar(i,j);
               end
           end
           
           out.fwdSpread = y_nN1;
           out.mu = y_nN1 + driftTerm1;
           out.sigma = sqrt(spreadVariance1);
           out.c_nN1 = c_nN1;
           
        end
        
        function out = CMSDigitalArgDate(ck1F,valueDate,toDate,tpDate,tenor1,tau)
           % probability that CMS rate > strike
            %CMS 1
           to      = toDate.DateDiff(valueDate)/365.0;
           if tenor1 < 1 && tenor1 >= 1.0/12.0
               tenorMonth = round(12*tenor1);
                tN1Date = toDate.AddDate(tenorMonth,'month');
           elseif tenor1 >= 1
               tN1Date = toDate.AddDate(tenor1,'year');
           else
               disp('unimplemented')
           end
           
            % if tenor is 0.25(3M) then use tenor(0.25) instead
           % model(zeroCurve)'s freq
           tauIn = tau;
           if abs(tenor1- 0.25) < 1e-8
                tauIn = tenor1;
           end
           
           tN1   =  tN1Date.DateDiff(valueDate)/365.0;
           
           p_nN1 = ck1F.PV01Date(valueDate,toDate,tN1Date,tauIn);
           
           df_Tn1 = ck1F.DF(to);
           df_TN1 = ck1F.DF(tN1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           
           c_nN1 = ck1F.C_nNDate(valueDate,toDate,tN1Date,tauIn,p_nN1,y_nN1);
           g_nN1 = ck1F.G_nNDate(valueDate,toDate,tN1Date,tauIn,p_nN1);
           h_nTerm1 = ck1F.H_nTermDate(valueDate,toDate,tpDate);
           
           varCovar = ck1F.LocalVariance(0,to);
           
           driftTerm=0.0;
           for i=1:1
               for j=1:1
                    driftTerm = driftTerm + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
               end
           end
           
           variance = 0.0;
           for i=1:1
               for j=1:1
                    variance = variance + c_nN1(i) * c_nN1(j) * varCovar(i,j);
               end
           end
           
           out.fwdCMS = y_nN1;
           out.mu = y_nN1 + driftTerm;
           out.sigma = sqrt(variance);
           out.c_nN = c_nN1;
           out.varCovar = varCovar;
        end
        
        function out = CMSDigital(ck1F,to,tp,tenor1,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsDigitalArg = ck1F.CMSDigitalArg(to,tp,tenor1,tau);
           mu = cmsDigitalArg.mu;
           sigma = cmsDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSDigitalDate(ck1F,valueDate,toDate,tpDate,tenor1,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsDigitalArg = ck1F.CMSDigitalArgDate(valueDate,toDate,tpDate,tenor1,tau);
           mu = cmsDigitalArg.mu;
           sigma = cmsDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSDigitalRange(ck1F,to,tp,tenor1,tau,lower,upper)
            lower_value = ck1F.CMSDigital(to,tp,tenor1,tau,lower);
            upper_value = ck1F.CMSDigital(to,tp,tenor1,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CMSDigitalRangeDate(ck1F,valueDate,toDate,tpDate,tenor1,tau,lower,upper)
            lower_value = ck1F.CMSDigitalDate(valueDate,toDate,tpDate,tenor1,tau,lower);
            upper_value = ck1F.CMSDigitalDate(valueDate,toDate,tpDate,tenor1,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CDDigitalDate(ck1F,valueDate,toDate,tpDate,tenor,tau,strike)
           
           % probability that CD > strikeCD 
           % probability that CMS tenor1 - CMS tenor 2 > strike
           
           % CD
           cdDigitalArg = ck1F.CDDigitalArgDate(valueDate,toDate,tpDate,tenor,tau);
           mu = cdDigitalArg.mu;
           sigma = cdDigitalArg.sigma;
           c_nN = cdDigitalArg.c_nN;
           varCovar = cdDigitalArg.varCovar;
           deltaT = cdDigitalArg.deltaT;
           
%            %CMS Spread
%            cmsSpreadDigitalArg = lgm2F.CMSSpreadDigitalArgDate(valueDate,toDate,tpDate,tenor1,tenor2,tau);
%            
%            c_nN1 = cmsSpreadDigitalArg.c_nN1;
%            c_nN2 = cmsSpreadDigitalArg.c_nN2;
%            
%            muS = cmsSpreadDigitalArg.mu;
%            sigmaS = cmsSpreadDigitalArg.sigma;
%            
           % covariance between CMS Rate & CMS Spread
%            covar = 0.0;
%            for i=1:2
%                for j=1:2
%                     covar = covar + c_nN(i) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
%                end
%            end
           
%            corr = covar/(sigma*sigmaS);
           
           z1 = (mu + log(1.0 + deltaT*strike))/sigma;
%            z2 = (muS - strikeS)/sigmaS;
           if ~isreal(z1) || ~isfinite(z1)
               out = 0.0;
           else
               out = H_ncdf(z1);
           end
             
%            mbar = [0 0];
%            corrBar = [1.0 corr;corr 1.0];
%            xbar = [z1 z2];
%            out = mvncdf(xbar, mbar, corrBar);
           
        end
        
        function out = CDDigitalRangeDate(ck1F,valueDate,toDate,tpDate,tenor,tau,lower,upper)
%             lower_value = ck1F.CDDigitalDate(valueDate,toDate,tpDate,tenor,tau,lower);
%             upper_value = ck1F.CDDigitalDate(valueDate,toDate,tpDate,tenor,tau,upper);
%             out = lower_value - upper_value;
%             
            upper_value = ck1F.CDDigitalDate(valueDate,toDate,tpDate,tenor,tau,upper);
            lower_value = ck1F.CDDigitalDate(valueDate,toDate,tpDate,tenor,tau,lower);
            out = upper_value - lower_value;
            
        end
        
        function out = CMSSpreadDigitalArg(ck1F,to,tp,tenor1,tenor2,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           p_nN1 = ck1F.PV01(to,to+tenor1,tau);
           df_Tn1 = ck1F.DF(to);
           df_TN1 = ck1F.DF(to + tenor1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           c_nN1 = ck1F.C_nN(to,to + tenor1,tau,p_nN1,y_nN1);
           g_nN1 = ck1F.G_nN(to,to + tenor1,tau,p_nN1);
           h_nTerm1 = ck1F.H_nTerm(to,tp);
           
           %CMS2
           p_nN2 = ck1F.PV01(to,to+tenor2,tau);
           df_Tn2 = ck1F.DF(to);
           df_TN2 = ck1F.DF(to + tenor2);
           y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
           c_nN2 = ck1F.C_nN(to,to + tenor2,tau,p_nN2,y_nN2);
           g_nN2 = ck1F.G_nN(to,to + tenor2,tau,p_nN2);
           h_nTerm2 = ck1F.H_nTerm(to,tp);
           
           varCovar = ck1F.LocalVariance(0,to);
           
           driftTerm1=0.0;
           driftTerm2=0.0;
           for i=1:1
               for j=1:1
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
                    driftTerm2 = driftTerm2 + c_nN2(i) * (h_nTerm2(j) + g_nN2(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           variance1 = 0.0;
           variance2 = 0.0;
           coVariance = 0.0;
           for i=1:1
               for j=1:1
                    spreadVariance = spreadVariance + (c_nN1(i)-c_nN2(i)) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
                    variance1 = variance1 + (c_nN1(i)) * (c_nN1(j)) * varCovar(i,j);
                    variance2 = variance2 + (c_nN2(i)) * (c_nN2(j)) * varCovar(i,j);
                    coVariance = coVariance + (c_nN1(i)) * (c_nN2(j)) * varCovar(i,j);
               end
           end
           
           out.fwdSpread = y_nN1 - y_nN2;
           out.mu = (y_nN1 + driftTerm1) - (y_nN2 + driftTerm2);
           out.pmu = driftTerm1 - driftTerm2;
           out.sigma = sqrt(spreadVariance);
           out.c_nN1 = c_nN1;
           out.c_nN2 = c_nN2;
           out.varCovar = varCovar;
           out.mu1 = y_nN1 + driftTerm1;
           out.mu2 = y_nN2 + driftTerm2;
           out.pmu1 = driftTerm1;
           out.pmu2 = driftTerm2;
           
           out.y1 = y_nN1;
           out.y2 = y_nN2;
           
           out.corr = coVariance/sqrt(variance1*variance2);
           out.sig1 = sqrt(variance1);
           out.sig2 = sqrt(variance2);
           
        end
        
        function out = CMSSpreadDigitalArgDate(ck1F,valueDate,toDate,tpDate,tenor1,tenor2,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           to      = toDate.DateDiff(valueDate)/365.0;
           tN1Date = toDate.AddDate(tenor1,'year');
           tN1   =  tN1Date.DateDiff(valueDate)/365.0;
           
           p_nN1 = ck1F.PV01Date(valueDate,toDate,tN1Date,tau);
           
           df_Tn1 = ck1F.DF(to);
           df_TN1 = ck1F.DF(tN1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           
           c_nN1 = ck1F.C_nNDate(valueDate,toDate,tN1Date,tau,p_nN1,y_nN1);
           g_nN1 = ck1F.G_nNDate(valueDate,toDate,tN1Date,tau,p_nN1);
           h_nTerm1 = ck1F.H_nTermDate(valueDate,toDate,tpDate);
           
           %CMS2
           tN2Date = toDate.AddDate(tenor2,'year');
           tN2   =  tN2Date.DateDiff(valueDate)/365.0;
           
           p_nN2 = ck1F.PV01Date(valueDate,toDate,tN2Date,tau);
           
           df_Tn2 = ck1F.DF(to);
           df_TN2 = ck1F.DF(tN2);
           y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
           
           c_nN2 = ck1F.C_nNDate(valueDate,toDate,tN2Date,tau,p_nN2,y_nN2);
           g_nN2 = ck1F.G_nNDate(valueDate,toDate,tN2Date,tau,p_nN2);
           h_nTerm2 = ck1F.H_nTermDate(valueDate,toDate,tpDate);
           
           varCovar = ck1F.LocalVariance(0,to);
           
           driftTerm1=0.0;
           driftTerm2=0.0;
           for i=1:1
               for j=1:1
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
                    driftTerm2 = driftTerm2 + c_nN2(i) * (h_nTerm2(j) + g_nN2(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           variance1 = 0.0;
           variance2 = 0.0;
           coVariance = 0.0;
           for i=1:1
               for j=1:1
                    spreadVariance = spreadVariance + (c_nN1(i)-c_nN2(i)) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
                    variance1 = variance1 + (c_nN1(i)) * (c_nN1(j)) * varCovar(i,j);
                    variance2 = variance2 + (c_nN2(i)) * (c_nN2(j)) * varCovar(i,j);
                    coVariance = coVariance + (c_nN1(i)) * (c_nN2(j)) * varCovar(i,j);
               end
           end
           
           out.fwdSpread = y_nN1 - y_nN2;
           out.mu = (y_nN1 + driftTerm1) - (y_nN2 + driftTerm2);
           out.pmu = driftTerm1 - driftTerm2;
           out.sigma = sqrt(spreadVariance);
           out.c_nN1 = c_nN1;
           out.c_nN2 = c_nN2;
           out.varCovar = varCovar;
           out.mu1 = y_nN1 + driftTerm1;
           out.mu2 = y_nN2 + driftTerm2;
           
           out.pmu1 = driftTerm1;
           out.pmu2 = driftTerm2;
           
           out.corr = coVariance/sqrt(variance1*variance2);
           out.sig1 = sqrt(variance1);
           out.sig2 = sqrt(variance2);
           
        end
        
        function out = CMSSpreadDigital(ck1F,to,tp,tenor1,tenor2,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsSpreadDigitalArg = ck1F.CMSSpreadDigitalArg(to,tp,tenor1,tenor2,tau);
           mu = cmsSpreadDigitalArg.mu;
           sigma = cmsSpreadDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSSpreadDigitalDate(ck1F,valueDate,toDate,tpDate,tenor1,tenor2,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsSpreadDigitalArg = ck1F.CMSSpreadDigitalArgDate(valueDate,toDate,tpDate,tenor1,tenor2,tau);
           mu = cmsSpreadDigitalArg.mu;
           sigma = cmsSpreadDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSSpreadDigitalRange(ck1F,to,tp,tenor1,tenor2,tau,lower,upper)
            lower_value = ck1F.CMSSpreadDigital(to,tp,tenor1,tenor2,tau,lower);
            upper_value = ck1F.CMSSpreadDigital(to,tp,tenor1,tenor2,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CMSSpreadDigitalRangeDate(ck1F,valueDate,toDate,tpDate,tenor1,tenor2,tau,lower,upper)
            lower_value = ck1F.CMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,lower);
            upper_value = ck1F.CMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CMSCMSSpreadDualDigital(ck1F,to,tp,tenor,tenor1,tenor2,tau,strike,strikeS)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsDigitalArg = ck1F.CMSDigitalArg(to,tp,tenor,tau);
           mu = cmsDigitalArg.mu;
           sigma = cmsDigitalArg.sigma;
           c_nN = cmsDigitalArg.c_nN;
           varCovar = cmsDigitalArg.varCovar;
           
           cmsSpreadDigitalArg = ck1F.CMSSpreadDigitalArg(to,tp,tenor1,tenor2,tau);
           muS = cmsSpreadDigitalArg.mu;
           sigmaS = cmsSpreadDigitalArg.sigma;
           c_nN1 = cmsSpreadDigitalArg.c_nN1;
           c_nN2 = cmsSpreadDigitalArg.c_nN2;
           varCovarS = cmsSpreadDigitalArg.varCovarS;
           
           % covariance between CMS Rate & CMS Spread
           covar = 0.0;
           for i=1:1
               for j=1:1
                    covar = covar + c_nN(i) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
               end
           end
           
           corr = covar/(sigma*sigmaS);
           
           z1 = (mu - strike)/sigma;
           z2 = (muS - strikeS)/sigmaS;
           
           mbar = [0 0];
           corrBar = [1.0 corr;corr 1.0];
           xbar = [z1 z2];
           out = mvncdf(xbar, mbar, corrBar);
           
        end
        
         function out = CMSCMSSpreadDualDigitalRange(ck1F,to,tp,tenor,tenor1,tenor2,tau,lower,upper,lowerS,upperS)
            value_ll = ck1F.CMSCMSSpreadDualDigital(to,tp,tenor,tenor1,tenor2,tau,lower,lowerS);
            value_lu = ck1F.CMSCMSSpreadDualDigital(to,tp,tenor,tenor1,tenor2,tau,lower,upperS);
            value_ul = ck1F.CMSCMSSpreadDualDigital(to,tp,tenor,tenor1,tenor2,tau,upper,lowerS);
            value_uu = ck1F.CMSCMSSpreadDualDigital(to,tp,tenor,tenor1,tenor2,tau,upper,upperS);
            out = value_ll - value_lu - value_ul + value_uu;
        end
        
        function atmSwaption = ATMSwaption(ck1F,tx,tnr)
            tau = 0.25;
            pv01 = ck1F.PV01(tx,tx+tnr,tau);
            fsr  = ck1F.Fsr(tx,tx+tnr,tau);
            c_nN = ck1F.C_nN(tx,tx+tnr,tau,pv01,fsr);
            variances = ck1F.LocalVariance(0,tx);
            totalVar = 0.0;
            for i=1:1
                for j=1:1
                    totalVar = totalVar + c_nN(i)*c_nN(j)*variances(i,j);
                end
            end
            atmSwaption =  pv01 * sqrt(totalVar)/sqrt(2.0*pi);
        end
        
        function spreadInfo = SpreadInfo(ck1F,tx,tnr1,tnr2)
            tau = 0.25;
            cmsSpreadDigitalArg =  ck1F.CMSSpreadDigitalArg(tx,tx+1.0,tnr1,tnr2,tau);
            spreadInfo = cmsSpreadDigitalArg;
            spreadInfo.normSigma = cmsSpreadDigitalArg.sigma/sqrt(tx);
        end
        
        function impCorrel = ImpCorrel(ck1F,tx,tnr1,tnr2)
            
            tau = 0.25;
            pv01_1 = ck1F.PV01(tx,tx+tnr1,tau);
            fsr_1  = ck1F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = ck1F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = ck1F.PV01(tx,tx+tnr2,tau);
            fsr_2  = ck1F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = ck1F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            variances = ck1F.LocalVariance(0,tx);
            var1 = 0;
            var2 = 0;
            coVar = 0;
            for j=1:1
                for k=1:1
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                    var2 = var2 + c_nN_2(j)*c_nN_2(k)*variances(j,k);
                    coVar = coVar + c_nN_1(j)*c_nN_2(k)*variances(j,k);
                end
            end
            impCorrel = coVar/sqrt(var1)/sqrt(var2);
        end
        
        function impCovar = ImpCovar(ck1F,tx,tnr1,tnr2)
            tau = 0.25;
            pv01_1 = ck1F.PV01(tx,tx+tnr1,tau);
            fsr_1  = ck1F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = ck1F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = ck1F.PV01(tx,tx+tnr2,tau);
            fsr_2  = ck1F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = ck1F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            variances = ck1F.LocalVariance(0,tx);
            var1 = 0;
            var2 = 0;
            coVar = 0;
            for j=1:1
                for k=1:1
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                    var2 = var2 + c_nN_2(j)*c_nN_2(k)*variances(j,k);
                    coVar = coVar + c_nN_1(j)*c_nN_2(k)*variances(j,k);
                end
            end
            impCovar = coVar;
        end
        
        
        function instFwdCorr = InsFwdCorr(ck1F,tx,tnr1,tnr2)
            tau = 0.25;
            tenor = ck1F.vol_r.tenor;
            tIdx = min(find(tx<=tenor));
            meanR_r = ck1F.meanR_r;
            vol_r = ck1F.vol_r;
            corr_rs = 1.0;
            covar = 0.0;
            var1 = 0;
            var2 = 0;
            
            var1 = exp(-2*meanR_r*tnr1)*vol_r.quote(tIdx)*vol_r.quote(tIdx);
            var2 = exp(-2*meanR_r*tnr2)*vol_r.quote(tIdx)*vol_r.quote(tIdx);
            covar = exp(-meanR_r*(tnr1+tnr2))*vol_r.quote(tIdx)*vol_r.quote(tIdx);
            
            instFwdCorr = covar/sqrt(var1*var2);
        end
        
        
        
        function instFwdCorrTest = InsFwdCorrTest(ck1F,tx,tnr1,tnr2)
            tau = 0.25;
            sigma_r = ck1F.sigma_Test(tx,ck1F.vol_r);
            a_r = ck1F.meanR_r;
            
            covar = 0.0;
            var1 = 0;
            var2 = 0;
            var1 =  exp(-2.0*a_r*tnr1)*sigma_r*sigma_r;
            var2 =  exp(-2.0*a_r*tnr2)*sigma_r*sigma_r;
            covar = exp(-1.0*a_r*(tnr1+ tnr2))*sigma_r*sigma_r;
            instFwdCorrTest = covar/sqrt(var1*var2);
        end
        
        
        function instFwdSwapCorrTest = InsSwapFwdCorrTest(ck1F,tx,tnr1,tnr2)
            tau = 0.25;
            C_tT1 = zeros(1,1);
            pv01_1 = ck1F.PV01(tx,tx+tnr1,tau);
            fsr_1  = ck1F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = ck1F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            C_tT1(1) = c_nN_1(1);
            
            C_tT2 = zeros(1,1);
            pv01_2 = ck1F.PV01(tx,tx+tnr2,tau);
            fsr_2  = ck1F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = ck1F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            C_tT2(1) = c_nN_2(1);
            
            dAlpha_t = zeros(1,1);
            dAlpha_t(1) = ck1F.sigma_Test(tx,ck1F.vol_r);
            corr = zeros(1,1);
            corr(1,1) = 1.0;
            
            covar = 0.0;
            var1 = 0.0;
            var2 = 0.0;
            for i=1:1
                for j=1:1
                    var1 = var1 + C_tT1(i)*C_tT1(j)*dAlpha_t(i)*dAlpha_t(j)*corr(i,j);
                    var2 = var2 + C_tT2(i)*C_tT2(j)*dAlpha_t(i)*dAlpha_t(j)*corr(i,j);
                    covar = covar + C_tT1(i)*C_tT2(j)*dAlpha_t(i)*dAlpha_t(j)*corr(i,j);
                end
            end
            instFwdSwapCorrTest = covar/sqrt(var1*var2);
        end
        
        function impVariance = ImpVariance(ck1F,tx,tnr1)
            tau = 0.25;
            pv01_1 = ck1F.PV01(tx,tx+tnr1,tau);
            fsr_1  = ck1F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = ck1F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            variances = ck1F.LocalVariance(0,tx);
            var1 = 0;
            for j=1:1
                for k=1:1
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                end
            end
            impVariance = var1;
        end
        
        function out = Rate_IndexGeneric(ck1F,rateType,valueDate,observeTime,numMatTime,tenor,modelStatesT)
            if strcmp(rateType,'CMS')
                out = CMS_PSA_SimpleDateGeneric(ck1F,valueDate,observeTime,numMatTime, tenor,modelStatesT);
            elseif strcmp(rateType,'LIBOR')
                out = Libor_SimpleDateGeneric(ck1F,valueDate,observeTime,numMatTime, tenor,modelStatesT);
            else
                error('unsupported rateType!')
            end
        end
        
        function out = Libor_SimpleDateGeneric(ck1F,valueDate,observeTime,numMatTime,tenor,modelStatesT)
            if observeTime < 0
                allFixings = ck1F.modelParams('allFixings');
                fixingDate = AddDate(valueDate,observeTime,'day');
                tenorMonth = round(tenor*12);
                fixingTicker = ['LIBOR' num2str(tenorMonth,'%d') 'M'];
                irIndexFixings = allFixings(fixingTicker);
                try
                    fixing = irIndexFixings(StrDate(fixingDate,'str'));
                    stateSize = size(modelStatesT,2);
                    out = ones(1,stateSize)*fixing;
                    
                catch exception
                    throw(exception)
                end
                
            else
                out = Libor_SimpleDate(ck1F,valueDate,observeTime,observeTime,numMatTime, tenor,modelStatesT);
            end
        end
        
        function out =  CMS_PSA_SimpleDateGeneric(ck1F,valueDate,observeTime,numMatTime, tenor,modelStatesT)
            if observeTime < 0
                allFixings = ck1F.modelParams('allFixings');
                fixingDate = AddDate(valueDate,observeTime,'day');
                fixingTicker = ['CMS' num2str(tenor,'%d') 'Y'];
                irIndexFixings = allFixings(fixingTicker);
                try
                    fixing = irIndexFixings(StrDate(fixingDate,'str'));
                    stateSize = size(modelStatesT,2);
                    out = ones(1,stateSize)*fixing;
                    
                catch exception
                    throw(exception)
                end
                
            else
                out = CMS_PSA_SimpleDate(ck1F,valueDate,observeTime,numMatTime, tenor,modelStatesT);
            end
        
        end
        
        function out = AssignExercise(ck1F,currentTimeStep,PricerColumn,exerciseValueIn,exerciseConditionIn,...
                                      minMaxFlagIn,decisionVar1In,nominal,numeraire,cachedModelStatesV)
                                  
           numMatTime = numeraire.numMatTime;
           
%            df_e = eqCOMDupireSpotGF.discountCurve.DF(currentTimeStep/365.0);
%            df_e = model.discountCurve.DF(currentTimeStep/365.0);
           onePayoff = ones(1,size(cachedModelStatesV,2));
           onePayoffD = ck1F.discountPayoffRisky(currentTimeStep,numMatTime ...
                            ,cachedModelStatesV,onePayoff);
                        
           originalValue = PricerColumn.payoff./onePayoffD;
           exerciseValue = exerciseValueIn;
           newValue = zeros(size(originalValue));
           exerciseITMCondition = exerciseConditionIn;
           

           minMaxFlag = minMaxFlagIn;
           decisionVar1 = decisionVar1In;
           %regression using polynomial of decisionVar1
           %upto nOrder 
%            nOrder = nOrderIn;

%            newValue = LSRegression1(originalValue,exerciseValue,exerciseITMCondition,minMaxFlag,decisionVar1,nOrder,nominal);
           
%            newValue = LSRegressionGeneric(originalValue,exerciseValue,exerciseITMCondition,minMaxFlag,decisionVar1,nominal);
           
           newValue = LSRegressionGeneric2(originalValue,exerciseValue,exerciseITMCondition,minMaxFlag,decisionVar1,nominal);
           
%            newValue = LSRegressionGeneric2_original(originalValue,exerciseValue,exerciseITMCondition,minMaxFlag,decisionVar1,nominal);
 
           %discount newValue and assign to the
           %pricerColumn payoff
           PricerColumn.payoff = newValue .* onePayoffD;
           out = PricerColumn.payoff; 
            
        end
        
        function out = AssignExercise_LSMC_3(ck1F,currentTimeStep,PricerColumn,exerciseValueIn,exerciseConditionIn,...
                                      minMaxFlagIn,decisionVar1In,nominal,numeraire,cachedModelStatesV)
                                  
           numMatTime = numeraire.numMatTime;
           
%            df_e = eqCOMDupireSpotGF.discountCurve.DF(currentTimeStep/365.0);
%            df_e = model.discountCurve.DF(currentTimeStep/365.0);
           onePayoff = ones(1,size(cachedModelStatesV,2));
           onePayoffD = ck1F.discountPayoffRisky(currentTimeStep,numMatTime ...
                            ,cachedModelStatesV,onePayoff);
                        
           originalValue = PricerColumn.payoff./onePayoffD;
           exerciseValue = exerciseValueIn;
           newValue = zeros(size(originalValue));
           exerciseITMCondition = exerciseConditionIn;
           

           minMaxFlag = minMaxFlagIn;
           decisionVar1 = decisionVar1In;
           %regression using polynomial of decisionVar1
           %upto nOrder 
%            nOrder = nOrderIn;

%            newValue = LSRegression1(originalValue,exerciseValue,exerciseITMCondition,minMaxFlag,decisionVar1,nOrder,nominal);
           
%            newValue = LSRegressionGeneric(originalValue,exerciseValue,exerciseITMCondition,minMaxFlag,decisionVar1,nominal);
           
%            newValue = LSRegressionGeneric2(originalValue,exerciseValue,exerciseITMCondition,minMaxFlag,decisionVar1,nominal);
           
%            newValue = LSRegressionGeneric2_original(originalValue,exerciseValue,exerciseITMCondition,minMaxFlag,decisionVar1,nominal);
            
           newValue = LSRegressionGeneric3_original(originalValue,exerciseValue,exerciseITMCondition,minMaxFlag,decisionVar1,nominal);
 
           %discount newValue and assign to the
           %pricerColumn payoff
           PricerColumn.payoff = newValue .* onePayoffD;
           out = PricerColumn.payoff; 
            
        end
        
        function out = computeInverseFloaterSwapGeneric2(ck1F,valueDate,steepenerParams,numMethod)
%           
            % model schedule generation
            dealStartDate = steepenerParams.startDate;
            dealStartEventTime = DateDiff(dealStartDate,valueDate);
            expiryDate = steepenerParams.endDate;
            dealEndEventTime = DateDiff(expiryDate,valueDate);
            %% structuredCouponSchedule start
            
            structuredCouponScheduleInt = steepenerParams.params.structuredCouponInfo.structuredCouponScheduleInt;
            
            % structuredCouponSchedule is a schedule of Structured Coupon
            % events
            
            % main backbone schedule
            structuredCouponScheduleSize = size(structuredCouponScheduleInt,1);
            structuredCouponSchedule= struct;
            structuredCouponSchedule.scheduleSize = structuredCouponScheduleSize;
            structuredCouponSchedule.eventType = steepenerParams.params.structuredCouponInfo.eventType;
            % coupon payment event happens at endDate
            % eventTime is endDate
            % payment date can be different from endDate
            
            % startTime,endTimeP should be included in timeStep Generation
            % with eventTime
            
            for i=1:structuredCouponSchedule.scheduleSize
                couponResetDate = H_Date(structuredCouponScheduleInt(i,1));
                couponStartDate = H_Date(structuredCouponScheduleInt(i,2));
                couponEndDate = H_Date(structuredCouponScheduleInt(i,3));
                couponPayDate = H_Date(structuredCouponScheduleInt(i,4));
                
                structuredCouponSchedule.dayCount(i) = couponEndDate.DateDiff(couponStartDate);
                structuredCouponSchedule.dayCountFraction(i) = structuredCouponSchedule.dayCount(i)/365.0; 
                
                couponEndDateP = couponEndDate.AddDate( -1 ,'day');
                
                structuredCouponSchedule.resetTime(i) = DateDiff(couponResetDate,valueDate);
                structuredCouponSchedule.startTime(i) = DateDiff(couponStartDate,valueDate);
                structuredCouponSchedule.endTimeP(i)  = DateDiff(couponEndDateP,valueDate);
                structuredCouponSchedule.endTime(i) = DateDiff(couponEndDate,valueDate);
                
                if strcmp(structuredCouponSchedule.eventType,'RESET')
                    structuredCouponSchedule.eventTime(i) = DateDiff(couponResetDate,valueDate);
                elseif strcmp(structuredCouponSchedule.eventType,'START')
                    structuredCouponSchedule.eventTime(i) = DateDiff(couponStartDate,valueDate);
                elseif strcmp(structuredCouponSchedule.eventType,'END')
                    structuredCouponSchedule.eventTime(i) = DateDiff(couponEndDate,valueDate);
                else % default is END : event happens at EndDate for StructuredCoupon
                    structuredCouponSchedule.eventTime(i) = DateDiff(couponEndDate,valueDate);
                end
                structuredCouponSchedule.payTime(i) = DateDiff(couponPayDate,valueDate);
                
            end
            
            
            tenorInverseFloater     = steepenerParams.params.structuredCouponInfo.tenorInverseFloater;
            
           %% rateType for structuredCoupon : default CMS
            if isfield(steepenerParams.params.structuredCouponInfo, 'rateType_tenorInverseFloater')
                rateType_tenorInverseFloater = steepenerParams.params.structuredCouponInfo.rateType_tenorInverseFloater;
            else
                rateType_tenorInverseFloater = 'CMS';
            end
            
            
            localFloor = steepenerParams.params.structuredCouponInfo.localFloor;
            localCap   = steepenerParams.params.structuredCouponInfo.localCap;
            multiplier = steepenerParams.params.structuredCouponInfo.multiplier;
            spread     = steepenerParams.params.structuredCouponInfo.spread;
            coupon     = steepenerParams.params.structuredCouponInfo.coupon;
           %% structuredCoupon schedule end
           
           %% call exercise event happens at the callExerciseDate
           %% callable schedule start
            callableScheduleInt = steepenerParams.params.callableInfo.callableScheduleInt;
            callableScheduleSize = size(callableScheduleInt,1);
            callableSchedule= struct;
            for i=1:callableScheduleSize
                callExerciseDate = H_Date(callableScheduleInt(i,1));
                callPayDate = H_Date(callableScheduleInt(i,2));
                callableSchedule.eventTime(i) = DateDiff(callExerciseDate,valueDate);
                callableSchedule.payTime(i) = DateDiff(callPayDate,valueDate);
                
                 callableSchedule.callPayment(i) = callableScheduleInt(i,3);
            end
            callableSchedule.scheduleSize = callableScheduleSize;
           
            isExercised = steepenerParams.params.callableInfo.isExercised;
           %% callable schedule end
           
          %% swapFloatingCouponSchedule start
            
            swapFloatingCouponScheduleInt = steepenerParams.params.swapFloatingCouponInfo.swapFloatingCouponScheduleInt;
            
            swapFloatingCouponSchedule= struct;
            swapFloatingCouponSchedule.scheduleSize = size(swapFloatingCouponScheduleInt,1);
            
            swapFloatingCouponSchedule.eventType = steepenerParams.params.swapFloatingCouponInfo.eventType;
            
            % floating coupon payment event happens at resetDate
            % eventTime is resetDate
            
            % resetTime,startTime,endTimeP should be included in timeStep Generation
            % with eventTime
            
            for i=1:swapFloatingCouponSchedule.scheduleSize
                flolatingCouponResetDate = H_Date(swapFloatingCouponScheduleInt(i,1));
                flolatingCouponStartDate = H_Date(swapFloatingCouponScheduleInt(i,2));
                flolatingCouponEndDate = H_Date(swapFloatingCouponScheduleInt(i,3));
                flolatingCouponPayDate = H_Date(swapFloatingCouponScheduleInt(i,4));
                
                swapFloatingCouponSchedule.dayCount(i) = flolatingCouponEndDate.DateDiff(flolatingCouponStartDate);
                swapFloatingCouponSchedule.dayCountFraction(i) = swapFloatingCouponSchedule.dayCount(i)/365.0; 
                
                flolatingCouponEndDateP = flolatingCouponEndDate.AddDate( -1 ,'day');
                
                swapFloatingCouponSchedule.resetTime(i) = DateDiff(flolatingCouponResetDate,valueDate);
                swapFloatingCouponSchedule.startTime(i) = DateDiff(flolatingCouponStartDate,valueDate);
                swapFloatingCouponSchedule.endTimeP(i)  = DateDiff(flolatingCouponEndDateP,valueDate);
                swapFloatingCouponSchedule.endTime(i) = DateDiff(flolatingCouponEndDate,valueDate);
                
                if strcmp(swapFloatingCouponSchedule.eventType,'RESET')
                    swapFloatingCouponSchedule.eventTime(i) = DateDiff(flolatingCouponResetDate,valueDate);
                elseif strcmp(swapFloatingCouponSchedule.eventType,'START')
                    swapFloatingCouponSchedule.eventTime(i) = DateDiff(flolatingCouponStartDate,valueDate);
                elseif strcmp(swapFloatingCouponSchedule.eventType,'END')
                    swapFloatingCouponSchedule.eventTime(i) = DateDiff(flolatingCouponEndDate,valueDate);
                else % default is RESET : for SwapFloatingCouponSchedule
                    swapFloatingCouponSchedule.eventTime(i) = DateDiff(flolatingCouponResetDate,valueDate);
                end
                
%                 swapFloatingCouponSchedule.eventTime(i) = DateDiff(flolatingCouponResetDate,valueDate);
                
                swapFloatingCouponSchedule.payTime(i) = DateDiff(flolatingCouponPayDate,valueDate);
                
            end
            
            swapFloating_tenor  = steepenerParams.params.swapFloatingCouponInfo.tenorFloat;
            
            %% rateType for swapFloatingCoupon : default LIBOR
            if isfield(steepenerParams.params.swapFloatingCouponInfo, 'rateType_tenorFloat')
                rateType_tenorFloat = steepenerParams.params.swapFloatingCouponInfo.rateType_tenorFloat;
            else
                rateType_tenorFloat = 'LIBOR';
            end
            
            swapFloating_multiplier = steepenerParams.params.swapFloatingCouponInfo.multiplier;
            swapFloating_spread     = steepenerParams.params.swapFloatingCouponInfo.spread;
           
          %% SwapFloatingCouponSchedule end
            
            
          %% pricing time Step Generation start
            
            % eventTimeStep Generate
            mcOneTimeStep = numMethod.mcOneTimeStep;
            
          %% eventTime from dealStartTime and dealEndTime
           eventTimeStep = dealStartEventTime;
           eventTimeStep = union(eventTimeStep,dealEndEventTime);
           
          %% eventTime From structuredCouponSchedule:eventTime(endTime) +
          %% startTime,endTimeP
          %% daily weekly timeStep for couponSchedule
          
          %% InverseFloater
          %% eventTime From structuredCouponSchedule:eventTime
            eventTimeStep = union(eventTimeStep, structuredCouponSchedule.resetTime);
            eventTimeStep = union(eventTimeStep,structuredCouponSchedule.eventTime);
%             eventTimeStep = union(eventTimeStep, structuredCouponSchedule.startTime);
            eventTimeStep = union(eventTimeStep, structuredCouponSchedule.endTime);
%             eventTimeStep = union(eventTimeStep, structuredCouponSchedule.endTimeP);
           %% eventTime From CallSchedule:eventTime
            eventTimeStep = union(eventTimeStep,callableSchedule.eventTime);
            eventTimeStep = sort(eventTimeStep,'ascend');
            
          %% eventTime From SwapFloatingCouponSchedule:eventTime
            eventTimeStep = union(eventTimeStep,swapFloatingCouponSchedule.resetTime);  
            eventTimeStep = union(eventTimeStep,swapFloatingCouponSchedule.eventTime);
            eventTimeStep = sort(eventTimeStep,'ascend');
            
            
            if dealStartEventTime < 0 
                dailyTimeSteps = [0:-1:dealStartEventTime];
                eventTimeStep = union(eventTimeStep,dailyTimeSteps);
            end
                
%           %% Daily Weekly Time Step Generation for AllTheTime ScheduleType, FromTo Schedule start
%             
%             % alive Coupon : event has not happend Yet
%             % from dealStartEventTime to 2nd AliveCoupon EventTime
%             % dailyTimeStep should be added to eventTimeStep
%             % for stable theta calculation
%              
%             aliveCouponIdxMax = max(find(structuredCouponSchedule.eventTime > 0));
%             aliveCouponIdxMin = min(find(structuredCouponSchedule.eventTime > 0));
%             
%             aliveCouponNums = aliveCouponIdxMax - aliveCouponIdxMin +1;
%             numOfAliveCouponDaily = min(aliveCouponNums,2);
%             if numOfAliveCouponDaily > 0
%               %% dailyTimeSteps for near couponEventTime, till dealStartEventTime and valueTime
%                 % for fixing handling and theta issue
%                 dailyTimeSteps = [structuredCouponSchedule.eventTime(aliveCouponIdxMin + numOfAliveCouponDaily - 1):-1:dealStartEventTime];
%                 dailyTimeSteps = union(dailyTimeSteps,0);
%                 eventTimeStep = union(eventTimeStep,dailyTimeSteps);
%               %% weeklyTimeSteps for far couponEventTime
%                 weeklyTimeSteps = [eventTimeStep(end):-1*mcOneTimeStep:structuredCouponSchedule.eventTime(aliveCouponIdxMin + numOfAliveCouponDaily - 1)];
%                 eventTimeStep = union(eventTimeStep,weeklyTimeSteps);
%             else
%               %% if no aliveCouponEventTime( if last StructuredCouponEventTime is passed but not paid yet)
%                 % if dealStartEventTime is passed then we add dailyTimeStep
%                 % from valueTime till dealStartEventTime 
%                 % for fixing handling
%                 
%                 if dealStartEventTime < 0 
%                     dailyTimeSteps = [0:-1:dealStartEventTime];
%                     eventTimeStep = union(eventTimeStep,dailyTimeSteps);
%                 end
%                 
%             end
           
          %% Daily Weekly Time Step Generation for AllTheTime ScheduleType, FromTo Schedule end
            
            eventTimeStep = sort(eventTimeStep,'ascend');
            
%           %% debugging purpose 20201013 start
%             passedTimeStep = eventTimeStep(eventTimeStep<0);
%             load('cmsSteepnerTimeStep20201013.mat','cmsSteepnerTimeStep20201013');
%             aliveTimeStep = sort(cmsSteepnerTimeStep20201013,'ascend');
%             eventTimeStep = [];
%             eventTimeStep = union(passedTimeStep,cmsSteepnerTimeStep20201013);
%             eventTimeStep = sort(eventTimeStep,'ascend');
%           
%           %% debugging purpose 20201013 end
            
          %% aliveTimeStep for mc path generation
            aliveTimeStep = eventTimeStep(eventTimeStep>=0);
            aliveTimeStep = sort(aliveTimeStep,'ascend');
            
            totalAliveTimeStepSize = length(aliveTimeStep);
            
           %% eventTimeStep Generation End
           
           %% numeraire info
            numMatTime = eventTimeStep(end);
            dfNumMat = ck1F.DFRisky(numMatTime/365.0);

            numeraire.numMatTime = numMatTime;
            numeraire.dfNumMat = dfNumMat;
              
            % should we use eventTimeIdx as future only time stes?
            % In this case we better use more generic approach
            % so that all times steps are included 
            
          %% pricing time Step Generation end
           
          %% MC init
            rng('default');
            rng(0);
            
            numOfFactors = ck1F.numOfFactors;
            NMC = numMethod.pathSize; 
            MCTimeStep = totalAliveTimeStepSize -1; % nowDate (0) should be skipped
%             Z= randn(NMC/2,MCTimeStep);
            Z= randn(NMC/2,numOfFactors*MCTimeStep);
            Z=[Z;-Z];
            Z=Z';

            %TimeInversion
            sizeZ = size(Z,1);
            A = zeros(sizeZ,sizeZ);
            for i=1:sizeZ
                A(i,sizeZ + 1 -i) = 1;
            end
            Z = A*Z;

%                 U = Z';
            U = Z;

           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = zeros(numOfFactors,NMC);
           
           nextX = initX;
            
          %% payoff init
           nominal = steepenerParams.params.nominal;
                      
%            payoffInfo.nominal = nominal;
% 
%            payoffInfo.couponSchedule = couponSchedule;
%            payoffInfo.callableSchedule = callableSchedule;
% %            payoffInfo.refixingSchedule = refixingSchedule;
           
           % EODFlag : if true then pricing as EndOfDate(cashflow on
           % valueDate wil be not included(already paid)
           % if false then pricing as Intraday ( cashflow on valueDate will 
           % be included)
           
           %EODFlag_Cashflow: Cashflow EOD Flag
           %EODFlag_Exercise: Exercise(Option) EOD Flag
           EODFlag_Cashflow = steepenerParams.params.EODFlag_Cashflow;
           EODFlag_Exercise = steepenerParams.params.EODFlag_Exercise;
           
           
           modelStatesSize = NMC;
           
          %% Define Pricing Columns
           % pricingColumns : nonCall,Callable  
           % we need steepenerCoupon as state variables for storing 
           % CMS Steepener Coupon
           
           % for the cashflow generated pricing columns as pricerInfo
           % we do use pricerInfo Structure
           
         %% (global) state variable initialize  : these correspond to ColumnVal[t] in generic pricing deal description
           
          % avg steeper coupon
%            steepenerCoupon = zeros(structuredCouponSchedule.scheduleSize,modelStatesSize);
           inverseFloaterCoupon = zeros(structuredCouponSchedule.scheduleSize,modelStatesSize);
           prefixedInverseFloatRate = zeros(structuredCouponSchedule.scheduleSize,modelStatesSize);
           
           swapFloatingCoupon = zeros(swapFloatingCouponSchedule.scheduleSize,modelStatesSize);
           prefixedSwapFloatingRate = zeros(swapFloatingCouponSchedule.scheduleSize,modelStatesSize);
           
           cachedModelStatesCoupon = zeros(structuredCouponSchedule.scheduleSize*numOfFactors,modelStatesSize);
  
          % for regression (2 modelState variables)
           cachedModelStatesCall = zeros(callableSchedule.scheduleSize*numOfFactors,modelStatesSize);
           
          % for swapFloatin 
           cachedModelStatesSwapFloating = zeros(swapFloatingCouponSchedule.scheduleSize*numOfFactors,modelStatesSize);
           
          %% PricingColumnGenericCashflow
          %% a structure that has cashflows with cashflowNames
          %% for CMS AVG structured we have nonCall, nonCallB, callable
           
            %nonCall
            cashflowNames = {'inverseFloaterCoupon','swapFloatingCoupon'};
            cashflowSizeMap = containers.Map({'inverseFloaterCoupon','swapFloatingCoupon'},{structuredCouponSchedule.scheduleSize, ...
                                            swapFloatingCouponSchedule.scheduleSize});
            
            cashflowInfo.cashflowNames = cashflowNames;
            cashflowInfo.cashflowSizeMap = cashflowSizeMap;
            cashflowInfo.pathSize = modelStatesSize;
            
           nonCall = PricingColumnInfoGenericCashflow(cashflowInfo);
           nonCallB = PricingColumnInfoGenericCashflow(cashflowInfo);
           callable = PricingColumnInfoGenericCashflow(cashflowInfo);
           
           
           % intermediate variables
           payoffStateA.cashflow = zeros(1,modelStatesSize);
           payoffStateA.cashflow_npv = zeros(1,modelStatesSize);
           
           % we don't need local variable for inverse floater
           
%            % variable to use accruedSumOfStockPrice for moving average calculation 
%            
%            cD = 0; % cumulated day count
%            cP = 0; 
%            dI = 0; % dummy idx for loop
%            dcfC = 0;
%            dcfN = 0;
%            dD = 0;
%       
%            idxL = zeros(1,modelStatesSize);
%            idxS = zeros(1,modelStatesSize);
% 
%            idxC = zeros(1,modelStatesSize);
%            idxN = zeros(1,modelStatesSize);
% 
%            spreadC = zeros(1,modelStatesSize);
%            spreadN = zeros(1,modelStatesSize);
%            probSpread = zeros(1,pathSize);
           
            % we use all path for regression
            % conversion ITM , puttable ITM ,  callable ITM
            % what's the use of Alive Path only Regression Sampling ?
           
          %% Path Generation Forward
          %% Store path dependent variables for coupon calculation 
          %% Store state variables to be used for AMC Regression
           
           %payoff descrption update index
           currentTimeIdx = 1;
           %MC numMethodState update index
           aliveTimeIdx = 1;
           % currentTimeStep is update in for statement
%            for currentTimeStep = eventTimeStep(1):1:eventTimeStep(end)
           for idxT = 1:1:length(eventTimeStep)
                currentTimeStep = eventTimeStep(idxT);
                
                if idxT < length(eventTimeStep)
                    nextTimeStep = eventTimeStep(idxT+1);
                end
                % evaluate scripts 
                % evaluate stateVariables(stores)
                % evaluate expression per each pricing columns
                % at the end of the loop we induct forward
                % from left to right
                % fictional payoff script columns are
                % cumCoupon,nonCall,Callable
                
                        
                % find the current
                % if find return errors then?
                try
                    
                    % cachedModelStateCoupon for modelState
                    idxCouponEvent = find(currentTimeStep == structuredCouponSchedule.eventTime);
                    if ~isempty(idxCouponEvent)
                        cachedModelStatesCoupon(numOfFactors*(idxCouponEvent-1)+1:numOfFactors*idxCouponEvent,:) = nextX;
                    end
                    
                    % cachedModelStateCall for modelState
                    idxCallEvent = find(currentTimeStep == callableSchedule.eventTime);
                    if ~isempty(idxCallEvent)
                        cachedModelStatesCall(numOfFactors*(idxCallEvent-1)+1:numOfFactors*idxCallEvent,:) = nextX;
                    end
                    
                    % cachedModelStateSwapFloatingEvent for modelState
                    idxSwapFloatingEvent = find(currentTimeStep == swapFloatingCouponSchedule.eventTime);
                    if ~isempty(idxSwapFloatingEvent)
                        cachedModelStatesSwapFloating(numOfFactors*(idxSwapFloatingEvent-1)+1:numOfFactors*idxSwapFloatingEvent,:) = nextX;
                    end
                    
%                     % find the corresponding nearest future coupon event
%                   %% structuredCouponSchedule event start
%                     idx = min(find(currentTimeStep <= structuredCouponSchedule.eventTime));
%                     if ~isempty(idx)
%                       %% stateVariable floatingCoupon
%                         idxL = ck1F.Rate_IndexGeneric(rateType_tenorL,valueDate,currentTimeStep,numMatTime, tenorL,nextX);
%                         idxS = ck1F.Rate_IndexGeneric(rateType_tenorS,valueDate,currentTimeStep,numMatTime, tenorS,nextX);
%                         
% %                         idxL = ck1F.CMS_PSA_SimpleDateGeneric(valueDate,currentTimeStep,numMatTime, tenorL,nextX);
% %                         idxS = ck1F.CMS_PSA_SimpleDateGeneric(valueDate,currentTimeStep,numMatTime, tenorS,nextX);
%                         
%                         dcfN = nextTimeStep;
%                         dcfC = currentTimeStep;
%                         % if coupon accrual condition is met
%                         % period startDate is included but 
%                         % endDate is not included
%                         
%                         if currentTimeStep >= structuredCouponSchedule.startTime(idx) ...
%                            && currentTimeStep < structuredCouponSchedule.eventTime(idx)
%                             
%                             dD = dcfN - dcfC;
%                             cD = cD + dD;
%                             cP = cP + (idxL-idxS)*dD;
%                             
%                         end
%                          
%                         % now if we are at couponSchedule eventTime
%                         % update floatingCoupon
%                         if currentTimeStep == structuredCouponSchedule.eventTime(idx)
% 
%                             steepenerCoupon(idx,:) =  max(min(multiplier * cP/cD + spread,localCap),localFloor) * structuredCouponSchedule.dayCountFraction(idx) * nominal;
%                             
%                             % resetting local variables for averaging
%                             cD = 0; % cumulated day count
%                             cP = 0; % cumulated steepener sum
%                             
%                             if idx == structuredCouponSchedule.scheduleSize
%                                 payoffStateA.cashflow = nominal*0.0 + steepenerCoupon(idx,:);
%                              else
%                                 payoffStateA.cashflow = steepenerCoupon(idx,:);
%                             end
%                             
%                             eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
%                             payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;
% 
%                              payTime = structuredCouponSchedule.payTime(idx);
%                              
%                              %process payoff cashflow 
%                              nonCall.ProcessAddPayoff(ck1F,currentTimeStep,payTime,numeraire,nextX, ...
%                                                      payoffStateA.cashflow,'steepenerCoupon',idx);
%      
%                         end
%                     end
                    
                 %% structuredCouponSchedule event end
                 
                 %% now if we are at structuredCouponSchedule eventTime
                   
                 %% structuredCouponSchedule event start
                 %% if inverseFloaterCoupon event happens at RESET Date
                 %% then we fix floatingCoupon at resetDate and we raise event at resetdate
                 %% if inverseFloaterCoupon event happens at END date then we reset inverseFloatRate first
                 %% at ResetDate and we refer this rate at EventDate(EndDate)
                    if strcmp(structuredCouponSchedule.eventType,'RESET')
                     %% now if we are at structuredCouponSchedule eventTime
                     %% structuredCouponSchedule event start

                        idx = find(currentTimeStep == structuredCouponSchedule.eventTime);
                        if ~isempty(idx)

                            inverseFloatRate = ck1F.Rate_IndexGeneric(rateType_tenorInverseFloater,valueDate,currentTimeStep,numMatTime,tenorInverseFloater,nextX);
%                             floatRate = ck1F.CMS_PSA_SimpleDateGeneric(valueDate,currentTimeStep,numMatTime,swapFloating_tenor,nextX);
%                             inverseFloaterCoupon(idx,:) =  (swapFloating_multiplier * floatRate + swapFloating_spread) * swapFloatingCouponSchedule.dayCountFraction(idxFloat) * nominal;
                            
                        %% inverseFloaterCoupon
                            
                            inverseFloaterCoupon(idx,:) =  max(min(coupon -  multiplier * inverseFloatRate,localCap),localFloor) * structuredCouponSchedule.dayCountFraction(idx) * nominal;
                            
                            payoffStateA.cashflow = inverseFloaterCoupon(idx,:);

%                          %% since we are paying floating CMS Coupon
%                              % we multiply -1 with floating coupon before we
%                              % process payoff
%                             payoffStateA.cashflow = payoffStateA.cashflow * -1.0;

                            eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                            payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;
                            payTime = structuredCouponSchedule.payTime(idx);

                            %process payoff cashflow 
                            nonCall.ProcessAddPayoff(ck1F,currentTimeStep,payTime,numeraire,nextX, ...
                                                    payoffStateA.cashflow,'inverseFloaterCoupon',idx);

                        end
                      %% structuredCouponSchedule event end
                       
                   elseif strcmp(structuredCouponSchedule.eventType,'END')
                     %% if we raise inverseFloaterCouponEvent at EndDate then
                     %% we need to raise fixing event at resetDate and store fixed inverseFloatRate as stateVariables
                     %% and reference it at ENDDate
                        idxInverseFloatReset = find(currentTimeStep == structuredCouponSchedule.resetTime);
                        if ~isempty(idxInverseFloatReset)
                            
                            inverseFloatRate = ck1F.Rate_IndexGeneric(rateType_tenorInverseFloater,valueDate,currentTimeStep,numMatTime,tenorInverseFloater,nextX);
%                             floatRate = ck1F.CMS_PSA_SimpleDateGeneric(valueDate,currentTimeStep,numMatTime,swapFloating_tenor,nextX);
                            
                            prefixedInverseFloatRate(idxInverseFloatReset,:) = inverseFloatRate;
                        end
                        
                        idxEnd = find(currentTimeStep == structuredCouponSchedule.eventTime);
                        if ~isempty(idxEnd)
                            % now if we are at swapFloatingCouponSchedule
                            % eventTime(=endDate)
                            % then we update swapFloatingCoupon
                            prefixedFloatRate = prefixedInverseFloatRate(idxEnd,:);
%                             inverseFloaterCoupon(idxEnd,:) =  (swapFloating_multiplier * prefixedFloatRate + swapFloating_spread) * swapFloatingCouponSchedule.dayCountFraction(idxEnd) * nominal;
                            inverseFloaterCoupon(idxEnd,:) =  max(min(coupon -  multiplier * prefixedFloatRate,localCap),localFloor) * structuredCouponSchedule.dayCountFraction(idxEnd) * nominal;

                            payoffStateA.cashflow = inverseFloaterCoupon(idxEnd,:);

%                          %% since we are paying floating CMS Coupon
%                              % we multiply -1 with floating coupon before we
%                              % process payoff
%                             payoffStateA.cashflow = payoffStateA.cashflow * -1.0;
                            
                            eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                            payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;

                            payTime = structuredCouponSchedule.payTime(idxEnd);

                            %process payoff cashflow 
                            nonCall.ProcessAddPayoff(ck1F,currentTimeStep,payTime,numeraire,nextX, ...
                                                    payoffStateA.cashflow,'inverseFloaterCoupon',idxEnd);
                            
                        end
                 %% swapFloatingCouponSchedule event end
                   else
                       error('not supported eventType')
                    end
                    
                 %% structuredCouponSchedule event end
                    
                 %% now if we are at swapFloatingCouponSchedule eventTime
                    
                 %% swapFloatingCouponSchedule event start
                 %% if swapFloatingCoupon event happens at RESET Date
                 %% then we fix floatingCoupon at resetDate and we raise event at resetdate
                 %% if swapFloatingCoupon event happens at END date
                 %% then we fix floatingCoupon at resetDate and we raise event at endDate
                 
                   if strcmp(swapFloatingCouponSchedule.eventType,'RESET')
                     %% now if we are at swapFloatingCouponSchedule eventTime
                     %% swapFloatingCouponSchedule event start

                        idxFloat = find(currentTimeStep == swapFloatingCouponSchedule.eventTime);
                        if ~isempty(idxFloat)

                            floatRate = ck1F.Rate_IndexGeneric(rateType_tenorFloat,valueDate,currentTimeStep,numMatTime,swapFloating_tenor,nextX);
%                             floatRate = ck1F.CMS_PSA_SimpleDateGeneric(valueDate,currentTimeStep,numMatTime,swapFloating_tenor,nextX);
                            
                            swapFloatingCoupon(idxFloat,:) =  (swapFloating_multiplier * floatRate + swapFloating_spread) * swapFloatingCouponSchedule.dayCountFraction(idxFloat) * nominal;

                            payoffStateA.cashflow = swapFloatingCoupon(idxFloat,:);

                         %% since we are paying floating CMS Coupon
                             % we multiply -1 with floating coupon before we
                             % process payoff
                            payoffStateA.cashflow = payoffStateA.cashflow * -1.0;

                            eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                            payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;
                            
                            payTime = swapFloatingCouponSchedule.payTime(idxFloat);

                            %process payoff cashflow 
                            nonCall.ProcessAddPayoff(ck1F,currentTimeStep,payTime,numeraire,nextX, ...
                                                    payoffStateA.cashflow,'swapFloatingCoupon',idxFloat);

                        end
                      %% swapFloatingCouponSchedule event end
                       
                   elseif strcmp(swapFloatingCouponSchedule.eventType,'END')
                     %% if we raise swapFloatingCouponEvent at EndDate then
                     %% we need to raise fixing event at resetDate and store fixed floatRate as stateVariables
                     %% and reference it at ENDDate
                        idxFloatReset = find(currentTimeStep == swapFloatingCouponSchedule.resetTime);
                        if ~isempty(idxFloatReset)
                            
                            floatRate = ck1F.Rate_IndexGeneric(rateType_tenorFloat,valueDate,currentTimeStep,numMatTime,swapFloating_tenor,nextX);
%                             floatRate = ck1F.CMS_PSA_SimpleDateGeneric(valueDate,currentTimeStep,numMatTime,swapFloating_tenor,nextX);
                            
                            prefixedSwapFloatingRate(idxFloatReset,:) = floatRate;
                        end
                        
                        idxEnd = find(currentTimeStep == swapFloatingCouponSchedule.eventTime);
                        if ~isempty(idxEnd)
                            % now if we are at swapFloatingCouponSchedule
                            % eventTime(=endDate)
                            % then we update swapFloatingCoupon
                            prefixedFloatRate = prefixedSwapFloatingRate(idxEnd,:);
                            swapFloatingCoupon(idxEnd,:) =  (swapFloating_multiplier * prefixedFloatRate + swapFloating_spread) * swapFloatingCouponSchedule.dayCountFraction(idxEnd) * nominal;

                            payoffStateA.cashflow = swapFloatingCoupon(idxEnd,:);

                         %% since we are paying floating CMS Coupon
                             % we multiply -1 with floating coupon before we
                             % process payoff
                            payoffStateA.cashflow = payoffStateA.cashflow * -1.0;
                            
                            eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                            payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;

                            payTime = swapFloatingCouponSchedule.payTime(idxEnd);

                            %process payoff cashflow 
                            nonCall.ProcessAddPayoff(ck1F,currentTimeStep,payTime,numeraire,nextX, ...
                                                    payoffStateA.cashflow,'swapFloatingCoupon',idxEnd);
                            
                        end
                 %% swapFloatingCouponSchedule event end
                   else
                       error('not supported eventType')
                   end
                    % if currentTimeSteps is today or future then inductMCForward
                    if currentTimeStep >= 0 && currentTimeStep < eventTimeStep(end)
                        mcmcOut   = inductMCForwardNew(ck1F,currentTimeStep,nextTimeStep,...
                                    nextX,U(numOfFactors*(aliveTimeIdx-1)+1:numOfFactors*aliveTimeIdx,:));
                        nextX = mcmcOut.nextX;
                        %MC numMethodState update
                        aliveTimeIdx = aliveTimeIdx + 1;
                    end
                    
                    currentTimeIdx = currentTimeIdx +1;
                catch exception
                    throw(exception)
                end
                
                
           end
           
          %% Forward Induction End
          
           % backward pricingColumn
           % nonCallB,
           % callable
           
           
          %% Backward induction start
           for idxT=length(eventTimeStep):-1:1
               
               currentTimeStep = eventTimeStep(idxT);
               try
                  if strcmp(structuredCouponSchedule.eventType,'RESET') 
                    %% if swapFloatingCoupon event happens at resetDate so, in
                    %% backward induction it should be positioned
                    %% before call event

                     %% structuredCouponSchedule event start  
                       idxCoupon = find(currentTimeStep == structuredCouponSchedule.eventTime);
                       % if currentTime  is structuredCouponSchedule.eventTime
                       if ~isempty(idxCoupon)

                             % we use cached(saved) floatingCoupon that
                             % was generated from Forward Induction
                             payTime = structuredCouponSchedule.payTime(idxCoupon);
                             payoffStateA.cashflow = inverseFloaterCoupon(idxCoupon,:);

%                              cachedModelStatesSwapFloatingV = cachedModelStatesSwapFloating(numOfFactors*(idxFloat-1)+1:numOfFactors*idxFloat,:);
                             cachedModelStatesCouponV = cachedModelStatesCoupon(numOfFactors*(idxCoupon-1)+1:numOfFactors*idxCoupon,:);
                             
%                           %% since we are paying floating CMS Coupon
%                              % we multiply -1 with floating coupon before we
%                              % process payoff
%                              payoffStateA.cashflow = payoffStateA.cashflow *-1.0;
                             
                             eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                             payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;

                             % at coupon EventDate AddFunctor
                             %process payoff cashflow 
                             nonCallB.ProcessAddPayoff(ck1F,currentTimeStep,payTime,numeraire,cachedModelStatesCouponV, ...
                                                   payoffStateA.cashflow,'inverseFloaterCoupon',idxCoupon);

                             callable.ProcessAddPayoff(ck1F,currentTimeStep,payTime,numeraire,cachedModelStatesCouponV, ...
                                                   payoffStateA.cashflow,'inverseFloaterCoupon',idxCoupon);

                       end

                     %% structuredCouponSchedule event end  
                  end
                  
                  if strcmp(swapFloatingCouponSchedule.eventType,'RESET') 
                    %% if swapFloatingCoupon event happens at resetDate so, in
                    %% backward induction it should be positioned
                    %% before call event

                     %% swapFloatingCoupon event start  
                       idxFloat = find(currentTimeStep == swapFloatingCouponSchedule.eventTime);
                       % if currentTime  is floatingCouponSchedule.eventTime
                       if ~isempty(idxFloat)

                             % we use cached(saved) floatingCoupon that
                             % was generated from Forward Induction
                             payTime = swapFloatingCouponSchedule.payTime(idxFloat);
                             payoffStateA.cashflow = swapFloatingCoupon(idxFloat,:);

                             cachedModelStatesSwapFloatingV = cachedModelStatesSwapFloating(numOfFactors*(idxFloat-1)+1:numOfFactors*idxFloat,:);
                             
                          %% since we are paying floating CMS Coupon
                             % we multiply -1 with floating coupon before we
                             % process payoff
                             payoffStateA.cashflow = payoffStateA.cashflow *-1.0;
                             
                             eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                             payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;

                             % at coupon EventDate AddFunctor
                             %process payoff cashflow 
                             nonCallB.ProcessAddPayoff(ck1F,currentTimeStep,payTime,numeraire,cachedModelStatesSwapFloatingV, ...
                                                   payoffStateA.cashflow,'swapFloatingCoupon',idxFloat);

                             callable.ProcessAddPayoff(ck1F,currentTimeStep,payTime,numeraire,cachedModelStatesSwapFloatingV, ...
                                                   payoffStateA.cashflow,'swapFloatingCoupon',idxFloat);

                       end

                     %% swapFloatingCoupon event end  
                  end
                 %% issuer callable event
                 %% AssignExercise Functor : applied only when eventTime >=0
                 %% if EODAdjust is YES : endOfDate conversion option has expired
                 %% if EODAdjust is NO : intraDay, and option is still alive
                   idxCall = find(currentTimeStep == callableSchedule.eventTime);
                   if ~isempty(idxCall)
                       if isExercised 
                           
                            payTime = callableSchedule.payTime(idxCall);
                            
                            % future payment <- is this necessary ?
                            if payTime >= 0
                                
                               %find the corresponding callPayment;
                               callPayment = callableSchedule.callPayment(idxCall);
                               %look up the corresponding cachedModelStatesCall for
                               %call decision variables
                               cachedModelStatesCallV = cachedModelStatesCall(numOfFactors*(idxCall-1)+1:numOfFactors*idxCall,:);

                               % exercised by notional amount
                               exerciseValue = callPayment*nominal*ones(1,modelStatesSize);
                               
                               % discount exerciseValues from payTime to eventTime
                               df_ep = ck1F.discountFactorRisky(currentTimeStep, payTime,numMatTime,cachedModelStatesCallV);
                               for k=1:modelStatesSize
                                   exerciseValue(k) = exerciseValue(k) * df_ep(k);
                               end
                               
                               outCallablePayoff = callable.ProcessAssignPayoff(ck1F,currentTimeStep,payTime,numeraire,cachedModelStatesCallV, ...
                                     exerciseValue);
                                
                            end
                       else
                           
                           if currentTimeStep >=0 && EODAdjust(EODFlag_Exercise,currentTimeStep)

                               %find the corresponding callPayment;
                               callPayment = callableSchedule.callPayment(idxCall);
                               %look up the corresponding cachedModelStatesCall for
                               %call decision variables
                               cachedModelStatesCallV = cachedModelStatesCall(numOfFactors*(idxCall-1)+1:numOfFactors*idxCall,:);

                               exerciseValue = zeros(1,modelStatesSize);
                               exerciseITMCondition = zeros(1,modelStatesSize);

                               for idxEx=1:modelStatesSize
                                   exerciseValue(idxEx) = callPayment*nominal; 
                                   exerciseITMCondition(idxEx) = 1;
                               end

                               payTime = callableSchedule.payTime(idxCall);

                               % discount exerciseValues from payTime to eventTime
                               df_ep = ck1F.discountFactorRisky(currentTimeStep, payTime,numMatTime,cachedModelStatesCallV);
                               for k=1:modelStatesSize
                                   exerciseValue(k) = exerciseValue(k) * df_ep(k);
                               end

                               % callable is minimizing the payoff
                               minMaxFlag   = 'MIN';
                               decisionVar1 =  cachedModelStatesCallV;
                               outCallablePayoff = AssignExercise(ck1F,currentTimeStep,callable,exerciseValue,exerciseITMCondition,...
                                          minMaxFlag,decisionVar1,nominal,numeraire,cachedModelStatesCallV);

                           end
                       
                       end
                   end
                   
                 %% coupon event
                  if strcmp(structuredCouponSchedule.eventType,'END')
                       idxCoupon = find(currentTimeStep == structuredCouponSchedule.eventTime);
                       % if currentTime  is couponSchedule.eventTime
                       if ~isempty(idxCoupon)

                           if currentTimeStep == structuredCouponSchedule.eventTime(idxCoupon)
                                 % we use cached(saved) floating Coupon that
                                 % was generated from Forward Induction
                                 payTime = structuredCouponSchedule.payTime(idxCoupon);

                                 if idxCoupon == structuredCouponSchedule.scheduleSize
                                    payoffStateA.cashflow = nominal*0.0 + inverseFloaterCoupon(idxCoupon,:);
                                 else
                                    payoffStateA.cashflow = inverseFloaterCoupon(idxCoupon,:);
                                 end

                                 eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                                 payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;

                                 cachedModelStatesCouponV = cachedModelStatesCoupon(numOfFactors*(idxCoupon-1)+1:numOfFactors*idxCoupon,:);

                                 % at coupon EventDate AddFunctor
                                 %process payoff cashflow 
                                 nonCallB.ProcessAddPayoff(ck1F,currentTimeStep,payTime,numeraire,cachedModelStatesCouponV, ...
                                                       payoffStateA.cashflow,'inverseFloaterCoupon',idxCoupon);

                                 callable.ProcessAddPayoff(ck1F,currentTimeStep,payTime,numeraire,cachedModelStatesCouponV, ...
                                                       payoffStateA.cashflow,'inverseFloaterCoupon',idxCoupon);

                           end
                       end
                  end
                 
                  if strcmp(swapFloatingCouponSchedule.eventType,'END')  
                    %% if swapFloatingCoupon event happens at endDate 
                    %% backward induction it should be positioned
                    %% after call event

                     %% swapFloatingCoupon event start  
                       idxFloat = find(currentTimeStep == swapFloatingCouponSchedule.eventTime);
                       % if currentTime  is floatingCouponSchedule.eventTime
                       if ~isempty(idxFloat)

                             % we use cached(saved) floatingCoupon that
                             % was generated from Forward Induction
                             payTime = swapFloatingCouponSchedule.payTime(idxFloat);
                             payoffStateA.cashflow = swapFloatingCoupon(idxFloat,:);

                             cachedModelStatesSwapFloatingV = cachedModelStatesSwapFloating(numOfFactors*(idxFloat-1)+1:numOfFactors*idxFloat,:);

                          %% since we are paying floating CMS Coupon
                             % we multiply -1 with floating coupon before we
                             % process payoff
                             payoffStateA.cashflow = payoffStateA.cashflow *-1.0;
                             
                             eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                             payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;

                             % at coupon EventDate AddFunctor
                             %process payoff cashflow 
                             nonCallB.ProcessAddPayoff(ck1F,currentTimeStep,payTime,numeraire,cachedModelStatesSwapFloatingV, ...
                                                   payoffStateA.cashflow,'swapFloatingCoupon',idxFloat);

                             callable.ProcessAddPayoff(ck1F,currentTimeStep,payTime,numeraire,cachedModelStatesSwapFloatingV, ...
                                                   payoffStateA.cashflow,'swapFloatingCoupon',idxFloat);

                       end

                     %% swapFloatingCoupon event end 
                  end
                 
                   
               catch exception
                    throw(exception)
               end    
           end
           
           
          %% Finalize output
           nonCall.npv = mean(nonCall.payoff);
           nonCallB.npv = mean(nonCallB.payoff);
           callable.npv = mean(callable.payoff);
       
           %forward
           out.nonCall = nonCall; 
           %backward
           out.nonCallB = nonCallB; 
           out.callable = callable;
           
           out.eventTimeStep = eventTimeStep';
           
        end
        
        function out = computeInverseFloaterSwapGeneric2_LSMC(ck1F,valueDate,steepenerParams,numMethod)
%           
            % model schedule generation
            dealStartDate = steepenerParams.startDate;
            dealStartEventTime = DateDiff(dealStartDate,valueDate);
            expiryDate = steepenerParams.endDate;
            dealEndEventTime = DateDiff(expiryDate,valueDate);
            %% structuredCouponSchedule start
            
            structuredCouponScheduleInt = steepenerParams.params.structuredCouponInfo.structuredCouponScheduleInt;
            
            % structuredCouponSchedule is a schedule of Structured Coupon
            % events
            
            % main backbone schedule
            structuredCouponScheduleSize = size(structuredCouponScheduleInt,1);
            structuredCouponSchedule= struct;
            structuredCouponSchedule.scheduleSize = structuredCouponScheduleSize;
            structuredCouponSchedule.eventType = steepenerParams.params.structuredCouponInfo.eventType;
            % coupon payment event happens at endDate
            % eventTime is endDate
            % payment date can be different from endDate
            
            % startTime,endTimeP should be included in timeStep Generation
            % with eventTime
            
            for i=1:structuredCouponSchedule.scheduleSize
                couponResetDate = H_Date(structuredCouponScheduleInt(i,1));
                couponStartDate = H_Date(structuredCouponScheduleInt(i,2));
                couponEndDate = H_Date(structuredCouponScheduleInt(i,3));
                couponPayDate = H_Date(structuredCouponScheduleInt(i,4));
                
                structuredCouponSchedule.dayCount(i) = couponEndDate.DateDiff(couponStartDate);
                structuredCouponSchedule.dayCountFraction(i) = structuredCouponSchedule.dayCount(i)/365.0; 
                
                couponEndDateP = couponEndDate.AddDate( -1 ,'day');
                
                structuredCouponSchedule.resetTime(i) = DateDiff(couponResetDate,valueDate);
                structuredCouponSchedule.startTime(i) = DateDiff(couponStartDate,valueDate);
                structuredCouponSchedule.endTimeP(i)  = DateDiff(couponEndDateP,valueDate);
                structuredCouponSchedule.endTime(i) = DateDiff(couponEndDate,valueDate);
                
                if strcmp(structuredCouponSchedule.eventType,'RESET')
                    structuredCouponSchedule.eventTime(i) = DateDiff(couponResetDate,valueDate);
                elseif strcmp(structuredCouponSchedule.eventType,'START')
                    structuredCouponSchedule.eventTime(i) = DateDiff(couponStartDate,valueDate);
                elseif strcmp(structuredCouponSchedule.eventType,'END')
                    structuredCouponSchedule.eventTime(i) = DateDiff(couponEndDate,valueDate);
                else % default is END : event happens at EndDate for StructuredCoupon
                    structuredCouponSchedule.eventTime(i) = DateDiff(couponEndDate,valueDate);
                end
                structuredCouponSchedule.payTime(i) = DateDiff(couponPayDate,valueDate);
                
            end
            
            
            tenorInverseFloater     = steepenerParams.params.structuredCouponInfo.tenorInverseFloater;
            
           %% rateType for structuredCoupon : default CMS
            if isfield(steepenerParams.params.structuredCouponInfo, 'rateType_tenorInverseFloater')
                rateType_tenorInverseFloater = steepenerParams.params.structuredCouponInfo.rateType_tenorInverseFloater;
            else
                rateType_tenorInverseFloater = 'CMS';
            end
            
            
            localFloor = steepenerParams.params.structuredCouponInfo.localFloor;
            localCap   = steepenerParams.params.structuredCouponInfo.localCap;
            multiplier = steepenerParams.params.structuredCouponInfo.multiplier;
            spread     = steepenerParams.params.structuredCouponInfo.spread;
            coupon     = steepenerParams.params.structuredCouponInfo.coupon;
           %% structuredCoupon schedule end
           
           %% call exercise event happens at the callExerciseDate
           %% callable schedule start
            callableScheduleInt = steepenerParams.params.callableInfo.callableScheduleInt;
            callableScheduleSize = size(callableScheduleInt,1);
            callableSchedule= struct;
            for i=1:callableScheduleSize
                callExerciseDate = H_Date(callableScheduleInt(i,1));
                callPayDate = H_Date(callableScheduleInt(i,2));
                callableSchedule.eventTime(i) = DateDiff(callExerciseDate,valueDate);
                callableSchedule.payTime(i) = DateDiff(callPayDate,valueDate);
                
                 callableSchedule.callPayment(i) = callableScheduleInt(i,3);
            end
            callableSchedule.scheduleSize = callableScheduleSize;
           
            isExercised = steepenerParams.params.callableInfo.isExercised;
           %% callable schedule end
           
          %% swapFloatingCouponSchedule start
            
            swapFloatingCouponScheduleInt = steepenerParams.params.swapFloatingCouponInfo.swapFloatingCouponScheduleInt;
            
            swapFloatingCouponSchedule= struct;
            swapFloatingCouponSchedule.scheduleSize = size(swapFloatingCouponScheduleInt,1);
            
            swapFloatingCouponSchedule.eventType = steepenerParams.params.swapFloatingCouponInfo.eventType;
            
            % floating coupon payment event happens at resetDate
            % eventTime is resetDate
            
            % resetTime,startTime,endTimeP should be included in timeStep Generation
            % with eventTime
            
            for i=1:swapFloatingCouponSchedule.scheduleSize
                flolatingCouponResetDate = H_Date(swapFloatingCouponScheduleInt(i,1));
                flolatingCouponStartDate = H_Date(swapFloatingCouponScheduleInt(i,2));
                flolatingCouponEndDate = H_Date(swapFloatingCouponScheduleInt(i,3));
                flolatingCouponPayDate = H_Date(swapFloatingCouponScheduleInt(i,4));
                
                swapFloatingCouponSchedule.dayCount(i) = flolatingCouponEndDate.DateDiff(flolatingCouponStartDate);
                swapFloatingCouponSchedule.dayCountFraction(i) = swapFloatingCouponSchedule.dayCount(i)/365.0; 
                
                flolatingCouponEndDateP = flolatingCouponEndDate.AddDate( -1 ,'day');
                
                swapFloatingCouponSchedule.resetTime(i) = DateDiff(flolatingCouponResetDate,valueDate);
                swapFloatingCouponSchedule.startTime(i) = DateDiff(flolatingCouponStartDate,valueDate);
                swapFloatingCouponSchedule.endTimeP(i)  = DateDiff(flolatingCouponEndDateP,valueDate);
                swapFloatingCouponSchedule.endTime(i) = DateDiff(flolatingCouponEndDate,valueDate);
                
                if strcmp(swapFloatingCouponSchedule.eventType,'RESET')
                    swapFloatingCouponSchedule.eventTime(i) = DateDiff(flolatingCouponResetDate,valueDate);
                elseif strcmp(swapFloatingCouponSchedule.eventType,'START')
                    swapFloatingCouponSchedule.eventTime(i) = DateDiff(flolatingCouponStartDate,valueDate);
                elseif strcmp(swapFloatingCouponSchedule.eventType,'END')
                    swapFloatingCouponSchedule.eventTime(i) = DateDiff(flolatingCouponEndDate,valueDate);
                else % default is RESET : for SwapFloatingCouponSchedule
                    swapFloatingCouponSchedule.eventTime(i) = DateDiff(flolatingCouponResetDate,valueDate);
                end
                
%                 swapFloatingCouponSchedule.eventTime(i) = DateDiff(flolatingCouponResetDate,valueDate);
                
                swapFloatingCouponSchedule.payTime(i) = DateDiff(flolatingCouponPayDate,valueDate);
                
            end
            
            swapFloating_tenor  = steepenerParams.params.swapFloatingCouponInfo.tenorFloat;
            
            %% rateType for swapFloatingCoupon : default LIBOR
            if isfield(steepenerParams.params.swapFloatingCouponInfo, 'rateType_tenorFloat')
                rateType_tenorFloat = steepenerParams.params.swapFloatingCouponInfo.rateType_tenorFloat;
            else
                rateType_tenorFloat = 'LIBOR';
            end
            
            swapFloating_multiplier = steepenerParams.params.swapFloatingCouponInfo.multiplier;
            swapFloating_spread     = steepenerParams.params.swapFloatingCouponInfo.spread;
           
          %% SwapFloatingCouponSchedule end
            
            
          %% pricing time Step Generation start
            
            % eventTimeStep Generate
            mcOneTimeStep = numMethod.mcOneTimeStep;
            
          %% eventTime from dealStartTime and dealEndTime
           eventTimeStep = dealStartEventTime;
           eventTimeStep = union(eventTimeStep,dealEndEventTime);
           
          %% eventTime From structuredCouponSchedule:eventTime(endTime) +
          %% startTime,endTimeP
          %% daily weekly timeStep for couponSchedule
          
          %% InverseFloater
          %% eventTime From structuredCouponSchedule:eventTime
            eventTimeStep = union(eventTimeStep, structuredCouponSchedule.resetTime);
            eventTimeStep = union(eventTimeStep,structuredCouponSchedule.eventTime);
%             eventTimeStep = union(eventTimeStep, structuredCouponSchedule.startTime);
            eventTimeStep = union(eventTimeStep, structuredCouponSchedule.endTime);
%             eventTimeStep = union(eventTimeStep, structuredCouponSchedule.endTimeP);
           %% eventTime From CallSchedule:eventTime
            eventTimeStep = union(eventTimeStep,callableSchedule.eventTime);
            eventTimeStep = sort(eventTimeStep,'ascend');
            
          %% eventTime From SwapFloatingCouponSchedule:eventTime
            eventTimeStep = union(eventTimeStep,swapFloatingCouponSchedule.resetTime);  
            eventTimeStep = union(eventTimeStep,swapFloatingCouponSchedule.eventTime);
            eventTimeStep = sort(eventTimeStep,'ascend');
            
            
            if dealStartEventTime < 0 
                dailyTimeSteps = [0:-1:dealStartEventTime];
                eventTimeStep = union(eventTimeStep,dailyTimeSteps);
            end
          
          %% Daily Weekly Time Step Generation for AllTheTime ScheduleType, FromTo Schedule end
            
            eventTimeStep = sort(eventTimeStep,'ascend');
            
          %% aliveTimeStep for mc path generation
            aliveTimeStep = eventTimeStep(eventTimeStep>=0);
            aliveTimeStep = sort(aliveTimeStep,'ascend');
            
            totalAliveTimeStepSize = length(aliveTimeStep);
            
           %% eventTimeStep Generation End
           
           %% numeraire info
            numMatTime = eventTimeStep(end);
            dfNumMat = ck1F.DFRisky(numMatTime/365.0);

            numeraire.numMatTime = numMatTime;
            numeraire.dfNumMat = dfNumMat;
              
            % should we use eventTimeIdx as future only time stes?
            % In this case we better use more generic approach
            % so that all times steps are included 
            
          %% pricing time Step Generation end
           
          %% MC init
            rng('default');
            rng(0);
            
            numOfFactors = ck1F.numOfFactors;
            NMC = numMethod.pathSize; 
            MCTimeStep = totalAliveTimeStepSize -1; % nowDate (0) should be skipped
%             Z= randn(NMC/2,MCTimeStep);
            Z= randn(NMC/2,numOfFactors*MCTimeStep);
            Z=[Z;-Z];
            Z=Z';

            %TimeInversion
            sizeZ = size(Z,1);
            A = zeros(sizeZ,sizeZ);
            for i=1:sizeZ
                A(i,sizeZ + 1 -i) = 1;
            end
            Z = A*Z;

%                 U = Z';
            U = Z;

           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = zeros(numOfFactors,NMC);
           
           nextX = initX;
            
          %% payoff init
           nominal = steepenerParams.params.nominal;

           % EODFlag : if true then pricing as EndOfDate(cashflow on
           % valueDate wil be not included(already paid)
           % if false then pricing as Intraday ( cashflow on valueDate will 
           % be included)
           
           %EODFlag_Cashflow: Cashflow EOD Flag
           %EODFlag_Exercise: Exercise(Option) EOD Flag
           EODFlag_Cashflow = steepenerParams.params.EODFlag_Cashflow;
           EODFlag_Exercise = steepenerParams.params.EODFlag_Exercise;
           
           
           modelStatesSize = NMC;
           
          %% Define Pricing Columns
           % pricingColumns : nonCall,Callable  
           % we need inverseFloaterCoupon as state variables for storing 
           % inverse Floater Coupon
           
           % for the cashflow generated pricing columns as pricerInfo
           % we do use pricerInfo Structure
           
         %% (global) state variable initialize  : these correspond to ColumnVal[t] in generic pricing deal description
           
          % avg steeper coupon
           inverseFloaterCoupon = zeros(structuredCouponSchedule.scheduleSize,modelStatesSize);
           prefixedInverseFloatRate = zeros(structuredCouponSchedule.scheduleSize,modelStatesSize);
           
           swapFloatingCoupon = zeros(swapFloatingCouponSchedule.scheduleSize,modelStatesSize);
           prefixedSwapFloatingRate = zeros(swapFloatingCouponSchedule.scheduleSize,modelStatesSize);
           
           cachedModelStatesCoupon = zeros(structuredCouponSchedule.scheduleSize*numOfFactors,modelStatesSize);
  
          % for regression (2 modelState variables)
           cachedModelStatesCall = zeros(callableSchedule.scheduleSize*numOfFactors,modelStatesSize);
           
          % for swapFloatin 
           cachedModelStatesSwapFloating = zeros(swapFloatingCouponSchedule.scheduleSize*numOfFactors,modelStatesSize);
           
          %% PricingColumnGenericCashflow
          %% a structure that has cashflows with cashflowNames
          %% for CMS AVG structured we have nonCall, nonCallB, callable
           
            %nonCall
            cashflowNames = {'inverseFloaterCoupon','swapFloatingCoupon'};
            cashflowSizeMap = containers.Map({'inverseFloaterCoupon','swapFloatingCoupon'},{structuredCouponSchedule.scheduleSize, ...
                                            swapFloatingCouponSchedule.scheduleSize});
            
            cashflowInfo.cashflowNames = cashflowNames;
            cashflowInfo.cashflowSizeMap = cashflowSizeMap;
            cashflowInfo.pathSize = modelStatesSize;
            
           nonCall = PricingColumnInfoGenericCashflow(cashflowInfo);
           nonCallB = PricingColumnInfoGenericCashflow(cashflowInfo);
           callable = PricingColumnInfoGenericCashflow(cashflowInfo);
           
           
           % intermediate variables
           payoffStateA.cashflow = zeros(1,modelStatesSize);
           payoffStateA.cashflow_npv = zeros(1,modelStatesSize);
           
           % we don't need local variable for inverse floater
           
%            % variable to use accruedSumOfStockPrice for moving average calculation 
%            
%            cD = 0; % cumulated day count
%            cP = 0; 
%            dI = 0; % dummy idx for loop
%            dcfC = 0;
%            dcfN = 0;
%            dD = 0;
%       
%            idxL = zeros(1,modelStatesSize);
%            idxS = zeros(1,modelStatesSize);
% 
%            idxC = zeros(1,modelStatesSize);
%            idxN = zeros(1,modelStatesSize);
% 
%            spreadC = zeros(1,modelStatesSize);
%            spreadN = zeros(1,modelStatesSize);
%            probSpread = zeros(1,pathSize);
           
            % we use all path for regression
            % conversion ITM , puttable ITM ,  callable ITM
            % what's the use of Alive Path only Regression Sampling ?
           
          %% Path Generation Forward
          %% Store path dependent variables for coupon calculation 
          %% Store state variables to be used for AMC Regression
           
           %payoff descrption update index
           currentTimeIdx = 1;
           %MC numMethodState update index
           aliveTimeIdx = 1;
           % currentTimeStep is update in for statement
%            for currentTimeStep = eventTimeStep(1):1:eventTimeStep(end)
           for idxT = 1:1:length(eventTimeStep)
                currentTimeStep = eventTimeStep(idxT);
                
                if idxT < length(eventTimeStep)
                    nextTimeStep = eventTimeStep(idxT+1);
                end
                % evaluate scripts 
                % evaluate stateVariables(stores)
                % evaluate expression per each pricing columns
                % at the end of the loop we induct forward
                % from left to right
                % fictional payoff script columns are
                % cumCoupon,nonCall,Callable
                
                        
                % find the current
                % if find return errors then?
                try
                    
                    % cachedModelStateCoupon for modelState
                    idxCouponEvent = find(currentTimeStep == structuredCouponSchedule.eventTime);
                    if ~isempty(idxCouponEvent)
                        cachedModelStatesCoupon(numOfFactors*(idxCouponEvent-1)+1:numOfFactors*idxCouponEvent,:) = nextX;
                    end
                    
                    % cachedModelStateCall for modelState
                    idxCallEvent = find(currentTimeStep == callableSchedule.eventTime);
                    if ~isempty(idxCallEvent)
                        cachedModelStatesCall(numOfFactors*(idxCallEvent-1)+1:numOfFactors*idxCallEvent,:) = nextX;
                    end
                    
                    % cachedModelStateSwapFloatingEvent for modelState
                    idxSwapFloatingEvent = find(currentTimeStep == swapFloatingCouponSchedule.eventTime);
                    if ~isempty(idxSwapFloatingEvent)
                        cachedModelStatesSwapFloating(numOfFactors*(idxSwapFloatingEvent-1)+1:numOfFactors*idxSwapFloatingEvent,:) = nextX;
                    end
                 
                 %% now if we are at structuredCouponSchedule eventTime
                   
                 %% structuredCouponSchedule event start
                 %% if inverseFloaterCoupon event happens at RESET Date
                 %% then we fix floatingCoupon at resetDate and we raise event at resetdate
                 %% if inverseFloaterCoupon event happens at END date then we reset inverseFloatRate first
                 %% at ResetDate and we refer this rate at EventDate(EndDate)
                    if strcmp(structuredCouponSchedule.eventType,'RESET')
                     %% now if we are at structuredCouponSchedule eventTime
                     %% structuredCouponSchedule event start

                        idx = find(currentTimeStep == structuredCouponSchedule.eventTime);
                        if ~isempty(idx)

                            inverseFloatRate = ck1F.Rate_IndexGeneric(rateType_tenorInverseFloater,valueDate,currentTimeStep,numMatTime,tenorInverseFloater,nextX);
                            
                        %% inverseFloaterCoupon
                            
                            inverseFloaterCoupon(idx,:) =  max(min(coupon -  multiplier * inverseFloatRate,localCap),localFloor) * structuredCouponSchedule.dayCountFraction(idx) * nominal;
                            
                            payoffStateA.cashflow = inverseFloaterCoupon(idx,:);

                            eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                            payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;
                            payTime = structuredCouponSchedule.payTime(idx);

                            %process payoff cashflow 
                            nonCall.ProcessAddPayoff(ck1F,currentTimeStep,payTime,numeraire,nextX, ...
                                                    payoffStateA.cashflow,'inverseFloaterCoupon',idx);

                        end
                      %% structuredCouponSchedule event end
                       
                   elseif strcmp(structuredCouponSchedule.eventType,'END')
                     %% if we raise inverseFloaterCouponEvent at EndDate then
                     %% we need to raise fixing event at resetDate and store fixed inverseFloatRate as stateVariables
                     %% and reference it at ENDDate
                        idxInverseFloatReset = find(currentTimeStep == structuredCouponSchedule.resetTime);
                        if ~isempty(idxInverseFloatReset)
                            
                            inverseFloatRate = ck1F.Rate_IndexGeneric(rateType_tenorInverseFloater,valueDate,currentTimeStep,numMatTime,tenorInverseFloater,nextX);
                            prefixedInverseFloatRate(idxInverseFloatReset,:) = inverseFloatRate;
                        end
                        
                        idxEnd = find(currentTimeStep == structuredCouponSchedule.eventTime);
                        if ~isempty(idxEnd)
                            prefixedFloatRate = prefixedInverseFloatRate(idxEnd,:);
                            inverseFloaterCoupon(idxEnd,:) =  max(min(coupon -  multiplier * prefixedFloatRate,localCap),localFloor) * structuredCouponSchedule.dayCountFraction(idxEnd) * nominal;

                            payoffStateA.cashflow = inverseFloaterCoupon(idxEnd,:);
                            
                            eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                            payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;

                            payTime = structuredCouponSchedule.payTime(idxEnd);

                            %process payoff cashflow 
                            nonCall.ProcessAddPayoff(ck1F,currentTimeStep,payTime,numeraire,nextX, ...
                                                    payoffStateA.cashflow,'inverseFloaterCoupon',idxEnd);
                            
                        end
                 %% swapFloatingCouponSchedule event end
                   else
                       error('not supported eventType')
                    end
                    
                 %% structuredCouponSchedule event end
                    
                 %% now if we are at swapFloatingCouponSchedule eventTime
                    
                 %% swapFloatingCouponSchedule event start
                 %% if swapFloatingCoupon event happens at RESET Date
                 %% then we fix floatingCoupon at resetDate and we raise event at resetdate
                 %% if swapFloatingCoupon event happens at END date
                 %% then we fix floatingCoupon at resetDate and we raise event at endDate
                 
                   if strcmp(swapFloatingCouponSchedule.eventType,'RESET')
                     %% now if we are at swapFloatingCouponSchedule eventTime
                     %% swapFloatingCouponSchedule event start

                        idxFloat = find(currentTimeStep == swapFloatingCouponSchedule.eventTime);
                        if ~isempty(idxFloat)

                            floatRate = ck1F.Rate_IndexGeneric(rateType_tenorFloat,valueDate,currentTimeStep,numMatTime,swapFloating_tenor,nextX);
                            
                            swapFloatingCoupon(idxFloat,:) =  (swapFloating_multiplier * floatRate + swapFloating_spread) * swapFloatingCouponSchedule.dayCountFraction(idxFloat) * nominal;

                            payoffStateA.cashflow = swapFloatingCoupon(idxFloat,:);

                         %% since we are paying floating CMS Coupon
                             % we multiply -1 with floating coupon before we
                             % process payoff
                            payoffStateA.cashflow = payoffStateA.cashflow * -1.0;

                            eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                            payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;
                            
                            payTime = swapFloatingCouponSchedule.payTime(idxFloat);

                            %process payoff cashflow 
                            nonCall.ProcessAddPayoff(ck1F,currentTimeStep,payTime,numeraire,nextX, ...
                                                    payoffStateA.cashflow,'swapFloatingCoupon',idxFloat);

                        end
                      %% swapFloatingCouponSchedule event end
                       
                   elseif strcmp(swapFloatingCouponSchedule.eventType,'END')
                     %% if we raise swapFloatingCouponEvent at EndDate then
                     %% we need to raise fixing event at resetDate and store fixed floatRate as stateVariables
                     %% and reference it at ENDDate
                        idxFloatReset = find(currentTimeStep == swapFloatingCouponSchedule.resetTime);
                        if ~isempty(idxFloatReset)
                            
                            floatRate = ck1F.Rate_IndexGeneric(rateType_tenorFloat,valueDate,currentTimeStep,numMatTime,swapFloating_tenor,nextX);
                            
                            prefixedSwapFloatingRate(idxFloatReset,:) = floatRate;
                        end
                        
                        idxEnd = find(currentTimeStep == swapFloatingCouponSchedule.eventTime);
                        if ~isempty(idxEnd)
                            % now if we are at swapFloatingCouponSchedule
                            % eventTime(=endDate)
                            % then we update swapFloatingCoupon
                            prefixedFloatRate = prefixedSwapFloatingRate(idxEnd,:);
                            swapFloatingCoupon(idxEnd,:) =  (swapFloating_multiplier * prefixedFloatRate + swapFloating_spread) * swapFloatingCouponSchedule.dayCountFraction(idxEnd) * nominal;

                            payoffStateA.cashflow = swapFloatingCoupon(idxEnd,:);

                         %% since we are paying floating CMS Coupon
                             % we multiply -1 with floating coupon before we
                             % process payoff
                            payoffStateA.cashflow = payoffStateA.cashflow * -1.0;
                            
                            eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                            payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;

                            payTime = swapFloatingCouponSchedule.payTime(idxEnd);

                            %process payoff cashflow 
                            nonCall.ProcessAddPayoff(ck1F,currentTimeStep,payTime,numeraire,nextX, ...
                                                    payoffStateA.cashflow,'swapFloatingCoupon',idxEnd);
                            
                        end
                 %% swapFloatingCouponSchedule event end
                   else
                       error('not supported eventType')
                   end
                   
                   % if currentTimeSteps is today or future then inductMCForward
                    if currentTimeStep >= 0 && currentTimeStep < eventTimeStep(end)
                        mcmcOut   = inductMCForwardNew(ck1F,currentTimeStep,nextTimeStep,...
                                    nextX,U(numOfFactors*(aliveTimeIdx-1)+1:numOfFactors*aliveTimeIdx,:));
                        nextX = mcmcOut.nextX;
                        %MC numMethodState update
                        aliveTimeIdx = aliveTimeIdx + 1;
                    end
                    
                    currentTimeIdx = currentTimeIdx +1;
                catch exception
                    throw(exception)
                end
                
                
           end
           
          %% Forward Induction End
          
           % backward pricingColumn
           % nonCallB,
           % callable
           
           
          %% Backward induction start
           for idxT=length(eventTimeStep):-1:1
               
               currentTimeStep = eventTimeStep(idxT);
               try
                  if strcmp(structuredCouponSchedule.eventType,'RESET') 
                    %% if swapFloatingCoupon event happens at resetDate so, in
                    %% backward induction it should be positioned
                    %% before call event

                     %% structuredCouponSchedule event start  
                       idxCoupon = find(currentTimeStep == structuredCouponSchedule.eventTime);
                       % if currentTime  is structuredCouponSchedule.eventTime
                       if ~isempty(idxCoupon)

                             % we use cached(saved) floatingCoupon that
                             % was generated from Forward Induction
                             payTime = structuredCouponSchedule.payTime(idxCoupon);
                             payoffStateA.cashflow = inverseFloaterCoupon(idxCoupon,:);

                             cachedModelStatesCouponV = cachedModelStatesCoupon(numOfFactors*(idxCoupon-1)+1:numOfFactors*idxCoupon,:);
                             
                             eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                             payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;

                             % at coupon EventDate AddFunctor
                             %process payoff cashflow 
                             nonCallB.ProcessAddPayoff(ck1F,currentTimeStep,payTime,numeraire,cachedModelStatesCouponV, ...
                                                   payoffStateA.cashflow,'inverseFloaterCoupon',idxCoupon);

                             callable.ProcessAddPayoff(ck1F,currentTimeStep,payTime,numeraire,cachedModelStatesCouponV, ...
                                                   payoffStateA.cashflow,'inverseFloaterCoupon',idxCoupon);

                       end

                     %% structuredCouponSchedule event end  
                  end
                  
                  if strcmp(swapFloatingCouponSchedule.eventType,'RESET') 
                    %% if swapFloatingCoupon event happens at resetDate so, in
                    %% backward induction it should be positioned
                    %% before call event

                     %% swapFloatingCoupon event start  
                       idxFloat = find(currentTimeStep == swapFloatingCouponSchedule.eventTime);
                       % if currentTime  is floatingCouponSchedule.eventTime
                       if ~isempty(idxFloat)

                             % we use cached(saved) floatingCoupon that
                             % was generated from Forward Induction
                             payTime = swapFloatingCouponSchedule.payTime(idxFloat);
                             payoffStateA.cashflow = swapFloatingCoupon(idxFloat,:);

                             cachedModelStatesSwapFloatingV = cachedModelStatesSwapFloating(numOfFactors*(idxFloat-1)+1:numOfFactors*idxFloat,:);
                             
                          %% since we are paying floating CMS Coupon
                             % we multiply -1 with floating coupon before we
                             % process payoff
                             payoffStateA.cashflow = payoffStateA.cashflow *-1.0;
                             
                             eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                             payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;

                             % at coupon EventDate AddFunctor
                             %process payoff cashflow 
                             nonCallB.ProcessAddPayoff(ck1F,currentTimeStep,payTime,numeraire,cachedModelStatesSwapFloatingV, ...
                                                   payoffStateA.cashflow,'swapFloatingCoupon',idxFloat);

                             callable.ProcessAddPayoff(ck1F,currentTimeStep,payTime,numeraire,cachedModelStatesSwapFloatingV, ...
                                                   payoffStateA.cashflow,'swapFloatingCoupon',idxFloat);

                       end

                     %% swapFloatingCoupon event end  
                  end
                 %% issuer callable event
                 %% AssignExercise Functor : applied only when eventTime >=0
                 %% if EODAdjust is YES : endOfDate conversion option has expired
                 %% if EODAdjust is NO : intraDay, and option is still alive
                   idxCall = find(currentTimeStep == callableSchedule.eventTime);
                   if ~isempty(idxCall)
                       if isExercised 
                           
                            payTime = callableSchedule.payTime(idxCall);
                            
                            % future payment <- is this necessary ?
                            if payTime >= 0
                                
                               %find the corresponding callPayment;
                               callPayment = callableSchedule.callPayment(idxCall);
                               %look up the corresponding cachedModelStatesCall for
                               %call decision variables
                               cachedModelStatesCallV = cachedModelStatesCall(numOfFactors*(idxCall-1)+1:numOfFactors*idxCall,:);

                               % exercised by notional amount
                               exerciseValue = callPayment*nominal*ones(1,modelStatesSize);
                               
                               % discount exerciseValues from payTime to eventTime
                               df_ep = ck1F.discountFactorRisky(currentTimeStep, payTime,numMatTime,cachedModelStatesCallV);
                               for k=1:modelStatesSize
                                   exerciseValue(k) = exerciseValue(k) * df_ep(k);
                               end
                               
                               outCallablePayoff = callable.ProcessAssignPayoff(ck1F,currentTimeStep,payTime,numeraire,cachedModelStatesCallV, ...
                                     exerciseValue);
                                
                            end
                       else
                           
                           if currentTimeStep >=0 && EODAdjust(EODFlag_Exercise,currentTimeStep)

                               %find the corresponding callPayment;
                               callPayment = callableSchedule.callPayment(idxCall);
                               %look up the corresponding cachedModelStatesCall for
                               %call decision variables
                               cachedModelStatesCallV = cachedModelStatesCall(numOfFactors*(idxCall-1)+1:numOfFactors*idxCall,:);

                               exerciseValue = zeros(1,modelStatesSize);
                               exerciseITMCondition = zeros(1,modelStatesSize);

                               for idxEx=1:modelStatesSize
                                   exerciseValue(idxEx) = callPayment*nominal; 
                                   exerciseITMCondition(idxEx) = 1;
                               end

                               payTime = callableSchedule.payTime(idxCall);

                               % discount exerciseValues from payTime to eventTime
                               df_ep = ck1F.discountFactorRisky(currentTimeStep, payTime,numMatTime,cachedModelStatesCallV);
                               for k=1:modelStatesSize
                                   exerciseValue(k) = exerciseValue(k) * df_ep(k);
                               end

                               % callable is minimizing the payoff
                               minMaxFlag   = 'MIN';
                               decisionVar1 =  cachedModelStatesCallV;
                               outCallablePayoff = AssignExercise(ck1F,currentTimeStep,callable,exerciseValue,exerciseITMCondition,...
                                          minMaxFlag,decisionVar1,nominal,numeraire,cachedModelStatesCallV);

                           end
                       
                       end
                   end
                   
                 %% coupon event
                  if strcmp(structuredCouponSchedule.eventType,'END')
                       idxCoupon = find(currentTimeStep == structuredCouponSchedule.eventTime);
                       % if currentTime  is couponSchedule.eventTime
                       if ~isempty(idxCoupon)

                           if currentTimeStep == structuredCouponSchedule.eventTime(idxCoupon)
                                 % we use cached(saved) floating Coupon that
                                 % was generated from Forward Induction
                                 payTime = structuredCouponSchedule.payTime(idxCoupon);

                                 if idxCoupon == structuredCouponSchedule.scheduleSize
                                    payoffStateA.cashflow = nominal*0.0 + inverseFloaterCoupon(idxCoupon,:);
                                 else
                                    payoffStateA.cashflow = inverseFloaterCoupon(idxCoupon,:);
                                 end

                                 eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                                 payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;

                                 cachedModelStatesCouponV = cachedModelStatesCoupon(numOfFactors*(idxCoupon-1)+1:numOfFactors*idxCoupon,:);

                                 % at coupon EventDate AddFunctor
                                 %process payoff cashflow 
                                 nonCallB.ProcessAddPayoff(ck1F,currentTimeStep,payTime,numeraire,cachedModelStatesCouponV, ...
                                                       payoffStateA.cashflow,'inverseFloaterCoupon',idxCoupon);

                                 callable.ProcessAddPayoff(ck1F,currentTimeStep,payTime,numeraire,cachedModelStatesCouponV, ...
                                                       payoffStateA.cashflow,'inverseFloaterCoupon',idxCoupon);

                           end
                       end
                  end
                 
                  if strcmp(swapFloatingCouponSchedule.eventType,'END')  
                    %% if swapFloatingCoupon event happens at endDate 
                    %% backward induction it should be positioned
                    %% after call event

                     %% swapFloatingCoupon event start  
                       idxFloat = find(currentTimeStep == swapFloatingCouponSchedule.eventTime);
                       % if currentTime  is floatingCouponSchedule.eventTime
                       if ~isempty(idxFloat)

                             % we use cached(saved) floatingCoupon that
                             % was generated from Forward Induction
                             payTime = swapFloatingCouponSchedule.payTime(idxFloat);
                             payoffStateA.cashflow = swapFloatingCoupon(idxFloat,:);

                             cachedModelStatesSwapFloatingV = cachedModelStatesSwapFloating(numOfFactors*(idxFloat-1)+1:numOfFactors*idxFloat,:);

                          %% since we are paying floating CMS Coupon
                             % we multiply -1 with floating coupon before we
                             % process payoff
                             payoffStateA.cashflow = payoffStateA.cashflow *-1.0;
                             
                             eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                             payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;

                             % at coupon EventDate AddFunctor
                             %process payoff cashflow 
                             nonCallB.ProcessAddPayoff(ck1F,currentTimeStep,payTime,numeraire,cachedModelStatesSwapFloatingV, ...
                                                   payoffStateA.cashflow,'swapFloatingCoupon',idxFloat);

                             callable.ProcessAddPayoff(ck1F,currentTimeStep,payTime,numeraire,cachedModelStatesSwapFloatingV, ...
                                                   payoffStateA.cashflow,'swapFloatingCoupon',idxFloat);

                       end

                     %% swapFloatingCoupon event end 
                  end
                 
                   
               catch exception
                    throw(exception)
               end    
           end
           
           
          %% Finalize output
           nonCall.npv = mean(nonCall.payoff);
           nonCallB.npv = mean(nonCallB.payoff);
           callable.npv = mean(callable.payoff);
       
           %forward
           out.nonCall = nonCall; 
           %backward
           out.nonCallB = nonCallB; 
           out.callable = callable;
           
           out.eventTimeStep = eventTimeStep';
           
        end
        
        
        function out = computeInverseFloaterSwapGeneric3_LSMC(ck1F,valueDate,steepenerParams,numMethod)
%           
            % model schedule generation
            dealStartDate = steepenerParams.startDate;
            dealStartEventTime = DateDiff(dealStartDate,valueDate);
            expiryDate = steepenerParams.endDate;
            dealEndEventTime = DateDiff(expiryDate,valueDate);
            %% structuredCouponSchedule start
            
            structuredCouponScheduleInt = steepenerParams.params.structuredCouponInfo.structuredCouponScheduleInt;
            
            % structuredCouponSchedule is a schedule of Structured Coupon
            % events
            
            % main backbone schedule
            structuredCouponScheduleSize = size(structuredCouponScheduleInt,1);
            structuredCouponSchedule= struct;
            structuredCouponSchedule.scheduleSize = structuredCouponScheduleSize;
            structuredCouponSchedule.eventType = steepenerParams.params.structuredCouponInfo.eventType;
            % coupon payment event happens at endDate
            % eventTime is endDate
            % payment date can be different from endDate
            
            % startTime,endTimeP should be included in timeStep Generation
            % with eventTime
            
            for i=1:structuredCouponSchedule.scheduleSize
                couponResetDate = H_Date(structuredCouponScheduleInt(i,1));
                couponStartDate = H_Date(structuredCouponScheduleInt(i,2));
                couponEndDate = H_Date(structuredCouponScheduleInt(i,3));
                couponPayDate = H_Date(structuredCouponScheduleInt(i,4));
                
                structuredCouponSchedule.dayCount(i) = couponEndDate.DateDiff(couponStartDate);
                structuredCouponSchedule.dayCountFraction(i) = structuredCouponSchedule.dayCount(i)/365.0; 
                
                couponEndDateP = couponEndDate.AddDate( -1 ,'day');
                
                structuredCouponSchedule.resetTime(i) = DateDiff(couponResetDate,valueDate);
                structuredCouponSchedule.startTime(i) = DateDiff(couponStartDate,valueDate);
                structuredCouponSchedule.endTimeP(i)  = DateDiff(couponEndDateP,valueDate);
                structuredCouponSchedule.endTime(i) = DateDiff(couponEndDate,valueDate);
                
                if strcmp(structuredCouponSchedule.eventType,'RESET')
                    structuredCouponSchedule.eventTime(i) = DateDiff(couponResetDate,valueDate);
                elseif strcmp(structuredCouponSchedule.eventType,'START')
                    structuredCouponSchedule.eventTime(i) = DateDiff(couponStartDate,valueDate);
                elseif strcmp(structuredCouponSchedule.eventType,'END')
                    structuredCouponSchedule.eventTime(i) = DateDiff(couponEndDate,valueDate);
                else % default is END : event happens at EndDate for StructuredCoupon
                    structuredCouponSchedule.eventTime(i) = DateDiff(couponEndDate,valueDate);
                end
                structuredCouponSchedule.payTime(i) = DateDiff(couponPayDate,valueDate);
                
            end
            
            
            tenorInverseFloater     = steepenerParams.params.structuredCouponInfo.tenorInverseFloater;
            
           %% rateType for structuredCoupon : default CMS
            if isfield(steepenerParams.params.structuredCouponInfo, 'rateType_tenorInverseFloater')
                rateType_tenorInverseFloater = steepenerParams.params.structuredCouponInfo.rateType_tenorInverseFloater;
            else
                rateType_tenorInverseFloater = 'CMS';
            end
            
            
            localFloor = steepenerParams.params.structuredCouponInfo.localFloor;
            localCap   = steepenerParams.params.structuredCouponInfo.localCap;
            multiplier = steepenerParams.params.structuredCouponInfo.multiplier;
            spread     = steepenerParams.params.structuredCouponInfo.spread;
            coupon     = steepenerParams.params.structuredCouponInfo.coupon;
           %% structuredCoupon schedule end
           
           %% call exercise event happens at the callExerciseDate
           %% callable schedule start
            callableScheduleInt = steepenerParams.params.callableInfo.callableScheduleInt;
            callableScheduleSize = size(callableScheduleInt,1);
            callableSchedule= struct;
            for i=1:callableScheduleSize
                callExerciseDate = H_Date(callableScheduleInt(i,1));
                callPayDate = H_Date(callableScheduleInt(i,2));
                callableSchedule.eventTime(i) = DateDiff(callExerciseDate,valueDate);
                callableSchedule.payTime(i) = DateDiff(callPayDate,valueDate);
                
                 callableSchedule.callPayment(i) = callableScheduleInt(i,3);
            end
            callableSchedule.scheduleSize = callableScheduleSize;
           
            isExercised = steepenerParams.params.callableInfo.isExercised;
           %% callable schedule end
           
          %% swapFloatingCouponSchedule start
            
            swapFloatingCouponScheduleInt = steepenerParams.params.swapFloatingCouponInfo.swapFloatingCouponScheduleInt;
            
            swapFloatingCouponSchedule= struct;
            swapFloatingCouponSchedule.scheduleSize = size(swapFloatingCouponScheduleInt,1);
            
            swapFloatingCouponSchedule.eventType = steepenerParams.params.swapFloatingCouponInfo.eventType;
            
            % floating coupon payment event happens at resetDate
            % eventTime is resetDate
            
            % resetTime,startTime,endTimeP should be included in timeStep Generation
            % with eventTime
            
            for i=1:swapFloatingCouponSchedule.scheduleSize
                flolatingCouponResetDate = H_Date(swapFloatingCouponScheduleInt(i,1));
                flolatingCouponStartDate = H_Date(swapFloatingCouponScheduleInt(i,2));
                flolatingCouponEndDate = H_Date(swapFloatingCouponScheduleInt(i,3));
                flolatingCouponPayDate = H_Date(swapFloatingCouponScheduleInt(i,4));
                
                swapFloatingCouponSchedule.dayCount(i) = flolatingCouponEndDate.DateDiff(flolatingCouponStartDate);
                swapFloatingCouponSchedule.dayCountFraction(i) = swapFloatingCouponSchedule.dayCount(i)/365.0; 
                
                flolatingCouponEndDateP = flolatingCouponEndDate.AddDate( -1 ,'day');
                
                swapFloatingCouponSchedule.resetTime(i) = DateDiff(flolatingCouponResetDate,valueDate);
                swapFloatingCouponSchedule.startTime(i) = DateDiff(flolatingCouponStartDate,valueDate);
                swapFloatingCouponSchedule.endTimeP(i)  = DateDiff(flolatingCouponEndDateP,valueDate);
                swapFloatingCouponSchedule.endTime(i) = DateDiff(flolatingCouponEndDate,valueDate);
                
                if strcmp(swapFloatingCouponSchedule.eventType,'RESET')
                    swapFloatingCouponSchedule.eventTime(i) = DateDiff(flolatingCouponResetDate,valueDate);
                elseif strcmp(swapFloatingCouponSchedule.eventType,'START')
                    swapFloatingCouponSchedule.eventTime(i) = DateDiff(flolatingCouponStartDate,valueDate);
                elseif strcmp(swapFloatingCouponSchedule.eventType,'END')
                    swapFloatingCouponSchedule.eventTime(i) = DateDiff(flolatingCouponEndDate,valueDate);
                else % default is RESET : for SwapFloatingCouponSchedule
                    swapFloatingCouponSchedule.eventTime(i) = DateDiff(flolatingCouponResetDate,valueDate);
                end
                
%                 swapFloatingCouponSchedule.eventTime(i) = DateDiff(flolatingCouponResetDate,valueDate);
                
                swapFloatingCouponSchedule.payTime(i) = DateDiff(flolatingCouponPayDate,valueDate);
                
            end
            
            swapFloating_tenor  = steepenerParams.params.swapFloatingCouponInfo.tenorFloat;
            
            %% rateType for swapFloatingCoupon : default LIBOR
            if isfield(steepenerParams.params.swapFloatingCouponInfo, 'rateType_tenorFloat')
                rateType_tenorFloat = steepenerParams.params.swapFloatingCouponInfo.rateType_tenorFloat;
            else
                rateType_tenorFloat = 'LIBOR';
            end
            
            swapFloating_multiplier = steepenerParams.params.swapFloatingCouponInfo.multiplier;
            swapFloating_spread     = steepenerParams.params.swapFloatingCouponInfo.spread;
           
          %% SwapFloatingCouponSchedule end
            
            
          %% pricing time Step Generation start
            
            % eventTimeStep Generate
            mcOneTimeStep = numMethod.mcOneTimeStep;
            
          %% eventTime from dealStartTime and dealEndTime
           eventTimeStep = dealStartEventTime;
           eventTimeStep = union(eventTimeStep,dealEndEventTime);
           
          %% eventTime From structuredCouponSchedule:eventTime(endTime) +
          %% startTime,endTimeP
          %% daily weekly timeStep for couponSchedule
          
          %% InverseFloater
          %% eventTime From structuredCouponSchedule:eventTime
            eventTimeStep = union(eventTimeStep, structuredCouponSchedule.resetTime);
            eventTimeStep = union(eventTimeStep,structuredCouponSchedule.eventTime);
%             eventTimeStep = union(eventTimeStep, structuredCouponSchedule.startTime);
            eventTimeStep = union(eventTimeStep, structuredCouponSchedule.endTime);
%             eventTimeStep = union(eventTimeStep, structuredCouponSchedule.endTimeP);
           %% eventTime From CallSchedule:eventTime
            eventTimeStep = union(eventTimeStep,callableSchedule.eventTime);
            eventTimeStep = sort(eventTimeStep,'ascend');
            
          %% eventTime From SwapFloatingCouponSchedule:eventTime
            eventTimeStep = union(eventTimeStep,swapFloatingCouponSchedule.resetTime);  
            eventTimeStep = union(eventTimeStep,swapFloatingCouponSchedule.eventTime);
            eventTimeStep = sort(eventTimeStep,'ascend');
            
            
            if dealStartEventTime < 0 
                dailyTimeSteps = [0:-1:dealStartEventTime];
                eventTimeStep = union(eventTimeStep,dailyTimeSteps);
            end
          
          %% Daily Weekly Time Step Generation for AllTheTime ScheduleType, FromTo Schedule end
            
            eventTimeStep = sort(eventTimeStep,'ascend');
            
          %% aliveTimeStep for mc path generation
            aliveTimeStep = eventTimeStep(eventTimeStep>=0);
            aliveTimeStep = sort(aliveTimeStep,'ascend');
            
            totalAliveTimeStepSize = length(aliveTimeStep);
            
           %% eventTimeStep Generation End
           
           %% numeraire info
            numMatTime = eventTimeStep(end);
            dfNumMat = ck1F.DFRisky(numMatTime/365.0);

            numeraire.numMatTime = numMatTime;
            numeraire.dfNumMat = dfNumMat;
              
            % should we use eventTimeIdx as future only time stes?
            % In this case we better use more generic approach
            % so that all times steps are included 
            
          %% pricing time Step Generation end
           
          %% MC init
            rng('default');
            rng(0);
            
            numOfFactors = ck1F.numOfFactors;
            NMC = numMethod.pathSize; 
            MCTimeStep = totalAliveTimeStepSize -1; % nowDate (0) should be skipped
%             Z= randn(NMC/2,MCTimeStep);
            Z= randn(NMC/2,numOfFactors*MCTimeStep);
            Z=[Z;-Z];
            Z=Z';

            %TimeInversion
            sizeZ = size(Z,1);
            A = zeros(sizeZ,sizeZ);
            for i=1:sizeZ
                A(i,sizeZ + 1 -i) = 1;
            end
            Z = A*Z;

%                 U = Z';
            U = Z;

           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = zeros(numOfFactors,NMC);
           
           nextX = initX;
            
          %% payoff init
           nominal = steepenerParams.params.nominal;

           % EODFlag : if true then pricing as EndOfDate(cashflow on
           % valueDate wil be not included(already paid)
           % if false then pricing as Intraday ( cashflow on valueDate will 
           % be included)
           
           %EODFlag_Cashflow: Cashflow EOD Flag
           %EODFlag_Exercise: Exercise(Option) EOD Flag
           EODFlag_Cashflow = steepenerParams.params.EODFlag_Cashflow;
           EODFlag_Exercise = steepenerParams.params.EODFlag_Exercise;
           
           
           modelStatesSize = NMC;
           
          %% Define Pricing Columns
           % pricingColumns : nonCall,Callable  
           % we need inverseFloaterCoupon as state variables for storing 
           % inverse Floater Coupon
           
           % for the cashflow generated pricing columns as pricerInfo
           % we do use pricerInfo Structure
           
         %% (global) state variable initialize  : these correspond to ColumnVal[t] in generic pricing deal description
           
          % avg steeper coupon
           inverseFloaterCoupon = zeros(structuredCouponSchedule.scheduleSize,modelStatesSize);
           prefixedInverseFloatRate = zeros(structuredCouponSchedule.scheduleSize,modelStatesSize);
           
           swapFloatingCoupon = zeros(swapFloatingCouponSchedule.scheduleSize,modelStatesSize);
           prefixedSwapFloatingRate = zeros(swapFloatingCouponSchedule.scheduleSize,modelStatesSize);
           
           cachedModelStatesCoupon = zeros(structuredCouponSchedule.scheduleSize*numOfFactors,modelStatesSize);
  
          % for regression (2 modelState variables)
           cachedModelStatesCall = zeros(callableSchedule.scheduleSize*numOfFactors,modelStatesSize);
           
          % for swapFloatin 
           cachedModelStatesSwapFloating = zeros(swapFloatingCouponSchedule.scheduleSize*numOfFactors,modelStatesSize);
           
          %% PricingColumnGenericCashflow
          %% a structure that has cashflows with cashflowNames
          %% for CMS AVG structured we have nonCall, nonCallB, callable
           
            %nonCall
            cashflowNames = {'inverseFloaterCoupon','swapFloatingCoupon'};
            cashflowSizeMap = containers.Map({'inverseFloaterCoupon','swapFloatingCoupon'},{structuredCouponSchedule.scheduleSize, ...
                                            swapFloatingCouponSchedule.scheduleSize});
            
            cashflowInfo.cashflowNames = cashflowNames;
            cashflowInfo.cashflowSizeMap = cashflowSizeMap;
            cashflowInfo.pathSize = modelStatesSize;
            
           nonCall = PricingColumnInfoGenericCashflow(cashflowInfo);
           nonCallB = PricingColumnInfoGenericCashflow(cashflowInfo);
           callable = PricingColumnInfoGenericCashflow(cashflowInfo);
           
           
           % intermediate variables
           payoffStateA.cashflow = zeros(1,modelStatesSize);
           payoffStateA.cashflow_npv = zeros(1,modelStatesSize);
           
           % we don't need local variable for inverse floater
           
%            % variable to use accruedSumOfStockPrice for moving average calculation 
%            
%            cD = 0; % cumulated day count
%            cP = 0; 
%            dI = 0; % dummy idx for loop
%            dcfC = 0;
%            dcfN = 0;
%            dD = 0;
%       
%            idxL = zeros(1,modelStatesSize);
%            idxS = zeros(1,modelStatesSize);
% 
%            idxC = zeros(1,modelStatesSize);
%            idxN = zeros(1,modelStatesSize);
% 
%            spreadC = zeros(1,modelStatesSize);
%            spreadN = zeros(1,modelStatesSize);
%            probSpread = zeros(1,pathSize);
           
            % we use all path for regression
            % conversion ITM , puttable ITM ,  callable ITM
            % what's the use of Alive Path only Regression Sampling ?
           
          %% Path Generation Forward
          %% Store path dependent variables for coupon calculation 
          %% Store state variables to be used for AMC Regression
           
           %payoff descrption update index
           currentTimeIdx = 1;
           %MC numMethodState update index
           aliveTimeIdx = 1;
           % currentTimeStep is update in for statement
%            for currentTimeStep = eventTimeStep(1):1:eventTimeStep(end)
           for idxT = 1:1:length(eventTimeStep)
                currentTimeStep = eventTimeStep(idxT);
                
                if idxT < length(eventTimeStep)
                    nextTimeStep = eventTimeStep(idxT+1);
                end
                % evaluate scripts 
                % evaluate stateVariables(stores)
                % evaluate expression per each pricing columns
                % at the end of the loop we induct forward
                % from left to right
                % fictional payoff script columns are
                % cumCoupon,nonCall,Callable
                
                        
                % find the current
                % if find return errors then?
                try
                    
                    % cachedModelStateCoupon for modelState
                    idxCouponEvent = find(currentTimeStep == structuredCouponSchedule.eventTime);
                    if ~isempty(idxCouponEvent)
                        cachedModelStatesCoupon(numOfFactors*(idxCouponEvent-1)+1:numOfFactors*idxCouponEvent,:) = nextX;
                    end
                    
                    % cachedModelStateCall for modelState
                    idxCallEvent = find(currentTimeStep == callableSchedule.eventTime);
                    if ~isempty(idxCallEvent)
                        cachedModelStatesCall(numOfFactors*(idxCallEvent-1)+1:numOfFactors*idxCallEvent,:) = nextX;
                    end
                    
                    % cachedModelStateSwapFloatingEvent for modelState
                    idxSwapFloatingEvent = find(currentTimeStep == swapFloatingCouponSchedule.eventTime);
                    if ~isempty(idxSwapFloatingEvent)
                        cachedModelStatesSwapFloating(numOfFactors*(idxSwapFloatingEvent-1)+1:numOfFactors*idxSwapFloatingEvent,:) = nextX;
                    end
                 
                 %% now if we are at structuredCouponSchedule eventTime
                   
                 %% structuredCouponSchedule event start
                 %% if inverseFloaterCoupon event happens at RESET Date
                 %% then we fix floatingCoupon at resetDate and we raise event at resetdate
                 %% if inverseFloaterCoupon event happens at END date then we reset inverseFloatRate first
                 %% at ResetDate and we refer this rate at EventDate(EndDate)
                    if strcmp(structuredCouponSchedule.eventType,'RESET')
                     %% now if we are at structuredCouponSchedule eventTime
                     %% structuredCouponSchedule event start

                        idx = find(currentTimeStep == structuredCouponSchedule.eventTime);
                        if ~isempty(idx)

                            inverseFloatRate = ck1F.Rate_IndexGeneric(rateType_tenorInverseFloater,valueDate,currentTimeStep,numMatTime,tenorInverseFloater,nextX);
                            
                        %% inverseFloaterCoupon
                            
                            inverseFloaterCoupon(idx,:) =  max(min(coupon -  multiplier * inverseFloatRate,localCap),localFloor) * structuredCouponSchedule.dayCountFraction(idx) * nominal;
                            
                            payoffStateA.cashflow = inverseFloaterCoupon(idx,:);

                            eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                            payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;
                            payTime = structuredCouponSchedule.payTime(idx);

                            %process payoff cashflow 
                            nonCall.ProcessAddPayoff(ck1F,currentTimeStep,payTime,numeraire,nextX, ...
                                                    payoffStateA.cashflow,'inverseFloaterCoupon',idx);

                        end
                      %% structuredCouponSchedule event end
                       
                   elseif strcmp(structuredCouponSchedule.eventType,'END')
                     %% if we raise inverseFloaterCouponEvent at EndDate then
                     %% we need to raise fixing event at resetDate and store fixed inverseFloatRate as stateVariables
                     %% and reference it at ENDDate
                        idxInverseFloatReset = find(currentTimeStep == structuredCouponSchedule.resetTime);
                        if ~isempty(idxInverseFloatReset)
                            
                            inverseFloatRate = ck1F.Rate_IndexGeneric(rateType_tenorInverseFloater,valueDate,currentTimeStep,numMatTime,tenorInverseFloater,nextX);
                            prefixedInverseFloatRate(idxInverseFloatReset,:) = inverseFloatRate;
                        end
                        
                        idxEnd = find(currentTimeStep == structuredCouponSchedule.eventTime);
                        if ~isempty(idxEnd)
                            prefixedFloatRate = prefixedInverseFloatRate(idxEnd,:);
                            inverseFloaterCoupon(idxEnd,:) =  max(min(coupon -  multiplier * prefixedFloatRate,localCap),localFloor) * structuredCouponSchedule.dayCountFraction(idxEnd) * nominal;

                            payoffStateA.cashflow = inverseFloaterCoupon(idxEnd,:);
                            
                            eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                            payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;

                            payTime = structuredCouponSchedule.payTime(idxEnd);

                            %process payoff cashflow 
                            nonCall.ProcessAddPayoff(ck1F,currentTimeStep,payTime,numeraire,nextX, ...
                                                    payoffStateA.cashflow,'inverseFloaterCoupon',idxEnd);
                            
                        end
                 %% swapFloatingCouponSchedule event end
                   else
                       error('not supported eventType')
                    end
                    
                 %% structuredCouponSchedule event end
                    
                 %% now if we are at swapFloatingCouponSchedule eventTime
                    
                 %% swapFloatingCouponSchedule event start
                 %% if swapFloatingCoupon event happens at RESET Date
                 %% then we fix floatingCoupon at resetDate and we raise event at resetdate
                 %% if swapFloatingCoupon event happens at END date
                 %% then we fix floatingCoupon at resetDate and we raise event at endDate
                 
                   if strcmp(swapFloatingCouponSchedule.eventType,'RESET')
                     %% now if we are at swapFloatingCouponSchedule eventTime
                     %% swapFloatingCouponSchedule event start

                        idxFloat = find(currentTimeStep == swapFloatingCouponSchedule.eventTime);
                        if ~isempty(idxFloat)

                            floatRate = ck1F.Rate_IndexGeneric(rateType_tenorFloat,valueDate,currentTimeStep,numMatTime,swapFloating_tenor,nextX);
                            
                            swapFloatingCoupon(idxFloat,:) =  (swapFloating_multiplier * floatRate + swapFloating_spread) * swapFloatingCouponSchedule.dayCountFraction(idxFloat) * nominal;

                            payoffStateA.cashflow = swapFloatingCoupon(idxFloat,:);

                         %% since we are paying floating CMS Coupon
                             % we multiply -1 with floating coupon before we
                             % process payoff
                            payoffStateA.cashflow = payoffStateA.cashflow * -1.0;

                            eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                            payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;
                            
                            payTime = swapFloatingCouponSchedule.payTime(idxFloat);

                            %process payoff cashflow 
                            nonCall.ProcessAddPayoff(ck1F,currentTimeStep,payTime,numeraire,nextX, ...
                                                    payoffStateA.cashflow,'swapFloatingCoupon',idxFloat);

                        end
                      %% swapFloatingCouponSchedule event end
                       
                   elseif strcmp(swapFloatingCouponSchedule.eventType,'END')
                     %% if we raise swapFloatingCouponEvent at EndDate then
                     %% we need to raise fixing event at resetDate and store fixed floatRate as stateVariables
                     %% and reference it at ENDDate
                        idxFloatReset = find(currentTimeStep == swapFloatingCouponSchedule.resetTime);
                        if ~isempty(idxFloatReset)
                            
                            floatRate = ck1F.Rate_IndexGeneric(rateType_tenorFloat,valueDate,currentTimeStep,numMatTime,swapFloating_tenor,nextX);
                            
                            prefixedSwapFloatingRate(idxFloatReset,:) = floatRate;
                        end
                        
                        idxEnd = find(currentTimeStep == swapFloatingCouponSchedule.eventTime);
                        if ~isempty(idxEnd)
                            % now if we are at swapFloatingCouponSchedule
                            % eventTime(=endDate)
                            % then we update swapFloatingCoupon
                            prefixedFloatRate = prefixedSwapFloatingRate(idxEnd,:);
                            swapFloatingCoupon(idxEnd,:) =  (swapFloating_multiplier * prefixedFloatRate + swapFloating_spread) * swapFloatingCouponSchedule.dayCountFraction(idxEnd) * nominal;

                            payoffStateA.cashflow = swapFloatingCoupon(idxEnd,:);

                         %% since we are paying floating CMS Coupon
                             % we multiply -1 with floating coupon before we
                             % process payoff
                            payoffStateA.cashflow = payoffStateA.cashflow * -1.0;
                            
                            eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                            payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;

                            payTime = swapFloatingCouponSchedule.payTime(idxEnd);

                            %process payoff cashflow 
                            nonCall.ProcessAddPayoff(ck1F,currentTimeStep,payTime,numeraire,nextX, ...
                                                    payoffStateA.cashflow,'swapFloatingCoupon',idxEnd);
                            
                        end
                 %% swapFloatingCouponSchedule event end
                   else
                       error('not supported eventType')
                   end
                   
                   % if currentTimeSteps is today or future then inductMCForward
                    if currentTimeStep >= 0 && currentTimeStep < eventTimeStep(end)
                        mcmcOut   = inductMCForwardNew(ck1F,currentTimeStep,nextTimeStep,...
                                    nextX,U(numOfFactors*(aliveTimeIdx-1)+1:numOfFactors*aliveTimeIdx,:));
                        nextX = mcmcOut.nextX;
                        %MC numMethodState update
                        aliveTimeIdx = aliveTimeIdx + 1;
                    end
                    
                    currentTimeIdx = currentTimeIdx +1;
                catch exception
                    throw(exception)
                end
                
                
           end
           
          %% Forward Induction End
          
           % backward pricingColumn
           % nonCallB,
           % callable
           
           
          %% Backward induction start
           for idxT=length(eventTimeStep):-1:1
               
               currentTimeStep = eventTimeStep(idxT);
               try
                  if strcmp(structuredCouponSchedule.eventType,'RESET') 
                    %% if swapFloatingCoupon event happens at resetDate so, in
                    %% backward induction it should be positioned
                    %% before call event

                     %% structuredCouponSchedule event start  
                       idxCoupon = find(currentTimeStep == structuredCouponSchedule.eventTime);
                       % if currentTime  is structuredCouponSchedule.eventTime
                       if ~isempty(idxCoupon)

                             % we use cached(saved) floatingCoupon that
                             % was generated from Forward Induction
                             payTime = structuredCouponSchedule.payTime(idxCoupon);
                             payoffStateA.cashflow = inverseFloaterCoupon(idxCoupon,:);

                             cachedModelStatesCouponV = cachedModelStatesCoupon(numOfFactors*(idxCoupon-1)+1:numOfFactors*idxCoupon,:);
                             
                             eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                             payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;

                             % at coupon EventDate AddFunctor
                             %process payoff cashflow 
                             nonCallB.ProcessAddPayoff(ck1F,currentTimeStep,payTime,numeraire,cachedModelStatesCouponV, ...
                                                   payoffStateA.cashflow,'inverseFloaterCoupon',idxCoupon);

                             callable.ProcessAddPayoff(ck1F,currentTimeStep,payTime,numeraire,cachedModelStatesCouponV, ...
                                                   payoffStateA.cashflow,'inverseFloaterCoupon',idxCoupon);

                       end

                     %% structuredCouponSchedule event end  
                  end
                  
                  if strcmp(swapFloatingCouponSchedule.eventType,'RESET') 
                    %% if swapFloatingCoupon event happens at resetDate so, in
                    %% backward induction it should be positioned
                    %% before call event

                     %% swapFloatingCoupon event start  
                       idxFloat = find(currentTimeStep == swapFloatingCouponSchedule.eventTime);
                       % if currentTime  is floatingCouponSchedule.eventTime
                       if ~isempty(idxFloat)

                             % we use cached(saved) floatingCoupon that
                             % was generated from Forward Induction
                             payTime = swapFloatingCouponSchedule.payTime(idxFloat);
                             payoffStateA.cashflow = swapFloatingCoupon(idxFloat,:);

                             cachedModelStatesSwapFloatingV = cachedModelStatesSwapFloating(numOfFactors*(idxFloat-1)+1:numOfFactors*idxFloat,:);
                             
                          %% since we are paying floating CMS Coupon
                             % we multiply -1 with floating coupon before we
                             % process payoff
                             payoffStateA.cashflow = payoffStateA.cashflow *-1.0;
                             
                             eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                             payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;

                             % at coupon EventDate AddFunctor
                             %process payoff cashflow 
                             nonCallB.ProcessAddPayoff(ck1F,currentTimeStep,payTime,numeraire,cachedModelStatesSwapFloatingV, ...
                                                   payoffStateA.cashflow,'swapFloatingCoupon',idxFloat);

                             callable.ProcessAddPayoff(ck1F,currentTimeStep,payTime,numeraire,cachedModelStatesSwapFloatingV, ...
                                                   payoffStateA.cashflow,'swapFloatingCoupon',idxFloat);

                       end

                     %% swapFloatingCoupon event end  
                  end
                 %% issuer callable event
                 %% AssignExercise Functor : applied only when eventTime >=0
                 %% if EODAdjust is YES : endOfDate conversion option has expired
                 %% if EODAdjust is NO : intraDay, and option is still alive
                   idxCall = find(currentTimeStep == callableSchedule.eventTime);
                   if ~isempty(idxCall)
                       if isExercised 
                           
                            payTime = callableSchedule.payTime(idxCall);
                            
                            % future payment <- is this necessary ?
                            if payTime >= 0
                                
                               %find the corresponding callPayment;
                               callPayment = callableSchedule.callPayment(idxCall);
                               %look up the corresponding cachedModelStatesCall for
                               %call decision variables
                               cachedModelStatesCallV = cachedModelStatesCall(numOfFactors*(idxCall-1)+1:numOfFactors*idxCall,:);

                               % exercised by notional amount
                               exerciseValue = callPayment*nominal*ones(1,modelStatesSize);
                               
                               % discount exerciseValues from payTime to eventTime
                               df_ep = ck1F.discountFactorRisky(currentTimeStep, payTime,numMatTime,cachedModelStatesCallV);
                               for k=1:modelStatesSize
                                   exerciseValue(k) = exerciseValue(k) * df_ep(k);
                               end
                               
                               outCallablePayoff = callable.ProcessAssignPayoff(ck1F,currentTimeStep,payTime,numeraire,cachedModelStatesCallV, ...
                                     exerciseValue);
                                
                            end
                       else
                           
                           if currentTimeStep >=0 && EODAdjust(EODFlag_Exercise,currentTimeStep)

                               %find the corresponding callPayment;
                               callPayment = callableSchedule.callPayment(idxCall);
                               %look up the corresponding cachedModelStatesCall for
                               %call decision variables
                               cachedModelStatesCallV = cachedModelStatesCall(numOfFactors*(idxCall-1)+1:numOfFactors*idxCall,:);

                               exerciseValue = zeros(1,modelStatesSize);
                               exerciseITMCondition = zeros(1,modelStatesSize);

                               for idxEx=1:modelStatesSize
                                   exerciseValue(idxEx) = callPayment*nominal; 
                                   exerciseITMCondition(idxEx) = 1;
                               end

                               payTime = callableSchedule.payTime(idxCall);

                               % discount exerciseValues from payTime to eventTime
                               df_ep = ck1F.discountFactorRisky(currentTimeStep, payTime,numMatTime,cachedModelStatesCallV);
                               for k=1:modelStatesSize
                                   exerciseValue(k) = exerciseValue(k) * df_ep(k);
                               end

                               % callable is minimizing the payoff
                               minMaxFlag   = 'MIN';
                               decisionVar1 =  cachedModelStatesCallV;
                               
%                                outCallablePayoff = AssignExercise(ck1F,currentTimeStep,callable,exerciseValue,exerciseITMCondition,...
%                                           minMaxFlag,decisionVar1,nominal,numeraire,cachedModelStatesCallV);
                                      
                               outCallablePayoff = AssignExercise_LSMC_3(ck1F,currentTimeStep,callable,exerciseValue,exerciseITMCondition,...
                                          minMaxFlag,decisionVar1,nominal,numeraire,cachedModelStatesCallV);

                           end
                       
                       end
                   end
                   
                 %% coupon event
                  if strcmp(structuredCouponSchedule.eventType,'END')
                       idxCoupon = find(currentTimeStep == structuredCouponSchedule.eventTime);
                       % if currentTime  is couponSchedule.eventTime
                       if ~isempty(idxCoupon)

                           if currentTimeStep == structuredCouponSchedule.eventTime(idxCoupon)
                                 % we use cached(saved) floating Coupon that
                                 % was generated from Forward Induction
                                 payTime = structuredCouponSchedule.payTime(idxCoupon);

                                 if idxCoupon == structuredCouponSchedule.scheduleSize
                                    payoffStateA.cashflow = nominal*0.0 + inverseFloaterCoupon(idxCoupon,:);
                                 else
                                    payoffStateA.cashflow = inverseFloaterCoupon(idxCoupon,:);
                                 end

                                 eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                                 payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;

                                 cachedModelStatesCouponV = cachedModelStatesCoupon(numOfFactors*(idxCoupon-1)+1:numOfFactors*idxCoupon,:);

                                 % at coupon EventDate AddFunctor
                                 %process payoff cashflow 
                                 nonCallB.ProcessAddPayoff(ck1F,currentTimeStep,payTime,numeraire,cachedModelStatesCouponV, ...
                                                       payoffStateA.cashflow,'inverseFloaterCoupon',idxCoupon);

                                 callable.ProcessAddPayoff(ck1F,currentTimeStep,payTime,numeraire,cachedModelStatesCouponV, ...
                                                       payoffStateA.cashflow,'inverseFloaterCoupon',idxCoupon);

                           end
                       end
                  end
                 
                  if strcmp(swapFloatingCouponSchedule.eventType,'END')  
                    %% if swapFloatingCoupon event happens at endDate 
                    %% backward induction it should be positioned
                    %% after call event

                     %% swapFloatingCoupon event start  
                       idxFloat = find(currentTimeStep == swapFloatingCouponSchedule.eventTime);
                       % if currentTime  is floatingCouponSchedule.eventTime
                       if ~isempty(idxFloat)

                             % we use cached(saved) floatingCoupon that
                             % was generated from Forward Induction
                             payTime = swapFloatingCouponSchedule.payTime(idxFloat);
                             payoffStateA.cashflow = swapFloatingCoupon(idxFloat,:);

                             cachedModelStatesSwapFloatingV = cachedModelStatesSwapFloating(numOfFactors*(idxFloat-1)+1:numOfFactors*idxFloat,:);

                          %% since we are paying floating CMS Coupon
                             % we multiply -1 with floating coupon before we
                             % process payoff
                             payoffStateA.cashflow = payoffStateA.cashflow *-1.0;
                             
                             eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                             payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;

                             % at coupon EventDate AddFunctor
                             %process payoff cashflow 
                             nonCallB.ProcessAddPayoff(ck1F,currentTimeStep,payTime,numeraire,cachedModelStatesSwapFloatingV, ...
                                                   payoffStateA.cashflow,'swapFloatingCoupon',idxFloat);

                             callable.ProcessAddPayoff(ck1F,currentTimeStep,payTime,numeraire,cachedModelStatesSwapFloatingV, ...
                                                   payoffStateA.cashflow,'swapFloatingCoupon',idxFloat);

                       end

                     %% swapFloatingCoupon event end 
                  end
                 
                   
               catch exception
                    throw(exception)
               end    
           end
           
           
          %% Finalize output
           nonCall.npv = mean(nonCall.payoff);
           nonCallB.npv = mean(nonCallB.payoff);
           callable.npv = mean(callable.payoff);
       
           %forward
           out.nonCall = nonCall; 
           %backward
           out.nonCallB = nonCallB; 
           out.callable = callable;
           
           out.eventTimeStep = eventTimeStep';
           
        end
        
        function out = CalibrateToATMDiagSwaptionNormal(ck1F,black,calibrationFlag)
          % Market Data Object , Vanilla Object
          
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
            
            expirySize = size(expiry,2);
            tenorSize = size(tenor,2);
            
            blackSwaptionPrice = zeros(expirySize,tenorSize);

            for k=1:expirySize
                for j=1:tenorSize
                    blackSwaptionPrice(k,j) = black.NormalATMSwaption(expiry(k),tenor(j));
               end
            end
            
            swaptionVolSurface.expiry = expiry;
            swaptionVolSurface.tenor = tenor;
            swaptionVolSurface.swaptionVol = swaptionVol; % volSurface with interpolator
            
          %% select the interpolated target diagonal swaption with expiry &
          %% tenor & linear interpolated vol
            % size for interpolated yearly call (swaption)
            
            target_size = ck1F.targetSize;

            params.target_size = target_size;
            params.matu = ck1F.matu;
            params.freq = ck1F.freq;
            params.swaption_expiry_use_yn = zeros(target_size,1);
            
            params.vol_r.tenor = 1:1:target_size;
            params.vol_r.tenor = params.vol_r.tenor';
            params.vol_r.quote = 0.01*ones(target_size,1);
            
            params.diagSwaption.expiry = zeros(target_size,1);
            params.diagSwaption.tenor  = zeros(target_size,1);
            params.diagSwaption.quote  = zeros(target_size,target_size);
            params.diagSwaption.strike = zeros(target_size,1);
            params.diagSwaption.blackPrice = zeros(target_size,1);
            
            if ck1F.matu <= 2.0
                params.swaption_expiry_use_yn(1) = 1;
                params.diagSwaption.expiry =  ones(1,1);
                params.diagSwaption.expiry(1) =  swaptionVolSurface.expiry(1);
                params.diagSwaption.tenor =  ones(1,1);
                params.diagSwaption.tenor(1) =  swaptionVolSurface.tenor(1);
                params.diagSwaption.quote = ones(1,1);
                params.diagSwaption.quote(1,1) = swaptionVolSurface.swaptionVol(expiry(1),tenor(1));
            else
                temp_tenor   = zeros(target_size,1);
                temp_expiry = zeros(target_size,1);
                for i=2:target_size
                    temp_tenor(i-1)  = params.vol_r.tenor(target_size - (i-1));
                    temp_expiry(i-1) = params.matu - temp_tenor(i-1); 
                end
                
                upper_expiry = swaptionVolSurface.expiry(end);
                upper_tenor  = swaptionVolSurface.tenor(end);
                
                lower_expiry = swaptionVolSurface.expiry(1);
                
                temp_idx= 1;
                for i=1:target_size
                    for j=temp_idx:target_size
                        % only those swaption whose expiry date is away.
                        if temp_expiry(j) > 0
                            params.vol_r.tenor(i) = temp_expiry(j);
                            params.diagSwaption.expiry(i) = temp_expiry(j);
                            params.diagSwaption.tenor(i)  = temp_tenor(j);
                            if(temp_expiry(j) > upper_expiry && temp_tenor(j) <= upper_tenor)
                                params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(upper_expiry,temp_tenor(j));
                            elseif(temp_expiry(j) <= upper_expiry && temp_tenor(j) > upper_tenor)
                                params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(temp_expiry(j),upper_tenor);    
                            else
                                if temp_expiry(j) < lower_expiry
                                    params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(lower_expiry,temp_tenor(j));
                                else
                                    params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(temp_expiry(j),temp_tenor(j));
                                end
                            end
                            
                            params.swaption_expiry_use_yn(i) = 1;
                            temp_idx = j+1;
                            break;
                        end
                    end
                    
                    if i>=2
                        params.vol_r.tenor(i) = params.vol_r.tenor(i-1) + 1;
                    end
                end
                
            end
            
            % set the new model parameters in accordance with the target
            % instruments maturities
            
            ck1F.vol_r    = params.vol_r;
            tau = 1.0/(ck1F.freq);
          %% target swaption price and forward swap rate
            for i=1:target_size
                if params.swaption_expiry_use_yn(i)
                    Ti = params.diagSwaption.expiry(i);
                    Tj = params.diagSwaption.tenor(i);
                    blackVolQuote = params.diagSwaption.quote(i,i);
                    params.diagSwaption.strike(i) = black.Fsr(Ti,Ti + Tj,tau);
%                     params.diagSwaption.blackPrice(i) = black.BlackSwaptionWithVol(Ti,Tj,params.diagSwaption.strike(i),blackVolQuote);
                    params.diagSwaption.blackPrice(i) = black.NormalSwaptionWithVol(Ti,Tj,params.diagSwaption.strike(i),blackVolQuote);
                    
                end
            end
            
           %% target swaption price and forward swap rate
            % tracking calibration
            
            out.blackSwaptionPrice = params.diagSwaption.blackPrice;
            out.ck1FSwaptionPrice = zeros(target_size,1);
            out.priceRelDiff = zeros(target_size,1);
            out.priceRmseTotal = 0.0;
            out.vol_r = params.vol_r;
            
            out.market_swaption = zeros(target_size,1);
            out.model_swaption = zeros(target_size,1);
            out.re = zeros(target_size,1);
            out.rmseTotal = 0;
            
            tvar = [];
            calibrationType = 'global';
            if nargin < 3
                calibrationType = 'global'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;
            targetSwaptionSize = 0;
            switch calibrationType
                case 'global'
                    
                    optParams.params = params;
                    
                    for j=1:length(ck1F.vol_r.quote)
                        if params.swaption_expiry_use_yn(j) == 1
                            targetSwaptionSize = targetSwaptionSize +1;
                        end
                    end
                    
                    optParams.vol_r_size = targetSwaptionSize;
                    
                    optParams.modelParamsSize = optParams.vol_r_size;
                    
                    if ck1F.matu <= 2.0
                        optParams.numOfEquation = targetSwaptionSize; % # of target swaption instruments
                    else
                        optParams.numOfEquation = targetSwaptionSize; % # of target swaption instruments
                    end
                    
                    out.modelParamsSize = optParams.modelParamsSize;
                    out.numOfEquation = optParams.numOfEquation;
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    lb = zeros(optParams.modelParamsSize,1);
                    ub = zeros(optParams.modelParamsSize,1);
                    
                    for j=1:optParams.vol_r_size
                        tvar(j) = params.vol_r.quote(j);
                        lb(j)= 0.0001;
                        ub(j)= 10;
                    end
                       
%                     targetfun = @(tvar) TargetFunctionck1FATMSwaptionDiagGlobal(ck1F,tvar,optParams);
%                     options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
                    x=  zeros(optParams.numOfEquation,1);
                    [ret, popt, info, covar]=levmar('TargetFunctionCK1FATMSwaptionNormalDiagGlobal',tvar,x, 200, options,'unc',ck1F,optParams);
                    
                    
                    % assigning model parameters
                    k=1;
                    for j=1:optParams.vol_r_size
                       ck1F.vol_r.quote(j) = max(popt(j),0.0001);
                       k=k+1;
                    end
                    % flat extrapolation of Alpha
                    % AlphaSize != length(Alpha)
                    
                    for j=k:length(ck1F.vol_r.quote)
                       ck1F.vol_r.quote(j) = ck1F.vol_r.quote(optParams.vol_r_size);
                    end
                    
                    out.vol_r.quote = ck1F.vol_r.quote;
                    
                    N=0;
                    for i=1:params.target_size
                        if params.swaption_expiry_use_yn(i) == 1
                            Ti = params.diagSwaption.expiry(i);
                            Tj = params.diagSwaption.tenor(i);
                            out.market_swaption(i) = params.diagSwaption.blackPrice(i);
                            out.model_swaption(i)  = ck1F.ATMSwaption(Ti,Tj);
                            out.re(i) = out.model_swaption(i)/out.market_swaption(i) - 1.0;
                            out.rmseTotal = out.rmseTotal + out.re(i)*out.re(i);
                            N=N+1; 
                        end
                    end
                    out.rmseTotal = sqrt(out.rmseTotal/N);
               
                otherwise
                    disp('unImplemented')
            end
        end
        
        function out = CalibrateToATMDiagSwaption(ck1F,black,calibrationFlag)
          % Market Data Object , Vanilla Object
          
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
            
            expirySize = size(expiry,2);
            tenorSize = size(tenor,2);
            
            blackSwaptionPrice = zeros(expirySize,tenorSize);

            for k=1:expirySize
                for j=1:tenorSize
                    blackSwaptionPrice(k,j) = black.BlackATMSwaption(expiry(k),tenor(j));
               end
            end
            
            swaptionVolSurface.expiry = expiry;
            swaptionVolSurface.tenor = tenor;
            swaptionVolSurface.swaptionVol = swaptionVol; % volSurface with interpolator
            
          %% select the interpolated target diagonal swaption with expiry &
          %% tenor & linear interpolated vol
            % size for interpolated yearly call (swaption)
            
            target_size = ck1F.targetSize;

            params.target_size = target_size;
            params.matu = ck1F.matu;
            params.freq = ck1F.freq;
            params.swaption_expiry_use_yn = zeros(target_size,1);
            
            params.vol_r.tenor = 1:1:target_size;
            params.vol_r.tenor = params.vol_r.tenor';
            params.vol_r.quote = 0.01*ones(target_size,1);
            
            params.diagSwaption.expiry = zeros(target_size,1);
            params.diagSwaption.tenor  = zeros(target_size,1);
            params.diagSwaption.quote  = zeros(target_size,target_size);
            params.diagSwaption.strike = zeros(target_size,1);
            params.diagSwaption.blackPrice = zeros(target_size,1);
            
            if ck1F.matu <= 2.0
                params.swaption_expiry_use_yn(1) = 1;
                params.diagSwaption.expiry =  ones(1,1);
                params.diagSwaption.expiry(1) =  swaptionVolSurface.expiry(1);
                params.diagSwaption.tenor =  ones(1,1);
                params.diagSwaption.tenor(1) =  swaptionVolSurface.tenor(1);
                params.diagSwaption.quote = ones(1,1);
                params.diagSwaption.quote(1,1) = swaptionVolSurface.swaptionVol(expiry(1),tenor(1));
            else
                temp_tenor   = zeros(target_size,1);
                temp_expiry = zeros(target_size,1);
                for i=2:target_size
                    temp_tenor(i-1)  = params.vol_r.tenor(target_size - (i-1));
                    temp_expiry(i-1) = params.matu - temp_tenor(i-1); 
                end
                
                upper_expiry = swaptionVolSurface.expiry(end);
                upper_tenor  = swaptionVolSurface.tenor(end);
                
                lower_expiry = swaptionVolSurface.expiry(1);
                
                temp_idx= 1;
                for i=1:target_size
                    for j=temp_idx:target_size
                        % only those swaption whose expiry date is away.
                        if temp_expiry(j) > 0
                            params.vol_r.tenor(i) = temp_expiry(j);
                            params.diagSwaption.expiry(i) = temp_expiry(j);
                            params.diagSwaption.tenor(i)  = temp_tenor(j);
                            if(temp_expiry(j) > upper_expiry && temp_tenor(j) <= upper_tenor)
                                params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(upper_expiry,temp_tenor(j));
                            elseif(temp_expiry(j) <= upper_expiry && temp_tenor(j) > upper_tenor)
                                params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(temp_expiry(j),upper_tenor);    
                            else
                                if temp_expiry(j) < lower_expiry
                                    params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(lower_expiry,temp_tenor(j));
                                else
                                    params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(temp_expiry(j),temp_tenor(j));
                                end
                            end
                            
                            params.swaption_expiry_use_yn(i) = 1;
                            temp_idx = j+1;
                            break;
                        end
                    end
                    
                    if i>=2
                        params.vol_r.tenor(i) = params.vol_r.tenor(i-1) + 1;
                    end
                end
                
            end
            
            % set the new model parameters in accordance with the target
            % instruments maturities
            
            ck1F.vol_r    = params.vol_r;
            tau = 1.0/(ck1F.freq);
          %% target swaption price and forward swap rate
            for i=1:target_size
                if params.swaption_expiry_use_yn(i)
                    Ti = params.diagSwaption.expiry(i);
                    Tj = params.diagSwaption.tenor(i);
                    blackVolQuote = params.diagSwaption.quote(i,i);
                    params.diagSwaption.strike(i) = black.Fsr(Ti,Ti + Tj,tau);
%                     params.diagSwaption.blackPrice(i) = black.BlackSwaptionWithVol(Ti,Tj,params.diagSwaption.strike(i),blackVolQuote);
                    params.diagSwaption.blackPrice(i) = black.BlackSwaptionWithVolTau(Ti,Tj,params.diagSwaption.strike(i),blackVolQuote,tau);
                    
                end
            end
            
           %% target swaption price and forward swap rate
            % tracking calibration
            
            out.blackSwaptionPrice = params.diagSwaption.blackPrice;
            out.ck1FSwaptionPrice = zeros(target_size,1);
            out.priceRelDiff = zeros(target_size,1);
            out.priceRmseTotal = 0.0;
            out.vol_r = params.vol_r;
            
            out.market_swaption = zeros(target_size,1);
            out.model_swaption = zeros(target_size,1);
            out.re = zeros(target_size,1);
            out.rmseTotal = 0;
            
            tvar = [];
            calibrationType = 'global';
            if nargin < 3
                calibrationType = 'global'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;
            targetSwaptionSize = 0;
            switch calibrationType
                case 'global'
                    
                    optParams.params = params;
                    
                    for j=1:length(ck1F.vol_r.quote)
                        if params.swaption_expiry_use_yn(j) == 1
                            targetSwaptionSize = targetSwaptionSize +1;
                        end
                    end
                    
                    optParams.vol_r_size = targetSwaptionSize;
                    
                    optParams.modelParamsSize = optParams.vol_r_size;
                    
                    if ck1F.matu <= 2.0
                        optParams.numOfEquation = targetSwaptionSize; % # of target swaption instruments
                    else
                        optParams.numOfEquation = targetSwaptionSize; % # of target swaption instruments
                    end
                    
                    out.modelParamsSize = optParams.modelParamsSize;
                    out.numOfEquation = optParams.numOfEquation;
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    lb = zeros(optParams.modelParamsSize,1);
                    ub = zeros(optParams.modelParamsSize,1);
                    
                    for j=1:optParams.vol_r_size
                        tvar(j) = params.vol_r.quote(j);
                        lb(j)= 0.0001;
                        ub(j)= 10;
                    end
                       
%                     targetfun = @(tvar) TargetFunctionck1FATMSwaptionDiagGlobal(ck1F,tvar,optParams);
%                     options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
                    x=  zeros(optParams.numOfEquation,1);
                    [ret, popt, info, covar]=levmar('TargetFunctionCK1FATMSwaptionDiagGlobal',tvar,x, 200, options,'unc',ck1F,optParams);
                    
                    
                    % assigning model parameters
                    k=1;
                    for j=1:optParams.vol_r_size
                       ck1F.vol_r.quote(j) = max(popt(j),0.0001);
                       k=k+1;
                    end
                    % flat extrapolation of Alpha
                    % AlphaSize != length(Alpha)
                    
                    for j=k:length(ck1F.vol_r.quote)
                       ck1F.vol_r.quote(j) = ck1F.vol_r.quote(optParams.vol_r_size);
                    end
                    
                    out.vol_r.quote = ck1F.vol_r.quote;
                    
                    N=0;
                    for i=1:params.target_size
                        if params.swaption_expiry_use_yn(i) == 1
                            Ti = params.diagSwaption.expiry(i);
                            Tj = params.diagSwaption.tenor(i);
                            out.market_swaption(i) = params.diagSwaption.blackPrice(i);
                            out.model_swaption(i)  = ck1F.ATMSwaption(Ti,Tj);
                            out.re(i) = out.model_swaption(i)/out.market_swaption(i) - 1.0;
                            out.rmseTotal = out.rmseTotal + out.re(i)*out.re(i);
                            N=N+1; 
                        end
                    end
                    out.rmseTotal = sqrt(out.rmseTotal/N);
               
                otherwise
                    disp('unImplemented')
            end
        end

        function out = TargetFunctionATMSwaptionGlobal(ck1F,tvar,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            ck1FSwaptionPrice = zeros(1,size(tenor,2));
            
            for i=1:length(ck1F.vol_r.quote)
                ck1F.vol_r.quote(i) = tvar(i);
            end
            out= 0;
            
            for j=1:length(expiry)
                for k=1:length(tenor)
                    ck1FSwaptionPrice(j,k) = ck1F.ATMSwaption(expiry(j),tenor(k));
                    out = out + (ck1FSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0)*(ck1FSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0)*10000;
                end
            end
        end   
        
        function out = TargetFunctionATMSwaption(ck1F,idxNow,tvar,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            ck1FSwaptionPrice = zeros(1,size(tenor,2));
            
            ck1F.vol_r.quote(idxNow) = tvar(1);
            
            out= 0;
            for j=1:size(tenor,2)
                ck1FSwaptionPrice(j) = ck1F.ATMSwaption(expiry(idxNow),tenor(j));
                out = out + (ck1FSwaptionPrice(j)/blackSwaptionPrice(idxNow,j)-1.0)*(ck1FSwaptionPrice(j)/blackSwaptionPrice(idxNow,j)-1.0)*10000;
            end
            
        end        
        
%         function out = CalibrateToATMSwaption(ck1F,black,calibrationFlag)
%             out.vol_r = ck1F.vol_r;
%             % Calculate Black ATM Swaption Prices
%             swaptionVol = black.swaptionVol;
%             swaptionVolData = black.mktData('swaptionVol');
%             swaptionVolrawData =  swaptionVolData.params('rawData');
%             
%             expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
%             tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
%             
%             expirySize = size(expiry,2);
%             tenorSize = size(tenor,2);
%             
%             blackSwaptionPrice = zeros(expirySize,tenorSize);
%             for k=1:expirySize
%                 for j=1:tenorSize
%                     blackSwaptionPrice(k,j) = black.BlackATMSwaption(expiry(k),tenor(j));
%                 end
%             end
%             
%             params.expiry = expiry;
%             params.tenor = tenor;
%             params.blackSwaptionPrice = blackSwaptionPrice;
%             params.expirySize = expirySize;
%             params.tenorSize =  tenorSize;
%             
%             out.blackSwaptionPrice = blackSwaptionPrice;
%             out.ck1FSwaptionPrice = zeros(expirySize,tenorSize);
%             out.relDiff = zeros(expirySize,tenorSize);
%             out.rmseTotal=0.0;
%             
%             calibrationType = 'bootstrap';
%             if nargin < 3
%                 calibrationType = 'bootstrap'; 
%             else
%                 calibrationType = calibrationFlag;
%             end
%             N=0;
%             switch calibrationType
%                 case 'bootstrap'
%                     for k=1:expirySize
%                         if k ~= 1
%                             ck1F.vol_r.quote(k) = ck1F.vol_r.quote(k-1);
%                             ck1F.vol_s.quote(k) = ck1F.vol_s.quote(k-1);
%                             ck1F.corr_rs.quote(k) = ck1F.corr_rs.quote(k-1);
%                         end    
%                         tvar = zeros(1,3);
%                         tvar(1) = ck1F.vol_r.quote(k);
%                         tvar(2) = ck1F.vol_s.quote(k);
%                         tvar(3) = ck1F.corr_rs.quote(k);
%                         
%                         lb(1) = 0.0001;
%                         lb(2) = 0.0001;
%                         ub(1) = 10.0;
%                         ub(2) = 10.0;
%                         
%                         lb(3) = -0.99;
%                         ub(3) = 0.0;
%                         
%                         targetfun = @(tvar) Targetck1FATMSwaption(ck1F,k,tvar,params);
% 
%                         options=optimset('Algorithm','interior-point');    
%                         [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);
% 
%                         out.vol_r.quote(k) = x(1);
%                         out.vol_s.quote(k) = x(2);
%                         out.corr_rs.quote(k) = x(3);
%                         
%                         ck1F.vol_r.quote(k) = x(1);
%                         ck1F.vol_s.quote(k) = x(2);
%                         ck1F.corr_rs.quote(k) = x(3);
%                         
%                         for j=1:tenorSize
%                             out.ck1FSwaptionPrice(k,j) = ck1F.ATMSwaption(expiry(k),tenor(j));
%                             out.relDiff(k,j) = out.ck1FSwaptionPrice(k,j) /out.blackSwaptionPrice(k,j)-1;
%                             out.rmseTotal = out.rmseTotal + out.relDiff(k,j)*out.relDiff(k,j);
%                             N=N+1;
%                         end
%                     end
%                     out.rmseTotal = sqrt(out.rmseTotal/N);
%                     out.vol_r = ck1F.vol_r;
%                     out.vol_s = ck1F.vol_s;
%                     out.corr_rs = ck1F.corr_rs;
%                     out.ck1F = ck1F;
%                 case 'global'
%                     for j=1:length(ck1F.vol_r.quote)
%                         tvar(j) = ck1F.vol_r.quote(j);
%                         tvar(length(ck1F.vol_r.quote)+j) = ck1F.vol_s.quote(j);
%                         lb(j)= 0.0001;
%                         lb(length(ck1F.vol_r.quote)+j)= 0.0001;
%                         ub(j)= 10;
%                         ub(length(ck1F.vol_r.quote)+j)= 10;
%                     end
%                     
%                     outSize = params.expirySize*params.tenorSize; 
%                     
%                     if 1
%                         x =  zeros(outSize,1);
%                         options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                         tic
%                         [ret, popt, info, covar]=levmar('TargetFunctionck1FATMSwaptionSurfaceGlobal',tvar,x, 200, options,'unc',ck1F,params);
%                         toc
%                         for j=1:length(ck1F.vol_r.quote)
%                             out.vol_r.quote(j) = popt(j);
%                             out.vol_s.quote(j) = popt(j+length(ck1F.vol_r.quote));
%                             ck1F.vol_r.quote(j) = popt(j);
%                             ck1F.vol_s.quote(j) = popt(j+length(ck1F.vol_s.quote));
%                         end
%                     else
%                         targetfun = @(tvar) TargetFunctionATMSwaptionGlobal(ck1F,tvar,params);
%                         options=optimset('Algorithm','interior-point');
%     %                     options=optimset('Algorithm','trust-region-reflective');
%                         [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);
% 
%                         for j=1:length(ck1F.vol_r.quote)
%                             out.vol_r.quote(j) = x(j);
%                             out.vol_s.quote(j) = x(j+length(ck1F.vol_r.quote));
%                             ck1F.vol_r.quote(j) = x(j);
%                             ck1F.vol_s.quote(j) = x(j+length(ck1F.vol_s.quote));
%                         end
%                     end
%                     N=0;
%                     for j=1:expirySize
%                         for k=1:tenorSize
%                             out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
%                             out.ck1FSwaptionPrice(j,k) = ck1F.ATMSwaption(expiry(j),tenor(k));
%                             out.relDiff(j,k) = out.ck1FSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
%                             out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
%                             N=N+1;
%                         end
%                     end
%                    
%                     out.rmseTotal = sqrt(out.rmseTotal/N);
%                 otherwise
%                     disp('unImplemented')
%             end
%         end    
    end
    
end



In [ ]:
classdef CK1F < IRModel
    %UNTITLED Summary of this class goes here
    %   Detailed explanation goes here
    
    properties
        % vol functor
        vol_r
        meanR_r
        matu
        joint_swap_tenor
        targetSize
        freq % 
        numOfFactors
        numOfFactor
        modelType
        
        instrumentStartDate
        instrumentEndDate
        calibrationTargetTenor
    end
    
    methods
        function ck1F = CK1F(params)
            if nargin > 0
                
                ck1F.zeroCurve = params.zeroCurve;
                
                ck1F.mktData = params.mktData;
                ck1F.modelParams = params.modelParams;
                
                ck1F.vol_r = params.modelParams('vol_r');
                ck1F.meanR_r =  params.modelParams('meanR_r');
                
                % make matu and targetSize as optional
%                 ck1F.matu =  params.modelParams('matu');
%                 ck1F.targetSize = params.modelParams('targetSize');
                if isKey(params.modelParams,'matu')
                    ck1F.matu = params.modelParams('matu');
                else
                    ck1F.matu = 15.0; % default
                end
                
                
                if isKey(params.modelParams,'targetSize')
                    ck1F.targetSize = params.modelParams('targetSize');
                else
                    ck1F.targetSize = 15; % default
                end
                
                
                
                ck1F.joint_swap_tenor =  params.modelParams('joint_swap_tenor');
                ck1F.freq  =params.modelParams('freq');
                
                 % make instrument specific info as calibration input
                 
                if isKey(params.modelParams,'instrumentStartDate')
                    ck1F.instrumentStartDate = params.modelParams('instrumentStartDate');
                else
                    % default as same as evaluationDate
                    ck1F.instrumentStartDate = H_Date(ck1F.zeroCurve.asOfDate); % default
                end
                
                if isKey(params.modelParams,'instrumentEndDate')
                    ck1F.instrumentEndDate = params.modelParams('instrumentEndDate');
                else
                    % default as instrumentStartDate + matu
%                     ck1F.instrumentEndDate = ck1F.instrumentStartDate.AddDate(ck1F.matu,'year'); % default
                     % default as instrumentStartDate + matu
                    ck1F.instrumentEndDate = ck1F.instrumentStartDate.AddDate(round(ck1F.matu*365),'year'); % default
                end
                
                if isKey(params.modelParams,'calibrationTargetTenor')
                    ck1F.calibrationTargetTenor = params.modelParams('calibrationTargetTenor');
                else
                    % default as 1,2,3,...,14
                    ck1F.calibrationTargetTenor = 1:1:14; % default
                end
                
                ck1F.numOfFactors = 1;
                ck1F.numOfFactor = 1;
                ck1F.modelType = 'CK1F';
                
            end
           
        end
        
        function out = SimpleSRPrOverHedge(ck1F,spreadC,spreadN,ralower,raupper)
            buffer = 0.001;
            outVec = 0.0;
            if spreadC < ralower - buffer
                outVec = 0.0;
            elseif spreadC >= ralower - buffer && spreadC < ralower
                outVec = 1.0/buffer * (spreadC - (ralower - buffer));
            elseif spreadC >= ralower && spreadC < raupper
                outVec = 1.0;
            elseif spreadC >= raupper && spreadC < raupper + buffer
                outVec = -1.0/buffer *(spreadC - (raupper + buffer));
            else
                outVec = 0.0;
            end
            
            out = outVec;
        end
        
        function out = SimpleSRPr(ck1F,spreadC,spreadN,ralower,raupper)
            outVec = 0.0;
            if spreadC >= ralower && spreadC <= raupper
                outVec = 1.0;
            end
            out = outVec;
        end
        
        
        function out = TRZ_SRPr(ck1F,curr,next,L,U)
            length = 1.0;
            remain = 1.0;
            if (curr >= L && curr <= U && next >= L && next <= U)
                length = remain;
            elseif ( curr >= L && curr <= U && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr >= L && curr <= U && next < L ) 
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr > U && next >= L && next <= U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next < L )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next >= L && next <= U )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next > U )
                length = 0.0;
            elseif ( curr < L && next < L)
                length = 0.0;
            else
                disp('error')
            end
            
            out = length;
        end
        
        function out = SRPr(ck1F,spreadC,spreadN,ralower,raupper,simpleYN,pathSize)
            
            outVec = zeros(1,pathSize);
            
            if simpleYN == 1
                for i =1:pathSize
                    outVec(i) = ck1F.SimpleSRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            elseif simpleYN == 2
                for i =1:pathSize
                    outVec(i) = ck1F.SimpleSRPrOverHedge(spreadC(i),spreadN(i),ralower,raupper);
                end
            else
                for i =1:pathSize
                    outVec(i) = ck1F.TRZ_SRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            end
            
            out = outVec;
        end
        
        function out =  CMS_PSA_SimpleDate(ck1F,valueDate,observeTime,numMatTime, tenor,modelStatesT)
           %CMS
           tau = 0.25;
           to      = observeTime/365.0;
           toDate = valueDate.AddDate(observeTime,'day');
           tNDate = toDate.AddDate(tenor,'year');
           tN   =  tNDate.DateDiff(valueDate)/365.0;
           
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           p_nN = ck1F.PV01Date(valueDate,toDate,tNDate,tau);
           
           df_Tn = ck1F.DF(to);
           df_TN = ck1F.DF(tN);
           y_nN = (df_Tn - df_TN)/p_nN;
           
           c_nN = ck1F.C_nNDate(valueDate,toDate,tNDate,tau,p_nN,y_nN);
           g_nN = ck1F.G_nNDate(valueDate,toDate,tNDate,tau,p_nN);
           h_nTerm = ck1F.H_nTermDate(valueDate,toDate,numMatDate);
           varCovar = ck1F.LocalVariance(0,to);
           
           stateSize = size(modelStatesT,1);
           cms_PCA_SimpleOut = zeros(1,stateSize);
           
           driftTerm = 0;
           for i=1:1
               for j=1:1
                   driftTerm = driftTerm + (c_nN(i) * h_nTerm(j) + c_nN(i) * g_nN(j))*varCovar(i,j);
               end
           end
           
           for i=1:stateSize
               cms_PCA_SimpleOut(i) = y_nN + driftTerm + c_nN(1)*modelStatesT(1,i); 
           end
           
           out = cms_PCA_SimpleOut;
           
            
        end
        
        function out = HWGeneralVariance(ck1F,from,to,T1,T2,MRS1,MRS2,sigmaTimes,sigmaValue1,sigmaValue2,corrValue)
            
            sigmaSize = length(sigmaValue1);
            
            for i=1:length(sigmaTimes)
                if from<= sigmaTimes(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:length(sigmaTimes)
                if to<= sigmaTimes(i)
                    break;
                end
            end
            endIdx = i;
            
            
            newExpScale = zeros(4,1);
            lastExpScale = zeros(4,1);
            variances = zeros(4,1);
            
            lastU  = from;
            lastExpScale(2) = exp(MRS1*lastU);
            lastExpScale(3) = exp(MRS2*lastU);
            lastExpScale(4) = exp((MRS1 + MRS2)*lastU);
            
            for i=startIdx:endIdx
                if (i< endIdx) U = sigmaTimes(i);else U = to;end;
                if (i< sigmaSize) sig1 = sigmaValue1(i);else sig1 = sigmaValue1(sigmaSize);end;
                if (i< sigmaSize) sig2 = sigmaValue2(i);else sig2 = sigmaValue2(sigmaSize);end;
                if (i< sigmaSize) corr12 = corrValue(i);else corr12 = corrValue(sigmaSize);end;
                
                variances(1) = variances(1) + corr12 * sig1 * sig2 * (U - lastU);
                
                newExpScale(2) = exp(MRS1*U);
                variances(2) = variances(2) + corr12 * sig1 * sig2 * (1.0/MRS1) * (newExpScale(2)-lastExpScale(2));
                lastExpScale(2) = newExpScale(2);
                
                newExpScale(3) = exp(MRS2*U);
                variances(3) = variances(3) + corr12 * sig1 * sig2 * (1.0/MRS2) * (newExpScale(3)-lastExpScale(3));
                lastExpScale(3) = newExpScale(3);
                
                newExpScale(4) = exp((MRS1+MRS2)*U);
                variances(4) = variances(4) + corr12 * sig1 * sig2 * (1.0/(MRS1+MRS2)) * (newExpScale(4)-lastExpScale(4));
                lastExpScale(4) = newExpScale(4);
                
                lastU = U;
                
            end
            
            value = (1.0/ MRS1)*(1.0/MRS2)*variances(1);
            value = value + -1.0*exp(-1.0*MRS1*T1)*(1.0/ MRS1)*(1.0/MRS2)*variances(2);
            value = value + -1.0*exp(-1.0*MRS2*T2)*(1.0/ MRS1)*(1.0/MRS2)*variances(3);
            value = value +  1.0*exp(-1.0*MRS1*T1)*exp(-1.0*MRS2*T2)*(1.0/ MRS1)*(1.0/MRS2)*variances(4);

            out  = value;
        end
        
        function out = fwdZcVariance(ck1F, from, to, T1, T2)
            
            MRS_r = ck1F.meanR_r;
            sigma_r = ck1F.vol_r.quote;
            correl_1 = ones(length(sigma_r),1);
            sigmaTimes = ck1F.vol_r.tenor;
            localVariance = zeros(6,1);
            
            localVariance(1) = ck1F.HWGeneralVariance(from,to,T1,T1,MRS_r,MRS_r,sigmaTimes,sigma_r,sigma_r,correl_1);
            localVariance(2) = ck1F.HWGeneralVariance(from,to,to,to,MRS_r,MRS_r,sigmaTimes,sigma_r,sigma_r,correl_1);
            
            value = localVariance(1) - localVariance(2);
            
            measureChange = zeros(8,1);
            
            measureChange(1) = ck1F.HWGeneralVariance(from,to,T1,T2,MRS_r,MRS_r,sigmaTimes,sigma_r,sigma_r,correl_1);
            measureChange(2) = ck1F.HWGeneralVariance(from,to,to,T2,MRS_r,MRS_r,sigmaTimes,sigma_r,sigma_r,correl_1);
            
            measureChangeTerm =                     1.0 * (measureChange(1) - measureChange(2));
            
            value = value + -2.0*measureChangeTerm;
            
            out = value;
            
        end
        
        function out = discountFactor(ck1F, startTime, endTime,numMatTime,modelStatesT)
           zcT = ck1F.DF(endTime/365.0);
           zct = ck1F.DF(startTime/365.0);
           beta = ck1F.Beta(startTime/365.0,endTime/365.0);
           zcDriftTerm = ck1F.fwdZcVariance(0,startTime/365.0,endTime/365.0,numMatTime/365.0);
           %numOfPath
           stateSize = size(modelStatesT,2);
           discountFactorOut = zeros(1,stateSize);
           
           for i=1:stateSize
               discountFactorOut(i) = zcT/zct * exp(-0.5*zcDriftTerm - beta(1)*modelStatesT(1,i)); 
           end
           
           out = discountFactorOut;
        end
        
        function out = discountPayoff(ck1F,eventTime,numMatTime ...
                ,modelStatesT,cashflow)
            
            dfNumMat = ck1F.DF(numMatTime/365.0);
            % for discountPayoff T=numMatTime
            % P(0,TnumMat)/P(Te,Tnummat)*Payoff
            %
            discountT = ck1F.discountFactor(eventTime, numMatTime,numMatTime,modelStatesT);
            stateSize = size(modelStatesT,2);
            payoff = zeros(1,stateSize);
            for i=1:stateSize
                payoff(i)= dfNumMat/discountT(i)*cashflow(i);
            end    
            out = payoff;
        end
        
        function out = stateLocalDrift(ck1F,fromTime, toTime)
            drift = zeros(1,1);
            drift(1,1) =  exp(-1.0*ck1F.meanR_r*(toTime-fromTime)/365.0);
            out = drift;
        end
        
        function out = stateLocalVariance(ck1F,fromTime, toTime)
            out = ck1F.LocalVariance(fromTime/365.0,toTime/365.0);
        end
        
        function variances = LocalVariance(ck1F,from,to)
%             sig =[];
            sig(1) = ck1F.vol_r;
%             corr =[];
            corr(1,1) = 1.0;
            
%             meanR = [];
            meanR(1) = ck1F.meanR_r;
            
            alphaSize = length(ck1F.vol_r.quote);
            
            for i=1:length(sig(1).tenor)
                if from<= sig(1).tenor(i)
                    break;
                end
            end
            startIdx = i;
            
%             endIdx = size(sig(1).tenor,1); 
            for i=1:length(sig(1).tenor)
                if to<= sig(1).tenor(i)
                    break;
                end
            end
            endIdx = i;
            
%             tiny = {};
%             lastExpScale={};
%             newExpScale = {};
            for i=1:1
                for j=1:1
                    if(abs(meanR(i)+meanR(j)) < 1.0E-12) tiny(i,j) = 1; else tiny(i,j) = 0; end;
                    lastExpScale(i,j) =  exp((meanR(i)+meanR(j))*from);
                end
            end
            
            lastU  = from;
            
%             variances=[];
%             vol =[];
%             correl ={};
           variances = zeros(1,1);
           for i=startIdx:endIdx
                if (i< endIdx) U = sig(1).tenor(i);else U = to;end;
                if (i< alphaSize) vol(1) = sig(1).quote(i);else vol(1) = sig(1).quote(alphaSize);end;
                correl(1,1) =1.0;
                for j=1:1
                    for k=1:1
                        newExpScale(j,k) =  exp((meanR(j)+meanR(k))*U);
                        if(tiny(j,k)~=1)
                            scale =newExpScale(j,k)-lastExpScale(j,k);
                        else
                            scale = U - lastU;
                        end
                        variances(j,k) = variances(j,k) + vol(j)*vol(k)*correl(j,k)...
                                       * scale;
                        lastExpScale(j,k) = newExpScale(j,k);
                    end
                end
                lastU = U;
           end
           
            for j=1:1
                for k=1:1
                    if(tiny(j,k)~= 1)
                        variances(j,k) = variances(j,k)/(meanR(j)+meanR(k))*exp(-1.0*(meanR(j)+meanR(k))*to);
                    end
                end
            end
        end
        
        function out = sigma_Test(ck1F,t,sigma)
            
            sum = 0.0;
            prev_t = 0.0;
            inRangeYN = false;
            for i=1:length(sigma.tenor)
                alpha = sigma.quote(i);
                if t<= sigma.tenor(i)
                    sum = alpha;
                    inRangeYN = true;
                    break;
                end
            end
            
            if ~inRangeYN 
                sum = alpha;
            end
            out = sum;
        end

        function beta = Beta(ck1F,t,T)
            a_r = ck1F.meanR_r;
            beta =[];
            if(abs(a_r) > 1.0E-12)
                beta(1) = (1.0-exp(-a_r*(T-t)))/a_r;
            else
                beta(1) = (T-t);
            end
        end
        
        function beta = BetaDate(ck1F,valueDate,tDate,TDate)
            t = tDate.DateDiff(valueDate)/365.0;
            T = TDate.DateDiff(valueDate)/365.0;
            
            a_r = ck1F.meanR_r;
            beta =[];
            
            if(abs(a_r) > 1.0E-12)
                beta(1) = (1.0-exp(-a_r*(T-t)))/a_r;
            else
                beta(1) = (T-t);
            end
            
        end
        
        function c_nN = C_nN(ck1F,tn,tN,tau,pnN,ynN)
            p = @ck1F.DF;
            c_nN=[0.0];
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = ck1F.Beta(tn,tp);
                df = p(tp);
                c_nN(1) = c_nN(1) +tau * beta(1) * df;
            end
            
            c_nN(1) = c_nN(1)* ynN;
            
            c_nN(1) = c_nN(1) + beta(1)*df;
            
            c_nN(1) = c_nN(1)/pnN;
            
        end
        
        function c_nN = C_nNDate(ck1F,valueDate,tnDate,tNDate,tau,pnN,ynN)
            tn = tnDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            
            addMonthUnit = round(12*tau);
            cnt = round((tN-tn)/tau);
            
            p = @ck1F.DF;
            c_nN=[0.0];
            
            for i=1:cnt
                tpDate = tnDate.AddDate(i*addMonthUnit,'month');
                tp = tpDate.DateDiff(valueDate)/365.0;
                beta = ck1F.BetaDate(valueDate,tnDate,tpDate);
                df = p(tp);
                c_nN(1) = c_nN(1) +tau * beta(1) * df;
                
            end
            
            c_nN(1) = c_nN(1)* ynN;
            
            c_nN(1) = c_nN(1) + beta(1)*df;
            
            c_nN(1) = c_nN(1)/pnN;
            
        end
        
         function g_nN = G_nN(ck1F,tn,tN,tau,pnN)
            p = @ck1F.DF;
            g_nN=[0.0];
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = ck1F.Beta(tn,tp);
                df = p(tp);
                g_nN(1) = g_nN(1) +tau * beta(1) * df;
            end
            
            g_nN(1) = g_nN(1)/pnN;
            
         end
        
        function g_nN = G_nNDate(ck1F,valueDate,tnDate,tNDate,tau,pnN)
            tn = tnDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            addMonthUnit = round(12*tau);
            cnt = round((tN-tn)/tau);
            
            p = @ck1F.DF;
            g_nN=[0.0];
            
            for i=1:cnt
                tpDate = tnDate.AddDate(i*addMonthUnit,'month');
                tp = tpDate.DateDiff(valueDate)/365.0;
                beta = ck1F.BetaDate(valueDate,tnDate,tpDate);
                df = p(tp);
                g_nN(1) = g_nN(1) +tau * beta(1) * df;
                
            end
            
            g_nN(1) = g_nN(1)/pnN;
            
        end 
        
        function h_nTerm = H_nTerm(ck1F,tn,tTerm)
           h_nTerm =[];
           beta = ck1F.Beta(tn,tTerm);
           h_nTerm(1) = -1.0*beta(1);
           
        end
        
        function h_nTerm = H_nTermDate(ck1F,valueDate,tnDate,tTermDate)
           h_nTerm =[];
           beta = ck1F.BetaDate(valueDate,tnDate,tTermDate);
           h_nTerm(1) = -1.0*beta(1);
           
        end
        
        
        function out = CMSDigitalArg(ck1F,to,tp,tenor1,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           sizeN = 1;
           p_nN1 = ck1F.PV01(to,to+tenor1,tau);
           df_Tn1 = ck1F.DF(to);
           df_TN1 = ck1F.DF(to + tenor1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           c_nN1 = ck1F.C_nN(to,to + tenor1,tau,p_nN1,y_nN1);
           g_nN1 = ck1F.G_nN(to,to + tenor1,tau,p_nN1);
           h_nTerm1 = ck1F.H_nTerm(to,tp);
           
           varCovar = ck1F.LocalVariance(0,to);
           
           driftTerm1=0.0;
           for i=1:sizeN
               for j=1:sizeN
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance1 = 0.0;
           for i=1:sizeN
               for j=1:sizeN
                    spreadVariance1 = spreadVariance1 + c_nN1(i) * c_nN1(j) * varCovar(i,j);
               end
           end
           
           out.fwdSpread = y_nN1;
           out.mu = y_nN1 + driftTerm1;
           out.sigma = sqrt(spreadVariance1);
           out.c_nN1 = c_nN1;
           
        end
        
        function out = CMSDigital(ck1F,to,tp,tenor1,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsDigitalArg = ck1F.CMSDigitalArg(to,tp,tenor1,tau);
           mu = cmsDigitalArg.mu;
           sigma = cmsDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSDigitalRange(ck1F,to,tp,tenor1,tau,lower,upper)
            lower_value = ck1F.CMSDigital(to,tp,tenor1,tau,lower);
            upper_value = ck1F.CMSDigital(to,tp,tenor1,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CMSSpreadDigitalArg(ck1F,to,tp,tenor1,tenor2,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           p_nN1 = ck1F.PV01(to,to+tenor1,tau);
           df_Tn1 = ck1F.DF(to);
           df_TN1 = ck1F.DF(to + tenor1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           c_nN1 = ck1F.C_nN(to,to + tenor1,tau,p_nN1,y_nN1);
           g_nN1 = ck1F.G_nN(to,to + tenor1,tau,p_nN1);
           h_nTerm1 = ck1F.H_nTerm(to,tp);
           
           %CMS2
           p_nN2 = ck1F.PV01(to,to+tenor2,tau);
           df_Tn2 = ck1F.DF(to);
           df_TN2 = ck1F.DF(to + tenor2);
           y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
           c_nN2 = ck1F.C_nN(to,to + tenor2,tau,p_nN2,y_nN2);
           g_nN2 = ck1F.G_nN(to,to + tenor2,tau,p_nN2);
           h_nTerm2 = ck1F.H_nTerm(to,tp);
           
           varCovar = ck1F.LocalVariance(0,to);
           
           driftTerm1=0.0;
           driftTerm2=0.0;
           for i=1:1
               for j=1:1
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
                    driftTerm2 = driftTerm2 + c_nN2(i) * (h_nTerm2(j) + g_nN2(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           variance1 = 0.0;
           variance2 = 0.0;
           coVariance = 0.0;
           for i=1:1
               for j=1:1
                    spreadVariance = spreadVariance + (c_nN1(i)-c_nN2(i)) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
                    variance1 = variance1 + (c_nN1(i)) * (c_nN1(j)) * varCovar(i,j);
                    variance2 = variance2 + (c_nN2(i)) * (c_nN2(j)) * varCovar(i,j);
                    coVariance = coVariance + (c_nN1(i)) * (c_nN2(j)) * varCovar(i,j);
               end
           end
           
           out.fwdSpread = y_nN1 - y_nN2;
           out.mu = (y_nN1 + driftTerm1) - (y_nN2 + driftTerm2);
           out.pmu = driftTerm1 - driftTerm2;
           out.sigma = sqrt(spreadVariance);
           out.c_nN1 = c_nN1;
           out.c_nN2 = c_nN2;
           out.varCovar = varCovar;
           out.mu1 = y_nN1 + driftTerm1;
           out.mu2 = y_nN2 + driftTerm2;
           out.pmu1 = driftTerm1;
           out.pmu2 = driftTerm2;
           
           out.y1 = y_nN1;
           out.y2 = y_nN2;
           
           out.corr = coVariance/sqrt(variance1*variance2);
           out.sig1 = sqrt(variance1);
           out.sig2 = sqrt(variance2);
           
        end
        
        function out = CMSSpreadDigitalArgDate(ck1F,valueDate,toDate,tpDate,tenor1,tenor2,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           to      = toDate.DateDiff(valueDate)/365.0;
           tN1Date = toDate.AddDate(tenor1,'year');
           tN1   =  tN1Date.DateDiff(valueDate)/365.0;
           
           p_nN1 = ck1F.PV01Date(valueDate,toDate,tN1Date,tau);
           
           df_Tn1 = ck1F.DF(to);
           df_TN1 = ck1F.DF(tN1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           
           c_nN1 = ck1F.C_nNDate(valueDate,toDate,tN1Date,tau,p_nN1,y_nN1);
           g_nN1 = ck1F.G_nNDate(valueDate,toDate,tN1Date,tau,p_nN1);
           h_nTerm1 = ck1F.H_nTermDate(valueDate,toDate,tpDate);
           
           %CMS2
           tN2Date = toDate.AddDate(tenor2,'year');
           tN2   =  tN2Date.DateDiff(valueDate)/365.0;
           
           p_nN2 = ck1F.PV01Date(valueDate,toDate,tN2Date,tau);
           
           df_Tn2 = ck1F.DF(to);
           df_TN2 = ck1F.DF(tN2);
           y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
           
           c_nN2 = ck1F.C_nNDate(valueDate,toDate,tN2Date,tau,p_nN2,y_nN2);
           g_nN2 = ck1F.G_nNDate(valueDate,toDate,tN2Date,tau,p_nN2);
           h_nTerm2 = ck1F.H_nTermDate(valueDate,toDate,tpDate);
           
           varCovar = ck1F.LocalVariance(0,to);
           
           driftTerm1=0.0;
           driftTerm2=0.0;
           for i=1:1
               for j=1:1
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
                    driftTerm2 = driftTerm2 + c_nN2(i) * (h_nTerm2(j) + g_nN2(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           variance1 = 0.0;
           variance2 = 0.0;
           coVariance = 0.0;
           for i=1:1
               for j=1:1
                    spreadVariance = spreadVariance + (c_nN1(i)-c_nN2(i)) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
                    variance1 = variance1 + (c_nN1(i)) * (c_nN1(j)) * varCovar(i,j);
                    variance2 = variance2 + (c_nN2(i)) * (c_nN2(j)) * varCovar(i,j);
                    coVariance = coVariance + (c_nN1(i)) * (c_nN2(j)) * varCovar(i,j);
               end
           end
           
           out.fwdSpread = y_nN1 - y_nN2;
           out.mu = (y_nN1 + driftTerm1) - (y_nN2 + driftTerm2);
           out.pmu = driftTerm1 - driftTerm2;
           out.sigma = sqrt(spreadVariance);
           out.c_nN1 = c_nN1;
           out.c_nN2 = c_nN2;
           out.varCovar = varCovar;
           out.mu1 = y_nN1 + driftTerm1;
           out.mu2 = y_nN2 + driftTerm2;
           
           out.pmu1 = driftTerm1;
           out.pmu2 = driftTerm2;
           
           out.corr = coVariance/sqrt(variance1*variance2);
           out.sig1 = sqrt(variance1);
           out.sig2 = sqrt(variance2);
           
        end
        
        function out = CMSSpreadDigital(ck1F,to,tp,tenor1,tenor2,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsSpreadDigitalArg = ck1F.CMSSpreadDigitalArg(to,tp,tenor1,tenor2,tau);
           mu = cmsSpreadDigitalArg.mu;
           sigma = cmsSpreadDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSSpreadDigitalDate(ck1F,valueDate,toDate,tpDate,tenor1,tenor2,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsSpreadDigitalArg = ck1F.CMSSpreadDigitalArgDate(valueDate,toDate,tpDate,tenor1,tenor2,tau);
           mu = cmsSpreadDigitalArg.mu;
           sigma = cmsSpreadDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSSpreadDigitalRange(ck1F,to,tp,tenor1,tenor2,tau,lower,upper)
            lower_value = ck1F.CMSSpreadDigital(to,tp,tenor1,tenor2,tau,lower);
            upper_value = ck1F.CMSSpreadDigital(to,tp,tenor1,tenor2,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CMSSpreadDigitalRangeDate(ck1F,valueDate,toDate,tpDate,tenor1,tenor2,tau,lower,upper)
            lower_value = ck1F.CMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,lower);
            upper_value = ck1F.CMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,upper);
            out = lower_value - upper_value;
        end
        
        function atmSwaption = ATMSwaption(ck1F,tx,tnr)
            tau = 0.25;
            pv01 = ck1F.PV01(tx,tx+tnr,tau);
            fsr  = ck1F.Fsr(tx,tx+tnr,tau);
            c_nN = ck1F.C_nN(tx,tx+tnr,tau,pv01,fsr);
            variances = ck1F.LocalVariance(0,tx);
            totalVar = 0.0;
            for i=1:1
                for j=1:1
                    totalVar = totalVar + c_nN(i)*c_nN(j)*variances(i,j);
                end
            end
            atmSwaption =  pv01 * sqrt(totalVar)/sqrt(2.0*pi);
        end
        
        function bondPutOption = BondPutOptionOld(ck1F,t0,tN,strike,xStar)
            
            p = @ck1F.DF;
            beta_nN = ck1F.Beta(t0,tN);
            localStd = sqrt(ck1F.LocalVariance(0.0,t0));
            zStar = xStar/localStd;
            std1 = beta_nN*localStd;
            
            d1 = zStar;
            d2 = d1 + std1;
            bondPutOption = p(t0)*strike*H_ncdf(-d1) - p(tN)*H_ncdf(-d2);
            
            if bondPutOption < 0
                error('bondPutOption is negative!')
            end
        end
        
        function bondPutOption = BondPutOption(ck1F,t0,tN,strike)
            
            p = @ck1F.DF;
            beta_nN = ck1F.Beta(t0,tN);
            localStd = sqrt(ck1F.LocalVariance(0.0,t0));
%             zStar = xStar/localStd;
            std1 = beta_nN*localStd;
            
            d1 = (log((p(tN)/p(t0))/strike)-0.5*std1*std1)/std1;
            d2 = d1 + std1;
            bondPutOption = p(t0)*strike*H_ncdf(-d1) - p(tN)*H_ncdf(-d2);
            
            if bondPutOption < 0
                error('bondPutOption is negative!')
            end
        end
        
        function atmSwaption = ATMSwaptionJamsOld(ck1F,tx,tnr)
            tau = 0.25;
            % ATM Strike ( or fixed coupon)
            fsr  = ck1F.Fsr(tx,tx+tnr,tau);
            
            cnt = round(tnr/tau);
            
            % find the swaption exercise boundary using root finding method
            
            targetG = @(x) sum(arrayfun(@(i) discountFactor(ck1F,tx*365.0,(tx+tau*i)*365.0,tx*365.0,x),1:cnt));
            
            targetF = @(x) (1.0 - discountFactor(ck1F,tx*365.0,(tx+tnr)*365.0,tx*365.0,x)-fsr*tau*targetG(x));
            
            xStar = fzero(targetF,[-0.1,0.1]);
            
            atmSwaption = 0.0;
            
            effectivePutOptionStrike = discountFactor(ck1F,tx*365.0,(tx+tnr)*365.0,tx*365.0,xStar);
            atmSwaption = ck1F.BondPutOption(tx,tx + tnr,effectivePutOptionStrike,xStar);
            for i=1:cnt
                effectivePutOptionStrike = discountFactor(ck1F,tx*365.0,(tx+tau*i)*365.0,tx*365.0,xStar);
                atmSwaption = atmSwaption + fsr * tau * ck1F.BondPutOption(tx,tx + tau*i,effectivePutOptionStrike,xStar);
            end
            
%             atmSwaption =  ck1F.DF(tx)*atmSwaption;
            
        end
        
        function atmSwaption = ATMSwaptionJams(ck1F,tx,tnr)
            tau = 0.25;
            % ATM Strike ( or fixed coupon)
            fsr  = ck1F.Fsr(tx,tx+tnr,tau);
            
            cnt = round(tnr/tau);
            
            % find the swaption exercise boundary using root finding method
            
            targetG = @(x) sum(arrayfun(@(i) discountFactor(ck1F,tx*365.0,(tx+tau*i)*365.0,tx*365.0,x),1:cnt));
            
            targetF = @(x) (1.0 - discountFactor(ck1F,tx*365.0,(tx+tnr)*365.0,tx*365.0,x)-fsr*tau*targetG(x));
            
            xStar = fzero(targetF,[-0.1,0.1]);
            
            atmSwaption = 0.0;
            
            effectivePutOptionStrike = discountFactor(ck1F,tx*365.0,(tx+tnr)*365.0,tx*365.0,xStar);
            atmSwaption = ck1F.BondPutOption(tx,tx + tnr,effectivePutOptionStrike);
            for i=1:cnt
                effectivePutOptionStrike = discountFactor(ck1F,tx*365.0,(tx+tau*i)*365.0,tx*365.0,xStar);
                atmSwaption = atmSwaption + fsr * tau * ck1F.BondPutOption(tx,tx + tau*i,effectivePutOptionStrike);
            end
            
%             atmSwaption =  ck1F.DF(tx)*atmSwaption;
            
        end
        
        function spreadInfo = SpreadInfo(ck1F,tx,tnr1,tnr2)
            tau = 0.25;
            cmsSpreadDigitalArg =  ck1F.CMSSpreadDigitalArg(tx,tx+1.0,tnr1,tnr2,tau);
            spreadInfo = cmsSpreadDigitalArg;
            spreadInfo.normSigma = cmsSpreadDigitalArg.sigma/sqrt(tx);
        end
        
        function impCorrel = ImpCorrel(ck1F,tx,tnr1,tnr2)
            
            tau = 0.25;
            pv01_1 = ck1F.PV01(tx,tx+tnr1,tau);
            fsr_1  = ck1F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = ck1F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = ck1F.PV01(tx,tx+tnr2,tau);
            fsr_2  = ck1F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = ck1F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            variances = ck1F.LocalVariance(0,tx);
            var1 = 0;
            var2 = 0;
            coVar = 0;
            for j=1:1
                for k=1:1
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                    var2 = var2 + c_nN_2(j)*c_nN_2(k)*variances(j,k);
                    coVar = coVar + c_nN_1(j)*c_nN_2(k)*variances(j,k);
                end
            end
            impCorrel = coVar/sqrt(var1)/sqrt(var2);
        end
        
        function impCovar = ImpCovar(ck1F,tx,tnr1,tnr2)
            tau = 0.25;
            pv01_1 = ck1F.PV01(tx,tx+tnr1,tau);
            fsr_1  = ck1F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = ck1F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = ck1F.PV01(tx,tx+tnr2,tau);
            fsr_2  = ck1F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = ck1F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            variances = ck1F.LocalVariance(0,tx);
            var1 = 0;
            var2 = 0;
            coVar = 0;
            for j=1:1
                for k=1:1
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                    var2 = var2 + c_nN_2(j)*c_nN_2(k)*variances(j,k);
                    coVar = coVar + c_nN_1(j)*c_nN_2(k)*variances(j,k);
                end
            end
            impCovar = coVar;
        end
        
        
        function instFwdCorr = InsFwdCorr(ck1F,tx,tnr1,tnr2)
            tau = 0.25;
            tenor = ck1F.vol_r.tenor;
            tIdx = min(find(tx<=tenor));
            meanR_r = ck1F.meanR_r;
            vol_r = ck1F.vol_r;
            corr_rs = 1.0;
            covar = 0.0;
            var1 = 0;
            var2 = 0;
            
            var1 = exp(-2*meanR_r*tnr1)*vol_r.quote(tIdx)*vol_r.quote(tIdx);
            var2 = exp(-2*meanR_r*tnr2)*vol_r.quote(tIdx)*vol_r.quote(tIdx);
            covar = exp(-meanR_r*(tnr1+tnr2))*vol_r.quote(tIdx)*vol_r.quote(tIdx);
            
            instFwdCorr = covar/sqrt(var1*var2);
        end
        
        
        
        function instFwdCorrTest = InsFwdCorrTest(ck1F,tx,tnr1,tnr2)
            tau = 0.25;
            sigma_r = ck1F.sigma_Test(tx,ck1F.vol_r);
            a_r = ck1F.meanR_r;
            
            covar = 0.0;
            var1 = 0;
            var2 = 0;
            var1 =  exp(-2.0*a_r*tnr1)*sigma_r*sigma_r;
            var2 =  exp(-2.0*a_r*tnr2)*sigma_r*sigma_r;
            covar = exp(-1.0*a_r*(tnr1+ tnr2))*sigma_r*sigma_r;
            instFwdCorrTest = covar/sqrt(var1*var2);
        end
        
        
        function instFwdSwapCorrTest = InsSwapFwdCorrTest(ck1F,tx,tnr1,tnr2)
            tau = 0.25;
            C_tT1 = zeros(1,1);
            pv01_1 = ck1F.PV01(tx,tx+tnr1,tau);
            fsr_1  = ck1F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = ck1F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            C_tT1(1) = c_nN_1(1);
            
            C_tT2 = zeros(1,1);
            pv01_2 = ck1F.PV01(tx,tx+tnr2,tau);
            fsr_2  = ck1F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = ck1F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            C_tT2(1) = c_nN_2(1);
            
            dAlpha_t = zeros(1,1);
            dAlpha_t(1) = ck1F.sigma_Test(tx,ck1F.vol_r);
            corr = zeros(1,1);
            corr(1,1) = 1.0;
            
            covar = 0.0;
            var1 = 0.0;
            var2 = 0.0;
            for i=1:1
                for j=1:1
                    var1 = var1 + C_tT1(i)*C_tT1(j)*dAlpha_t(i)*dAlpha_t(j)*corr(i,j);
                    var2 = var2 + C_tT2(i)*C_tT2(j)*dAlpha_t(i)*dAlpha_t(j)*corr(i,j);
                    covar = covar + C_tT1(i)*C_tT2(j)*dAlpha_t(i)*dAlpha_t(j)*corr(i,j);
                end
            end
            instFwdSwapCorrTest = covar/sqrt(var1*var2);
        end
        
        function impVariance = ImpVariance(ck1F,tx,tnr1)
            tau = 0.25;
            pv01_1 = ck1F.PV01(tx,tx+tnr1,tau);
            fsr_1  = ck1F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = ck1F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            variances = ck1F.LocalVariance(0,tx);
            var1 = 0;
            for j=1:1
                for k=1:1
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                end
            end
            impVariance = var1;
        end
        
        function out = CalibrateToATMDiagSwaptionNormal(ck1F,black,calibrationFlag)
          % Market Data Object , Vanilla Object
          
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
            
            expirySize = size(expiry,2);
            tenorSize = size(tenor,2);
            
            blackSwaptionPrice = zeros(expirySize,tenorSize);

            for k=1:expirySize
                for j=1:tenorSize
                    blackSwaptionPrice(k,j) = black.NormalATMSwaption(expiry(k),tenor(j));
               end
            end
            
            swaptionVolSurface.expiry = expiry;
            swaptionVolSurface.tenor = tenor;
            swaptionVolSurface.swaptionVol = swaptionVol; % volSurface with interpolator
            
          %% select the interpolated target diagonal swaption with expiry &
          %% tenor & linear interpolated vol
            % size for interpolated yearly call (swaption)
            
            target_size = ck1F.targetSize;

            params.target_size = target_size;
            params.matu = ck1F.matu;
            params.freq = ck1F.freq;
            params.swaption_expiry_use_yn = zeros(target_size,1);
            
            params.vol_r.tenor = 1:1:target_size;
            params.vol_r.tenor = params.vol_r.tenor';
            params.vol_r.quote = 0.01*ones(target_size,1);
            
            params.diagSwaption.expiry = zeros(target_size,1);
            params.diagSwaption.tenor  = zeros(target_size,1);
            params.diagSwaption.quote  = zeros(target_size,target_size);
            params.diagSwaption.strike = zeros(target_size,1);
            params.diagSwaption.blackPrice = zeros(target_size,1);
            
            if ck1F.matu <= 2.0
                params.swaption_expiry_use_yn(1) = 1;
                params.diagSwaption.expiry =  ones(1,1);
                params.diagSwaption.expiry(1) =  swaptionVolSurface.expiry(1);
                params.diagSwaption.tenor =  ones(1,1);
                params.diagSwaption.tenor(1) =  swaptionVolSurface.tenor(1);
                params.diagSwaption.quote = ones(1,1);
                params.diagSwaption.quote(1,1) = swaptionVolSurface.swaptionVol(expiry(1),tenor(1));
            else
                temp_tenor   = zeros(target_size,1);
                temp_expiry = zeros(target_size,1);
                for i=2:target_size
                    temp_tenor(i-1)  = params.vol_r.tenor(target_size - (i-1));
                    temp_expiry(i-1) = params.matu - temp_tenor(i-1); 
                end
                
                upper_expiry = swaptionVolSurface.expiry(end);
                upper_tenor  = swaptionVolSurface.tenor(end);
                
                lower_expiry = swaptionVolSurface.expiry(1);
                
                temp_idx= 1;
                for i=1:target_size
                    for j=temp_idx:target_size
                        % only those swaption whose expiry date is away.
                        if temp_expiry(j) > 0
                            params.vol_r.tenor(i) = temp_expiry(j);
                            params.diagSwaption.expiry(i) = temp_expiry(j);
                            params.diagSwaption.tenor(i)  = temp_tenor(j);
                            if(temp_expiry(j) > upper_expiry && temp_tenor(j) <= upper_tenor)
                                params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(upper_expiry,temp_tenor(j));
                            elseif(temp_expiry(j) <= upper_expiry && temp_tenor(j) > upper_tenor)
                                params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(temp_expiry(j),upper_tenor);    
                            else
                                if temp_expiry(j) < lower_expiry
                                    params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(lower_expiry,temp_tenor(j));
                                else
                                    params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(temp_expiry(j),temp_tenor(j));
                                end
                            end
                            
                            params.swaption_expiry_use_yn(i) = 1;
                            temp_idx = j+1;
                            break;
                        end
                    end
                    
                    if i>=2
                        params.vol_r.tenor(i) = params.vol_r.tenor(i-1) + 1;
                    end
                end
                
            end
            
            % set the new model parameters in accordance with the target
            % instruments maturities
            
            ck1F.vol_r    = params.vol_r;
            tau = 1.0/(ck1F.freq);
          %% target swaption price and forward swap rate
            for i=1:target_size
                if params.swaption_expiry_use_yn(i)
                    Ti = params.diagSwaption.expiry(i);
                    Tj = params.diagSwaption.tenor(i);
                    blackVolQuote = params.diagSwaption.quote(i,i);
                    params.diagSwaption.strike(i) = black.Fsr(Ti,Ti + Tj,tau);
%                     params.diagSwaption.blackPrice(i) = black.BlackSwaptionWithVol(Ti,Tj,params.diagSwaption.strike(i),blackVolQuote);
                    params.diagSwaption.blackPrice(i) = black.NormalSwaptionWithVol(Ti,Tj,params.diagSwaption.strike(i),blackVolQuote);
                    
                end
            end
            
           %% target swaption price and forward swap rate
            % tracking calibration
            
            out.blackSwaptionPrice = params.diagSwaption.blackPrice;
            out.ck1FSwaptionPrice = zeros(target_size,1);
            out.priceRelDiff = zeros(target_size,1);
            out.priceRmseTotal = 0.0;
            out.vol_r = params.vol_r;
            
            out.market_swaption = zeros(target_size,1);
            out.model_swaption = zeros(target_size,1);
            out.re = zeros(target_size,1);
            out.rmseTotal = 0;
            
            tvar = [];
            calibrationType = 'global';
            if nargin < 3
                calibrationType = 'global'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;
            targetSwaptionSize = 0;
            switch calibrationType
                case 'global'
                    
                    optParams.params = params;
                    
                    for j=1:length(ck1F.vol_r.quote)
                        if params.swaption_expiry_use_yn(j) == 1
                            targetSwaptionSize = targetSwaptionSize +1;
                        end
                    end
                    
                    optParams.vol_r_size = targetSwaptionSize;
                    
                    optParams.modelParamsSize = optParams.vol_r_size;
                    
                    if ck1F.matu <= 2.0
                        optParams.numOfEquation = targetSwaptionSize; % # of target swaption instruments
                    else
                        optParams.numOfEquation = targetSwaptionSize; % # of target swaption instruments
                    end
                    
                    out.modelParamsSize = optParams.modelParamsSize;
                    out.numOfEquation = optParams.numOfEquation;
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    lb = zeros(optParams.modelParamsSize,1);
                    ub = zeros(optParams.modelParamsSize,1);
                    
                    for j=1:optParams.vol_r_size
                        tvar(j) = params.vol_r.quote(j);
                        lb(j)= 0.0001;
                        ub(j)= 10;
                    end
                       
%                     targetfun = @(tvar) TargetFunctionck1FATMSwaptionDiagGlobal(ck1F,tvar,optParams);
%                     options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
                    x=  zeros(optParams.numOfEquation,1);
                    [ret, popt, info, covar]=levmar('TargetFunctionCK1FATMSwaptionNormalDiagGlobal',tvar,x, 200, options,'unc',ck1F,optParams);
                    
                    
                    % assigning model parameters
                    k=1;
                    for j=1:optParams.vol_r_size
                       ck1F.vol_r.quote(j) = max(popt(j),0.0001);
                       k=k+1;
                    end
                    % flat extrapolation of Alpha
                    % AlphaSize != length(Alpha)
                    
                    for j=k:length(ck1F.vol_r.quote)
                       ck1F.vol_r.quote(j) = ck1F.vol_r.quote(optParams.vol_r_size);
                    end
                    
                    out.vol_r.quote = ck1F.vol_r.quote;
                    
                    N=0;
                    for i=1:params.target_size
                        if params.swaption_expiry_use_yn(i) == 1
                            Ti = params.diagSwaption.expiry(i);
                            Tj = params.diagSwaption.tenor(i);
                            out.market_swaption(i) = params.diagSwaption.blackPrice(i);
                            out.model_swaption(i)  = ck1F.ATMSwaption(Ti,Tj);
                            out.market_vol(i) = params.diagSwaption.quote(i,i);
                            out.re(i) = out.model_swaption(i)/out.market_swaption(i) - 1.0;
                            out.rmseTotal = out.rmseTotal + out.re(i)*out.re(i);
                            N=N+1; 
                        end
                    end
                    out.rmseTotal = sqrt(out.rmseTotal/N);
               
                otherwise
                    disp('unImplemented')
            end
        end
        
        function out = CalibrateToATMDiagSwaptionForward(ck1F,black,calibrationFlag)
          % instrumentStartDate 
          % instrumentEndDate
          % calibrationTargetTenor as input
          
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
            
            expirySize = size(expiry,2);
            tenorSize = size(tenor,2);
            
            blackSwaptionPrice = zeros(expirySize,tenorSize);

            for k=1:expirySize
                for j=1:tenorSize
                    blackSwaptionPrice(k,j) = black.BlackATMSwaption(expiry(k),tenor(j));
               end
            end
            
            swaptionVolSurface.expiry = expiry;
            swaptionVolSurface.tenor = tenor;
            swaptionVolSurface.swaptionVol = swaptionVol; % volSurface with interpolator
            
          %% select the interpolated target diagonal swaption with expiry &
          %% tenor & linear interpolated vol
            % size for interpolated yearly call (swaption)
            
%             target_size = ck1F.targetSize;
%             calibrationTargetTenor = 1,2,3,...,14  -> length 14
            target_size = length(ck1F.calibrationTargetTenor);
            
            params.target_size = target_size;
            
%             params.matu = ck1F.matu;
            % evalDate == ck1F.zeroCurve.asOfDate
%             params.matu = DateDiff(ck1F.instrumentEndDate,ck1F.zeroCurve.asOfDate)/365.0; 
            
            params.freq = ck1F.freq;
            params.swaption_expiry_use_yn = zeros(target_size,1);
            
            params.vol_r.tenor = [];
            
            dummyExpiry = zeros(length(target_size),1);
            dummyTenor = zeros(length(target_size),1);
            
            exerciseDate = cell(length(target_size),1);
            numberOfAliveTarget = 0;
            for i=1:target_size
                exerciseDate{i} = ck1F.instrumentStartDate.AddDate(ck1F.calibrationTargetTenor(i),'year');
                dummyExpiry(i) = DateDiff(exerciseDate{i},ck1F.zeroCurve.asOfDate)/365.0;
                dummyTenor(i) =  DateDiff(ck1F.instrumentEndDate,exerciseDate{i})/365.0;
                if dummyExpiry(i) > 0 && dummyTenor(i) > 0
                    numberOfAliveTarget = numberOfAliveTarget + 1;
                    params.vol_r.tenor = [params.vol_r.tenor;dummyExpiry(i)];
                end
            end
            
%             params.vol_r.tenor = params.vol_r.tenor';
            params.vol_r.quote = 0.01*ones(length(params.vol_r.tenor),1);
            
            params.diagSwaption.expiry = zeros(target_size,1);
            params.diagSwaption.tenor  = zeros(target_size,1);
            params.diagSwaption.quote  = zeros(target_size,target_size);
            params.diagSwaption.strike = zeros(target_size,1);
            params.diagSwaption.blackPrice = zeros(target_size,1);
            
            if numberOfAliveTarget < 2
                params.swaption_expiry_use_yn(1) = 1;
                params.diagSwaption.expiry =  ones(1,1);
                params.diagSwaption.expiry(1) =  swaptionVolSurface.expiry(1);
                params.diagSwaption.tenor =  ones(1,1);
                params.diagSwaption.tenor(1) =  swaptionVolSurface.tenor(1);
                params.diagSwaption.quote = ones(1,1);
                params.diagSwaption.quote(1,1) = swaptionVolSurface.swaptionVol(expiry(1),tenor(1));
                
                params.vol_r.tenor = ones(1,1);
                params.vol_r.quote = 0.01*ones(length(params.vol_r.tenor),1);
            else
                temp_tenor   = zeros(numberOfAliveTarget,1);
                temp_expiry = zeros(numberOfAliveTarget,1);
                
%                 temp_expiry = dummyExpiry;
%                 temp_tenor = dummyTenor;
                j=1;
                for i=1:target_size
                    if dummyExpiry(i) > 0 && dummyTenor(i) > 0
                        temp_expiry(j) = dummyExpiry(i);
                        temp_tenor(j)  = dummyTenor(i);
                        j = j + 1;
                    end 
                end
                
                upper_expiry = swaptionVolSurface.expiry(end);
                upper_tenor  = swaptionVolSurface.tenor(end);
                
                lower_expiry = swaptionVolSurface.expiry(1);
                lower_tenor = swaptionVolSurface.tenor(1);
                
                temp_idx= 1;
                for i=1:target_size
                    for j=temp_idx:target_size
                        % only those swaption whose expiry date is away.
                        if dummyExpiry(j) > 0 && dummyTenor(j) > 0
                            params.vol_r.tenor(i) = dummyExpiry(j);
                            params.diagSwaption.expiry(i) = dummyExpiry(j);
                            params.diagSwaption.tenor(i)  = dummyTenor(j);
                            if(dummyExpiry(j) > upper_expiry && dummyTenor(j) <= upper_tenor)
                                params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(upper_expiry,dummyTenor(j));
                            elseif(dummyExpiry(j) <= upper_expiry && dummyTenor(j) > upper_tenor)
                                params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(dummyExpiry(j),upper_tenor);    
                            else
                                if dummyTenor(j) < lower_tenor
                                    params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(dummyExpiry(j),lower_tenor);
                                else
                                    if dummyExpiry(j) < lower_expiry
                                        params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(lower_expiry,dummyTenor(j));
                                    else
                                        params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(dummyExpiry(j),dummyTenor(j));
                                    end
                                end
                            end
                            
                            params.swaption_expiry_use_yn(i) = 1;
                            temp_idx = j+1;
                            break;
                        end
                    end
                    
%                     if i>=2
%                         params.vol_r.tenor(i) = params.vol_r.tenor(i-1) + 1;
%                     end
                end
                
            end
            
            % set the new model parameters in accordance with the target
            % instruments maturities
            
            ck1F.vol_r    = params.vol_r;
            tau = 1.0/(ck1F.freq);
          %% target swaption price and forward swap rate
            for i=1:target_size
                if params.swaption_expiry_use_yn(i)
                    Ti = params.diagSwaption.expiry(i);
                    Tj = params.diagSwaption.tenor(i);
                    blackVolQuote = params.diagSwaption.quote(i,i);
                    params.diagSwaption.strike(i) = black.Fsr(Ti,Ti + Tj,tau);
%                     params.diagSwaption.blackPrice(i) = black.BlackSwaptionWithVol(Ti,Tj,params.diagSwaption.strike(i),blackVolQuote);
                    params.diagSwaption.blackPrice(i) = black.BlackSwaptionWithVolTau(Ti,Tj,params.diagSwaption.strike(i),blackVolQuote,tau);
                    
                end
            end
            
           %% target swaption price and forward swap rate
            % tracking calibration
            
            out.blackSwaptionPrice = params.diagSwaption.blackPrice;
            out.ck1FSwaptionPrice = zeros(target_size,1);
            out.priceRelDiff = zeros(target_size,1);
            out.priceRmseTotal = 0.0;
            out.vol_r = params.vol_r;
            
            out.market_swaption = zeros(target_size,1);
            out.model_swaption = zeros(target_size,1);
            out.re = zeros(target_size,1);
            out.rmseTotal = 0;
            
            tvar = [];
            calibrationType = 'global';
            if nargin < 3
                calibrationType = 'global'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;
            targetSwaptionSize = 0;
            switch calibrationType
                case 'global'
                    
                    optParams.params = params;
                    
%                     for j=1:length(ck1F.vol_r.quote)
%                         if params.swaption_expiry_use_yn(j) == 1
%                             targetSwaptionSize = targetSwaptionSize +1;
%                         end
%                     end
                    
                    targetSwaptionSize = length(ck1F.vol_r.quote);
                    
                    optParams.vol_r_size = targetSwaptionSize;
                    
                    optParams.modelParamsSize = optParams.vol_r_size;
                    
                    optParams.numOfEquation = targetSwaptionSize;
%                     
%                     if ck1F.matu <= 2.0
%                         optParams.numOfEquation = targetSwaptionSize; % # of target swaption instruments
%                     else
%                         optParams.numOfEquation = targetSwaptionSize; % # of target swaption instruments
%                     end
                    
                    out.modelParamsSize = optParams.modelParamsSize;
                    out.numOfEquation = optParams.numOfEquation;
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    lb = zeros(optParams.modelParamsSize,1);
                    ub = zeros(optParams.modelParamsSize,1);
                    
                    for j=1:optParams.vol_r_size
                        tvar(j) = params.vol_r.quote(j);
                        lb(j)= 0.0001;
                        ub(j)= 10;
                    end
                       
%                     targetfun = @(tvar) TargetFunctionck1FATMSwaptionDiagGlobal(ck1F,tvar,optParams);
%                     options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
                    x=  zeros(optParams.numOfEquation,1);
                    [ret, popt, info, covar]=levmar('TargetFunctionCK1FATMSwaptionDiagGlobal',tvar,x, 200, options,'unc',ck1F,optParams);
                    
                    
                    % assigning model parameters
                    k=1;
                    for j=1:optParams.vol_r_size
                       ck1F.vol_r.quote(j) = max(popt(j),0.0001);
                       k=k+1;
                    end
                    % flat extrapolation of Alpha
                    % AlphaSize != length(Alpha)
                    
                    for j=k:length(ck1F.vol_r.quote)
                       ck1F.vol_r.quote(j) = ck1F.vol_r.quote(optParams.vol_r_size);
                    end
                    
                    out.vol_r.quote = ck1F.vol_r.quote;
                    
                    N=0;
                    for i=1:params.target_size
                        if params.swaption_expiry_use_yn(i) == 1
                            Ti = params.diagSwaption.expiry(i);
                            Tj = params.diagSwaption.tenor(i);
                            out.market_swaption(i) = params.diagSwaption.blackPrice(i);
                            out.model_swaption(i)  = ck1F.ATMSwaption(Ti,Tj);
                            out.re(i) = out.model_swaption(i)/out.market_swaption(i) - 1.0;
                            out.rmseTotal = out.rmseTotal + out.re(i)*out.re(i);
                            N=N+1; 
                        end
                    end
                    out.rmseTotal = sqrt(out.rmseTotal/N);
               
                otherwise
                    disp('unImplemented')
            end
        end
        
        function out = CalibrateToATMDiagSwaptionBackward(ck1F,black,calibrationFlag)
          % instrumentStartDate 
          % instrumentEndDate
          % calibrationTargetTenor as input
          
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
            
            expirySize = size(expiry,2);
            tenorSize = size(tenor,2);
            
            blackSwaptionPrice = zeros(expirySize,tenorSize);

            for k=1:expirySize
                for j=1:tenorSize
                    blackSwaptionPrice(k,j) = black.BlackATMSwaption(expiry(k),tenor(j));
               end
            end
            
            swaptionVolSurface.expiry = expiry;
            swaptionVolSurface.tenor = tenor;
            swaptionVolSurface.swaptionVol = swaptionVol; % volSurface with interpolator
            
          %% select the interpolated target diagonal swaption with expiry &
          %% tenor & linear interpolated vol
            % size for interpolated yearly call (swaption)
            
%             target_size = ck1F.targetSize;
%             calibrationTargetTenor = 1,2,3,...,14  -> length 14
            target_size = length(ck1F.calibrationTargetTenor) + 1;

            
            params.target_size = target_size;
            params.target_size = target_size;
            
%             params.matu = ck1F.matu;
            % evalDate == ck1F.zeroCurve.asOfDate
            params.matu = DateDiff(ck1F.instrumentEndDate,ck1F.zeroCurve.asOfDate)/365.0; 
            
            params.freq = ck1F.freq;
            params.swaption_expiry_use_yn = zeros(target_size,1);
            
%             params.vol_r.tenor = 1:1:target_size;
%             vol_r.tenor = calibrationTargetTenor,
%             calibrarationTargetTenor  + 1
            addedTenor = ck1F.calibrationTargetTenor(end) +1;
            params.vol_r.tenor = [ck1F.calibrationTargetTenor,addedTenor];
            
            params.vol_r.tenor = params.vol_r.tenor';
            params.vol_r.quote = 0.01*ones(target_size,1);
            
            params.diagSwaption.expiry = zeros(target_size,1);
            params.diagSwaption.tenor  = zeros(target_size,1);
            params.diagSwaption.quote  = zeros(target_size,target_size);
            params.diagSwaption.strike = zeros(target_size,1);
            params.diagSwaption.blackPrice = zeros(target_size,1);
            
            if ck1F.matu <= 2.0
                params.swaption_expiry_use_yn(1) = 1;
                params.diagSwaption.expiry =  ones(1,1);
                params.diagSwaption.expiry(1) =  swaptionVolSurface.expiry(1);
                params.diagSwaption.tenor =  ones(1,1);
                params.diagSwaption.tenor(1) =  swaptionVolSurface.tenor(1);
                params.diagSwaption.quote = ones(1,1);
                params.diagSwaption.quote(1,1) = swaptionVolSurface.swaptionVol(expiry(1),tenor(1));
            else
                temp_tenor   = zeros(target_size,1);
                temp_expiry = zeros(target_size,1);
                for i=2:target_size
                    temp_tenor(i-1)  = params.vol_r.tenor(target_size - (i-1));
                    temp_expiry(i-1) = params.matu - temp_tenor(i-1); 
                end
                
                upper_expiry = swaptionVolSurface.expiry(end);
                upper_tenor  = swaptionVolSurface.tenor(end);
                
                lower_expiry = swaptionVolSurface.expiry(1);
                
                temp_idx= 1;
                for i=1:target_size
                    for j=temp_idx:target_size
                        % only those swaption whose expiry date is away.
                        if temp_expiry(j) > 0
                            params.vol_r.tenor(i) = temp_expiry(j);
                            params.diagSwaption.expiry(i) = temp_expiry(j);
                            params.diagSwaption.tenor(i)  = temp_tenor(j);
                            if(temp_expiry(j) > upper_expiry && temp_tenor(j) <= upper_tenor)
                                params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(upper_expiry,temp_tenor(j));
                            elseif(temp_expiry(j) <= upper_expiry && temp_tenor(j) > upper_tenor)
                                params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(temp_expiry(j),upper_tenor);    
                            else
                                if temp_expiry(j) < lower_expiry
                                    params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(lower_expiry,temp_tenor(j));
                                else
                                    params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(temp_expiry(j),temp_tenor(j));
                                end
                            end
                            
                            params.swaption_expiry_use_yn(i) = 1;
                            temp_idx = j+1;
                            break;
                        end
                    end
                    
                    if i>=2
                        params.vol_r.tenor(i) = params.vol_r.tenor(i-1) + 1;
                    end
                end
                
            end
            
            % set the new model parameters in accordance with the target
            % instruments maturities
            
            ck1F.vol_r    = params.vol_r;
            tau = 1.0/(ck1F.freq);
          %% target swaption price and forward swap rate
            for i=1:target_size
                if params.swaption_expiry_use_yn(i)
                    Ti = params.diagSwaption.expiry(i);
                    Tj = params.diagSwaption.tenor(i);
                    blackVolQuote = params.diagSwaption.quote(i,i);
                    params.diagSwaption.strike(i) = black.Fsr(Ti,Ti + Tj,tau);
%                     params.diagSwaption.blackPrice(i) = black.BlackSwaptionWithVol(Ti,Tj,params.diagSwaption.strike(i),blackVolQuote);
                    params.diagSwaption.blackPrice(i) = black.BlackSwaptionWithVolTau(Ti,Tj,params.diagSwaption.strike(i),blackVolQuote,tau);
                    
                end
            end
            
           %% target swaption price and forward swap rate
            % tracking calibration
            
            out.blackSwaptionPrice = params.diagSwaption.blackPrice;
            out.ck1FSwaptionPrice = zeros(target_size,1);
            out.priceRelDiff = zeros(target_size,1);
            out.priceRmseTotal = 0.0;
            out.vol_r = params.vol_r;
            
            out.market_swaption = zeros(target_size,1);
            out.model_swaption = zeros(target_size,1);
            out.re = zeros(target_size,1);
            out.rmseTotal = 0;
            
            tvar = [];
            calibrationType = 'global';
            if nargin < 3
                calibrationType = 'global'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;
            targetSwaptionSize = 0;
            switch calibrationType
                case 'global'
                    
                    optParams.params = params;
                    
                    for j=1:length(ck1F.vol_r.quote)
                        if params.swaption_expiry_use_yn(j) == 1
                            targetSwaptionSize = targetSwaptionSize +1;
                        end
                    end
                    
                    optParams.vol_r_size = targetSwaptionSize;
                    
                    optParams.modelParamsSize = optParams.vol_r_size;
                    
                    if ck1F.matu <= 2.0
                        optParams.numOfEquation = targetSwaptionSize; % # of target swaption instruments
                    else
                        optParams.numOfEquation = targetSwaptionSize; % # of target swaption instruments
                    end
                    
                    out.modelParamsSize = optParams.modelParamsSize;
                    out.numOfEquation = optParams.numOfEquation;
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    lb = zeros(optParams.modelParamsSize,1);
                    ub = zeros(optParams.modelParamsSize,1);
                    
                    for j=1:optParams.vol_r_size
                        tvar(j) = params.vol_r.quote(j);
                        lb(j)= 0.0001;
                        ub(j)= 10;
                    end
                       
%                     targetfun = @(tvar) TargetFunctionck1FATMSwaptionDiagGlobal(ck1F,tvar,optParams);
%                     options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
                    x=  zeros(optParams.numOfEquation,1);
                    [ret, popt, info, covar]=levmar('TargetFunctionCK1FATMSwaptionDiagGlobal',tvar,x, 200, options,'unc',ck1F,optParams);
                    
                    
                    % assigning model parameters
                    k=1;
                    for j=1:optParams.vol_r_size
                       ck1F.vol_r.quote(j) = max(popt(j),0.0001);
                       k=k+1;
                    end
                    % flat extrapolation of Alpha
                    % AlphaSize != length(Alpha)
                    
                    for j=k:length(ck1F.vol_r.quote)
                       ck1F.vol_r.quote(j) = ck1F.vol_r.quote(optParams.vol_r_size);
                    end
                    
                    out.vol_r.quote = ck1F.vol_r.quote;
                    
                    N=0;
                    for i=1:params.target_size
                        if params.swaption_expiry_use_yn(i) == 1
                            Ti = params.diagSwaption.expiry(i);
                            Tj = params.diagSwaption.tenor(i);
                            out.market_swaption(i) = params.diagSwaption.blackPrice(i);
                            out.model_swaption(i)  = ck1F.ATMSwaption(Ti,Tj);
                            out.re(i) = out.model_swaption(i)/out.market_swaption(i) - 1.0;
                            out.rmseTotal = out.rmseTotal + out.re(i)*out.re(i);
                            N=N+1; 
                        end
                    end
                    out.rmseTotal = sqrt(out.rmseTotal/N);
               
                otherwise
                    disp('unImplemented')
            end
        end
        
        function out = CalibrateToATMDiagSwaption(ck1F,black,calibrationFlag)
          % Market Data Object , Vanilla Object
          
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
            
            expirySize = size(expiry,2);
            tenorSize = size(tenor,2);
            
            blackSwaptionPrice = zeros(expirySize,tenorSize);

            for k=1:expirySize
                for j=1:tenorSize
                    blackSwaptionPrice(k,j) = black.BlackATMSwaption(expiry(k),tenor(j));
               end
            end
            
            swaptionVolSurface.expiry = expiry;
            swaptionVolSurface.tenor = tenor;
            swaptionVolSurface.swaptionVol = swaptionVol; % volSurface with interpolator
            
          %% select the interpolated target diagonal swaption with expiry &
          %% tenor & linear interpolated vol
            % size for interpolated yearly call (swaption)
            
            target_size = ck1F.targetSize;

            params.target_size = target_size;
            params.matu = ck1F.matu;
            params.freq = ck1F.freq;
            params.swaption_expiry_use_yn = zeros(target_size,1);
            
            params.vol_r.tenor = 1:1:target_size;
            params.vol_r.tenor = params.vol_r.tenor';
            params.vol_r.quote = 0.01*ones(target_size,1);
            
            params.diagSwaption.expiry = zeros(target_size,1);
            params.diagSwaption.tenor  = zeros(target_size,1);
            params.diagSwaption.quote  = zeros(target_size,target_size);
            params.diagSwaption.strike = zeros(target_size,1);
            params.diagSwaption.blackPrice = zeros(target_size,1);
            
            if ck1F.matu <= 2.0
                params.swaption_expiry_use_yn(1) = 1;
                params.diagSwaption.expiry =  ones(1,1);
                params.diagSwaption.expiry(1) =  swaptionVolSurface.expiry(1);
                params.diagSwaption.tenor =  ones(1,1);
                params.diagSwaption.tenor(1) =  swaptionVolSurface.tenor(1);
                params.diagSwaption.quote = ones(1,1);
                params.diagSwaption.quote(1,1) = swaptionVolSurface.swaptionVol(expiry(1),tenor(1));
            else
                temp_tenor   = zeros(target_size,1);
                temp_expiry = zeros(target_size,1);
                for i=2:target_size
                    temp_tenor(i-1)  = params.vol_r.tenor(target_size - (i-1));
                    temp_expiry(i-1) = params.matu - temp_tenor(i-1); 
                end
                
                upper_expiry = swaptionVolSurface.expiry(end);
                upper_tenor  = swaptionVolSurface.tenor(end);
                
                lower_expiry = swaptionVolSurface.expiry(1);
                
                temp_idx= 1;
                for i=1:target_size
                    for j=temp_idx:target_size
                        % only those swaption whose expiry date is away.
                        if temp_expiry(j) > 0
                            params.vol_r.tenor(i) = temp_expiry(j);
                            params.diagSwaption.expiry(i) = temp_expiry(j);
                            params.diagSwaption.tenor(i)  = temp_tenor(j);
                            if(temp_expiry(j) > upper_expiry && temp_tenor(j) <= upper_tenor)
                                params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(upper_expiry,temp_tenor(j));
                            elseif(temp_expiry(j) <= upper_expiry && temp_tenor(j) > upper_tenor)
                                params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(temp_expiry(j),upper_tenor);    
                            else
                                if temp_expiry(j) < lower_expiry
                                    params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(lower_expiry,temp_tenor(j));
                                else
                                    params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(temp_expiry(j),temp_tenor(j));
                                end
                            end
                            
                            params.swaption_expiry_use_yn(i) = 1;
                            temp_idx = j+1;
                            break;
                        end
                    end
                    
                    if i>=2
                        params.vol_r.tenor(i) = params.vol_r.tenor(i-1) + 1;
                    end
                end
                
            end
            
            % set the new model parameters in accordance with the target
            % instruments maturities
            
            ck1F.vol_r    = params.vol_r;
            tau = 1.0/(ck1F.freq);
          %% target swaption price and forward swap rate
            for i=1:target_size
                if params.swaption_expiry_use_yn(i)
                    Ti = params.diagSwaption.expiry(i);
                    Tj = params.diagSwaption.tenor(i);
                    blackVolQuote = params.diagSwaption.quote(i,i);
                    params.diagSwaption.strike(i) = black.Fsr(Ti,Ti + Tj,tau);
                    params.diagSwaption.blackPrice(i) = black.BlackSwaptionWithVol(Ti,Tj,params.diagSwaption.strike(i),blackVolQuote);
         
                end
            end
            
           %% target swaption price and forward swap rate
            % tracking calibration
            
            out.blackSwaptionPrice = params.diagSwaption.blackPrice;
            out.ck1FSwaptionPrice = zeros(target_size,1);
            out.priceRelDiff = zeros(target_size,1);
            out.priceRmseTotal = 0.0;
            out.vol_r = params.vol_r;
            
            out.market_swaption = zeros(target_size,1);
            out.model_swaption = zeros(target_size,1);
            out.re = zeros(target_size,1);
            out.rmseTotal = 0;
            
            tvar = [];
            calibrationType = 'global';
            if nargin < 3
                calibrationType = 'global'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;
            targetSwaptionSize = 0;
            switch calibrationType
                case 'global'
                    
                    optParams.params = params;
                    
                    for j=1:length(ck1F.vol_r.quote)
                        if params.swaption_expiry_use_yn(j) == 1
                            targetSwaptionSize = targetSwaptionSize +1;
                        end
                    end
                    
                    optParams.vol_r_size = targetSwaptionSize;
                    
                    optParams.modelParamsSize = optParams.vol_r_size;
                    
                    if ck1F.matu <= 2.0
                        optParams.numOfEquation = targetSwaptionSize; % # of target swaption instruments
                    else
                        optParams.numOfEquation = targetSwaptionSize; % # of target swaption instruments
                    end
                    
                    out.modelParamsSize = optParams.modelParamsSize;
                    out.numOfEquation = optParams.numOfEquation;
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    lb = zeros(optParams.modelParamsSize,1);
                    ub = zeros(optParams.modelParamsSize,1);
                    
                    for j=1:optParams.vol_r_size
                        tvar(j) = params.vol_r.quote(j);
                        lb(j)= 0.0001;
                        ub(j)= 10;
                    end
                       
%                     targetfun = @(tvar) TargetFunctionck1FATMSwaptionDiagGlobal(ck1F,tvar,optParams);
%                     options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
                    x=  zeros(optParams.numOfEquation,1);
%                    [ret, popt, info, covar]=levmar('TargetFunctionCK1FATMSwaptionDiagGlobal',tvar,x, 200, options,'unc',ck1F,optParams);
                     [ret, popt, info, covar]=levmar('TargetFunctionCK1FATMSwaptionDiagGlobal',tvar,x, 500, options,'unc',ck1F,optParams);
                   
                    
                    % assigning model parameters
                    k=1;
                    for j=1:optParams.vol_r_size
                       ck1F.vol_r.quote(j) = max(popt(j),0.0001);
                       k=k+1;
                    end
                    % flat extrapolation of Alpha
                    % AlphaSize != length(Alpha)
                    
                    for j=k:length(ck1F.vol_r.quote)
                       ck1F.vol_r.quote(j) = ck1F.vol_r.quote(optParams.vol_r_size);
                    end
                    
                    out.vol_r.quote = ck1F.vol_r.quote;
                    
                    N=0;
                    for i=1:params.target_size
                        if params.swaption_expiry_use_yn(i) == 1
                            Ti = params.diagSwaption.expiry(i);
                            Tj = params.diagSwaption.tenor(i);
                            out.market_swaption(i) = params.diagSwaption.blackPrice(i);
                            out.model_swaption(i)  = ck1F.ATMSwaption(Ti,Tj);
                            out.re(i) = out.model_swaption(i)/out.market_swaption(i) - 1.0;
                            out.rmseTotal = out.rmseTotal + out.re(i)*out.re(i);
                            N=N+1; 
                        end
                    end
                    out.rmseTotal = sqrt(out.rmseTotal/N);
               
                case 'globalJams'
                    
                    optParams.params = params;
                    
                    for j=1:length(ck1F.vol_r.quote)
                        if params.swaption_expiry_use_yn(j) == 1
                            targetSwaptionSize = targetSwaptionSize +1;
                        end
                    end
                    
                    optParams.vol_r_size = targetSwaptionSize;
                    
                    optParams.modelParamsSize = optParams.vol_r_size;
                    
                    if ck1F.matu <= 2.0
                        optParams.numOfEquation = targetSwaptionSize; % # of target swaption instruments
                    else
                        optParams.numOfEquation = targetSwaptionSize; % # of target swaption instruments
                    end
                    
                    out.modelParamsSize = optParams.modelParamsSize;
                    out.numOfEquation = optParams.numOfEquation;
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    lb = zeros(optParams.modelParamsSize,1);
                    ub = zeros(optParams.modelParamsSize,1);
                    
                    for j=1:optParams.vol_r_size
                        tvar(j) = params.vol_r.quote(j);
                        lb(j)= 0.0001;
                        ub(j)= 10;
                    end
                       
%                     targetfun = @(tvar) TargetFunctionck1FATMSwaptionDiagGlobal(ck1F,tvar,optParams);
%                     options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
                    x=  zeros(optParams.numOfEquation,1);
%                    [ret, popt, info, covar]=levmar('TargetFunctionCK1FATMSwaptionDiagGlobal',tvar,x, 200, options,'unc',ck1F,optParams);
                     [ret, popt, info, covar]=levmar('TargetFunctionCK1FATMSwaptionJamsDiagGlobal',tvar,x, 500, options,'unc',ck1F,optParams);
                   
                    
                    % assigning model parameters
                    k=1;
                    for j=1:optParams.vol_r_size
                       ck1F.vol_r.quote(j) = max(popt(j),0.0001);
                       k=k+1;
                    end
                    % flat extrapolation of Alpha
                    % AlphaSize != length(Alpha)
                    
                    for j=k:length(ck1F.vol_r.quote)
                       ck1F.vol_r.quote(j) = ck1F.vol_r.quote(optParams.vol_r_size);
                    end
                    
                    out.vol_r.quote = ck1F.vol_r.quote;
                    
                    N=0;
                    for i=1:params.target_size
                        if params.swaption_expiry_use_yn(i) == 1
                            Ti = params.diagSwaption.expiry(i);
                            Tj = params.diagSwaption.tenor(i);
                            out.market_swaption(i) = params.diagSwaption.blackPrice(i);
                            out.model_swaption(i)  = ck1F.ATMSwaptionJams(Ti,Tj);
                            out.re(i) = out.model_swaption(i)/out.market_swaption(i) - 1.0;
                            out.rmseTotal = out.rmseTotal + out.re(i)*out.re(i);
                            N=N+1; 
                        end
                    end
                    out.rmseTotal = sqrt(out.rmseTotal/N);
               
                otherwise
                    disp('unImplemented')
            end
        end

        function out = TargetFunctionATMSwaptionGlobal(ck1F,tvar,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            ck1FSwaptionPrice = zeros(1,size(tenor,2));
            
            for i=1:length(ck1F.vol_r.quote)
                ck1F.vol_r.quote(i) = tvar(i);
            end
            out= 0;
            
            for j=1:length(expiry)
                for k=1:length(tenor)
                    ck1FSwaptionPrice(j,k) = ck1F.ATMSwaption(expiry(j),tenor(k));
                    out = out + (ck1FSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0)*(ck1FSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0)*10000;
                end
            end
        end   
        
        function out = TargetFunctionATMSwaption(ck1F,idxNow,tvar,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            ck1FSwaptionPrice = zeros(1,size(tenor,2));
            
            ck1F.vol_r.quote(idxNow) = tvar(1);
            
            out= 0;
            for j=1:size(tenor,2)
                ck1FSwaptionPrice(j) = ck1F.ATMSwaption(expiry(idxNow),tenor(j));
                out = out + (ck1FSwaptionPrice(j)/blackSwaptionPrice(idxNow,j)-1.0)*(ck1FSwaptionPrice(j)/blackSwaptionPrice(idxNow,j)-1.0)*10000;
            end
            
        end
        
        function out = CalibrateToATMSwaptionTenorStrip(ck1F,black,swapTenor,calibrationFlag)
            % Calculate Black ATM Swaption Prices
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
            
            expirySize = size(expiry,2);
            tenorSize = size(tenor,2);
            
            swaptionVolSurface.expiry = expiry;
            swaptionVolSurface.tenor = tenor;
            swaptionVolSurface.swaptionVol = swaptionVol; % volSurface with interpolator
            
          %% target instruments set up
            params.swaption_expiry_size = expirySize;
            params.swaption_tenor_size = tenorSize;
            
            params.swaption_calibration_yn = zeros(expirySize,tenorSize);
            
            params.swaptionSurface.expiry = swaptionVolSurface.expiry;
            params.swaptionSurface.tenor  = swaptionVolSurface.tenor;
            params.swaptionSurface.quote  = zeros(expirySize,tenorSize);
            params.swaptionSurface.strike = zeros(expirySize,tenorSize);
            params.swaptionSurface.blackSwaptionPrice = zeros(expirySize,tenorSize);
            
            % setting target diagonal swaption
            for i=1:expirySize
                for j=1:tenorSize
                    if swapTenor == params.swaptionSurface.tenor(j)
                        params.swaption_calibration_yn(i,j) = 1;
                        Ti = params.swaptionSurface.expiry(i);
                        Tj = params.swaptionSurface.tenor(j);
                        blackVolQuote = swaptionVolSurface.swaptionVol(expiry(i),tenor(j));
                        params.swaptionSurface.quote(i,j) = blackVolQuote;
                        params.swaptionSurface.strike(i,j) = black.Fsr(Ti,Ti + Tj,0.25);
                        params.swaptionSurface.blackSwaptionPrice(i,j) = black.BlackSwaptionWithVol(Ti,Tj,params.swaptionSurface.strike(i,j),blackVolQuote);
                    end
                end

            end
            
            % initialize model vol
            
            ck1F.vol_r.tenor = params.swaptionSurface.expiry;
            ck1F.vol_r.quote = 0.01*ones(length(ck1F.vol_r.tenor),1);
            
            out.vol_r = ck1F.vol_r;
            
            out.blackSwaptionPrice = params.swaptionSurface.blackSwaptionPrice;
            out.ck1FSwaptionPrice = zeros(expirySize,tenorSize);
            out.relDiff = zeros(expirySize,tenorSize);
            out.rmseTotal=0.0;
            
            calibrationType = 'bootstrap';
            if nargin < 3
                calibrationType = 'bootstrap'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;
            switch calibrationType
                case 'global'
                    for j=1:length(ck1F.vol_r.quote)
                        tvar(j) = ck1F.vol_r.quote(j);
                        lb(j)= 0.0001;
                        ub(j)= 10;
                    end
                    outSize = params.swaption_expiry_size; 

                    x =  zeros(outSize,1);
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionck1FATMSwaptionTenorStripGlobal',tvar,x, 200, options,'unc',ck1F,params);
                    toc
                    
                    for j=1:length(ck1F.vol_r.quote)
                        out.vol_r.quote(j) = popt(j);
                        ck1F.vol_r.quote(j) = popt(j);
                    end

                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            if  params.swaption_calibration_yn(j,k) == 1
%                                 out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                                out.ck1FSwaptionPrice(j,k) = ck1F.ATMSwaption(expiry(j),tenor(k));
                                out.relDiff(j,k) = out.ck1FSwaptionPrice(j,k) /out.blackSwaptionPrice(j,k)-1;
                                out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                                N=N+1;
                            end
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                otherwise
                    disp('unImplemented')
            end
        end
        
        function out = CalibrateToATMSwaptionTenorDiagSwaption(ck1F,black,joint_swap_tenor,calibrationFlag)
          % Market Data Object , Vanilla Object
          
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
            
            expirySize = size(expiry,2);
            tenorSize = size(tenor,2);
            
            swaptionVolSurface.expiry = expiry;
            swaptionVolSurface.tenor = tenor;
            swaptionVolSurface.swaptionVol = swaptionVol; % volSurface with interpolator
            
          %% select the interpolated target diagonal swaption with expiry &
          %% tenor & linear interpolated vol
            % size for interpolated yearly call (swaption)
            
            % dummy
%             target_size = lgm1F.targetSize;

            params.matu = ck1F.matu;
            params.freq = ck1F.freq;
            params.joint_swap_tenor = joint_swap_tenor;
            
%             params.dh_initial = 1.0;
            params.vol_initial = 0.01;
            params.smallNumber = 0.000001;
%             
            % set the target swaption expiry & tenor
            remainingMatu = params.matu - floor(params.matu);
            max_swaption_expiry_size = ceil(params.matu);
            
            if remainingMatu == 0
                remainingMatu = 1.0;
            end
            
            if params.matu -1  < params.joint_swap_tenor
                max_swaption_tenor_size = floor(params.joint_swap_tenor);
            else
                max_swaption_tenor_size = floor(params.matu - params.smallNumber);
            end
            
            params.targetSwaption.expiry = zeros(max_swaption_expiry_size,1);
            params.targetSwaption.tenor  = zeros(max_swaption_tenor_size,1);
            params.targetSwaption.quote  = zeros(max_swaption_expiry_size,max_swaption_tenor_size);
            params.swaption_use_yn = zeros(max_swaption_expiry_size,max_swaption_tenor_size);
            
            for i=1:max_swaption_expiry_size
                params.targetSwaption.expiry(i) = remainingMatu + i-1;
            end
            
            for i=1:max_swaption_tenor_size
                params.targetSwaption.tenor(i) = i;
            end
            
            params.target_size = 0;
            for i=1:max_swaption_expiry_size
                swaptionExpiry = params.targetSwaption.expiry(i);
                for j=1:max_swaption_tenor_size
                    swaptionTenor = params.targetSwaption.tenor(j);
                    diag_yn = 0;
                    tenor_yn = 0;
                    
                    if abs(swaptionExpiry + swaptionTenor - params.matu) < params.smallNumber 
                        diag_yn = 1;
                    end
                    
                    if (swaptionTenor == params.joint_swap_tenor) && ((swaptionExpiry <= params.matu) || abs(swaptionExpiry - params.matu) < params.smallNumber)
                        tenor_yn = 1;
                    end
                    
                    if (diag_yn) || (tenor_yn)
                        params.swaption_use_yn(i,j) = 1;
                        params.target_size = params.target_size +1;
                    end
                end
            end
            
            params.target_expiry_idx = zeros(params.target_size,1);
            params.target_tenor_idx = zeros(params.target_size,1);
            params.targetSwaption.strike = zeros(params.target_size,1);
            params.targetSwaption.blackPrice = zeros(params.target_size,1);
            
            idx = 1;
            for i=1:max_swaption_expiry_size
                for j=1:max_swaption_tenor_size
                    if params.swaption_use_yn(i,j) == 1
                        params.target_expiry_idx(idx) = i;
                        params.target_tenor_idx(idx) = j;
                        idx = idx +1;
                    end
                end
            end
            
%             dh_size = floor( params.matu) + 1 + params.joint_swap_tenor + 1;
%             params.dH1.tenor = zeros(dh_size,1);
%             params.dH1.quote = zeros(dh_size,1);
            
%             for i=1:dh_size
%                 params.dH1.tenor(i) = remainingMatu + i-1;
%                 params.dH1.quote(i) = params.dh_initial;
%             end
            
            vol_size = floor(params.matu - params.smallNumber) + 2;
            params.vol_r.tenor = zeros(vol_size,1);
            params.vol_r.quote = zeros(vol_size,1);
            
            for i=1:vol_size
                params.vol_r.tenor(i) = remainingMatu + i-1;
                params.vol_r.quote(i) = params.vol_initial;
            end
            
            upper_expiry = swaptionVolSurface.expiry(end);
            upper_tenor  = swaptionVolSurface.tenor(end);
            lower_expiry = swaptionVolSurface.expiry(1);
            
            tau = 1.0/(ck1F.freq);
            for i=1:params.target_size
                expiry_idx = params.target_expiry_idx(i);
                tenor_idx = params.target_tenor_idx(i);
                swaptionExpiry = params.targetSwaption.expiry(expiry_idx);
                swaptionTenor = params.targetSwaption.tenor(tenor_idx);
                
                if(swaptionExpiry > upper_expiry && swaptionTenor <= upper_tenor)
                    params.targetSwaption.quote(expiry_idx,tenor_idx) = swaptionVolSurface.swaptionVol(upper_expiry,swaptionTenor);
                elseif(swaptionExpiry <= upper_expiry && swaptionTenor > upper_tenor)
                    params.targetSwaption.quote(expiry_idx,tenor_idx) = swaptionVolSurface.swaptionVol(swaptionExpiry,upper_tenor);    
                else
                    if swaptionExpiry < lower_expiry
                        params.targetSwaption.quote(expiry_idx,tenor_idx) = swaptionVolSurface.swaptionVol(lower_expiry,swaptionTenor);
                    else
                        params.targetSwaption.quote(expiry_idx,tenor_idx) = swaptionVolSurface.swaptionVol(swaptionExpiry,swaptionTenor);
                    end
                end
                
                params.targetSwaption.strike(i) = black.Fsr(swaptionExpiry,swaptionExpiry + swaptionTenor,tau);
%                 annuity = black.PV01(swaptionExpiry,swaptionExpiry + swaptionTenor,0.25);
                blackVolQuote = params.targetSwaption.quote(expiry_idx,tenor_idx); 
%                 params.targetSwaption.blackPrice(i) = black.BlackSwaptionWithVol(swaptionExpiry,swaptionTenor,params.targetSwaption.strike(i),blackVolQuote);
                params.targetSwaption.blackPrice(i) = black.BlackSwaptionWithVolTau(swaptionExpiry,swaptionTenor,params.targetSwaption.strike(i),blackVolQuote,tau);
            end
            
            
            % set the new model parameters in accordance with the target
            % instruments maturities
            
%             ck1F.dH1    = params.dH1;
            ck1F.vol_r = params.vol_r;
            
          
            
           %% target swaption price and forward swap rate
            % tracking calibration
            
            params.swaption_size = params.target_size;
            out.swaption_size = params.target_size;
            
            out.blackSwaptionPrice = params.targetSwaption.blackPrice;
            out.ck1FSwaptionPrice = zeros(params.target_size,1);
            
%             out.swaptionPriceRelDiff = zeros(params.target_size,1);
%             out.swaptionPriceRmseTotal = 0.0;
%             
            out.re = zeros(params.target_size,1);
            out.rmseTotal = 0.0;
            
%             out.dH1 = params.dH1;
            out.vol_r = params.vol_r;
            
            out.market_swaption = zeros(params.target_size,1);
            out.model_swaption = zeros(params.target_size,1);
            
            
            tvar = [];
            calibrationType = 'global';
            if nargin < 3
                calibrationType = 'global'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;

            switch calibrationType
                case 'global'
                    
                    optParams.params = params;
                    
%                     optParams.dH_size = length(params.dH1.quote);  
                    optParams.vol_r_size = length(params.vol_r.quote);
                    
                    % dH1(end-1) = dH1(end)
%                     optParams.modelParamsSize = optParams.dH_size-1 + optParams.Alpha_size;
                    optParams.modelParamsSize = optParams.vol_r_size;
                    
%                     optParams.numOfEquation = params.target_size + ...  % target swaption
%                                               optParams.Alpha_size -2 + ... % tikhonov regularization gamma constraint
%                                               optParams.dH_size;     % constraint on dH 
                                          
                    optParams.numOfEquation = params.target_size;  % target swaption
                    
                    out.modelParamsSize = optParams.modelParamsSize;
                    out.numOfEquation = optParams.numOfEquation;
                    out.target_size = params.target_size;
                    out.vol_r_size = optParams.vol_r_size;
%                     out.dH_size = optParams.dH_size;
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    lb = zeros(optParams.modelParamsSize,1);
                    ub = zeros(optParams.modelParamsSize,1);
                    
%                     for j=1:optParams.dH_size-1
%                         tvar(j) = params.dH1.quote(j);
%                         lb(j)= -10.0;
%                         ub(j)= 10.0;
%                     end
                    
                    for j=1:optParams.vol_r_size
                        tvar(j) = params.vol_r.quote(j);
                        lb(j)= 0.0001;
                        ub(j)= 10.0;
                    end
                       
%                     targetfun = @(tvar) TargetFunctionLGM1FATMSwaptionDiagGlobal(lgm1F,tvar,optParams);
%                     options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
                    x=  zeros(optParams.numOfEquation,1);
                    [ret, popt, info, covar]=levmar('TargetFunctionCK1FATMSwaptionTenorSwaptionDiagGlobal',tvar,x, 200, options,'unc',ck1F,optParams);
%                     [ret, popt, info, covar]=levmar('TargetFunctionLGM1FATMSwaptionTenorSwaptionDiagGlobal',tvar,x, 1, options,'unc',lgm1F,optParams);
                    
                    
                    % assigning model parameters
%                     for j=1:optParams.dH_size-1
%                        ck1F.dH1.quote(j) = max(abs(popt(j)),0.0001);
%                     end
%                     ck1F.dH1.quote(optParams.dH_size) = ck1F.dH1.quote(optParams.dH_size-1);
                    
                    for j=1:optParams.vol_r_size
                       ck1F.vol_r.quote(j) = max(abs(popt(j)),0.0001);
                    end

%                     out.dH1.quote = ck1F.dH1.quote;
                    out.vol_r.quote = ck1F.vol_r.quote;
                    
                    N=0;
                    for i=1:params.target_size
                        expiry_idx = params.target_expiry_idx(i);
                        tenor_idx = params.target_tenor_idx(i);
                        swaptionExpiry = params.targetSwaption.expiry(expiry_idx);
                        swaptionTenor = params.targetSwaption.tenor(tenor_idx);
                        out.market_swaption(i) = params.targetSwaption.blackPrice(i);
                        out.model_swaption(i) = ck1F.ATMSwaption(swaptionExpiry,swaptionTenor);
                        out.re(i) = out.model_swaption(i)/out.market_swaption(i) - 1.0;
                        out.rmseTotal = out.rmseTotal + out.re(i)*out.re(i);
                        N=N+1;
                    end
                    
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    
                case 'globalJams'
                    
                    optParams.params = params;
                    
%                     optParams.dH_size = length(params.dH1.quote);  
                    optParams.vol_r_size = length(params.vol_r.quote);
                    
                    % dH1(end-1) = dH1(end)
%                     optParams.modelParamsSize = optParams.dH_size-1 + optParams.Alpha_size;
                    optParams.modelParamsSize = optParams.vol_r_size;
                    
%                     optParams.numOfEquation = params.target_size + ...  % target swaption
%                                               optParams.Alpha_size -2 + ... % tikhonov regularization gamma constraint
%                                               optParams.dH_size;     % constraint on dH 
                                          
                    optParams.numOfEquation = params.target_size;  % target swaption
                    
                    out.modelParamsSize = optParams.modelParamsSize;
                    out.numOfEquation = optParams.numOfEquation;
                    out.target_size = params.target_size;
                    out.vol_r_size = optParams.vol_r_size;
%                     out.dH_size = optParams.dH_size;
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    lb = zeros(optParams.modelParamsSize,1);
                    ub = zeros(optParams.modelParamsSize,1);
                    
%                     for j=1:optParams.dH_size-1
%                         tvar(j) = params.dH1.quote(j);
%                         lb(j)= -10.0;
%                         ub(j)= 10.0;
%                     end
                    
                    for j=1:optParams.vol_r_size
                        tvar(j) = params.vol_r.quote(j);
                        lb(j)= 0.0001;
                        ub(j)= 10.0;
                    end
                       
%                     targetfun = @(tvar) TargetFunctionLGM1FATMSwaptionDiagGlobal(lgm1F,tvar,optParams);
%                     options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
                    x=  zeros(optParams.numOfEquation,1);
                    [ret, popt, info, covar]=levmar('TargetFunctionCK1FATMSwaptionTenorSwaptionJamsDiagGlobal',tvar,x, 200, options,'unc',ck1F,optParams);
%                     [ret, popt, info, covar]=levmar('TargetFunctionLGM1FATMSwaptionTenorSwaptionDiagGlobal',tvar,x, 1, options,'unc',lgm1F,optParams);
                    
                    
                    % assigning model parameters
%                     for j=1:optParams.dH_size-1
%                        ck1F.dH1.quote(j) = max(abs(popt(j)),0.0001);
%                     end
%                     ck1F.dH1.quote(optParams.dH_size) = ck1F.dH1.quote(optParams.dH_size-1);
                    
                    for j=1:optParams.vol_r_size
                       ck1F.vol_r.quote(j) = max(abs(popt(j)),0.0001);
                    end

%                     out.dH1.quote = ck1F.dH1.quote;
                    out.vol_r.quote = ck1F.vol_r.quote;
                    
                    N=0;
                    for i=1:params.target_size
                        expiry_idx = params.target_expiry_idx(i);
                        tenor_idx = params.target_tenor_idx(i);
                        swaptionExpiry = params.targetSwaption.expiry(expiry_idx);
                        swaptionTenor = params.targetSwaption.tenor(tenor_idx);
                        out.market_swaption(i) = params.targetSwaption.blackPrice(i);
                        out.model_swaption(i) = ck1F.ATMSwaption(swaptionExpiry,swaptionTenor);
                        out.re(i) = out.model_swaption(i)/out.market_swaption(i) - 1.0;
                        out.rmseTotal = out.rmseTotal + out.re(i)*out.re(i);
                        N=N+1;
                    end
                    
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    
                otherwise
                    disp('unImplemented')
            end
        end 
        
%         function out = CalibrateToATMSwaption(ck1F,black,calibrationFlag)
%             out.vol_r = ck1F.vol_r;
%             % Calculate Black ATM Swaption Prices
%             swaptionVol = black.swaptionVol;
%             swaptionVolData = black.mktData('swaptionVol');
%             swaptionVolrawData =  swaptionVolData.params('rawData');
%             
%             expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
%             tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
%             
%             expirySize = size(expiry,2);
%             tenorSize = size(tenor,2);
%             
%             blackSwaptionPrice = zeros(expirySize,tenorSize);
%             for k=1:expirySize
%                 for j=1:tenorSize
%                     blackSwaptionPrice(k,j) = black.BlackATMSwaption(expiry(k),tenor(j));
%                 end
%             end
%             
%             params.expiry = expiry;
%             params.tenor = tenor;
%             params.blackSwaptionPrice = blackSwaptionPrice;
%             params.expirySize = expirySize;
%             params.tenorSize =  tenorSize;
%             
%             out.blackSwaptionPrice = blackSwaptionPrice;
%             out.ck1FSwaptionPrice = zeros(expirySize,tenorSize);
%             out.relDiff = zeros(expirySize,tenorSize);
%             out.rmseTotal=0.0;
%             
%             calibrationType = 'bootstrap';
%             if nargin < 3
%                 calibrationType = 'bootstrap'; 
%             else
%                 calibrationType = calibrationFlag;
%             end
%             N=0;
%             switch calibrationType
%                 case 'bootstrap'
%                     for k=1:expirySize
%                         if k ~= 1
%                             ck1F.vol_r.quote(k) = ck1F.vol_r.quote(k-1);
%                             ck1F.vol_s.quote(k) = ck1F.vol_s.quote(k-1);
%                             ck1F.corr_rs.quote(k) = ck1F.corr_rs.quote(k-1);
%                         end    
%                         tvar = zeros(1,3);
%                         tvar(1) = ck1F.vol_r.quote(k);
%                         tvar(2) = ck1F.vol_s.quote(k);
%                         tvar(3) = ck1F.corr_rs.quote(k);
%                         
%                         lb(1) = 0.0001;
%                         lb(2) = 0.0001;
%                         ub(1) = 10.0;
%                         ub(2) = 10.0;
%                         
%                         lb(3) = -0.99;
%                         ub(3) = 0.0;
%                         
%                         targetfun = @(tvar) Targetck1FATMSwaption(ck1F,k,tvar,params);
% 
%                         options=optimset('Algorithm','interior-point');    
%                         [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);
% 
%                         out.vol_r.quote(k) = x(1);
%                         out.vol_s.quote(k) = x(2);
%                         out.corr_rs.quote(k) = x(3);
%                         
%                         ck1F.vol_r.quote(k) = x(1);
%                         ck1F.vol_s.quote(k) = x(2);
%                         ck1F.corr_rs.quote(k) = x(3);
%                         
%                         for j=1:tenorSize
%                             out.ck1FSwaptionPrice(k,j) = ck1F.ATMSwaption(expiry(k),tenor(j));
%                             out.relDiff(k,j) = out.ck1FSwaptionPrice(k,j) /out.blackSwaptionPrice(k,j)-1;
%                             out.rmseTotal = out.rmseTotal + out.relDiff(k,j)*out.relDiff(k,j);
%                             N=N+1;
%                         end
%                     end
%                     out.rmseTotal = sqrt(out.rmseTotal/N);
%                     out.vol_r = ck1F.vol_r;
%                     out.vol_s = ck1F.vol_s;
%                     out.corr_rs = ck1F.corr_rs;
%                     out.ck1F = ck1F;
%                 case 'global'
%                     for j=1:length(ck1F.vol_r.quote)
%                         tvar(j) = ck1F.vol_r.quote(j);
%                         tvar(length(ck1F.vol_r.quote)+j) = ck1F.vol_s.quote(j);
%                         lb(j)= 0.0001;
%                         lb(length(ck1F.vol_r.quote)+j)= 0.0001;
%                         ub(j)= 10;
%                         ub(length(ck1F.vol_r.quote)+j)= 10;
%                     end
%                     
%                     outSize = params.expirySize*params.tenorSize; 
%                     
%                     if 1
%                         x =  zeros(outSize,1);
%                         options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                         tic
%                         [ret, popt, info, covar]=levmar('TargetFunctionck1FATMSwaptionSurfaceGlobal',tvar,x, 200, options,'unc',ck1F,params);
%                         toc
%                         for j=1:length(ck1F.vol_r.quote)
%                             out.vol_r.quote(j) = popt(j);
%                             out.vol_s.quote(j) = popt(j+length(ck1F.vol_r.quote));
%                             ck1F.vol_r.quote(j) = popt(j);
%                             ck1F.vol_s.quote(j) = popt(j+length(ck1F.vol_s.quote));
%                         end
%                     else
%                         targetfun = @(tvar) TargetFunctionATMSwaptionGlobal(ck1F,tvar,params);
%                         options=optimset('Algorithm','interior-point');
%     %                     options=optimset('Algorithm','trust-region-reflective');
%                         [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);
% 
%                         for j=1:length(ck1F.vol_r.quote)
%                             out.vol_r.quote(j) = x(j);
%                             out.vol_s.quote(j) = x(j+length(ck1F.vol_r.quote));
%                             ck1F.vol_r.quote(j) = x(j);
%                             ck1F.vol_s.quote(j) = x(j+length(ck1F.vol_s.quote));
%                         end
%                     end
%                     N=0;
%                     for j=1:expirySize
%                         for k=1:tenorSize
%                             out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
%                             out.ck1FSwaptionPrice(j,k) = ck1F.ATMSwaption(expiry(j),tenor(k));
%                             out.relDiff(j,k) = out.ck1FSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
%                             out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
%                             N=N+1;
%                         end
%                     end
%                    
%                     out.rmseTotal = sqrt(out.rmseTotal/N);
%                 otherwise
%                     disp('unImplemented')
%             end
%         end    
    end
    
end



In [ ]:
function [output] = chol_mod(varargin)
%default params    
    a = 1e-13;
    type = 'lower';
    %%%%%%%% Input parameters
    % Input at l e a s t a matrix C
    if(nargin <1)
        disp('Scaled Spec t r a l Method : Not enough input arguments .' );
        disp('Input : C, [ a ] . Die now . .');
        return
    end
    % This i s our target matrix
    C = varargin {1};
    
    % s t a r t ing pos i t i on f o r eigenvalue value . a >= 0
    if(nargin >= 2 && isstring(varargin{2}))
        type = varargin{2};
    end
    
    % s t a r t ing pos i t i on f o r eigenvalue value . a >= 0
    if(nargin >= 3 && varargin{3} >=0)
        a = varargin{3};
    end
    
    maxIter = 10;
    
    loopNb = 0;
    result = 1;
    covar = zeros(size(C,1),size(C,2));
    diagV = zeros(size(C,1),1);
    
    while (loopNb < maxIter && result > 0)
        [outputL,result] = chol(C,type);
        if result > 0
            %compute nearest positive definite correlation matrix
            diagV =         sqrt(diag(C));
            invDiagV = 1.0./sqrt(diag(C));
            correlM = diag(invDiagV)*C*diag(invDiagV);
            outputSpectral = posdef_scaled_spectral(correlM);
            newCov = diag(diagV)*outputSpectral.C*diag(diagV);
            C = newCov;
        else
            output = outputL;
            break;
        end
    end
    
end



In [ ]:
classdef CheyetteTermRisky < IRModelRisky
    %UNTITLED Summary of this class goes here
    %   Detailed explanation goes here
    
    properties
        % vol functor
        smoothAlpha;
        numOfFactor;
        lambda = {};
        kappa = [];
        tau = [];
        corr_fwdTermSize = 0;
        corr_fwdTerm = {};
        rhoTerm = {};
        inv_H_tau = [];
    end
    
    methods
        function ch = CheyetteTermRisky(params)
            if nargin > 0
                ch.zeroCurve = params.zeroCurve;
                ch.discountCurve = params.discountCurve;
                ch.mktData = params.mktData;
                ch.modelParams = params.modelParams;
                
                ch.smoothAlpha =  params.modelParams('smoothAlpha');
                ch.numOfFactor = params.modelParams('numOfFactor');
                sizeN = ch.numOfFactor;
                ch.lambda = cell(sizeN,1);
                ch.lambda = params.modelParams('lambda');
                
                ch.kappa = ones(sizeN,1);
                ch.kappa =  params.modelParams('kappa');
               
                ch.tau = ones(sizeN,1);
                ch.tau =  params.modelParams('tau');
                ch.inv_H_tau = ones(sizeN,sizeN);
                H_tau = ones(sizeN,sizeN);
                for i=1:sizeN
                    for j=1:sizeN
                        H_tau(i,j) = exp(-ch.kappa(j)* ch.tau(i));
                    end
                end
                ch.inv_H_tau = inv(H_tau);
                
%                 ch.corr_fwd = zeros(sizeN,sizeN);
                ch.corr_fwdTermSize = params.modelParams('corr_fwdTermSize');
                ch.corr_fwdTerm = cell(ch.corr_fwdTermSize,1);
                
                ch.corr_fwdTerm =  params.modelParams('corr_fwdTerm');
                
                ch.rhoTerm = cell(ch.corr_fwdTermSize,1);
                
                for idx =1:ch.corr_fwdTermSize
                    [PC,V]=eig(ch.corr_fwdTerm{idx,1}.quote);
                    PCW= PC*sqrt(V);
                    for i=1:sizeN
                        for j=1:sizeN
                            ch.rhoTerm{idx,1}.quote(i,j)=PCW(i,j);
                        end
                    end
                end
                
           
                
            end
        end
        
        function out = SimpleSRPrOverHedge(ch,spreadC,spreadN,ralower,raupper)
            buffer = 0.001;
            outVec = 0.0;
            if spreadC < ralower - buffer
                outVec = 0.0;
            elseif spreadC >= ralower - buffer && spreadC < ralower
                outVec = 1.0/buffer * (spreadC - (ralower - buffer));
            elseif spreadC >= ralower && spreadC < raupper
                outVec = 1.0;
            elseif spreadC >= raupper && spreadC < raupper + buffer
                outVec = -1.0/buffer *(spreadC - (raupper + buffer));
            else
                outVec = 0.0;
            end
            
            out = outVec;
        end
        
        function out = SimpleSRPr(ch,spreadC,spreadN,ralower,raupper)
            outVec = 0.0;
            if spreadC >= ralower && spreadC <= raupper
                outVec = 1.0;
            end
            out = outVec;
        end
        
        
        function out = TRZ_SRPr(ch,curr,next,L,U)
            length = 1.0;
            remain = 1.0;
            if (curr >= L && curr <= U && next >= L && next <= U)
                length = remain;
            elseif ( curr >= L && curr <= U && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr >= L && curr <= U && next < L ) 
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr > U && next >= L && next <= U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next < L )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next >= L && next <= U )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next > U )
                length = 0.0;
            elseif ( curr < L && next < L)
                length = 0.0;
            else
                disp('error')
            end
            
            out = length;
        end
        
        function out = SRPr(ch,spreadC,spreadN,ralower,raupper,simpleYN,pathSize)
            
            outVec = zeros(1,pathSize);
            
            if simpleYN == 1
                for i =1:pathSize
                    outVec(i) = ch.SimpleSRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            elseif simpleYN == 2
                for i =1:pathSize
                    outVec(i) = ch.SimpleSRPrOverHedge(spreadC(i),spreadN(i),ralower,raupper);
                end
            else
                for i =1:pathSize
                    outVec(i) = ch.TRZ_SRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            end
            
            out = outVec;
        end
        
        function out =  CMS_PSA_SimpleDate(ch,valueDate,observeTime,numMatTime, tenor,modelStatesT)
           %CMS
           tau = 0.25;
           to      = observeTime/365.0;
           toDate = valueDate.AddDate(observeTime,'day');
           tNDate = toDate.AddDate(tenor,'year');
           tN   =  tNDate.DateDiff(valueDate)/365.0;
           
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           p_nN = ch.PV01Date(valueDate,toDate,tNDate,tau);
           
           df_Tn = ch.DF(to);
           df_TN = ch.DF(tN);
           y_nN = (df_Tn - df_TN)/p_nN;
           
           c_nN = ch.C_nNDate(valueDate,toDate,tNDate,tau,p_nN,y_nN);
           g_nN = ch.G_nNDate(valueDate,toDate,tNDate,tau,p_nN);
           h_nTerm = ch.H_nTermDate(valueDate,toDate,numMatDate);
           varCovar = ch.LocalVariance(0,to);
           
           sizeN = ch.numOfFactor;
            
           stateSize = size(modelStatesT,2);
           cms_PCA_SimpleOut = zeros(1,stateSize);
           
           driftTerm = 0;
           for i=1:sizeN
               for j=1:sizeN
                   driftTerm = driftTerm + (c_nN(i) * h_nTerm(j) + c_nN(i) * g_nN(j))*varCovar(i,j);
               end
           end
           
           for i=1:stateSize
               cms_PCA_SimpleOut(i) = y_nN + driftTerm;
               for j=1:sizeN
                cms_PCA_SimpleOut(i) = cms_PCA_SimpleOut(i) + c_nN(j)*modelStatesT(j,i);
               end
           end
           
           out = cms_PCA_SimpleOut;
           
            
        end
        
        function out = CheyetteGeneralVariance(ch,from,to,T1,T2)
            
            sizeN = ch.numOfFactor;
            plambda =cell(sizeN,1);
            for i=1:sizeN
                plambda{i,1}= ch.lambda{i,1};
            end
            
            sigma_f = zeros(sizeN,sizeN);
            
            % eta is short rate volatlity
            % eta2 is short rate covariance matrix

            for i=1:sizeN
                for j=1:sizeN
                    sigma_f(i,j)=sqrt(plambda{i,1}.quote(1)*plambda{i,1}.quote(1))*ch.rhoTerm{1,1}.quote(i,j);
                end
            end
            
           eta = zeros(sizeN,sizeN);
           eta = ch.inv_H_tau*sigma_f;
           eta2 = eta*eta';
 
           sigmaTimes = plambda{1,1}.tenor;
           sigmaSize = length(plambda{1,1}.tenor);
            
            for i=1:sigmaSize
                if from<= sigmaTimes(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:sigmaSize
                if to<= sigmaTimes(i)
                    break;
                end
            end
            endIdx = i;
            
            lastExpScale = zeros(4,sizeN,sizeN);
            newExpScale  = zeros(4,sizeN,sizeN);
            variances = zeros(4,sizeN,sizeN);
            meanR = zeros(sizeN,1);
            
            for i=1:sizeN
                meanR(i) = ch.kappa(i);
            end
%             newExpScale = zeros(4,1);
%             lastExpScale = zeros(4,1);
%             variances = zeros(4,1);
            
            lastU  = from;
            
            for j=1:sizeN
                for k=1:sizeN
                    lastExpScale(2,j,k) = exp(meanR(j)*lastU);
                    lastExpScale(3,j,k) = exp(meanR(k)*lastU);
                    lastExpScale(4,j,k) = exp((meanR(j)+meanR(k))*lastU);
                end
            end
            
            for i=startIdx:endIdx
                if (i< endIdx) U = sigmaTimes(i);else U = to;end;
                if (i< sigmaSize) tindex = i;else tindex = sigmaSize;end;
            % eta is short rate volatlity
                for j=1:sizeN
                    for k=1:sizeN
                        sigma_f(j,k)=sqrt(plambda{j,1}.quote(tindex)*plambda{j,1}.quote(tindex))*ch.rhoTerm{tindex,1}.quote(j,k);
                    end
                end
                
                eta = ch.inv_H_tau*sigma_f;
                eta2 = eta*eta';
                
                for j=1:sizeN
                    for k=1:sizeN
                        newExpScale(2,j,k) = exp(meanR(j)*U);
                        newExpScale(3,j,k) = exp(meanR(k)*U);
                        newExpScale(4,j,k) = exp((meanR(j)+meanR(k))*U);
                    
                        variances(1,j,k) = variances(1,j,k) + eta2(j,k) * (U - lastU);
                        variances(2,j,k) = variances(2,j,k) + eta2(j,k) * (1.0/ meanR(j))*(newExpScale(2,j,k) - lastExpScale(2,j,k));
                        variances(3,j,k) = variances(3,j,k) + eta2(j,k) * (1.0/ meanR(k))*(newExpScale(3,j,k) - lastExpScale(3,j,k));
                        variances(4,j,k) = variances(4,j,k) + eta2(j,k) * (1.0/ (meanR(j)+meanR(k)))*(newExpScale(4,j,k) - lastExpScale(4,j,k));
                    end
                end
                
                lastU = U;
                for j=1:sizeN
                    for k=1:sizeN
                        lastExpScale(2,j,k) = newExpScale(2,j,k);
                        lastExpScale(3,j,k) = newExpScale(3,j,k);
                        lastExpScale(4,j,k) = newExpScale(4,j,k);
                    end
                end
                
            end
            
            value = 0;
            
            for j=1:sizeN
                for k=1:sizeN
                    value = value + (1.0/ meanR(j))*(1.0/meanR(k))*variances(1,j,k);
                end
            end
            
            for j=1:sizeN
                for k=1:sizeN
                    value = value + -1.0*exp(-1.0*meanR(j)*T1)*(1.0/ meanR(j))*(1.0/meanR(k))*variances(2,j,k);
                end
            end
            
            for j=1:sizeN
                for k=1:sizeN
                    value = value + -1.0*exp(-1.0*meanR(k)*T2)*(1.0/ meanR(j))*(1.0/meanR(k))*variances(3,j,k);
                end
            end
            
            for j=1:sizeN
                for k=1:sizeN
                    value = value +  1.0*exp(-1.0*meanR(j)*T1)*exp(-1.0*meanR(k)*T2)*(1.0/ meanR(j))*(1.0/meanR(k))*variances(4,j,k);
                end
            end
            
            out  = value;
        end
        
        function out = fwdZcVariance(ch, from, to, T1, T2)
            
            localVariance = zeros(2,1);
            
            localVariance(1) = ch.CheyetteGeneralVariance(from,to,T1,T1);
            localVariance(2) = ch.CheyetteGeneralVariance(from,to,to,to);
            
            
            value = localVariance(1) - localVariance(2);
            
            measureChange = zeros(2,1);
            
            measureChange(1) = ch.CheyetteGeneralVariance(from,to,T1,T2);
            measureChange(2) = ch.CheyetteGeneralVariance(from,to,to,T2);
            
            measureChangeTerm = measureChange(1) - measureChange(2);
            
            value = value + -2.0*measureChangeTerm;
            
            out = value;
            
        end
        
        function out = discountFactorRisky(ch, startTime, endTime,numMatTime,modelStatesT)
           zcT = ch.DFRisky(endTime/365.0);
           zct = ch.DFRisky(startTime/365.0);
           beta = ch.Beta(startTime/365.0,endTime/365.0);
           zcDriftTerm = ch.fwdZcVariance(0,startTime/365.0,endTime/365.0,numMatTime/365.0);
           %numOfPath
           stateSize = size(modelStatesT,2);
           discountFactorOut = zeros(1,stateSize);
           
           sizeN = ch.numOfFactor;
           
           for i=1:stateSize
               discountFactorOut(i) = zcT/zct * exp(-0.5*zcDriftTerm);
               for j=1:sizeN
                    discountFactorOut(i) = discountFactorOut(i) * exp(-beta(j)*modelStatesT(j,i));
               end
           end
           
           out = discountFactorOut;
        end
        
        function out = discountFactor(ch, startTime, endTime,numMatTime,modelStatesT)
           zcT = ch.DF(endTime/365.0);
           zct = ch.DF(startTime/365.0);
           beta = ch.Beta(startTime/365.0,endTime/365.0);
           zcDriftTerm = ch.fwdZcVariance(0,startTime/365.0,endTime/365.0,numMatTime/365.0);
           %numOfPath
           stateSize = size(modelStatesT,2);
           discountFactorOut = zeros(1,stateSize);
           
           sizeN = ch.numOfFactor;
           
           for i=1:stateSize
               discountFactorOut(i) = zcT/zct * exp(-0.5*zcDriftTerm);
               for j=1:sizeN
                    discountFactorOut(i) = discountFactorOut(i) * exp(-beta(j)*modelStatesT(j,i));
               end
           end
           
           out = discountFactorOut;
        end
        
        function out = discountPayoffRisky(ch,eventTime,numMatTime ...
                ,modelStatesT,cashflow)
            
            dfNumMat = ch.DFRisky(numMatTime/365.0);
            % for discountPayoff T=numMatTime
            % P(0,TnumMat)/P(Te,Tnummat)*Payoff
            %
            discountT = ch.discountFactorRisky(eventTime, numMatTime,numMatTime,modelStatesT);
            stateSize = size(modelStatesT,2);
            payoff = zeros(1,stateSize);
            for i=1:stateSize
                payoff(i)= dfNumMat/discountT(i)*cashflow(i);
            end    
            out = payoff;
        end
        
        function out = discountPayoff(ch,eventTime,numMatTime ...
                ,modelStatesT,cashflow)
            
            dfNumMat = ch.DF(numMatTime/365.0);
            % for discountPayoff T=numMatTime
            % P(0,TnumMat)/P(Te,Tnummat)*Payoff
            %
            discountT = ch.discountFactor(eventTime, numMatTime,numMatTime,modelStatesT);
            stateSize = size(modelStatesT,2);
            payoff = zeros(1,stateSize);
            for i=1:stateSize
                payoff(i)= dfNumMat/discountT(i)*cashflow(i);
            end    
            out = payoff;
        end
        
        function out = stateLocalDrift(ch,fromTime, toTime)
            sizeN = ch.numOfFactor;
            drift = zeros(sizeN,sizeN);
            for i=1:sizeN
                drift(i,i) =  exp(-1.0*ch.kappa(i)*(toTime-fromTime)/365.0);
            end
            out = drift;
        end
        
        function out = stateLocalVariance(ch,fromTime, toTime)
            out = ch.LocalVariance(fromTime/365.0,toTime/365.0);
        end
        
        function variances = LocalVariance(ch,from,to)
            sizeN = ch.numOfFactor;
            plambda =cell(sizeN,1);
            for i=1:sizeN
                plambda{i,1}= ch.lambda{i,1};
            end
            sigma_f =[];
            sigma_f = zeros(sizeN,sizeN);
%% converting forward rate volatlity into short rate volatlitity
%% eta is short rate volatlity
            for i=1:sizeN
                for j=1:sizeN
%                     sigma_f(i,j)=plambda{i,1}.quote(1)*ch.Fwd(0,ch.tau(i))*ch.rho(i,j);
%                     sigma_f(i,j)=plambda{i,1}.quote(1)*ch.rho(i,j);
                    sigma_f(i,j)=sqrt(plambda{i,1}.quote(1)*plambda{i,1}.quote(1))*ch.rhoTerm{1,1}.quote(i,j);
                end
            end
            
           eta =[];
           eta = zeros(sizeN,sizeN);
           eta = ch.inv_H_tau*sigma_f;
           eta2 = eta*eta';
           %%
           alphaSize = length(plambda{1,1}.tenor);
            
           for i=1:alphaSize
               if from<= plambda{1,1}.tenor(i);
                   break;
               end
           end
            
           startIdx = i;
            
%             endIdx = size(sig(1).tenor,1); 
           for i=1:alphaSize
               if to<= plambda{1,1}.tenor(i);
                   break;
               end
           end
           endIdx = i;
            
           lastU  = from;
           
           tiny = zeros(sizeN,sizeN);
           lastExpScale = zeros(sizeN,sizeN);
           newExpScale  = zeros(sizeN,sizeN);
           scale = 0.0;
           meanR = zeros(sizeN,1);
           for i=1:sizeN
                meanR(i) = ch.kappa(i);
           end
           for i=1:sizeN
                for j=1:sizeN
                    if(abs(meanR(i)+meanR(j)) < 1.0E-12) tiny(i,j) = 1; else tiny(i,j) = 0; end;
                    lastExpScale(i,j) =  exp((meanR(i)+meanR(j))*from);
                end
           end
            
           variances = zeros(sizeN,sizeN);
           tindex = 1;
           for i=startIdx:endIdx
                if (i< endIdx) U = plambda{1,1}.tenor(i);else U = to;end;
                if (i< alphaSize) tindex = i;else tindex = alphaSize;end;
            %% converting forward rate volatlity into short rate volatlitity
            %% eta is short rate volatlity
                for j=1:sizeN
                    for k=1:sizeN
%                         sigma_f(j,k)=plambda{j,1}.quote(tindex)*ch.Fwd(0,ch.tau(j))*ch.rho(j,k);
%                         sigma_f(j,k)=plambda{j,1}.quote(tindex)*ch.rho(j,k);
                        sigma_f(j,k)=sqrt(plambda{j,1}.quote(tindex)*plambda{j,1}.quote(tindex))*ch.rhoTerm{tindex,1}.quote(j,k);
                    end
                end
                
                eta = ch.inv_H_tau*sigma_f;
                eta2 = eta*eta';
            %%
                for j=1:sizeN
                    for k=1:sizeN
                        newExpScale(j,k) =  exp((meanR(j)+meanR(k))*U);
                        if(tiny(j,k)~=1)
                            scale = newExpScale(j,k)-lastExpScale(j,k);
                        else
                            scale = U - lastU;
                        end
                        variances(j,k) = variances(j,k) + eta2(j,k) * scale;
                        lastExpScale(j,k) = newExpScale(j,k);
                    end
                end
                lastU = U;
           end
           
           for j=1:sizeN
                for k=1:sizeN
                    if(tiny(j,k)~= 1)
                        variances(j,k) = variances(j,k)/(meanR(j)+meanR(k))*exp(-1.0*(meanR(j)+meanR(k))*to);
                    end
                end
           end
            
            
        end

        function beta = Beta(ch,t,T)
            beta = [];
            sizeN = ch.numOfFactor;
            beta = zeros(sizeN,1);
            
            for i=1:sizeN
                if(abs(ch.kappa(i)) > 1.0E-12)
                    beta(i) = (1.0-exp(-ch.kappa(i)*(T-t)))/ch.kappa(i);
                else
                    beta(i) = (T-t);
                end
            end
        end
        
        function beta = BetaDate(ch,valueDate,tDate,TDate)
            t = tDate.DateDiff(valueDate)/365.0;
            T = TDate.DateDiff(valueDate)/365.0;
            
            sizeN = ch.numOfFactor;
            beta = zeros(sizeN,1);
            
            for i=1:sizeN
                if(abs(ch.kappa(i)) > 1.0E-12)
                    beta(i) = (1.0-exp(-ch.kappa(i)*(T-t)))/ch.kappa(i);
                else
                    beta(i) = (T-t);
                end
            end
            
        end
        
        function c_nN = C_nN(ch,tn,tN,tau,pnN,ynN)
            p = @ch.DF;
            c_nN = [];
            sizeN = ch.numOfFactor;
            c_nN = zeros(sizeN,1);
            
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = ch.Beta(tn,tp);
                df = p(tp);
                for j=1:sizeN
                    c_nN(j) = c_nN(j) +tau * beta(j) * df;
                end
            end
            
            for j=1:sizeN
                c_nN(j) = c_nN(j)* ynN;
            end
            
            for j=1:sizeN
                c_nN(j) = c_nN(j) + beta(j)*df;
            end
            
            for j=1:sizeN
                c_nN(j) = c_nN(j)/pnN;
            end
           
        end
        
        function c_nN = C_nNDate(ch,valueDate,tnDate,tNDate,tau,pnN,ynN)
            tn = tnDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            
            addMonthUnit = round(12*tau);
            cnt = round((tN-tn)/tau);
            
            p = @ch.DF;
            sizeN = ch.numOfFactor;
            c_nN = zeros(sizeN,1);
            
            for i=1:cnt
                tpDate = tnDate.AddDate(i*addMonthUnit,'month');
                tp = tpDate.DateDiff(valueDate)/365.0;
                beta = ch.BetaDate(valueDate,tnDate,tpDate);
                df = p(tp);
                for j=1:sizeN
                    c_nN(j) = c_nN(j) +tau * beta(j) * df;
                end
            end
            
            for j=1:sizeN
                c_nN(j) = c_nN(j)* ynN;
            end
            
            for j=1:sizeN
                c_nN(j) = c_nN(j) + beta(j)*df;
            end
            
            for j=1:sizeN
                c_nN(j) = c_nN(j)/pnN;
            end
        end
        
        function g_nN = G_nN(ch,tn,tN,tau,pnN)
            p = @ch.DF;
            g_nN = [];
            sizeN = ch.numOfFactor;
            g_nN = zeros(sizeN,1);
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = ch.Beta(tn,tp);
                df = p(tp);
                for j=1:sizeN
                    g_nN(j) = g_nN(j) +tau * beta(j) * df;
                end
            end
            
            for j=1:sizeN
                g_nN(j) = g_nN(j)/pnN;
            end
            
        end
        
        function g_nN = G_nNDate(ch,valueDate,tnDate,tNDate,tau,pnN)
            tn = tnDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            addMonthUnit = round(12*tau);
            cnt = round((tN-tn)/tau);
            
            p = @ch.DF;
            sizeN = ch.numOfFactor;
            g_nN = zeros(sizeN,1);
            
            for i=1:cnt
                tpDate = tnDate.AddDate(i*addMonthUnit,'month');
                tp = tpDate.DateDiff(valueDate)/365.0;
                beta = ch.BetaDate(valueDate,tnDate,tpDate);
                df = p(tp);
                for j=1:sizeN
                    g_nN(j) = g_nN(j) +tau * beta(j) * df;
                end
            end
            
            for j=1:sizeN
                g_nN(j) = g_nN(j)/pnN;
            end
        end 
        
        function h_nTerm = H_nTerm(ch,tn,tTerm)
           h_nTerm =[];
           sizeN = ch.numOfFactor;
           h_nTerm = zeros(sizeN,1);
           beta = ch.Beta(tn,tTerm);
           
           for j=1:sizeN
                h_nTerm(j) = -1.0*beta(j);
           end
          
        end
        
        function h_nTerm = H_nTermDate(ch,valueDate,tnDate,tTermDate)
           h_nTerm =[];
           sizeN = ch.numOfFactor;
           h_nTerm = zeros(sizeN,1);
           beta = ch.BetaDate(valueDate,tnDate,tTermDate);
           for j=1:sizeN
                h_nTerm(j) = -1.0*beta(j);
           end
        end
        
        function out = findTindex(ch,t,sigma)
            
            sum = 0.0;
            prev_t = 0.0;
            inRangeYN = false;
            for i=1:length(sigma.tenor)
                alpha = i;
                if t<= sigma.tenor(i)
                    sum = alpha;
                    inRangeYN = true;
                    break;
                end
            end
            
            if ~inRangeYN 
                sum = alpha;
            end
            out = sum;
        end
        
        function instFwdCorrTest = InsFwdCorrTest(ch,tx,tnr1,tnr2)
            sizeN = ch.numOfFactor;
            plambda =cell(sizeN,1);
            for i=1:sizeN
                plambda{i,1}= ch.lambda{i,1};
            end
            sigma_f =[];
            sigma_f = zeros(sizeN,sizeN);
%             labmdaTest.quote = plambda{1,1}.quote;
%             lambdaTest.tenor = plambda{1,1}.tenor;
            tindex = ch.findTindex(tx+1e-6,plambda{1,1});
            
            for j=1:sizeN
                for k=1:sizeN
%                     sigma_f(j,k)=plambda{j,1}.quote(tindex)*ch.Fwd(0,ch.tau(j))*ch.rho(j,k);
                    sigma_f(j,k)=plambda{j,1}.quote(tindex)*ch.rhoTerm{tindex,1}.quote(j,k);
                end
            end
                
            eta = ch.inv_H_tau*sigma_f;
            eta2 = eta*eta';
            
            KappaTau1 = zeros(2,1);
            KappaTau2 = zeros(2,1);
            
            for i=1:sizeN
                KappaTau1(i) = exp(-ch.kappa(i)*tnr1);
                KappaTau2(i) = exp(-ch.kappa(i)*tnr2);
            end
            
            
            covar = 0.0;
            var1 = 0;
            var2 = 0;
            var1 =  KappaTau1'*eta2*KappaTau1;
            var2 =  KappaTau2'*eta2*KappaTau2;
            covar =  KappaTau1'*eta2*KappaTau2;
            
            instFwdCorrTest = covar/sqrt(var1*var2);
        end
        
        function InstSwapFwdCorrTest = InsSwapFwdCorrTest(ch,tx,tnr1,tnr2)
            sizeN = ch.numOfFactor;
            plambda =cell(sizeN,1);
            for i=1:sizeN
                plambda{i,1}= ch.lambda{i,1};
            end
            sigma_f =[];
            sigma_f = zeros(sizeN,sizeN);
            tindex = ch.findTindex(tx+1e-6,plambda{1,1});
            
            for j=1:sizeN
                for k=1:sizeN
%                     sigma_f(j,k)=plambda{j,1}.quote(tindex)*ch.Fwd(0,ch.tau(j))*ch.rho(j,k);
                    sigma_f(j,k)=plambda{j,1}.quote(tindex)*ch.rhoTerm{tindex,1}.quote(j,k);
                end
            end
                
            eta = ch.inv_H_tau*sigma_f;
            eta2 = eta*eta';
            
            KappaTau1 = zeros(2,1);
            KappaTau2 = zeros(2,1);
            
            tau = 0.25;
            p_nN1 = ch.PV01(tx,tx+tnr1,tau);
            df_Tn1 = ch.DF(tx);
            df_TN1 = ch.DF(tx + tnr1);
            y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
            c_nN1 = ch.C_nN(tx,tx + tnr1,tau,p_nN1,y_nN1);
            
            p_nN2 = ch.PV01(tx,tx+tnr2,tau);
            df_Tn2 = ch.DF(tx);
            df_TN2 = ch.DF(tx + tnr2);
            y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
            c_nN2 = ch.C_nN(tx,tx + tnr2,tau,p_nN2,y_nN2);
            
            for i=1:sizeN
                KappaTau1(i) = c_nN1(i)*exp(-ch.kappa(i)*tnr1);
                KappaTau2(i) = c_nN2(i)*exp(-ch.kappa(i)*tnr2);
            end
            
            
            covar = 0.0;
            var1 = 0;
            var2 = 0;
            var1 =  KappaTau1'*eta2*KappaTau1;
            var2 =  KappaTau2'*eta2*KappaTau2;
            covar =  KappaTau1'*eta2*KappaTau2;
            
            InstSwapFwdCorrTest = covar/sqrt(var1*var2);
        end
        
        function out = CMSDigitalArg(ch,to,tp,tenor1,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           sizeN = ch.numOfFactor;
           p_nN1 = ch.PV01(to,to+tenor1,tau);
           df_Tn1 = ch.DF(to);
           df_TN1 = ch.DF(to + tenor1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           c_nN1 = ch.C_nN(to,to + tenor1,tau,p_nN1,y_nN1);
           g_nN1 = ch.G_nN(to,to + tenor1,tau,p_nN1);
           h_nTerm1 = ch.H_nTerm(to,tp);
           
           varCovar = ch.LocalVariance(0,to);
           
           driftTerm1=0.0;
           for i=1:sizeN
               for j=1:sizeN
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           for i=1:sizeN
               for j=1:sizeN
                    spreadVariance = spreadVariance + (c_nN1(i)) * (c_nN1(j)) * varCovar(i,j);
               end
           end
           
           out.mu = (y_nN1 + driftTerm1);
           if spreadVariance >=0
            out.sigma = sqrt(spreadVariance);
           else
            out.sigma = 0.0;
           end
           
           out.c_nN1 = c_nN1;
           out.varCovar = varCovar;
           
        end
        
        function out = CMSDigital(ch,to,tp,tenor1,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsDigitalArg = ch.CMSDigitalArg(to,tp,tenor1,tau);
           mu = cmsDigitalArg.mu;
           sigma = cmsDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSDigitalRange(ch,to,tp,tenor1,tau,lower,upper)
            lower_value = ch.CMSDigital(to,tp,tenor1,tau,lower);
            upper_value = ch.CMSDigital(to,tp,tenor1,tau,upper);
            out = lower_value - upper_value;
        end
        
        
        function out = CMSSpreadDigitalArg(ch,to,tp,tenor1,tenor2,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           sizeN = ch.numOfFactor;
           p_nN1 = ch.PV01(to,to+tenor1,tau);
           df_Tn1 = ch.DF(to);
           df_TN1 = ch.DF(to + tenor1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           c_nN1 = ch.C_nN(to,to + tenor1,tau,p_nN1,y_nN1);
           g_nN1 = ch.G_nN(to,to + tenor1,tau,p_nN1);
           h_nTerm1 = ch.H_nTerm(to,tp);
           
           %CMS2
           p_nN2 = ch.PV01(to,to+tenor2,tau);
           df_Tn2 = ch.DF(to);
           df_TN2 = ch.DF(to + tenor2);
           y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
           c_nN2 = ch.C_nN(to,to + tenor2,tau,p_nN2,y_nN2);
           g_nN2 = ch.G_nN(to,to + tenor2,tau,p_nN2);
           h_nTerm2 = ch.H_nTerm(to,tp);
           
           varCovar = ch.LocalVariance(0,to);
           
           driftTerm1=0.0;
           driftTerm2=0.0;
           for i=1:sizeN
               for j=1:sizeN
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
                    driftTerm2 = driftTerm2 + c_nN2(i) * (h_nTerm2(j) + g_nN2(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           for i=1:sizeN
               for j=1:sizeN
                    spreadVariance = spreadVariance + (c_nN1(i)-c_nN2(i)) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
               end
           end
           
           out.mu = (y_nN1 + driftTerm1) - (y_nN2 + driftTerm2);
           out.sigma = sqrt(spreadVariance);
           out.c_nN1 = c_nN1;
           out.c_nN2 = c_nN2;
           out.varCovar = varCovar;
           
        end
        
        function out = CMSSpreadDigitalArgDate(ch,valueDate,toDate,tpDate,tenor1,tenor2,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           sizeN = ch.numOfFactor;
           
           to      = toDate.DateDiff(valueDate)/365.0;
           tN1Date = toDate.AddDate(tenor1,'year');
           tN1   =  tN1Date.DateDiff(valueDate)/365.0;
           
           p_nN1 = ch.PV01Date(valueDate,toDate,tN1Date,tau);
           
           df_Tn1 = ch.DF(to);
           df_TN1 = ch.DF(tN1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           
           c_nN1 = ch.C_nNDate(valueDate,toDate,tN1Date,tau,p_nN1,y_nN1);
           g_nN1 = ch.G_nNDate(valueDate,toDate,tN1Date,tau,p_nN1);
           h_nTerm1 = ch.H_nTermDate(valueDate,toDate,tpDate);
           
           %CMS2
           tN2Date = toDate.AddDate(tenor2,'year');
           tN2   =  tN2Date.DateDiff(valueDate)/365.0;
           
           p_nN2 = ch.PV01Date(valueDate,toDate,tN2Date,tau);
           
           df_Tn2 = ch.DF(to);
           df_TN2 = ch.DF(tN2);
           y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
           
           c_nN2 = ch.C_nNDate(valueDate,toDate,tN2Date,tau,p_nN2,y_nN2);
           g_nN2 = ch.G_nNDate(valueDate,toDate,tN2Date,tau,p_nN2);
           h_nTerm2 = ch.H_nTermDate(valueDate,toDate,tpDate);
           
           varCovar = ch.LocalVariance(0,to);
           
           driftTerm1=0.0;
           driftTerm2=0.0;
           for i=1:sizeN
               for j=1:sizeN
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
                    driftTerm2 = driftTerm2 + c_nN2(i) * (h_nTerm2(j) + g_nN2(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           for i=1:sizeN
               for j=1:sizeN
                    spreadVariance = spreadVariance + (c_nN1(i)-c_nN2(i)) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
               end
           end
           
           out.mu = (y_nN1 + driftTerm1) - (y_nN2 + driftTerm2);
           out.sigma = sqrt(spreadVariance);
           out.c_nN1 = c_nN1;
           out.c_nN2 = c_nN2;
           out.varCovar = varCovar;
           
        end
        
        function out = CMSSpreadDigital(ch,to,tp,tenor1,tenor2,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsSpreadDigitalArg = ch.CMSSpreadDigitalArg(to,tp,tenor1,tenor2,tau);
           mu = cmsSpreadDigitalArg.mu;
           sigma = cmsSpreadDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSSpreadDigitalDate(ch,valueDate,toDate,tpDate,tenor1,tenor2,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsSpreadDigitalArg = ch.CMSSpreadDigitalArgDate(valueDate,toDate,tpDate,tenor1,tenor2,tau);
           mu = cmsSpreadDigitalArg.mu;
           sigma = cmsSpreadDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSSpreadDigitalRange(ch,to,tp,tenor1,tenor2,tau,lower,upper)
            lower_value = ch.CMSSpreadDigital(to,tp,tenor1,tenor2,tau,lower);
            upper_value = ch.CMSSpreadDigital(to,tp,tenor1,tenor2,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CMSSpreadDigitalRangeDate(ch,valueDate,toDate,tpDate,tenor1,tenor2,tau,lower,upper)
            lower_value = ch.CMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,lower);
            upper_value = ch.CMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,upper);
            out = lower_value - upper_value;
        end
        
        function atmSwaption = ATMSwaption(ch,tx,tnr)
            sizeN = ch.numOfFactor;
            tau = 0.25;
            pv01 = ch.PV01(tx,tx+tnr,tau);
            fsr  = ch.Fsr(tx,tx+tnr,tau);
            c_nN = ch.C_nN(tx,tx+tnr,tau,pv01,fsr);
            variances = ch.LocalVariance(0,tx);
            totalVar = 0.0;
            for i=1:sizeN
                for j=1:sizeN
                    totalVar = totalVar + c_nN(i)*c_nN(j)*variances(i,j);
                end
            end
            if totalVar <=0
                totalVar = 0.0;
            end
            atmSwaption =  pv01 * sqrt(totalVar)/sqrt(2.0*pi);
        end
        
        function atmSwaptionNormalVol = ATMSwaptionNormalVol(ch,tx,tnr)
            sizeN = ch.numOfFactor;
            tau = 0.25;
            pv01 = ch.PV01(tx,tx+tnr,tau);
            fsr  = ch.Fsr(tx,tx+tnr,tau);
            c_nN = ch.C_nN(tx,tx+tnr,tau,pv01,fsr);
            variances = ch.LocalVariance(0,tx);
            totalVar = 0.0;
            for i=1:sizeN
                for j=1:sizeN
                    totalVar = totalVar + c_nN(i)*c_nN(j)*variances(i,j);
                end
            end
            if totalVar <=0
                totalVar = 0.0;
            end
            atmSwaptionNormalVol =  sqrt(totalVar/tx);
        end
        
        function capletOut = Caplet(ch,tx,x,tnr)
        %blackCaplet:a function that computes a blackCaplet price
        %arguments
        %tx =(annualized)time to reset
        %sig= caplet volatility
        %x  = caplet strike
        %tnr=(annualized)time to payment
        %NA =Notional Amount
        
            tau = 0.25;
            p = @ch.DF;
            beta = ch.Beta(tx,tx + tnr);
            sizeN = ch.numOfFactor;
            variances = ch.LocalVariance(0,tx);
            totalVar = 0.0;
            for i=1:sizeN
                for j=1:sizeN
                    totalVar = totalVar + beta(i)*beta(j)*variances(i,j);
                end
            end
            sig = sqrt(totalVar);
            d1= (log(p(tx)/p(tx + tnr)*1.0/(1+tnr*x))+(sig^2/2.0))/(sig);
            d2 = d1 -sig;
            capletOut = p(tx)*H_ncdf(d1) - p(tx+tnr)*(1+tnr*x)*H_ncdf(d2);
        end
        
        function capOut = Cap(ch,tx,x,tnr)
            capOut = sum(arrayfun(@(i) ch.Caplet(tnr*i,x,tnr),1:round(tx/tnr)-1));
        end
        
         function atmSwaptionImpVol = ATMSwaptionImpVol(ch,tx,tnr)
            atmSwaption = ch.ATMSwaption(tx,tnr);
            
            a = 0.001;
            b = 10;
            Tol = 1e-10;
            MaxIter = 1000;
            tau = 0.25;
            fsr  = ch.Fsr(tx,tx+tnr,tau);
            pv01 = ch.PV01(tx,tx+tnr,tau);
            impVol = BisecBSIV('P',fsr,fsr, ...
                                          0.0,tx,a,b,atmSwaption/pv01,Tol,MaxIter);
            atmSwaptionImpVol =  impVol;
        end
        
        function impCorrel = ImpCorrel(ch,tx,tnr1,tnr2)
            tau = 0.25;
            pv01_1 = ch.PV01(tx,tx+tnr1,tau);
            fsr_1  = ch.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = ch.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = ch.PV01(tx,tx+tnr2,tau);
            fsr_2  = ch.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = ch.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            variances = ch.LocalVariance(0,tx);
            var1 = 0;
            var2 = 0;
            coVar = 0;
            sizeN = ch.numOfFactor;
            for j=1:sizeN
                for k=1:sizeN
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                    var2 = var2 + c_nN_2(j)*c_nN_2(k)*variances(j,k);
                    coVar = coVar + c_nN_1(j)*c_nN_2(k)*variances(j,k);
                end
            end
            impCorrel = coVar/sqrt(var1)/sqrt(var2);
        end
        
        function impVariance = ImpVariance(ch,tx,tnr1)
            tau = 0.25;
            pv01_1 = ch.PV01(tx,tx+tnr1,tau);
            fsr_1  = ch.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = ch.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            variances = ch.LocalVariance(0,tx);
            var1 = 0;
            sizeN = ch.numOfFactor;
            for j=1:sizeN
                for k=1:sizeN
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                end
            end
            impVariance = var1;
        end
        
        function out = RebonatoFwdCorr(ch,tx,tnr1,tnr2)
           rho_inf = 0.35;
           rho_decay = 0.15;
           out =  rho_inf + (1-rho_inf)*exp(-rho_decay*abs(tnr1-tnr2));
        end
        
        function out = TargetFunctionATMSwaptionAndCapBootStrapLevMar(ch,tvar,idxNow,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            chSwaptionPrice = zeros(length(tenor),1);
            
            capExpiry = params.capExpiry;
            blackCapPrice = params.blackCapPrice;
            blackCapStrike = params.blackCapStrike;
            
            useCap = false;
            if expiry(idxNow) <= capExpiry(length(capExpiry))
                if capExpiry(idxNow) == expiry(idxNow)
                    useCap = true;
                end
            end
            sizeN = ch.numOfFactor;
            for j=1:sizeN
                ch.lambda{j,1}.quote(idxNow) = tvar(j);
            end
            
            out= zeros(length(tenor)+ 1,1);
            for k=1:length(tenor)
                chSwaptionPrice(k) = ch.ATMSwaption(expiry(idxNow),tenor(k));
                out(k) = chSwaptionPrice(k)/blackSwaptionPrice(idxNow,k)-1.0;
            end
            
            if useCap
                chCapPrice = ch.Cap(capExpiry(idxNow),blackCapStrike(idxNow),0.25);
                out(length(tenor)+1) = chCapPrice/blackCapPrice(idxNow)-1.0;
            end

        end
        
        function out = TargetFunctionATMSwaptionAndCapGlobalLevMar(ch,tvar,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            chSwaptionPrice = zeros(length(expiry),length(tenor));
            
            capExpiry = params.capExpiry;
            blackCapPrice = params.blackCapPrice;
            blackCapStrike = params.blackCapStrike;
            chCapPrice = zeros(length(capExpiry),1);
            
            sizeN = ch.numOfFactor;
            alphaSize = length(ch.lambda{1,1}.tenor);
            for i=1:sizeN
                for j=1:alphaSize
                    ch.lambda{i,1}.quote(j) = tvar((i-1)*alphaSize +j);
                end
            end
            
            sizeOut = length(expiry) * length(tenor);
            sizeOut = sizeOut + length(capExpiry);
            
            out= zeros(sizeOut,1);
            for j=1:length(expiry)
                for k=1:length(tenor)
                    chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                    out((j-1)*length(tenor) + k) = chSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0;
                end
            end
            
            for j=1:length(capExpiry)
                chCapPrice(j) = ch.Cap(capExpiry(j),blackCapStrike(j),0.25);
                out(length(expiry)*length(tenor) + j) = chCapPrice(j)/blackCapPrice(j) - 1.0;
            end
        end 
        
        function out = TargetFunctionATMSwaptionBootStrapLevMar(ch,tvar,idxNow,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            chSwaptionPrice = zeros(length(tenor),1);
            
            sizeN = ch.numOfFactor;
            for j=1:sizeN
                ch.lambda{j,1}.quote(idxNow) = tvar(j);
            end
            
            out= zeros(sizeN,1);
            for k=1:length(tenor)
                chSwaptionPrice(k) = ch.ATMSwaption(expiry(idxNow),tenor(k));
                out(k) = chSwaptionPrice(k)/blackSwaptionPrice(idxNow,k)-1.0;
            end

        end
        
        function out = TargetFunctionATMSwaptionBootStrapLevMarNormalVol(ch,tvar,idxNow,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionNormalVol = params.blackSwaptionNormalVol;
            chATMSwaptionNormalVol = zeros(1,length(tenor));
            
            sizeN = ch.numOfFactor;
            for j=1:sizeN
                ch.lambda{j,1}.quote(idxNow) = tvar(j);
            end
            
            out= zeros(sizeN,1);
            for k=1:length(tenor)
                chATMSwaptionNormalVol(k) = ch.ATMSwaptionNormalVol(expiry(idxNow),tenor(k));
                out(k) = chATMSwaptionNormalVol(k)/blackSwaptionNormalVol(idxNow,k)-1.0;
            end

        end
        
        function out = TargetFunctionATMSwaptionBootStrapLevMarNormalVariance(ch,tvar,idxNow,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionNormalVol = params.blackSwaptionNormalVol;
            chATMSwaptionNormalVol = zeros(1,length(tenor));
            
            sizeN = ch.numOfFactor;
            for j=1:sizeN
                ch.lambda{j,1}.quote(idxNow) = tvar(j);
            end
            
            out= zeros(sizeN,1);
            for k=1:length(tenor)
                chATMSwaptionNormalVol(k) = ch.ATMSwaptionNormalVol(expiry(idxNow),tenor(k));
                out(k) = (chATMSwaptionNormalVol(k)*chATMSwaptionNormalVol(k)*expiry(idxNow))/(blackSwaptionNormalVol(idxNow,k)*blackSwaptionNormalVol(idxNow,k)*expiry(idxNow))-1.0;
            end

        end
        
        function out = TargetFunctionATMSwaptionBootStrapLevMarAbs(ch,tvar,idxNow,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            chSwaptionPrice = zeros(length(tenor),1);
            
            sizeN = ch.numOfFactor;
            for j=1:sizeN
                ch.lambda{j,1}.quote(idxNow) = abs(tvar(j));
            end
            
            out= zeros(sizeN,1);
            for k=1:length(tenor)
                chSwaptionPrice(k) = ch.ATMSwaption(expiry(idxNow),tenor(k));
                out(k) = chSwaptionPrice(k)/blackSwaptionPrice(idxNow,k)-1.0;
            end

        end
        
        function out = TargetFunctionATMSwaptionGlobalLevMar(ch,tvar,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            chSwaptionPrice = zeros(length(expiry),length(tenor));
            
            sizeN = ch.numOfFactor;
            alphaSize = length(ch.lambda{1,1}.tenor);
            for i=1:sizeN
                for j=1:alphaSize
                    ch.lambda{i,1}.quote(j) = tvar((i-1)*alphaSize +j);
                end
            end
            
            sizeOut = length(expiry) * length(tenor);
            out= zeros(sizeOut,1);
            for j=1:length(expiry)
                for k=1:length(tenor)
                    chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                    out((j-1)*length(tenor) + k) = chSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0;
                end
            end
        end 
        
        function out = TargetFunctionATMSwaptionGlobalLevMarKappa(ch,tvar,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            chSwaptionPrice = zeros(length(expiry),length(tenor));
            
            sizeN = ch.numOfFactor;
            alphaSize = length(ch.lambda{1,1}.tenor);
            for i=1:sizeN
                for j=1:alphaSize
                    ch.lambda{i,1}.quote(j) = tvar((i-1)*alphaSize +j);
                end
            end
            
            for i=1:sizeN
                ch.kappa(i) = tvar(sizeN*alphaSize + i);
            end
            
            sizeOut = length(expiry) * length(tenor);
            out= zeros(sizeOut,1);
            for j=1:length(expiry)
                for k=1:length(tenor)
                    chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                    out((j-1)*length(tenor) + k) = chSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0;
                end
            end
        end 

        function out = TargetFunctionATMSwaptionGlobal(ch,tvar,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            chSwaptionPrice = zeros(1,size(tenor,2));
            
            sizeN = ch.numOfFactor;
            alphaSize = length(ch.lambda{1,1}.tenor);
            for i=1:sizeN
                for j=1:alphaSize
                    ch.lambda{i,1}.quote(j) = tvar((i-1)*alphaSize +j);
                end
            end
            
            out= 0;
            for j=1:length(expiry)
                for k=1:length(tenor)
                    chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                    out = out + (chSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0)*(chSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0)*10000;
                end
            end
        end   
        
        function out = TargetFunctionATMSwaptionGlobalImpVol(ch,tvar,params)
            a = 0.001;
            b = 10;
            Tol = 1e-10;
            MaxIter = 1000;

            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionImpVol = params.blackSwaptionImpVol;
            chSwaptionPrice = zeros(1,size(tenor,2));
            
            sizeN = ch.numOfFactor;
            alphaSize = length(ch.lambda{1,1}.tenor);
            for i=1:sizeN
                for j=1:alphaSize
                    ch.lambda{i,1}.quote(j) = tvar((i-1)*alphaSize +j);
                end
            end
            
            out= 0;
            for j=1:length(expiry)
                for k=1:length(tenor)
                    chATMSwaptionImpVol(j,k) = ch.ATMSwaptionImpVol(expiry(j),tenor(k));
                    out = out + (chATMSwaptionImpVol(j,k)- blackSwaptionImpVol(j,k))*(chATMSwaptionImpVol(j,k)- blackSwaptionImpVol(j,k))*10000;
                end
            end
        end   
        
        function out = TargetFunctionATMSwaptionGlobalNormalVol(ch,tvar,params)
            
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionNormalVol = params.blackSwaptionNormalVol;
            chATMSwaptionNormalVol = zeros(length(expiry),length(tenor));
            
            sizeN = ch.numOfFactor;
            alphaSize = length(ch.lambda{1,1}.tenor);
            for i=1:sizeN
                for j=1:alphaSize
                    ch.lambda{i,1}.quote(j) = tvar((i-1)*alphaSize +j);
                end
            end
            
            out= 0;
            for j=1:length(expiry)
                for k=1:length(tenor)
                    chATMSwaptionNormalVol(j,k) = ch.ATMSwaptionNormalVol(expiry(j),tenor(k));
                    out = out + (chATMSwaptionNormalVol(j,k)/blackSwaptionNormalVol(j,k)-1.0)*(chATMSwaptionNormalVol(j,k)/blackSwaptionNormalVol(j,k)-1.0)*10000;
                end
            end
        end   
        
        function out = CalibrateToATMSwaption(ch,black,calibrationFlag)
            tic;
            
            sizeN = ch.numOfFactor;
            alphaSize = length(ch.lambda{1,1}.tenor);
            for i=1:sizeN
               for j=1:alphaSize
                   out.lambda{i,1} = ch.lambda{i,1};
               end
            end
            
          % Calculate Black ATM Swaption Prices
          
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
            
            expirySize = size(expiry,2);
            tenorSize = size(tenor,2);
            
            blackSwaptionPrice = zeros(expirySize,tenorSize);
            blackSwaptionImpVol = zeros(expirySize,tenorSize);
%             blackSwaptionNormalVol = zeros(expirySize,tenorSize);
            
            for k=1:expirySize
                for j=1:tenorSize
                    blackSwaptionPrice(k,j)  = black.BlackATMSwaption(expiry(k),tenor(j));
                    blackSwaptionImpVol(k,j) = black.swaptionVol(expiry(k),tenor(j));
%                     blackSwaptionNormalVol(k,j) = black.ATMSwaptionNormalVol(expiry(k),tenor(j));
                end
            end
            
            params.expiry = expiry;
            params.tenor = tenor;
            params.blackSwaptionPrice = blackSwaptionPrice;
            params.blackSwaptionImpVol = blackSwaptionImpVol;
%             params.blackSwaptionNormalVol = blackSwaptionNormalVol;
            params.expirySize = expirySize;
            params.tenorSize =  tenorSize;
            
            params.swaption_calibration_yn = ones(expirySize,tenorSize);
            
            out.blackSwaptionPrice = blackSwaptionPrice;
            out.chSwaptionPrice = zeros(expirySize,tenorSize);
            out.relDiff = zeros(expirySize,tenorSize);
            out.rmseTotal=0.0;
            out.impVol2Total = 0.0;
            out.normalVol2Total = 0.0;
            out.normalVarianceTotal = 0.0;
            
            calibrationType = 'bootstrap';
            if nargin < 3
                calibrationType = 'bootstrap'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;
            tvar = [];
            
            switch calibrationType
                case 'bootstrapLevMar'
                    tvar = zeros(sizeN,1);
                    for i=1:expirySize
                        idxNow = i;
                        for j=1:sizeN
                             tvar(j) = ch.lambda{j,1}.quote(i);
                             lb(j) = 0.0001;
                             ub(j) = 10.0;
                        end
                        
                        targetfun = @(tvar) TargetFunctionATMSwaptionBootStrapLevMar(ch,tvar,idxNow,params);
                        options=optimset('Algorithm','levenberg-marquardt');
%                         options=optimset('Algorithm','trust-region-reflective');
                        [x, fval] = lsqnonlin(targetfun,tvar,lb,ub,options);
                        
                        for j=1:sizeN
                             out.lambda{j,1}.quote(i) = x(j); 
                             ch.lambda{j,1}.quote(i) = x(j); 
                        end
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.calibrationTime = toc;
                case 'bootstrapLevMarNormalVol'
                    tvar = zeros(sizeN,1);
                    for i=1:expirySize
                        idxNow = i;
                        for j=1:sizeN
                             tvar(j) = ch.lambda{j,1}.quote(i);
                             lb(j) = 0.0001;
                             ub(j) = 10.0;
                        end
                        
                        targetfun = @(tvar) TargetFunctionATMSwaptionBootStrapLevMarNormalVol(ch,tvar,idxNow,params);
                        options=optimset('Algorithm','levenberg-marquardt');
%                         options=optimset('Algorithm','trust-region-reflective');
                        [x, fval] = lsqnonlin(targetfun,tvar,lb,ub,options);
                        
                        for j=1:sizeN
                             out.lambda{j,1}.quote(i) = x(j); 
                             ch.lambda{j,1}.quote(i) = x(j); 
                        end
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.blackSwaptionImpVol(j,k) = blackSwaptionImpVol(j,k);
                            out.blackSwaptionNormalVol(j,k) = blackSwaptionNormalVol(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.chSwaptionImpVol(j,k) = ch.ATMSwaptionImpVol(expiry(j),tenor(k));
                            out.chSwaptionNormalVol(j,k) = ch.ATMSwaptionNormalVol(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.ImpVolDiff(j,k) = out.chSwaptionImpVol(j,k) - out.blackSwaptionImpVol(j,k);
                            out.NormalVolDiff(j,k) = out.chSwaptionNormalVol(j,k)/out.blackSwaptionNormalVol(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            out.impVol2Total = out.impVol2Total + out.ImpVolDiff(j,k)*out.ImpVolDiff(j,k);
                            out.normalVol2Total = out.normalVol2Total + out.NormalVolDiff(j,k)*out.NormalVolDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.impVol2Total = sqrt(out.impVol2Total/N);
                    out.normalVol2Total = sqrt(out.normalVol2Total/N);
                    out.calibrationTime = toc;
                    
                case 'bootstrapLevMarNormalVariance'
                    tvar = zeros(sizeN,1);
                    for i=1:expirySize
                        idxNow = i;
                        for j=1:sizeN
                             tvar(j) = ch.lambda{j,1}.quote(i);
                             lb(j) = 0.0001;
                             ub(j) = 10.0;
                        end
                        
                        targetfun = @(tvar) TargetFunctionATMSwaptionBootStrapLevMarNormalVariance(ch,tvar,idxNow,params);
                        options=optimset('Algorithm','levenberg-marquardt');
%                         options=optimset('Algorithm','trust-region-reflective');
                        [x, fval] = lsqnonlin(targetfun,tvar,lb,ub,options);
                        
                        for j=1:sizeN
                             out.lambda{j,1}.quote(i) = x(j); 
                             ch.lambda{j,1}.quote(i) = x(j); 
                        end
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.blackSwaptionImpVol(j,k) = blackSwaptionImpVol(j,k);
                            out.blackSwaptionNormalVol(j,k) = blackSwaptionNormalVol(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.chSwaptionImpVol(j,k) = ch.ATMSwaptionImpVol(expiry(j),tenor(k));
                            out.chSwaptionNormalVol(j,k) = ch.ATMSwaptionNormalVol(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.ImpVolDiff(j,k) = out.chSwaptionImpVol(j,k) - out.blackSwaptionImpVol(j,k);
                            out.NormalVolDiff(j,k) = out.chSwaptionNormalVol(j,k)/out.blackSwaptionNormalVol(j,k)-1;
                            out.NormalVarianceDiff(j,k) = (out.chSwaptionNormalVol(j,k)*out.chSwaptionNormalVol(j,k)*expiry(j))/(out.blackSwaptionNormalVol(j,k)*out.blackSwaptionNormalVol(j,k)*expiry(j))-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            out.impVol2Total = out.impVol2Total + out.ImpVolDiff(j,k)*out.ImpVolDiff(j,k);
                            out.normalVol2Total = out.normalVol2Total + out.NormalVolDiff(j,k)*out.NormalVolDiff(j,k);
                            out.normalVarianceTotal = out.normalVarianceTotal + out.NormalVarianceDiff(j,k)*out.NormalVarianceDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.impVol2Total = sqrt(out.impVol2Total/N);
                    out.normalVol2Total = sqrt(out.normalVol2Total/N);
                    out.normalVarianceTotal = sqrt(out.normalVarianceTotal/N);
                    out.calibrationTime = toc;
                    
                case 'globalLevMar'
                    for i=1:sizeN
                        for j=1:alphaSize
                             tvar((i-1)*alphaSize +j) = ch.lambda{i,1}.quote(j);
                             lb((i-1)*alphaSize +j) = 0.0001;
                             ub((i-1)*alphaSize +j) = 10.0;
                        end
                    end
                    
                       
                    targetfun = @(tvar) TargetFunctionATMSwaptionGlobalLevMar(ch,tvar,params);
%                     options=optimset('Algorithm','levenberg-marquardt');
                    options=optimset('Algorithm','levenberg-marquardt','MaxFunEvals',700);
                    [x, fval] = lsqnonlin(targetfun,tvar,lb,ub,options);
                    
                    for i=1:sizeN
                        for j=1:alphaSize
                             out.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                             ch.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                        end
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.calibrationTime = toc;
                
                 case 'dummy'
                    
                    for i=1:sizeN
                        for j=1:alphaSize
                             out.lambda{i,1}.quote(j) = ch.lambda{i,1}.quote(j); 
                        end
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.calibrationTime = toc;
                    
                    
                case 'globalLevMarKappa'
                    for i=1:sizeN
                        for j=1:alphaSize
                             tvar((i-1)*alphaSize +j) = ch.lambda{i,1}.quote(j);
                             lb((i-1)*alphaSize +j) = 0.0001;
                             ub((i-1)*alphaSize +j) = 10.0;
                        end
                    end
                    
                    for i=1:sizeN
                        tvar(sizeN*alphaSize + i) = ch.kappa(i);
                        lb(sizeN*alphaSize + i) = 0.0001;
                        ub(sizeN*alphaSize + i) = 10.0;
                    end
                       
                    targetfun = @(tvar) TargetFunctionATMSwaptionGlobalLevMarKappa(ch,tvar,params);
                    options=optimset('Algorithm','levenberg-marquardt');
                    [x, fval] = lsqnonlin(targetfun,tvar,lb,ub,options);
                    
                    for i=1:sizeN
                        for j=1:alphaSize
                             out.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                             ch.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                        end
                    end
                    
                    for i=1:sizeN
                        out.kappa(i) = x(sizeN*alphaSize+i);
                        ch.kappa(i)  = x(sizeN*alphaSize+i);
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);    
                    
                case 'global'
                    optParams.params = params;
                    optParams.modelParamsSize = sizeN * alphaSize;
                    optParams.numOfEquation = params.expirySize*params.tenorSize + ...  % target cap & swaption
                                              sizeN*(alphaSize -2);   % tikhonov regularization gamma constraint
                    
                    tvar = zeros(sizeN*alphaSize,1);
                    for i=1:sizeN
                        for j=1:alphaSize
                             tvar((i-1)*alphaSize +j) = ch.lambda{i,1}.quote(j);
                        end
                    end
                    
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
                    
                    x=  zeros(optParams.numOfEquation,1);
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionCheyetteATMSwaptionSurfaceGlobal',tvar,x, 500, options,'unc',ch,optParams);
                    toc

%                     targetfun = @(tvar) TargetFunctionATMSwaptionGlobal(ch,tvar,params);
% %                     options=optimset('Algorithm','interior-point');
%                      options=optimset('Algorithm','trust-region-reflective');
%                     [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);
                    
                    for i=1:sizeN
                        for j=1:alphaSize
                             out.lambda{i,1}.quote(j) = max(abs(popt((i-1)*alphaSize +j)),0.0001); 
                             ch.lambda{i,1}.quote(j) = max(abs(popt((i-1)*alphaSize +j)),0.0001); 
                        end
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                 case 'globalFmin'
                    tvar = zeros(sizeN*alphaSize,1);
                    for i=1:sizeN
                        for j=1:alphaSize
                             tvar((i-1)*alphaSize +j) = ch.lambda{i,1}.quote(j);
                             lb((i-1)*alphaSize +j) = 0.01;
                             ub((i-1)*alphaSize +j) = 10.0;
                        end
                    end
                    
                       
                    targetfun = @(tvar) TargetFunctionATMSwaptionGlobal(ch,tvar,params);
%                     options=optimset('Algorithm','interior-point');
                     options=optimset('Algorithm','trust-region-reflective');
                    [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);
                    
                    for i=1:sizeN
                        for j=1:alphaSize
                             out.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                             ch.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                        end
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                 case 'globalImpVol'
                    for i=1:sizeN
                        for j=1:alphaSize
                             tvar((i-1)*alphaSize +j) = ch.lambda{i,1}.quote(j);
                             lb((i-1)*alphaSize +j) = 0.0001;
                             ub((i-1)*alphaSize +j) = 10.0;
                        end
                    end
                    
                       
                    targetfun = @(tvar) TargetFunctionATMSwaptionGlobalImpVol(ch,tvar,params);
%                     options=optimset('Algorithm','interior-point');
                     options=optimset('Algorithm','trust-region-reflective');
                    [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);
                    
                    for i=1:sizeN
                        for j=1:alphaSize
                             out.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                             ch.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                        end
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.blaskSwaptionImpVol(j,k) = blackSwaptionImpVol(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.chSwaptionImpVol(j,k) = ch.ATMSwaptionImpVol(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            our.volDiff(j,k) = out.chSwaptionImpVol(j,k) - out.blaskSwaptionImpVol(j,k);
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            out.vol2Total = out.vol2Total + our.volDiff(j,k)*our.volDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.vol2Total = sqrt(out.vol2Total/N);
                    
                case 'globalNormalVol'
                    for i=1:sizeN
                        for j=1:alphaSize
                             tvar((i-1)*alphaSize +j) = ch.lambda{i,1}.quote(j);
                             lb((i-1)*alphaSize +j) = 0.0001;
                             ub((i-1)*alphaSize +j) = 10.0;
                        end
                    end
                    
                       
                    targetfun = @(tvar) TargetFunctionATMSwaptionGlobalNormalVol(ch,tvar,params);
%                     options=optimset('Algorithm','interior-point');
                     options=optimset('Algorithm','trust-region-reflective');
                    [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);
                    
                    for i=1:sizeN
                        for j=1:alphaSize
                             out.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                             ch.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                        end
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.blackSwaptionImpVol(j,k) = blaskSwaptionImpVol(j,k);
                            out.blackSwaptionNormalVol(j,k) = blackSwaptionNormalVol(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.chSwaptionImpVol(j,k) = ch.ATMSwaptionImpVol(expiry(j),tenor(k));
                            out.chSwaptionNormalVol(j,k) = ch.ATMSwaptionNormalVol(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.ImpVolDiff(j,k) = out.chSwaptionImpVol(j,k) - out.blackSwaptionImpVol(j,k);
                            out.NormalVolDiff(j,k) = out.chSwaptionNormalVol(j,k)/out.blackSwaptionNormalVol(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            out.impVol2Total = out.impVol2Total + out.ImpVolDiff(j,k)*out.ImpVolDiff(j,k);
                            out.normalVol2Total = out.normalVol2Total + out.NormalVolDiff(j,k)*out.NormalVolDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.impVol2Total = sqrt(out.impVol2Total/N);
                    out.normalVol2Total = sqrt(out.normalVol2Total/N);
                
                otherwise
                    disp('unImplemented')
            end
        end
        
        function out = CalibrateToATMSwaptionAndCap(ch,black,calibrationFlag)
            tic;
            
            sizeN = ch.numOfFactor;
            alphaSize = length(ch.lambda{1,1}.tenor);
            for i=1:sizeN
               for j=1:alphaSize
                   out.lambda{i,1} = ch.lambda{i,1};
               end
            end
            
          %% Calculate Black ATM Swaption Prices
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
            
            expirySize = size(expiry,2);
            tenorSize = size(tenor,2);
            
            blackSwaptionPrice = zeros(expirySize,tenorSize);
            blackSwaptionImpVol = zeros(expirySize,tenorSize);
            for k=1:expirySize
                for j=1:tenorSize
                    blackSwaptionPrice(k,j)  = black.BlackATMSwaption(expiry(k),tenor(j));
                    blackSwaptionImpVol(k,j) = black.swaptionVol(expiry(k),tenor(j));
                end
            end
            
          %% Calculate Black ATM Cap Prices
           capVol = black.capVol;
           capVolData = black.mktData('capVol');
           capVolrawData =  capVolData.params('rawData');
            
           capExpiry = capVolrawData(:,1);
           capExpirySize = length(capExpiry);
            
           blackCapPrice = zeros(capExpirySize,1);
           blackCapImpVol = zeros(capExpirySize,1);
           blackCapStrike = zeros(capExpirySize,1);
           for k=1:capExpirySize
               blackCapStrike(k) = black.Fsr(0.25,capExpiry(k) -0.25,0.25);
               blackCapPrice(k) = black.BlackCap(capExpiry(k),blackCapStrike(k),0.25);
               blackCapImpVol(k) = black.capVol(capExpiry(k));
           end
          
          
            %%
            params.expiry = expiry;
            params.tenor = tenor;
            params.blackSwaptionPrice = blackSwaptionPrice;
            params.blackSwaptionImpVol = blackSwaptionImpVol;
            params.expirySize = expirySize;
            params.tenorSize =  tenorSize;
            
            params.capExpiry = capExpiry;
            params.blackCapPrice = blackCapPrice;
            params.blackCapStrike = blackCapStrike;
            params.blackCapImpVol = blackCapImpVol;
            params.capExpirySize = capExpirySize;
            
            out.blackSwaptionPrice = blackSwaptionPrice;
            out.chSwaptionPrice = zeros(expirySize,tenorSize);
            out.relDiff = zeros(expirySize,tenorSize);
            out.rmseTotal=0.0;
            
            out.blackCapPrice = blackCapPrice;
            out.chCapPrice = zeros(capExpirySize,1);
            out.capRelDiff = zeros(capExpirySize,1);
            
            calibrationType = 'bootstrap';
            if nargin < 3
                calibrationType = 'bootstrap'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;
            tvar = [];
            
            switch calibrationType
                case 'bootstrapLevMar'
                    tvar = zeros(sizeN,1);
                    for i=1:expirySize
                        idxNow = i;
                        for j=1:sizeN
                             tvar(j) = ch.lambda{j,1}.quote(i);
                             lb(j) = 0.0;
                             ub(j) = 10.0;
                        end
                        
                        targetfun = @(tvar) TargetFunctionATMSwaptionAndCapBootStrapLevMar(ch,tvar,idxNow,params);
                        options=optimset('Algorithm','levenberg-marquardt');
%                         options=optimset('Algorithm','trust-region-reflective');
                        [x, fval] = lsqnonlin(targetfun,tvar,lb,ub,options);
                        
                        for j=1:sizeN
                             out.lambda{j,1}.quote(i) = x(j); 
                             ch.lambda{j,1}.quote(i) = x(j); 
                        end
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                    
                    for j=1:capExpirySize
                        out.chCapPrice(j) = ch.Cap(capExpiry(j),blackCapStrike(j),0.25);
                        out.capRelDiff(j) = out.chCapPrice(j)/out.blackCapPrice(j) -1.0;
                        out.rmseTotal = out.rmseTotal +  out.capRelDiff(j)* out.capRelDiff(j);
                        N=N+1;
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                out.calibrationTime = toc;
                
                case 'globalLevMar'
                    for i=1:sizeN
                        for j=1:alphaSize
                             tvar((i-1)*alphaSize +j) = ch.lambda{i,1}.quote(j);
                             lb((i-1)*alphaSize +j) = 0.0;
                             ub((i-1)*alphaSize +j) = 10.0;
                        end
                    end
                    
                       
                    targetfun = @(tvar) TargetFunctionATMSwaptionAndCapGlobalLevMar(ch,tvar,params);
                    options=optimset('Algorithm','levenberg-marquardt');
                    [x, fval] = lsqnonlin(targetfun,tvar,lb,ub,options);
                    
                    for i=1:sizeN
                        for j=1:alphaSize
                             out.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                             ch.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                        end
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                    
                    for j=1:capExpirySize
                        out.chCapPrice(j) = ch.Cap(capExpiry(j),blackCapStrike(j),0.25);
                        out.capRelDiff(j) = out.chCapPrice(j)/out.blackCapPrice(j) -1.0;
                        out.rmseTotal = out.rmseTotal +  out.capRelDiff(j)* out.capRelDiff(j);
                        N=N+1;
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.calibrationTime = toc;
                   
                otherwise
                    disp('unImplemented')
            end
        end    
    end
    
end



In [ ]:
classdef CheyetteTerm < IRModel
    %UNTITLED Summary of this class goes here
    %   Detailed explanation goes here
    
    properties
        % vol functor
        smoothAlpha;
        numOfFactor;
        lambda = {};
        kappa = [];
        tau = [];
        corr_fwdTermSize = 0;
        corr_fwdTerm = {};
        rhoTerm = {};
        inv_H_tau = [];
        swaption_calibration_yn=[];
    end
    
    methods
        function ch = CheyetteTerm(params)
            if nargin > 0
                ch.mktData =  params.mktData;
                ch.modelParams = params.modelParams;
                
                ch.zeroCurve = params.zeroCurve;
                
                ch.smoothAlpha =  params.modelParams('smoothAlpha');
                
                ch.numOfFactor = params.modelParams('numOfFactor');
                sizeN = ch.numOfFactor;
                ch.lambda = cell(sizeN,1);
                ch.lambda = params.modelParams('lambda');
                
                ch.kappa = ones(sizeN,1);
                ch.kappa =  params.modelParams('kappa');
               
                ch.tau = ones(sizeN,1);
                ch.tau =  params.modelParams('tau');
                ch.inv_H_tau = ones(sizeN,sizeN);
                H_tau = ones(sizeN,sizeN);
                for i=1:sizeN
                    for j=1:sizeN
                        H_tau(i,j) = exp(-ch.kappa(j)* ch.tau(i));
                    end
                end
                ch.inv_H_tau = inv(H_tau);
                
%                 ch.corr_fwdTerm = zeros(sizeN,sizeN);
                ch.corr_fwdTermSize = params.modelParams('corr_fwdTermSize');
                ch.corr_fwdTerm = cell(ch.corr_fwdTermSize,1);
                
                ch.corr_fwdTerm =  params.modelParams('corr_fwdTerm');
                
                ch.rhoTerm = cell(ch.corr_fwdTermSize,1);
                
                for idx =1:ch.corr_fwdTermSize
                    [PC,V]=eig(ch.corr_fwdTerm{idx,1}.quote);
                    PCW= PC*sqrt(V);
                    for i=1:sizeN
                        for j=1:sizeN
                            ch.rhoTerm{idx,1}.quote(i,j)=PCW(i,j);
                        end
                    end
                end
                ch.swaption_calibration_yn = params.modelParams('swaptionCalibrationYN');
           
                
            end
        end
        
        function out = SimpleSRPrOverHedge(ch,spreadC,spreadN,ralower,raupper)
            buffer = 0.001;
            outVec = 0.0;
            if spreadC < ralower - buffer
                outVec = 0.0;
            elseif spreadC >= ralower - buffer && spreadC < ralower
                outVec = 1.0/buffer * (spreadC - (ralower - buffer))
            elseif spreadC >= ralower && spreadC < raupper
                outVec = 1.0;
            elseif spreadC >= raupper && spreadC < raupper + buffer
                outVec = -1.0/buffer *(spreadC - (raupper + buffer))
            else
                outVec = 0.0;
            end
            
            out = outVec;
        end
        
        function out = SimpleSRPr(ch,spreadC,spreadN,ralower,raupper)
            outVec = 0.0;
            if spreadC >= ralower && spreadC <= raupper
                outVec = 1.0;
            end
            out = outVec;
        end
        
        
        function out = TRZ_SRPr(ch,curr,next,L,U)
            length = 1.0;
            remain = 1.0;
            if (curr >= L && curr <= U && next >= L && next <= U)
                length = remain;
            elseif ( curr >= L && curr <= U && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr >= L && curr <= U && next < L ) 
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr > U && next >= L && next <= U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next < L )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next >= L && next <= U )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next > U )
                length = 0.0;
            elseif ( curr < L && next < L)
                length = 0.0;
            else
                disp('error')
            end
            
            out = length;
        end
        
        function out = SRPr(ch,spreadC,spreadN,ralower,raupper,simpleYN,pathSize)
            
            outVec = zeros(1,pathSize);
            
            if simpleYN == 1
                for i =1:pathSize
                    outVec(i) = ch.SimpleSRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            elseif simpleYN == 2
                for i =1:pathSize
                    outVec(i) = ch.SimpleSRPrOverHedge(spreadC(i),spreadN(i),ralower,raupper);
                end
            else
                for i =1:pathSize
                    outVec(i) = ch.TRZ_SRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            end
            
            out = outVec;
        end
        
        function out =  CMS_PSA_SimpleDate(ch,valueDate,observeTime,numMatTime, tenor,modelStatesT)
           %CMS
           tau = 0.25;
           to      = observeTime/365.0;
           toDate = valueDate.AddDate(observeTime,'day');
           tNDate = toDate.AddDate(tenor,'year');
           tN   =  tNDate.DateDiff(valueDate)/365.0;
           
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           p_nN = ch.PV01Date(valueDate,toDate,tNDate,tau);
           
           df_Tn = ch.DF(to);
           df_TN = ch.DF(tN);
           y_nN = (df_Tn - df_TN)/p_nN;
           
           c_nN = ch.C_nNDate(valueDate,toDate,tNDate,tau,p_nN,y_nN);
           g_nN = ch.G_nNDate(valueDate,toDate,tNDate,tau,p_nN);
           h_nTerm = ch.H_nTermDate(valueDate,toDate,numMatDate);
           varCovar = ch.LocalVariance(0,to);
           
           sizeN = ch.numOfFactor;
            
           stateSize = size(modelStatesT,2);
           cms_PCA_SimpleOut = zeros(1,stateSize);
           
           driftTerm = 0;
           for i=1:sizeN
               for j=1:sizeN
                   driftTerm = driftTerm + (c_nN(i) * h_nTerm(j) + c_nN(i) * g_nN(j))*varCovar(i,j);
               end
           end
           
           for i=1:stateSize
               cms_PCA_SimpleOut(i) = y_nN + driftTerm;
               for j=1:sizeN
                cms_PCA_SimpleOut(i) = cms_PCA_SimpleOut(i) + c_nN(j)*modelStatesT(j,i);
               end
           end
           
           out = cms_PCA_SimpleOut;
           
            
        end
        
        function out = CheyetteGeneralVariance(ch,from,to,T1,T2)
            
            sizeN = ch.numOfFactor;
            plambda =cell(sizeN,1);
            for i=1:sizeN
                plambda{i,1}= ch.lambda{i,1};
            end
            
            sigma_f = zeros(sizeN,sizeN);
            
            % eta is short rate volatlity
            % eta2 is short rate covariance matrix

            for i=1:sizeN
                for j=1:sizeN
                    sigma_f(i,j)=sqrt(plambda{i,1}.quote(1)*plambda{i,1}.quote(1))*ch.rhoTerm{1,1}.quote(i,j);
                end
            end
            
           eta = zeros(sizeN,sizeN);
           eta = ch.inv_H_tau*sigma_f;
           eta2 = eta*eta';
 
           sigmaTimes = plambda{1,1}.tenor;
           sigmaSize = length(plambda{1,1}.tenor);
            
            for i=1:sigmaSize
                if from<= sigmaTimes(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:sigmaSize
                if to<= sigmaTimes(i)
                    break;
                end
            end
            endIdx = i;
            
            lastExpScale = zeros(4,sizeN,sizeN);
            newExpScale  = zeros(4,sizeN,sizeN);
            variances = zeros(4,sizeN,sizeN);
            meanR = zeros(sizeN,1);
            
            for i=1:sizeN
                meanR(i) = ch.kappa(i);
            end
%             newExpScale = zeros(4,1);
%             lastExpScale = zeros(4,1);
%             variances = zeros(4,1);
            
            lastU  = from;
            
            for j=1:sizeN
                for k=1:sizeN
                    lastExpScale(2,j,k) = exp(meanR(j)*lastU);
                    lastExpScale(3,j,k) = exp(meanR(k)*lastU);
                    lastExpScale(4,j,k) = exp((meanR(j)+meanR(k))*lastU);
                end
            end
            
            for i=startIdx:endIdx
                if (i< endIdx) U = sigmaTimes(i);else U = to;end;
                if (i< sigmaSize) tindex = i;else tindex = sigmaSize;end;
            % eta is short rate volatlity
                for j=1:sizeN
                    for k=1:sizeN
                        sigma_f(j,k)=sqrt(plambda{j,1}.quote(tindex)*plambda{j,1}.quote(tindex))*ch.rhoTerm{tindex,1}.quote(j,k);
                    end
                end
                
                eta = ch.inv_H_tau*sigma_f;
                eta2 = eta*eta';
                
                for j=1:sizeN
                    for k=1:sizeN
                        newExpScale(2,j,k) = exp(meanR(j)*U);
                        newExpScale(3,j,k) = exp(meanR(k)*U);
                        newExpScale(4,j,k) = exp((meanR(j)+meanR(k))*U);
                    
                        variances(1,j,k) = variances(1,j,k) + eta2(j,k) * (U - lastU);
                        variances(2,j,k) = variances(2,j,k) + eta2(j,k) * (1.0/ meanR(j))*(newExpScale(2,j,k) - lastExpScale(2,j,k));
                        variances(3,j,k) = variances(3,j,k) + eta2(j,k) * (1.0/ meanR(k))*(newExpScale(3,j,k) - lastExpScale(3,j,k));
                        variances(4,j,k) = variances(4,j,k) + eta2(j,k) * (1.0/ (meanR(j)+meanR(k)))*(newExpScale(4,j,k) - lastExpScale(4,j,k));
                    end
                end
                
                lastU = U;
                for j=1:sizeN
                    for k=1:sizeN
                        lastExpScale(2,j,k) = newExpScale(2,j,k);
                        lastExpScale(3,j,k) = newExpScale(3,j,k);
                        lastExpScale(4,j,k) = newExpScale(4,j,k);
                    end
                end
                
            end
            
            value = 0;
            
            for j=1:sizeN
                for k=1:sizeN
                    value = value + (1.0/ meanR(j))*(1.0/meanR(k))*variances(1,j,k);
                end
            end
            
            for j=1:sizeN
                for k=1:sizeN
                    value = value + -1.0*exp(-1.0*meanR(j)*T1)*(1.0/ meanR(j))*(1.0/meanR(k))*variances(2,j,k);
                end
            end
            
            for j=1:sizeN
                for k=1:sizeN
                    value = value + -1.0*exp(-1.0*meanR(k)*T2)*(1.0/ meanR(j))*(1.0/meanR(k))*variances(3,j,k);
                end
            end
            
            for j=1:sizeN
                for k=1:sizeN
                    value = value +  1.0*exp(-1.0*meanR(j)*T1)*exp(-1.0*meanR(k)*T2)*(1.0/ meanR(j))*(1.0/meanR(k))*variances(4,j,k);
                end
            end
            
            out  = value;
        end
        
        function out = fwdZcVariance(ch, from, to, T1, T2)
            
            localVariance = zeros(2,1);
            
            localVariance(1) = ch.CheyetteGeneralVariance(from,to,T1,T1);
            localVariance(2) = ch.CheyetteGeneralVariance(from,to,to,to);
            
            
            value = localVariance(1) - localVariance(2);
            
            measureChange = zeros(2,1);
            
            measureChange(1) = ch.CheyetteGeneralVariance(from,to,T1,T2);
            measureChange(2) = ch.CheyetteGeneralVariance(from,to,to,T2);
            
            measureChangeTerm = measureChange(1) - measureChange(2);
            
            value = value + -2.0*measureChangeTerm;
            
            out = value;
            
        end
        
        function out = discountFactor(ch, startTime, endTime,numMatTime,modelStatesT)
           zcT = ch.DF(endTime/365.0);
           zct = ch.DF(startTime/365.0);
           beta = ch.Beta(startTime/365.0,endTime/365.0);
           zcDriftTerm = ch.fwdZcVariance(0,startTime/365.0,endTime/365.0,numMatTime/365.0);
           %numOfPath
           stateSize = size(modelStatesT,2);
           discountFactorOut = zeros(1,stateSize);
           
           sizeN = ch.numOfFactor;
           
           for i=1:stateSize
               discountFactorOut(i) = zcT/zct * exp(-0.5*zcDriftTerm);
               for j=1:sizeN
                    discountFactorOut(i) = discountFactorOut(i) * exp(-beta(j)*modelStatesT(j,i));
               end
           end
           
           out = discountFactorOut;
        end
        
        function out = discountPayoff(ch,eventTime,numMatTime ...
                ,modelStatesT,cashflow)
            
            dfNumMat = ch.DF(numMatTime/365.0);
            % for discountPayoff T=numMatTime
            % P(0,TnumMat)/P(Te,Tnummat)*Payoff
            %
            discountT = ch.discountFactor(eventTime, numMatTime,numMatTime,modelStatesT);
            stateSize = size(modelStatesT,2);
            payoff = zeros(1,stateSize);
            for i=1:stateSize
                payoff(i)= dfNumMat/discountT(i)*cashflow(i);
            end    
            out = payoff;
        end
        
        function out = stateLocalDrift(ch,fromTime, toTime)
            sizeN = ch.numOfFactor;
            drift = zeros(sizeN,sizeN);
            for i=1:sizeN
                drift(i,i) =  exp(-1.0*ch.kappa(i)*(toTime-fromTime)/365.0);
            end
            out = drift;
        end
        
        function out = stateLocalVariance(ch,fromTime, toTime)
            out = ch.LocalVariance(fromTime/365.0,toTime/365.0);
        end
        
        function variances = LocalVariance(ch,from,to)
            sizeN = ch.numOfFactor;
            plambda =cell(sizeN,1);
            for i=1:sizeN
                plambda{i,1}= ch.lambda{i,1};
            end
            sigma_f =[];
            sigma_f = zeros(sizeN,sizeN);
%% converting forward rate volatlity into short rate volatlitity
%% eta is short rate volatlity
            for i=1:sizeN
                for j=1:sizeN
%                     sigma_f(i,j)=plambda{i,1}.quote(1)*ch.Fwd(0,ch.tau(i))*ch.rho(i,j);
%                     sigma_f(i,j)=plambda{i,1}.quote(1)*ch.rho(i,j);
                    sigma_f(i,j)=sqrt(plambda{i,1}.quote(1)*plambda{i,1}.quote(1))*ch.rhoTerm{1,1}.quote(i,j);
                end
            end
            
           eta =[];
           eta = zeros(sizeN,sizeN);
           eta = ch.inv_H_tau*sigma_f;
           eta2 = eta*eta';
           %%
           alphaSize = length(plambda{1,1}.tenor);
            
           for i=1:alphaSize
               if from<= plambda{1,1}.tenor(i);
                   break;
               end
           end
            
           startIdx = i;
            
%             endIdx = size(sig(1).tenor,1); 
           for i=1:alphaSize
               if to<= plambda{1,1}.tenor(i);
                   break;
               end
           end
           endIdx = i;
            
           lastU  = from;
           
           tiny = zeros(sizeN,sizeN);
           lastExpScale = zeros(sizeN,sizeN);
           newExpScale  = zeros(sizeN,sizeN);
           scale = 0.0;
           meanR = zeros(sizeN,1);
           for i=1:sizeN
                meanR(i) = ch.kappa(i);
           end
           for i=1:sizeN
                for j=1:sizeN
                    if(abs(meanR(i)+meanR(j)) < 1.0E-12) tiny(i,j) = 1; else tiny(i,j) = 0; end;
                    lastExpScale(i,j) =  exp((meanR(i)+meanR(j))*from);
                end
           end
            
           variances = zeros(sizeN,sizeN);
           tindex = 1;
           for i=startIdx:endIdx
                if (i< endIdx) U = plambda{1,1}.tenor(i);else U = to;end;
                if (i< alphaSize) tindex = i;else tindex = alphaSize;end;
            %% converting forward rate volatlity into short rate volatlitity
            %% eta is short rate volatlity
                for j=1:sizeN
                    for k=1:sizeN
%                         sigma_f(j,k)=plambda{j,1}.quote(tindex)*ch.Fwd(0,ch.tau(j))*ch.rho(j,k);
%                         sigma_f(j,k)=plambda{j,1}.quote(tindex)*ch.rho(j,k);
                        sigma_f(j,k)=sqrt(plambda{j,1}.quote(tindex)*plambda{j,1}.quote(tindex))*ch.rhoTerm{tindex,1}.quote(j,k);
                    end
                end
                
                eta = ch.inv_H_tau*sigma_f;
                eta2 = eta*eta';
            %%
                for j=1:sizeN
                    for k=1:sizeN
                        newExpScale(j,k) =  exp((meanR(j)+meanR(k))*U);
                        if(tiny(j,k)~=1)
                            scale = newExpScale(j,k)-lastExpScale(j,k);
                        else
                            scale = U - lastU;
                        end
                        variances(j,k) = variances(j,k) + eta2(j,k) * scale;
                        lastExpScale(j,k) = newExpScale(j,k);
                    end
                end
                lastU = U;
           end
           
           for j=1:sizeN
                for k=1:sizeN
                    if(tiny(j,k)~= 1)
                        variances(j,k) = variances(j,k)/(meanR(j)+meanR(k))*exp(-1.0*(meanR(j)+meanR(k))*to);
                    end
                end
           end
            
            
        end

        function beta = Beta(ch,t,T)
            beta = [];
            sizeN = ch.numOfFactor;
            beta = zeros(sizeN,1);
            
            for i=1:sizeN
                if(abs(ch.kappa(i)) > 1.0E-12)
                    beta(i) = (1.0-exp(-ch.kappa(i)*(T-t)))/ch.kappa(i);
                else
                    beta(i) = (T-t);
                end
            end
        end
        
        function beta = BetaDate(ch,valueDate,tDate,TDate)
            t = tDate.DateDiff(valueDate)/365.0;
            T = TDate.DateDiff(valueDate)/365.0;
            
            sizeN = ch.numOfFactor;
            beta = zeros(sizeN,1);
            
            for i=1:sizeN
                if(abs(ch.kappa(i)) > 1.0E-12)
                    beta(i) = (1.0-exp(-ch.kappa(i)*(T-t)))/ch.kappa(i);
                else
                    beta(i) = (T-t);
                end
            end
            
        end
        
        function c_nN = C_nN(ch,tn,tN,tau,pnN,ynN)
            p = @ch.DF;
            c_nN = [];
            sizeN = ch.numOfFactor;
            c_nN = zeros(sizeN,1);
            
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = ch.Beta(tn,tp);
                df = p(tp);
                for j=1:sizeN
                    c_nN(j) = c_nN(j) +tau * beta(j) * df;
                end
            end
            
            for j=1:sizeN
                c_nN(j) = c_nN(j)* ynN;
            end
            
            for j=1:sizeN
                c_nN(j) = c_nN(j) + beta(j)*df;
            end
            
            for j=1:sizeN
                c_nN(j) = c_nN(j)/pnN;
            end
           
        end
        
        function c_nN = C_nNDate(ch,valueDate,tnDate,tNDate,tau,pnN,ynN)
            tn = tnDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            
            addMonthUnit = round(12*tau);
            cnt = round((tN-tn)/tau);
            
            p = @ch.DF;
            sizeN = ch.numOfFactor;
            c_nN = zeros(sizeN,1);
            
            for i=1:cnt
                tpDate = tnDate.AddDate(i*addMonthUnit,'month');
                tp = tpDate.DateDiff(valueDate)/365.0;
                beta = ch.BetaDate(valueDate,tnDate,tpDate);
                df = p(tp);
                for j=1:sizeN
                    c_nN(j) = c_nN(j) +tau * beta(j) * df;
                end
            end
            
            for j=1:sizeN
                c_nN(j) = c_nN(j)* ynN;
            end
            
            for j=1:sizeN
                c_nN(j) = c_nN(j) + beta(j)*df;
            end
            
            for j=1:sizeN
                c_nN(j) = c_nN(j)/pnN;
            end
        end
        
        function g_nN = G_nN(ch,tn,tN,tau,pnN)
            p = @ch.DF;
            g_nN = [];
            sizeN = ch.numOfFactor;
            g_nN = zeros(sizeN,1);
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = ch.Beta(tn,tp);
                df = p(tp);
                for j=1:sizeN
                    g_nN(j) = g_nN(j) +tau * beta(j) * df;
                end
            end
            
            for j=1:sizeN
                g_nN(j) = g_nN(j)/pnN;
            end
            
        end
        
        function g_nN = G_nNDate(ch,valueDate,tnDate,tNDate,tau,pnN)
            tn = tnDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            addMonthUnit = round(12*tau);
            cnt = round((tN-tn)/tau);
            
            p = @ch.DF;
            sizeN = ch.numOfFactor;
            g_nN = zeros(sizeN,1);
            
            for i=1:cnt
                tpDate = tnDate.AddDate(i*addMonthUnit,'month');
                tp = tpDate.DateDiff(valueDate)/365.0;
                beta = ch.BetaDate(valueDate,tnDate,tpDate);
                df = p(tp);
                for j=1:sizeN
                    g_nN(j) = g_nN(j) +tau * beta(j) * df;
                end
            end
            
            for j=1:sizeN
                g_nN(j) = g_nN(j)/pnN;
            end
        end 
        
        function h_nTerm = H_nTerm(ch,tn,tTerm)
           h_nTerm =[];
           sizeN = ch.numOfFactor;
           h_nTerm = zeros(sizeN,1);
           beta = ch.Beta(tn,tTerm);
           
           for j=1:sizeN
                h_nTerm(j) = -1.0*beta(j);
           end
          
        end
        
        function h_nTerm = H_nTermDate(ch,valueDate,tnDate,tTermDate)
           h_nTerm =[];
           sizeN = ch.numOfFactor;
           h_nTerm = zeros(sizeN,1);
           beta = ch.BetaDate(valueDate,tnDate,tTermDate);
           for j=1:sizeN
                h_nTerm(j) = -1.0*beta(j);
           end
        end
        
        
        
        function out = CMSDigitalArg(ch,to,tp,tenor1,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           sizeN = ch.numOfFactor;
           p_nN1 = ch.PV01(to,to+tenor1,tau);
           df_Tn1 = ch.DF(to);
           df_TN1 = ch.DF(to + tenor1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           c_nN1 = ch.C_nN(to,to + tenor1,tau,p_nN1,y_nN1);
           g_nN1 = ch.G_nN(to,to + tenor1,tau,p_nN1);
           h_nTerm1 = ch.H_nTerm(to,tp);
           
           varCovar = ch.LocalVariance(0,to);
           
           driftTerm1=0.0;
           for i=1:sizeN
               for j=1:sizeN
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           for i=1:sizeN
               for j=1:sizeN
                    spreadVariance = spreadVariance + (c_nN1(i)) * (c_nN1(j)) * varCovar(i,j);
               end
           end
           
           out.mu = (y_nN1 + driftTerm1);
           if spreadVariance >=0
            out.sigma = sqrt(spreadVariance);
           else
            out.sigma = 0.0;
           end
           
           out.c_nN1 = c_nN1;
           out.varCovar = varCovar;
           
        end
        
        function out = CMSDigital(ch,to,tp,tenor1,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsDigitalArg = ch.CMSDigitalArg(to,tp,tenor1,tau);
           mu = cmsDigitalArg.mu;
           sigma = cmsDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSDigitalRange(ch,to,tp,tenor1,tau,lower,upper)
            lower_value = ch.CMSDigital(to,tp,tenor1,tau,lower);
            upper_value = ch.CMSDigital(to,tp,tenor1,tau,upper);
            out = lower_value - upper_value;
        end
        
        
        function out = CMSSpreadDigitalArg(ch,to,tp,tenor1,tenor2,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           sizeN = ch.numOfFactor;
           p_nN1 = ch.PV01(to,to+tenor1,tau);
           df_Tn1 = ch.DF(to);
           df_TN1 = ch.DF(to + tenor1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           c_nN1 = ch.C_nN(to,to + tenor1,tau,p_nN1,y_nN1);
           g_nN1 = ch.G_nN(to,to + tenor1,tau,p_nN1);
           h_nTerm1 = ch.H_nTerm(to,tp);
           
           %CMS2
           p_nN2 = ch.PV01(to,to+tenor2,tau);
           df_Tn2 = ch.DF(to);
           df_TN2 = ch.DF(to + tenor2);
           y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
           c_nN2 = ch.C_nN(to,to + tenor2,tau,p_nN2,y_nN2);
           g_nN2 = ch.G_nN(to,to + tenor2,tau,p_nN2);
           h_nTerm2 = ch.H_nTerm(to,tp);
           
           varCovar = ch.LocalVariance(0,to);
           
           driftTerm1=0.0;
           driftTerm2=0.0;
           for i=1:sizeN
               for j=1:sizeN
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
                    driftTerm2 = driftTerm2 + c_nN2(i) * (h_nTerm2(j) + g_nN2(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           variance1 = 0.0;
           variance2 = 0.0;
           coVariance = 0.0;
           for i=1:sizeN
               for j=1:sizeN
                    spreadVariance = spreadVariance + (c_nN1(i)-c_nN2(i)) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
                    variance1 = variance1 + (c_nN1(i)) * (c_nN1(j)) * varCovar(i,j);
                    variance2 = variance2 + (c_nN2(i)) * (c_nN2(j)) * varCovar(i,j);
                    coVariance = coVariance + (c_nN1(i)) * (c_nN2(j)) * varCovar(i,j);
               end
           end
           
           out.fwdSpread = y_nN1 - y_nN2;
           out.mu = (y_nN1 + driftTerm1) - (y_nN2 + driftTerm2);
           out.sigma = sqrt(spreadVariance);
           out.c_nN1 = c_nN1;
           out.c_nN2 = c_nN2;
           out.varCovar = varCovar;
           out.corr = coVariance/sqrt(variance1*variance2);
           
           out.mu1 = y_nN1 + driftTerm1;
           out.mu2 = y_nN2 + driftTerm2;
           
           out.sig1 = sqrt(variance1);
           out.sig2 = sqrt(variance2);
           
           out.pmu = driftTerm1 - driftTerm2;
           out.pmu1 = driftTerm1;
           out.pmu2 = driftTerm2;
           
           out.y1 = y_nN1;
           out.y2 = y_nN2;
           
        end
        
        function out = CMSSpreadDigitalArgDate(ch,valueDate,toDate,tpDate,tenor1,tenor2,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           sizeN = ch.numOfFactor;
           
           to      = toDate.DateDiff(valueDate)/365.0;
           tN1Date = toDate.AddDate(tenor1,'year');
           tN1   =  tN1Date.DateDiff(valueDate)/365.0;
           
           p_nN1 = ch.PV01Date(valueDate,toDate,tN1Date,tau);
           
           df_Tn1 = ch.DF(to);
           df_TN1 = ch.DF(tN1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           
           c_nN1 = ch.C_nNDate(valueDate,toDate,tN1Date,tau,p_nN1,y_nN1);
           g_nN1 = ch.G_nNDate(valueDate,toDate,tN1Date,tau,p_nN1);
           h_nTerm1 = ch.H_nTermDate(valueDate,toDate,tpDate);
           
           %CMS2
           tN2Date = toDate.AddDate(tenor2,'year');
           tN2   =  tN2Date.DateDiff(valueDate)/365.0;
           
           p_nN2 = ch.PV01Date(valueDate,toDate,tN2Date,tau);
           
           df_Tn2 = ch.DF(to);
           df_TN2 = ch.DF(tN2);
           y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
           
           c_nN2 = ch.C_nNDate(valueDate,toDate,tN2Date,tau,p_nN2,y_nN2);
           g_nN2 = ch.G_nNDate(valueDate,toDate,tN2Date,tau,p_nN2);
           h_nTerm2 = ch.H_nTermDate(valueDate,toDate,tpDate);
           
           varCovar = ch.LocalVariance(0,to);
           
           driftTerm1=0.0;
           driftTerm2=0.0;
           for i=1:sizeN
               for j=1:sizeN
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
                    driftTerm2 = driftTerm2 + c_nN2(i) * (h_nTerm2(j) + g_nN2(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           for i=1:sizeN
               for j=1:sizeN
                    spreadVariance = spreadVariance + (c_nN1(i)-c_nN2(i)) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
               end
           end
           
           out.mu = (y_nN1 + driftTerm1) - (y_nN2 + driftTerm2);
           out.sigma = sqrt(spreadVariance);
           out.c_nN1 = c_nN1;
           out.c_nN2 = c_nN2;
           out.varCovar = varCovar;
           
        end
        
        function out = CMSSpreadDigital(ch,to,tp,tenor1,tenor2,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsSpreadDigitalArg = ch.CMSSpreadDigitalArg(to,tp,tenor1,tenor2,tau);
           mu = cmsSpreadDigitalArg.mu;
           sigma = cmsSpreadDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSSpreadDigitalDate(ch,valueDate,toDate,tpDate,tenor1,tenor2,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsSpreadDigitalArg = ch.CMSSpreadDigitalArgDate(valueDate,toDate,tpDate,tenor1,tenor2,tau);
           mu = cmsSpreadDigitalArg.mu;
           sigma = cmsSpreadDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSSpreadDigitalRange(ch,to,tp,tenor1,tenor2,tau,lower,upper)
            lower_value = ch.CMSSpreadDigital(to,tp,tenor1,tenor2,tau,lower);
            upper_value = ch.CMSSpreadDigital(to,tp,tenor1,tenor2,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CMSSpreadDigitalRangeDate(ch,valueDate,toDate,tpDate,tenor1,tenor2,tau,lower,upper)
            lower_value = ch.CMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,lower);
            upper_value = ch.CMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,upper);
            out = lower_value - upper_value;
        end
        
        function atmSwaption = ATMSwaption(ch,tx,tnr)
            sizeN = ch.numOfFactor;
            tau = 0.25;
            pv01 = ch.PV01(tx,tx+tnr,tau);
            fsr  = ch.Fsr(tx,tx+tnr,tau);
            c_nN = ch.C_nN(tx,tx+tnr,tau,pv01,fsr);
            variances = ch.LocalVariance(0,tx);
            totalVar = 0.0;
            for i=1:sizeN
                for j=1:sizeN
                    totalVar = totalVar + c_nN(i)*c_nN(j)*variances(i,j);
                end
            end
            if totalVar <=0
                totalVar = 0.0;
            end
            atmSwaption =  pv01 * sqrt(totalVar)/sqrt(2.0*pi);
        end
        
        function atmSwaptionNormalVol = ATMSwaptionNormalVol(ch,tx,tnr)
            sizeN = ch.numOfFactor;
            tau = 0.25;
            pv01 = ch.PV01(tx,tx+tnr,tau);
            fsr  = ch.Fsr(tx,tx+tnr,tau);
            c_nN = ch.C_nN(tx,tx+tnr,tau,pv01,fsr);
            variances = ch.LocalVariance(0,tx);
            totalVar = 0.0;
            for i=1:sizeN
                for j=1:sizeN
                    totalVar = totalVar + c_nN(i)*c_nN(j)*variances(i,j);
                end
            end
            if totalVar <=0
                totalVar = 0.0;
            end
            atmSwaptionNormalVol =  sqrt(totalVar/tx);
        end
        
        function capletOut = Caplet(ch,tx,x,tnr)
        %blackCaplet:a function that computes a blackCaplet price
        %arguments
        %tx =(annualized)time to reset
        %sig= caplet volatility
        %x  = caplet strike
        %tnr=(annualized)time to payment
        %NA =Notional Amount
        
            tau = 0.25;
            p = @ch.DF;
            beta = ch.Beta(tx,tx + tnr);
            sizeN = ch.numOfFactor;
            variances = ch.LocalVariance(0,tx);
            totalVar = 0.0;
            for i=1:sizeN
                for j=1:sizeN
                    totalVar = totalVar + beta(i)*beta(j)*variances(i,j);
                end
            end
            sig = sqrt(totalVar);
            d1= (log(p(tx)/p(tx + tnr)*1.0/(1+tnr*x))+(sig^2/2.0))/(sig);
            d2 = d1 -sig;
            capletOut = p(tx)*H_ncdf(d1) - p(tx+tnr)*(1+tnr*x)*H_ncdf(d2);
        end
        
        function capOut = Cap(ch,tx,x,tnr)
            capOut = sum(arrayfun(@(i) ch.Caplet(tnr*i,x,tnr),1:round(tx/tnr)-1));
        end
        
         function atmSwaptionImpVol = ATMSwaptionImpVol(ch,tx,tnr)
            atmSwaption = ch.ATMSwaption(tx,tnr);
            
            a = 0.001;
            b = 10;
            Tol = 1e-10;
            MaxIter = 1000;
            tau = 0.25;
            fsr  = ch.Fsr(tx,tx+tnr,tau);
            pv01 = ch.PV01(tx,tx+tnr,tau);
            impVol = BisecBSIV('P',fsr,fsr, ...
                                          0.0,tx,a,b,atmSwaption/pv01,Tol,MaxIter);
            atmSwaptionImpVol =  impVol;
         end
        
         function out = findTindex(ch,t,sigma)
            
            sum = 0.0;
            prev_t = 0.0;
            inRangeYN = false;
            for i=1:length(sigma.tenor)
                alpha = i;
                if t<= sigma.tenor(i)
                    sum = alpha;
                    inRangeYN = true;
                    break;
                end
            end
            
            if ~inRangeYN 
                sum = alpha;
            end
            out = sum;
        end
        
        function instFwdCorrTest = InsFwdCorrTest(ch,tx,tnr1,tnr2)
            sizeN = ch.numOfFactor;
            plambda =cell(sizeN,1);
            for i=1:sizeN
                plambda{i,1}= ch.lambda{i,1};
            end
            
            sigma_f = zeros(sizeN,sizeN);
            
%             tindex = ch.findTindex(tx+1e-6,plambda{1,1});
            tenor = ch.lambda{1,1}.tenor;
            tindex = min(find(tx<=tenor));
            
            for j=1:sizeN
                for k=1:sizeN
                    sigma_f(j,k)=plambda{j,1}.quote(tindex)*ch.rhoTerm{tindex,1}.quote(j,k);
                end
            end
                
            eta = ch.inv_H_tau*sigma_f;
            eta2 = eta*eta';
            
            KappaTau1 = zeros(sizeN,1);
            KappaTau2 = zeros(sizeN,1);
            
            for i=1:sizeN
                KappaTau1(i) = exp(-ch.kappa(i)*tnr1);
                KappaTau2(i) = exp(-ch.kappa(i)*tnr2);
            end
            
            
            covar = 0.0;
            var1 = 0;
            var2 = 0;
            var1 =  KappaTau1'*eta2*KappaTau1;
            var2 =  KappaTau2'*eta2*KappaTau2;
            covar =  KappaTau1'*eta2*KappaTau2;
            
            instFwdCorrTest = covar/sqrt(var1*var2);
        end
        
        function instFwdCorrTestMatrix = InsFwdCorrTest2(ch,tx,tnr1,tau)
            if (tnr1(length(tnr1)) <= tx)
                disp('beyond the limit')
                instFwdCorrTestMatrix = 0;
            else

                % find T0 such that T0 =min{T_i|tx <= T_i}
                ipos = find(tnr1-tx > 0);
                T0 = tnr1(ipos(1));
                idx = ipos(1);
                newtnr = tnr1(idx:end);
                % make a matrix that spans T0,...,TN
                coVar = zeros(length(newtnr),length(newtnr));

                sizeN = ch.numOfFactor;
                plambda =cell(sizeN,1);
                for i=1:sizeN
                    plambda{i,1}= ch.lambda{i,1};
                end

                sigma_f = zeros(sizeN,sizeN);

    %             tindex = ch.findTindex(tx+1e-6,plambda{1,1});
                tenor = ch.lambda{1,1}.tenor;
                tindex = min(find(tx<=tenor));

                for j=1:sizeN
                    for k=1:sizeN
                        sigma_f(j,k)=plambda{j,1}.quote(tindex)*ch.rhoTerm{tindex}.quote(j,k);
                    end
                end

                eta = ch.inv_H_tau*sigma_f;
                eta2 = eta*eta';


                deltaG_tT = zeros(length(newtnr),sizeN);
                
                for i=1:length(newtnr)
                    G_tT1 = ch.Beta(tx,newtnr(i)+tau);
                    G_tT2 = ch.Beta(tx,newtnr(i));
                    for j=1:sizeN
                        deltaG_tT(i,j) = (G_tT1(j) - G_tT2(j))/tau;
                    end
                end

                
                testLoadingD = deltaG_tT*eta;

                coVar =  deltaG_tT*eta*eta'*deltaG_tT';
                instFwdCovarTestMatrix = coVar;
                %normalize the correlation matrix and calculate the loading matrix D
                rInv_m = zeros(length(newtnr),length(newtnr));
                for i=1:length(newtnr)
                    rInv_m(i,i) = 1.0/sqrt(coVar(i,i));
                end
                instFwdCorrTestMatrix = zeros(length(newtnr),length(newtnr));
                instFwdCorrTestMatrix = rInv_m*instFwdCovarTestMatrix*rInv_m';

            end
            
        end
        
        function InstSwapFwdCorrTest = InsSwapFwdCorrTest(ch,tx,tnr1,tnr2)
            sizeN = ch.numOfFactor;
            plambda =cell(sizeN,1);
            for i=1:sizeN
                plambda{i,1}= ch.lambda{i,1};
            end
            sigma_f = zeros(sizeN,sizeN);
            tenor = ch.lambda{1,1}.tenor;
            tindex = min(find(tx<=tenor));
            
            for j=1:sizeN
                for k=1:sizeN
                    sigma_f(j,k)=plambda{j,1}.quote(tindex)*ch.rhoTerm{tindex,1}.quote(j,k);
                end
            end
                
            eta = ch.inv_H_tau*sigma_f;
            eta2 = eta*eta';
            
            KappaTau1 = zeros(sizeN,1);
            KappaTau2 = zeros(sizeN,1);
            
            tau = 0.25;
            p_nN1 = ch.PV01(tx,tx+tnr1,tau);
            df_Tn1 = ch.DF(tx);
            df_TN1 = ch.DF(tx + tnr1);
            y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
            c_nN1 = ch.C_nN(tx,tx + tnr1,tau,p_nN1,y_nN1);
            
            p_nN2 = ch.PV01(tx,tx+tnr2,tau);
            df_Tn2 = ch.DF(tx);
            df_TN2 = ch.DF(tx + tnr2);
            y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
            c_nN2 = ch.C_nN(tx,tx + tnr2,tau,p_nN2,y_nN2);
            
            for i=1:sizeN
                KappaTau1(i) = c_nN1(i)*exp(-ch.kappa(i)*tnr1);
                KappaTau2(i) = c_nN2(i)*exp(-ch.kappa(i)*tnr2);
            end
            
            
            covar = 0.0;
            var1 = 0;
            var2 = 0;
            var1 =  KappaTau1'*eta2*KappaTau1;
            var2 =  KappaTau2'*eta2*KappaTau2;
            covar =  KappaTau1'*eta2*KappaTau2;
            
            InstSwapFwdCorrTest = covar/sqrt(var1*var2);
        end
        
        function spreadInfo = SpreadInfo(ch,tx,tnr1,tnr2)
            tau = 0.25;
            cmsSpreadDigitalArg =  ch.CMSSpreadDigitalArg(tx,tx+1.0,tnr1,tnr2,tau);
            spreadInfo = cmsSpreadDigitalArg;
            spreadInfo.normSigma = cmsSpreadDigitalArg.sigma/sqrt(tx);
        end
        
        function impCorrel = ImpCorrel(ch,tx,tnr1,tnr2)
            tau = 0.25;
            pv01_1 = ch.PV01(tx,tx+tnr1,tau);
            fsr_1  = ch.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = ch.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = ch.PV01(tx,tx+tnr2,tau);
            fsr_2  = ch.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = ch.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            variances = ch.LocalVariance(0,tx);
            var1 = 0;
            var2 = 0;
            coVar = 0;
            sizeN = ch.numOfFactor;
            for j=1:sizeN
                for k=1:sizeN
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                    var2 = var2 + c_nN_2(j)*c_nN_2(k)*variances(j,k);
                    coVar = coVar + c_nN_1(j)*c_nN_2(k)*variances(j,k);
                end
            end
            impCorrel = coVar/sqrt(var1)/sqrt(var2);
        end
        
        function impVariance = ImpVariance(ch,tx,tnr1)
            tau = 0.25;
            pv01_1 = ch.PV01(tx,tx+tnr1,tau);
            fsr_1  = ch.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = ch.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            variances = ch.LocalVariance(0,tx);
            var1 = 0;
            
            sizeN = ch.numOfFactor;
            for j=1:sizeN
                for k=1:sizeN
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                end
            end
            impVariance = var1;
        end
        
        function impCovar = ImpCovar(ch,tx,tnr1,tnr2)
            tau = 0.25;
            pv01_1 = ch.PV01(tx,tx+tnr1,tau);
            fsr_1  = ch.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = ch.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = ch.PV01(tx,tx+tnr2,tau);
            fsr_2  = ch.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = ch.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            variances = ch.LocalVariance(0,tx);
            var1 = 0;
            var2 = 0;
            coVar = 0;
            
            sizeN = ch.numOfFactor;
            for j=1:sizeN
                for k=1:sizeN
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                    var2 = var2 + c_nN_2(j)*c_nN_2(k)*variances(j,k);
                    coVar = coVar + c_nN_1(j)*c_nN_2(k)*variances(j,k);
                end
            end
            impCovar = coVar;
        end
        
        function out = RebonatoFwdCorr(ch,tx,tnr1,tnr2)
           rho_inf = 0.7;
           rho_decay = 0.15;
           out =  rho_inf + (1-rho_inf)*exp(-rho_decay*abs(tnr1-tnr2));
        end
        
        function out = RebonatoFwdCorr3(ch,tx,tnr1,tnr2)
           rho_inf = 0.4;
           rho_decay = 0.15;
           rho_alpha = 0.1;
           beta = rho_decay*exp(-rho_alpha*min(tnr1-tx,tnr2-tx));
           out =  rho_inf + (1-rho_inf)*exp(-beta*abs(tnr1-tnr2));
        end
        
        function out = TargetFunctionATMSwaptionAndCapBootStrapLevMar(ch,tvar,idxNow,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            chSwaptionPrice = zeros(length(tenor),1);
            
            capExpiry = params.capExpiry;
            blackCapPrice = params.blackCapPrice;
            blackCapStrike = params.blackCapStrike;
            
            useCap = false;
            if expiry(idxNow) <= capExpiry(length(capExpiry))
                if capExpiry(idxNow) == expiry(idxNow)
                    useCap = true;
                end
            end
            sizeN = ch.numOfFactor;
            for j=1:sizeN
                ch.lambda{j,1}.quote(idxNow) = tvar(j);
            end
            
            out= zeros(length(tenor)+ 1,1);
            for k=1:length(tenor)
                chSwaptionPrice(k) = ch.ATMSwaption(expiry(idxNow),tenor(k));
                out(k) = chSwaptionPrice(k)/blackSwaptionPrice(idxNow,k)-1.0;
            end
            
            if useCap
                chCapPrice = ch.Cap(capExpiry(idxNow),blackCapStrike(idxNow),0.25);
                out(length(tenor)+1) = chCapPrice/blackCapPrice(idxNow)-1.0;
            end

        end
        
        function out = TargetFunctionATMSwaptionAndCapGlobalLevMar(ch,tvar,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            chSwaptionPrice = zeros(length(expiry),length(tenor));
            
            capExpiry = params.capExpiry;
            blackCapPrice = params.blackCapPrice;
            blackCapStrike = params.blackCapStrike;
            chCapPrice = zeros(length(capExpiry),1);
            
            sizeN = ch.numOfFactor;
            alphaSize = length(ch.lambda{1,1}.tenor);
            for i=1:sizeN
                for j=1:alphaSize
                    ch.lambda{i,1}.quote(j) = tvar((i-1)*alphaSize +j);
                end
            end
            
            sizeOut = length(expiry) * length(tenor);
            sizeOut = sizeOut + length(capExpiry);
            
            out= zeros(sizeOut,1);
            for j=1:length(expiry)
                for k=1:length(tenor)
                    chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                    out((j-1)*length(tenor) + k) = chSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0;
                end
            end
            
            for j=1:length(capExpiry)
                chCapPrice(j) = ch.Cap(capExpiry(j),blackCapStrike(j),0.25);
                out(length(expiry)*length(tenor) + j) = chCapPrice(j)/blackCapPrice(j) - 1.0;
            end
        end 
        
        function out = TargetFunctionATMSwaptionBootStrapLevMar(ch,tvar,idxNow,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            chSwaptionPrice = zeros(length(tenor),1);
            
            sizeN = ch.numOfFactor;
            for j=1:sizeN
                ch.lambda{j,1}.quote(idxNow) = tvar(j);
            end
            
            out= zeros(sizeN,1);
            for k=1:length(tenor)
                chSwaptionPrice(k) = ch.ATMSwaption(expiry(idxNow),tenor(k));
                out(k) = chSwaptionPrice(k)/blackSwaptionPrice(idxNow,k)-1.0;
            end

        end
        
        function out = TargetFunctionATMSwaptionBootStrapLevMarNormalVol(ch,tvar,idxNow,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionNormalVol = params.blackSwaptionNormalVol;
            chATMSwaptionNormalVol = zeros(1,length(tenor));
            
            sizeN = ch.numOfFactor;
            for j=1:sizeN
                ch.lambda{j,1}.quote(idxNow) = tvar(j);
            end
            
            out= zeros(sizeN,1);
            for k=1:length(tenor)
                chATMSwaptionNormalVol(k) = ch.ATMSwaptionNormalVol(expiry(idxNow),tenor(k));
                out(k) = chATMSwaptionNormalVol(k)/blackSwaptionNormalVol(idxNow,k)-1.0;
            end

        end
        
        function out = TargetFunctionATMSwaptionBootStrapLevMarNormalVariance(ch,tvar,idxNow,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionNormalVol = params.blackSwaptionNormalVol;
            chATMSwaptionNormalVol = zeros(1,length(tenor));
            
            sizeN = ch.numOfFactor;
            for j=1:sizeN
                ch.lambda{j,1}.quote(idxNow) = tvar(j);
            end
            
            out= zeros(sizeN,1);
            for k=1:length(tenor)
                chATMSwaptionNormalVol(k) = ch.ATMSwaptionNormalVol(expiry(idxNow),tenor(k));
                out(k) = (chATMSwaptionNormalVol(k)*chATMSwaptionNormalVol(k)*expiry(idxNow))/(blackSwaptionNormalVol(idxNow,k)*blackSwaptionNormalVol(idxNow,k)*expiry(idxNow))-1.0;
            end

        end
        
        function out = TargetFunctionATMSwaptionBootStrapLevMarAbs(ch,tvar,idxNow,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            chSwaptionPrice = zeros(length(tenor),1);
            
            sizeN = ch.numOfFactor;
            for j=1:sizeN
                ch.lambda{j,1}.quote(idxNow) = abs(tvar(j));
            end
            
            out= zeros(sizeN,1);
            for k=1:length(tenor)
                chSwaptionPrice(k) = ch.ATMSwaption(expiry(idxNow),tenor(k));
                out(k) = chSwaptionPrice(k)/blackSwaptionPrice(idxNow,k)-1.0;
            end

        end
        
        function out = TargetFunctionATMSwaptionGlobalLevMar(ch,tvar,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            chSwaptionPrice = zeros(length(expiry),length(tenor));
            
            sizeN = ch.numOfFactor;
            alphaSize = length(ch.lambda{1,1}.tenor);
            for i=1:sizeN
                for j=1:alphaSize
                    ch.lambda{i,1}.quote(j) = tvar((i-1)*alphaSize +j);
                end
            end
            
            sizeOut = length(expiry) * length(tenor);
            out= zeros(sizeOut,1);
            for j=1:length(expiry)
                for k=1:length(tenor)
                    chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                    out((j-1)*length(tenor) + k) = chSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0;
                end
            end
        end 
        
        function out = TargetFunctionATMSwaptionGlobalLevMarKappa(ch,tvar,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            chSwaptionPrice = zeros(length(expiry),length(tenor));
            
            sizeN = ch.numOfFactor;
            alphaSize = length(ch.lambda{1,1}.tenor);
            for i=1:sizeN
                for j=1:alphaSize
                    ch.lambda{i,1}.quote(j) = tvar((i-1)*alphaSize +j);
                end
            end
            
            for i=1:sizeN
                ch.kappa(i) = tvar(sizeN*alphaSize + i);
            end
            
            sizeOut = length(expiry) * length(tenor);
            out= zeros(sizeOut,1);
            for j=1:length(expiry)
                for k=1:length(tenor)
                    chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                    out((j-1)*length(tenor) + k) = chSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0;
                end
            end
        end 

        function out = TargetFunctionATMSwaptionGlobal(ch,tvar,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            chSwaptionPrice = zeros(1,size(tenor,2));
            
            sizeN = ch.numOfFactor;
            alphaSize = length(ch.lambda{1,1}.tenor);
            for i=1:sizeN
                for j=1:alphaSize
                    ch.lambda{i,1}.quote(j) = tvar((i-1)*alphaSize +j);
                end
            end
            
            out= 0;
            for j=1:length(expiry)
                for k=1:length(tenor)
                    chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                    out = out + (chSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0)*(chSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0)*10000;
                end
            end
        end   
        
        function out = TargetFunctionATMSwaptionGlobalImpVol(ch,tvar,params)
            a = 0.001;
            b = 10;
            Tol = 1e-10;
            MaxIter = 1000;

            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionImpVol = params.blackSwaptionImpVol;
            chSwaptionPrice = zeros(1,size(tenor,2));
            
            sizeN = ch.numOfFactor;
            alphaSize = length(ch.lambda{1,1}.tenor);
            for i=1:sizeN
                for j=1:alphaSize
                    ch.lambda{i,1}.quote(j) = tvar((i-1)*alphaSize +j);
                end
            end
            
            out= 0;
            for j=1:length(expiry)
                for k=1:length(tenor)
                    chATMSwaptionImpVol(j,k) = ch.ATMSwaptionImpVol(expiry(j),tenor(k));
                    out = out + (chATMSwaptionImpVol(j,k)- blackSwaptionImpVol(j,k))*(chATMSwaptionImpVol(j,k)- blackSwaptionImpVol(j,k))*10000;
                end
            end
        end   
        
        function out = TargetFunctionATMSwaptionGlobalNormalVol(ch,tvar,params)
            
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionNormalVol = params.blackSwaptionNormalVol;
            chATMSwaptionNormalVol = zeros(length(expiry),length(tenor));
            
            sizeN = ch.numOfFactor;
            alphaSize = length(ch.lambda{1,1}.tenor);
            for i=1:sizeN
                for j=1:alphaSize
                    ch.lambda{i,1}.quote(j) = tvar((i-1)*alphaSize +j);
                end
            end
            
            out= 0;
            for j=1:length(expiry)
                for k=1:length(tenor)
                    chATMSwaptionNormalVol(j,k) = ch.ATMSwaptionNormalVol(expiry(j),tenor(k));
                    out = out + (chATMSwaptionNormalVol(j,k)/blackSwaptionNormalVol(j,k)-1.0)*(chATMSwaptionNormalVol(j,k)/blackSwaptionNormalVol(j,k)-1.0)*10000;
                end
            end
        end   
        
        function out = CalibrateToATMSwaption(ch,black,calibrationFlag)
            tic;
            
            sizeN = ch.numOfFactor;
            alphaSize = length(ch.lambda{1,1}.tenor);
            for i=1:sizeN
               for j=1:alphaSize
                   out.lambda{i,1} = ch.lambda{i,1};
               end
            end
            
          % Calculate Black ATM Swaption Prices
          
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
            
            expirySize = size(expiry,2);
            tenorSize = size(tenor,2);
            
            blackSwaptionPrice = zeros(expirySize,tenorSize);
            blackSwaptionImpVol = zeros(expirySize,tenorSize);
            blackSwaptionVega = zeros(expirySize,tenorSize);
            blackSwaptionFsr = zeros(expirySize,tenorSize);
%             blackSwaptionNormalVol = zeros(expirySize,tenorSize);
            
            for k=1:expirySize
                for j=1:tenorSize
                    blackSwaptionPrice(k,j)  = black.BlackATMSwaption(expiry(k),tenor(j));
                    blackSwaptionImpVol(k,j) = black.swaptionVol(expiry(k),tenor(j));
                    blackSwaptionFsr(k,j) = black.Fsr(expiry(k),expiry(k) + tenor(j),0.25);
                    blackSwaptionVega(k,j) = black.BlackSwaptionVegaWithVol(expiry(k),tenor(j),blackSwaptionFsr(k,j),blackSwaptionImpVol(k,j));
%                     blackSwaptionNormalVol(k,j) = black.ATMSwaptionNormalVol(expiry(k),tenor(j));
                end
            end
            
            params.expiry = expiry;
            params.tenor = tenor;
            params.blackSwaptionPrice = blackSwaptionPrice;
            params.blackSwaptionImpVol = blackSwaptionImpVol;
            params.blackSwaptionFsr = blackSwaptionFsr;
            params.blackSwaptionVega = blackSwaptionVega;
%             params.blackSwaptionNormalVol = blackSwaptionNormalVol;
            params.expirySize = expirySize;
            params.tenorSize =  tenorSize;
            
            params.swaption_calibration_yn = ch.swaption_calibration_yn;
            
            out.blackSwaptionPrice = blackSwaptionPrice;
            out.blackSwaptionVega = blackSwaptionVega;
            out.blackSwaptionFsr = blackSwaptionFsr;
            
            out.chSwaptionPrice = zeros(expirySize,tenorSize);
            out.relDiff = zeros(expirySize,tenorSize);
            out.relVolDiff = zeros(expirySize,tenorSize);
            out.rmseTotal=0.0;
            out.rmseVolTotal = 0.0;
            
            out.impVol2Total = 0.0;
            out.normalVol2Total = 0.0;
            out.normalVarianceTotal = 0.0;
            
            calibrationType = 'bootstrap';
            if nargin < 3
                calibrationType = 'bootstrap'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;
            tvar = [];
            
            switch calibrationType
                case 'bootstrapLevMar'
                    tvar = zeros(sizeN,1);
                    for i=1:expirySize
                        idxNow = i;
                        for j=1:sizeN
                             tvar(j) = ch.lambda{j,1}.quote(i);
                             lb(j) = 0.0001;
                             ub(j) = 10.0;
                        end
                        
                        targetfun = @(tvar) TargetFunctionATMSwaptionBootStrapLevMar(ch,tvar,idxNow,params);
                        options=optimset('Algorithm','levenberg-marquardt');
%                         options=optimset('Algorithm','trust-region-reflective');
                        [x, fval] = lsqnonlin(targetfun,tvar,lb,ub,options);
                        
                        for j=1:sizeN
                             out.lambda{j,1}.quote(i) = x(j); 
                             ch.lambda{j,1}.quote(i) = x(j); 
                        end
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.calibrationTime = toc;
                case 'bootstrapLevMarNormalVol'
                    tvar = zeros(sizeN,1);
                    for i=1:expirySize
                        idxNow = i;
                        for j=1:sizeN
                             tvar(j) = ch.lambda{j,1}.quote(i);
                             lb(j) = 0.0001;
                             ub(j) = 10.0;
                        end
                        
                        targetfun = @(tvar) TargetFunctionATMSwaptionBootStrapLevMarNormalVol(ch,tvar,idxNow,params);
                        options=optimset('Algorithm','levenberg-marquardt');
%                         options=optimset('Algorithm','trust-region-reflective');
                        [x, fval] = lsqnonlin(targetfun,tvar,lb,ub,options);
                        
                        for j=1:sizeN
                             out.lambda{j,1}.quote(i) = x(j); 
                             ch.lambda{j,1}.quote(i) = x(j); 
                        end
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.blackSwaptionImpVol(j,k) = blackSwaptionImpVol(j,k);
                            out.blackSwaptionNormalVol(j,k) = blackSwaptionNormalVol(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.chSwaptionImpVol(j,k) = ch.ATMSwaptionImpVol(expiry(j),tenor(k));
                            out.chSwaptionNormalVol(j,k) = ch.ATMSwaptionNormalVol(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.ImpVolDiff(j,k) = out.chSwaptionImpVol(j,k) - out.blackSwaptionImpVol(j,k);
                            out.NormalVolDiff(j,k) = out.chSwaptionNormalVol(j,k)/out.blackSwaptionNormalVol(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            out.impVol2Total = out.impVol2Total + out.ImpVolDiff(j,k)*out.ImpVolDiff(j,k);
                            out.normalVol2Total = out.normalVol2Total + out.NormalVolDiff(j,k)*out.NormalVolDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.impVol2Total = sqrt(out.impVol2Total/N);
                    out.normalVol2Total = sqrt(out.normalVol2Total/N);
                    out.calibrationTime = toc;
                    
                case 'bootstrapLevMarNormalVariance'
                    tvar = zeros(sizeN,1);
                    for i=1:expirySize
                        idxNow = i;
                        for j=1:sizeN
                             tvar(j) = ch.lambda{j,1}.quote(i);
                             lb(j) = 0.0001;
                             ub(j) = 10.0;
                        end
                        
                        targetfun = @(tvar) TargetFunctionATMSwaptionBootStrapLevMarNormalVariance(ch,tvar,idxNow,params);
                        options=optimset('Algorithm','levenberg-marquardt');
%                         options=optimset('Algorithm','trust-region-reflective');
                        [x, fval] = lsqnonlin(targetfun,tvar,lb,ub,options);
                        
                        for j=1:sizeN
                             out.lambda{j,1}.quote(i) = x(j); 
                             ch.lambda{j,1}.quote(i) = x(j); 
                        end
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.blackSwaptionImpVol(j,k) = blackSwaptionImpVol(j,k);
                            out.blackSwaptionNormalVol(j,k) = blackSwaptionNormalVol(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.chSwaptionImpVol(j,k) = ch.ATMSwaptionImpVol(expiry(j),tenor(k));
                            out.chSwaptionNormalVol(j,k) = ch.ATMSwaptionNormalVol(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.ImpVolDiff(j,k) = out.chSwaptionImpVol(j,k) - out.blackSwaptionImpVol(j,k);
                            out.NormalVolDiff(j,k) = out.chSwaptionNormalVol(j,k)/out.blackSwaptionNormalVol(j,k)-1;
                            out.NormalVarianceDiff(j,k) = (out.chSwaptionNormalVol(j,k)*out.chSwaptionNormalVol(j,k)*expiry(j))/(out.blackSwaptionNormalVol(j,k)*out.blackSwaptionNormalVol(j,k)*expiry(j))-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            out.impVol2Total = out.impVol2Total + out.ImpVolDiff(j,k)*out.ImpVolDiff(j,k);
                            out.normalVol2Total = out.normalVol2Total + out.NormalVolDiff(j,k)*out.NormalVolDiff(j,k);
                            out.normalVarianceTotal = out.normalVarianceTotal + out.NormalVarianceDiff(j,k)*out.NormalVarianceDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.impVol2Total = sqrt(out.impVol2Total/N);
                    out.normalVol2Total = sqrt(out.normalVol2Total/N);
                    out.normalVarianceTotal = sqrt(out.normalVarianceTotal/N);
                    out.calibrationTime = toc;
                    
                case 'globalLevMar'
                    for i=1:sizeN
                        for j=1:alphaSize
                             tvar((i-1)*alphaSize +j) = ch.lambda{i,1}.quote(j);
                             lb((i-1)*alphaSize +j) = 0.0001;
                             ub((i-1)*alphaSize +j) = 10.0;
                        end
                    end
                    
                       
                    targetfun = @(tvar) TargetFunctionATMSwaptionGlobalLevMar(ch,tvar,params);
%                     options=optimset('Algorithm','levenberg-marquardt');
                    options=optimset('Algorithm','levenberg-marquardt','MaxFunEvals',700);
                    [x, fval] = lsqnonlin(targetfun,tvar,lb,ub,options);
                    
                    for i=1:sizeN
                        for j=1:alphaSize
                             out.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                             ch.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                        end
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.calibrationTime = toc;
                
                 case 'dummy'
                    
                    for i=1:sizeN
                        for j=1:alphaSize
                             out.lambda{i,1}.quote(j) = ch.lambda{i,1}.quote(j); 
                        end
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.calibrationTime = toc;
                    
                    
                case 'globalLevMarKappa'
                    for i=1:sizeN
                        for j=1:alphaSize
                             tvar((i-1)*alphaSize +j) = ch.lambda{i,1}.quote(j);
                             lb((i-1)*alphaSize +j) = 0.0001;
                             ub((i-1)*alphaSize +j) = 10.0;
                        end
                    end
                    
                    for i=1:sizeN
                        tvar(sizeN*alphaSize + i) = ch.kappa(i);
                        lb(sizeN*alphaSize + i) = 0.0001;
                        ub(sizeN*alphaSize + i) = 10.0;
                    end
                       
                    targetfun = @(tvar) TargetFunctionATMSwaptionGlobalLevMarKappa(ch,tvar,params);
                    options=optimset('Algorithm','levenberg-marquardt');
                    [x, fval] = lsqnonlin(targetfun,tvar,lb,ub,options);
                    
                    for i=1:sizeN
                        for j=1:alphaSize
                             out.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                             ch.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                        end
                    end
                    
                    for i=1:sizeN
                        out.kappa(i) = x(sizeN*alphaSize+i);
                        ch.kappa(i)  = x(sizeN*alphaSize+i);
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);    
                    
                case 'global'
                    optParams.params = params;
                    optParams.modelParamsSize = sizeN * alphaSize;
                    optParams.numOfEquation = params.expirySize*params.tenorSize + ...  % target cap & swaption
                                              sizeN*(alphaSize -2);   % tikhonov regularization gamma constraint
                    
                    tvar = zeros(sizeN*alphaSize,1);
                    for i=1:sizeN
                        for j=1:alphaSize
                             tvar((i-1)*alphaSize +j) = ch.lambda{i,1}.quote(j);
                        end
                    end
                    
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
                    
                    x=  zeros(optParams.numOfEquation,1);
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionCheyetteATMSwaptionSurfaceGlobal',tvar,x, 500, options,'unc',ch,optParams);
                    toc

%                     targetfun = @(tvar) TargetFunctionATMSwaptionGlobal(ch,tvar,params);
% %                     options=optimset('Algorithm','interior-point');
%                      options=optimset('Algorithm','trust-region-reflective');
%                     [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);
                    
                    for i=1:sizeN
                        for j=1:alphaSize
                             out.lambda{i,1}.quote(j) = max(abs(popt((i-1)*alphaSize +j)),0.0001); 
                             ch.lambda{i,1}.quote(j) = max(abs(popt((i-1)*alphaSize +j)),0.0001); 
                        end
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                 case 'globalVegaWeight'
                    optParams.params = params;
                    optParams.modelParamsSize = sizeN * alphaSize;
                    optParams.numOfEquation = params.expirySize*params.tenorSize + ...  % target cap & swaption
                                              sizeN*(alphaSize -2);   % tikhonov regularization gamma constraint
                    
                    tvar = zeros(sizeN*alphaSize,1);
                    for i=1:sizeN
                        for j=1:alphaSize
                             tvar((i-1)*alphaSize +j) = ch.lambda{i,1}.quote(j);
                        end
                    end
                    
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
                    
                    x=  zeros(optParams.numOfEquation,1);
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionCheyetteATMSwaptionSurfaceGlobalVegaWeight',tvar,x, 500, options,'unc',ch,optParams);
                    toc

%                     targetfun = @(tvar) TargetFunctionATMSwaptionGlobal(ch,tvar,params);
% %                     options=optimset('Algorithm','interior-point');
%                      options=optimset('Algorithm','trust-region-reflective');
%                     [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);
                    
                    for i=1:sizeN
                        for j=1:alphaSize
                             out.lambda{i,1}.quote(j) = max(abs(popt((i-1)*alphaSize +j)),0.0001); 
                             ch.lambda{i,1}.quote(j) = max(abs(popt((i-1)*alphaSize +j)),0.0001); 
                        end
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.relVolDiff(j,k) = (out.chSwaptionPrice(j,k) - out.blackATMSwaptionPrice(j,k))/out.blackSwaptionVega(j,k);
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            out.rmseVolTotal = out.rmseVolTotal + out.relVolDiff(j,k)*out.relVolDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.rmseVolTotal = sqrt(out.rmseVolTotal/N);
                    
                 case 'globalFmin'
                    tvar = zeros(sizeN*alphaSize,1);
                    for i=1:sizeN
                        for j=1:alphaSize
                             tvar((i-1)*alphaSize +j) = ch.lambda{i,1}.quote(j);
                             lb((i-1)*alphaSize +j) = 0.01;
                             ub((i-1)*alphaSize +j) = 10.0;
                        end
                    end
                    
                       
                    targetfun = @(tvar) TargetFunctionATMSwaptionGlobal(ch,tvar,params);
%                     options=optimset('Algorithm','interior-point');
                     options=optimset('Algorithm','trust-region-reflective');
                    [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);
                    
                    for i=1:sizeN
                        for j=1:alphaSize
                             out.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                             ch.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                        end
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                 case 'globalImpVol'
                    for i=1:sizeN
                        for j=1:alphaSize
                             tvar((i-1)*alphaSize +j) = ch.lambda{i,1}.quote(j);
                             lb((i-1)*alphaSize +j) = 0.0001;
                             ub((i-1)*alphaSize +j) = 10.0;
                        end
                    end
                    
                       
                    targetfun = @(tvar) TargetFunctionATMSwaptionGlobalImpVol(ch,tvar,params);
%                     options=optimset('Algorithm','interior-point');
                     options=optimset('Algorithm','trust-region-reflective');
                    [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);
                    
                    for i=1:sizeN
                        for j=1:alphaSize
                             out.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                             ch.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                        end
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.blaskSwaptionImpVol(j,k) = blackSwaptionImpVol(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.chSwaptionImpVol(j,k) = ch.ATMSwaptionImpVol(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            our.volDiff(j,k) = out.chSwaptionImpVol(j,k) - out.blaskSwaptionImpVol(j,k);
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            out.vol2Total = out.vol2Total + our.volDiff(j,k)*our.volDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.vol2Total = sqrt(out.vol2Total/N);
                    
                case 'globalNormalVol'
                    for i=1:sizeN
                        for j=1:alphaSize
                             tvar((i-1)*alphaSize +j) = ch.lambda{i,1}.quote(j);
                             lb((i-1)*alphaSize +j) = 0.0001;
                             ub((i-1)*alphaSize +j) = 10.0;
                        end
                    end
                    
                       
                    targetfun = @(tvar) TargetFunctionATMSwaptionGlobalNormalVol(ch,tvar,params);
%                     options=optimset('Algorithm','interior-point');
                     options=optimset('Algorithm','trust-region-reflective');
                    [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);
                    
                    for i=1:sizeN
                        for j=1:alphaSize
                             out.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                             ch.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                        end
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.blackSwaptionImpVol(j,k) = blaskSwaptionImpVol(j,k);
                            out.blackSwaptionNormalVol(j,k) = blackSwaptionNormalVol(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.chSwaptionImpVol(j,k) = ch.ATMSwaptionImpVol(expiry(j),tenor(k));
                            out.chSwaptionNormalVol(j,k) = ch.ATMSwaptionNormalVol(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.ImpVolDiff(j,k) = out.chSwaptionImpVol(j,k) - out.blackSwaptionImpVol(j,k);
                            out.NormalVolDiff(j,k) = out.chSwaptionNormalVol(j,k)/out.blackSwaptionNormalVol(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            out.impVol2Total = out.impVol2Total + out.ImpVolDiff(j,k)*out.ImpVolDiff(j,k);
                            out.normalVol2Total = out.normalVol2Total + out.NormalVolDiff(j,k)*out.NormalVolDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.impVol2Total = sqrt(out.impVol2Total/N);
                    out.normalVol2Total = sqrt(out.normalVol2Total/N);
                
                otherwise
                    disp('unImplemented')
            end
        end
        
        function out = CalibrateToATMSwaptionAndCap(ch,black,calibrationFlag)
            tic;
            
            sizeN = ch.numOfFactor;
            alphaSize = length(ch.lambda{1,1}.tenor);
            for i=1:sizeN
               for j=1:alphaSize
                   out.lambda{i,1} = ch.lambda{i,1};
               end
            end
            
          %% Calculate Black ATM Swaption Prices
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
            
            expirySize = size(expiry,2);
            tenorSize = size(tenor,2);
            
            blackSwaptionPrice = zeros(expirySize,tenorSize);
            blackSwaptionImpVol = zeros(expirySize,tenorSize);
            for k=1:expirySize
                for j=1:tenorSize
                    blackSwaptionPrice(k,j)  = black.BlackATMSwaption(expiry(k),tenor(j));
                    blackSwaptionImpVol(k,j) = black.swaptionVol(expiry(k),tenor(j));
                end
            end
            
          %% Calculate Black ATM Cap Prices
           capVol = black.capVol;
           capVolData = black.mktData('capVol');
           capVolrawData =  capVolData.params('rawData');
            
           capExpiry = capVolrawData(:,1);
           capExpirySize = length(capExpiry);
            
           blackCapPrice = zeros(capExpirySize,1);
           blackCapImpVol = zeros(capExpirySize,1);
           blackCapStrike = zeros(capExpirySize,1);
           for k=1:capExpirySize
               blackCapStrike(k) = black.Fsr(0.25,capExpiry(k) -0.25,0.25);
               blackCapPrice(k) = black.BlackCap(capExpiry(k),blackCapStrike(k),0.25);
               blackCapImpVol(k) = black.capVol(capExpiry(k));
           end
          
          
            %%
            params.expiry = expiry;
            params.tenor = tenor;
            params.blackSwaptionPrice = blackSwaptionPrice;
            params.blackSwaptionImpVol = blackSwaptionImpVol;
            params.expirySize = expirySize;
            params.tenorSize =  tenorSize;
            
            params.capExpiry = capExpiry;
            params.blackCapPrice = blackCapPrice;
            params.blackCapStrike = blackCapStrike;
            params.blackCapImpVol = blackCapImpVol;
            params.capExpirySize = capExpirySize;
            
            out.blackSwaptionPrice = blackSwaptionPrice;
            out.chSwaptionPrice = zeros(expirySize,tenorSize);
            out.relDiff = zeros(expirySize,tenorSize);
            out.rmseTotal=0.0;
            
            out.blackCapPrice = blackCapPrice;
            out.chCapPrice = zeros(capExpirySize,1);
            out.capRelDiff = zeros(capExpirySize,1);
            
            calibrationType = 'bootstrap';
            if nargin < 3
                calibrationType = 'bootstrap'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;
            tvar = [];
            
            switch calibrationType
                case 'bootstrapLevMar'
                    tvar = zeros(sizeN,1);
                    for i=1:expirySize
                        idxNow = i;
                        for j=1:sizeN
                             tvar(j) = ch.lambda{j,1}.quote(i);
                             lb(j) = 0.0;
                             ub(j) = 10.0;
                        end
                        
                        targetfun = @(tvar) TargetFunctionATMSwaptionAndCapBootStrapLevMar(ch,tvar,idxNow,params);
                        options=optimset('Algorithm','levenberg-marquardt');
%                         options=optimset('Algorithm','trust-region-reflective');
                        [x, fval] = lsqnonlin(targetfun,tvar,lb,ub,options);
                        
                        for j=1:sizeN
                             out.lambda{j,1}.quote(i) = x(j); 
                             ch.lambda{j,1}.quote(i) = x(j); 
                        end
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                    
                    for j=1:capExpirySize
                        out.chCapPrice(j) = ch.Cap(capExpiry(j),blackCapStrike(j),0.25);
                        out.capRelDiff(j) = out.chCapPrice(j)/out.blackCapPrice(j) -1.0;
                        out.rmseTotal = out.rmseTotal +  out.capRelDiff(j)* out.capRelDiff(j);
                        N=N+1;
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                out.calibrationTime = toc;
                
                case 'globalLevMar'
                    for i=1:sizeN
                        for j=1:alphaSize
                             tvar((i-1)*alphaSize +j) = ch.lambda{i,1}.quote(j);
                             lb((i-1)*alphaSize +j) = 0.0;
                             ub((i-1)*alphaSize +j) = 10.0;
                        end
                    end
                    
                       
                    targetfun = @(tvar) TargetFunctionATMSwaptionAndCapGlobalLevMar(ch,tvar,params);
                    options=optimset('Algorithm','levenberg-marquardt');
                    [x, fval] = lsqnonlin(targetfun,tvar,lb,ub,options);
                    
                    for i=1:sizeN
                        for j=1:alphaSize
                             out.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                             ch.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                        end
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                    
                    for j=1:capExpirySize
                        out.chCapPrice(j) = ch.Cap(capExpiry(j),blackCapStrike(j),0.25);
                        out.capRelDiff(j) = out.chCapPrice(j)/out.blackCapPrice(j) -1.0;
                        out.rmseTotal = out.rmseTotal +  out.capRelDiff(j)* out.capRelDiff(j);
                        N=N+1;
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.calibrationTime = toc;
                   
                otherwise
                    disp('unImplemented')
            end
        end    
    end
    
end



In [ ]:
classdef CheyetteRisky < IRModelRisky
    %UNTITLED Summary of this class goes here
    %   Detailed explanation goes here
    
    properties
        % vol functor
        smoothAlpha;
        numOfFactor;
        lambda = {};
        kappa = [];
        tau = [];
        corr_fwd = [];
        rho = [];
        inv_H_tau = [];
    end
    
    methods
        function ch = CheyetteRisky(params)
            if nargin > 0
                ch.zeroCurve = params.zeroCurve;
                ch.discountCurve = params.discountCurve;
                ch.mktData = params.mktData;
                ch.modelParams = params.modelParams;
                
                ch.smoothAlpha =  params.modelParams('smoothAlpha');
                ch.numOfFactor = params.modelParams('numOfFactor');
                sizeN = ch.numOfFactor;
                ch.lambda = cell(sizeN,1);
                ch.lambda = params.modelParams('lambda');
                
                ch.kappa = ones(sizeN,1);
                ch.kappa =  params.modelParams('kappa');
               
                ch.tau = ones(sizeN,1);
                ch.tau =  params.modelParams('tau');
                ch.inv_H_tau = ones(sizeN,sizeN);
                H_tau = ones(sizeN,sizeN);
                for i=1:sizeN
                    for j=1:sizeN
                        H_tau(i,j) = exp(-ch.kappa(j)* ch.tau(i));
                    end
                end
                ch.inv_H_tau = inv(H_tau);
                
                ch.corr_fwd = zeros(sizeN,sizeN);
                ch.corr_fwd =  params.modelParams('corr_fwd');
                
                ch.rho = zeros(sizeN,sizeN); 
                [PC,V]=eig(ch.corr_fwd);
                PCW= PC*sqrt(V);
                for i=1:sizeN
                    for j=1:sizeN
                        ch.rho(i,j)=PCW(i,j);
                    end
                end
                
           
                
            end
        end
        
        function out = SimpleSRPrOverHedge(ch,spreadC,spreadN,ralower,raupper)
            buffer = 0.001;
            outVec = 0.0;
            if spreadC < ralower - buffer
                outVec = 0.0;
            elseif spreadC >= ralower - buffer && spreadC < ralower
                outVec = 1.0/buffer * (spreadC - (ralower - buffer));
            elseif spreadC >= ralower && spreadC < raupper
                outVec = 1.0;
            elseif spreadC >= raupper && spreadC < raupper + buffer
                outVec = -1.0/buffer *(spreadC - (raupper + buffer));
            else
                outVec = 0.0;
            end
            
            out = outVec;
        end
        
        function out = SimpleSRPr(ch,spreadC,spreadN,ralower,raupper)
            outVec = 0.0;
            if spreadC >= ralower && spreadC <= raupper
                outVec = 1.0;
            end
            out = outVec;
        end
        
        
        function out = TRZ_SRPr(ch,curr,next,L,U)
            length = 1.0;
            remain = 1.0;
            if (curr >= L && curr <= U && next >= L && next <= U)
                length = remain;
            elseif ( curr >= L && curr <= U && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr >= L && curr <= U && next < L ) 
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr > U && next >= L && next <= U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next < L )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next >= L && next <= U )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next > U )
                length = 0.0;
            elseif ( curr < L && next < L)
                length = 0.0;
            else
                disp('error')
            end
            
            out = length;
        end
        
        function out = SRPr(ch,spreadC,spreadN,ralower,raupper,simpleYN,pathSize)
            
            outVec = zeros(1,pathSize);
            
            if simpleYN == 1
                for i =1:pathSize
                    outVec(i) = ch.SimpleSRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            elseif simpleYN == 2
                for i =1:pathSize
                    outVec(i) = ch.SimpleSRPrOverHedge(spreadC(i),spreadN(i),ralower,raupper);
                end
            else
                for i =1:pathSize
                    outVec(i) = ch.TRZ_SRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            end
            
            out = outVec;
        end
        
        function out =  Libor_SimpleDate(ch,valueDate,evalTime, fwdStartTime, numMatTime, tenor,modelStatesT)
           %CMS
           fwdStartDate = valueDate.AddDate(fwdStartTime,'day');
           fwdEndDate = fwdStartDate.AddDate(tenor,'year');
           fwdEndTime = fwdEndDate.DateDiff(valueDate);
           
           deltaT = (fwdEndTime - fwdStartTime)/365.0;
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           
           stateSize = size(modelStatesT,2);
           df_Tn = zeros(1,stateSize);
           df_TN = zeros(1,stateSize);
           
           df_Tn = ch.discountFactor(evalTime,fwdStartTime,numMatTime,modelStatesT);
           df_TN = ch.discountFactor(evalTime,fwdEndTime,numMatTime,modelStatesT);
           
           liborOut = zeros(1,stateSize);
           
           for i=1:stateSize
                liborOut(i) = (df_Tn(i)/df_TN(i) - 1.0)/deltaT;
           end
           
           out = liborOut;
            
        end
        
        function out =  CMS_PSA_SimpleDate(ch,valueDate,observeTime,numMatTime, tenor,modelStatesT)
           %CMS
           tau = 0.25;
           to      = observeTime/365.0;
           toDate = valueDate.AddDate(observeTime,'day');
           tNDate = toDate.AddDate(tenor,'year');
           tN   =  tNDate.DateDiff(valueDate)/365.0;
           
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           p_nN = ch.PV01Date(valueDate,toDate,tNDate,tau);
           
           df_Tn = ch.DF(to);
           df_TN = ch.DF(tN);
           y_nN = (df_Tn - df_TN)/p_nN;
           
           c_nN = ch.C_nNDate(valueDate,toDate,tNDate,tau,p_nN,y_nN);
           g_nN = ch.G_nNDate(valueDate,toDate,tNDate,tau,p_nN);
           h_nTerm = ch.H_nTermDate(valueDate,toDate,numMatDate);
           varCovar = ch.LocalVariance(0,to);
           
           sizeN = ch.numOfFactor;
            
           stateSize = size(modelStatesT,2);
           cms_PCA_SimpleOut = zeros(1,stateSize);
           
           driftTerm = 0;
           for i=1:sizeN
               for j=1:sizeN
                   driftTerm = driftTerm + (c_nN(i) * h_nTerm(j) + c_nN(i) * g_nN(j))*varCovar(i,j);
               end
           end
           
           for i=1:stateSize
               cms_PCA_SimpleOut(i) = y_nN + driftTerm;
               for j=1:sizeN
                cms_PCA_SimpleOut(i) = cms_PCA_SimpleOut(i) + c_nN(j)*modelStatesT(j,i);
               end
           end
           
           out = cms_PCA_SimpleOut;
           
            
        end
        
        function out = CheyetteGeneralVariance(ch,from,to,T1,T2)
            
            sizeN = ch.numOfFactor;
            plambda =cell(sizeN,1);
            for i=1:sizeN
                plambda{i,1}= ch.lambda{i,1};
            end
            
            sigma_f = zeros(sizeN,sizeN);
            
            % eta is short rate volatlity
            % eta2 is short rate covariance matrix

            for i=1:sizeN
                for j=1:sizeN
                    sigma_f(i,j)=sqrt(plambda{i,1}.quote(1)*plambda{i,1}.quote(1))*ch.rho(i,j);
                end
            end
            
           eta = zeros(sizeN,sizeN);
           eta = ch.inv_H_tau*sigma_f;
           eta2 = eta*eta';
 
           sigmaTimes = plambda{1,1}.tenor;
           sigmaSize = length(plambda{1,1}.tenor);
            
            for i=1:sigmaSize
                if from<= sigmaTimes(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:sigmaSize
                if to<= sigmaTimes(i)
                    break;
                end
            end
            endIdx = i;
            
            lastExpScale = zeros(4,sizeN,sizeN);
            newExpScale  = zeros(4,sizeN,sizeN);
            variances = zeros(4,sizeN,sizeN);
            meanR = zeros(sizeN,1);
            
            for i=1:sizeN
                meanR(i) = ch.kappa(i);
            end
%             newExpScale = zeros(4,1);
%             lastExpScale = zeros(4,1);
%             variances = zeros(4,1);
            
            lastU  = from;
            
            for j=1:sizeN
                for k=1:sizeN
                    lastExpScale(2,j,k) = exp(meanR(j)*lastU);
                    lastExpScale(3,j,k) = exp(meanR(k)*lastU);
                    lastExpScale(4,j,k) = exp((meanR(j)+meanR(k))*lastU);
                end
            end
            
            for i=startIdx:endIdx
                if (i< endIdx) U = sigmaTimes(i);else U = to;end;
                if (i< sigmaSize) tindex = i;else tindex = sigmaSize;end;
            % eta is short rate volatlity
                for j=1:sizeN
                    for k=1:sizeN
                        sigma_f(j,k)=sqrt(plambda{j,1}.quote(tindex)*plambda{j,1}.quote(tindex))*ch.rho(j,k);
                    end
                end
                
                eta = ch.inv_H_tau*sigma_f;
                eta2 = eta*eta';
                
                for j=1:sizeN
                    for k=1:sizeN
                        newExpScale(2,j,k) = exp(meanR(j)*U);
                        newExpScale(3,j,k) = exp(meanR(k)*U);
                        newExpScale(4,j,k) = exp((meanR(j)+meanR(k))*U);
                    
                        variances(1,j,k) = variances(1,j,k) + eta2(j,k) * (U - lastU);
                        variances(2,j,k) = variances(2,j,k) + eta2(j,k) * (1.0/ meanR(j))*(newExpScale(2,j,k) - lastExpScale(2,j,k));
                        variances(3,j,k) = variances(3,j,k) + eta2(j,k) * (1.0/ meanR(k))*(newExpScale(3,j,k) - lastExpScale(3,j,k));
                        variances(4,j,k) = variances(4,j,k) + eta2(j,k) * (1.0/ (meanR(j)+meanR(k)))*(newExpScale(4,j,k) - lastExpScale(4,j,k));
                    end
                end
                
                lastU = U;
                for j=1:sizeN
                    for k=1:sizeN
                        lastExpScale(2,j,k) = newExpScale(2,j,k);
                        lastExpScale(3,j,k) = newExpScale(3,j,k);
                        lastExpScale(4,j,k) = newExpScale(4,j,k);
                    end
                end
                
            end
            
            value = 0;
            
            for j=1:sizeN
                for k=1:sizeN
                    value = value + (1.0/ meanR(j))*(1.0/meanR(k))*variances(1,j,k);
                end
            end
            
            for j=1:sizeN
                for k=1:sizeN
                    value = value + -1.0*exp(-1.0*meanR(j)*T1)*(1.0/ meanR(j))*(1.0/meanR(k))*variances(2,j,k);
                end
            end
            
            for j=1:sizeN
                for k=1:sizeN
                    value = value + -1.0*exp(-1.0*meanR(k)*T2)*(1.0/ meanR(j))*(1.0/meanR(k))*variances(3,j,k);
                end
            end
            
            for j=1:sizeN
                for k=1:sizeN
                    value = value +  1.0*exp(-1.0*meanR(j)*T1)*exp(-1.0*meanR(k)*T2)*(1.0/ meanR(j))*(1.0/meanR(k))*variances(4,j,k);
                end
            end
            
            out  = value;
        end
        
        function out = fwdZcVariance(ch, from, to, T1, T2)
            
            localVariance = zeros(2,1);
            
            localVariance(1) = ch.CheyetteGeneralVariance(from,to,T1,T1);
            localVariance(2) = ch.CheyetteGeneralVariance(from,to,to,to);
            
            
            value = localVariance(1) - localVariance(2);
            
            measureChange = zeros(2,1);
            
            measureChange(1) = ch.CheyetteGeneralVariance(from,to,T1,T2);
            measureChange(2) = ch.CheyetteGeneralVariance(from,to,to,T2);
            
            measureChangeTerm = measureChange(1) - measureChange(2);
            
            value = value + -2.0*measureChangeTerm;
            
            out = value;
            
        end
        
        function out = discountFactorRisky(ch, startTime, endTime,numMatTime,modelStatesT)
           zcT = ch.DFRisky(endTime/365.0);
           zct = ch.DFRisky(startTime/365.0);
           beta = ch.Beta(startTime/365.0,endTime/365.0);
           zcDriftTerm = ch.fwdZcVariance(0,startTime/365.0,endTime/365.0,numMatTime/365.0);
           %numOfPath
           stateSize = size(modelStatesT,2);
           discountFactorOut = zeros(1,stateSize);
           
           sizeN = ch.numOfFactor;
           
           for i=1:stateSize
               discountFactorOut(i) = zcT/zct * exp(-0.5*zcDriftTerm);
               for j=1:sizeN
                    discountFactorOut(i) = discountFactorOut(i) * exp(-beta(j)*modelStatesT(j,i));
               end
           end
           
           out = discountFactorOut;
        end
        
        function out = discountFactor(ch, startTime, endTime,numMatTime,modelStatesT)
           zcT = ch.DF(endTime/365.0);
           zct = ch.DF(startTime/365.0);
           beta = ch.Beta(startTime/365.0,endTime/365.0);
           zcDriftTerm = ch.fwdZcVariance(0,startTime/365.0,endTime/365.0,numMatTime/365.0);
           %numOfPath
           stateSize = size(modelStatesT,2);
           discountFactorOut = zeros(1,stateSize);
           
           sizeN = ch.numOfFactor;
           
           for i=1:stateSize
               discountFactorOut(i) = zcT/zct * exp(-0.5*zcDriftTerm);
               for j=1:sizeN
                    discountFactorOut(i) = discountFactorOut(i) * exp(-beta(j)*modelStatesT(j,i));
               end
           end
           
           out = discountFactorOut;
        end
        
        function out = discountPayoffRisky(ch,eventTime,numMatTime ...
                ,modelStatesT,cashflow)
            
            dfNumMat = ch.DFRisky(numMatTime/365.0);
            % for discountPayoff T=numMatTime
            % P(0,TnumMat)/P(Te,Tnummat)*Payoff
            %
            discountT = ch.discountFactorRisky(eventTime, numMatTime,numMatTime,modelStatesT);
            stateSize = size(modelStatesT,2);
            payoff = zeros(1,stateSize);
            for i=1:stateSize
                payoff(i)= dfNumMat/discountT(i)*cashflow(i);
            end    
            out = payoff;
        end
        
        function out = discountPayoff(ch,eventTime,numMatTime ...
                ,modelStatesT,cashflow)
            
            dfNumMat = ch.DF(numMatTime/365.0);
            % for discountPayoff T=numMatTime
            % P(0,TnumMat)/P(Te,Tnummat)*Payoff
            %
            discountT = ch.discountFactor(eventTime, numMatTime,numMatTime,modelStatesT);
            stateSize = size(modelStatesT,2);
            payoff = zeros(1,stateSize);
            for i=1:stateSize
                payoff(i)= dfNumMat/discountT(i)*cashflow(i);
            end    
            out = payoff;
        end
        
        function out = stateLocalDrift(ch,fromTime, toTime)
            sizeN = ch.numOfFactor;
            drift = zeros(sizeN,sizeN);
            for i=1:sizeN
                drift(i,i) =  exp(-1.0*ch.kappa(i)*(toTime-fromTime)/365.0);
            end
            out = drift;
        end
        
        function out = stateLocalVariance(ch,fromTime, toTime)
            out = ch.LocalVariance(fromTime/365.0,toTime/365.0);
        end
        
        function variances = LocalVariance(ch,from,to)
            sizeN = ch.numOfFactor;
            plambda =cell(sizeN,1);
            for i=1:sizeN
                plambda{i,1}= ch.lambda{i,1};
            end
            sigma_f =[];
            sigma_f = zeros(sizeN,sizeN);
%% converting forward rate volatlity into short rate volatlitity
%% eta is short rate volatlity
            for i=1:sizeN
                for j=1:sizeN
%                     sigma_f(i,j)=plambda{i,1}.quote(1)*ch.Fwd(0,ch.tau(i))*ch.rho(i,j);
%                     sigma_f(i,j)=plambda{i,1}.quote(1)*ch.rho(i,j);
                    sigma_f(i,j)=sqrt(plambda{i,1}.quote(1)*plambda{i,1}.quote(1))*ch.rho(i,j);
                end
            end
            
           eta =[];
           eta = zeros(sizeN,sizeN);
           eta = ch.inv_H_tau*sigma_f;
           eta2 = eta*eta';
           %%
           alphaSize = length(plambda{1,1}.tenor);
            
           for i=1:alphaSize
               if from<= plambda{1,1}.tenor(i);
                   break;
               end
           end
            
           startIdx = i;
            
%             endIdx = size(sig(1).tenor,1); 
           for i=1:alphaSize
               if to<= plambda{1,1}.tenor(i);
                   break;
               end
           end
           endIdx = i;
            
           lastU  = from;
           
           tiny = zeros(sizeN,sizeN);
           lastExpScale = zeros(sizeN,sizeN);
           newExpScale  = zeros(sizeN,sizeN);
           scale = 0.0;
           meanR = zeros(sizeN,1);
           for i=1:sizeN
                meanR(i) = ch.kappa(i);
           end
           for i=1:sizeN
                for j=1:sizeN
                    if(abs(meanR(i)+meanR(j)) < 1.0E-12) tiny(i,j) = 1; else tiny(i,j) = 0; end;
                    lastExpScale(i,j) =  exp((meanR(i)+meanR(j))*from);
                end
           end
            
           variances = zeros(sizeN,sizeN);
           tindex = 1;
           for i=startIdx:endIdx
                if (i< endIdx) U = plambda{1,1}.tenor(i);else U = to;end;
                if (i< alphaSize) tindex = i;else tindex = alphaSize;end;
            %% converting forward rate volatlity into short rate volatlitity
            %% eta is short rate volatlity
                for j=1:sizeN
                    for k=1:sizeN
%                         sigma_f(j,k)=plambda{j,1}.quote(tindex)*ch.Fwd(0,ch.tau(j))*ch.rho(j,k);
%                         sigma_f(j,k)=plambda{j,1}.quote(tindex)*ch.rho(j,k);
                        sigma_f(j,k)=sqrt(plambda{j,1}.quote(tindex)*plambda{j,1}.quote(tindex))*ch.rho(j,k);
                    end
                end
                
                eta = ch.inv_H_tau*sigma_f;
                eta2 = eta*eta';
            %%
                for j=1:sizeN
                    for k=1:sizeN
                        newExpScale(j,k) =  exp((meanR(j)+meanR(k))*U);
                        if(tiny(j,k)~=1)
                            scale = newExpScale(j,k)-lastExpScale(j,k);
                        else
                            scale = U - lastU;
                        end
                        variances(j,k) = variances(j,k) + eta2(j,k) * scale;
                        lastExpScale(j,k) = newExpScale(j,k);
                    end
                end
                lastU = U;
           end
           
           for j=1:sizeN
                for k=1:sizeN
                    if(tiny(j,k)~= 1)
                        variances(j,k) = variances(j,k)/(meanR(j)+meanR(k))*exp(-1.0*(meanR(j)+meanR(k))*to);
                    end
                end
           end
            
            
        end

        function beta = Beta(ch,t,T)
            beta = [];
            sizeN = ch.numOfFactor;
            beta = zeros(sizeN,1);
            
            for i=1:sizeN
                if(abs(ch.kappa(i)) > 1.0E-12)
                    beta(i) = (1.0-exp(-ch.kappa(i)*(T-t)))/ch.kappa(i);
                else
                    beta(i) = (T-t);
                end
            end
        end
        
        function beta = BetaDate(ch,valueDate,tDate,TDate)
            t = tDate.DateDiff(valueDate)/365.0;
            T = TDate.DateDiff(valueDate)/365.0;
            
            sizeN = ch.numOfFactor;
            beta = zeros(sizeN,1);
            
            for i=1:sizeN
                if(abs(ch.kappa(i)) > 1.0E-12)
                    beta(i) = (1.0-exp(-ch.kappa(i)*(T-t)))/ch.kappa(i);
                else
                    beta(i) = (T-t);
                end
            end
            
        end
        
        function c_nN = C_nN(ch,tn,tN,tau,pnN,ynN)
            p = @ch.DF;
            c_nN = [];
            sizeN = ch.numOfFactor;
            c_nN = zeros(sizeN,1);
            
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = ch.Beta(tn,tp);
                df = p(tp);
                for j=1:sizeN
                    c_nN(j) = c_nN(j) +tau * beta(j) * df;
                end
            end
            
            for j=1:sizeN
                c_nN(j) = c_nN(j)* ynN;
            end
            
            for j=1:sizeN
                c_nN(j) = c_nN(j) + beta(j)*df;
            end
            
            for j=1:sizeN
                c_nN(j) = c_nN(j)/pnN;
            end
           
        end
        
        function c_nN = C_nNDate(ch,valueDate,tnDate,tNDate,tau,pnN,ynN)
            tn = tnDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            
            addMonthUnit = round(12*tau);
            cnt = round((tN-tn)/tau);
            
            p = @ch.DF;
            sizeN = ch.numOfFactor;
            c_nN = zeros(sizeN,1);
            
            for i=1:cnt
                tpDate = tnDate.AddDate(i*addMonthUnit,'month');
                tp = tpDate.DateDiff(valueDate)/365.0;
                beta = ch.BetaDate(valueDate,tnDate,tpDate);
                df = p(tp);
                for j=1:sizeN
                    c_nN(j) = c_nN(j) +tau * beta(j) * df;
                end
            end
            
            for j=1:sizeN
                c_nN(j) = c_nN(j)* ynN;
            end
            
            for j=1:sizeN
                c_nN(j) = c_nN(j) + beta(j)*df;
            end
            
            for j=1:sizeN
                c_nN(j) = c_nN(j)/pnN;
            end
        end
        
        function g_nN = G_nN(ch,tn,tN,tau,pnN)
            p = @ch.DF;
            g_nN = [];
            sizeN = ch.numOfFactor;
            g_nN = zeros(sizeN,1);
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = ch.Beta(tn,tp);
                df = p(tp);
                for j=1:sizeN
                    g_nN(j) = g_nN(j) +tau * beta(j) * df;
                end
            end
            
            for j=1:sizeN
                g_nN(j) = g_nN(j)/pnN;
            end
            
        end
        
        function g_nN = G_nNDate(ch,valueDate,tnDate,tNDate,tau,pnN)
            tn = tnDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            addMonthUnit = round(12*tau);
            cnt = round((tN-tn)/tau);
            
            p = @ch.DF;
            sizeN = ch.numOfFactor;
            g_nN = zeros(sizeN,1);
            
            for i=1:cnt
                tpDate = tnDate.AddDate(i*addMonthUnit,'month');
                tp = tpDate.DateDiff(valueDate)/365.0;
                beta = ch.BetaDate(valueDate,tnDate,tpDate);
                df = p(tp);
                for j=1:sizeN
                    g_nN(j) = g_nN(j) +tau * beta(j) * df;
                end
            end
            
            for j=1:sizeN
                g_nN(j) = g_nN(j)/pnN;
            end
        end 
        
        function h_nTerm = H_nTerm(ch,tn,tTerm)
           h_nTerm =[];
           sizeN = ch.numOfFactor;
           h_nTerm = zeros(sizeN,1);
           beta = ch.Beta(tn,tTerm);
           
           for j=1:sizeN
                h_nTerm(j) = -1.0*beta(j);
           end
          
        end
        
        function h_nTerm = H_nTermDate(ch,valueDate,tnDate,tTermDate)
           h_nTerm =[];
           sizeN = ch.numOfFactor;
           h_nTerm = zeros(sizeN,1);
           beta = ch.BetaDate(valueDate,tnDate,tTermDate);
           for j=1:sizeN
                h_nTerm(j) = -1.0*beta(j);
           end
        end
        
        function out = findTindex(ch,t,sigma)
            
            sum = 0.0;
            prev_t = 0.0;
            inRangeYN = false;
            for i=1:length(sigma.tenor)
                alpha = i;
                if t<= sigma.tenor(i)
                    sum = alpha;
                    inRangeYN = true;
                    break;
                end
            end
            
            if ~inRangeYN 
                sum = alpha;
            end
            out = sum;
        end
        
        function instFwdCorrTest = InsFwdCorrTest(ch,tx,tnr1,tnr2)
            sizeN = ch.numOfFactor;
            plambda =cell(sizeN,1);
            for i=1:sizeN
                plambda{i,1}= ch.lambda{i,1};
            end
            sigma_f =[];
            sigma_f = zeros(sizeN,sizeN);
%             labmdaTest.quote = plambda{1,1}.quote;
%             lambdaTest.tenor = plambda{1,1}.tenor;
            tindex = ch.findTindex(tx+1e-6,plambda{1,1});
            
            for j=1:sizeN
                for k=1:sizeN
%                     sigma_f(j,k)=plambda{j,1}.quote(tindex)*ch.Fwd(0,ch.tau(j))*ch.rho(j,k);
                    sigma_f(j,k)=plambda{j,1}.quote(tindex)*ch.rho(j,k);
                end
            end
                
            eta = ch.inv_H_tau*sigma_f;
            eta2 = eta*eta';
            
            KappaTau1 = zeros(2,1);
            KappaTau2 = zeros(2,1);
            
            for i=1:sizeN
                KappaTau1(i) = exp(-ch.kappa(i)*tnr1);
                KappaTau2(i) = exp(-ch.kappa(i)*tnr2);
            end
            
            
            covar = 0.0;
            var1 = 0;
            var2 = 0;
            var1 =  KappaTau1'*eta2*KappaTau1;
            var2 =  KappaTau2'*eta2*KappaTau2;
            covar =  KappaTau1'*eta2*KappaTau2;
            
            instFwdCorrTest = covar/sqrt(var1*var2);
        end
        
        function InstSwapFwdCorrTest = InsSwapFwdCorrTest(ch,tx,tnr1,tnr2)
            sizeN = ch.numOfFactor;
            plambda =cell(sizeN,1);
            for i=1:sizeN
                plambda{i,1}= ch.lambda{i,1};
            end
            sigma_f =[];
            sigma_f = zeros(sizeN,sizeN);
            tindex = ch.findTindex(tx+1e-6,plambda{1,1});
            
            for j=1:sizeN
                for k=1:sizeN
%                     sigma_f(j,k)=plambda{j,1}.quote(tindex)*ch.Fwd(0,ch.tau(j))*ch.rho(j,k);
                    sigma_f(j,k)=plambda{j,1}.quote(tindex)*ch.rho(j,k);
                end
            end
                
            eta = ch.inv_H_tau*sigma_f;
            eta2 = eta*eta';
            
            KappaTau1 = zeros(2,1);
            KappaTau2 = zeros(2,1);
            
            tau = 0.25;
            p_nN1 = ch.PV01(tx,tx+tnr1,tau);
            df_Tn1 = ch.DF(tx);
            df_TN1 = ch.DF(tx + tnr1);
            y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
            c_nN1 = ch.C_nN(tx,tx + tnr1,tau,p_nN1,y_nN1);
            
            p_nN2 = ch.PV01(tx,tx+tnr2,tau);
            df_Tn2 = ch.DF(tx);
            df_TN2 = ch.DF(tx + tnr2);
            y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
            c_nN2 = ch.C_nN(tx,tx + tnr2,tau,p_nN2,y_nN2);
            
            for i=1:sizeN
                KappaTau1(i) = c_nN1(i)*exp(-ch.kappa(i)*tnr1);
                KappaTau2(i) = c_nN2(i)*exp(-ch.kappa(i)*tnr2);
            end
            
            
            covar = 0.0;
            var1 = 0;
            var2 = 0;
            var1 =  KappaTau1'*eta2*KappaTau1;
            var2 =  KappaTau2'*eta2*KappaTau2;
            covar =  KappaTau1'*eta2*KappaTau2;
            
            InstSwapFwdCorrTest = covar/sqrt(var1*var2);
        end
        
        function out = CDDigitalArgDate(ch,valueDate,toDate,tpDate,tenor1,tau)
           % probability that CMS rate > strike
            %CMS 1
           sizeN = ch.numOfFactor;
           toTime =  toDate.DateDiff(valueDate);
           to      = toTime/365.0;
           if tenor1 < 1 && tenor1 >= 1.0/12.0
               tenorMonth = round(12*tenor1);
                tN1Date = toDate.AddDate(tenorMonth,'month');
           elseif tenor1 >= 1
               tN1Date = toDate.AddDate(tenor1,'year');
           else
               disp('unimplemented')
           end
           
           tN1Time = tN1Date.DateDiff(valueDate);
           tN1   =  tN1Time/365.0;
           
           deltaT = (tN1Time - toTime)/365.0;
           
           df_Tn1 = ch.DF(to);
           df_TN1 = ch.DF(tN1);
           
           c_nN1 = ch.BetaDate(valueDate,toDate,tN1Date);
           c_nTerm = ch.BetaDate(valueDate,toDate,tpDate);
           
           varCovar = ch.LocalVariance(0,to);
           
           driftTerm=0.0;
           for i=1:sizeN
               for j=1:sizeN
                    driftTerm = driftTerm + c_nN1(i) * c_nTerm(j) * varCovar(i,j);
               end
           end
           
           variance = 0.0;
           for i=1:sizeN
               for j=1:sizeN
                    variance = variance + c_nN1(i) * c_nN1(j) * varCovar(i,j);
               end
           end
           
%            out.mu = 0.5*variance  -1.0*log(df_TN1/df_Tn1) - driftTerm;
           out.mu = -0.5*variance  +1.0*log(df_TN1/df_Tn1) + driftTerm;
           
           out.sigma = sqrt(variance);
           out.c_nN = c_nN1;
           out.varCovar = varCovar;
           out.deltaT = deltaT;
        end
        
        function out = CMSDigitalArg(ch,to,tp,tenor1,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           sizeN = ch.numOfFactor;
           p_nN1 = ch.PV01(to,to+tenor1,tau);
           df_Tn1 = ch.DF(to);
           df_TN1 = ch.DF(to + tenor1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           c_nN1 = ch.C_nN(to,to + tenor1,tau,p_nN1,y_nN1);
           g_nN1 = ch.G_nN(to,to + tenor1,tau,p_nN1);
           h_nTerm1 = ch.H_nTerm(to,tp);
           
           varCovar = ch.LocalVariance(0,to);
           
           driftTerm1=0.0;
           for i=1:sizeN
               for j=1:sizeN
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           for i=1:sizeN
               for j=1:sizeN
                    spreadVariance = spreadVariance + (c_nN1(i)) * (c_nN1(j)) * varCovar(i,j);
               end
           end
           
           out.mu = (y_nN1 + driftTerm1);
           if spreadVariance >=0
            out.sigma = sqrt(spreadVariance);
           else
            out.sigma = 0.0;
           end
           
           out.c_nN1 = c_nN1;
           out.varCovar = varCovar;
           
        end
        
        function out = CMSDigital(ch,to,tp,tenor1,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsDigitalArg = ch.CMSDigitalArg(to,tp,tenor1,tau);
           mu = cmsDigitalArg.mu;
           sigma = cmsDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSDigitalRange(ch,to,tp,tenor1,tau,lower,upper)
            lower_value = ch.CMSDigital(to,tp,tenor1,tau,lower);
            upper_value = ch.CMSDigital(to,tp,tenor1,tau,upper);
            out = lower_value - upper_value;
        end
        
        
        function out = CMSSpreadDigitalArg(ch,to,tp,tenor1,tenor2,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           sizeN = ch.numOfFactor;
           p_nN1 = ch.PV01(to,to+tenor1,tau);
           df_Tn1 = ch.DF(to);
           df_TN1 = ch.DF(to + tenor1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           c_nN1 = ch.C_nN(to,to + tenor1,tau,p_nN1,y_nN1);
           g_nN1 = ch.G_nN(to,to + tenor1,tau,p_nN1);
           h_nTerm1 = ch.H_nTerm(to,tp);
           
           %CMS2
           p_nN2 = ch.PV01(to,to+tenor2,tau);
           df_Tn2 = ch.DF(to);
           df_TN2 = ch.DF(to + tenor2);
           y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
           c_nN2 = ch.C_nN(to,to + tenor2,tau,p_nN2,y_nN2);
           g_nN2 = ch.G_nN(to,to + tenor2,tau,p_nN2);
           h_nTerm2 = ch.H_nTerm(to,tp);
           
           varCovar = ch.LocalVariance(0,to);
           
           driftTerm1=0.0;
           driftTerm2=0.0;
           for i=1:sizeN
               for j=1:sizeN
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
                    driftTerm2 = driftTerm2 + c_nN2(i) * (h_nTerm2(j) + g_nN2(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           for i=1:sizeN
               for j=1:sizeN
                    spreadVariance = spreadVariance + (c_nN1(i)-c_nN2(i)) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
               end
           end
           
           out.mu = (y_nN1 + driftTerm1) - (y_nN2 + driftTerm2);
           out.sigma = sqrt(spreadVariance);
           out.c_nN1 = c_nN1;
           out.c_nN2 = c_nN2;
           out.varCovar = varCovar;
           
        end
        
        function out = CMSSpreadDigitalArgDate(ch,valueDate,toDate,tpDate,tenor1,tenor2,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           sizeN = ch.numOfFactor;
           
           to      = toDate.DateDiff(valueDate)/365.0;
           tN1Date = toDate.AddDate(tenor1,'year');
           tN1   =  tN1Date.DateDiff(valueDate)/365.0;
           
           p_nN1 = ch.PV01Date(valueDate,toDate,tN1Date,tau);
           
           df_Tn1 = ch.DF(to);
           df_TN1 = ch.DF(tN1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           
           c_nN1 = ch.C_nNDate(valueDate,toDate,tN1Date,tau,p_nN1,y_nN1);
           g_nN1 = ch.G_nNDate(valueDate,toDate,tN1Date,tau,p_nN1);
           h_nTerm1 = ch.H_nTermDate(valueDate,toDate,tpDate);
           
           %CMS2
           tN2Date = toDate.AddDate(tenor2,'year');
           tN2   =  tN2Date.DateDiff(valueDate)/365.0;
           
           p_nN2 = ch.PV01Date(valueDate,toDate,tN2Date,tau);
           
           df_Tn2 = ch.DF(to);
           df_TN2 = ch.DF(tN2);
           y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
           
           c_nN2 = ch.C_nNDate(valueDate,toDate,tN2Date,tau,p_nN2,y_nN2);
           g_nN2 = ch.G_nNDate(valueDate,toDate,tN2Date,tau,p_nN2);
           h_nTerm2 = ch.H_nTermDate(valueDate,toDate,tpDate);
           
           varCovar = ch.LocalVariance(0,to);
           
           driftTerm1=0.0;
           driftTerm2=0.0;
           for i=1:sizeN
               for j=1:sizeN
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
                    driftTerm2 = driftTerm2 + c_nN2(i) * (h_nTerm2(j) + g_nN2(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           for i=1:sizeN
               for j=1:sizeN
                    spreadVariance = spreadVariance + (c_nN1(i)-c_nN2(i)) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
               end
           end
           
           out.mu = (y_nN1 + driftTerm1) - (y_nN2 + driftTerm2);
           out.sigma = sqrt(spreadVariance);
           out.c_nN1 = c_nN1;
           out.c_nN2 = c_nN2;
           out.varCovar = varCovar;
           
        end
        
        function out = CMSSpreadDigital(ch,to,tp,tenor1,tenor2,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsSpreadDigitalArg = ch.CMSSpreadDigitalArg(to,tp,tenor1,tenor2,tau);
           mu = cmsSpreadDigitalArg.mu;
           sigma = cmsSpreadDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSSpreadDigitalDate(ch,valueDate,toDate,tpDate,tenor1,tenor2,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsSpreadDigitalArg = ch.CMSSpreadDigitalArgDate(valueDate,toDate,tpDate,tenor1,tenor2,tau);
           mu = cmsSpreadDigitalArg.mu;
           sigma = cmsSpreadDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CDDigitalDate(ch,valueDate,toDate,tpDate,tenor,tau,strike)
           
           % probability that CD > strikeCD 
           
           % CD
           cdDigitalArg = ch.CDDigitalArgDate(valueDate,toDate,tpDate,tenor,tau);
           mu = cdDigitalArg.mu;
           sigma = cdDigitalArg.sigma;
           c_nN = cdDigitalArg.c_nN;
           varCovar = cdDigitalArg.varCovar;
           deltaT = cdDigitalArg.deltaT;
           
           
           z1 = (mu + log(1.0 + deltaT*strike))/sigma;
           if ~isreal(z1) || ~isfinite(z1)
               out = 0.0;
           else
               out = H_ncdf(z1);
           end
        end
        
        function out = CMSSpreadDigitalRange(ch,to,tp,tenor1,tenor2,tau,lower,upper)
            lower_value = ch.CMSSpreadDigital(to,tp,tenor1,tenor2,tau,lower);
            upper_value = ch.CMSSpreadDigital(to,tp,tenor1,tenor2,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CMSSpreadDigitalRangeDate(ch,valueDate,toDate,tpDate,tenor1,tenor2,tau,lower,upper)
            lower_value = ch.CMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,lower);
            upper_value = ch.CMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CDDigitalRangeDate(ch,valueDate,toDate,tpDate,tenor,tau,lower,upper)
            upper_value = ch.CDDigitalDate(valueDate,toDate,tpDate,tenor,tau,upper);
            lower_value = ch.CDDigitalDate(valueDate,toDate,tpDate,tenor,tau,lower);
            out = upper_value - lower_value;
            
        end
        
        function atmSwaption = ATMSwaption(ch,tx,tnr)
            sizeN = ch.numOfFactor;
            tau = 0.25;
            pv01 = ch.PV01(tx,tx+tnr,tau);
            fsr  = ch.Fsr(tx,tx+tnr,tau);
            c_nN = ch.C_nN(tx,tx+tnr,tau,pv01,fsr);
            variances = ch.LocalVariance(0,tx);
            totalVar = 0.0;
            for i=1:sizeN
                for j=1:sizeN
                    totalVar = totalVar + c_nN(i)*c_nN(j)*variances(i,j);
                end
            end
            if totalVar <=0
                totalVar = 0.0;
            end
            atmSwaption =  pv01 * sqrt(totalVar)/sqrt(2.0*pi);
        end
        
        function atmSwaptionNormalVol = ATMSwaptionNormalVol(ch,tx,tnr)
            sizeN = ch.numOfFactor;
            tau = 0.25;
            pv01 = ch.PV01(tx,tx+tnr,tau);
            fsr  = ch.Fsr(tx,tx+tnr,tau);
            c_nN = ch.C_nN(tx,tx+tnr,tau,pv01,fsr);
            variances = ch.LocalVariance(0,tx);
            totalVar = 0.0;
            for i=1:sizeN
                for j=1:sizeN
                    totalVar = totalVar + c_nN(i)*c_nN(j)*variances(i,j);
                end
            end
            if totalVar <=0
                totalVar = 0.0;
            end
            atmSwaptionNormalVol =  sqrt(totalVar/tx);
        end
        
        function capletOut = Caplet(ch,tx,x,tnr)
        %blackCaplet:a function that computes a blackCaplet price
        %arguments
        %tx =(annualized)time to reset
        %sig= caplet volatility
        %x  = caplet strike
        %tnr=(annualized)time to payment
        %NA =Notional Amount
        
            tau = 0.25;
            p = @ch.DF;
            beta = ch.Beta(tx,tx + tnr);
            sizeN = ch.numOfFactor;
            variances = ch.LocalVariance(0,tx);
            totalVar = 0.0;
            for i=1:sizeN
                for j=1:sizeN
                    totalVar = totalVar + beta(i)*beta(j)*variances(i,j);
                end
            end
            sig = sqrt(totalVar);
            d1= (log(p(tx)/p(tx + tnr)*1.0/(1+tnr*x))+(sig^2/2.0))/(sig);
            d2 = d1 -sig;
            capletOut = p(tx)*H_ncdf(d1) - p(tx+tnr)*(1+tnr*x)*H_ncdf(d2);
        end
        
        function capOut = Cap(ch,tx,x,tnr)
            capOut = sum(arrayfun(@(i) ch.Caplet(tnr*i,x,tnr),1:round(tx/tnr)-1));
        end
        
         function atmSwaptionImpVol = ATMSwaptionImpVol(ch,tx,tnr)
            atmSwaption = ch.ATMSwaption(tx,tnr);
            
            a = 0.001;
            b = 10;
            Tol = 1e-10;
            MaxIter = 1000;
            tau = 0.25;
            fsr  = ch.Fsr(tx,tx+tnr,tau);
            pv01 = ch.PV01(tx,tx+tnr,tau);
            impVol = BisecBSIV('P',fsr,fsr, ...
                                          0.0,tx,a,b,atmSwaption/pv01,Tol,MaxIter);
            atmSwaptionImpVol =  impVol;
        end
        
        function impCorrel = ImpCorrel(ch,tx,tnr1,tnr2)
            tau = 0.25;
            pv01_1 = ch.PV01(tx,tx+tnr1,tau);
            fsr_1  = ch.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = ch.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = ch.PV01(tx,tx+tnr2,tau);
            fsr_2  = ch.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = ch.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            variances = ch.LocalVariance(0,tx);
            var1 = 0;
            var2 = 0;
            coVar = 0;
            sizeN = ch.numOfFactor;
            for j=1:sizeN
                for k=1:sizeN
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                    var2 = var2 + c_nN_2(j)*c_nN_2(k)*variances(j,k);
                    coVar = coVar + c_nN_1(j)*c_nN_2(k)*variances(j,k);
                end
            end
            impCorrel = coVar/sqrt(var1)/sqrt(var2);
        end
        
        function impVariance = ImpVariance(ch,tx,tnr1)
            tau = 0.25;
            pv01_1 = ch.PV01(tx,tx+tnr1,tau);
            fsr_1  = ch.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = ch.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            variances = ch.LocalVariance(0,tx);
            var1 = 0;
            sizeN = ch.numOfFactor;
            for j=1:sizeN
                for k=1:sizeN
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                end
            end
            impVariance = var1;
        end
        
        function out = RebonatoFwdCorr(ch,tx,tnr1,tnr2)
           rho_inf = 0.35;
           rho_decay = 0.15;
           out =  rho_inf + (1-rho_inf)*exp(-rho_decay*abs(tnr1-tnr2));
        end
        
        function out = TargetFunctionATMSwaptionAndCapBootStrapLevMar(ch,tvar,idxNow,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            chSwaptionPrice = zeros(length(tenor),1);
            
            capExpiry = params.capExpiry;
            blackCapPrice = params.blackCapPrice;
            blackCapStrike = params.blackCapStrike;
            
            useCap = false;
            if expiry(idxNow) <= capExpiry(length(capExpiry))
                if capExpiry(idxNow) == expiry(idxNow)
                    useCap = true;
                end
            end
            sizeN = ch.numOfFactor;
            for j=1:sizeN
                ch.lambda{j,1}.quote(idxNow) = tvar(j);
            end
            
            out= zeros(length(tenor)+ 1,1);
            for k=1:length(tenor)
                chSwaptionPrice(k) = ch.ATMSwaption(expiry(idxNow),tenor(k));
                out(k) = chSwaptionPrice(k)/blackSwaptionPrice(idxNow,k)-1.0;
            end
            
            if useCap
                chCapPrice = ch.Cap(capExpiry(idxNow),blackCapStrike(idxNow),0.25);
                out(length(tenor)+1) = chCapPrice/blackCapPrice(idxNow)-1.0;
            end

        end
        
        function out = TargetFunctionATMSwaptionAndCapGlobalLevMar(ch,tvar,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            chSwaptionPrice = zeros(length(expiry),length(tenor));
            
            capExpiry = params.capExpiry;
            blackCapPrice = params.blackCapPrice;
            blackCapStrike = params.blackCapStrike;
            chCapPrice = zeros(length(capExpiry),1);
            
            sizeN = ch.numOfFactor;
            alphaSize = length(ch.lambda{1,1}.tenor);
            for i=1:sizeN
                for j=1:alphaSize
                    ch.lambda{i,1}.quote(j) = tvar((i-1)*alphaSize +j);
                end
            end
            
            sizeOut = length(expiry) * length(tenor);
            sizeOut = sizeOut + length(capExpiry);
            
            out= zeros(sizeOut,1);
            for j=1:length(expiry)
                for k=1:length(tenor)
                    chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                    out((j-1)*length(tenor) + k) = chSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0;
                end
            end
            
            for j=1:length(capExpiry)
                chCapPrice(j) = ch.Cap(capExpiry(j),blackCapStrike(j),0.25);
                out(length(expiry)*length(tenor) + j) = chCapPrice(j)/blackCapPrice(j) - 1.0;
            end
        end 
        
        function out = TargetFunctionATMSwaptionBootStrapLevMar(ch,tvar,idxNow,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            chSwaptionPrice = zeros(length(tenor),1);
            
            sizeN = ch.numOfFactor;
            for j=1:sizeN
                ch.lambda{j,1}.quote(idxNow) = tvar(j);
            end
            
            out= zeros(sizeN,1);
            for k=1:length(tenor)
                chSwaptionPrice(k) = ch.ATMSwaption(expiry(idxNow),tenor(k));
                out(k) = chSwaptionPrice(k)/blackSwaptionPrice(idxNow,k)-1.0;
            end

        end
        
        function out = TargetFunctionATMSwaptionBootStrapLevMarNormalVol(ch,tvar,idxNow,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionNormalVol = params.blackSwaptionNormalVol;
            chATMSwaptionNormalVol = zeros(1,length(tenor));
            
            sizeN = ch.numOfFactor;
            for j=1:sizeN
                ch.lambda{j,1}.quote(idxNow) = tvar(j);
            end
            
            out= zeros(sizeN,1);
            for k=1:length(tenor)
                chATMSwaptionNormalVol(k) = ch.ATMSwaptionNormalVol(expiry(idxNow),tenor(k));
                out(k) = chATMSwaptionNormalVol(k)/blackSwaptionNormalVol(idxNow,k)-1.0;
            end

        end
        
        function out = TargetFunctionATMSwaptionBootStrapLevMarNormalVariance(ch,tvar,idxNow,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionNormalVol = params.blackSwaptionNormalVol;
            chATMSwaptionNormalVol = zeros(1,length(tenor));
            
            sizeN = ch.numOfFactor;
            for j=1:sizeN
                ch.lambda{j,1}.quote(idxNow) = tvar(j);
            end
            
            out= zeros(sizeN,1);
            for k=1:length(tenor)
                chATMSwaptionNormalVol(k) = ch.ATMSwaptionNormalVol(expiry(idxNow),tenor(k));
                out(k) = (chATMSwaptionNormalVol(k)*chATMSwaptionNormalVol(k)*expiry(idxNow))/(blackSwaptionNormalVol(idxNow,k)*blackSwaptionNormalVol(idxNow,k)*expiry(idxNow))-1.0;
            end

        end
        
        function out = TargetFunctionATMSwaptionBootStrapLevMarAbs(ch,tvar,idxNow,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            chSwaptionPrice = zeros(length(tenor),1);
            
            sizeN = ch.numOfFactor;
            for j=1:sizeN
                ch.lambda{j,1}.quote(idxNow) = abs(tvar(j));
            end
            
            out= zeros(sizeN,1);
            for k=1:length(tenor)
                chSwaptionPrice(k) = ch.ATMSwaption(expiry(idxNow),tenor(k));
                out(k) = chSwaptionPrice(k)/blackSwaptionPrice(idxNow,k)-1.0;
            end

        end
        
        function out = TargetFunctionATMSwaptionGlobalLevMar(ch,tvar,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            chSwaptionPrice = zeros(length(expiry),length(tenor));
            
            sizeN = ch.numOfFactor;
            alphaSize = length(ch.lambda{1,1}.tenor);
            for i=1:sizeN
                for j=1:alphaSize
                    ch.lambda{i,1}.quote(j) = tvar((i-1)*alphaSize +j);
                end
            end
            
            sizeOut = length(expiry) * length(tenor);
            out= zeros(sizeOut,1);
            for j=1:length(expiry)
                for k=1:length(tenor)
                    chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                    out((j-1)*length(tenor) + k) = chSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0;
                end
            end
        end 
        
        function out = TargetFunctionATMSwaptionGlobalLevMarKappa(ch,tvar,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            chSwaptionPrice = zeros(length(expiry),length(tenor));
            
            sizeN = ch.numOfFactor;
            alphaSize = length(ch.lambda{1,1}.tenor);
            for i=1:sizeN
                for j=1:alphaSize
                    ch.lambda{i,1}.quote(j) = tvar((i-1)*alphaSize +j);
                end
            end
            
            for i=1:sizeN
                ch.kappa(i) = tvar(sizeN*alphaSize + i);
            end
            
            sizeOut = length(expiry) * length(tenor);
            out= zeros(sizeOut,1);
            for j=1:length(expiry)
                for k=1:length(tenor)
                    chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                    out((j-1)*length(tenor) + k) = chSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0;
                end
            end
        end 

        function out = TargetFunctionATMSwaptionGlobal(ch,tvar,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            chSwaptionPrice = zeros(1,size(tenor,2));
            
            sizeN = ch.numOfFactor;
            alphaSize = length(ch.lambda{1,1}.tenor);
            for i=1:sizeN
                for j=1:alphaSize
                    ch.lambda{i,1}.quote(j) = tvar((i-1)*alphaSize +j);
                end
            end
            
            out= 0;
            for j=1:length(expiry)
                for k=1:length(tenor)
                    chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                    out = out + (chSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0)*(chSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0)*10000;
                end
            end
        end   
        
        function out = TargetFunctionATMSwaptionGlobalImpVol(ch,tvar,params)
            a = 0.001;
            b = 10;
            Tol = 1e-10;
            MaxIter = 1000;

            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionImpVol = params.blackSwaptionImpVol;
            chSwaptionPrice = zeros(1,size(tenor,2));
            
            sizeN = ch.numOfFactor;
            alphaSize = length(ch.lambda{1,1}.tenor);
            for i=1:sizeN
                for j=1:alphaSize
                    ch.lambda{i,1}.quote(j) = tvar((i-1)*alphaSize +j);
                end
            end
            
            out= 0;
            for j=1:length(expiry)
                for k=1:length(tenor)
                    chATMSwaptionImpVol(j,k) = ch.ATMSwaptionImpVol(expiry(j),tenor(k));
                    out = out + (chATMSwaptionImpVol(j,k)- blackSwaptionImpVol(j,k))*(chATMSwaptionImpVol(j,k)- blackSwaptionImpVol(j,k))*10000;
                end
            end
        end   
        
        function out = TargetFunctionATMSwaptionGlobalNormalVol(ch,tvar,params)
            
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionNormalVol = params.blackSwaptionNormalVol;
            chATMSwaptionNormalVol = zeros(length(expiry),length(tenor));
            
            sizeN = ch.numOfFactor;
            alphaSize = length(ch.lambda{1,1}.tenor);
            for i=1:sizeN
                for j=1:alphaSize
                    ch.lambda{i,1}.quote(j) = tvar((i-1)*alphaSize +j);
                end
            end
            
            out= 0;
            for j=1:length(expiry)
                for k=1:length(tenor)
                    chATMSwaptionNormalVol(j,k) = ch.ATMSwaptionNormalVol(expiry(j),tenor(k));
                    out = out + (chATMSwaptionNormalVol(j,k)/blackSwaptionNormalVol(j,k)-1.0)*(chATMSwaptionNormalVol(j,k)/blackSwaptionNormalVol(j,k)-1.0)*10000;
                end
            end
        end   
        
        function out = CalibrateToATMSwaption(ch,black,calibrationFlag)
            tic;
            
            sizeN = ch.numOfFactor;
            alphaSize = length(ch.lambda{1,1}.tenor);
            for i=1:sizeN
               for j=1:alphaSize
                   out.lambda{i,1} = ch.lambda{i,1};
               end
            end
            
          % Calculate Black ATM Swaption Prices
          
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
            
            expirySize = size(expiry,2);
            tenorSize = size(tenor,2);
            
            blackSwaptionPrice = zeros(expirySize,tenorSize);
            blackSwaptionImpVol = zeros(expirySize,tenorSize);
%             blackSwaptionNormalVol = zeros(expirySize,tenorSize);
            
            for k=1:expirySize
                for j=1:tenorSize
                    blackSwaptionPrice(k,j)  = black.BlackATMSwaption(expiry(k),tenor(j));
                    blackSwaptionImpVol(k,j) = black.swaptionVol(expiry(k),tenor(j));
%                     blackSwaptionNormalVol(k,j) = black.ATMSwaptionNormalVol(expiry(k),tenor(j));
                end
            end
            
            params.expiry = expiry;
            params.tenor = tenor;
            params.blackSwaptionPrice = blackSwaptionPrice;
            params.blackSwaptionImpVol = blackSwaptionImpVol;
%             params.blackSwaptionNormalVol = blackSwaptionNormalVol;
            params.expirySize = expirySize;
            params.tenorSize =  tenorSize;
            
            params.swaption_calibration_yn = ones(expirySize,tenorSize);
            
            out.blackSwaptionPrice = blackSwaptionPrice;
            out.chSwaptionPrice = zeros(expirySize,tenorSize);
            out.relDiff = zeros(expirySize,tenorSize);
            out.rmseTotal=0.0;
            out.impVol2Total = 0.0;
            out.normalVol2Total = 0.0;
            out.normalVarianceTotal = 0.0;
            
            calibrationType = 'bootstrap';
            if nargin < 3
                calibrationType = 'bootstrap'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;
            tvar = [];
            
            switch calibrationType
                case 'bootstrapLevMar'
                    tvar = zeros(sizeN,1);
                    for i=1:expirySize
                        idxNow = i;
                        for j=1:sizeN
                             tvar(j) = ch.lambda{j,1}.quote(i);
                             lb(j) = 0.0001;
                             ub(j) = 10.0;
                        end
                        
                        targetfun = @(tvar) TargetFunctionATMSwaptionBootStrapLevMar(ch,tvar,idxNow,params);
                        options=optimset('Algorithm','levenberg-marquardt');
%                         options=optimset('Algorithm','trust-region-reflective');
                        [x, fval] = lsqnonlin(targetfun,tvar,lb,ub,options);
                        
                        for j=1:sizeN
                             out.lambda{j,1}.quote(i) = x(j); 
                             ch.lambda{j,1}.quote(i) = x(j); 
                        end
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.calibrationTime = toc;
                case 'bootstrapLevMarNormalVol'
                    tvar = zeros(sizeN,1);
                    for i=1:expirySize
                        idxNow = i;
                        for j=1:sizeN
                             tvar(j) = ch.lambda{j,1}.quote(i);
                             lb(j) = 0.0001;
                             ub(j) = 10.0;
                        end
                        
                        targetfun = @(tvar) TargetFunctionATMSwaptionBootStrapLevMarNormalVol(ch,tvar,idxNow,params);
                        options=optimset('Algorithm','levenberg-marquardt');
%                         options=optimset('Algorithm','trust-region-reflective');
                        [x, fval] = lsqnonlin(targetfun,tvar,lb,ub,options);
                        
                        for j=1:sizeN
                             out.lambda{j,1}.quote(i) = x(j); 
                             ch.lambda{j,1}.quote(i) = x(j); 
                        end
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.blackSwaptionImpVol(j,k) = blackSwaptionImpVol(j,k);
                            out.blackSwaptionNormalVol(j,k) = blackSwaptionNormalVol(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.chSwaptionImpVol(j,k) = ch.ATMSwaptionImpVol(expiry(j),tenor(k));
                            out.chSwaptionNormalVol(j,k) = ch.ATMSwaptionNormalVol(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.ImpVolDiff(j,k) = out.chSwaptionImpVol(j,k) - out.blackSwaptionImpVol(j,k);
                            out.NormalVolDiff(j,k) = out.chSwaptionNormalVol(j,k)/out.blackSwaptionNormalVol(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            out.impVol2Total = out.impVol2Total + out.ImpVolDiff(j,k)*out.ImpVolDiff(j,k);
                            out.normalVol2Total = out.normalVol2Total + out.NormalVolDiff(j,k)*out.NormalVolDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.impVol2Total = sqrt(out.impVol2Total/N);
                    out.normalVol2Total = sqrt(out.normalVol2Total/N);
                    out.calibrationTime = toc;
                    
                case 'bootstrapLevMarNormalVariance'
                    tvar = zeros(sizeN,1);
                    for i=1:expirySize
                        idxNow = i;
                        for j=1:sizeN
                             tvar(j) = ch.lambda{j,1}.quote(i);
                             lb(j) = 0.0001;
                             ub(j) = 10.0;
                        end
                        
                        targetfun = @(tvar) TargetFunctionATMSwaptionBootStrapLevMarNormalVariance(ch,tvar,idxNow,params);
                        options=optimset('Algorithm','levenberg-marquardt');
%                         options=optimset('Algorithm','trust-region-reflective');
                        [x, fval] = lsqnonlin(targetfun,tvar,lb,ub,options);
                        
                        for j=1:sizeN
                             out.lambda{j,1}.quote(i) = x(j); 
                             ch.lambda{j,1}.quote(i) = x(j); 
                        end
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.blackSwaptionImpVol(j,k) = blackSwaptionImpVol(j,k);
                            out.blackSwaptionNormalVol(j,k) = blackSwaptionNormalVol(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.chSwaptionImpVol(j,k) = ch.ATMSwaptionImpVol(expiry(j),tenor(k));
                            out.chSwaptionNormalVol(j,k) = ch.ATMSwaptionNormalVol(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.ImpVolDiff(j,k) = out.chSwaptionImpVol(j,k) - out.blackSwaptionImpVol(j,k);
                            out.NormalVolDiff(j,k) = out.chSwaptionNormalVol(j,k)/out.blackSwaptionNormalVol(j,k)-1;
                            out.NormalVarianceDiff(j,k) = (out.chSwaptionNormalVol(j,k)*out.chSwaptionNormalVol(j,k)*expiry(j))/(out.blackSwaptionNormalVol(j,k)*out.blackSwaptionNormalVol(j,k)*expiry(j))-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            out.impVol2Total = out.impVol2Total + out.ImpVolDiff(j,k)*out.ImpVolDiff(j,k);
                            out.normalVol2Total = out.normalVol2Total + out.NormalVolDiff(j,k)*out.NormalVolDiff(j,k);
                            out.normalVarianceTotal = out.normalVarianceTotal + out.NormalVarianceDiff(j,k)*out.NormalVarianceDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.impVol2Total = sqrt(out.impVol2Total/N);
                    out.normalVol2Total = sqrt(out.normalVol2Total/N);
                    out.normalVarianceTotal = sqrt(out.normalVarianceTotal/N);
                    out.calibrationTime = toc;
                    
                case 'globalLevMar'
                    for i=1:sizeN
                        for j=1:alphaSize
                             tvar((i-1)*alphaSize +j) = ch.lambda{i,1}.quote(j);
                             lb((i-1)*alphaSize +j) = 0.0001;
                             ub((i-1)*alphaSize +j) = 10.0;
                        end
                    end
                    
                       
                    targetfun = @(tvar) TargetFunctionATMSwaptionGlobalLevMar(ch,tvar,params);
%                     options=optimset('Algorithm','levenberg-marquardt');
                    options=optimset('Algorithm','levenberg-marquardt','MaxFunEvals',700);
                    [x, fval] = lsqnonlin(targetfun,tvar,lb,ub,options);
                    
                    for i=1:sizeN
                        for j=1:alphaSize
                             out.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                             ch.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                        end
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.calibrationTime = toc;
                
                 case 'dummy'
                    
                    for i=1:sizeN
                        for j=1:alphaSize
                             out.lambda{i,1}.quote(j) = ch.lambda{i,1}.quote(j); 
                        end
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.calibrationTime = toc;
                    
                    
                case 'globalLevMarKappa'
                    for i=1:sizeN
                        for j=1:alphaSize
                             tvar((i-1)*alphaSize +j) = ch.lambda{i,1}.quote(j);
                             lb((i-1)*alphaSize +j) = 0.0001;
                             ub((i-1)*alphaSize +j) = 10.0;
                        end
                    end
                    
                    for i=1:sizeN
                        tvar(sizeN*alphaSize + i) = ch.kappa(i);
                        lb(sizeN*alphaSize + i) = 0.0001;
                        ub(sizeN*alphaSize + i) = 10.0;
                    end
                       
                    targetfun = @(tvar) TargetFunctionATMSwaptionGlobalLevMarKappa(ch,tvar,params);
                    options=optimset('Algorithm','levenberg-marquardt');
                    [x, fval] = lsqnonlin(targetfun,tvar,lb,ub,options);
                    
                    for i=1:sizeN
                        for j=1:alphaSize
                             out.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                             ch.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                        end
                    end
                    
                    for i=1:sizeN
                        out.kappa(i) = x(sizeN*alphaSize+i);
                        ch.kappa(i)  = x(sizeN*alphaSize+i);
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);    
                    
                case 'global'
                    optParams.params = params;
                    optParams.modelParamsSize = sizeN * alphaSize;
                    optParams.numOfEquation = params.expirySize*params.tenorSize + ...  % target cap & swaption
                                              sizeN*(alphaSize -2);   % tikhonov regularization gamma constraint
                    
                    tvar = zeros(sizeN*alphaSize,1);
                    for i=1:sizeN
                        for j=1:alphaSize
                             tvar((i-1)*alphaSize +j) = ch.lambda{i,1}.quote(j);
                        end
                    end
                    
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
                    
                    x=  zeros(optParams.numOfEquation,1);
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionCheyetteATMSwaptionSurfaceGlobal',tvar,x, 500, options,'unc',ch,optParams);
                    toc

%                     targetfun = @(tvar) TargetFunctionATMSwaptionGlobal(ch,tvar,params);
% %                     options=optimset('Algorithm','interior-point');
%                      options=optimset('Algorithm','trust-region-reflective');
%                     [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);
                    
                    for i=1:sizeN
                        for j=1:alphaSize
                             out.lambda{i,1}.quote(j) = max(abs(popt((i-1)*alphaSize +j)),0.0001); 
                             ch.lambda{i,1}.quote(j) = max(abs(popt((i-1)*alphaSize +j)),0.0001); 
                        end
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                 case 'globalFmin'
                    tvar = zeros(sizeN*alphaSize,1);
                    for i=1:sizeN
                        for j=1:alphaSize
                             tvar((i-1)*alphaSize +j) = ch.lambda{i,1}.quote(j);
                             lb((i-1)*alphaSize +j) = 0.01;
                             ub((i-1)*alphaSize +j) = 10.0;
                        end
                    end
                    
                       
                    targetfun = @(tvar) TargetFunctionATMSwaptionGlobal(ch,tvar,params);
%                     options=optimset('Algorithm','interior-point');
                     options=optimset('Algorithm','trust-region-reflective');
                    [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);
                    
                    for i=1:sizeN
                        for j=1:alphaSize
                             out.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                             ch.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                        end
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                 case 'globalImpVol'
                    for i=1:sizeN
                        for j=1:alphaSize
                             tvar((i-1)*alphaSize +j) = ch.lambda{i,1}.quote(j);
                             lb((i-1)*alphaSize +j) = 0.0001;
                             ub((i-1)*alphaSize +j) = 10.0;
                        end
                    end
                    
                       
                    targetfun = @(tvar) TargetFunctionATMSwaptionGlobalImpVol(ch,tvar,params);
%                     options=optimset('Algorithm','interior-point');
                     options=optimset('Algorithm','trust-region-reflective');
                    [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);
                    
                    for i=1:sizeN
                        for j=1:alphaSize
                             out.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                             ch.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                        end
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.blaskSwaptionImpVol(j,k) = blackSwaptionImpVol(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.chSwaptionImpVol(j,k) = ch.ATMSwaptionImpVol(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            our.volDiff(j,k) = out.chSwaptionImpVol(j,k) - out.blaskSwaptionImpVol(j,k);
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            out.vol2Total = out.vol2Total + our.volDiff(j,k)*our.volDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.vol2Total = sqrt(out.vol2Total/N);
                    
                case 'globalNormalVol'
                    for i=1:sizeN
                        for j=1:alphaSize
                             tvar((i-1)*alphaSize +j) = ch.lambda{i,1}.quote(j);
                             lb((i-1)*alphaSize +j) = 0.0001;
                             ub((i-1)*alphaSize +j) = 10.0;
                        end
                    end
                    
                       
                    targetfun = @(tvar) TargetFunctionATMSwaptionGlobalNormalVol(ch,tvar,params);
%                     options=optimset('Algorithm','interior-point');
                     options=optimset('Algorithm','trust-region-reflective');
                    [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);
                    
                    for i=1:sizeN
                        for j=1:alphaSize
                             out.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                             ch.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                        end
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.blackSwaptionImpVol(j,k) = blaskSwaptionImpVol(j,k);
                            out.blackSwaptionNormalVol(j,k) = blackSwaptionNormalVol(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.chSwaptionImpVol(j,k) = ch.ATMSwaptionImpVol(expiry(j),tenor(k));
                            out.chSwaptionNormalVol(j,k) = ch.ATMSwaptionNormalVol(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.ImpVolDiff(j,k) = out.chSwaptionImpVol(j,k) - out.blackSwaptionImpVol(j,k);
                            out.NormalVolDiff(j,k) = out.chSwaptionNormalVol(j,k)/out.blackSwaptionNormalVol(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            out.impVol2Total = out.impVol2Total + out.ImpVolDiff(j,k)*out.ImpVolDiff(j,k);
                            out.normalVol2Total = out.normalVol2Total + out.NormalVolDiff(j,k)*out.NormalVolDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.impVol2Total = sqrt(out.impVol2Total/N);
                    out.normalVol2Total = sqrt(out.normalVol2Total/N);
                
                otherwise
                    disp('unImplemented')
            end
        end
        
        function out = CalibrateToATMSwaptionAndCap(ch,black,calibrationFlag)
            tic;
            
            sizeN = ch.numOfFactor;
            alphaSize = length(ch.lambda{1,1}.tenor);
            for i=1:sizeN
               for j=1:alphaSize
                   out.lambda{i,1} = ch.lambda{i,1};
               end
            end
            
          %% Calculate Black ATM Swaption Prices
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
            
            expirySize = size(expiry,2);
            tenorSize = size(tenor,2);
            
            blackSwaptionPrice = zeros(expirySize,tenorSize);
            blackSwaptionImpVol = zeros(expirySize,tenorSize);
            for k=1:expirySize
                for j=1:tenorSize
                    blackSwaptionPrice(k,j)  = black.BlackATMSwaption(expiry(k),tenor(j));
                    blackSwaptionImpVol(k,j) = black.swaptionVol(expiry(k),tenor(j));
                end
            end
            
          %% Calculate Black ATM Cap Prices
           capVol = black.capVol;
           capVolData = black.mktData('capVol');
           capVolrawData =  capVolData.params('rawData');
            
           capExpiry = capVolrawData(:,1);
           capExpirySize = length(capExpiry);
            
           blackCapPrice = zeros(capExpirySize,1);
           blackCapImpVol = zeros(capExpirySize,1);
           blackCapStrike = zeros(capExpirySize,1);
           for k=1:capExpirySize
               blackCapStrike(k) = black.Fsr(0.25,capExpiry(k) -0.25,0.25);
               blackCapPrice(k) = black.BlackCap(capExpiry(k),blackCapStrike(k),0.25);
               blackCapImpVol(k) = black.capVol(capExpiry(k));
           end
          
          
            %%
            params.expiry = expiry;
            params.tenor = tenor;
            params.blackSwaptionPrice = blackSwaptionPrice;
            params.blackSwaptionImpVol = blackSwaptionImpVol;
            params.expirySize = expirySize;
            params.tenorSize =  tenorSize;
            
            params.capExpiry = capExpiry;
            params.blackCapPrice = blackCapPrice;
            params.blackCapStrike = blackCapStrike;
            params.blackCapImpVol = blackCapImpVol;
            params.capExpirySize = capExpirySize;
            
            out.blackSwaptionPrice = blackSwaptionPrice;
            out.chSwaptionPrice = zeros(expirySize,tenorSize);
            out.relDiff = zeros(expirySize,tenorSize);
            out.rmseTotal=0.0;
            
            out.blackCapPrice = blackCapPrice;
            out.chCapPrice = zeros(capExpirySize,1);
            out.capRelDiff = zeros(capExpirySize,1);
            
            calibrationType = 'bootstrap';
            if nargin < 3
                calibrationType = 'bootstrap'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;
            tvar = [];
            
            switch calibrationType
                case 'bootstrapLevMar'
                    tvar = zeros(sizeN,1);
                    for i=1:expirySize
                        idxNow = i;
                        for j=1:sizeN
                             tvar(j) = ch.lambda{j,1}.quote(i);
                             lb(j) = 0.0;
                             ub(j) = 10.0;
                        end
                        
                        targetfun = @(tvar) TargetFunctionATMSwaptionAndCapBootStrapLevMar(ch,tvar,idxNow,params);
                        options=optimset('Algorithm','levenberg-marquardt');
%                         options=optimset('Algorithm','trust-region-reflective');
                        [x, fval] = lsqnonlin(targetfun,tvar,lb,ub,options);
                        
                        for j=1:sizeN
                             out.lambda{j,1}.quote(i) = x(j); 
                             ch.lambda{j,1}.quote(i) = x(j); 
                        end
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                    
                    for j=1:capExpirySize
                        out.chCapPrice(j) = ch.Cap(capExpiry(j),blackCapStrike(j),0.25);
                        out.capRelDiff(j) = out.chCapPrice(j)/out.blackCapPrice(j) -1.0;
                        out.rmseTotal = out.rmseTotal +  out.capRelDiff(j)* out.capRelDiff(j);
                        N=N+1;
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                out.calibrationTime = toc;
                
                case 'globalLevMar'
                    for i=1:sizeN
                        for j=1:alphaSize
                             tvar((i-1)*alphaSize +j) = ch.lambda{i,1}.quote(j);
                             lb((i-1)*alphaSize +j) = 0.0;
                             ub((i-1)*alphaSize +j) = 10.0;
                        end
                    end
                    
                       
                    targetfun = @(tvar) TargetFunctionATMSwaptionAndCapGlobalLevMar(ch,tvar,params);
                    options=optimset('Algorithm','levenberg-marquardt');
                    [x, fval] = lsqnonlin(targetfun,tvar,lb,ub,options);
                    
                    for i=1:sizeN
                        for j=1:alphaSize
                             out.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                             ch.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                        end
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                    
                    for j=1:capExpirySize
                        out.chCapPrice(j) = ch.Cap(capExpiry(j),blackCapStrike(j),0.25);
                        out.capRelDiff(j) = out.chCapPrice(j)/out.blackCapPrice(j) -1.0;
                        out.rmseTotal = out.rmseTotal +  out.capRelDiff(j)* out.capRelDiff(j);
                        N=N+1;
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.calibrationTime = toc;
                   
                otherwise
                    disp('unImplemented')
            end
        end    
    end
    
end



In [ ]:
classdef Cheyette < IRModel
    %UNTITLED Summary of this class goes here
    %   Detailed explanation goes here
    
    properties
        % vol functor
        smoothAlpha;
        numOfFactor;
        lambda = {};
        kappa = [];
        tau = [];
        corr_fwd = [];
        rho = [];
        inv_H_tau = [];
        swaption_calibration_yn=[];
    end
    
    methods
        function ch = Cheyette(params)
            if nargin > 0
                ch.mktData =  params.mktData;
                ch.modelParams = params.modelParams;
                
                ch.zeroCurve = params.zeroCurve;
                
                ch.smoothAlpha =  params.modelParams('smoothAlpha');
                
                ch.numOfFactor = params.modelParams('numOfFactor');
                sizeN = ch.numOfFactor;
                ch.lambda = cell(sizeN,1);
                ch.lambda = params.modelParams('lambda');
                
                ch.kappa = ones(sizeN,1);
                ch.kappa =  params.modelParams('kappa');
               
                ch.tau = ones(sizeN,1);
                ch.tau =  params.modelParams('tau');
                ch.inv_H_tau = ones(sizeN,sizeN);
                H_tau = ones(sizeN,sizeN);
                for i=1:sizeN
                    for j=1:sizeN
                        H_tau(i,j) = exp(-ch.kappa(j)* ch.tau(i));
                    end
                end
                ch.inv_H_tau = inv(H_tau);
                
                ch.corr_fwd = zeros(sizeN,sizeN);
                ch.corr_fwd =  params.modelParams('corr_fwd');
                
                ch.rho = zeros(sizeN,sizeN); 
                [PC,V]=eig(ch.corr_fwd);
                PCW= PC*sqrt(V);
                for i=1:sizeN
                    for j=1:sizeN
                        ch.rho(i,j)=PCW(i,j);
                    end
                end
                
                ch.swaption_calibration_yn = params.modelParams('swaptionCalibrationYN');
           
                
            end
        end
        
        function out = SimpleSRPrOverHedge(ch,spreadC,spreadN,ralower,raupper)
            buffer = 0.001;
            outVec = 0.0;
            if spreadC < ralower - buffer
                outVec = 0.0;
            elseif spreadC >= ralower - buffer && spreadC < ralower
                outVec = 1.0/buffer * (spreadC - (ralower - buffer))
            elseif spreadC >= ralower && spreadC < raupper
                outVec = 1.0;
            elseif spreadC >= raupper && spreadC < raupper + buffer
                outVec = -1.0/buffer *(spreadC - (raupper + buffer))
            else
                outVec = 0.0;
            end
            
            out = outVec;
        end
        
        function out = SimpleSRPr(ch,spreadC,spreadN,ralower,raupper)
            outVec = 0.0;
            if spreadC >= ralower && spreadC <= raupper
                outVec = 1.0;
            end
            out = outVec;
        end
        
        
        function out = TRZ_SRPr(ch,curr,next,L,U)
            length = 1.0;
            remain = 1.0;
            if (curr >= L && curr <= U && next >= L && next <= U)
                length = remain;
            elseif ( curr >= L && curr <= U && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr >= L && curr <= U && next < L ) 
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr > U && next >= L && next <= U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next < L )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next >= L && next <= U )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next > U )
                length = 0.0;
            elseif ( curr < L && next < L)
                length = 0.0;
            else
                disp('error')
            end
            
            out = length;
        end
        
        function out = SRPr(ch,spreadC,spreadN,ralower,raupper,simpleYN,pathSize)
            
            outVec = zeros(1,pathSize);
            
            if simpleYN == 1
                for i =1:pathSize
                    outVec(i) = ch.SimpleSRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            elseif simpleYN == 2
                for i =1:pathSize
                    outVec(i) = ch.SimpleSRPrOverHedge(spreadC(i),spreadN(i),ralower,raupper);
                end
            else
                for i =1:pathSize
                    outVec(i) = ch.TRZ_SRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            end
            
            out = outVec;
        end
        
        function out =  CMS_PSA_SimpleDate(ch,valueDate,observeTime,numMatTime, tenor,modelStatesT)
           %CMS
           tau = 0.25;
           to      = observeTime/365.0;
           toDate = valueDate.AddDate(observeTime,'day');
           tNDate = toDate.AddDate(tenor,'year');
           tN   =  tNDate.DateDiff(valueDate)/365.0;
           
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           p_nN = ch.PV01Date(valueDate,toDate,tNDate,tau);
           
           df_Tn = ch.DF(to);
           df_TN = ch.DF(tN);
           y_nN = (df_Tn - df_TN)/p_nN;
           
           c_nN = ch.C_nNDate(valueDate,toDate,tNDate,tau,p_nN,y_nN);
           g_nN = ch.G_nNDate(valueDate,toDate,tNDate,tau,p_nN);
           h_nTerm = ch.H_nTermDate(valueDate,toDate,numMatDate);
           varCovar = ch.LocalVariance(0,to);
           
           sizeN = ch.numOfFactor;
            
           stateSize = size(modelStatesT,2);
           cms_PCA_SimpleOut = zeros(1,stateSize);
           
           driftTerm = 0;
           for i=1:sizeN
               for j=1:sizeN
                   driftTerm = driftTerm + (c_nN(i) * h_nTerm(j) + c_nN(i) * g_nN(j))*varCovar(i,j);
               end
           end
           
           for i=1:stateSize
               cms_PCA_SimpleOut(i) = y_nN + driftTerm;
               for j=1:sizeN
                cms_PCA_SimpleOut(i) = cms_PCA_SimpleOut(i) + c_nN(j)*modelStatesT(j,i);
               end
           end
           
           out = cms_PCA_SimpleOut;
           
            
        end
        
        function out = CheyetteGeneralVariance(ch,from,to,T1,T2)
            
            sizeN = ch.numOfFactor;
            plambda =cell(sizeN,1);
            for i=1:sizeN
                plambda{i,1}= ch.lambda{i,1};
            end
            
            sigma_f = zeros(sizeN,sizeN);
            
            % eta is short rate volatlity
            % eta2 is short rate covariance matrix

            for i=1:sizeN
                for j=1:sizeN
                    sigma_f(i,j)=sqrt(plambda{i,1}.quote(1)*plambda{i,1}.quote(1))*ch.rho(i,j);
                end
            end
            
           eta = zeros(sizeN,sizeN);
           eta = ch.inv_H_tau*sigma_f;
           eta2 = eta*eta';
 
           sigmaTimes = plambda{1,1}.tenor;
           sigmaSize = length(plambda{1,1}.tenor);
            
            for i=1:sigmaSize
                if from<= sigmaTimes(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:sigmaSize
                if to<= sigmaTimes(i)
                    break;
                end
            end
            endIdx = i;
            
            lastExpScale = zeros(4,sizeN,sizeN);
            newExpScale  = zeros(4,sizeN,sizeN);
            variances = zeros(4,sizeN,sizeN);
            meanR = zeros(sizeN,1);
            
            for i=1:sizeN
                meanR(i) = ch.kappa(i);
            end
%             newExpScale = zeros(4,1);
%             lastExpScale = zeros(4,1);
%             variances = zeros(4,1);
            
            lastU  = from;
            
            for j=1:sizeN
                for k=1:sizeN
                    lastExpScale(2,j,k) = exp(meanR(j)*lastU);
                    lastExpScale(3,j,k) = exp(meanR(k)*lastU);
                    lastExpScale(4,j,k) = exp((meanR(j)+meanR(k))*lastU);
                end
            end
            
            for i=startIdx:endIdx
                if (i< endIdx) U = sigmaTimes(i);else U = to;end;
                if (i< sigmaSize) tindex = i;else tindex = sigmaSize;end;
            % eta is short rate volatlity
                for j=1:sizeN
                    for k=1:sizeN
                        sigma_f(j,k)=sqrt(plambda{j,1}.quote(tindex)*plambda{j,1}.quote(tindex))*ch.rho(j,k);
                    end
                end
                
                eta = ch.inv_H_tau*sigma_f;
                eta2 = eta*eta';
                
                for j=1:sizeN
                    for k=1:sizeN
                        newExpScale(2,j,k) = exp(meanR(j)*U);
                        newExpScale(3,j,k) = exp(meanR(k)*U);
                        newExpScale(4,j,k) = exp((meanR(j)+meanR(k))*U);
                    
                        variances(1,j,k) = variances(1,j,k) + eta2(j,k) * (U - lastU);
                        variances(2,j,k) = variances(2,j,k) + eta2(j,k) * (1.0/ meanR(j))*(newExpScale(2,j,k) - lastExpScale(2,j,k));
                        variances(3,j,k) = variances(3,j,k) + eta2(j,k) * (1.0/ meanR(k))*(newExpScale(3,j,k) - lastExpScale(3,j,k));
                        variances(4,j,k) = variances(4,j,k) + eta2(j,k) * (1.0/ (meanR(j)+meanR(k)))*(newExpScale(4,j,k) - lastExpScale(4,j,k));
                    end
                end
                
                lastU = U;
                for j=1:sizeN
                    for k=1:sizeN
                        lastExpScale(2,j,k) = newExpScale(2,j,k);
                        lastExpScale(3,j,k) = newExpScale(3,j,k);
                        lastExpScale(4,j,k) = newExpScale(4,j,k);
                    end
                end
                
            end
            
            value = 0;
            
            for j=1:sizeN
                for k=1:sizeN
                    value = value + (1.0/ meanR(j))*(1.0/meanR(k))*variances(1,j,k);
                end
            end
            
            for j=1:sizeN
                for k=1:sizeN
                    value = value + -1.0*exp(-1.0*meanR(j)*T1)*(1.0/ meanR(j))*(1.0/meanR(k))*variances(2,j,k);
                end
            end
            
            for j=1:sizeN
                for k=1:sizeN
                    value = value + -1.0*exp(-1.0*meanR(k)*T2)*(1.0/ meanR(j))*(1.0/meanR(k))*variances(3,j,k);
                end
            end
            
            for j=1:sizeN
                for k=1:sizeN
                    value = value +  1.0*exp(-1.0*meanR(j)*T1)*exp(-1.0*meanR(k)*T2)*(1.0/ meanR(j))*(1.0/meanR(k))*variances(4,j,k);
                end
            end
            
            out  = value;
        end
        
        function out = fwdZcVariance(ch, from, to, T1, T2)
            
            localVariance = zeros(2,1);
            
            localVariance(1) = ch.CheyetteGeneralVariance(from,to,T1,T1);
            localVariance(2) = ch.CheyetteGeneralVariance(from,to,to,to);
            
            
            value = localVariance(1) - localVariance(2);
            
            measureChange = zeros(2,1);
            
            measureChange(1) = ch.CheyetteGeneralVariance(from,to,T1,T2);
            measureChange(2) = ch.CheyetteGeneralVariance(from,to,to,T2);
            
            measureChangeTerm = measureChange(1) - measureChange(2);
            
            value = value + -2.0*measureChangeTerm;
            
            out = value;
            
        end
        
        function out = discountFactor(ch, startTime, endTime,numMatTime,modelStatesT)
           zcT = ch.DF(endTime/365.0);
           zct = ch.DF(startTime/365.0);
           beta = ch.Beta(startTime/365.0,endTime/365.0);
           zcDriftTerm = ch.fwdZcVariance(0,startTime/365.0,endTime/365.0,numMatTime/365.0);
           %numOfPath
           stateSize = size(modelStatesT,2);
           discountFactorOut = zeros(1,stateSize);
           
           sizeN = ch.numOfFactor;
           
           for i=1:stateSize
               discountFactorOut(i) = zcT/zct * exp(-0.5*zcDriftTerm);
               for j=1:sizeN
                    discountFactorOut(i) = discountFactorOut(i) * exp(-beta(j)*modelStatesT(j,i));
               end
           end
           
           out = discountFactorOut;
        end
        
        function out = discountPayoff(ch,eventTime,numMatTime ...
                ,modelStatesT,cashflow)
            
            dfNumMat = ch.DF(numMatTime/365.0);
            % for discountPayoff T=numMatTime
            % P(0,TnumMat)/P(Te,Tnummat)*Payoff
            %
            discountT = ch.discountFactor(eventTime, numMatTime,numMatTime,modelStatesT);
            stateSize = size(modelStatesT,2);
            payoff = zeros(1,stateSize);
            for i=1:stateSize
                payoff(i)= dfNumMat/discountT(i)*cashflow(i);
            end    
            out = payoff;
        end
        
        function out = stateLocalDrift(ch,fromTime, toTime)
            sizeN = ch.numOfFactor;
            drift = zeros(sizeN,sizeN);
            for i=1:sizeN
                drift(i,i) =  exp(-1.0*ch.kappa(i)*(toTime-fromTime)/365.0);
            end
            out = drift;
        end
        
        function out = stateLocalVariance(ch,fromTime, toTime)
            out = ch.LocalVariance(fromTime/365.0,toTime/365.0);
        end
        
        function variances = LocalVariance(ch,from,to)
            sizeN = ch.numOfFactor;
            plambda =cell(sizeN,1);
            for i=1:sizeN
                plambda{i,1}= ch.lambda{i,1};
            end
            sigma_f =[];
            sigma_f = zeros(sizeN,sizeN);
%% converting forward rate volatlity into short rate volatlitity
%% eta is short rate volatlity
            for i=1:sizeN
                for j=1:sizeN
%                     sigma_f(i,j)=plambda{i,1}.quote(1)*ch.Fwd(0,ch.tau(i))*ch.rho(i,j);
%                     sigma_f(i,j)=plambda{i,1}.quote(1)*ch.rho(i,j);
                    sigma_f(i,j)=sqrt(plambda{i,1}.quote(1)*plambda{i,1}.quote(1))*ch.rho(i,j);
                end
            end
            
           eta =[];
           eta = zeros(sizeN,sizeN);
           eta = ch.inv_H_tau*sigma_f;
           eta2 = eta*eta';
           %%
           alphaSize = length(plambda{1,1}.tenor);
            
           for i=1:alphaSize
               if from<= plambda{1,1}.tenor(i);
                   break;
               end
           end
            
           startIdx = i;
            
%             endIdx = size(sig(1).tenor,1); 
           for i=1:alphaSize
               if to<= plambda{1,1}.tenor(i);
                   break;
               end
           end
           endIdx = i;
            
           lastU  = from;
           
           tiny = zeros(sizeN,sizeN);
           lastExpScale = zeros(sizeN,sizeN);
           newExpScale  = zeros(sizeN,sizeN);
           scale = 0.0;
           meanR = zeros(sizeN,1);
           for i=1:sizeN
                meanR(i) = ch.kappa(i);
           end
           for i=1:sizeN
                for j=1:sizeN
                    if(abs(meanR(i)+meanR(j)) < 1.0E-12) tiny(i,j) = 1; else tiny(i,j) = 0; end;
                    lastExpScale(i,j) =  exp((meanR(i)+meanR(j))*from);
                end
           end
            
           variances = zeros(sizeN,sizeN);
           tindex = 1;
           for i=startIdx:endIdx
                if (i< endIdx) U = plambda{1,1}.tenor(i);else U = to;end;
                if (i< alphaSize) tindex = i;else tindex = alphaSize;end;
            %% converting forward rate volatlity into short rate volatlitity
            %% eta is short rate volatlity
                for j=1:sizeN
                    for k=1:sizeN
%                         sigma_f(j,k)=plambda{j,1}.quote(tindex)*ch.Fwd(0,ch.tau(j))*ch.rho(j,k);
%                         sigma_f(j,k)=plambda{j,1}.quote(tindex)*ch.rho(j,k);
                        sigma_f(j,k)=sqrt(plambda{j,1}.quote(tindex)*plambda{j,1}.quote(tindex))*ch.rho(j,k);
                    end
                end
                
                eta = ch.inv_H_tau*sigma_f;
                eta2 = eta*eta';
            %%
                for j=1:sizeN
                    for k=1:sizeN
                        newExpScale(j,k) =  exp((meanR(j)+meanR(k))*U);
                        if(tiny(j,k)~=1)
                            scale = newExpScale(j,k)-lastExpScale(j,k);
                        else
                            scale = U - lastU;
                        end
                        variances(j,k) = variances(j,k) + eta2(j,k) * scale;
                        lastExpScale(j,k) = newExpScale(j,k);
                    end
                end
                lastU = U;
           end
           
           for j=1:sizeN
                for k=1:sizeN
                    if(tiny(j,k)~= 1)
                        variances(j,k) = variances(j,k)/(meanR(j)+meanR(k))*exp(-1.0*(meanR(j)+meanR(k))*to);
                    end
                end
           end
            
            
        end

        function beta = Beta(ch,t,T)
            beta = [];
            sizeN = ch.numOfFactor;
            beta = zeros(sizeN,1);
            
            for i=1:sizeN
                if(abs(ch.kappa(i)) > 1.0E-12)
                    beta(i) = (1.0-exp(-ch.kappa(i)*(T-t)))/ch.kappa(i);
                else
                    beta(i) = (T-t);
                end
            end
        end
        
        function beta = BetaDate(ch,valueDate,tDate,TDate)
            t = tDate.DateDiff(valueDate)/365.0;
            T = TDate.DateDiff(valueDate)/365.0;
            
            sizeN = ch.numOfFactor;
            beta = zeros(sizeN,1);
            
            for i=1:sizeN
                if(abs(ch.kappa(i)) > 1.0E-12)
                    beta(i) = (1.0-exp(-ch.kappa(i)*(T-t)))/ch.kappa(i);
                else
                    beta(i) = (T-t);
                end
            end
            
        end
        
        function c_nN = C_nN(ch,tn,tN,tau,pnN,ynN)
            p = @ch.DF;
            c_nN = [];
            sizeN = ch.numOfFactor;
            c_nN = zeros(sizeN,1);
            
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = ch.Beta(tn,tp);
                df = p(tp);
                for j=1:sizeN
                    c_nN(j) = c_nN(j) +tau * beta(j) * df;
                end
            end
            
            for j=1:sizeN
                c_nN(j) = c_nN(j)* ynN;
            end
            
            for j=1:sizeN
                c_nN(j) = c_nN(j) + beta(j)*df;
            end
            
            for j=1:sizeN
                c_nN(j) = c_nN(j)/pnN;
            end
           
        end
        
        function c_nN = C_nNDate(ch,valueDate,tnDate,tNDate,tau,pnN,ynN)
            tn = tnDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            
            addMonthUnit = round(12*tau);
            cnt = round((tN-tn)/tau);
            
            p = @ch.DF;
            sizeN = ch.numOfFactor;
            c_nN = zeros(sizeN,1);
            
            for i=1:cnt
                tpDate = tnDate.AddDate(i*addMonthUnit,'month');
                tp = tpDate.DateDiff(valueDate)/365.0;
                beta = ch.BetaDate(valueDate,tnDate,tpDate);
                df = p(tp);
                for j=1:sizeN
                    c_nN(j) = c_nN(j) +tau * beta(j) * df;
                end
            end
            
            for j=1:sizeN
                c_nN(j) = c_nN(j)* ynN;
            end
            
            for j=1:sizeN
                c_nN(j) = c_nN(j) + beta(j)*df;
            end
            
            for j=1:sizeN
                c_nN(j) = c_nN(j)/pnN;
            end
        end
        
        function g_nN = G_nN(ch,tn,tN,tau,pnN)
            p = @ch.DF;
            g_nN = [];
            sizeN = ch.numOfFactor;
            g_nN = zeros(sizeN,1);
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = ch.Beta(tn,tp);
                df = p(tp);
                for j=1:sizeN
                    g_nN(j) = g_nN(j) +tau * beta(j) * df;
                end
            end
            
            for j=1:sizeN
                g_nN(j) = g_nN(j)/pnN;
            end
            
        end
        
        function g_nN = G_nNDate(ch,valueDate,tnDate,tNDate,tau,pnN)
            tn = tnDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            addMonthUnit = round(12*tau);
            cnt = round((tN-tn)/tau);
            
            p = @ch.DF;
            sizeN = ch.numOfFactor;
            g_nN = zeros(sizeN,1);
            
            for i=1:cnt
                tpDate = tnDate.AddDate(i*addMonthUnit,'month');
                tp = tpDate.DateDiff(valueDate)/365.0;
                beta = ch.BetaDate(valueDate,tnDate,tpDate);
                df = p(tp);
                for j=1:sizeN
                    g_nN(j) = g_nN(j) +tau * beta(j) * df;
                end
            end
            
            for j=1:sizeN
                g_nN(j) = g_nN(j)/pnN;
            end
        end 
        
        function h_nTerm = H_nTerm(ch,tn,tTerm)
           h_nTerm =[];
           sizeN = ch.numOfFactor;
           h_nTerm = zeros(sizeN,1);
           beta = ch.Beta(tn,tTerm);
           
           for j=1:sizeN
                h_nTerm(j) = -1.0*beta(j);
           end
          
        end
        
        function h_nTerm = H_nTermDate(ch,valueDate,tnDate,tTermDate)
           h_nTerm =[];
           sizeN = ch.numOfFactor;
           h_nTerm = zeros(sizeN,1);
           beta = ch.BetaDate(valueDate,tnDate,tTermDate);
           for j=1:sizeN
                h_nTerm(j) = -1.0*beta(j);
           end
        end
        
        
        
        function out = CMSDigitalArg(ch,to,tp,tenor1,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           sizeN = ch.numOfFactor;
           p_nN1 = ch.PV01(to,to+tenor1,tau);
           df_Tn1 = ch.DF(to);
           df_TN1 = ch.DF(to + tenor1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           c_nN1 = ch.C_nN(to,to + tenor1,tau,p_nN1,y_nN1);
           g_nN1 = ch.G_nN(to,to + tenor1,tau,p_nN1);
           h_nTerm1 = ch.H_nTerm(to,tp);
           
           varCovar = ch.LocalVariance(0,to);
           
           driftTerm1=0.0;
           for i=1:sizeN
               for j=1:sizeN
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           for i=1:sizeN
               for j=1:sizeN
                    spreadVariance = spreadVariance + (c_nN1(i)) * (c_nN1(j)) * varCovar(i,j);
               end
           end
           
           out.mu = (y_nN1 + driftTerm1);
           if spreadVariance >=0
            out.sigma = sqrt(spreadVariance);
           else
            out.sigma = 0.0;
           end
           
           out.c_nN1 = c_nN1;
           out.varCovar = varCovar;
           
        end
        
        function out = CMSDigital(ch,to,tp,tenor1,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsDigitalArg = ch.CMSDigitalArg(to,tp,tenor1,tau);
           mu = cmsDigitalArg.mu;
           sigma = cmsDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSDigitalRange(ch,to,tp,tenor1,tau,lower,upper)
            lower_value = ch.CMSDigital(to,tp,tenor1,tau,lower);
            upper_value = ch.CMSDigital(to,tp,tenor1,tau,upper);
            out = lower_value - upper_value;
        end
        
        
        function out = CMSSpreadDigitalArg(ch,to,tp,tenor1,tenor2,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           sizeN = ch.numOfFactor;
           p_nN1 = ch.PV01(to,to+tenor1,tau);
           df_Tn1 = ch.DF(to);
           df_TN1 = ch.DF(to + tenor1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           c_nN1 = ch.C_nN(to,to + tenor1,tau,p_nN1,y_nN1);
           g_nN1 = ch.G_nN(to,to + tenor1,tau,p_nN1);
           h_nTerm1 = ch.H_nTerm(to,tp);
           
           %CMS2
           p_nN2 = ch.PV01(to,to+tenor2,tau);
           df_Tn2 = ch.DF(to);
           df_TN2 = ch.DF(to + tenor2);
           y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
           c_nN2 = ch.C_nN(to,to + tenor2,tau,p_nN2,y_nN2);
           g_nN2 = ch.G_nN(to,to + tenor2,tau,p_nN2);
           h_nTerm2 = ch.H_nTerm(to,tp);
           
           varCovar = ch.LocalVariance(0,to);
           
           driftTerm1=0.0;
           driftTerm2=0.0;
           for i=1:sizeN
               for j=1:sizeN
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
                    driftTerm2 = driftTerm2 + c_nN2(i) * (h_nTerm2(j) + g_nN2(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           variance1 = 0.0;
           variance2 = 0.0;
           coVariance = 0.0;
           for i=1:sizeN
               for j=1:sizeN
                    spreadVariance = spreadVariance + (c_nN1(i)-c_nN2(i)) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
                    variance1 = variance1 + (c_nN1(i)) * (c_nN1(j)) * varCovar(i,j);
                    variance2 = variance2 + (c_nN2(i)) * (c_nN2(j)) * varCovar(i,j);
                    coVariance = coVariance + (c_nN1(i)) * (c_nN2(j)) * varCovar(i,j);
               end
           end
           
           out.fwdSpread = y_nN1 - y_nN2;
           out.mu = (y_nN1 + driftTerm1) - (y_nN2 + driftTerm2);
           out.sigma = sqrt(spreadVariance);
           out.c_nN1 = c_nN1;
           out.c_nN2 = c_nN2;
           out.varCovar = varCovar;
           out.corr = coVariance/sqrt(variance1*variance2);
           
           out.mu1 = y_nN1 + driftTerm1;
           out.mu2 = y_nN2 + driftTerm2;
           
           out.sig1 = sqrt(variance1);
           out.sig2 = sqrt(variance2);
           
           out.pmu = driftTerm1 - driftTerm2;
           out.pmu1 = driftTerm1;
           out.pmu2 = driftTerm2;
           
           out.y1 = y_nN1;
           out.y2 = y_nN2;
           
        end
        
        function out = CMSSpreadDigitalArgDate(ch,valueDate,toDate,tpDate,tenor1,tenor2,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           sizeN = ch.numOfFactor;
           
           to      = toDate.DateDiff(valueDate)/365.0;
           tN1Date = toDate.AddDate(tenor1,'year');
           tN1   =  tN1Date.DateDiff(valueDate)/365.0;
           
           p_nN1 = ch.PV01Date(valueDate,toDate,tN1Date,tau);
           
           df_Tn1 = ch.DF(to);
           df_TN1 = ch.DF(tN1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           
           c_nN1 = ch.C_nNDate(valueDate,toDate,tN1Date,tau,p_nN1,y_nN1);
           g_nN1 = ch.G_nNDate(valueDate,toDate,tN1Date,tau,p_nN1);
           h_nTerm1 = ch.H_nTermDate(valueDate,toDate,tpDate);
           
           %CMS2
           tN2Date = toDate.AddDate(tenor2,'year');
           tN2   =  tN2Date.DateDiff(valueDate)/365.0;
           
           p_nN2 = ch.PV01Date(valueDate,toDate,tN2Date,tau);
           
           df_Tn2 = ch.DF(to);
           df_TN2 = ch.DF(tN2);
           y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
           
           c_nN2 = ch.C_nNDate(valueDate,toDate,tN2Date,tau,p_nN2,y_nN2);
           g_nN2 = ch.G_nNDate(valueDate,toDate,tN2Date,tau,p_nN2);
           h_nTerm2 = ch.H_nTermDate(valueDate,toDate,tpDate);
           
           varCovar = ch.LocalVariance(0,to);
           
           driftTerm1=0.0;
           driftTerm2=0.0;
           for i=1:sizeN
               for j=1:sizeN
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
                    driftTerm2 = driftTerm2 + c_nN2(i) * (h_nTerm2(j) + g_nN2(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           for i=1:sizeN
               for j=1:sizeN
                    spreadVariance = spreadVariance + (c_nN1(i)-c_nN2(i)) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
               end
           end
           
           out.mu = (y_nN1 + driftTerm1) - (y_nN2 + driftTerm2);
           out.sigma = sqrt(spreadVariance);
           out.c_nN1 = c_nN1;
           out.c_nN2 = c_nN2;
           out.varCovar = varCovar;
           
        end
        
        function out = CMSSpreadDigital(ch,to,tp,tenor1,tenor2,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsSpreadDigitalArg = ch.CMSSpreadDigitalArg(to,tp,tenor1,tenor2,tau);
           mu = cmsSpreadDigitalArg.mu;
           sigma = cmsSpreadDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSSpreadDigitalDate(ch,valueDate,toDate,tpDate,tenor1,tenor2,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsSpreadDigitalArg = ch.CMSSpreadDigitalArgDate(valueDate,toDate,tpDate,tenor1,tenor2,tau);
           mu = cmsSpreadDigitalArg.mu;
           sigma = cmsSpreadDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSSpreadDigitalRange(ch,to,tp,tenor1,tenor2,tau,lower,upper)
            lower_value = ch.CMSSpreadDigital(to,tp,tenor1,tenor2,tau,lower);
            upper_value = ch.CMSSpreadDigital(to,tp,tenor1,tenor2,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CMSSpreadDigitalRangeDate(ch,valueDate,toDate,tpDate,tenor1,tenor2,tau,lower,upper)
            lower_value = ch.CMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,lower);
            upper_value = ch.CMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,upper);
            out = lower_value - upper_value;
        end
        
        function atmSwaption = ATMSwaption(ch,tx,tnr)
            sizeN = ch.numOfFactor;
            tau = 0.25;
            pv01 = ch.PV01(tx,tx+tnr,tau);
            fsr  = ch.Fsr(tx,tx+tnr,tau);
            c_nN = ch.C_nN(tx,tx+tnr,tau,pv01,fsr);
            variances = ch.LocalVariance(0,tx);
            totalVar = 0.0;
            for i=1:sizeN
                for j=1:sizeN
                    totalVar = totalVar + c_nN(i)*c_nN(j)*variances(i,j);
                end
            end
            if totalVar <=0
                totalVar = 0.0;
            end
            atmSwaption =  pv01 * sqrt(totalVar)/sqrt(2.0*pi);
        end
        
        function atmSwaptionNormalVol = ATMSwaptionNormalVol(ch,tx,tnr)
            sizeN = ch.numOfFactor;
            tau = 0.25;
            pv01 = ch.PV01(tx,tx+tnr,tau);
            fsr  = ch.Fsr(tx,tx+tnr,tau);
            c_nN = ch.C_nN(tx,tx+tnr,tau,pv01,fsr);
            variances = ch.LocalVariance(0,tx);
            totalVar = 0.0;
            for i=1:sizeN
                for j=1:sizeN
                    totalVar = totalVar + c_nN(i)*c_nN(j)*variances(i,j);
                end
            end
            if totalVar <=0
                totalVar = 0.0;
            end
            atmSwaptionNormalVol =  sqrt(totalVar/tx);
        end
        
        function capletOut = Caplet(ch,tx,x,tnr)
        %blackCaplet:a function that computes a blackCaplet price
        %arguments
        %tx =(annualized)time to reset
        %sig= caplet volatility
        %x  = caplet strike
        %tnr=(annualized)time to payment
        %NA =Notional Amount
        
            tau = 0.25;
            p = @ch.DF;
            beta = ch.Beta(tx,tx + tnr);
            sizeN = ch.numOfFactor;
            variances = ch.LocalVariance(0,tx);
            totalVar = 0.0;
            for i=1:sizeN
                for j=1:sizeN
                    totalVar = totalVar + beta(i)*beta(j)*variances(i,j);
                end
            end
            sig = sqrt(totalVar);
            d1= (log(p(tx)/p(tx + tnr)*1.0/(1+tnr*x))+(sig^2/2.0))/(sig);
            d2 = d1 -sig;
            capletOut = p(tx)*H_ncdf(d1) - p(tx+tnr)*(1+tnr*x)*H_ncdf(d2);
        end
        
        function capOut = Cap(ch,tx,x,tnr)
            capOut = sum(arrayfun(@(i) ch.Caplet(tnr*i,x,tnr),1:round(tx/tnr)-1));
        end
        
         function atmSwaptionImpVol = ATMSwaptionImpVol(ch,tx,tnr)
            atmSwaption = ch.ATMSwaption(tx,tnr);
            
            a = 0.001;
            b = 10;
            Tol = 1e-10;
            MaxIter = 1000;
            tau = 0.25;
            fsr  = ch.Fsr(tx,tx+tnr,tau);
            pv01 = ch.PV01(tx,tx+tnr,tau);
            impVol = BisecBSIV('P',fsr,fsr, ...
                                          0.0,tx,a,b,atmSwaption/pv01,Tol,MaxIter);
            atmSwaptionImpVol =  impVol;
         end
        
         function out = findTindex(ch,t,sigma)
            
            sum = 0.0;
            prev_t = 0.0;
            inRangeYN = false;
            for i=1:length(sigma.tenor)
                alpha = i;
                if t<= sigma.tenor(i)
                    sum = alpha;
                    inRangeYN = true;
                    break;
                end
            end
            
            if ~inRangeYN 
                sum = alpha;
            end
            out = sum;
        end
        
        function instFwdCorrTest = InsFwdCorrTest(ch,tx,tnr1,tnr2)
            sizeN = ch.numOfFactor;
            plambda =cell(sizeN,1);
            for i=1:sizeN
                plambda{i,1}= ch.lambda{i,1};
            end
            
            sigma_f = zeros(sizeN,sizeN);
            
%             tindex = ch.findTindex(tx+1e-6,plambda{1,1});
            tenor = ch.lambda{1,1}.tenor;
            tindex = min(find(tx<=tenor));
            
            for j=1:sizeN
                for k=1:sizeN
                    sigma_f(j,k)=plambda{j,1}.quote(tindex)*ch.rho(j,k);
                end
            end
                
            eta = ch.inv_H_tau*sigma_f;
            eta2 = eta*eta';
            
            KappaTau1 = zeros(sizeN,1);
            KappaTau2 = zeros(sizeN,1);
            
            for i=1:sizeN
                KappaTau1(i) = exp(-ch.kappa(i)*tnr1);
                KappaTau2(i) = exp(-ch.kappa(i)*tnr2);
            end
            
            
            covar = 0.0;
            var1 = 0;
            var2 = 0;
            var1 =  KappaTau1'*eta2*KappaTau1;
            var2 =  KappaTau2'*eta2*KappaTau2;
            covar =  KappaTau1'*eta2*KappaTau2;
            
            instFwdCorrTest = covar/sqrt(var1*var2);
        end
        
        function instFwdCorrTestMatrix = InsFwdCorrTest2(ch,tx,tnr1,tau)
            if (tnr1(length(tnr1)) <= tx)
                disp('beyond the limit')
                instFwdCorrTestMatrix = 0;
            else

                % find T0 such that T0 =min{T_i|tx <= T_i}
                ipos = find(tnr1-tx > 0);
                T0 = tnr1(ipos(1));
                idx = ipos(1);
                newtnr = tnr1(idx:end);
                % make a matrix that spans T0,...,TN
                coVar = zeros(length(newtnr),length(newtnr));

                sizeN = ch.numOfFactor;
                plambda =cell(sizeN,1);
                for i=1:sizeN
                    plambda{i,1}= ch.lambda{i,1};
                end

                sigma_f = zeros(sizeN,sizeN);

    %             tindex = ch.findTindex(tx+1e-6,plambda{1,1});
                tenor = ch.lambda{1,1}.tenor;
                tindex = min(find(tx<=tenor));

                for j=1:sizeN
                    for k=1:sizeN
                        sigma_f(j,k)=plambda{j,1}.quote(tindex)*ch.rho(j,k);
                    end
                end

                eta = ch.inv_H_tau*sigma_f;
                eta2 = eta*eta';


                deltaG_tT = zeros(length(newtnr),sizeN);
                
                for i=1:length(newtnr)
                    G_tT1 = ch.Beta(tx,newtnr(i)+tau);
                    G_tT2 = ch.Beta(tx,newtnr(i));
                    for j=1:sizeN
                        deltaG_tT(i,j) = (G_tT1(j) - G_tT2(j))/tau;
                    end
                end

                
                testLoadingD = deltaG_tT*eta;

                coVar =  deltaG_tT*eta*eta'*deltaG_tT';
                instFwdCovarTestMatrix = coVar;
                %normalize the correlation matrix and calculate the loading matrix D
                rInv_m = zeros(length(newtnr),length(newtnr));
                for i=1:length(newtnr)
                    rInv_m(i,i) = 1.0/sqrt(coVar(i,i));
                end
                instFwdCorrTestMatrix = zeros(length(newtnr),length(newtnr));
                instFwdCorrTestMatrix = rInv_m*instFwdCovarTestMatrix*rInv_m';

            end
            
        end
        
        function InstSwapFwdCorrTest = InsSwapFwdCorrTest(ch,tx,tnr1,tnr2)
            sizeN = ch.numOfFactor;
            plambda =cell(sizeN,1);
            for i=1:sizeN
                plambda{i,1}= ch.lambda{i,1};
            end
            sigma_f = zeros(sizeN,sizeN);
            tenor = ch.lambda{1,1}.tenor;
            tindex = min(find(tx<=tenor));
            
            for j=1:sizeN
                for k=1:sizeN
                    sigma_f(j,k)=plambda{j,1}.quote(tindex)*ch.rho(j,k);
                end
            end
                
            eta = ch.inv_H_tau*sigma_f;
            eta2 = eta*eta';
            
            KappaTau1 = zeros(sizeN,1);
            KappaTau2 = zeros(sizeN,1);
            
            tau = 0.25;
            p_nN1 = ch.PV01(tx,tx+tnr1,tau);
            df_Tn1 = ch.DF(tx);
            df_TN1 = ch.DF(tx + tnr1);
            y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
            c_nN1 = ch.C_nN(tx,tx + tnr1,tau,p_nN1,y_nN1);
            
            p_nN2 = ch.PV01(tx,tx+tnr2,tau);
            df_Tn2 = ch.DF(tx);
            df_TN2 = ch.DF(tx + tnr2);
            y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
            c_nN2 = ch.C_nN(tx,tx + tnr2,tau,p_nN2,y_nN2);
            
            for i=1:sizeN
                KappaTau1(i) = c_nN1(i)*exp(-ch.kappa(i)*tnr1);
                KappaTau2(i) = c_nN2(i)*exp(-ch.kappa(i)*tnr2);
            end
            
            
            covar = 0.0;
            var1 = 0;
            var2 = 0;
            var1 =  KappaTau1'*eta2*KappaTau1;
            var2 =  KappaTau2'*eta2*KappaTau2;
            covar =  KappaTau1'*eta2*KappaTau2;
            
            InstSwapFwdCorrTest = covar/sqrt(var1*var2);
        end
        
        function spreadInfo = SpreadInfo(ch,tx,tnr1,tnr2)
            tau = 0.25;
            cmsSpreadDigitalArg =  ch.CMSSpreadDigitalArg(tx,tx+1.0,tnr1,tnr2,tau);
            spreadInfo = cmsSpreadDigitalArg;
            spreadInfo.normSigma = cmsSpreadDigitalArg.sigma/sqrt(tx);
        end
        
        function impCorrel = ImpCorrel(ch,tx,tnr1,tnr2)
            tau = 0.25;
            pv01_1 = ch.PV01(tx,tx+tnr1,tau);
            fsr_1  = ch.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = ch.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = ch.PV01(tx,tx+tnr2,tau);
            fsr_2  = ch.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = ch.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            variances = ch.LocalVariance(0,tx);
            var1 = 0;
            var2 = 0;
            coVar = 0;
            sizeN = ch.numOfFactor;
            for j=1:sizeN
                for k=1:sizeN
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                    var2 = var2 + c_nN_2(j)*c_nN_2(k)*variances(j,k);
                    coVar = coVar + c_nN_1(j)*c_nN_2(k)*variances(j,k);
                end
            end
            impCorrel = coVar/sqrt(var1)/sqrt(var2);
        end
        
        function impVariance = ImpVariance(ch,tx,tnr1)
            tau = 0.25;
            pv01_1 = ch.PV01(tx,tx+tnr1,tau);
            fsr_1  = ch.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = ch.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            variances = ch.LocalVariance(0,tx);
            var1 = 0;
            
            sizeN = ch.numOfFactor;
            for j=1:sizeN
                for k=1:sizeN
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                end
            end
            impVariance = var1;
        end
        
        function impCovar = ImpCovar(ch,tx,tnr1,tnr2)
            tau = 0.25;
            pv01_1 = ch.PV01(tx,tx+tnr1,tau);
            fsr_1  = ch.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = ch.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = ch.PV01(tx,tx+tnr2,tau);
            fsr_2  = ch.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = ch.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            variances = ch.LocalVariance(0,tx);
            var1 = 0;
            var2 = 0;
            coVar = 0;
            
            sizeN = ch.numOfFactor;
            for j=1:sizeN
                for k=1:sizeN
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                    var2 = var2 + c_nN_2(j)*c_nN_2(k)*variances(j,k);
                    coVar = coVar + c_nN_1(j)*c_nN_2(k)*variances(j,k);
                end
            end
            impCovar = coVar;
        end
        
        function out = RebonatoFwdCorr(ch,tx,tnr1,tnr2)
           rho_inf = 0.7;
           rho_decay = 0.15;
           out =  rho_inf + (1-rho_inf)*exp(-rho_decay*abs(tnr1-tnr2));
        end
        
        function out = RebonatoFwdCorr3(ch,tx,tnr1,tnr2)
           rho_inf = 0.4;
           rho_decay = 0.15;
           rho_alpha = 0.1;
           beta = rho_decay*exp(-rho_alpha*min(tnr1-tx,tnr2-tx));
           out =  rho_inf + (1-rho_inf)*exp(-beta*abs(tnr1-tnr2));
        end
        
        function out = TargetFunctionATMSwaptionAndCapBootStrapLevMar(ch,tvar,idxNow,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            chSwaptionPrice = zeros(length(tenor),1);
            
            capExpiry = params.capExpiry;
            blackCapPrice = params.blackCapPrice;
            blackCapStrike = params.blackCapStrike;
            
            useCap = false;
            if expiry(idxNow) <= capExpiry(length(capExpiry))
                if capExpiry(idxNow) == expiry(idxNow)
                    useCap = true;
                end
            end
            sizeN = ch.numOfFactor;
            for j=1:sizeN
                ch.lambda{j,1}.quote(idxNow) = tvar(j);
            end
            
            out= zeros(length(tenor)+ 1,1);
            for k=1:length(tenor)
                chSwaptionPrice(k) = ch.ATMSwaption(expiry(idxNow),tenor(k));
                out(k) = chSwaptionPrice(k)/blackSwaptionPrice(idxNow,k)-1.0;
            end
            
            if useCap
                chCapPrice = ch.Cap(capExpiry(idxNow),blackCapStrike(idxNow),0.25);
                out(length(tenor)+1) = chCapPrice/blackCapPrice(idxNow)-1.0;
            end

        end
        
        function out = TargetFunctionATMSwaptionAndCapGlobalLevMar(ch,tvar,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            chSwaptionPrice = zeros(length(expiry),length(tenor));
            
            capExpiry = params.capExpiry;
            blackCapPrice = params.blackCapPrice;
            blackCapStrike = params.blackCapStrike;
            chCapPrice = zeros(length(capExpiry),1);
            
            sizeN = ch.numOfFactor;
            alphaSize = length(ch.lambda{1,1}.tenor);
            for i=1:sizeN
                for j=1:alphaSize
                    ch.lambda{i,1}.quote(j) = tvar((i-1)*alphaSize +j);
                end
            end
            
            sizeOut = length(expiry) * length(tenor);
            sizeOut = sizeOut + length(capExpiry);
            
            out= zeros(sizeOut,1);
            for j=1:length(expiry)
                for k=1:length(tenor)
                    chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                    out((j-1)*length(tenor) + k) = chSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0;
                end
            end
            
            for j=1:length(capExpiry)
                chCapPrice(j) = ch.Cap(capExpiry(j),blackCapStrike(j),0.25);
                out(length(expiry)*length(tenor) + j) = chCapPrice(j)/blackCapPrice(j) - 1.0;
            end
        end 
        
        function out = TargetFunctionATMSwaptionBootStrapLevMar(ch,tvar,idxNow,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            chSwaptionPrice = zeros(length(tenor),1);
            
            sizeN = ch.numOfFactor;
            for j=1:sizeN
                ch.lambda{j,1}.quote(idxNow) = tvar(j);
            end
            
            out= zeros(sizeN,1);
            for k=1:length(tenor)
                chSwaptionPrice(k) = ch.ATMSwaption(expiry(idxNow),tenor(k));
                out(k) = chSwaptionPrice(k)/blackSwaptionPrice(idxNow,k)-1.0;
            end

        end
        
        function out = TargetFunctionATMSwaptionBootStrapLevMarNormalVol(ch,tvar,idxNow,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionNormalVol = params.blackSwaptionNormalVol;
            chATMSwaptionNormalVol = zeros(1,length(tenor));
            
            sizeN = ch.numOfFactor;
            for j=1:sizeN
                ch.lambda{j,1}.quote(idxNow) = tvar(j);
            end
            
            out= zeros(sizeN,1);
            for k=1:length(tenor)
                chATMSwaptionNormalVol(k) = ch.ATMSwaptionNormalVol(expiry(idxNow),tenor(k));
                out(k) = chATMSwaptionNormalVol(k)/blackSwaptionNormalVol(idxNow,k)-1.0;
            end

        end
        
        function out = TargetFunctionATMSwaptionBootStrapLevMarNormalVariance(ch,tvar,idxNow,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionNormalVol = params.blackSwaptionNormalVol;
            chATMSwaptionNormalVol = zeros(1,length(tenor));
            
            sizeN = ch.numOfFactor;
            for j=1:sizeN
                ch.lambda{j,1}.quote(idxNow) = tvar(j);
            end
            
            out= zeros(sizeN,1);
            for k=1:length(tenor)
                chATMSwaptionNormalVol(k) = ch.ATMSwaptionNormalVol(expiry(idxNow),tenor(k));
                out(k) = (chATMSwaptionNormalVol(k)*chATMSwaptionNormalVol(k)*expiry(idxNow))/(blackSwaptionNormalVol(idxNow,k)*blackSwaptionNormalVol(idxNow,k)*expiry(idxNow))-1.0;
            end

        end
        
        function out = TargetFunctionATMSwaptionBootStrapLevMarAbs(ch,tvar,idxNow,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            chSwaptionPrice = zeros(length(tenor),1);
            
            sizeN = ch.numOfFactor;
            for j=1:sizeN
                ch.lambda{j,1}.quote(idxNow) = abs(tvar(j));
            end
            
            out= zeros(sizeN,1);
            for k=1:length(tenor)
                chSwaptionPrice(k) = ch.ATMSwaption(expiry(idxNow),tenor(k));
                out(k) = chSwaptionPrice(k)/blackSwaptionPrice(idxNow,k)-1.0;
            end

        end
        
        function out = TargetFunctionATMSwaptionGlobalLevMar(ch,tvar,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            chSwaptionPrice = zeros(length(expiry),length(tenor));
            
            sizeN = ch.numOfFactor;
            alphaSize = length(ch.lambda{1,1}.tenor);
            for i=1:sizeN
                for j=1:alphaSize
                    ch.lambda{i,1}.quote(j) = tvar((i-1)*alphaSize +j);
                end
            end
            
            sizeOut = length(expiry) * length(tenor);
            out= zeros(sizeOut,1);
            for j=1:length(expiry)
                for k=1:length(tenor)
                    chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                    out((j-1)*length(tenor) + k) = chSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0;
                end
            end
        end 
        
        function out = TargetFunctionATMSwaptionGlobalLevMarKappa(ch,tvar,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            chSwaptionPrice = zeros(length(expiry),length(tenor));
            
            sizeN = ch.numOfFactor;
            alphaSize = length(ch.lambda{1,1}.tenor);
            for i=1:sizeN
                for j=1:alphaSize
                    ch.lambda{i,1}.quote(j) = tvar((i-1)*alphaSize +j);
                end
            end
            
            for i=1:sizeN
                ch.kappa(i) = tvar(sizeN*alphaSize + i);
            end
            
            sizeOut = length(expiry) * length(tenor);
            out= zeros(sizeOut,1);
            for j=1:length(expiry)
                for k=1:length(tenor)
                    chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                    out((j-1)*length(tenor) + k) = chSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0;
                end
            end
        end 

        function out = TargetFunctionATMSwaptionGlobal(ch,tvar,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            chSwaptionPrice = zeros(1,size(tenor,2));
            
            sizeN = ch.numOfFactor;
            alphaSize = length(ch.lambda{1,1}.tenor);
            for i=1:sizeN
                for j=1:alphaSize
                    ch.lambda{i,1}.quote(j) = tvar((i-1)*alphaSize +j);
                end
            end
            
            out= 0;
            for j=1:length(expiry)
                for k=1:length(tenor)
                    chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                    out = out + (chSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0)*(chSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0)*10000;
                end
            end
        end   
        
        function out = TargetFunctionATMSwaptionGlobalImpVol(ch,tvar,params)
            a = 0.001;
            b = 10;
            Tol = 1e-10;
            MaxIter = 1000;

            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionImpVol = params.blackSwaptionImpVol;
            chSwaptionPrice = zeros(1,size(tenor,2));
            
            sizeN = ch.numOfFactor;
            alphaSize = length(ch.lambda{1,1}.tenor);
            for i=1:sizeN
                for j=1:alphaSize
                    ch.lambda{i,1}.quote(j) = tvar((i-1)*alphaSize +j);
                end
            end
            
            out= 0;
            for j=1:length(expiry)
                for k=1:length(tenor)
                    chATMSwaptionImpVol(j,k) = ch.ATMSwaptionImpVol(expiry(j),tenor(k));
                    out = out + (chATMSwaptionImpVol(j,k)- blackSwaptionImpVol(j,k))*(chATMSwaptionImpVol(j,k)- blackSwaptionImpVol(j,k))*10000;
                end
            end
        end   
        
        function out = TargetFunctionATMSwaptionGlobalNormalVol(ch,tvar,params)
            
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionNormalVol = params.blackSwaptionNormalVol;
            chATMSwaptionNormalVol = zeros(length(expiry),length(tenor));
            
            sizeN = ch.numOfFactor;
            alphaSize = length(ch.lambda{1,1}.tenor);
            for i=1:sizeN
                for j=1:alphaSize
                    ch.lambda{i,1}.quote(j) = tvar((i-1)*alphaSize +j);
                end
            end
            
            out= 0;
            for j=1:length(expiry)
                for k=1:length(tenor)
                    chATMSwaptionNormalVol(j,k) = ch.ATMSwaptionNormalVol(expiry(j),tenor(k));
                    out = out + (chATMSwaptionNormalVol(j,k)/blackSwaptionNormalVol(j,k)-1.0)*(chATMSwaptionNormalVol(j,k)/blackSwaptionNormalVol(j,k)-1.0)*10000;
                end
            end
        end   
        
        function out = CalibrateToATMSwaption(ch,black,calibrationFlag)
            tic;
            
            sizeN = ch.numOfFactor;
            alphaSize = length(ch.lambda{1,1}.tenor);
            for i=1:sizeN
               for j=1:alphaSize
                   out.lambda{i,1} = ch.lambda{i,1};
               end
            end
            
          % Calculate Black ATM Swaption Prices
          
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
            
            expirySize = size(expiry,2);
            tenorSize = size(tenor,2);
            
            blackSwaptionPrice = zeros(expirySize,tenorSize);
            blackSwaptionImpVol = zeros(expirySize,tenorSize);
            blackSwaptionVega = zeros(expirySize,tenorSize);
            blackSwaptionFsr = zeros(expirySize,tenorSize);
%             blackSwaptionNormalVol = zeros(expirySize,tenorSize);
            
            for k=1:expirySize
                for j=1:tenorSize
                    blackSwaptionPrice(k,j)  = black.BlackATMSwaption(expiry(k),tenor(j));
                    blackSwaptionImpVol(k,j) = black.swaptionVol(expiry(k),tenor(j));
                    blackSwaptionFsr(k,j) = black.Fsr(expiry(k),expiry(k) + tenor(j),0.25);
                    blackSwaptionVega(k,j) = black.BlackSwaptionVegaWithVol(expiry(k),tenor(j),blackSwaptionFsr(k,j),blackSwaptionImpVol(k,j));
%                     blackSwaptionNormalVol(k,j) = black.ATMSwaptionNormalVol(expiry(k),tenor(j));
                end
            end
            
            params.expiry = expiry;
            params.tenor = tenor;
            params.blackSwaptionPrice = blackSwaptionPrice;
            params.blackSwaptionImpVol = blackSwaptionImpVol;
            params.blackSwaptionFsr = blackSwaptionFsr;
            params.blackSwaptionVega = blackSwaptionVega;
%             params.blackSwaptionNormalVol = blackSwaptionNormalVol;
            params.expirySize = expirySize;
            params.tenorSize =  tenorSize;
            
            params.swaption_calibration_yn = ch.swaption_calibration_yn;
            
            out.blackSwaptionPrice = blackSwaptionPrice;
            out.blackSwaptionVega = blackSwaptionVega;
            out.blackSwaptionFsr = blackSwaptionFsr;
            
            out.chSwaptionPrice = zeros(expirySize,tenorSize);
            out.relDiff = zeros(expirySize,tenorSize);
            out.relVolDiff = zeros(expirySize,tenorSize);
            out.rmseTotal=0.0;
            out.rmseVolTotal = 0.0;
            
            out.impVol2Total = 0.0;
            out.normalVol2Total = 0.0;
            out.normalVarianceTotal = 0.0;
            
            calibrationType = 'bootstrap';
            if nargin < 3
                calibrationType = 'bootstrap'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;
            tvar = [];
            
            switch calibrationType
                case 'bootstrapLevMar'
                    tvar = zeros(sizeN,1);
                    for i=1:expirySize
                        idxNow = i;
                        for j=1:sizeN
                             tvar(j) = ch.lambda{j,1}.quote(i);
                             lb(j) = 0.0001;
                             ub(j) = 10.0;
                        end
                        
                        targetfun = @(tvar) TargetFunctionATMSwaptionBootStrapLevMar(ch,tvar,idxNow,params);
                        options=optimset('Algorithm','levenberg-marquardt');
%                         options=optimset('Algorithm','trust-region-reflective');
                        [x, fval] = lsqnonlin(targetfun,tvar,lb,ub,options);
                        
                        for j=1:sizeN
                             out.lambda{j,1}.quote(i) = x(j); 
                             ch.lambda{j,1}.quote(i) = x(j); 
                        end
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.calibrationTime = toc;
                case 'bootstrapLevMarNormalVol'
                    tvar = zeros(sizeN,1);
                    for i=1:expirySize
                        idxNow = i;
                        for j=1:sizeN
                             tvar(j) = ch.lambda{j,1}.quote(i);
                             lb(j) = 0.0001;
                             ub(j) = 10.0;
                        end
                        
                        targetfun = @(tvar) TargetFunctionATMSwaptionBootStrapLevMarNormalVol(ch,tvar,idxNow,params);
                        options=optimset('Algorithm','levenberg-marquardt');
%                         options=optimset('Algorithm','trust-region-reflective');
                        [x, fval] = lsqnonlin(targetfun,tvar,lb,ub,options);
                        
                        for j=1:sizeN
                             out.lambda{j,1}.quote(i) = x(j); 
                             ch.lambda{j,1}.quote(i) = x(j); 
                        end
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.blackSwaptionImpVol(j,k) = blackSwaptionImpVol(j,k);
                            out.blackSwaptionNormalVol(j,k) = blackSwaptionNormalVol(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.chSwaptionImpVol(j,k) = ch.ATMSwaptionImpVol(expiry(j),tenor(k));
                            out.chSwaptionNormalVol(j,k) = ch.ATMSwaptionNormalVol(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.ImpVolDiff(j,k) = out.chSwaptionImpVol(j,k) - out.blackSwaptionImpVol(j,k);
                            out.NormalVolDiff(j,k) = out.chSwaptionNormalVol(j,k)/out.blackSwaptionNormalVol(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            out.impVol2Total = out.impVol2Total + out.ImpVolDiff(j,k)*out.ImpVolDiff(j,k);
                            out.normalVol2Total = out.normalVol2Total + out.NormalVolDiff(j,k)*out.NormalVolDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.impVol2Total = sqrt(out.impVol2Total/N);
                    out.normalVol2Total = sqrt(out.normalVol2Total/N);
                    out.calibrationTime = toc;
                    
                case 'bootstrapLevMarNormalVariance'
                    tvar = zeros(sizeN,1);
                    for i=1:expirySize
                        idxNow = i;
                        for j=1:sizeN
                             tvar(j) = ch.lambda{j,1}.quote(i);
                             lb(j) = 0.0001;
                             ub(j) = 10.0;
                        end
                        
                        targetfun = @(tvar) TargetFunctionATMSwaptionBootStrapLevMarNormalVariance(ch,tvar,idxNow,params);
                        options=optimset('Algorithm','levenberg-marquardt');
%                         options=optimset('Algorithm','trust-region-reflective');
                        [x, fval] = lsqnonlin(targetfun,tvar,lb,ub,options);
                        
                        for j=1:sizeN
                             out.lambda{j,1}.quote(i) = x(j); 
                             ch.lambda{j,1}.quote(i) = x(j); 
                        end
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.blackSwaptionImpVol(j,k) = blackSwaptionImpVol(j,k);
                            out.blackSwaptionNormalVol(j,k) = blackSwaptionNormalVol(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.chSwaptionImpVol(j,k) = ch.ATMSwaptionImpVol(expiry(j),tenor(k));
                            out.chSwaptionNormalVol(j,k) = ch.ATMSwaptionNormalVol(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.ImpVolDiff(j,k) = out.chSwaptionImpVol(j,k) - out.blackSwaptionImpVol(j,k);
                            out.NormalVolDiff(j,k) = out.chSwaptionNormalVol(j,k)/out.blackSwaptionNormalVol(j,k)-1;
                            out.NormalVarianceDiff(j,k) = (out.chSwaptionNormalVol(j,k)*out.chSwaptionNormalVol(j,k)*expiry(j))/(out.blackSwaptionNormalVol(j,k)*out.blackSwaptionNormalVol(j,k)*expiry(j))-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            out.impVol2Total = out.impVol2Total + out.ImpVolDiff(j,k)*out.ImpVolDiff(j,k);
                            out.normalVol2Total = out.normalVol2Total + out.NormalVolDiff(j,k)*out.NormalVolDiff(j,k);
                            out.normalVarianceTotal = out.normalVarianceTotal + out.NormalVarianceDiff(j,k)*out.NormalVarianceDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.impVol2Total = sqrt(out.impVol2Total/N);
                    out.normalVol2Total = sqrt(out.normalVol2Total/N);
                    out.normalVarianceTotal = sqrt(out.normalVarianceTotal/N);
                    out.calibrationTime = toc;
                    
                case 'globalLevMar'
                    for i=1:sizeN
                        for j=1:alphaSize
                             tvar((i-1)*alphaSize +j) = ch.lambda{i,1}.quote(j);
                             lb((i-1)*alphaSize +j) = 0.0001;
                             ub((i-1)*alphaSize +j) = 10.0;
                        end
                    end
                    
                       
                    targetfun = @(tvar) TargetFunctionATMSwaptionGlobalLevMar(ch,tvar,params);
%                     options=optimset('Algorithm','levenberg-marquardt');
                    options=optimset('Algorithm','levenberg-marquardt','MaxFunEvals',700);
                    [x, fval] = lsqnonlin(targetfun,tvar,lb,ub,options);
                    
                    for i=1:sizeN
                        for j=1:alphaSize
                             out.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                             ch.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                        end
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.calibrationTime = toc;
                
                 case 'dummy'
                    
                    for i=1:sizeN
                        for j=1:alphaSize
                             out.lambda{i,1}.quote(j) = ch.lambda{i,1}.quote(j); 
                        end
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.calibrationTime = toc;
                    
                    
                case 'globalLevMarKappa'
                    for i=1:sizeN
                        for j=1:alphaSize
                             tvar((i-1)*alphaSize +j) = ch.lambda{i,1}.quote(j);
                             lb((i-1)*alphaSize +j) = 0.0001;
                             ub((i-1)*alphaSize +j) = 10.0;
                        end
                    end
                    
                    for i=1:sizeN
                        tvar(sizeN*alphaSize + i) = ch.kappa(i);
                        lb(sizeN*alphaSize + i) = 0.0001;
                        ub(sizeN*alphaSize + i) = 10.0;
                    end
                       
                    targetfun = @(tvar) TargetFunctionATMSwaptionGlobalLevMarKappa(ch,tvar,params);
                    options=optimset('Algorithm','levenberg-marquardt');
                    [x, fval] = lsqnonlin(targetfun,tvar,lb,ub,options);
                    
                    for i=1:sizeN
                        for j=1:alphaSize
                             out.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                             ch.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                        end
                    end
                    
                    for i=1:sizeN
                        out.kappa(i) = x(sizeN*alphaSize+i);
                        ch.kappa(i)  = x(sizeN*alphaSize+i);
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);    
                    
                case 'global'
                    optParams.params = params;
                    optParams.modelParamsSize = sizeN * alphaSize;
                    optParams.numOfEquation = params.expirySize*params.tenorSize + ...  % target cap & swaption
                                              sizeN*(alphaSize -2);   % tikhonov regularization gamma constraint
                    
                    tvar = zeros(sizeN*alphaSize,1);
                    for i=1:sizeN
                        for j=1:alphaSize
                             tvar((i-1)*alphaSize +j) = ch.lambda{i,1}.quote(j);
                        end
                    end
                    
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
                    
                    x=  zeros(optParams.numOfEquation,1);
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionCheyetteATMSwaptionSurfaceGlobal',tvar,x, 500, options,'unc',ch,optParams);
                    toc

%                     targetfun = @(tvar) TargetFunctionATMSwaptionGlobal(ch,tvar,params);
% %                     options=optimset('Algorithm','interior-point');
%                      options=optimset('Algorithm','trust-region-reflective');
%                     [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);
                    
                    for i=1:sizeN
                        for j=1:alphaSize
                             out.lambda{i,1}.quote(j) = max(abs(popt((i-1)*alphaSize +j)),0.0001); 
                             ch.lambda{i,1}.quote(j) = max(abs(popt((i-1)*alphaSize +j)),0.0001); 
                        end
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                 case 'globalVegaWeight'
                    optParams.params = params;
                    optParams.modelParamsSize = sizeN * alphaSize;
                    optParams.numOfEquation = params.expirySize*params.tenorSize + ...  % target cap & swaption
                                              sizeN*(alphaSize -2);   % tikhonov regularization gamma constraint
                    
                    tvar = zeros(sizeN*alphaSize,1);
                    for i=1:sizeN
                        for j=1:alphaSize
                             tvar((i-1)*alphaSize +j) = ch.lambda{i,1}.quote(j);
                        end
                    end
                    
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
                    
                    x=  zeros(optParams.numOfEquation,1);
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionCheyetteATMSwaptionSurfaceGlobalVegaWeight',tvar,x, 500, options,'unc',ch,optParams);
                    toc

%                     targetfun = @(tvar) TargetFunctionATMSwaptionGlobal(ch,tvar,params);
% %                     options=optimset('Algorithm','interior-point');
%                      options=optimset('Algorithm','trust-region-reflective');
%                     [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);
                    
                    for i=1:sizeN
                        for j=1:alphaSize
                             out.lambda{i,1}.quote(j) = max(abs(popt((i-1)*alphaSize +j)),0.0001); 
                             ch.lambda{i,1}.quote(j) = max(abs(popt((i-1)*alphaSize +j)),0.0001); 
                        end
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.relVolDiff(j,k) = (out.chSwaptionPrice(j,k) - out.blackATMSwaptionPrice(j,k))/out.blackSwaptionVega(j,k);
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            out.rmseVolTotal = out.rmseVolTotal + out.relVolDiff(j,k)*out.relVolDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.rmseVolTotal = sqrt(out.rmseVolTotal/N);
                    
                 case 'globalFmin'
                    tvar = zeros(sizeN*alphaSize,1);
                    for i=1:sizeN
                        for j=1:alphaSize
                             tvar((i-1)*alphaSize +j) = ch.lambda{i,1}.quote(j);
                             lb((i-1)*alphaSize +j) = 0.01;
                             ub((i-1)*alphaSize +j) = 10.0;
                        end
                    end
                    
                       
                    targetfun = @(tvar) TargetFunctionATMSwaptionGlobal(ch,tvar,params);
%                     options=optimset('Algorithm','interior-point');
                     options=optimset('Algorithm','trust-region-reflective');
                    [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);
                    
                    for i=1:sizeN
                        for j=1:alphaSize
                             out.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                             ch.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                        end
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                 case 'globalImpVol'
                    for i=1:sizeN
                        for j=1:alphaSize
                             tvar((i-1)*alphaSize +j) = ch.lambda{i,1}.quote(j);
                             lb((i-1)*alphaSize +j) = 0.0001;
                             ub((i-1)*alphaSize +j) = 10.0;
                        end
                    end
                    
                       
                    targetfun = @(tvar) TargetFunctionATMSwaptionGlobalImpVol(ch,tvar,params);
%                     options=optimset('Algorithm','interior-point');
                     options=optimset('Algorithm','trust-region-reflective');
                    [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);
                    
                    for i=1:sizeN
                        for j=1:alphaSize
                             out.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                             ch.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                        end
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.blaskSwaptionImpVol(j,k) = blackSwaptionImpVol(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.chSwaptionImpVol(j,k) = ch.ATMSwaptionImpVol(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            our.volDiff(j,k) = out.chSwaptionImpVol(j,k) - out.blaskSwaptionImpVol(j,k);
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            out.vol2Total = out.vol2Total + our.volDiff(j,k)*our.volDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.vol2Total = sqrt(out.vol2Total/N);
                    
                case 'globalNormalVol'
                    for i=1:sizeN
                        for j=1:alphaSize
                             tvar((i-1)*alphaSize +j) = ch.lambda{i,1}.quote(j);
                             lb((i-1)*alphaSize +j) = 0.0001;
                             ub((i-1)*alphaSize +j) = 10.0;
                        end
                    end
                    
                       
                    targetfun = @(tvar) TargetFunctionATMSwaptionGlobalNormalVol(ch,tvar,params);
%                     options=optimset('Algorithm','interior-point');
                     options=optimset('Algorithm','trust-region-reflective');
                    [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);
                    
                    for i=1:sizeN
                        for j=1:alphaSize
                             out.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                             ch.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                        end
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.blackSwaptionImpVol(j,k) = blaskSwaptionImpVol(j,k);
                            out.blackSwaptionNormalVol(j,k) = blackSwaptionNormalVol(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.chSwaptionImpVol(j,k) = ch.ATMSwaptionImpVol(expiry(j),tenor(k));
                            out.chSwaptionNormalVol(j,k) = ch.ATMSwaptionNormalVol(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.ImpVolDiff(j,k) = out.chSwaptionImpVol(j,k) - out.blackSwaptionImpVol(j,k);
                            out.NormalVolDiff(j,k) = out.chSwaptionNormalVol(j,k)/out.blackSwaptionNormalVol(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            out.impVol2Total = out.impVol2Total + out.ImpVolDiff(j,k)*out.ImpVolDiff(j,k);
                            out.normalVol2Total = out.normalVol2Total + out.NormalVolDiff(j,k)*out.NormalVolDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.impVol2Total = sqrt(out.impVol2Total/N);
                    out.normalVol2Total = sqrt(out.normalVol2Total/N);
                
                otherwise
                    disp('unImplemented')
            end
        end
        
        function out = CalibrateToATMSwaptionAndCap(ch,black,calibrationFlag)
            tic;
            
            sizeN = ch.numOfFactor;
            alphaSize = length(ch.lambda{1,1}.tenor);
            for i=1:sizeN
               for j=1:alphaSize
                   out.lambda{i,1} = ch.lambda{i,1};
               end
            end
            
          %% Calculate Black ATM Swaption Prices
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
            
            expirySize = size(expiry,2);
            tenorSize = size(tenor,2);
            
            blackSwaptionPrice = zeros(expirySize,tenorSize);
            blackSwaptionImpVol = zeros(expirySize,tenorSize);
            for k=1:expirySize
                for j=1:tenorSize
                    blackSwaptionPrice(k,j)  = black.BlackATMSwaption(expiry(k),tenor(j));
                    blackSwaptionImpVol(k,j) = black.swaptionVol(expiry(k),tenor(j));
                end
            end
            
          %% Calculate Black ATM Cap Prices
           capVol = black.capVol;
           capVolData = black.mktData('capVol');
           capVolrawData =  capVolData.params('rawData');
            
           capExpiry = capVolrawData(:,1);
           capExpirySize = length(capExpiry);
            
           blackCapPrice = zeros(capExpirySize,1);
           blackCapImpVol = zeros(capExpirySize,1);
           blackCapStrike = zeros(capExpirySize,1);
           for k=1:capExpirySize
               blackCapStrike(k) = black.Fsr(0.25,capExpiry(k) -0.25,0.25);
               blackCapPrice(k) = black.BlackCap(capExpiry(k),blackCapStrike(k),0.25);
               blackCapImpVol(k) = black.capVol(capExpiry(k));
           end
          
          
            %%
            params.expiry = expiry;
            params.tenor = tenor;
            params.blackSwaptionPrice = blackSwaptionPrice;
            params.blackSwaptionImpVol = blackSwaptionImpVol;
            params.expirySize = expirySize;
            params.tenorSize =  tenorSize;
            
            params.capExpiry = capExpiry;
            params.blackCapPrice = blackCapPrice;
            params.blackCapStrike = blackCapStrike;
            params.blackCapImpVol = blackCapImpVol;
            params.capExpirySize = capExpirySize;
            
            out.blackSwaptionPrice = blackSwaptionPrice;
            out.chSwaptionPrice = zeros(expirySize,tenorSize);
            out.relDiff = zeros(expirySize,tenorSize);
            out.rmseTotal=0.0;
            
            out.blackCapPrice = blackCapPrice;
            out.chCapPrice = zeros(capExpirySize,1);
            out.capRelDiff = zeros(capExpirySize,1);
            
            calibrationType = 'bootstrap';
            if nargin < 3
                calibrationType = 'bootstrap'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;
            tvar = [];
            
            switch calibrationType
                case 'bootstrapLevMar'
                    tvar = zeros(sizeN,1);
                    for i=1:expirySize
                        idxNow = i;
                        for j=1:sizeN
                             tvar(j) = ch.lambda{j,1}.quote(i);
                             lb(j) = 0.0;
                             ub(j) = 10.0;
                        end
                        
                        targetfun = @(tvar) TargetFunctionATMSwaptionAndCapBootStrapLevMar(ch,tvar,idxNow,params);
                        options=optimset('Algorithm','levenberg-marquardt');
%                         options=optimset('Algorithm','trust-region-reflective');
                        [x, fval] = lsqnonlin(targetfun,tvar,lb,ub,options);
                        
                        for j=1:sizeN
                             out.lambda{j,1}.quote(i) = x(j); 
                             ch.lambda{j,1}.quote(i) = x(j); 
                        end
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                    
                    for j=1:capExpirySize
                        out.chCapPrice(j) = ch.Cap(capExpiry(j),blackCapStrike(j),0.25);
                        out.capRelDiff(j) = out.chCapPrice(j)/out.blackCapPrice(j) -1.0;
                        out.rmseTotal = out.rmseTotal +  out.capRelDiff(j)* out.capRelDiff(j);
                        N=N+1;
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                out.calibrationTime = toc;
                
                case 'globalLevMar'
                    for i=1:sizeN
                        for j=1:alphaSize
                             tvar((i-1)*alphaSize +j) = ch.lambda{i,1}.quote(j);
                             lb((i-1)*alphaSize +j) = 0.0;
                             ub((i-1)*alphaSize +j) = 10.0;
                        end
                    end
                    
                       
                    targetfun = @(tvar) TargetFunctionATMSwaptionAndCapGlobalLevMar(ch,tvar,params);
                    options=optimset('Algorithm','levenberg-marquardt');
                    [x, fval] = lsqnonlin(targetfun,tvar,lb,ub,options);
                    
                    for i=1:sizeN
                        for j=1:alphaSize
                             out.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                             ch.lambda{i,1}.quote(j) = x((i-1)*alphaSize +j); 
                        end
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.chSwaptionPrice(j,k) = ch.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.chSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                    
                    for j=1:capExpirySize
                        out.chCapPrice(j) = ch.Cap(capExpiry(j),blackCapStrike(j),0.25);
                        out.capRelDiff(j) = out.chCapPrice(j)/out.blackCapPrice(j) -1.0;
                        out.rmseTotal = out.rmseTotal +  out.capRelDiff(j)* out.capRelDiff(j);
                        N=N+1;
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.calibrationTime = toc;
                   
                otherwise
                    disp('unImplemented')
            end
        end    
    end
    
end



In [ ]:
function value = chebyshev (x , c , a , b)
    y = (2* x -a -b) /(b -a) ;
    y2 = 2* y ;
    b0 = c (12) ;
    b1 = 0.0;
    b2 = 0.0;
    b2 = b1 ; b1 = b0 ; b0 = y2 * b1 - b2 + c (11) ;
    b2 = b1 ; b1 = b0 ; b0 = y2 * b1 - b2 + c (10) ;
    b2 = b1 ; b1 = b0 ; b0 = y2 * b1 - b2 + c (9) ;
    b2 = b1 ; b1 = b0 ; b0 = y2 * b1 - b2 + c (8) ;
    b2 = b1 ; b1 = b0 ; b0 = y2 * b1 - b2 + c (7) ;
    b2 = b1 ; b1 = b0 ; b0 = y2 * b1 - b2 + c (6) ;
    b2 = b1 ; b1 = b0 ; b0 = y2 * b1 - b2 + c (5) ;
    b2 = b1 ; b1 = b0 ; b0 = y2 * b1 - b2 + c (4) ;
    b2 = b1 ; b1 = b0 ; b0 = y2 * b1 - b2 + c (3) ;
    b2 = b1 ; b1 = b0 ; b0 = y2 * b1 - b2 + c (2) ;
    value = y* b0 - b1 +0.5* c (1) ;
end

In [ ]:
classdef CDRangeAccrualSwap < Security
    %RangeAccrual is the SuperClass of all IRModel
    % properties: zeroCurve 
    % methods : DF, FWD,PV01,FSR
    properties(SetAccess = public)
        nominal
        coupon
        couponFreq
        tenor
        lower
        upper
        rangeDays
        callYN
        schedule
        couponSchedule
        callSchedule
        manualScheduleYN
        callStrike;
        
        %floatingLeg
        floatingSchedule
        floatingYN
        floatingSpread
        floatingIndex
        
        allFixings
        
    end
    
    methods
        % constructor
        function rangeAccrual = CDRangeAccrualSwap(params,swapParams)
                rangeAccrual.payCcy = params.payCcy;
                rangeAccrual.startDate = params.startDate;
                rangeAccrual.maturity = params.maturity;
                
                %rangeAccrual.maturityDate = params.startDate.AddDate(params.maturity,'year');
                rangeAccrual.nominal = params.params('nominal');
                rangeAccrual.coupon = params.params('coupon');
                rangeAccrual.couponFreq= params.params('couponFreq');
                rangeAccrual.tenor = params.params('tenor');
                rangeAccrual.lower = params.params('lower');
                rangeAccrual.upper = params.params('upper');
                rangeAccrual.rangeDays = params.params('rangeDays');
                rangeAccrual.callYN = params.params('callYN');
                rangeAccrual.manualScheduleYN = params.params('manualScheduleYN');
                rangeAccrual.callStrike = params.params('callStrike');
                
                if rangeAccrual.manualScheduleYN
                    rangeAccrual.couponSchedule = rangeAccrual.ManualScheduleGenerate(params.params('couponScheduleInt'));
                    rangeAccrual.callSchedule = rangeAccrual.ManualScheduleGenerate(params.params('callScheduleInt'));
                    %floatingSchedue
                    rangeAccrual.floatingSchedule = rangeAccrual.ManualScheduleGenerate(swapParams.params('floatingScheduleInt'));
                else
                    maturityDate = params.startDate.AddDate(params.maturity,'year');
                    rangeAccrual.schedule = rangeAccrual.ScheduleGenerate(rangeAccrual.startDate,maturityDate,rangeAccrual.couponFreq);
                end
                
                %floatingLeg
                rangeAccrual.floatingYN = swapParams.params('floatingYN');
                rangeAccrual.floatingSpread = swapParams.params('floatingSpread');
                rangeAccrual.floatingIndex = swapParams.params('floatingIndex');
                
                rangeAccrual.allFixings = swapParams.params('allFixings');
        end
        
        function out = ManualScheduleGenerate(rangeAccrual,scheduleInt)
            scheduleSize = size(scheduleInt,1);
            out.numOfPeriod = scheduleSize;
            for i=1:scheduleSize
                out.resetDates(i) = H_Date(scheduleInt(i,1));
                out.startDates(i) = H_Date(scheduleInt(i,2));
                out.endDates(i) = H_Date(scheduleInt(i,3));
                out.payDates(i) = H_Date(scheduleInt(i,4));
                out.dayCount(i) = out.endDates(i).DateDiff(out.startDates(i));
                out.dayCountFraction(i) = out.dayCount(i)/ 365.0;
            end
            
            
        end
        
        function out = ScheduleGenerate(rangeAccrual,initialDate,maturityDate,freq)
            schedule = ScheduleGenerate(initialDate,maturityDate,freq);
            out.startDates= schedule.startDates;
            out.resetDates = schedule.resetDates;
            out.endDates = schedule.endDates;
            out.payDates = schedule.payDates;
            out.dayCount = schedule.dayCount;
            out.dayCountFraction = schedule.dayCountFraction;
            out.numOfPeriod = schedule.numOfPeriod;
        end
        
        function out = numMethodTimeStepsGenerate(rangeAccrual,valueDate,schedule,mcOneTimeStep)
            
            periodCount = schedule.numOfPeriod;
            out.timeStepsPerPeriod =  cell(periodCount,1);
            
            out.startTimeIdx = zeros(periodCount,1);
            out.endM1TimeIdx = zeros(periodCount,1);
            
            out.numOfTimeSteps = zeros(periodCount,1);
            out.totalTimeSteps = [];
            out.totalTimeStepsSize = 0;
            numOfAlivePeriod = 0;
            
            for i=1:periodCount
                timeSteps = [];
                if DateDiff(valueDate,schedule.payDates(i)) >= 0
                    out.timeStepsPerPeriod(i) = timeSteps;
                    continue;
                end
                
                numOfAlivePeriod = numOfAlivePeriod + 1;
                if DateDiff(valueDate,schedule.startDates(i)) >= 0
                    startDate = valueDate;
                    endDate = schedule.endDates(i);
                    % we exclude end date from the 
                    endDate = endDate.AddDate(-1,'day');
                    
                    timeSteps = [];
                    % startDate should be included
                    startTime = startDate.DateDiff(valueDate);
                    timeSteps = [timeSteps; startTime];
                    
                    while 1
                        endTime = endDate.DateDiff(valueDate);
                        timeSteps = [timeSteps; endTime];
                        endDate = endDate.AddDate(-1*mcOneTimeStep,'day');
                        if endDate.DateDiff(startDate) < 0 break;end
                    end
                    timeSteps = sort(unique(timeSteps),'ascend');
                    %out.timeStepsPerPeriod {i,1}.timeSteps = [];
                    out.timeStepsPerPeriod{i,1}.timeSteps = timeSteps;
                    out.numOfTimeSteps(i) = length(timeSteps);
                    out.totalTimeSteps = [out.totalTimeSteps; timeSteps];
                    
                    % 1st period
                    out.startTimeIdx(i) = 1;
                    out.endM1TimeIdx(i) = length(timeSteps);
                    
                else
                    startDate = schedule.startDates(i);
                    endDate = schedule.endDates(i);
                    % we exclude end date from the 
                    endDate = endDate.AddDate(-1,'day');
                    
                    timeSteps = [];
                    % startDate should be included
                    startTime = startDate.DateDiff(valueDate);
                    timeSteps = [timeSteps; startTime];
                    
                    while 1
                        endTime = endDate.DateDiff(valueDate);
                        timeSteps = [timeSteps; endTime];
                        
                        endDate = endDate.AddDate(-1*mcOneTimeStep,'day');
                        if endDate.DateDiff(startDate) < 0 break;end
                    end
                    timeSteps = sort(unique(timeSteps),'ascend');
                    out.timeStepsPerPeriod{i,1}.timeSteps = timeSteps;
                    out.numOfTimeSteps(i) = length(timeSteps);
                    out.totalTimeSteps = [out.totalTimeSteps; timeSteps];
                    
                    % 1st period
                    out.startTimeIdx(i) = out.endM1TimeIdx(i-1) + 1;
                    out.endM1TimeIdx(i) = out.endM1TimeIdx(i-1) + length(timeSteps);
                    
                end
            end
            
            % endDate of the last period (= Security's maturityDate should
            % be included
            
            endTime = DateDiff(schedule.endDates(periodCount),valueDate);
            out.totalTimeSteps = [out.totalTimeSteps;endTime];
            out.totalTimeSteps = sort(unique(out.totalTimeSteps),'ascend');
            out.totalTimeStepsSize = length(out.totalTimeSteps);
            
            out.periodCount = periodCount;
            out.numOfAlivePeriod = numOfAlivePeriod;
        end
        
        function out = generateModelStates(rangeAccrual,valueDate,schedule,numMethodInfo,model)
            
            pathSize = numMethodInfo.pathSize;
            numOfFactors = numMethodInfo.numOfFactors;
            totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
            totalTimeSteps = numMethodInfo.totalTimeSteps;
            Z = numMethodInfo.Z;
            modelStates = zeros(numOfFactors*totalTimeStepsSize,pathSize);
            
            currentStep = totalTimeSteps(1);
            for i=1:totalTimeStepsSize-1
                nextStep = totalTimeSteps(i+1);
                sqrtDt = sqrt((nextStep-currentStep)/365.0);
                stateLocalVariance = model.stateLocalVariance(currentStep,nextStep);
                L = chol(stateLocalVariance);
                prevModelStates = modelStates(numOfFactors*(i-1)+1:numOfFactors*(i-1)+2,:);
                dW = Z(numOfFactors*i+1:numOfFactors*i+2,:);
                modelStates(numOfFactors*i+1:numOfFactors*i+2,:) = prevModelStates + L*dW; 
                currentStep = nextStep;
            end
            
            out = modelStates;
        end
        
        function out = computePriceMCForward(rangeAccrual,valueDate,model)
            schedule = rangeAccrual.schedule;
            
            % numMethodTimeSteps Generate
            mcOneTimeStep = 1;
            outInfo = rangeAccrual.numMethodTimeStepsGenerate(valueDate,schedule,mcOneTimeStep);
            
            numMethodInfo.timeStepsInfo = outInfo;
            
            % mc random number init
            %pathSize = 2^16;
            pathSize = 5000;
            
            %pathSize = 2^10;
            
            rng('default');
            rng(0);
            
            numOfFactors = 2;
            
            Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
            Z=[Z,-Z];
            
            numMethodInfo.pathSize = pathSize;
            numMethodInfo.numOfFactors = numOfFactors;
            numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
            numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
            numMethodInfo.Z = Z;
            
            % numeraire info
            numMatTime = numMethodInfo.totalTimeSteps(end);
            dfNumMat = model.DF(numMatTime/365.0);
            
            %generate model states for the given timeSteps
            modelStates = rangeAccrual.generateModelStates(valueDate,schedule,numMethodInfo,model);
            
            % induct backward and calculate cashflow
            
            
            periodCount = schedule.numOfPeriod;
            
            nonCall.npv = 0;
            
            nonCall.cashflow_e = zeros(periodCount + 1,1);
            nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
            nonCall.rangeProb_e = zeros(periodCount + 1,1);
            
            nonCall.df_e = zeros(periodCount + 1,1);
            nonCall.df_c = zeros(periodCount + 1,1);
            
            % sum of cashflow_npv
            nonCall.payoff = zeros(1,pathSize);
            
            nonCall.payoffStateVectors_cashflow = zeros(periodCount+1,pathSize);
            nonCall.payoffStateVectors_cashflow_npv = zeros(periodCount+1,pathSize);
            nonCall.payoffStateVector_df = zeros(periodCount + 1,pathSize);
            
            payoffStateVectors_cashflow = zeros(1, pathSize);
            payoffStateVectors_cashflow_npv = zeros(1, pathSize);
            payoffStateVectors_df = zeros(1,pathSize);
            
            % we calculate cashflow backward from maturityDate to valueDate
            % rangeAccrual coupons are added at the startDate of the Period
            % At maturity
            
            nominal = rangeAccrual.nominal;
            coupon = rangeAccrual.coupon;
            
            currentTime = numMethodInfo.totalTimeSteps(end);
            totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
            currentTimeIdx = totalTimeStepsSize;
            
            modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
            payoffStateVectors_cashflow = nominal*ones(1,pathSize);
            nonCall.payoffStateVectors_cashflow(end,:) = payoffStateVectors_cashflow;
            
            payoffStateVectors_cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,payoffStateVectors_cashflow);
            
            nonCall.payoffStateVectors_cashflow_npv(end,:) = payoffStateVectors_cashflow_npv;
            
            nonCall.cashflow_npv_e(end) = mean(payoffStateVectors_cashflow_npv);
            
            nonCall.payoff = nonCall.payoff + payoffStateVectors_cashflow_npv;
            
            payoffStateVectors_df = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,1.0*ones(1,pathSize));
            
            nonCall.payoffStateVector_df(end,:) = payoffStateVectors_df;
            nonCall.df_e(end) = mean(nonCall.payoffStateVector_df(end,:));
            nonCall.df_c(end) = model.DF(currentTime/365.0);
%             
%             nonCall.cashflow_npv_df(end,:) =  nonCall.npv/nominal;
%             nonCall.cashflow_npv_df_e(end) = mean(nonCall.cashflow_npv_df(end,:));
            
            lastTimeIdx = currentTimeIdx;
            totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
            
            for i=periodCount:-1:1
                % past schedule neglect
                if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                endM1TimeIdx = numMethodInfo.timeStepsInfo.endM1TimeIdx(i);
                
                %% FromToInduction start
                
                %modelsStates induct backward(lastTimeIdx to endM1TimeIdx)
                while endM1TimeIdx < lastTimeIdx
                    lastTimeIdx = lastTimeIdx -1;
                    if lastTimeIdx == endM1TimeIdx break;end
                end
                
                currentTimeIdx = lastTimeIdx;
                currentTime = totalTimeSteps(currentTimeIdx);
                % now we are at endM1TimeIdx
                % variables for coupon valuation initialized
                ralower = rangeAccrual.lower;
                raupper = rangeAccrual.upper;
                
                ralowerS = rangeAccrual.lowerS;
                raupperS = rangeAccrual.upperS;
                
                tenorCMS = rangeAccrual.tenor;
                
                tenorLong = rangeAccrual.tenorL;
                tenorShort = rangeAccrual.tenorS;
                
                cD = 0; % cumulated day count fraction
                cP = 0; 
                dI = 0; % dummy idx for loop
                dcfC = 0;
                dcfN = 0;
                dD = 0;
                idxCMS = zeros(1,pathSize);
                
                idxL = zeros(1,pathSize);
                idxS = zeros(1,pathSize);
                
                idxC = zeros(1,pathSize);
                idxN = zeros(1,pathSize);
                
                spreadC = zeros(1,pathSize);
                spreadN = zeros(1,pathSize);
                probSpread = zeros(1,pathSize);
                simpleYN = true;
                numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                
                for j=numOfTimeSteps:-1:1
                    % evaluate dealdescription
                    % last step is omitted
                    currentTime = totalTimeSteps(currentTimeIdx);
                    dcfC = currentTime/365.0;
                    if j~= numOfTimeSteps
                        dD = (dcfN-dcfC);
                    end
                    
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
                    idxCMS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorCMS,modelStateT);
                    idxC = idxCMS;
                    
                    idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                    idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                    spreadC = idxL - idxS;
                     
%                     probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                    probSpread = model.DRPr(idxC,idxN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS,simpleYN,pathSize)*dD;
                    
                    cD = cD + dD;
                    cP = cP + probSpread;
  
                    dcfN = dcfC;
                    
                    idxN = idxC;
                    spreadN = spreadC;
                    
                    %induct backward to the next time step
                    % one time backward induction
                    lastTimeIdx = currentTimeIdx;
                    
                    if startTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        currentTimeIdx = lastTimeIdx;
                    end
                    
                    
                end
                
                %% FromToInduction end
                %% AtDate operation
                % Adding a new cashflow to the payoff
                
                dcf = schedule.dayCountFraction(i);
                payoffStateVectors_cashflow = nominal * coupon * dcf * cP/cD;
                
                % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                % i.e. currentTimeIdx has not changed
                % no need to upodate modelStateT
                % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                % since cashflow's are added at the startDate(i) we need to
                % multiply cashflow with the df(startDate(i), payDate(i))
                payTime = DateDiff(schedule.payDates(i),valueDate);
                df_se = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);
                
                for k=1:pathSize
                    payoffStateVectors_cashflow(k) = payoffStateVectors_cashflow(k) + df_se(k);
                end
                
                nonCall.payoffStateVectors_cashflow(i,:) = payoffStateVectors_cashflow;
                
                nonCall.cashflow_e(i) = mean(payoffStateVectors_cashflow);
                nonCall.rangeProb_e(i) = mean(nonCall.cashflow_e(i)/(nominal * coupon * dcf));
                
                payoffStateVectors_cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,payoffStateVectors_cashflow);
            
                nonCall.cashflow_npv_e(i) = mean(payoffStateVectors_cashflow_npv);
                
                nonCall.payoffStateVectors_cashflow_npv(i,:) = payoffStateVectors_cashflow_npv;
                nonCall.payoff = nonCall.payoff + payoffStateVectors_cashflow_npv;
                
                payoffStateVectors_df = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,1.0*ones(1,pathSize));
                
                nonCall.payoffStateVector_df(i,:) = payoffStateVectors_df;
                nonCall.df_e(i) = mean(nonCall.payoffStateVector_df(i,:));
                nonCall.df_c(i) = model.DF(currentTime/365.0);
                
            end
            
            nonCall.npv = mean(nonCall.payoff);
  
            out.cashflow= nonCall.cashflow_e;
            out.cashflowNpv = nonCall.cashflow_npv_e;
            out.df_e = nonCall.df_e;
            out.df_c = nonCall.df_c;
            out.daycountFraction = schedule.dayCountFraction(i);
            out.rangeProb = nonCall.rangeProb_e;
            out.npv = nonCall.npv;
           
        end
        
        function out = computePrice(rangeAccrual,valueDate,model)
            schedule = rangeAccrual.schedule;
            periodCount = schedule.numOfPeriod;
            rangeProb = zeros(periodCount,1);
            discountFactor = zeros(periodCount,1);
            daycountFraction = zeros(periodCount,1);
            cashflow = zeros(periodCount,1);
            cashflowNpv = zeros(periodCount,1);
            rangeDays = rangeAccrual.rangeDays;
            npv = 0;
            for i=1:periodCount
                if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end;
                
                observeDate = schedule.startDates(i);
                if DateDiff(valueDate,schedule.startDates(i)) >= 0
                    observeDate = valueDate;
                    observeDate = AddDate(observeDate,1,'day');
                else
                    observeDate = schedule.startDates(i);
                end
                
                observeDays = DateDiff(schedule.endDates(i),schedule.startDates(i));
                days = DateDiff(schedule.endDates(i), observeDate);
                discountFactor(i) = model.DF( DateDiff(schedule.payDates(i),valueDate)/365.0);
                daycountFraction(i) = schedule.dayCountFraction(i);
                for j=1:days
                    toDate = observeDate;
                    tpDate = schedule.payDates(i);
                    tenor1 = rangeAccrual.tenorL;
                    tenor2 = rangeAccrual.tenorS;
                    tau =0.25;
                    lower = rangeAccrual.lower;
                    upper = rangeAccrual.upper;
                    rangeValue = model.CMSSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,lower,upper);
%                     rangeValue = 1.0;
                    rangeProb(i) = rangeProb(i) + rangeValue;
                    observeDate = AddDate(observeDate,1,'day');
                end
                cashflow(i) = rangeAccrual.nominal * rangeAccrual.coupon * (rangeProb(i) + rangeDays(i))/observeDays * daycountFraction(i);
                cashflowNpv(i) = cashflow(i) * discountFactor(i);
                npv = npv + cashflowNpv(i);
                % bug fixed observeDays -> days
                rangeProb(i) = rangeProb(i)/days;
                
                if i == periodCount
                    npv = npv + rangeAccrual.nominal * discountFactor(i);
                end
            end
            out.cashflow= cashflow;
            out.cashflowNpv = cashflowNpv;
            out.discountFactor = discountFactor;
            out.daycountFraction = daycountFraction;
            out.rangeProb = rangeProb;
            out.npv = npv;
        end
        
%         function out = discountPayoff(rangeAccrual,eventTime,numMatTime ...
%                 ,model,modelStatesT,cashflow)
%             
%             dfNumMat = model.DF(numMatTime/365.0);
%             % for discountPayoff T=numMatTime
%             % P(0,TnumMat)/P(Te,Tnummat)*Payoff
%             %
%             discountT = model.discountFactor(eventTime, numMatTime,numMatTime,modelStatesT);
%             stateSize = size(modelStatesT,2);
%             payoff = zeros(1,stateSize);
%             for i=1:stateSize
%                 payoff(i)= dfNumMat/discountT(i)*cashflow(i);
%             end    
%             out = payoff;
%         end
        
        
        
    end
    
end



In [ ]:
classdef CDRangeAccrual < Security
    %RangeAccrual is the SuperClass of all IRModel
    % properties: zeroCurve 
    % methods : DF, FWD,PV01,FSR
    properties(SetAccess = public)
        nominal
        coupon
        couponFreq
        tenor
        lower
        upper
        rangeDays
        callYN
        schedule
        couponSchedule
        callSchedule
        manualScheduleYN
        callStrike
        
    end
    
    methods
        % constructor
        function rangeAccrual = CDRangeAccrual(params)
                security = Security(params);
                rangeAccrual.payCcy = security.payCcy;
                rangeAccrual.startDate = security.startDate;
                rangeAccrual.endDate = security.endDate;
                rangeAccrual.maturity = security.maturity;
                
%                 rangeAccrual.payCcy = params.payCcy;
%                 rangeAccrual.startDate = params.startDate;
%                 rangeAccrual.maturity = params.maturity;
                
                
                %rangeAccrual.maturityDate = params.startDate.AddDate(params.maturity,'year');
                rangeAccrual.nominal = params.params('nominal');
                rangeAccrual.coupon = params.params('coupon');
                rangeAccrual.couponFreq= params.params('couponFreq');
                rangeAccrual.tenor = params.params('tenor');
                rangeAccrual.lower = params.params('lower');
                rangeAccrual.upper = params.params('upper');
                rangeAccrual.rangeDays = params.params('rangeDays');
                rangeAccrual.callYN = params.params('callYN');
                rangeAccrual.manualScheduleYN = params.params('manualScheduleYN');
                rangeAccrual.callStrike = params.params('callStrike');
                
                if rangeAccrual.manualScheduleYN
                    rangeAccrual.couponSchedule = rangeAccrual.ManualScheduleGenerate(params.params('couponScheduleInt'));
                    rangeAccrual.callSchedule = rangeAccrual.ManualScheduleGenerate(params.params('callScheduleInt'));
                    
                
                else
                    maturityDate = params.startDate.AddDate(params.maturity,'year');
                    rangeAccrual.schedule = rangeAccrual.ScheduleGenerate(rangeAccrual.startDate,maturityDate,rangeAccrual.couponFreq);
                
                end
                 
        end
        
        function out = ManualScheduleGenerate(rangeAccrual,scheduleInt)
            scheduleSize = size(scheduleInt,1);
            out.numOfPeriod = scheduleSize;
            for i=1:scheduleSize
                out.resetDates(i) = H_Date(scheduleInt(i,1));
                out.startDates(i) = H_Date(scheduleInt(i,2));
                out.endDates(i) = H_Date(scheduleInt(i,3));
                out.payDates(i) = H_Date(scheduleInt(i,4));
                out.dayCount(i) = out.endDates(i).DateDiff(out.startDates(i));
                out.dayCountFraction(i) = out.dayCount(i)/ 365.0;
            end
            
            
        end
        
        function out = ScheduleGenerate(rangeAccrual,initialDate,maturityDate,freq)
            schedule = ScheduleGenerate(initialDate,maturityDate,freq);
            out.startDates= schedule.startDates;
            out.resetDates = schedule.resetDates;
            out.endDates = schedule.endDates;
            out.payDates = schedule.payDates;
            out.dayCount = schedule.dayCount;
            out.dayCountFraction = schedule.dayCountFraction;
            out.numOfPeriod = schedule.numOfPeriod;
        end
        
        function out = numMethodTimeStepsGenerate(rangeAccrual,valueDate,schedule,mcOneTimeStep)
            
            periodCount = schedule.numOfPeriod;
            out.timeStepsPerPeriod =  cell(periodCount,1);
            
            out.startTimeIdx = zeros(periodCount,1);
            out.endM1TimeIdx = zeros(periodCount,1);
            
            out.numOfTimeSteps = zeros(periodCount,1);
            out.totalTimeSteps = [];
            out.totalTimeStepsSize = 0;
            numOfAlivePeriod = 0;
            
            for i=1:periodCount
                timeSteps = [];
                if DateDiff(valueDate,schedule.payDates(i)) >= 0
                    out.timeStepsPerPeriod(i) = timeSteps;
                    continue;
                end
                
                numOfAlivePeriod = numOfAlivePeriod + 1;
                if DateDiff(valueDate,schedule.startDates(i)) >= 0
                    startDate = valueDate;
                    endDate = schedule.endDates(i);
                    % we exclude end date from the 
                    endDate = endDate.AddDate(-1,'day');
                    
                    timeSteps = [];
                    % startDate should be included
                    startTime = startDate.DateDiff(valueDate);
                    timeSteps = [timeSteps; startTime];
                    
                    while 1
                        endTime = endDate.DateDiff(valueDate);
                        timeSteps = [timeSteps; endTime];
                        endDate = endDate.AddDate(-1*mcOneTimeStep,'day');
                        if endDate.DateDiff(startDate) < 0 break;end
                    end
                    timeSteps = sort(unique(timeSteps),'ascend');
                    %out.timeStepsPerPeriod {i,1}.timeSteps = [];
                    out.timeStepsPerPeriod{i,1}.timeSteps = timeSteps;
                    out.numOfTimeSteps(i) = length(timeSteps);
                    out.totalTimeSteps = [out.totalTimeSteps; timeSteps];
                    
                    % 1st period
                    out.startTimeIdx(i) = 1;
                    out.endM1TimeIdx(i) = length(timeSteps);
                    
                else
                    startDate = schedule.startDates(i);
                    endDate = schedule.endDates(i);
                    % we exclude end date from the 
                    endDate = endDate.AddDate(-1,'day');
                    
                    timeSteps = [];
                    % startDate should be included
                    startTime = startDate.DateDiff(valueDate);
                    timeSteps = [timeSteps; startTime];
                    
                    while 1
                        endTime = endDate.DateDiff(valueDate);
                        timeSteps = [timeSteps; endTime];
                        
                        endDate = endDate.AddDate(-1*mcOneTimeStep,'day');
                        if endDate.DateDiff(startDate) < 0 break;end
                    end
                    timeSteps = sort(unique(timeSteps),'ascend');
                    out.timeStepsPerPeriod{i,1}.timeSteps = timeSteps;
                    out.numOfTimeSteps(i) = length(timeSteps);
                    out.totalTimeSteps = [out.totalTimeSteps; timeSteps];
                    
                    % 1st period
                    out.startTimeIdx(i) = out.endM1TimeIdx(i-1) + 1;
                    out.endM1TimeIdx(i) = out.endM1TimeIdx(i-1) + length(timeSteps);
                    
                end
            end
            
            % endDate of the last period (= Security's maturityDate should
            % be included
            
            endTime = DateDiff(schedule.endDates(periodCount),valueDate);
            out.totalTimeSteps = [out.totalTimeSteps;endTime];
            out.totalTimeSteps = sort(unique(out.totalTimeSteps),'ascend');
            out.totalTimeStepsSize = length(out.totalTimeSteps);
            
            out.periodCount = periodCount;
            out.numOfAlivePeriod = numOfAlivePeriod;
        end
        
        function out = generateModelStates(rangeAccrual,valueDate,schedule,numMethodInfo,model)
            
            pathSize = numMethodInfo.pathSize;
            numOfFactors = numMethodInfo.numOfFactors;
            totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
            totalTimeSteps = numMethodInfo.totalTimeSteps;
            Z = numMethodInfo.Z;
            modelStates = zeros(numOfFactors*totalTimeStepsSize,pathSize);
            
            currentStep = totalTimeSteps(1);
            for i=1:totalTimeStepsSize-1
                nextStep = totalTimeSteps(i+1);
                sqrtDt = sqrt((nextStep-currentStep)/365.0);
                stateLocalVariance = model.stateLocalVariance(currentStep,nextStep);
                L = chol(stateLocalVariance);
                prevModelStates = modelStates(numOfFactors*(i-1)+1:numOfFactors*(i-1)+2,:);
                dW = Z(numOfFactors*i+1:numOfFactors*i+2,:);
                modelStates(numOfFactors*i+1:numOfFactors*i+2,:) = prevModelStates + L*dW; 
                currentStep = nextStep;
            end
            
            out = modelStates;
        end
        
        function out = computePriceMCForward(rangeAccrual,valueDate,model)
            schedule = rangeAccrual.schedule;
            
            % numMethodTimeSteps Generate
            mcOneTimeStep = 1;
            outInfo = rangeAccrual.numMethodTimeStepsGenerate(valueDate,schedule,mcOneTimeStep);
            
            numMethodInfo.timeStepsInfo = outInfo;
            
            % mc random number init
            %pathSize = 2^16;
            pathSize = 5000;
            
            %pathSize = 2^10;
            
            rng('default');
            rng(0);
            
            numOfFactors = 2;
            
            Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
            Z=[Z,-Z];
            
            numMethodInfo.pathSize = pathSize;
            numMethodInfo.numOfFactors = numOfFactors;
            numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
            numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
            numMethodInfo.Z = Z;
            
            % numeraire info
            numMatTime = numMethodInfo.totalTimeSteps(end);
            dfNumMat = model.DF(numMatTime/365.0);
            
            %generate model states for the given timeSteps
            modelStates = rangeAccrual.generateModelStates(valueDate,schedule,numMethodInfo,model);
            
            % induct backward and calculate cashflow
            
            
            periodCount = schedule.numOfPeriod;
            
            nonCall.npv = 0;
            
            nonCall.cashflow_e = zeros(periodCount + 1,1);
            nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
            nonCall.rangeProb_e = zeros(periodCount + 1,1);
            
            nonCall.df_e = zeros(periodCount + 1,1);
            nonCall.df_c = zeros(periodCount + 1,1);
            
            % sum of cashflow_npv
            nonCall.payoff = zeros(1,pathSize);
            
            nonCall.payoffStateVectors_cashflow = zeros(periodCount+1,pathSize);
            nonCall.payoffStateVectors_cashflow_npv = zeros(periodCount+1,pathSize);
            nonCall.payoffStateVector_df = zeros(periodCount + 1,pathSize);
            
            payoffStateVectors_cashflow = zeros(1, pathSize);
            payoffStateVectors_cashflow_npv = zeros(1, pathSize);
            payoffStateVectors_df = zeros(1,pathSize);
            
            % we calculate cashflow backward from maturityDate to valueDate
            % rangeAccrual coupons are added at the startDate of the Period
            % At maturity
            
            nominal = rangeAccrual.nominal;
            coupon = rangeAccrual.coupon;
            
            currentTime = numMethodInfo.totalTimeSteps(end);
            totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
            currentTimeIdx = totalTimeStepsSize;
            
            modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
            payoffStateVectors_cashflow = nominal*ones(1,pathSize);
            nonCall.payoffStateVectors_cashflow(end,:) = payoffStateVectors_cashflow;
            
            payoffStateVectors_cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,payoffStateVectors_cashflow);
            
            nonCall.payoffStateVectors_cashflow_npv(end,:) = payoffStateVectors_cashflow_npv;
            
            nonCall.cashflow_npv_e(end) = mean(payoffStateVectors_cashflow_npv);
            
            nonCall.payoff = nonCall.payoff + payoffStateVectors_cashflow_npv;
            
            payoffStateVectors_df = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,1.0*ones(1,pathSize));
            
            nonCall.payoffStateVector_df(end,:) = payoffStateVectors_df;
            nonCall.df_e(end) = mean(nonCall.payoffStateVector_df(end,:));
            nonCall.df_c(end) = model.DF(currentTime/365.0);
%             
%             nonCall.cashflow_npv_df(end,:) =  nonCall.npv/nominal;
%             nonCall.cashflow_npv_df_e(end) = mean(nonCall.cashflow_npv_df(end,:));
            
            lastTimeIdx = currentTimeIdx;
            totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
            
            for i=periodCount:-1:1
                % past schedule neglect
                if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                endM1TimeIdx = numMethodInfo.timeStepsInfo.endM1TimeIdx(i);
                
                %% FromToInduction start
                
                %modelsStates induct backward(lastTimeIdx to endM1TimeIdx)
                while endM1TimeIdx < lastTimeIdx
                    lastTimeIdx = lastTimeIdx -1;
                    if lastTimeIdx == endM1TimeIdx break;end
                end
                
                currentTimeIdx = lastTimeIdx;
                currentTime = totalTimeSteps(currentTimeIdx);
                % now we are at endM1TimeIdx
                % variables for coupon valuation initialized
                ralower = rangeAccrual.lower;
                raupper = rangeAccrual.upper;
                
                ralowerS = rangeAccrual.lowerS;
                raupperS = rangeAccrual.upperS;
                
                tenorCMS = rangeAccrual.tenor;
                
                tenorLong = rangeAccrual.tenorL;
                tenorShort = rangeAccrual.tenorS;
                
                cD = 0; % cumulated day count fraction
                cP = 0; 
                dI = 0; % dummy idx for loop
                dcfC = 0;
                dcfN = 0;
                dD = 0;
                idxCMS = zeros(1,pathSize);
                
                idxL = zeros(1,pathSize);
                idxS = zeros(1,pathSize);
                
                idxC = zeros(1,pathSize);
                idxN = zeros(1,pathSize);
                
                spreadC = zeros(1,pathSize);
                spreadN = zeros(1,pathSize);
                probSpread = zeros(1,pathSize);
                simpleYN = true;
                numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                
                for j=numOfTimeSteps:-1:1
                    % evaluate dealdescription
                    % last step is omitted
                    currentTime = totalTimeSteps(currentTimeIdx);
                    dcfC = currentTime/365.0;
                    if j~= numOfTimeSteps
                        dD = (dcfN-dcfC);
                    end
                    
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
                    idxCMS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorCMS,modelStateT);
                    idxC = idxCMS;
                    
                    idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                    idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                    spreadC = idxL - idxS;
                     
%                     probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                    probSpread = model.DRPr(idxC,idxN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS,simpleYN,pathSize)*dD;
                    
                    cD = cD + dD;
                    cP = cP + probSpread;
  
                    dcfN = dcfC;
                    
                    idxN = idxC;
                    spreadN = spreadC;
                    
                    %induct backward to the next time step
                    % one time backward induction
                    lastTimeIdx = currentTimeIdx;
                    
                    if startTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        currentTimeIdx = lastTimeIdx;
                    end
                    
                    
                end
                
                %% FromToInduction end
                %% AtDate operation
                % Adding a new cashflow to the payoff
                
                dcf = schedule.dayCountFraction(i);
                payoffStateVectors_cashflow = nominal * coupon * dcf * cP/cD;
                
                % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                % i.e. currentTimeIdx has not changed
                % no need to upodate modelStateT
                % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                % since cashflow's are added at the startDate(i) we need to
                % multiply cashflow with the df(startDate(i), payDate(i))
                payTime = DateDiff(schedule.payDates(i),valueDate);
                df_se = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);
                
                for k=1:pathSize
                    payoffStateVectors_cashflow(k) = payoffStateVectors_cashflow(k) + df_se(k);
                end
                
                nonCall.payoffStateVectors_cashflow(i,:) = payoffStateVectors_cashflow;
                
                nonCall.cashflow_e(i) = mean(payoffStateVectors_cashflow);
                nonCall.rangeProb_e(i) = mean(nonCall.cashflow_e(i)/(nominal * coupon * dcf));
                
                payoffStateVectors_cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,payoffStateVectors_cashflow);
            
                nonCall.cashflow_npv_e(i) = mean(payoffStateVectors_cashflow_npv);
                
                nonCall.payoffStateVectors_cashflow_npv(i,:) = payoffStateVectors_cashflow_npv;
                nonCall.payoff = nonCall.payoff + payoffStateVectors_cashflow_npv;
                
                payoffStateVectors_df = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,1.0*ones(1,pathSize));
                
                nonCall.payoffStateVector_df(i,:) = payoffStateVectors_df;
                nonCall.df_e(i) = mean(nonCall.payoffStateVector_df(i,:));
                nonCall.df_c(i) = model.DF(currentTime/365.0);
                
            end
            
            nonCall.npv = mean(nonCall.payoff);
  
            out.cashflow= nonCall.cashflow_e;
            out.cashflowNpv = nonCall.cashflow_npv_e;
            out.df_e = nonCall.df_e;
            out.df_c = nonCall.df_c;
            out.daycountFraction = schedule.dayCountFraction(i);
            out.rangeProb = nonCall.rangeProb_e;
            out.npv = nonCall.npv;
           
        end
        
        function out = computePrice(rangeAccrual,valueDate,model)
            schedule = rangeAccrual.schedule;
            periodCount = schedule.numOfPeriod;
            rangeProb = zeros(periodCount,1);
            discountFactor = zeros(periodCount,1);
            daycountFraction = zeros(periodCount,1);
            cashflow = zeros(periodCount,1);
            cashflowNpv = zeros(periodCount,1);
            rangeDays = rangeAccrual.rangeDays;
            npv = 0;
            for i=1:periodCount
                if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end;
                
                observeDate = schedule.startDates(i);
                if DateDiff(valueDate,schedule.startDates(i)) >= 0
                    observeDate = valueDate;
                    observeDate = AddDate(observeDate,1,'day');
                else
                    observeDate = schedule.startDates(i);
                end
                
                observeDays = DateDiff(schedule.endDates(i),schedule.startDates(i));
                days = DateDiff(schedule.endDates(i), observeDate);
                discountFactor(i) = model.DF( DateDiff(schedule.payDates(i),valueDate)/365.0);
                daycountFraction(i) = schedule.dayCountFraction(i);
                for j=1:days
                    toDate = observeDate;
                    tpDate = schedule.payDates(i);
                    tenor1 = rangeAccrual.tenorL;
                    tenor2 = rangeAccrual.tenorS;
                    tau =0.25;
                    lower = rangeAccrual.lower;
                    upper = rangeAccrual.upper;
                    rangeValue = model.CMSSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,lower,upper);
%                     rangeValue = 1.0;
                    rangeProb(i) = rangeProb(i) + rangeValue;
                    observeDate = AddDate(observeDate,1,'day');
                end
                cashflow(i) = rangeAccrual.nominal * rangeAccrual.coupon * (rangeProb(i) + rangeDays(i))/observeDays * daycountFraction(i);
                cashflowNpv(i) = cashflow(i) * discountFactor(i);
                npv = npv + cashflowNpv(i);
                % bug fixed observeDays -> days
                rangeProb(i) = rangeProb(i)/days;
                
                if i == periodCount
                    npv = npv + rangeAccrual.nominal * discountFactor(i);
                end
            end
            out.cashflow= cashflow;
            out.cashflowNpv = cashflowNpv;
            out.discountFactor = discountFactor;
            out.daycountFraction = daycountFraction;
            out.rangeProb = rangeProb;
            out.npv = npv;
        end
        
%         function out = discountPayoff(rangeAccrual,eventTime,numMatTime ...
%                 ,model,modelStatesT,cashflow)
%             
%             dfNumMat = model.DF(numMatTime/365.0);
%             % for discountPayoff T=numMatTime
%             % P(0,TnumMat)/P(Te,Tnummat)*Payoff
%             %
%             discountT = model.discountFactor(eventTime, numMatTime,numMatTime,modelStatesT);
%             stateSize = size(modelStatesT,2);
%             payoff = zeros(1,stateSize);
%             for i=1:stateSize
%                 payoff(i)= dfNumMat/discountT(i)*cashflow(i);
%             end    
%             out = payoff;
%         end
        
        
        
    end
    
end



In [ ]:
classdef CDKTBSpreadCMSSpreadDualRangeAccrual < Security
    %RangeAccrual is the SuperClass of all IRModel
    % properties: zeroCurve 
    % methods : DF, FWD,PV01,FSR
    properties(SetAccess = public)
        nominal
        coupon
        couponFreq
        tenorCMSL
        tenorCMSS
        
        lowerCMSLS
        upperCMSLS
        
        tenorL
        tenorS
        lowerS
        upperS
        rangeDays
        callYN
        schedule
        couponSchedule
        callSchedule
        manualScheduleYN
        hedgeBuffer
    end
    
    methods
        % constructor
        function rangeAccrual = CDKTBSpreadCMSSpreadDualRangeAccrual(params)
                rangeAccrual.payCcy = params.payCcy;
                rangeAccrual.startDate = params.startDate;
                rangeAccrual.maturity = params.maturity;
                
                %rangeAccrual.maturityDate = params.startDate.AddDate(params.maturity,'year');
                rangeAccrual.nominal = params.params('nominal');
                rangeAccrual.coupon = params.params('coupon');
                rangeAccrual.couponFreq= params.params('couponFreq');
                rangeAccrual.tenorCMSL = params.params('tenorCMSL');
                rangeAccrual.tenorCMSS = params.params('tenorCMSS');
                rangeAccrual.lowerCMSLS = params.params('lowerCMSLS');
                rangeAccrual.upperCMSLS = params.params('upperCMSLS');

                rangeAccrual.tenorL = params.params('tenorL');
                rangeAccrual.tenorS = params.params('tenorS');
                rangeAccrual.lowerS = params.params('lowerS');
                rangeAccrual.upperS = params.params('upperS');
                rangeAccrual.rangeDays = params.params('rangeDays');
                rangeAccrual.callYN = params.params('callYN');
                rangeAccrual.manualScheduleYN = params.params('manualScheduleYN');
                if rangeAccrual.manualScheduleYN
                    rangeAccrual.couponSchedule = rangeAccrual.ManualScheduleGenerate(params.params('couponScheduleInt'));
                    rangeAccrual.callSchedule = rangeAccrual.ManualScheduleGenerate(params.params('callScheduleInt'));
                    
                
                else
                    maturityDate = params.startDate.AddDate(params.maturity,'year');
                    rangeAccrual.schedule = rangeAccrual.ScheduleGenerate(rangeAccrual.startDate,maturityDate,rangeAccrual.couponFreq);
                
                end
                
                if isKey(params.params,'hedgeBuffer')
                    rangeAccrual.hedgeBuffer = params.params('hedgeBuffer');
                else
                    rangeAccrual.hedgeBuffer = 0.0;
                end
                 
        end
        
        function out = ManualScheduleGenerate(rangeAccrual,scheduleInt)
            scheduleSize = size(scheduleInt,1);
            out.numOfPeriod = scheduleSize;
            for i=1:scheduleSize
                out.resetDates(i) = H_Date(scheduleInt(i,1));
                out.startDates(i) = H_Date(scheduleInt(i,2));
                out.endDates(i) = H_Date(scheduleInt(i,3));
                out.payDates(i) = H_Date(scheduleInt(i,4));
                out.dayCount(i) = out.endDates(i).DateDiff(out.startDates(i));
                out.dayCountFraction(i) = out.dayCount(i)/ 365.0;
            end
            
            
        end
        
        function out = ScheduleGenerate(rangeAccrual,initialDate,maturityDate,freq)
            schedule = ScheduleGenerate(initialDate,maturityDate,freq);
            out.startDates= schedule.startDates;
            out.resetDates = schedule.resetDates;
            out.endDates = schedule.endDates;
            out.payDates = schedule.payDates;
            out.dayCount = schedule.dayCount;
            out.dayCountFraction = schedule.dayCountFraction;
            out.numOfPeriod = schedule.numOfPeriod;
        end
        
        function out = numMethodTimeStepsGenerate(rangeAccrual,valueDate,schedule,mcOneTimeStep)
            
            periodCount = schedule.numOfPeriod;
            out.timeStepsPerPeriod =  cell(periodCount,1);
            
            out.startTimeIdx = zeros(periodCount,1);
            out.endM1TimeIdx = zeros(periodCount,1);
            
            out.numOfTimeSteps = zeros(periodCount,1);
            out.totalTimeSteps = [];
            out.totalTimeStepsSize = 0;
            numOfAlivePeriod = 0;
            
            for i=1:periodCount
                timeSteps = [];
                if DateDiff(valueDate,schedule.payDates(i)) >= 0
                    out.timeStepsPerPeriod(i) = timeSteps;
                    continue;
                end
                
                numOfAlivePeriod = numOfAlivePeriod + 1;
                if DateDiff(valueDate,schedule.startDates(i)) >= 0
                    startDate = valueDate;
                    endDate = schedule.endDates(i);
                    % we exclude end date from the 
                    endDate = endDate.AddDate(-1,'day');
                    
                    timeSteps = [];
                    % startDate should be included
                    startTime = startDate.DateDiff(valueDate);
                    timeSteps = [timeSteps; startTime];
                    
                    while 1
                        endTime = endDate.DateDiff(valueDate);
                        timeSteps = [timeSteps; endTime];
                        endDate = endDate.AddDate(-1*mcOneTimeStep,'day');
                        if endDate.DateDiff(startDate) < 0 break;end
                    end
                    timeSteps = sort(unique(timeSteps),'ascend');
                    %out.timeStepsPerPeriod {i,1}.timeSteps = [];
                    out.timeStepsPerPeriod{i,1}.timeSteps = timeSteps;
                    out.numOfTimeSteps(i) = length(timeSteps);
                    out.totalTimeSteps = [out.totalTimeSteps; timeSteps];
                    
                    % 1st period
                    out.startTimeIdx(i) = 1;
                    out.endM1TimeIdx(i) = length(timeSteps);
                    
                else
                    startDate = schedule.startDates(i);
                    endDate = schedule.endDates(i);
                    % we exclude end date from the 
                    endDate = endDate.AddDate(-1,'day');
                    
                    timeSteps = [];
                    % startDate should be included
                    startTime = startDate.DateDiff(valueDate);
                    timeSteps = [timeSteps; startTime];
                    
                    while 1
                        endTime = endDate.DateDiff(valueDate);
                        timeSteps = [timeSteps; endTime];
                        
                        endDate = endDate.AddDate(-1*mcOneTimeStep,'day');
                        if endDate.DateDiff(startDate) < 0 break;end
                    end
                    timeSteps = sort(unique(timeSteps),'ascend');
                    out.timeStepsPerPeriod{i,1}.timeSteps = timeSteps;
                    out.numOfTimeSteps(i) = length(timeSteps);
                    out.totalTimeSteps = [out.totalTimeSteps; timeSteps];
                    
                    % 1st period
                    out.startTimeIdx(i) = out.endM1TimeIdx(i-1) + 1;
                    out.endM1TimeIdx(i) = out.endM1TimeIdx(i-1) + length(timeSteps);
                    
                end
            end
            
            % endDate of the last period (= Security's maturityDate should
            % be included
            
            endTime = DateDiff(schedule.endDates(periodCount),valueDate);
            out.totalTimeSteps = [out.totalTimeSteps;endTime];
            out.totalTimeSteps = sort(unique(out.totalTimeSteps),'ascend');
            out.totalTimeStepsSize = length(out.totalTimeSteps);
            
            out.periodCount = periodCount;
            out.numOfAlivePeriod = numOfAlivePeriod;
        end
        
        function out = generateModelStates(rangeAccrual,valueDate,schedule,numMethodInfo,model)
            
            pathSize = numMethodInfo.pathSize;
            numOfFactors = numMethodInfo.numOfFactors;
            totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
            totalTimeSteps = numMethodInfo.totalTimeSteps;
            Z = numMethodInfo.Z;
            modelStates = zeros(numOfFactors*totalTimeStepsSize,pathSize);
            
            currentStep = totalTimeSteps(1);
            for i=1:totalTimeStepsSize-1
                nextStep = totalTimeSteps(i+1);
                sqrtDt = sqrt((nextStep-currentStep)/365.0);
                stateLocalVariance = model.stateLocalVariance(currentStep,nextStep);
                L = chol(stateLocalVariance);
                prevModelStates = modelStates(numOfFactors*(i-1)+1:numOfFactors*(i-1)+2,:);
                dW = Z(numOfFactors*i+1:numOfFactors*i+2,:);
                modelStates(numOfFactors*i+1:numOfFactors*i+2,:) = prevModelStates + L*dW; 
                currentStep = nextStep;
            end
            
            out = modelStates;
        end
        
        function out = computePriceMCForward(rangeAccrual,valueDate,model)
            schedule = rangeAccrual.schedule;
            
            % numMethodTimeSteps Generate
            mcOneTimeStep = 1;
            outInfo = rangeAccrual.numMethodTimeStepsGenerate(valueDate,schedule,mcOneTimeStep);
            
            numMethodInfo.timeStepsInfo = outInfo;
            
            % mc random number init
            %pathSize = 2^16;
            pathSize = 5000;
            
            %pathSize = 2^10;
            
            rng('default');
            rng(0);
            
            numOfFactors = 2;
            
            Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
            Z=[Z,-Z];
            
            numMethodInfo.pathSize = pathSize;
            numMethodInfo.numOfFactors = numOfFactors;
            numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
            numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
            numMethodInfo.Z = Z;
            
            % numeraire info
            numMatTime = numMethodInfo.totalTimeSteps(end);
            dfNumMat = model.DF(numMatTime/365.0);
            
            %generate model states for the given timeSteps
            modelStates = rangeAccrual.generateModelStates(valueDate,schedule,numMethodInfo,model);
            
            % induct backward and calculate cashflow
            
            
            periodCount = schedule.numOfPeriod;
            
            nonCall.npv = 0;
            
            nonCall.cashflow_e = zeros(periodCount + 1,1);
            nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
            nonCall.rangeProb_e = zeros(periodCount + 1,1);
            
            nonCall.df_e = zeros(periodCount + 1,1);
            nonCall.df_c = zeros(periodCount + 1,1);
            
            % sum of cashflow_npv
            nonCall.payoff = zeros(1,pathSize);
            
            nonCall.payoffStateVectors_cashflow = zeros(periodCount+1,pathSize);
            nonCall.payoffStateVectors_cashflow_npv = zeros(periodCount+1,pathSize);
            nonCall.payoffStateVector_df = zeros(periodCount + 1,pathSize);
            
            payoffStateVectors_cashflow = zeros(1, pathSize);
            payoffStateVectors_cashflow_npv = zeros(1, pathSize);
            payoffStateVectors_df = zeros(1,pathSize);
            
            % we calculate cashflow backward from maturityDate to valueDate
            % rangeAccrual coupons are added at the startDate of the Period
            % At maturity
            
            nominal = rangeAccrual.nominal;
            coupon = rangeAccrual.coupon;
            
            currentTime = numMethodInfo.totalTimeSteps(end);
            totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
            currentTimeIdx = totalTimeStepsSize;
            
            modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
            payoffStateVectors_cashflow = nominal*ones(1,pathSize);
            nonCall.payoffStateVectors_cashflow(end,:) = payoffStateVectors_cashflow;
            
            payoffStateVectors_cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,payoffStateVectors_cashflow);
            
            nonCall.payoffStateVectors_cashflow_npv(end,:) = payoffStateVectors_cashflow_npv;
            
            nonCall.cashflow_npv_e(end) = mean(payoffStateVectors_cashflow_npv);
            
            nonCall.payoff = nonCall.payoff + payoffStateVectors_cashflow_npv;
            
            payoffStateVectors_df = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,1.0*ones(1,pathSize));
            
            nonCall.payoffStateVector_df(end,:) = payoffStateVectors_df;
            nonCall.df_e(end) = mean(nonCall.payoffStateVector_df(end,:));
            nonCall.df_c(end) = model.DF(currentTime/365.0);
%             
%             nonCall.cashflow_npv_df(end,:) =  nonCall.npv/nominal;
%             nonCall.cashflow_npv_df_e(end) = mean(nonCall.cashflow_npv_df(end,:));
            
            lastTimeIdx = currentTimeIdx;
            totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
            
            for i=periodCount:-1:1
                % past schedule neglect
                if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                endM1TimeIdx = numMethodInfo.timeStepsInfo.endM1TimeIdx(i);
                
                %% FromToInduction start
                
                %modelsStates induct backward(lastTimeIdx to endM1TimeIdx)
                while endM1TimeIdx < lastTimeIdx
                    lastTimeIdx = lastTimeIdx -1;
                    if lastTimeIdx == endM1TimeIdx break;end
                end
                
                currentTimeIdx = lastTimeIdx;
                currentTime = totalTimeSteps(currentTimeIdx);
                % now we are at endM1TimeIdx
                % variables for coupon valuation initialized
                ralower = rangeAccrual.lower;
                raupper = rangeAccrual.upper;
                
                ralowerS = rangeAccrual.lowerS;
                raupperS = rangeAccrual.upperS;
                
                tenorCMS = rangeAccrual.tenor;
                
                tenorLong = rangeAccrual.tenorL;
                tenorShort = rangeAccrual.tenorS;
                
                cD = 0; % cumulated day count fraction
                cP = 0; 
                dI = 0; % dummy idx for loop
                dcfC = 0;
                dcfN = 0;
                dD = 0;
                idxCMS = zeros(1,pathSize);
                
                idxL = zeros(1,pathSize);
                idxS = zeros(1,pathSize);
                
                idxC = zeros(1,pathSize);
                idxN = zeros(1,pathSize);
                
                spreadC = zeros(1,pathSize);
                spreadN = zeros(1,pathSize);
                probSpread = zeros(1,pathSize);
                simpleYN = true;
                numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                
                for j=numOfTimeSteps:-1:1
                    % evaluate dealdescription
                    % last step is omitted
                    currentTime = totalTimeSteps(currentTimeIdx);
                    dcfC = currentTime/365.0;
                    if j~= numOfTimeSteps
                        dD = (dcfN-dcfC);
                    end
                    
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
                    idxCMS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorCMS,modelStateT);
                    idxC = idxCMS;
                    
                    idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                    idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                    spreadC = idxL - idxS;
                     
%                     probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                    probSpread = model.DRPr(idxC,idxN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS,simpleYN,pathSize)*dD;
                    
                    cD = cD + dD;
                    cP = cP + probSpread;
  
                    dcfN = dcfC;
                    
                    idxN = idxC;
                    spreadN = spreadC;
                    
                    %induct backward to the next time step
                    % one time backward induction
                    lastTimeIdx = currentTimeIdx;
                    
                    if startTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        currentTimeIdx = lastTimeIdx;
                    end
                    
                    
                end
                
                %% FromToInduction end
                %% AtDate operation
                % Adding a new cashflow to the payoff
                
                dcf = schedule.dayCountFraction(i);
                payoffStateVectors_cashflow = nominal * coupon * dcf * cP/cD;
                
                % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                % i.e. currentTimeIdx has not changed
                % no need to upodate modelStateT
                % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                % since cashflow's are added at the startDate(i) we need to
                % multiply cashflow with the df(startDate(i), payDate(i))
                payTime = DateDiff(schedule.payDates(i),valueDate);
                df_se = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);
                
                for k=1:pathSize
                    payoffStateVectors_cashflow(k) = payoffStateVectors_cashflow(k) + df_se(k);
                end
                
                nonCall.payoffStateVectors_cashflow(i,:) = payoffStateVectors_cashflow;
                
                nonCall.cashflow_e(i) = mean(payoffStateVectors_cashflow);
                nonCall.rangeProb_e(i) = mean(nonCall.cashflow_e(i)/(nominal * coupon * dcf));
                
                payoffStateVectors_cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,payoffStateVectors_cashflow);
            
                nonCall.cashflow_npv_e(i) = mean(payoffStateVectors_cashflow_npv);
                
                nonCall.payoffStateVectors_cashflow_npv(i,:) = payoffStateVectors_cashflow_npv;
                nonCall.payoff = nonCall.payoff + payoffStateVectors_cashflow_npv;
                
                payoffStateVectors_df = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,1.0*ones(1,pathSize));
                
                nonCall.payoffStateVector_df(i,:) = payoffStateVectors_df;
                nonCall.df_e(i) = mean(nonCall.payoffStateVector_df(i,:));
                nonCall.df_c(i) = model.DF(currentTime/365.0);
                
            end
            
            nonCall.npv = mean(nonCall.payoff);
  
            out.cashflow= nonCall.cashflow_e;
            out.cashflowNpv = nonCall.cashflow_npv_e;
            out.df_e = nonCall.df_e;
            out.df_c = nonCall.df_c;
            out.daycountFraction = schedule.dayCountFraction(i);
            out.rangeProb = nonCall.rangeProb_e;
            out.npv = nonCall.npv;
           
        end
        
        function out = computePrice(rangeAccrual,valueDate,model)
            schedule = rangeAccrual.schedule;
            periodCount = schedule.numOfPeriod;
            rangeProb = zeros(periodCount,1);
            discountFactor = zeros(periodCount,1);
            daycountFraction = zeros(periodCount,1);
            cashflow = zeros(periodCount,1);
            cashflowNpv = zeros(periodCount,1);
            rangeDays = rangeAccrual.rangeDays;
            npv = 0;
            for i=1:periodCount
                if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end;
                
                observeDate = schedule.startDates(i);
                if DateDiff(valueDate,schedule.startDates(i)) >= 0
                    observeDate = valueDate;
                    observeDate = AddDate(observeDate,1,'day');
                else
                    observeDate = schedule.startDates(i);
                end
                
                observeDays = DateDiff(schedule.endDates(i),schedule.startDates(i));
                days = DateDiff(schedule.endDates(i), observeDate);
                discountFactor(i) = model.DF( DateDiff(schedule.payDates(i),valueDate)/365.0);
                daycountFraction(i) = schedule.dayCountFraction(i);
                for j=1:days
                    toDate = observeDate;
                    tpDate = schedule.payDates(i);
                    tenor1 = rangeAccrual.tenorL;
                    tenor2 = rangeAccrual.tenorS;
                    tau =0.25;
                    lower = rangeAccrual.lower;
                    upper = rangeAccrual.upper;
                    rangeValue = model.CMSSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,lower,upper);
%                     rangeValue = 1.0;
                    rangeProb(i) = rangeProb(i) + rangeValue;
                    observeDate = AddDate(observeDate,1,'day');
                end
                cashflow(i) = rangeAccrual.nominal * rangeAccrual.coupon * (rangeProb(i) + rangeDays(i))/observeDays * daycountFraction(i);
                cashflowNpv(i) = cashflow(i) * discountFactor(i);
                npv = npv + cashflowNpv(i);
                % bug fixed observeDays -> days
                rangeProb(i) = rangeProb(i)/days;
                
                if i == periodCount
                    npv = npv + rangeAccrual.nominal * discountFactor(i);
                end
            end
            out.cashflow= cashflow;
            out.cashflowNpv = cashflowNpv;
            out.discountFactor = discountFactor;
            out.daycountFraction = daycountFraction;
            out.rangeProb = rangeProb;
            out.npv = npv;
        end
        
%         function out = discountPayoff(rangeAccrual,eventTime,numMatTime ...
%                 ,model,modelStatesT,cashflow)
%             
%             dfNumMat = model.DF(numMatTime/365.0);
%             % for discountPayoff T=numMatTime
%             % P(0,TnumMat)/P(Te,Tnummat)*Payoff
%             %
%             discountT = model.discountFactor(eventTime, numMatTime,numMatTime,modelStatesT);
%             stateSize = size(modelStatesT,2);
%             payoff = zeros(1,stateSize);
%             for i=1:stateSize
%                 payoff(i)= dfNumMat/discountT(i)*cashflow(i);
%             end    
%             out = payoff;
%         end
        
        
        
    end
    
end



In [ ]:
classdef CDCMSSpreadRangeAccrualSwap < Security
    %RangeAccrual is the SuperClass of all IRModel
    % properties: zeroCurve 
    % methods : DF, FWD,PV01,FSR
    properties(SetAccess = public)
        nominal
        coupon
        couponFreq
        tenorL
        tenorS
        tenorCD
        lower
        upper
        lowerCD
        upperCD
        rangeDays
        callYN
        schedule
        couponSchedule
        callSchedule
        manualScheduleYN
        
        %floatingLeg
        floatingSchedule
        floatingSpread
        floatingIndex
        
        allFixings
        
    end
    
    methods
        % constructor
        function rangeAccrual = CDCMSSpreadRangeAccrualSwap(params,swapParams)
                rangeAccrual.payCcy = params.payCcy;
                rangeAccrual.startDate = params.startDate;
                rangeAccrual.maturity = params.maturity;
                
                %rangeAccrual.maturityDate = params.startDate.AddDate(params.maturity,'year');
                rangeAccrual.nominal = params.params('nominal');
                rangeAccrual.coupon = params.params('coupon');
                rangeAccrual.couponFreq= params.params('couponFreq');
                rangeAccrual.tenorL = params.params('tenorL');
                rangeAccrual.tenorS = params.params('tenorS');
                rangeAccrual.tenorCD = params.params('tenorCD');
                rangeAccrual.lower = params.params('lower');
                rangeAccrual.upper = params.params('upper');
                rangeAccrual.lowerCD = params.params('lowerCD');
                rangeAccrual.upperCD = params.params('upperCD');
                
                rangeAccrual.rangeDays = params.params('rangeDays');
                rangeAccrual.callYN = params.params('callYN');
                rangeAccrual.manualScheduleYN = params.params('manualScheduleYN');
                if rangeAccrual.manualScheduleYN
                    rangeAccrual.couponSchedule = rangeAccrual.ManualScheduleGenerate(params.params('couponScheduleInt'));
                    rangeAccrual.callSchedule = rangeAccrual.ManualScheduleGenerate(params.params('callScheduleInt'));
                    rangeAccrual.floatingSchedule = rangeAccrual.ManualScheduleGenerate(swapParams.params('floatingScheduleInt'));
                
                else
                    maturityDate = params.startDate.AddDate(params.maturity,'year');
                    rangeAccrual.schedule = rangeAccrual.ScheduleGenerate(rangeAccrual.startDate,maturityDate,rangeAccrual.couponFreq);
                
                end
                
                 %floatingLeg
                rangeAccrual.floatingSpread = swapParams.params('floatingSpread');
                rangeAccrual.floatingIndex = swapParams.params('floatingIndex');
                
                rangeAccrual.allFixings = swapParams.params('allFixings'); 
        end
        
        function out = ManualScheduleGenerate(rangeAccrual,scheduleInt)
            scheduleSize = size(scheduleInt,1);
            out.numOfPeriod = scheduleSize;
            for i=1:scheduleSize
                out.resetDates(i) = H_Date(scheduleInt(i,1));
                out.startDates(i) = H_Date(scheduleInt(i,2));
                out.endDates(i) = H_Date(scheduleInt(i,3));
                out.payDates(i) = H_Date(scheduleInt(i,4));
                out.dayCount(i) = out.endDates(i).DateDiff(out.startDates(i));
                out.dayCountFraction(i) = out.dayCount(i)/ 365.0;
            end
            
            
        end
        
        function out = ScheduleGenerate(rangeAccrual,initialDate,maturityDate,freq)
            schedule = ScheduleGenerate(initialDate,maturityDate,freq);
            out.startDates= schedule.startDates;
            out.resetDates = schedule.resetDates;
            out.endDates = schedule.endDates;
            out.payDates = schedule.payDates;
            out.dayCount = schedule.dayCount;
            out.dayCountFraction = schedule.dayCountFraction;
            out.numOfPeriod = schedule.numOfPeriod;
        end
        
        
    end
    
end



In [ ]:
classdef CDCMSSpreadDualRangeAccrualInverseFloater < Security
    %RangeAccrual is the SuperClass of all IRModel
    % properties: zeroCurve 
    % methods : DF, FWD,PV01,FSR
    properties(SetAccess = public)
        nominal
        coupon
        couponFreq
        tenor
        
        lower
        upper
        
        tenorL
        tenorS
        lowerS
        upperS
        rangeDays
        callYN
        schedule
        couponSchedule
        callSchedule
        manualScheduleYN
        hedgeBuffer
        
        couponLowerBound
    end
    
    methods
        % constructor
        function rangeAccrual = CDCMSSpreadDualRangeAccrualInverseFloater(params)
                rangeAccrual.payCcy = params.payCcy;
                rangeAccrual.startDate = params.startDate;
                rangeAccrual.maturity = params.maturity;
                
                %rangeAccrual.maturityDate = params.startDate.AddDate(params.maturity,'year');
                rangeAccrual.nominal = params.params('nominal');
                rangeAccrual.coupon = params.params('coupon');
                rangeAccrual.couponFreq= params.params('couponFreq');
                rangeAccrual.tenor = params.params('tenor');
                rangeAccrual.tenorL = params.params('tenorL');
                rangeAccrual.tenorS = params.params('tenorS');
                rangeAccrual.lower = params.params('lower');
                rangeAccrual.upper = params.params('upper');
                rangeAccrual.lowerS = params.params('lowerS');
                rangeAccrual.upperS = params.params('upperS');
                rangeAccrual.rangeDays = params.params('rangeDays');
                rangeAccrual.callYN = params.params('callYN');
                rangeAccrual.manualScheduleYN = params.params('manualScheduleYN');
                if rangeAccrual.manualScheduleYN
                    rangeAccrual.couponSchedule = rangeAccrual.ManualScheduleGenerate(params.params('couponScheduleInt'));
                    rangeAccrual.callSchedule = rangeAccrual.ManualScheduleGenerate(params.params('callScheduleInt'));
                    
                
                else
                    maturityDate = params.startDate.AddDate(params.maturity,'year');
                    rangeAccrual.schedule = rangeAccrual.ScheduleGenerate(rangeAccrual.startDate,maturityDate,rangeAccrual.couponFreq);
                
                end
                
                if isKey(params.params,'hedgeBuffer')
                    rangeAccrual.hedgeBuffer = params.params('hedgeBuffer');
                else
                    rangeAccrual.hedgeBuffer = 0.0;
                end
                
                rangeAccrual.couponLowerBound = params.params('couponLowerBound');
                 
        end
        
        function out = ManualScheduleGenerate(rangeAccrual,scheduleInt)
            scheduleSize = size(scheduleInt,1);
            out.numOfPeriod = scheduleSize;
            for i=1:scheduleSize
                out.resetDates(i) = H_Date(scheduleInt(i,1));
                out.startDates(i) = H_Date(scheduleInt(i,2));
                out.endDates(i) = H_Date(scheduleInt(i,3));
                out.payDates(i) = H_Date(scheduleInt(i,4));
                out.dayCount(i) = out.endDates(i).DateDiff(out.startDates(i));
                out.dayCountFraction(i) = out.dayCount(i)/ 365.0;
            end
            
            
        end
        
        function out = ScheduleGenerate(rangeAccrual,initialDate,maturityDate,freq)
            schedule = ScheduleGenerate(initialDate,maturityDate,freq);
            out.startDates= schedule.startDates;
            out.resetDates = schedule.resetDates;
            out.endDates = schedule.endDates;
            out.payDates = schedule.payDates;
            out.dayCount = schedule.dayCount;
            out.dayCountFraction = schedule.dayCountFraction;
            out.numOfPeriod = schedule.numOfPeriod;
        end
        
        function out = numMethodTimeStepsGenerate(rangeAccrual,valueDate,schedule,mcOneTimeStep)
            
            periodCount = schedule.numOfPeriod;
            out.timeStepsPerPeriod =  cell(periodCount,1);
            
            out.startTimeIdx = zeros(periodCount,1);
            out.endM1TimeIdx = zeros(periodCount,1);
            
            out.numOfTimeSteps = zeros(periodCount,1);
            out.totalTimeSteps = [];
            out.totalTimeStepsSize = 0;
            numOfAlivePeriod = 0;
            
            for i=1:periodCount
                timeSteps = [];
                if DateDiff(valueDate,schedule.payDates(i)) >= 0
                    out.timeStepsPerPeriod(i) = timeSteps;
                    continue;
                end
                
                numOfAlivePeriod = numOfAlivePeriod + 1;
                if DateDiff(valueDate,schedule.startDates(i)) >= 0
                    startDate = valueDate;
                    endDate = schedule.endDates(i);
                    % we exclude end date from the 
                    endDate = endDate.AddDate(-1,'day');
                    
                    timeSteps = [];
                    % startDate should be included
                    startTime = startDate.DateDiff(valueDate);
                    timeSteps = [timeSteps; startTime];
                    
                    while 1
                        endTime = endDate.DateDiff(valueDate);
                        timeSteps = [timeSteps; endTime];
                        endDate = endDate.AddDate(-1*mcOneTimeStep,'day');
                        if endDate.DateDiff(startDate) < 0 break;end
                    end
                    timeSteps = sort(unique(timeSteps),'ascend');
                    %out.timeStepsPerPeriod {i,1}.timeSteps = [];
                    out.timeStepsPerPeriod{i,1}.timeSteps = timeSteps;
                    out.numOfTimeSteps(i) = length(timeSteps);
                    out.totalTimeSteps = [out.totalTimeSteps; timeSteps];
                    
                    % 1st period
                    out.startTimeIdx(i) = 1;
                    out.endM1TimeIdx(i) = length(timeSteps);
                    
                else
                    startDate = schedule.startDates(i);
                    endDate = schedule.endDates(i);
                    % we exclude end date from the 
                    endDate = endDate.AddDate(-1,'day');
                    
                    timeSteps = [];
                    % startDate should be included
                    startTime = startDate.DateDiff(valueDate);
                    timeSteps = [timeSteps; startTime];
                    
                    while 1
                        endTime = endDate.DateDiff(valueDate);
                        timeSteps = [timeSteps; endTime];
                        
                        endDate = endDate.AddDate(-1*mcOneTimeStep,'day');
                        if endDate.DateDiff(startDate) < 0 break;end
                    end
                    timeSteps = sort(unique(timeSteps),'ascend');
                    out.timeStepsPerPeriod{i,1}.timeSteps = timeSteps;
                    out.numOfTimeSteps(i) = length(timeSteps);
                    out.totalTimeSteps = [out.totalTimeSteps; timeSteps];
                    
                    % 1st period
                    out.startTimeIdx(i) = out.endM1TimeIdx(i-1) + 1;
                    out.endM1TimeIdx(i) = out.endM1TimeIdx(i-1) + length(timeSteps);
                    
                end
            end
            
            % endDate of the last period (= Security's maturityDate should
            % be included
            
            endTime = DateDiff(schedule.endDates(periodCount),valueDate);
            out.totalTimeSteps = [out.totalTimeSteps;endTime];
            out.totalTimeSteps = sort(unique(out.totalTimeSteps),'ascend');
            out.totalTimeStepsSize = length(out.totalTimeSteps);
            
            out.periodCount = periodCount;
            out.numOfAlivePeriod = numOfAlivePeriod;
        end
        
        function out = generateModelStates(rangeAccrual,valueDate,schedule,numMethodInfo,model)
            
            pathSize = numMethodInfo.pathSize;
            numOfFactors = numMethodInfo.numOfFactors;
            totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
            totalTimeSteps = numMethodInfo.totalTimeSteps;
            Z = numMethodInfo.Z;
            modelStates = zeros(numOfFactors*totalTimeStepsSize,pathSize);
            
            currentStep = totalTimeSteps(1);
            for i=1:totalTimeStepsSize-1
                nextStep = totalTimeSteps(i+1);
                sqrtDt = sqrt((nextStep-currentStep)/365.0);
                stateLocalVariance = model.stateLocalVariance(currentStep,nextStep);
                L = chol(stateLocalVariance);
                prevModelStates = modelStates(numOfFactors*(i-1)+1:numOfFactors*(i-1)+2,:);
                dW = Z(numOfFactors*i+1:numOfFactors*i+2,:);
                modelStates(numOfFactors*i+1:numOfFactors*i+2,:) = prevModelStates + L*dW; 
                currentStep = nextStep;
            end
            
            out = modelStates;
        end
        
        function out = computePriceMCForward(rangeAccrual,valueDate,model)
            schedule = rangeAccrual.schedule;
            
            % numMethodTimeSteps Generate
            mcOneTimeStep = 1;
            outInfo = rangeAccrual.numMethodTimeStepsGenerate(valueDate,schedule,mcOneTimeStep);
            
            numMethodInfo.timeStepsInfo = outInfo;
            
            % mc random number init
            %pathSize = 2^16;
            pathSize = 5000;
            
            %pathSize = 2^10;
            
            rng('default');
            rng(0);
            
            numOfFactors = 2;
            
            Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
            Z=[Z,-Z];
            
            numMethodInfo.pathSize = pathSize;
            numMethodInfo.numOfFactors = numOfFactors;
            numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
            numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
            numMethodInfo.Z = Z;
            
            % numeraire info
            numMatTime = numMethodInfo.totalTimeSteps(end);
            dfNumMat = model.DF(numMatTime/365.0);
            
            %generate model states for the given timeSteps
            modelStates = rangeAccrual.generateModelStates(valueDate,schedule,numMethodInfo,model);
            
            % induct backward and calculate cashflow
            
            
            periodCount = schedule.numOfPeriod;
            
            nonCall.npv = 0;
            
            nonCall.cashflow_e = zeros(periodCount + 1,1);
            nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
            nonCall.rangeProb_e = zeros(periodCount + 1,1);
            
            nonCall.df_e = zeros(periodCount + 1,1);
            nonCall.df_c = zeros(periodCount + 1,1);
            
            % sum of cashflow_npv
            nonCall.payoff = zeros(1,pathSize);
            
            nonCall.payoffStateVectors_cashflow = zeros(periodCount+1,pathSize);
            nonCall.payoffStateVectors_cashflow_npv = zeros(periodCount+1,pathSize);
            nonCall.payoffStateVector_df = zeros(periodCount + 1,pathSize);
            
            payoffStateVectors_cashflow = zeros(1, pathSize);
            payoffStateVectors_cashflow_npv = zeros(1, pathSize);
            payoffStateVectors_df = zeros(1,pathSize);
            
            % we calculate cashflow backward from maturityDate to valueDate
            % rangeAccrual coupons are added at the startDate of the Period
            % At maturity
            
            nominal = rangeAccrual.nominal;
            coupon = rangeAccrual.coupon;
            
            currentTime = numMethodInfo.totalTimeSteps(end);
            totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
            currentTimeIdx = totalTimeStepsSize;
            
            modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
            payoffStateVectors_cashflow = nominal*ones(1,pathSize);
            nonCall.payoffStateVectors_cashflow(end,:) = payoffStateVectors_cashflow;
            
            payoffStateVectors_cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,payoffStateVectors_cashflow);
            
            nonCall.payoffStateVectors_cashflow_npv(end,:) = payoffStateVectors_cashflow_npv;
            
            nonCall.cashflow_npv_e(end) = mean(payoffStateVectors_cashflow_npv);
            
            nonCall.payoff = nonCall.payoff + payoffStateVectors_cashflow_npv;
            
            payoffStateVectors_df = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,1.0*ones(1,pathSize));
            
            nonCall.payoffStateVector_df(end,:) = payoffStateVectors_df;
            nonCall.df_e(end) = mean(nonCall.payoffStateVector_df(end,:));
            nonCall.df_c(end) = model.DF(currentTime/365.0);
%             
%             nonCall.cashflow_npv_df(end,:) =  nonCall.npv/nominal;
%             nonCall.cashflow_npv_df_e(end) = mean(nonCall.cashflow_npv_df(end,:));
            
            lastTimeIdx = currentTimeIdx;
            totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
            
            for i=periodCount:-1:1
                % past schedule neglect
                if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                endM1TimeIdx = numMethodInfo.timeStepsInfo.endM1TimeIdx(i);
                
                %% FromToInduction start
                
                %modelsStates induct backward(lastTimeIdx to endM1TimeIdx)
                while endM1TimeIdx < lastTimeIdx
                    lastTimeIdx = lastTimeIdx -1;
                    if lastTimeIdx == endM1TimeIdx break;end
                end
                
                currentTimeIdx = lastTimeIdx;
                currentTime = totalTimeSteps(currentTimeIdx);
                % now we are at endM1TimeIdx
                % variables for coupon valuation initialized
                ralower = rangeAccrual.lower;
                raupper = rangeAccrual.upper;
                
                ralowerS = rangeAccrual.lowerS;
                raupperS = rangeAccrual.upperS;
                
                tenorCMS = rangeAccrual.tenor;
                
                tenorLong = rangeAccrual.tenorL;
                tenorShort = rangeAccrual.tenorS;
                
                cD = 0; % cumulated day count fraction
                cP = 0; 
                dI = 0; % dummy idx for loop
                dcfC = 0;
                dcfN = 0;
                dD = 0;
                idxCMS = zeros(1,pathSize);
                
                idxL = zeros(1,pathSize);
                idxS = zeros(1,pathSize);
                
                idxC = zeros(1,pathSize);
                idxN = zeros(1,pathSize);
                
                spreadC = zeros(1,pathSize);
                spreadN = zeros(1,pathSize);
                probSpread = zeros(1,pathSize);
                simpleYN = true;
                numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                
                for j=numOfTimeSteps:-1:1
                    % evaluate dealdescription
                    % last step is omitted
                    currentTime = totalTimeSteps(currentTimeIdx);
                    dcfC = currentTime/365.0;
                    if j~= numOfTimeSteps
                        dD = (dcfN-dcfC);
                    end
                    
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
                    idxCMS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorCMS,modelStateT);
                    idxC = idxCMS;
                    
                    idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                    idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                    spreadC = idxL - idxS;
                     
%                     probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                    probSpread = model.DRPr(idxC,idxN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS,simpleYN,pathSize)*dD;
                    
                    cD = cD + dD;
                    cP = cP + probSpread;
  
                    dcfN = dcfC;
                    
                    idxN = idxC;
                    spreadN = spreadC;
                    
                    %induct backward to the next time step
                    % one time backward induction
                    lastTimeIdx = currentTimeIdx;
                    
                    if startTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        currentTimeIdx = lastTimeIdx;
                    end
                    
                    
                end
                
                %% FromToInduction end
                %% AtDate operation
                % Adding a new cashflow to the payoff
                
                dcf = schedule.dayCountFraction(i);
                payoffStateVectors_cashflow = nominal * coupon * dcf * cP/cD;
                
                % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                % i.e. currentTimeIdx has not changed
                % no need to upodate modelStateT
                % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                % since cashflow's are added at the startDate(i) we need to
                % multiply cashflow with the df(startDate(i), payDate(i))
                payTime = DateDiff(schedule.payDates(i),valueDate);
                df_se = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);
                
                for k=1:pathSize
                    payoffStateVectors_cashflow(k) = payoffStateVectors_cashflow(k) + df_se(k);
                end
                
                nonCall.payoffStateVectors_cashflow(i,:) = payoffStateVectors_cashflow;
                
                nonCall.cashflow_e(i) = mean(payoffStateVectors_cashflow);
                nonCall.rangeProb_e(i) = mean(nonCall.cashflow_e(i)/(nominal * coupon * dcf));
                
                payoffStateVectors_cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,payoffStateVectors_cashflow);
            
                nonCall.cashflow_npv_e(i) = mean(payoffStateVectors_cashflow_npv);
                
                nonCall.payoffStateVectors_cashflow_npv(i,:) = payoffStateVectors_cashflow_npv;
                nonCall.payoff = nonCall.payoff + payoffStateVectors_cashflow_npv;
                
                payoffStateVectors_df = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,1.0*ones(1,pathSize));
                
                nonCall.payoffStateVector_df(i,:) = payoffStateVectors_df;
                nonCall.df_e(i) = mean(nonCall.payoffStateVector_df(i,:));
                nonCall.df_c(i) = model.DF(currentTime/365.0);
                
            end
            
            nonCall.npv = mean(nonCall.payoff);
  
            out.cashflow= nonCall.cashflow_e;
            out.cashflowNpv = nonCall.cashflow_npv_e;
            out.df_e = nonCall.df_e;
            out.df_c = nonCall.df_c;
            out.daycountFraction = schedule.dayCountFraction(i);
            out.rangeProb = nonCall.rangeProb_e;
            out.npv = nonCall.npv;
           
        end
        
        function out = computePrice(rangeAccrual,valueDate,model)
            schedule = rangeAccrual.schedule;
            periodCount = schedule.numOfPeriod;
            rangeProb = zeros(periodCount,1);
            discountFactor = zeros(periodCount,1);
            daycountFraction = zeros(periodCount,1);
            cashflow = zeros(periodCount,1);
            cashflowNpv = zeros(periodCount,1);
            rangeDays = rangeAccrual.rangeDays;
            npv = 0;
            for i=1:periodCount
                if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end;
                
                observeDate = schedule.startDates(i);
                if DateDiff(valueDate,schedule.startDates(i)) >= 0
                    observeDate = valueDate;
                    observeDate = AddDate(observeDate,1,'day');
                else
                    observeDate = schedule.startDates(i);
                end
                
                observeDays = DateDiff(schedule.endDates(i),schedule.startDates(i));
                days = DateDiff(schedule.endDates(i), observeDate);
                discountFactor(i) = model.DF( DateDiff(schedule.payDates(i),valueDate)/365.0);
                daycountFraction(i) = schedule.dayCountFraction(i);
                for j=1:days
                    toDate = observeDate;
                    tpDate = schedule.payDates(i);
                    tenor1 = rangeAccrual.tenorL;
                    tenor2 = rangeAccrual.tenorS;
                    tau =0.25;
                    lower = rangeAccrual.lower;
                    upper = rangeAccrual.upper;
                    rangeValue = model.CMSSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,lower,upper);
%                     rangeValue = 1.0;
                    rangeProb(i) = rangeProb(i) + rangeValue;
                    observeDate = AddDate(observeDate,1,'day');
                end
                cashflow(i) = rangeAccrual.nominal * rangeAccrual.coupon * (rangeProb(i) + rangeDays(i))/observeDays * daycountFraction(i);
                cashflowNpv(i) = cashflow(i) * discountFactor(i);
                npv = npv + cashflowNpv(i);
                % bug fixed observeDays -> days
                rangeProb(i) = rangeProb(i)/days;
                
                if i == periodCount
                    npv = npv + rangeAccrual.nominal * discountFactor(i);
                end
            end
            out.cashflow= cashflow;
            out.cashflowNpv = cashflowNpv;
            out.discountFactor = discountFactor;
            out.daycountFraction = daycountFraction;
            out.rangeProb = rangeProb;
            out.npv = npv;
        end
        
%         function out = discountPayoff(rangeAccrual,eventTime,numMatTime ...
%                 ,model,modelStatesT,cashflow)
%             
%             dfNumMat = model.DF(numMatTime/365.0);
%             % for discountPayoff T=numMatTime
%             % P(0,TnumMat)/P(Te,Tnummat)*Payoff
%             %
%             discountT = model.discountFactor(eventTime, numMatTime,numMatTime,modelStatesT);
%             stateSize = size(modelStatesT,2);
%             payoff = zeros(1,stateSize);
%             for i=1:stateSize
%                 payoff(i)= dfNumMat/discountT(i)*cashflow(i);
%             end    
%             out = payoff;
%         end
        
        
        
    end
    
end



In [ ]:
classdef CDCMSSpreadDualRangeAccrual < Security
    %RangeAccrual is the SuperClass of all IRModel
    % properties: zeroCurve 
    % methods : DF, FWD,PV01,FSR
    properties(SetAccess = public)
        nominal
        coupon
        couponFreq
        tenor
        
        lower
        upper
        
        tenorL
        tenorS
        lowerS
        upperS
        rangeDays
        callYN
        schedule
        couponSchedule
        callSchedule
        manualScheduleYN
        hedgeBuffer
    end
    
    methods
        % constructor
        function rangeAccrual = CDCMSSpreadDualRangeAccrual(params)
                rangeAccrual.payCcy = params.payCcy;
                rangeAccrual.startDate = params.startDate;
                rangeAccrual.maturity = params.maturity;
                
                %rangeAccrual.maturityDate = params.startDate.AddDate(params.maturity,'year');
                rangeAccrual.nominal = params.params('nominal');
                rangeAccrual.coupon = params.params('coupon');
                rangeAccrual.couponFreq= params.params('couponFreq');
                rangeAccrual.tenor = params.params('tenor');
                rangeAccrual.tenorL = params.params('tenorL');
                rangeAccrual.tenorS = params.params('tenorS');
                rangeAccrual.lower = params.params('lower');
                rangeAccrual.upper = params.params('upper');
                rangeAccrual.lowerS = params.params('lowerS');
                rangeAccrual.upperS = params.params('upperS');
                rangeAccrual.rangeDays = params.params('rangeDays');
                rangeAccrual.callYN = params.params('callYN');
                rangeAccrual.manualScheduleYN = params.params('manualScheduleYN');
                if rangeAccrual.manualScheduleYN
                    rangeAccrual.couponSchedule = rangeAccrual.ManualScheduleGenerate(params.params('couponScheduleInt'));
                    rangeAccrual.callSchedule = rangeAccrual.ManualScheduleGenerate(params.params('callScheduleInt'));
                    
                
                else
                    maturityDate = params.startDate.AddDate(params.maturity,'year');
                    rangeAccrual.schedule = rangeAccrual.ScheduleGenerate(rangeAccrual.startDate,maturityDate,rangeAccrual.couponFreq);
                
                end
                
                if isKey(params.params,'hedgeBuffer')
                    rangeAccrual.hedgeBuffer = params.params('hedgeBuffer');
                else
                    rangeAccrual.hedgeBuffer = 0.0;
                end
                 
        end
        
        function out = ManualScheduleGenerate(rangeAccrual,scheduleInt)
            scheduleSize = size(scheduleInt,1);
            out.numOfPeriod = scheduleSize;
            for i=1:scheduleSize
                out.resetDates(i) = H_Date(scheduleInt(i,1));
                out.startDates(i) = H_Date(scheduleInt(i,2));
                out.endDates(i) = H_Date(scheduleInt(i,3));
                out.payDates(i) = H_Date(scheduleInt(i,4));
                out.dayCount(i) = out.endDates(i).DateDiff(out.startDates(i));
                out.dayCountFraction(i) = out.dayCount(i)/ 365.0;
            end
            
            
        end
        
        function out = ScheduleGenerate(rangeAccrual,initialDate,maturityDate,freq)
            schedule = ScheduleGenerate(initialDate,maturityDate,freq);
            out.startDates= schedule.startDates;
            out.resetDates = schedule.resetDates;
            out.endDates = schedule.endDates;
            out.payDates = schedule.payDates;
            out.dayCount = schedule.dayCount;
            out.dayCountFraction = schedule.dayCountFraction;
            out.numOfPeriod = schedule.numOfPeriod;
        end
        
        function out = numMethodTimeStepsGenerate(rangeAccrual,valueDate,schedule,mcOneTimeStep)
            
            periodCount = schedule.numOfPeriod;
            out.timeStepsPerPeriod =  cell(periodCount,1);
            
            out.startTimeIdx = zeros(periodCount,1);
            out.endM1TimeIdx = zeros(periodCount,1);
            
            out.numOfTimeSteps = zeros(periodCount,1);
            out.totalTimeSteps = [];
            out.totalTimeStepsSize = 0;
            numOfAlivePeriod = 0;
            
            for i=1:periodCount
                timeSteps = [];
                if DateDiff(valueDate,schedule.payDates(i)) >= 0
                    out.timeStepsPerPeriod(i) = timeSteps;
                    continue;
                end
                
                numOfAlivePeriod = numOfAlivePeriod + 1;
                if DateDiff(valueDate,schedule.startDates(i)) >= 0
                    startDate = valueDate;
                    endDate = schedule.endDates(i);
                    % we exclude end date from the 
                    endDate = endDate.AddDate(-1,'day');
                    
                    timeSteps = [];
                    % startDate should be included
                    startTime = startDate.DateDiff(valueDate);
                    timeSteps = [timeSteps; startTime];
                    
                    while 1
                        endTime = endDate.DateDiff(valueDate);
                        timeSteps = [timeSteps; endTime];
                        endDate = endDate.AddDate(-1*mcOneTimeStep,'day');
                        if endDate.DateDiff(startDate) < 0 break;end
                    end
                    timeSteps = sort(unique(timeSteps),'ascend');
                    %out.timeStepsPerPeriod {i,1}.timeSteps = [];
                    out.timeStepsPerPeriod{i,1}.timeSteps = timeSteps;
                    out.numOfTimeSteps(i) = length(timeSteps);
                    out.totalTimeSteps = [out.totalTimeSteps; timeSteps];
                    
                    % 1st period
                    out.startTimeIdx(i) = 1;
                    out.endM1TimeIdx(i) = length(timeSteps);
                    
                else
                    startDate = schedule.startDates(i);
                    endDate = schedule.endDates(i);
                    % we exclude end date from the 
                    endDate = endDate.AddDate(-1,'day');
                    
                    timeSteps = [];
                    % startDate should be included
                    startTime = startDate.DateDiff(valueDate);
                    timeSteps = [timeSteps; startTime];
                    
                    while 1
                        endTime = endDate.DateDiff(valueDate);
                        timeSteps = [timeSteps; endTime];
                        
                        endDate = endDate.AddDate(-1*mcOneTimeStep,'day');
                        if endDate.DateDiff(startDate) < 0 break;end
                    end
                    timeSteps = sort(unique(timeSteps),'ascend');
                    out.timeStepsPerPeriod{i,1}.timeSteps = timeSteps;
                    out.numOfTimeSteps(i) = length(timeSteps);
                    out.totalTimeSteps = [out.totalTimeSteps; timeSteps];
                    
                    % 1st period
                    out.startTimeIdx(i) = out.endM1TimeIdx(i-1) + 1;
                    out.endM1TimeIdx(i) = out.endM1TimeIdx(i-1) + length(timeSteps);
                    
                end
            end
            
            % endDate of the last period (= Security's maturityDate should
            % be included
            
            endTime = DateDiff(schedule.endDates(periodCount),valueDate);
            out.totalTimeSteps = [out.totalTimeSteps;endTime];
            out.totalTimeSteps = sort(unique(out.totalTimeSteps),'ascend');
            out.totalTimeStepsSize = length(out.totalTimeSteps);
            
            out.periodCount = periodCount;
            out.numOfAlivePeriod = numOfAlivePeriod;
        end
        
        function out = generateModelStates(rangeAccrual,valueDate,schedule,numMethodInfo,model)
            
            pathSize = numMethodInfo.pathSize;
            numOfFactors = numMethodInfo.numOfFactors;
            totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
            totalTimeSteps = numMethodInfo.totalTimeSteps;
            Z = numMethodInfo.Z;
            modelStates = zeros(numOfFactors*totalTimeStepsSize,pathSize);
            
            currentStep = totalTimeSteps(1);
            for i=1:totalTimeStepsSize-1
                nextStep = totalTimeSteps(i+1);
                sqrtDt = sqrt((nextStep-currentStep)/365.0);
                stateLocalVariance = model.stateLocalVariance(currentStep,nextStep);
                L = chol(stateLocalVariance);
                prevModelStates = modelStates(numOfFactors*(i-1)+1:numOfFactors*(i-1)+2,:);
                dW = Z(numOfFactors*i+1:numOfFactors*i+2,:);
                modelStates(numOfFactors*i+1:numOfFactors*i+2,:) = prevModelStates + L*dW; 
                currentStep = nextStep;
            end
            
            out = modelStates;
        end
        
        function out = computePriceMCForward(rangeAccrual,valueDate,model)
            schedule = rangeAccrual.schedule;
            
            % numMethodTimeSteps Generate
            mcOneTimeStep = 1;
            outInfo = rangeAccrual.numMethodTimeStepsGenerate(valueDate,schedule,mcOneTimeStep);
            
            numMethodInfo.timeStepsInfo = outInfo;
            
            % mc random number init
            %pathSize = 2^16;
            pathSize = 5000;
            
            %pathSize = 2^10;
            
            rng('default');
            rng(0);
            
            numOfFactors = 2;
            
            Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
            Z=[Z,-Z];
            
            numMethodInfo.pathSize = pathSize;
            numMethodInfo.numOfFactors = numOfFactors;
            numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
            numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
            numMethodInfo.Z = Z;
            
            % numeraire info
            numMatTime = numMethodInfo.totalTimeSteps(end);
            dfNumMat = model.DF(numMatTime/365.0);
            
            %generate model states for the given timeSteps
            modelStates = rangeAccrual.generateModelStates(valueDate,schedule,numMethodInfo,model);
            
            % induct backward and calculate cashflow
            
            
            periodCount = schedule.numOfPeriod;
            
            nonCall.npv = 0;
            
            nonCall.cashflow_e = zeros(periodCount + 1,1);
            nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
            nonCall.rangeProb_e = zeros(periodCount + 1,1);
            
            nonCall.df_e = zeros(periodCount + 1,1);
            nonCall.df_c = zeros(periodCount + 1,1);
            
            % sum of cashflow_npv
            nonCall.payoff = zeros(1,pathSize);
            
            nonCall.payoffStateVectors_cashflow = zeros(periodCount+1,pathSize);
            nonCall.payoffStateVectors_cashflow_npv = zeros(periodCount+1,pathSize);
            nonCall.payoffStateVector_df = zeros(periodCount + 1,pathSize);
            
            payoffStateVectors_cashflow = zeros(1, pathSize);
            payoffStateVectors_cashflow_npv = zeros(1, pathSize);
            payoffStateVectors_df = zeros(1,pathSize);
            
            % we calculate cashflow backward from maturityDate to valueDate
            % rangeAccrual coupons are added at the startDate of the Period
            % At maturity
            
            nominal = rangeAccrual.nominal;
            coupon = rangeAccrual.coupon;
            
            currentTime = numMethodInfo.totalTimeSteps(end);
            totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
            currentTimeIdx = totalTimeStepsSize;
            
            modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
            payoffStateVectors_cashflow = nominal*ones(1,pathSize);
            nonCall.payoffStateVectors_cashflow(end,:) = payoffStateVectors_cashflow;
            
            payoffStateVectors_cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,payoffStateVectors_cashflow);
            
            nonCall.payoffStateVectors_cashflow_npv(end,:) = payoffStateVectors_cashflow_npv;
            
            nonCall.cashflow_npv_e(end) = mean(payoffStateVectors_cashflow_npv);
            
            nonCall.payoff = nonCall.payoff + payoffStateVectors_cashflow_npv;
            
            payoffStateVectors_df = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,1.0*ones(1,pathSize));
            
            nonCall.payoffStateVector_df(end,:) = payoffStateVectors_df;
            nonCall.df_e(end) = mean(nonCall.payoffStateVector_df(end,:));
            nonCall.df_c(end) = model.DF(currentTime/365.0);
%             
%             nonCall.cashflow_npv_df(end,:) =  nonCall.npv/nominal;
%             nonCall.cashflow_npv_df_e(end) = mean(nonCall.cashflow_npv_df(end,:));
            
            lastTimeIdx = currentTimeIdx;
            totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
            
            for i=periodCount:-1:1
                % past schedule neglect
                if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                endM1TimeIdx = numMethodInfo.timeStepsInfo.endM1TimeIdx(i);
                
                %% FromToInduction start
                
                %modelsStates induct backward(lastTimeIdx to endM1TimeIdx)
                while endM1TimeIdx < lastTimeIdx
                    lastTimeIdx = lastTimeIdx -1;
                    if lastTimeIdx == endM1TimeIdx break;end
                end
                
                currentTimeIdx = lastTimeIdx;
                currentTime = totalTimeSteps(currentTimeIdx);
                % now we are at endM1TimeIdx
                % variables for coupon valuation initialized
                ralower = rangeAccrual.lower;
                raupper = rangeAccrual.upper;
                
                ralowerS = rangeAccrual.lowerS;
                raupperS = rangeAccrual.upperS;
                
                tenorCMS = rangeAccrual.tenor;
                
                tenorLong = rangeAccrual.tenorL;
                tenorShort = rangeAccrual.tenorS;
                
                cD = 0; % cumulated day count fraction
                cP = 0; 
                dI = 0; % dummy idx for loop
                dcfC = 0;
                dcfN = 0;
                dD = 0;
                idxCMS = zeros(1,pathSize);
                
                idxL = zeros(1,pathSize);
                idxS = zeros(1,pathSize);
                
                idxC = zeros(1,pathSize);
                idxN = zeros(1,pathSize);
                
                spreadC = zeros(1,pathSize);
                spreadN = zeros(1,pathSize);
                probSpread = zeros(1,pathSize);
                simpleYN = true;
                numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                
                for j=numOfTimeSteps:-1:1
                    % evaluate dealdescription
                    % last step is omitted
                    currentTime = totalTimeSteps(currentTimeIdx);
                    dcfC = currentTime/365.0;
                    if j~= numOfTimeSteps
                        dD = (dcfN-dcfC);
                    end
                    
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
                    idxCMS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorCMS,modelStateT);
                    idxC = idxCMS;
                    
                    idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                    idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                    spreadC = idxL - idxS;
                     
%                     probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                    probSpread = model.DRPr(idxC,idxN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS,simpleYN,pathSize)*dD;
                    
                    cD = cD + dD;
                    cP = cP + probSpread;
  
                    dcfN = dcfC;
                    
                    idxN = idxC;
                    spreadN = spreadC;
                    
                    %induct backward to the next time step
                    % one time backward induction
                    lastTimeIdx = currentTimeIdx;
                    
                    if startTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        currentTimeIdx = lastTimeIdx;
                    end
                    
                    
                end
                
                %% FromToInduction end
                %% AtDate operation
                % Adding a new cashflow to the payoff
                
                dcf = schedule.dayCountFraction(i);
                payoffStateVectors_cashflow = nominal * coupon * dcf * cP/cD;
                
                % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                % i.e. currentTimeIdx has not changed
                % no need to upodate modelStateT
                % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                % since cashflow's are added at the startDate(i) we need to
                % multiply cashflow with the df(startDate(i), payDate(i))
                payTime = DateDiff(schedule.payDates(i),valueDate);
                df_se = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);
                
                for k=1:pathSize
                    payoffStateVectors_cashflow(k) = payoffStateVectors_cashflow(k) + df_se(k);
                end
                
                nonCall.payoffStateVectors_cashflow(i,:) = payoffStateVectors_cashflow;
                
                nonCall.cashflow_e(i) = mean(payoffStateVectors_cashflow);
                nonCall.rangeProb_e(i) = mean(nonCall.cashflow_e(i)/(nominal * coupon * dcf));
                
                payoffStateVectors_cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,payoffStateVectors_cashflow);
            
                nonCall.cashflow_npv_e(i) = mean(payoffStateVectors_cashflow_npv);
                
                nonCall.payoffStateVectors_cashflow_npv(i,:) = payoffStateVectors_cashflow_npv;
                nonCall.payoff = nonCall.payoff + payoffStateVectors_cashflow_npv;
                
                payoffStateVectors_df = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,1.0*ones(1,pathSize));
                
                nonCall.payoffStateVector_df(i,:) = payoffStateVectors_df;
                nonCall.df_e(i) = mean(nonCall.payoffStateVector_df(i,:));
                nonCall.df_c(i) = model.DF(currentTime/365.0);
                
            end
            
            nonCall.npv = mean(nonCall.payoff);
  
            out.cashflow= nonCall.cashflow_e;
            out.cashflowNpv = nonCall.cashflow_npv_e;
            out.df_e = nonCall.df_e;
            out.df_c = nonCall.df_c;
            out.daycountFraction = schedule.dayCountFraction(i);
            out.rangeProb = nonCall.rangeProb_e;
            out.npv = nonCall.npv;
           
        end
        
        function out = computePrice(rangeAccrual,valueDate,model)
            schedule = rangeAccrual.schedule;
            periodCount = schedule.numOfPeriod;
            rangeProb = zeros(periodCount,1);
            discountFactor = zeros(periodCount,1);
            daycountFraction = zeros(periodCount,1);
            cashflow = zeros(periodCount,1);
            cashflowNpv = zeros(periodCount,1);
            rangeDays = rangeAccrual.rangeDays;
            npv = 0;
            for i=1:periodCount
                if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end;
                
                observeDate = schedule.startDates(i);
                if DateDiff(valueDate,schedule.startDates(i)) >= 0
                    observeDate = valueDate;
                    observeDate = AddDate(observeDate,1,'day');
                else
                    observeDate = schedule.startDates(i);
                end
                
                observeDays = DateDiff(schedule.endDates(i),schedule.startDates(i));
                days = DateDiff(schedule.endDates(i), observeDate);
                discountFactor(i) = model.DF( DateDiff(schedule.payDates(i),valueDate)/365.0);
                daycountFraction(i) = schedule.dayCountFraction(i);
                for j=1:days
                    toDate = observeDate;
                    tpDate = schedule.payDates(i);
                    tenor1 = rangeAccrual.tenorL;
                    tenor2 = rangeAccrual.tenorS;
                    tau =0.25;
                    lower = rangeAccrual.lower;
                    upper = rangeAccrual.upper;
                    rangeValue = model.CMSSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,lower,upper);
%                     rangeValue = 1.0;
                    rangeProb(i) = rangeProb(i) + rangeValue;
                    observeDate = AddDate(observeDate,1,'day');
                end
                cashflow(i) = rangeAccrual.nominal * rangeAccrual.coupon * (rangeProb(i) + rangeDays(i))/observeDays * daycountFraction(i);
                cashflowNpv(i) = cashflow(i) * discountFactor(i);
                npv = npv + cashflowNpv(i);
                % bug fixed observeDays -> days
                rangeProb(i) = rangeProb(i)/days;
                
                if i == periodCount
                    npv = npv + rangeAccrual.nominal * discountFactor(i);
                end
            end
            out.cashflow= cashflow;
            out.cashflowNpv = cashflowNpv;
            out.discountFactor = discountFactor;
            out.daycountFraction = daycountFraction;
            out.rangeProb = rangeProb;
            out.npv = npv;
        end
        
%         function out = discountPayoff(rangeAccrual,eventTime,numMatTime ...
%                 ,model,modelStatesT,cashflow)
%             
%             dfNumMat = model.DF(numMatTime/365.0);
%             % for discountPayoff T=numMatTime
%             % P(0,TnumMat)/P(Te,Tnummat)*Payoff
%             %
%             discountT = model.discountFactor(eventTime, numMatTime,numMatTime,modelStatesT);
%             stateSize = size(modelStatesT,2);
%             payoff = zeros(1,stateSize);
%             for i=1:stateSize
%                 payoff(i)= dfNumMat/discountT(i)*cashflow(i);
%             end    
%             out = payoff;
%         end
        
        
        
    end
    
end



In [ ]:
classdef CDAndCDKTBSpreadDualRA < Security
    %RangeAccrual is the SuperClass of all IRModel
    % properties: zeroCurve 
    % methods : DF, FWD,PV01,FSR
    properties(SetAccess = public)
        nominal
        couponInfo
        callInfo
        manualScheduleYN
        
    end
    
    methods
        % constructor
        function rangeAccrual = CDAndCDKTBSpreadDualRA(params)
                rangeAccrual.payCcy = params.payCcy;
                rangeAccrual.startDate = params.startDate;
                rangeAccrual.maturity = params.maturity;
                
                %rangeAccrual.maturityDate = params.startDate.AddDate(params.maturity,'year');
                rangeAccrual.nominal = params.params('nominal');
                rangeAccrual.coupon = params.params('coupon');
                rangeAccrual.couponFreq= params.params('couponFreq');
                rangeAccrual.tenor = params.params('tenor');
                rangeAccrual.tenorL = params.params('tenorL');
                rangeAccrual.tenorS = params.params('tenorS');
                rangeAccrual.lower = params.params('lower');
                rangeAccrual.upper = params.params('upper');
                rangeAccrual.lowerS = params.params('lowerS');
                rangeAccrual.upperS = params.params('upperS');
                rangeAccrual.rangeDays = params.params('rangeDays');
                rangeAccrual.callYN = params.params('callYN');
                rangeAccrual.manualScheduleYN = params.params('manualScheduleYN');
                if rangeAccrual.manualScheduleYN
                    rangeAccrual.couponSchedule = rangeAccrual.ManualScheduleGenerate(params.params('couponScheduleInt'));
                    rangeAccrual.callSchedule = rangeAccrual.ManualScheduleGenerate(params.params('callScheduleInt'));
                    
                
                else
                    maturityDate = params.startDate.AddDate(params.maturity,'year');
                    rangeAccrual.schedule = rangeAccrual.ScheduleGenerate(rangeAccrual.startDate,maturityDate,rangeAccrual.couponFreq);
                
                end
                 
        end
        
        function out = ManualScheduleGenerate(rangeAccrual,scheduleInt)
            scheduleSize = size(scheduleInt,1);
            out.numOfPeriod = scheduleSize;
            for i=1:scheduleSize
                out.resetDates(i) = H_Date(scheduleInt(i,1));
                out.startDates(i) = H_Date(scheduleInt(i,2));
                out.endDates(i) = H_Date(scheduleInt(i,3));
                out.payDates(i) = H_Date(scheduleInt(i,4));
                out.dayCount(i) = out.endDates(i).DateDiff(out.startDates(i));
                out.dayCountFraction(i) = out.dayCount(i)/ 365.0;
            end
            
            
        end
        
        function out = ScheduleGenerate(rangeAccrual,initialDate,maturityDate,freq)
            schedule = ScheduleGenerate(initialDate,maturityDate,freq);
            out.startDates= schedule.startDates;
            out.resetDates = schedule.resetDates;
            out.endDates = schedule.endDates;
            out.payDates = schedule.payDates;
            out.dayCount = schedule.dayCount;
            out.dayCountFraction = schedule.dayCountFraction;
            out.numOfPeriod = schedule.numOfPeriod;
        end
        
        function out = numMethodTimeStepsGenerate(rangeAccrual,valueDate,schedule,mcOneTimeStep)
            
            periodCount = schedule.numOfPeriod;
            out.timeStepsPerPeriod =  cell(periodCount,1);
            
            out.startTimeIdx = zeros(periodCount,1);
            out.endM1TimeIdx = zeros(periodCount,1);
            
            out.numOfTimeSteps = zeros(periodCount,1);
            out.totalTimeSteps = [];
            out.totalTimeStepsSize = 0;
            numOfAlivePeriod = 0;
            
            for i=1:periodCount
                timeSteps = [];
                if DateDiff(valueDate,schedule.payDates(i)) >= 0
                    out.timeStepsPerPeriod(i) = timeSteps;
                    continue;
                end
                
                numOfAlivePeriod = numOfAlivePeriod + 1;
                if DateDiff(valueDate,schedule.startDates(i)) >= 0
                    startDate = valueDate;
                    endDate = schedule.endDates(i);
                    % we exclude end date from the 
                    endDate = endDate.AddDate(-1,'day');
                    
                    timeSteps = [];
                    % startDate should be included
                    startTime = startDate.DateDiff(valueDate);
                    timeSteps = [timeSteps; startTime];
                    
                    while 1
                        endTime = endDate.DateDiff(valueDate);
                        timeSteps = [timeSteps; endTime];
                        endDate = endDate.AddDate(-1*mcOneTimeStep,'day');
                        if endDate.DateDiff(startDate) < 0 break;end
                    end
                    timeSteps = sort(unique(timeSteps),'ascend');
                    %out.timeStepsPerPeriod {i,1}.timeSteps = [];
                    out.timeStepsPerPeriod{i,1}.timeSteps = timeSteps;
                    out.numOfTimeSteps(i) = length(timeSteps);
                    out.totalTimeSteps = [out.totalTimeSteps; timeSteps];
                    
                    % 1st period
                    out.startTimeIdx(i) = 1;
                    out.endM1TimeIdx(i) = length(timeSteps);
                    
                else
                    startDate = schedule.startDates(i);
                    endDate = schedule.endDates(i);
                    % we exclude end date from the 
                    endDate = endDate.AddDate(-1,'day');
                    
                    timeSteps = [];
                    % startDate should be included
                    startTime = startDate.DateDiff(valueDate);
                    timeSteps = [timeSteps; startTime];
                    
                    while 1
                        endTime = endDate.DateDiff(valueDate);
                        timeSteps = [timeSteps; endTime];
                        
                        endDate = endDate.AddDate(-1*mcOneTimeStep,'day');
                        if endDate.DateDiff(startDate) < 0 break;end
                    end
                    timeSteps = sort(unique(timeSteps),'ascend');
                    out.timeStepsPerPeriod{i,1}.timeSteps = timeSteps;
                    out.numOfTimeSteps(i) = length(timeSteps);
                    out.totalTimeSteps = [out.totalTimeSteps; timeSteps];
                    
                    % 1st period
                    out.startTimeIdx(i) = out.endM1TimeIdx(i-1) + 1;
                    out.endM1TimeIdx(i) = out.endM1TimeIdx(i-1) + length(timeSteps);
                    
                end
            end
            
            % endDate of the last period (= Security's maturityDate should
            % be included
            
            endTime = DateDiff(schedule.endDates(periodCount),valueDate);
            out.totalTimeSteps = [out.totalTimeSteps;endTime];
            out.totalTimeSteps = sort(unique(out.totalTimeSteps),'ascend');
            out.totalTimeStepsSize = length(out.totalTimeSteps);
            
            out.periodCount = periodCount;
            out.numOfAlivePeriod = numOfAlivePeriod;
        end
        
        function out = generateModelStates(rangeAccrual,valueDate,schedule,numMethodInfo,model)
            
            pathSize = numMethodInfo.pathSize;
            numOfFactors = numMethodInfo.numOfFactors;
            totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
            totalTimeSteps = numMethodInfo.totalTimeSteps;
            Z = numMethodInfo.Z;
            modelStates = zeros(numOfFactors*totalTimeStepsSize,pathSize);
            
            currentStep = totalTimeSteps(1);
            for i=1:totalTimeStepsSize-1
                nextStep = totalTimeSteps(i+1);
                sqrtDt = sqrt((nextStep-currentStep)/365.0);
                stateLocalVariance = model.stateLocalVariance(currentStep,nextStep);
                L = chol(stateLocalVariance);
                prevModelStates = modelStates(numOfFactors*(i-1)+1:numOfFactors*(i-1)+2,:);
                dW = Z(numOfFactors*i+1:numOfFactors*i+2,:);
                modelStates(numOfFactors*i+1:numOfFactors*i+2,:) = prevModelStates + L*dW; 
                currentStep = nextStep;
            end
            
            out = modelStates;
        end
        
        function out = computePriceMCForward(rangeAccrual,valueDate,model)
            schedule = rangeAccrual.schedule;
            
            % numMethodTimeSteps Generate
            mcOneTimeStep = 1;
            outInfo = rangeAccrual.numMethodTimeStepsGenerate(valueDate,schedule,mcOneTimeStep);
            
            numMethodInfo.timeStepsInfo = outInfo;
            
            % mc random number init
            %pathSize = 2^16;
            pathSize = 5000;
            
            %pathSize = 2^10;
            
            rng('default');
            rng(0);
            
            numOfFactors = 2;
            
            Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
            Z=[Z,-Z];
            
            numMethodInfo.pathSize = pathSize;
            numMethodInfo.numOfFactors = numOfFactors;
            numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
            numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
            numMethodInfo.Z = Z;
            
            % numeraire info
            numMatTime = numMethodInfo.totalTimeSteps(end);
            dfNumMat = model.DF(numMatTime/365.0);
            
            %generate model states for the given timeSteps
            modelStates = rangeAccrual.generateModelStates(valueDate,schedule,numMethodInfo,model);
            
            % induct backward and calculate cashflow
            
            
            periodCount = schedule.numOfPeriod;
            
            nonCall.npv = 0;
            
            nonCall.cashflow_e = zeros(periodCount + 1,1);
            nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
            nonCall.rangeProb_e = zeros(periodCount + 1,1);
            
            nonCall.df_e = zeros(periodCount + 1,1);
            nonCall.df_c = zeros(periodCount + 1,1);
            
            % sum of cashflow_npv
            nonCall.payoff = zeros(1,pathSize);
            
            nonCall.payoffStateVectors_cashflow = zeros(periodCount+1,pathSize);
            nonCall.payoffStateVectors_cashflow_npv = zeros(periodCount+1,pathSize);
            nonCall.payoffStateVector_df = zeros(periodCount + 1,pathSize);
            
            payoffStateVectors_cashflow = zeros(1, pathSize);
            payoffStateVectors_cashflow_npv = zeros(1, pathSize);
            payoffStateVectors_df = zeros(1,pathSize);
            
            % we calculate cashflow backward from maturityDate to valueDate
            % rangeAccrual coupons are added at the startDate of the Period
            % At maturity
            
            nominal = rangeAccrual.nominal;
            coupon = rangeAccrual.coupon;
            
            currentTime = numMethodInfo.totalTimeSteps(end);
            totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
            currentTimeIdx = totalTimeStepsSize;
            
            modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
            payoffStateVectors_cashflow = nominal*ones(1,pathSize);
            nonCall.payoffStateVectors_cashflow(end,:) = payoffStateVectors_cashflow;
            
            payoffStateVectors_cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,payoffStateVectors_cashflow);
            
            nonCall.payoffStateVectors_cashflow_npv(end,:) = payoffStateVectors_cashflow_npv;
            
            nonCall.cashflow_npv_e(end) = mean(payoffStateVectors_cashflow_npv);
            
            nonCall.payoff = nonCall.payoff + payoffStateVectors_cashflow_npv;
            
            payoffStateVectors_df = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,1.0*ones(1,pathSize));
            
            nonCall.payoffStateVector_df(end,:) = payoffStateVectors_df;
            nonCall.df_e(end) = mean(nonCall.payoffStateVector_df(end,:));
            nonCall.df_c(end) = model.DF(currentTime/365.0);
%             
%             nonCall.cashflow_npv_df(end,:) =  nonCall.npv/nominal;
%             nonCall.cashflow_npv_df_e(end) = mean(nonCall.cashflow_npv_df(end,:));
            
            lastTimeIdx = currentTimeIdx;
            totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
            
            for i=periodCount:-1:1
                % past schedule neglect
                if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                endM1TimeIdx = numMethodInfo.timeStepsInfo.endM1TimeIdx(i);
                
                %% FromToInduction start
                
                %modelsStates induct backward(lastTimeIdx to endM1TimeIdx)
                while endM1TimeIdx < lastTimeIdx
                    lastTimeIdx = lastTimeIdx -1;
                    if lastTimeIdx == endM1TimeIdx break;end
                end
                
                currentTimeIdx = lastTimeIdx;
                currentTime = totalTimeSteps(currentTimeIdx);
                % now we are at endM1TimeIdx
                % variables for coupon valuation initialized
                ralower = rangeAccrual.lower;
                raupper = rangeAccrual.upper;
                
                ralowerS = rangeAccrual.lowerS;
                raupperS = rangeAccrual.upperS;
                
                tenorCMS = rangeAccrual.tenor;
                
                tenorLong = rangeAccrual.tenorL;
                tenorShort = rangeAccrual.tenorS;
                
                cD = 0; % cumulated day count fraction
                cP = 0; 
                dI = 0; % dummy idx for loop
                dcfC = 0;
                dcfN = 0;
                dD = 0;
                idxCMS = zeros(1,pathSize);
                
                idxL = zeros(1,pathSize);
                idxS = zeros(1,pathSize);
                
                idxC = zeros(1,pathSize);
                idxN = zeros(1,pathSize);
                
                spreadC = zeros(1,pathSize);
                spreadN = zeros(1,pathSize);
                probSpread = zeros(1,pathSize);
                simpleYN = true;
                numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                
                for j=numOfTimeSteps:-1:1
                    % evaluate dealdescription
                    % last step is omitted
                    currentTime = totalTimeSteps(currentTimeIdx);
                    dcfC = currentTime/365.0;
                    if j~= numOfTimeSteps
                        dD = (dcfN-dcfC);
                    end
                    
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
                    idxCMS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorCMS,modelStateT);
                    idxC = idxCMS;
                    
                    idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                    idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                    spreadC = idxL - idxS;
                     
%                     probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                    probSpread = model.DRPr(idxC,idxN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS,simpleYN,pathSize)*dD;
                    
                    cD = cD + dD;
                    cP = cP + probSpread;
  
                    dcfN = dcfC;
                    
                    idxN = idxC;
                    spreadN = spreadC;
                    
                    %induct backward to the next time step
                    % one time backward induction
                    lastTimeIdx = currentTimeIdx;
                    
                    if startTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        currentTimeIdx = lastTimeIdx;
                    end
                    
                    
                end
                
                %% FromToInduction end
                %% AtDate operation
                % Adding a new cashflow to the payoff
                
                dcf = schedule.dayCountFraction(i);
                payoffStateVectors_cashflow = nominal * coupon * dcf * cP/cD;
                
                % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                % i.e. currentTimeIdx has not changed
                % no need to upodate modelStateT
                % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                % since cashflow's are added at the startDate(i) we need to
                % multiply cashflow with the df(startDate(i), payDate(i))
                payTime = DateDiff(schedule.payDates(i),valueDate);
                df_se = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);
                
                for k=1:pathSize
                    payoffStateVectors_cashflow(k) = payoffStateVectors_cashflow(k) + df_se(k);
                end
                
                nonCall.payoffStateVectors_cashflow(i,:) = payoffStateVectors_cashflow;
                
                nonCall.cashflow_e(i) = mean(payoffStateVectors_cashflow);
                nonCall.rangeProb_e(i) = mean(nonCall.cashflow_e(i)/(nominal * coupon * dcf));
                
                payoffStateVectors_cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,payoffStateVectors_cashflow);
            
                nonCall.cashflow_npv_e(i) = mean(payoffStateVectors_cashflow_npv);
                
                nonCall.payoffStateVectors_cashflow_npv(i,:) = payoffStateVectors_cashflow_npv;
                nonCall.payoff = nonCall.payoff + payoffStateVectors_cashflow_npv;
                
                payoffStateVectors_df = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,1.0*ones(1,pathSize));
                
                nonCall.payoffStateVector_df(i,:) = payoffStateVectors_df;
                nonCall.df_e(i) = mean(nonCall.payoffStateVector_df(i,:));
                nonCall.df_c(i) = model.DF(currentTime/365.0);
                
            end
            
            nonCall.npv = mean(nonCall.payoff);
  
            out.cashflow= nonCall.cashflow_e;
            out.cashflowNpv = nonCall.cashflow_npv_e;
            out.df_e = nonCall.df_e;
            out.df_c = nonCall.df_c;
            out.daycountFraction = schedule.dayCountFraction(i);
            out.rangeProb = nonCall.rangeProb_e;
            out.npv = nonCall.npv;
           
        end
        
        function out = computePrice(rangeAccrual,valueDate,model)
            schedule = rangeAccrual.schedule;
            periodCount = schedule.numOfPeriod;
            rangeProb = zeros(periodCount,1);
            discountFactor = zeros(periodCount,1);
            daycountFraction = zeros(periodCount,1);
            cashflow = zeros(periodCount,1);
            cashflowNpv = zeros(periodCount,1);
            rangeDays = rangeAccrual.rangeDays;
            npv = 0;
            for i=1:periodCount
                if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end;
                
                observeDate = schedule.startDates(i);
                if DateDiff(valueDate,schedule.startDates(i)) >= 0
                    observeDate = valueDate;
                    observeDate = AddDate(observeDate,1,'day');
                else
                    observeDate = schedule.startDates(i);
                end
                
                observeDays = DateDiff(schedule.endDates(i),schedule.startDates(i));
                days = DateDiff(schedule.endDates(i), observeDate);
                discountFactor(i) = model.DF( DateDiff(schedule.payDates(i),valueDate)/365.0);
                daycountFraction(i) = schedule.dayCountFraction(i);
                for j=1:days
                    toDate = observeDate;
                    tpDate = schedule.payDates(i);
                    tenor1 = rangeAccrual.tenorL;
                    tenor2 = rangeAccrual.tenorS;
                    tau =0.25;
                    lower = rangeAccrual.lower;
                    upper = rangeAccrual.upper;
                    rangeValue = model.CMSSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,lower,upper);
%                     rangeValue = 1.0;
                    rangeProb(i) = rangeProb(i) + rangeValue;
                    observeDate = AddDate(observeDate,1,'day');
                end
                cashflow(i) = rangeAccrual.nominal * rangeAccrual.coupon * (rangeProb(i) + rangeDays(i))/observeDays * daycountFraction(i);
                cashflowNpv(i) = cashflow(i) * discountFactor(i);
                npv = npv + cashflowNpv(i);
                % bug fixed observeDays -> days
                rangeProb(i) = rangeProb(i)/days;
                
                if i == periodCount
                    npv = npv + rangeAccrual.nominal * discountFactor(i);
                end
            end
            out.cashflow= cashflow;
            out.cashflowNpv = cashflowNpv;
            out.discountFactor = discountFactor;
            out.daycountFraction = daycountFraction;
            out.rangeProb = rangeProb;
            out.npv = npv;
        end
        
%         function out = discountPayoff(rangeAccrual,eventTime,numMatTime ...
%                 ,model,modelStatesT,cashflow)
%             
%             dfNumMat = model.DF(numMatTime/365.0);
%             % for discountPayoff T=numMatTime
%             % P(0,TnumMat)/P(Te,Tnummat)*Payoff
%             %
%             discountT = model.discountFactor(eventTime, numMatTime,numMatTime,modelStatesT);
%             stateSize = size(modelStatesT,2);
%             payoff = zeros(1,stateSize);
%             for i=1:stateSize
%                 payoff(i)= dfNumMat/discountT(i)*cashflow(i);
%             end    
%             out = payoff;
%         end
        
        
        
    end
    
end



In [ ]:
classdef CDAndCDKTBSpreadDualRA < Security
    %RangeAccrual is the SuperClass of all IRModel
    % properties: zeroCurve 
    % methods : DF, FWD,PV01,FSR
    properties(SetAccess = public)
        nominal
        couponInfo
        callableInfo
        manualScheduleYN
        
    end
    
    methods
        % constructor
        function rangeAccrual = CDAndCDKTBSpreadDualRA(params)
                security = Security(params);
                rangeAccrual.payCcy = security.payCcy;
                rangeAccrual.startDate = security.startDate;
                rangeAccrual.maturity = security.maturity;
                rangeAccrual.endDate = security.endDate;
                
                rangeAccrual.couponInfo = params.params.couponInfo;
                rangeAccrual.callableInfo = params.params.callableInfo;
                rangeAccrual.manualScheduleYN = params.params.manualScheduleYN;
                
                if rangeAccrual.manualScheduleYN
                    rangeAccrual.couponInfo.couponSchedule = rangeAccrual.ManualScheduleGenerate(rangeAccrual.couponInfo.couponScheduleInt);
                    rangeAccrual.callableInfo.callableSchedule = rangeAccrual.ManualScheduleGenerateCall(rangeAccrual.callableInfo.callScheduleInt);
                    
                
                else
                    maturityDate = params.startDate.AddDate(params.maturity,'year');
                    rangeAccrual.schedule = rangeAccrual.ScheduleGenerate(rangeAccrual.startDate,maturityDate,rangeAccrual.couponFreq);

                end
                 
        end
        
        function out = ManualScheduleGenerate(rangeAccrual,scheduleInt)
            scheduleSize = size(scheduleInt,1);
            out.scheduleSize = scheduleSize;
            for i=1:scheduleSize
                out.resetDates(i) = H_Date(scheduleInt(i,1));
                out.startDates(i) = H_Date(scheduleInt(i,2));
                out.endDates(i) = H_Date(scheduleInt(i,3));
                out.payDates(i) = H_Date(scheduleInt(i,4));
                out.dayCount(i) = out.endDates(i).DateDiff(out.startDates(i));
                out.dayCountFraction(i) = out.dayCount(i)/ 365.0;
            end
            
            
        end
        
        function out = ManualScheduleGenerateCall(rangeAccrual,scheduleInt)
            scheduleSize = size(scheduleInt,1);
            out.scheduleSize = scheduleSize;
            for i=1:scheduleSize
                out.startDates(i) = H_Date(scheduleInt(i,1));
                out.payDates(i) = H_Date(scheduleInt(i,2));
            end
            
            
        end
        
        function out = ScheduleGenerate(rangeAccrual,initialDate,maturityDate,freq)
            schedule = ScheduleGenerate(initialDate,maturityDate,freq);
            out.startDates= schedule.startDates;
            out.resetDates = schedule.resetDates;
            out.endDates = schedule.endDates;
            out.payDates = schedule.payDates;
            out.dayCount = schedule.dayCount;
            out.dayCountFraction = schedule.dayCountFraction;
            out.numOfPeriod = schedule.numOfPeriod;
        end
        
        function out = numMethodTimeStepsGenerate(rangeAccrual,valueDate,schedule,mcOneTimeStep)
            
            periodCount = schedule.numOfPeriod;
            out.timeStepsPerPeriod =  cell(periodCount,1);
            
            out.startTimeIdx = zeros(periodCount,1);
            out.endM1TimeIdx = zeros(periodCount,1);
            
            out.numOfTimeSteps = zeros(periodCount,1);
            out.totalTimeSteps = [];
            out.totalTimeStepsSize = 0;
            numOfAlivePeriod = 0;
            
            for i=1:periodCount
                timeSteps = [];
                if DateDiff(valueDate,schedule.payDates(i)) >= 0
                    out.timeStepsPerPeriod(i) = timeSteps;
                    continue;
                end
                
                numOfAlivePeriod = numOfAlivePeriod + 1;
                if DateDiff(valueDate,schedule.startDates(i)) >= 0
                    startDate = valueDate;
                    endDate = schedule.endDates(i);
                    % we exclude end date from the 
                    endDate = endDate.AddDate(-1,'day');
                    
                    timeSteps = [];
                    % startDate should be included
                    startTime = startDate.DateDiff(valueDate);
                    timeSteps = [timeSteps; startTime];
                    
                    while 1
                        endTime = endDate.DateDiff(valueDate);
                        timeSteps = [timeSteps; endTime];
                        endDate = endDate.AddDate(-1*mcOneTimeStep,'day');
                        if endDate.DateDiff(startDate) < 0 break;end
                    end
                    timeSteps = sort(unique(timeSteps),'ascend');
                    %out.timeStepsPerPeriod {i,1}.timeSteps = [];
                    out.timeStepsPerPeriod{i,1}.timeSteps = timeSteps;
                    out.numOfTimeSteps(i) = length(timeSteps);
                    out.totalTimeSteps = [out.totalTimeSteps; timeSteps];
                    
                    % 1st period
                    out.startTimeIdx(i) = 1;
                    out.endM1TimeIdx(i) = length(timeSteps);
                    
                else
                    startDate = schedule.startDates(i);
                    endDate = schedule.endDates(i);
                    % we exclude end date from the 
                    endDate = endDate.AddDate(-1,'day');
                    
                    timeSteps = [];
                    % startDate should be included
                    startTime = startDate.DateDiff(valueDate);
                    timeSteps = [timeSteps; startTime];
                    
                    while 1
                        endTime = endDate.DateDiff(valueDate);
                        timeSteps = [timeSteps; endTime];
                        
                        endDate = endDate.AddDate(-1*mcOneTimeStep,'day');
                        if endDate.DateDiff(startDate) < 0 break;end
                    end
                    timeSteps = sort(unique(timeSteps),'ascend');
                    out.timeStepsPerPeriod{i,1}.timeSteps = timeSteps;
                    out.numOfTimeSteps(i) = length(timeSteps);
                    out.totalTimeSteps = [out.totalTimeSteps; timeSteps];
                    
                    % 1st period
                    out.startTimeIdx(i) = out.endM1TimeIdx(i-1) + 1;
                    out.endM1TimeIdx(i) = out.endM1TimeIdx(i-1) + length(timeSteps);
                    
                end
            end
            
            % endDate of the last period (= Security's maturityDate should
            % be included
            
            endTime = DateDiff(schedule.endDates(periodCount),valueDate);
            out.totalTimeSteps = [out.totalTimeSteps;endTime];
            out.totalTimeSteps = sort(unique(out.totalTimeSteps),'ascend');
            out.totalTimeStepsSize = length(out.totalTimeSteps);
            
            out.periodCount = periodCount;
            out.numOfAlivePeriod = numOfAlivePeriod;
        end
        
        function out = generateModelStates(rangeAccrual,valueDate,schedule,numMethodInfo,model)
            
            pathSize = numMethodInfo.pathSize;
            numOfFactors = numMethodInfo.numOfFactors;
            totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
            totalTimeSteps = numMethodInfo.totalTimeSteps;
            Z = numMethodInfo.Z;
            modelStates = zeros(numOfFactors*totalTimeStepsSize,pathSize);
            
            currentStep = totalTimeSteps(1);
            for i=1:totalTimeStepsSize-1
                nextStep = totalTimeSteps(i+1);
                sqrtDt = sqrt((nextStep-currentStep)/365.0);
                stateLocalVariance = model.stateLocalVariance(currentStep,nextStep);
                L = chol(stateLocalVariance);
                prevModelStates = modelStates(numOfFactors*(i-1)+1:numOfFactors*(i-1)+2,:);
                dW = Z(numOfFactors*i+1:numOfFactors*i+2,:);
                modelStates(numOfFactors*i+1:numOfFactors*i+2,:) = prevModelStates + L*dW; 
                currentStep = nextStep;
            end
            
            out = modelStates;
        end
        
        function out = computePriceMCForward(rangeAccrual,valueDate,model)
            schedule = rangeAccrual.schedule;
            
            % numMethodTimeSteps Generate
            mcOneTimeStep = 1;
            outInfo = rangeAccrual.numMethodTimeStepsGenerate(valueDate,schedule,mcOneTimeStep);
            
            numMethodInfo.timeStepsInfo = outInfo;
            
            % mc random number init
            %pathSize = 2^16;
            pathSize = 5000;
            
            %pathSize = 2^10;
            
            rng('default');
            rng(0);
            
            numOfFactors = 2;
            
            Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
            Z=[Z,-Z];
            
            numMethodInfo.pathSize = pathSize;
            numMethodInfo.numOfFactors = numOfFactors;
            numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
            numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
            numMethodInfo.Z = Z;
            
            % numeraire info
            numMatTime = numMethodInfo.totalTimeSteps(end);
            dfNumMat = model.DF(numMatTime/365.0);
            
            %generate model states for the given timeSteps
            modelStates = rangeAccrual.generateModelStates(valueDate,schedule,numMethodInfo,model);
            
            % induct backward and calculate cashflow
            
            
            periodCount = schedule.numOfPeriod;
            
            nonCall.npv = 0;
            
            nonCall.cashflow_e = zeros(periodCount + 1,1);
            nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
            nonCall.rangeProb_e = zeros(periodCount + 1,1);
            
            nonCall.df_e = zeros(periodCount + 1,1);
            nonCall.df_c = zeros(periodCount + 1,1);
            
            % sum of cashflow_npv
            nonCall.payoff = zeros(1,pathSize);
            
            nonCall.payoffStateVectors_cashflow = zeros(periodCount+1,pathSize);
            nonCall.payoffStateVectors_cashflow_npv = zeros(periodCount+1,pathSize);
            nonCall.payoffStateVector_df = zeros(periodCount + 1,pathSize);
            
            payoffStateVectors_cashflow = zeros(1, pathSize);
            payoffStateVectors_cashflow_npv = zeros(1, pathSize);
            payoffStateVectors_df = zeros(1,pathSize);
            
            % we calculate cashflow backward from maturityDate to valueDate
            % rangeAccrual coupons are added at the startDate of the Period
            % At maturity
            
            nominal = rangeAccrual.nominal;
            coupon = rangeAccrual.coupon;
            
            currentTime = numMethodInfo.totalTimeSteps(end);
            totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
            currentTimeIdx = totalTimeStepsSize;
            
            modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
            payoffStateVectors_cashflow = nominal*ones(1,pathSize);
            nonCall.payoffStateVectors_cashflow(end,:) = payoffStateVectors_cashflow;
            
            payoffStateVectors_cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,payoffStateVectors_cashflow);
            
            nonCall.payoffStateVectors_cashflow_npv(end,:) = payoffStateVectors_cashflow_npv;
            
            nonCall.cashflow_npv_e(end) = mean(payoffStateVectors_cashflow_npv);
            
            nonCall.payoff = nonCall.payoff + payoffStateVectors_cashflow_npv;
            
            payoffStateVectors_df = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,1.0*ones(1,pathSize));
            
            nonCall.payoffStateVector_df(end,:) = payoffStateVectors_df;
            nonCall.df_e(end) = mean(nonCall.payoffStateVector_df(end,:));
            nonCall.df_c(end) = model.DF(currentTime/365.0);
%             
%             nonCall.cashflow_npv_df(end,:) =  nonCall.npv/nominal;
%             nonCall.cashflow_npv_df_e(end) = mean(nonCall.cashflow_npv_df(end,:));
            
            lastTimeIdx = currentTimeIdx;
            totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
            
            for i=periodCount:-1:1
                % past schedule neglect
                if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                endM1TimeIdx = numMethodInfo.timeStepsInfo.endM1TimeIdx(i);
                
                %% FromToInduction start
                
                %modelsStates induct backward(lastTimeIdx to endM1TimeIdx)
                while endM1TimeIdx < lastTimeIdx
                    lastTimeIdx = lastTimeIdx -1;
                    if lastTimeIdx == endM1TimeIdx break;end
                end
                
                currentTimeIdx = lastTimeIdx;
                currentTime = totalTimeSteps(currentTimeIdx);
                % now we are at endM1TimeIdx
                % variables for coupon valuation initialized
                ralower = rangeAccrual.lower;
                raupper = rangeAccrual.upper;
                
                ralowerS = rangeAccrual.lowerS;
                raupperS = rangeAccrual.upperS;
                
                tenorCMS = rangeAccrual.tenor;
                
                tenorLong = rangeAccrual.tenorL;
                tenorShort = rangeAccrual.tenorS;
                
                cD = 0; % cumulated day count fraction
                cP = 0; 
                dI = 0; % dummy idx for loop
                dcfC = 0;
                dcfN = 0;
                dD = 0;
                idxCMS = zeros(1,pathSize);
                
                idxL = zeros(1,pathSize);
                idxS = zeros(1,pathSize);
                
                idxC = zeros(1,pathSize);
                idxN = zeros(1,pathSize);
                
                spreadC = zeros(1,pathSize);
                spreadN = zeros(1,pathSize);
                probSpread = zeros(1,pathSize);
                simpleYN = true;
                numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                
                for j=numOfTimeSteps:-1:1
                    % evaluate dealdescription
                    % last step is omitted
                    currentTime = totalTimeSteps(currentTimeIdx);
                    dcfC = currentTime/365.0;
                    if j~= numOfTimeSteps
                        dD = (dcfN-dcfC);
                    end
                    
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
                    idxCMS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorCMS,modelStateT);
                    idxC = idxCMS;
                    
                    idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                    idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                    spreadC = idxL - idxS;
                     
%                     probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                    probSpread = model.DRPr(idxC,idxN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS,simpleYN,pathSize)*dD;
                    
                    cD = cD + dD;
                    cP = cP + probSpread;
  
                    dcfN = dcfC;
                    
                    idxN = idxC;
                    spreadN = spreadC;
                    
                    %induct backward to the next time step
                    % one time backward induction
                    lastTimeIdx = currentTimeIdx;
                    
                    if startTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        currentTimeIdx = lastTimeIdx;
                    end
                    
                    
                end
                
                %% FromToInduction end
                %% AtDate operation
                % Adding a new cashflow to the payoff
                
                dcf = schedule.dayCountFraction(i);
                payoffStateVectors_cashflow = nominal * coupon * dcf * cP/cD;
                
                % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                % i.e. currentTimeIdx has not changed
                % no need to upodate modelStateT
                % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                % since cashflow's are added at the startDate(i) we need to
                % multiply cashflow with the df(startDate(i), payDate(i))
                payTime = DateDiff(schedule.payDates(i),valueDate);
                df_se = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);
                
                for k=1:pathSize
                    payoffStateVectors_cashflow(k) = payoffStateVectors_cashflow(k) + df_se(k);
                end
                
                nonCall.payoffStateVectors_cashflow(i,:) = payoffStateVectors_cashflow;
                
                nonCall.cashflow_e(i) = mean(payoffStateVectors_cashflow);
                nonCall.rangeProb_e(i) = mean(nonCall.cashflow_e(i)/(nominal * coupon * dcf));
                
                payoffStateVectors_cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,payoffStateVectors_cashflow);
            
                nonCall.cashflow_npv_e(i) = mean(payoffStateVectors_cashflow_npv);
                
                nonCall.payoffStateVectors_cashflow_npv(i,:) = payoffStateVectors_cashflow_npv;
                nonCall.payoff = nonCall.payoff + payoffStateVectors_cashflow_npv;
                
                payoffStateVectors_df = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,1.0*ones(1,pathSize));
                
                nonCall.payoffStateVector_df(i,:) = payoffStateVectors_df;
                nonCall.df_e(i) = mean(nonCall.payoffStateVector_df(i,:));
                nonCall.df_c(i) = model.DF(currentTime/365.0);
                
            end
            
            nonCall.npv = mean(nonCall.payoff);
  
            out.cashflow= nonCall.cashflow_e;
            out.cashflowNpv = nonCall.cashflow_npv_e;
            out.df_e = nonCall.df_e;
            out.df_c = nonCall.df_c;
            out.daycountFraction = schedule.dayCountFraction(i);
            out.rangeProb = nonCall.rangeProb_e;
            out.npv = nonCall.npv;
           
        end
        
        function out = computePrice(rangeAccrual,valueDate,model)
            schedule = rangeAccrual.schedule;
            periodCount = schedule.numOfPeriod;
            rangeProb = zeros(periodCount,1);
            discountFactor = zeros(periodCount,1);
            daycountFraction = zeros(periodCount,1);
            cashflow = zeros(periodCount,1);
            cashflowNpv = zeros(periodCount,1);
            rangeDays = rangeAccrual.rangeDays;
            npv = 0;
            for i=1:periodCount
                if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end;
                
                observeDate = schedule.startDates(i);
                if DateDiff(valueDate,schedule.startDates(i)) >= 0
                    observeDate = valueDate;
                    observeDate = AddDate(observeDate,1,'day');
                else
                    observeDate = schedule.startDates(i);
                end
                
                observeDays = DateDiff(schedule.endDates(i),schedule.startDates(i));
                days = DateDiff(schedule.endDates(i), observeDate);
                discountFactor(i) = model.DF( DateDiff(schedule.payDates(i),valueDate)/365.0);
                daycountFraction(i) = schedule.dayCountFraction(i);
                for j=1:days
                    toDate = observeDate;
                    tpDate = schedule.payDates(i);
                    tenor1 = rangeAccrual.tenorL;
                    tenor2 = rangeAccrual.tenorS;
                    tau =0.25;
                    lower = rangeAccrual.lower;
                    upper = rangeAccrual.upper;
                    rangeValue = model.CMSSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,lower,upper);
%                     rangeValue = 1.0;
                    rangeProb(i) = rangeProb(i) + rangeValue;
                    observeDate = AddDate(observeDate,1,'day');
                end
                cashflow(i) = rangeAccrual.nominal * rangeAccrual.coupon * (rangeProb(i) + rangeDays(i))/observeDays * daycountFraction(i);
                cashflowNpv(i) = cashflow(i) * discountFactor(i);
                npv = npv + cashflowNpv(i);
                % bug fixed observeDays -> days
                rangeProb(i) = rangeProb(i)/days;
                
                if i == periodCount
                    npv = npv + rangeAccrual.nominal * discountFactor(i);
                end
            end
            out.cashflow= cashflow;
            out.cashflowNpv = cashflowNpv;
            out.discountFactor = discountFactor;
            out.daycountFraction = daycountFraction;
            out.rangeProb = rangeProb;
            out.npv = npv;
        end
        
%         function out = discountPayoff(rangeAccrual,eventTime,numMatTime ...
%                 ,model,modelStatesT,cashflow)
%             
%             dfNumMat = model.DF(numMatTime/365.0);
%             % for discountPayoff T=numMatTime
%             % P(0,TnumMat)/P(Te,Tnummat)*Payoff
%             %
%             discountT = model.discountFactor(eventTime, numMatTime,numMatTime,modelStatesT);
%             stateSize = size(modelStatesT,2);
%             payoff = zeros(1,stateSize);
%             for i=1:stateSize
%                 payoff(i)= dfNumMat/discountT(i)*cashflow(i);
%             end    
%             out = payoff;
%         end
        
        
        
    end
    
end



In [ ]:
function out = CapletHWType2(zeroRate,fwdRate,strike,valueDate,resetDate,endDate,payDate,sigma,kappa,fixingAccraulFactor)

    resetTime = DateDiff(resetDate,valueDate)/365.0;
    endTime = DateDiff(endDate,valueDate)/365.0;
    payTime = DateDiff(payDate,valueDate)/365.0;

    df = exp(-1.0*zeroRate*payTime);
    
    if endTime < 0 
        out = 0;
        return;
        
    elseif endTime == 0 % option expiry is endTime
        out = max(fwdRate-strike,0);
        return;
    end
    
    if resetTime < 0  % option is expired
        
        deltaTime = endTime-resetTime;
        aa = 4.0 * exp(-1.0*kappa*endTime) - 1.0 * exp(-2.0*kappa*endTime) + 2.0 *kappa*endTime - 3.0;
        kappaTo3 = kappa*kappa*kappa;
        variance = sigma*sigma/2.0/kappaTo3*aa;
        vol = sqrt(variance);
        d1 = log((1.0 + deltaTime*fwdRate)/(1.0+deltaTime*strike))/vol + 0.5*vol;
        d2 = d1 - vol;
        out = (1.0 + deltaTime*fwdRate)*H_ncdf(d1) - (1.0 + deltaTime*strike)*H_ncdf(d2);
        out = out * df;
    else
    
        deltaTime = endTime-resetTime;
        aa = (exp(-1.0*kappa*deltaTime) + kappa*deltaTime - 1.0);
        bb = (1.0-exp(-1.0*kappa*deltaTime));
        cc = 2.0 * aa -1.0 * bb *bb *exp(-2.0*kappa*resetTime);
        kappaTo3 = kappa*kappa*kappa;
        variance = sigma*sigma/2.0/kappaTo3*cc;
        vol = sqrt(variance);
        d1 = log((1.0 + deltaTime*fwdRate)/(1.0+deltaTime*strike))/vol + 0.5*vol;
        d2 = d1 - vol;
        out = (1.0 + deltaTime*fwdRate)*H_ncdf(d1) - (1.0 + deltaTime*strike)*H_ncdf(d2);
        out = out * df;
        
        
    end
    
end



In [ ]:
function out = CapletHWType1(zeroRate,fwdRate,strike,valueDate,resetDate,endDate,payDate,sigma,kappa,fixingAccraulFactor)

    resetTime = DateDiff(resetDate,valueDate)/365.0;
    endTime = DateDiff(endDate,valueDate)/365.0;
    payTime = DateDiff(payDate,valueDate)/365.0;

    df = exp(-1.0*zeroRate*payTime);
    
    if resetTime < 0  % option is expired
        out = 0;
        return;
        
    elseif resetTime == 0 % at option expiry 
        out = max(fwdRate-strike,0);
        return;
    else
    
        deltaTime = endTime-resetTime;
        aa = (exp(-1.0*kappa*deltaTime) - 1.0);
        bb = (1.0-exp(-2.0*kappa*resetTime));
        kappaTo3 = kappa*kappa*kappa;
        variance = sigma*sigma/2.0/kappaTo3*aa*aa*bb;
        vol = sqrt(variance);
        d1 = log((1.0 + deltaTime*fwdRate)/(1.0+deltaTime*strike))/vol + 0.5*vol;
        d2 = d1 - vol;
        out = (1.0 + deltaTime*fwdRate)*H_ncdf(d1) - (1.0 + deltaTime*strike)*H_ncdf(d2);
        out = out * df;
        
        
    end
    
end



In [ ]:
function [P] = Call_RoughHeston_MF(S0,K,T,r,x,n,r_n)

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
% This f u n c t i o n p r i c e s a Ca l l o p t i on us ing Lewis ’ approach and
% Adams method f o r t h e c h a r a c t e r i s t i c func t i on , c o n s i d e r i n g
% t h e mul t i􀀀f a c t o r Heston model f o r t h e v o l a t i l i t y
%
%
% INPUT:
% S0 : i n i t i a l s p o t p r i c e
% K : s t r i k e p r i c e
% T : t ime t o e x p i r y ( in y e a r s )
% r : r i s k f r e e r a t e
% x : nu , lambda , rho , t h e t a ,V0 , a l ph a
% Rough Heston parame t e r s
% n : number o f f a c t o r s
% r_n : mu l t i f a c t o r c o e f f i c i e n t
%
%
% OUTPUT:
% P : European Ca l l p r i c e
%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%f u n c t i o n handl e f o r t h e Four i e r t rans form o f t h e
%c h a r a c t e r i s t i c f u n c t i o n
KTilda = log(S0/K) + r*T;

% integrand=@(u)(real(tr_fourier_MF(x,T,u,n,r_n).*exp(1i*log(K)*u)));
integrand=@(u)(real(tr_fourier_MF(x,T,u,n,r_n).*exp(1i*KTilda*u)));

%e x p l i c i t i n t e g r a t i o n
phiL = integral(integrand,0,100) ;

%Lewis ’ Pr i c ing formula
P = S0 -((sqrt(S0*K)*exp(-r*T/2)/pi)*phiL) ;

end

In [ ]:
function [P] = Call_RoughHeston ( S0 ,K,T, r , x )

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
% This f u n c t i o n p r i c e s a Ca l l o p t i on us ing Lewis ’ approach and
% Adams ’ method f o r t h e c h a r a c t e r i s t i c func t i on , c o n s i d e r i n g
% t h e Rough Heston model f o r t h e v o l a t i l i t y
%
%
% INPUT:
% S0 : i n i t i a l s p o t p r i c e
% K : s t r i k e p r i c e
% T : t ime t o e x p i r y ( in y e a r s )
% r : r i s k f r e e r a t e
% x : Rough Heston parame t e r s
%
%
% OUTPUT:
% P : European Ca l l p r i c e
%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

%f u n c t i o n handl e f o r t h e Four i e r t rans form o f t h e
%c h a r a c t e r i s t i c f u n c t i o n
KTilda = log(S0/K) + r*T;

% integrand=@(u) ( real( tr_fourier(x ,T, u).*exp(1i*log(K)*u) ) ) ;
integrand=@(u) ( real( tr_fourier(x ,T, u).*exp(1i*KTilda*u) ) ) ;

%e x p l i c i t i n t e g r a t i o n

phiL = integral( integrand , 0 , 100) ;

%Lewis ’ Pr i c ing formula
P = S0 - ( ( sqrt(S0*K)*exp(-r*T/2)/pi )*phiL ) ;

end

In [ ]:
function P = Call_Price_Heston (S,K,T,r,kappa,theta,sigma,rho,V,alpha,L)

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
% This f u n c t i o n p r i c e s a Ca l l o p t i on wi t h Matur i ty T and
% S t r i k e K us ing t h e c h a r a c t e r i s t i c f u n c t i o n o f t h e p r i c e and
% Lewis ’ approach
%
% No FFT used
%
%
% INPUT:
% S : i n i t i a l s p o t p r i c e
% r : r i s k f r e e r a t e
% kappa , t h e t a , sigma , rho : Heston model parame t e r s
% V : i n i t i a l v o l in Heston model
% al p h a : damping f a c t o r ( a l ph a >0)
% L : t r u n c a t i o n bound f o r t h e i n t e g r a l
%
%
% OUTPUT:
% P : European Ca l l p r i c e
%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%Heston c h a r a c t e r i s t i c f u n c t i o n
% 
% b=@(nu) ( kappa-1i*rho*sigma.*nu) ;
% gamma=@(nu) ( sqrt( sigma^2*(nu.^2+1i.*nu)+b(nu).^2));
% a=@(nu) (b(nu)./gamma(nu)).*sinh(T*0.5.*gamma(nu));
% c=@(nu) (gamma(nu).*coth(0.5*T.*gamma(nu))+b(nu));
% d=@(nu) ( kappa*theta*T.*b(nu)/sigma^2);
% 
% f=@(nu) (1i*(log(S)+r*T).*nu+d(nu));
% g=@(nu) (cosh(T*0.5.*gamma(nu))+a(nu)).^(2*kappa*theta/sigma^2) ;
% h=@(nu) (-(nu.^2+1i.*nu)*V./c(nu));

% phi=@(nu)(exp(f(nu)).*exp(h(nu))./g(nu));
Trap = 1;
lambda = 0.0;
tau = T;
q= 0.0;
v0 = V;
trap = 1;

load('laguerre192.mat','laguerre192');
quadrature_nodesAndWeights = laguerre192;
x = quadrature_nodesAndWeights(:,1);
w = quadrature_nodesAndWeights(:,2);
            
% [x w] = GenerateGaussLaguerre(32);

P  = LewisPrice311(S,K,r,q,T,theta,kappa,sigma,rho,v0,trap,x,w);
% 
% 
% phi=@(u)(LewisIntegrand311(u,kappa,theta,lambda,rho,sigma,tau,S,K,r,q,v0,Trap));
% 
% KTilda = log(S/K) + r*T;
% %f u n c t i o n handl e f o r t h e Four i e r t rans form o f t h e
% %c h a r a c t e r i s t i c f u n c t i o n
% % integrand=@(u)(real(phi(u-1i/2).*exp(1i*log(K)*u))./(u.^2+.25));
% % integrand=@(u)(real(phi(u-1i/2).*exp(1i*KTilda*u))./(u.^2+.25));
% integrand=@(u)(phi(u));
% 
% %e x p l i c i t i n t e g r a t i o n
% phi1 = integral(integrand,0,L) ;
% 
% 
% % Lewis ’ Pr i c ing formula
% P = S- ((sqrt(S*K)*exp(-r*T/2)/pi)*phi1);

end

In [ ]:
%calibration_lgm1f_CalibrateToATMSwaptionTenorDiagSwaption.m

clc
clear
% valueDate = '2018-01-09';
valueDate = '2017-02-17';

%% USD model building start
%% import and construct a zeroCurve
rawData = [0.002739726	0.999960071
0.083333333	0.998705285
0.166666667	0.997278569
0.25	0.99570906
0.5	0.990659472
0.75	0.985691562
1	0.980706602
2	0.958190889
3	0.935244778
4	0.91270743
5	0.890529514
7	0.846476648
10	0.781398058
12	0.739443263
15	0.680504997
20	0.593081133
25	0.519315395
30	0.456759825
    ];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'USD','USDZero'});
zeroCurveMktData = MktData(mktData);
zeroCurveMktData.params('convention') = {};
zeroCurve =  ZeroCurve(zeroCurveMktData);
USDAllMktData =containers.Map('zeroCurve',zeroCurve);

%% import and construct a discountCurve
rawData = [0.002739726	0.999960071
0.083333333	0.998705285
0.166666667	0.997278569
0.25	0.99570906
0.5	0.990659472
0.75	0.985691562
1	0.980706602
2	0.958190889
3	0.935244778
4	0.91270743
5	0.890529514
7	0.846476648
10	0.781398058
12	0.739443263
15	0.680504997
20	0.593081133
25	0.519315395
30	0.456759825
    ];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'USD','discountZero'});
discountCurveMktData = MktData(mktData);
discountCurveMktData.params('convention') = {};
discountCurve =  ZeroCurve(discountCurveMktData);
USDAllMktData('discountCurve') = discountCurve;

%% import and construct a swaptionVolSurface
rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
1	0.18785	0.21535	0.228	0.23405	0.23705	0.23735	0.2341	0.23	0.2259
2	0.22675	0.24245	0.25075	0.25335	0.25235	0.2504	0.2464	0.23945	0.2325
3	0.25885	0.26305	0.2647	0.263	0.26135	0.2569	0.2502	0.243325	0.23645
4	0.2736	0.27245	0.2692	0.266	0.263	0.25715	0.2503	0.243125	0.23595
5	0.27605	0.272	0.26905	0.26565	0.2627	0.2561	0.24885	0.24125	0.23365
7	0.2656	0.26325	0.25995	0.25655	0.25355	0.24835	0.24125	0.233575	0.2259
10	0.24575	0.2439	0.24245	0.23995	0.2385	0.23425	0.22875	0.2217	0.21465
15	0.2192	0.21675	0.2156	0.21345	0.21175	0.20865	0.20585	0.2007	0.19555
    ];

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawDataSwaption,valueDate,'USD','USDSwaptionVol'});
swaptionVolMktData = MktData(mktData);
USDAllMktData('swaptionVol') = swaptionVolMktData;

%% create  a BlackModel to calculate market Swaption
modelParams= containers.Map({'modelName'},{'Black'});
USDBlack = IRBlack(IRModel(Model(USDAllMktData,modelParams))); 

expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
swaptionPriceSurf.expiry= expiry;
swaptionPriceSurf.tenor = tenor;

tempdHTenor = zeros(size(expiry,1),size(tenor,1));
for i=1:size(expiry,1)
    for j=1:size(tenor,1)
%         swaptionPriceSurf.surface(i,j)= USDBlack.BlackATMSwaption(expiry(i),tenor(j));
%         swaptionVolSurf(i,j) = USDBlack.swaptionVol(expiry(i),tenor(j));
        tempdHTenor(i,j) = expiry(i) + tenor(j);
    end
end
%% create a LGM1FModel with initial modelParams

dH1.tenor = [1;2;3;4;5;7;10;15];

dH1.quote = ones(1,length(dH1.tenor))*1.0;
dH1Prev = dH1;
Alpha1.tenor =  [1;2;3;4;5;7;10;15];
Alpha1.quote = ones(1,length(dH1.tenor))*0.01;

joint_swap_tenor = 10;

smoothdH = 1.0;
smoothAlpha = 0.00;

targetSize = 15;

freq = 4;
matu = 15 + 1.0/365.0;

tic
modelParams= containers.Map({'dH1','Alpha1','matu','joint_swap_tenor','modelName','dH1Prev','smoothdH','smoothAlpha','targetSize','freq'}, ...
                        {dH1,Alpha1,matu,joint_swap_tenor, 'LGM1F',dH1Prev,smoothdH,smoothAlpha,targetSize,freq});
USDLGM1F = LGM1F(IRModel(Model(USDAllMktData,modelParams)));

USDLGM = LGM1F(IRModel(Model(USDAllMktData,modelParams)));
out = USDLGM.CalibrateToATMSwaptionTenorDiagSwaption(USDBlack,'global');
% out = USDLGM1F.CalibrateToATMDiagSwaption(KRWBlack,'global');
aaa =1.0;

dH1Calib = USDLGM.dH1;
Alpha1Calib = USDLGM.Alpha1;

modelParamsCalib= containers.Map({'dH1','Alpha1','matu','joint_swap_tenor','modelName','dH1Prev','smoothdH','smoothAlpha','targetSize','freq'}, ...
                    {dH1Calib,Alpha1Calib,matu,joint_swap_tenor,'LGM1F',dH1Prev,smoothdH,smoothAlpha,targetSize,freq});

USDLGMRisky = LGM1FRisky(IRModelRisky(Model(USDAllMktData,modelParamsCalib)));

%% USD model building end



In [ ]:
classdef Calendar < H_RootObject

    

    properties(SetAccess = public)
        name
        holidaysList

    end

    

    methods

        function calendar = Calendar()
            calendar.name = '';
            calendar.holidaysList = {};

        end

        

        function AddHoliday(calendar,h_Date)

            if strcmp(class(h_Date), 'H_Date')
                if calendar.IsBusinessDay(h_Date)    
                    calendar.holidaysList{end+1}=h_Date;
                end
            elseif strcmp(class(h_Date),'char') || strcmp(class(h_Date),'string')
                h_Date1 = H_Date(h_Date);
                if calendar.IsBusinessDay(h_Date1)
                    calendar.holidaysList{end+1}=h_Date1;
                end
            else

                error('wrong date type')

            end

        end

        

        function out = IsHoliday(calendar,h_Date)

            if strcmp(class(h_Date),'char') || strcmp(class(h_Date),'string')

                h_Date = H_Date(h_Date);

            end

            

            if IsWeekend(h_Date)

                out = true;

                return;

            else

                index = cellfun(@(x) x==h_Date, calendar.holidaysList, ...
                        'UniformOutput', 1);

                idx = find(index);

                if ~isempty(idx) 

                    out = true;

                    return;

                end

            end

            out = false;

        end

        

        function out = IsBusinessDay(calendar,h_Date)

            out = ~IsHoliday(calendar,h_Date);

        end

        

        function out = Adjust(calendar,d,bdc)

            if strcmp(bdc,'Unadjusted')

                out = d;

                return;

            end

            d1 = d;

            if strcmp(bdc,'Following') || ...
               strcmp(bdc,'ModifiedFollowing') || ...
               strcmp(bdc,'HalfMonthModifiedFollowing')

               while IsHoliday(calendar,d1)

                   d1 = d1.AddDate(1,'day');

               end

               

               if strcmp(bdc,'ModifiedFollowing') || ...
                    strcmp(bdc,'HalfMonthModifiedFollowing')

                   if d1.month ~= d.month 

                       out = calendar.Adjust(d, 'Preceding');

                       return;

                   end

                   

                   if strcmp(bdc,'HalfMonthModifiedFollowing')

                       if d.day <= 15 && d1.dayOfMonth > 15

                           out = calendar.Adjust(d, 'Preceding');

                           return;

                       end

                   end

                   

               end

            elseif strcmp(bdc,'Preceding') || ...
                   strcmp(bdc,'ModifiedPreceding')

               while IsHoliday(calendar,d1)

                   d1 = d1.AddDate(-1,'day');

               end

               

               if strcmp(bdc,'ModifiedPreceding') && ...
                       d1.month ~= d.month

                   out = calendar.Adjust(d,'Following');

                   return;

               end

            else

                error('unknown business day convention');

            end

            out = d1;

            

        end

        function out = AdvanceTenor(calendar,d,bdc,tenor,endOfMonth)
           if strcmp(tenor(end),'D')
                out = Advance(calendar,d,bdc,str2double(tenor(1:end-1)),'day',endOfMonth);
           elseif strcmp(tenor(end),'M')
                out = Advance(calendar,d,bdc,str2double(tenor(1:end-1)),'month',endOfMonth);
           elseif strcmp(tenor(end),'Y')
               out = Advance(calendar,d,bdc,str2double(tenor(1:end-1)),'year',endOfMonth);
           else
               error('unimplemented')
           end 
        end

        function out = Advance(calendar,d,bdc,n,unit,endOfMonth)

            if n == 0 

                out =  calendar.Adjust(d,bdc);

            elseif strcmp(unit,'day')

                d1 = d;

                if n > 0

                    while n > 0

                        d1 = d1.AddDate(1,'day');

                        while IsHoliday(calendar,d1)

                            d1 = d1.AddDate(1,'day');

                        end

                        n = n-1;

                    end

                else 

                    while n < 0

                        d1 = d1.AddDate(-1,'day');

                        while IsHoliday(calendar,d1)

                            d1 = d1.AddDate(-1,'day');

                        end

                        n = n +1;

                    end

                end

                

                out = d1;

                return;

            else

                d1 = d.AddDate(n,unit);

                % we are sure the unit is Months or Years

                if endOfMonth && IsEndOfMonth(d)

                    out =calendar.Adjust(EndOfMonth(d1),'Preceding');

                    return;

                end

                

                out =  Adjust(calendar,d1, bdc);

                return;

            end

        end

    end

    

end





In [ ]:
function rlag=calculateReturns(prices, lag)
% rlag=calculateReturns(prices, lag) returns the lagged returns based on
% the price series

% rlag=log(prices)-backshift(lag, log(prices));
prevPrices=backshift(lag, prices);
rlag=(prices-prevPrices)./prevPrices;




In [ ]:
function output = BootStrapZeroFromYC(ycData,date,curveName,globalDict)
    
    nowDate = date;
    %% get calendars
    if  isa(nowDate,'H_Date')
        nowDateH = nowDate;
    else
        nowDateH = H_Date(nowDate);
    end


%     fromDateStr = int2str(FngDate(nowDateH));
%     DataNm = curveName;
%     curveType = 'market';
%     yieldCurveMarketQueryStr1 = 'http://45.12.71.111:50100/data/market/yieldcurves/';
% 
%     queryStr = strcat(yieldCurveMarketQueryStr1,DataNm,'/',fromDateStr,'?type=',curveType);
%     dataJson = webread(queryStr);
% 
%     dataType = cell(length(dataJson.Yields),1);
%     dataTenor = cell(length(dataJson.Yields),1);
%     dataRate = cell(length(dataJson.Yields),1);
%     
%     for i=1:length(dataJson.Yields)
%         sizeStr = int2str(cast(dataJson.Yields(i).Tenor.Value,'int32'));
%         periodTypeStr = dataJson.Yields(i).Tenor.TenorCd.Cd;
%         dataTenor{i} = strcat(sizeStr,periodTypeStr);
%         dataType{i} = dataJson.Yields(i).YcInsTpNm;
%         dataRate{i} = dataJson.Yields(i).Rate;
%     end
%     
%     ycData = {};
%     ycData.dataType = dataType;
%     ycData.dataTenor = dataTenor;
%     ycData.dataRate = dataRate;
%     
%     output.ycData = ycData;
    
    if 1
        
        if strcmp(curveName,'KRW_KTB')
            ccy = 'KTB';
        else
            ccy = curveName(1:3);
        end
    
        yieldCurveSettings = globalDict('yieldCurveSettings');
        if isKey(yieldCurveSettings,curveName)
            yieldCurveParams = yieldCurveSettings(curveName);
        else
            yieldCurveParams = yieldCurveSettings('KRW_KTB');
        end
        
        zcCurveOut = YieldCurveBootStrap(ccy,nowDateH,ycData,yieldCurveParams);
        
        output.zcCurve = zcCurveOut;
    
    else
        output.zcCurve = ZeroCurve(MktData(containers.Map('asOfDate',nowDateH)));
        
    end
    
end





In [ ]:
function y = BlackFwdVega(F,K,vol,T)

% Bisection algorithm
% PutCall = "P"ut or "C"all
% F = Forward price
% K = Strike price
% vol = volatility
% T = Maturity

    d1= (log(F/K)+(vol^2/2.0)*T)/(vol*sqrt(T));
    
    y = sqrt(T)*H_ndf(d1);

    
end



In [ ]:
function y = BlackFwdDelta(F,K,vol,T)

% Bisection algorithm
% PutCall = "P"ut or "C"all
% F = Forward price
% K = Strike price
% vol = volatility
% T = Maturity

    d1= (log(F/K)+(vol^2/2.0)*T)/(vol*sqrt(T));
    
    y = -1.0*H_ncdf(-d1);

    
end



In [ ]:
classdef BlackFormula
    % Jim Gatheral R Code to Matlab
    % Convert each R library as blank class(function container)
    
    methods
        
%         function obj = BlackFormula()
%                 
%         end
        
        function out = BlackCall(obj,S0, K, T, sigma)
            k  = log(K./S0);
            sig= sigma.*sqrt(T);
            d1 = -k./sig+sig./2;
            d2 = d1 - sig;
            out = S0.*H_ncdf(d1) - K.*H_ncdf(d2);
            
        end
        
        function out = BlackPut(obj,S0, K, T, sigma)
            k  = log(K./S0);
            sig= sigma.*sqrt(T);
            d1 = -k./sig+sig/2;
            d2 = d1 - sig;
            out = K.*H_ncdf(-d2) - S0.*H_ncdf(-d1);
        end
        
        function out = ivCall(obj,S0, K, T, C)
            nK = length(K);
%             sigmaL = ones(nK,1)*1e-10;
            sigmaL = 1e-10;
            
            CL  = obj.BlackCall(S0, K, T, sigmaL);
%             sigmaH =ones(nK,1)*10.0;
            sigmaH =10.0;
            
            CH  = obj.BlackCall(S0, K, T, sigmaH);

            while (mean(sigmaH - sigmaL) > 1e-10)
                sigma = (sigmaL + sigmaH)./2;
                CM = obj.BlackCall(S0, K, T, sigma);
                CL = CL + (CM < C).*(CM-CL);
                sigmaL = sigmaL + (CM < C).*(sigma-sigmaL);
                CH = CH + (CM >= C).*(CM-CH);
                sigmaH = sigmaH + (CM >= C).*(sigma-sigmaH);
            end

            out = sigma;
        
        end
        
        function out = ivPut(obj,S0, K, T, P)
            % This function also works with vectors of strikes and option values  
            
            nK = length(K);
            sigmaL = 1e-10;
            PL  = obj.BlackPut(S0, K, T, sigmaL);
            sigmaH =10.0;
            PH  = obj.BlackPut(S0, K, T, sigmaH);

            while (mean(sigmaH - sigmaL) > 1e-10)
                sigma = (sigmaL + sigmaH)./2;
                PM = obj.BlackPut(S0, K, T, sigma);
                PL = PL + (PM < P).*(PM-PL);
                sigmaL = sigmaL + (PM < P).*(sigma-sigmaL);
                PH = PH + (PM >= P).*(PM-PH);
                sigmaH = sigmaH + (PM >= P).*(sigma-sigmaH);
            end

            out = sigma;
        
        end
        
        function out = ivOTM(obj,S0, K, T, V)
%             if (K > S0) out = obj.ivCall(S0,K,T,V); else out = obj.ivPut(S0,K,T,V);end;
            out = (S0 < K).*obj.ivCall(S0,K,T,V) + (S0 >= K).* obj.ivPut(S0,K,T,V);
        end
        
%         function out = ivOTM(obj,S0, K, T, V)
%             out = arrayfun(@(K) obj.ivOTMRaw(S0, K, T, V),K);
%         end
        
        function out = ivS(obj,Sf,T,AK)
            % Function to compute option prices and implied vols
            % given vector of final values of underlying
            
            nK = length(AK);
            N  = length(Sf);
            Sfbar = mean(Sf);
            V =ones(1,nK);
            ivBlack =ones(1,nK);
            for j= 1:nK 
                payoff = (Sf - AK(j)) .* (Sf > AK(j));
                V = mean(payoff);
                if (Sfbar< AK(j)) V_OTM = V; else V_OTM = V + AK(j) - Sfbar;end;
                ivBlack(j) =obj.ivOTM(Sfbar, AK(j), T, V_OTM);
            end
            out =  ivBlack;
        end
    end

end



In [ ]:
function y = BisecNormIV(PutCall,S,K,rf,T,a,b,MktPrice,Tol,MaxIter)

% Bisection algorithm
% PutCall = "P"ut or "C"all
% S = Spot price
% K = Strike price
% rf = Risk free rate
% T = Maturity
% a = Starting point lower value of vol
% b = Starting point upper value of vol
% MktPrice = Market price
% Tol = Tolerance
% MaxIter = maximum number of iterations

% Functions for the Black-Scholes call and put  
NormC = @(s,K,rf,v,T) (exp(-rf*T)*((s*exp(rf*T)-K)*H_ncdf((s*exp(rf*T)-K)/v./sqrt(T)) + v.*sqrt(T)*H_ndf((s*exp(rf*T)-K)/v./sqrt(T))));
NormP = @(s,K,rf,v,T) (exp(-rf*T)*((K-s*exp(rf*T))*H_ncdf((K-s*exp(rf*T))/v./sqrt(T)) + v.*sqrt(T)*H_ndf((s*exp(rf*T)-K)/v./sqrt(T))));
  
if strcmp(PutCall,'C')
	lowCdif  = MktPrice - NormC(S,K,rf,a,T);
	highCdif = MktPrice - NormC(S,K,rf,b,T);
else
	lowCdif  = MktPrice - NormP(S,K,rf,a,T);
	highCdif = MktPrice - NormP(S,K,rf,b,T);
end

if lowCdif*highCdif > 0
	y = -1;
else
	for x=1:MaxIter
		midP = (a+b)/2;
		if strcmp(PutCall,'C')
			midCdif = MktPrice - NormC(S,K,rf,midP,T);
		else
			midCdif = MktPrice - NormP(S,K,rf,midP,T);
		end
		if abs(midCdif)<Tol
			break
		else
			if midCdif>0
				a = midP;
			else
				b = midP;
			end
		end
	end
	y = midP;
end



In [ ]:
function y = BisecFwdMoneynessPADelta(delta,sig,T)

% Bisection algorithm
% delta
% sig = volatility
% T = Maturity
% a = Starting point lower value of fwdmoneyness
% b = Starting point upper value of fwdmoneyness
% Tol = Tolerance
% MaxIter = maximum number of iterations

% Tol = 1e-6;
Tol = 1e-10;
MaxIter =1000;


% Functions for the Black-Scholes call and put  
deltaPAC = @(k,v,T) ( 1.0*k*H_ncdf( 1.0*(-log(k)-0.5*v*v*T)/(v*sqrt(T))));
deltaPAP = @(k,v,T) (-1.0*k*H_ncdf(-1.0*(-log(k)-0.5*v*v*T)/(v*sqrt(T))));

% ATM
if abs(delta - 0.5) < 0.0001
    y = exp(-0.5*sig*sig*T);
    return;
end

if delta > 0 % Call
    a = 1.0;
    b = 2.0;
	lowCdif  = delta - deltaPAC(a,sig,T);
	highCdif = delta - deltaPAC(b,sig,T);
else
    a = 0.01;
    b = 1.0;
	lowCdif  = delta - deltaPAP(a,sig,T);
	highCdif = delta - deltaPAP(b,sig,T);
end

if lowCdif*highCdif > 0
	y = -1; % there's no solution
else
	for x=1:MaxIter
		midP = (a+b)/2;
		if delta > 0 
			midCdif = delta - deltaPAC(midP,sig,T);
		else
			midCdif = delta - deltaPAP(midP,sig,T);
		end
		if abs(midCdif)<Tol
			break
		else
			if midCdif>0
% 				a = midP;
                b = midP;
			else
% 				b = midP;
                a = midP;
			end
		end
	end
	y = midP;
end



In [ ]:
function y = BisecBSIV(PutCall,S,K,rf,T,a,b,MktPrice,Tol,MaxIter)

% Bisection algorithm
% PutCall = "P"ut or "C"all
% S = Spot price
% K = Strike price
% rf = Risk free rate
% T = Maturity
% a = Starting point lower value of vol
% b = Starting point upper value of vol
% MktPrice = Market price
% Tol = Tolerance
% MaxIter = maximum number of iterations

% Functions for the Black-Scholes call and put  
BSC = @(s,K,rf,v,T) (s.*normcdf((log(s./K) + (rf+v.^2./2).*T)./v./sqrt(T)) - K.*exp(-rf.*T).*normcdf((log(s./K) + (rf+v.^2./2).*T)./v./sqrt(T) - v.*sqrt(T)));
BSP = @(s,K,rf,v,T) (K.*exp(-rf.*T).*normcdf(-(log(s./K) + (rf+v.^2./2).*T)./v./sqrt(T) + v.*sqrt(T)) - s.*normcdf(-(log(s./K) + (rf+v.^2./2).*T)./v./sqrt(T)));
  
if strcmp(PutCall,'C')
	lowCdif  = MktPrice - BSC(S,K,rf,a,T);
	highCdif = MktPrice - BSC(S,K,rf,b,T);
else
	lowCdif  = MktPrice - BSP(S,K,rf,a,T);
	highCdif = MktPrice - BSP(S,K,rf,b,T);
end

if lowCdif*highCdif > 0
	y = -1;
else
	for x=1:MaxIter
		midP = (a+b)/2;
		if strcmp(PutCall,'C')
			midCdif = MktPrice - BSC(S,K,rf,midP,T);
		else
			midCdif = MktPrice - BSP(S,K,rf,midP,T);
		end
		if abs(midCdif)<Tol
			break
		else
			if midCdif>0
				a = midP;
			else
				b = midP;
			end
		end
	end
	y = midP;
end



In [ ]:
function y = BisecBlackFwdIV(PutCall,F,K,rf,T,a,b,MktPrice,Tol,MaxIter)

% Bisection algorithm
% PutCall = "P"ut or "C"all
% S = Spot price
% K = Strike price
% rf = Risk free rate
% T = Maturity
% a = Starting point lower value of vol
% b = Starting point upper value of vol
% MktPrice = Market price
% Tol = Tolerance
% MaxIter = maximum number of iterations


% Functions for the Black-Scholes call and put  
BSC = @(F,K,rf,v,T) (exp(-rf.*T).*(F.*H_ncdf((log(F./K) + (v.^2./2).*T)./v./sqrt(T)) - K.*H_ncdf((log(F./K) + (v.^2./2).*T)./v./sqrt(T) - v.*sqrt(T))));
BSP = @(F,K,rf,v,T) (exp(-rf.*T).*(K.*H_ncdf(-(log(F./K) + (v.^2./2).*T)./v./sqrt(T) + v.*sqrt(T)) - F.*H_ncdf(-(log(F./K) + (v.^2./2).*T)./v./sqrt(T))));
  
if strcmp(PutCall,'C')
	lowCdif  = MktPrice - BSC(F,K,rf,a,T);
	highCdif = MktPrice - BSC(F,K,rf,b,T);
else
	lowCdif  = MktPrice - BSP(F,K,rf,a,T);
	highCdif = MktPrice - BSP(F,K,rf,b,T);
end

if lowCdif*highCdif > 0
	y = -1;
else
	for x=1:MaxIter
		midP = (a+b)/2;
		if strcmp(PutCall,'C')
			midCdif = MktPrice - BSC(F,K,rf,midP,T);
		else
			midCdif = MktPrice - BSP(F,K,rf,midP,T);
		end
		if abs(midCdif)<Tol
			break
		else
			if midCdif>0
				a = midP;
			else
				b = midP;
			end
		end
	end
	y = midP;
end



In [ ]:
classdef BasisSwapHelper < H_RootObject
    % FixedRateLeg
    % properties: fixedSchedule 
    
    properties(SetAccess = public)
        
        rateType
        
        rate
        rateTenor
        
        gearing
        spread
        
        domesticIndex
        foreignIndex
        
        evaluationDate
        
        basisSwap
%         domesticDCurve
        discountCurve
        foreignDCurve
        
        earliestDate
        latestDate
        
    end
    
    methods
        % constructor
        function basisSwapHelper = BasisSwapHelper(params)
                if nargin > 0
                    basisSwapHelper.rateType = 'BasisSwap';
                    
                    basisSwapHelper.rate = params('rate');
                    basisSwapHelper.rateTenor = params('rateTenor');
                    
                    basisSwapHelper.gearing = params('domesticGearing');
                    basisSwapHelper.domesticIndex = params('domesticIndex');
                    basisSwapHelper.foreignIndex = params('foreignIndex');
                    
                    basisSwapHelper.evaluationDate = basisSwapHelper.domesticIndex.forwardCurve.asOfDate;
                    
%                     basisSwapHelper.domesticDCurve = params('domesticDCurve');
                    basisSwapHelper.discountCurve = params('discountCurve');
                    
                    % can be generalized later for dual curve input
                    
                    basisSwapHelper.foreignDCurve = basisSwapHelper.foreignIndex.forwardCurve;
                    
                    if basisSwapHelper.discountCurve.asOfDate ~= basisSwapHelper.domesticIndex.forwardCurve.asOfDate
                        error('forwardCurve asOfDate is different from discountCurve asOfDate')
                    end
                    
                    basisSwapHelper.basisSwap = MakeBasisSwap(basisSwapHelper);
                
                    out = InitializeDates(basisSwapHelper);

                    basisSwapHelper.earliestDate = out.earliestDate;
                    basisSwapHelper.latestDate = out.latestDate;
                end
        end
        
        function out = MakeBasisSwap(basisSwapHelper)
            domesticIndex = basisSwapHelper.domesticIndex;
            foreignIndex = basisSwapHelper.foreignIndex;
            
            domesticCalendar = domesticIndex.fixingCalendar;
            foreignCalendar = foreignIndex.fixingCalendar;
            
            jointCalendar = JointCalendar({domesticCalendar,foreignCalendar});
            
            fixingDays = max(domesticIndex.fixingDays,foreignIndex.fixingDays);
            
            referenceDate = domesticIndex.forwardCurve.asOfDate;
            
            startDate = jointCalendar.Advance(referenceDate,'Following',...
                                                fixingDays,'day',false);
            
            endDate = AddTenor(startDate,basisSwapHelper.rateTenor);
            % fixed & floating schedule 
            domesticSchedule = Schedule(startDate,endDate,domesticIndex.tenor ,jointCalendar,...
                                    domesticIndex.convention, 'Backward',false); 
            
            foreignSchedule = Schedule(startDate,endDate,foreignIndex.tenor,jointCalendar,...
                                    foreignIndex.convention, 'Backward',false);
             
             type = 'Payer';
%              nominal = 10000.0;
             domesticNominal = 1.0;
             domesticGearing = basisSwapHelper.gearing;
             domesticSpread = 0.0;
             domesticDayCounter = domesticIndex.dayCounter;
             
             foreignNominal = 1.0;
             foreignGearing = 1.0;
             foreignSpread = 0.0;
             foreignDayCounter = foreignIndex.dayCounter;
             
             notionalExchangeType = 'Both';
             
             basisSwap = BasisSwap(type,domesticNominal,domesticSchedule,domesticGearing,domesticSpread,domesticDayCounter, ...
                                   domesticIndex,... 
                                   foreignNominal,foreignSchedule,foreignGearing,foreignSpread,foreignDayCounter,...
                                   foreignIndex,...
                                   notionalExchangeType);
                                  
             out = basisSwap;
        end
        
        function out = InitializeDates(basisSwapHelper)
            
            %earliestDate
            earliestDate = basisSwapHelper.basisSwap.StartDate();
            latestDate = basisSwapHelper.basisSwap.EndDate();
            
            % need to be implemented ??
            % check later !!
            
%             floatEndValueDate = basisSwapHelper.vanillaSwap.floatingLeg.couponRates{end}.fixingEndDate; 
%             % if the last floating Index fixingEndDate is after
%             % latestDate(last accrualEndDate then we change
%             % the lastestDate into floatEndValueDate
%             
%             if DateDiff(floatEndValueDate,latestDate) > 0
%                 latestDate = floatEndValueDate;
%             end
            
            out.earliestDate = earliestDate;
            out.latestDate = latestDate;
           
        end
        
        function out = ImpliedQuote(basisSwapHelper)
            outInfo = basisSwapHelper.basisSwap.Calculate(basisSwapHelper.discountCurve,basisSwapHelper.foreignDCurve);
            out = outInfo.fairDomesticSpread;
            
        end
        
        % bootstrapping
        function out = QuoteError(basisSwapHelper)
            out = basisSwapHelper.rate - basisSwapHelper.ImpliedQuote();
        end
        
        function out = BootStrapError(basisSwapHelper,guess,idx)
            basisSwapHelper.Update(idx,guess);
            out = basisSwapHelper.QuoteError();
        end
        
        function Update(basisSwapHelper,idx,guess)
             % forwardCurve update
%             forwardRawData = basisSwapHelper.iborIndex.forwardCurve.params('rawData');
%             forwardRawData(idx,2) = guess;
%             basisSwapHelper.iborIndex.forwardCurve.params('rawData')= forwardRawData;
            
            % for singleCurve framework this is redundant
            
            % discountCurve update
            discountRawData = basisSwapHelper.discountCurve.params('rawData');
            discountRawData(idx,2) = guess;
            basisSwapHelper.discountCurve.params('rawData')= discountRawData;
            
        end
        
        
    end
    
end



In [ ]:
classdef BasisSwap < H_RootObject
    % FixedRateLeg
    % properties: fixedSchedule 
    
    properties(SetAccess = public)
        type
        domesticNominal
        domesticSchedule
        domesticGearing
        domesticSpread
        domesticDayCounter
        domesticIndex
        
        foreignNominal
        foreignSchedule
        foreignGearing
        foreignSpread
        foreignDayCounter
        foreignIndex
        notionalExchangeType
        
        domesticLeg
        foreignLeg
        
        NPV;
        fairDomesticSpread;
       
    end
    
    methods
        % constructor
        function basisSwap = BasisSwap(type,domesticNominal,domesticSchedule,domesticGearing,domesticSpread,domesticDayCounter,...
                                      domesticIndex,...
                                      foreignNominal,foreignSchedule,foreignGearing,foreignSpread,foreignDayCounter,...
                                      foreignIndex,...
                                      notionalExchangeType)
            
            basisSwap.type = type;
            basisSwap.domesticNominal = domesticNominal;
            basisSwap.domesticSchedule = domesticSchedule;
            basisSwap.domesticGearing = domesticGearing;
            basisSwap.domesticSpread = domesticSpread;
            basisSwap.domesticDayCounter = domesticDayCounter;
            basisSwap.domesticIndex = domesticIndex;
            
            basisSwap.foreignNominal = foreignNominal;
            basisSwap.foreignSchedule = foreignSchedule;
            basisSwap.foreignGearing = foreignGearing;
            basisSwap.foreignSpread = foreignSpread;
            basisSwap.foreignDayCounter = foreignDayCounter;
            basisSwap.foreignIndex = foreignIndex;
            
            basisSwap.notionalExchangeType = notionalExchangeType;
            
            basisSwap.domesticLeg = FixedRateLeg(domesticSchedule,domesticNominal,1.0,domesticDayCounter,domesticSchedule.convention);
                                    
            if strcmp(notionalExchangeType,'Both') || strcmp(notionalExchangeType,'Initial')
                couponRates = basisSwap.domesticLeg.couponRates;
                tempRates = cell(length(couponRates)+1,1);
                tempRates{1} = SimpleCashflow(-domesticNominal,domesticSchedule.startDate);
                for i=2:length(tempRates)
                    tempRates{i} = couponRates{i-1};
                end
                basisSwap.domesticLeg.couponRates = tempRates;
            end
            
            if strcmp(notionalExchangeType,'Both') || strcmp(notionalExchangeType,'Final')
                basisSwap.domesticLeg.couponRates{end+1} = SimpleCashflow(domesticNominal,domesticSchedule.endDate);
            end
            
            
            basisSwap.foreignLeg = FloatingRateLeg(foreignSchedule,foreignIndex,foreignGearing,foreignSpread,foreignNominal,...
                                      foreignDayCounter,foreignSchedule.convention);
                                  
            if strcmp(notionalExchangeType,'Both') || strcmp(notionalExchangeType,'Initial')
                couponRates = basisSwap.foreignLeg.couponRates;
                tempRates = cell(length(couponRates)+1,1);
                tempRates{1} = SimpleCashflow(-foreignNominal,foreignSchedule.startDate);
                for i=2:length(tempRates)
                    tempRates{i} = couponRates{i-1};
                end
                basisSwap.foreignLeg.couponRates = tempRates;
            end
            
            if strcmp(notionalExchangeType,'Both') || strcmp(notionalExchangeType,'Final')
                basisSwap.foreignLeg.couponRates{end+1} = SimpleCashflow(foreignNominal,foreignSchedule.endDate);
            end
            
        end
        
        function out = StartDate(basisSwap)
            d1 = basisSwap.domesticLeg.StartDate();
            d2 = basisSwap.foreignLeg.StartDate();
            
            if DateDiff(d1,d2) > 0
                out = d2;
            else
                out = d1;
            end
        end
        
        function out = EndDate(basisSwap)
            d1 = basisSwap.domesticLeg.EndDate();
            d2 = basisSwap.foreignLeg.EndDate();
            
            if DateDiff(d1,d2) > 0
                out = d1;
            else
                out = d2;
            end
        end
        
        function out = Calculate(basisSwap,domesticDCurve,foreignDCurve)
            % to be updated
            % very weird samsung basisSwap pricing logic :(
            % talk to CSY in the future
            
            domesticNPV = basisSwap.domesticLeg.AdjustedNPV(domesticDCurve);
            domesticBPS = basisSwap.domesticLeg.AdjustedBPS(domesticDCurve);
%             spreadNPV = basisSwap.floatingSpread * domesticBPS;
            foreignNPV = basisSwap.foreignLeg.NPV(foreignDCurve);
%             foreignBPS = basisSwap.fixedLeg.BPS(discountCurve);
            
            if strcmp(basisSwap.type,'Payer')
                out.NPV = domesticNPV - domesticBPS - foreignNPV;
                out.fairDomesticSpread =  -1.0*(out.NPV)/domesticBPS;
            else
                out.NPV = -1.0*(domesticNPV + domesticBPS - foreignNPV);
                out.fairDomesticSpread =  1.0*(out.NPV)/domesticBPS;
            end
            
        end
    end
    
end



In [ ]:
function y=backshift(day,x)
% y=backshift(day,x)
assert(day>=0);
y=[NaN(day,size(x,2), size(x, 3));x(1:end-day,:, :)];


In [ ]:
classdef AFVSimulation < GammaKernel
    
    methods
        function out = psiM(obj,psi,ev,w)
            beta2 = 2.0./psi-1+sqrt(2./psi).*sqrt(abs(2./psi-1));
            alpha = ev./(1+beta2);
            vf = alpha.*(sqrt(abs(beta2))+w).^2;
            out = vf;
        end
        
        function out = psiP(obj,psi,ev,u)
            p = 2.0./(1+psi);
            gam = ev./2.0.*(1+psi);
            vf = -(u<p).*gam.*log(u./p);
            out = vf;
        end

        function out = RSQESim(obj, params, xi,paths, steps, expiry)
            try
                eta = params.eta;
                lam = params.lam;
                H = params.al - 1/2;
                rho = params.rho;
                rho2m1 = sqrt(1 - rho * rho);
                eps0 = 1e-10;
                %     W = matrix(rnorm(steps * paths), nrow = steps, ncol = paths);
                rng('default');
                rng(0);
                
                W = randn(paths/2,steps);W=[W;-W];W=W';
                Wperp = randn(paths/2,steps);Wperp=[Wperp;-Wperp];Wperp=Wperp';
                Un = randn(paths/2,steps);Un=[Un;-Un];Un=Un';
                U = normcdf(Un);
                
                G00p = @(t) (arrayfun(@(t) obj.G00(params,t),t));
    
                dt = expiry/steps;
                sqrtDt = sqrt(dt);
                tj = (1:steps) * dt;
                xij = xi(tj);
                G00del = obj.G00(params,dt);
                

                G00j = [0, G00p(tj)];

                bstar = sqrt(diff(G00j)./dt);

                bstar1 = bstar(1); % bstar is average g over an interval
                u = zeros(steps,paths);
                v = ones(1,paths).* xi(0);
                xihat = ones(1,paths).*xij(1);
                x = zeros(1,paths);
                y = zeros(1,paths);
                w = zeros(1,paths);
                vf = zeros(1,paths);
                for j =1:steps 
                    varv = eta.^2 .* (xihat + 2.* H.* v)/(1+2.*H).* G00del;
                    psi =varv./xihat.^2;
                    vf = (psi < 3/2).*obj.psiM(psi, xihat,W(j,:))+(psi >= 3/2).*obj.psiP(psi, xihat, U(j,:));

                    u(j,:) = vf - xihat;
                    dw = (v + vf)./2 .* dt;
                    w  = w + dw;
                    dy = u(j,:)./(eta .* bstar1);
                    y = y + dy;
                    x = x - dw./2 + sqrt(dw) .* rho2m1 .* Wperp(j,:) + rho .* dy;

                    if (j < steps)
                        btilde = flip(bstar(2:(j+1)));

                        xihat = xij(j + 1) + (btilde * u(1:j,:))./bstar1;
                    end

                    idx11 = find(xihat <= eps0);
                    if ~isempty(idx11)
                        xihat(idx11) = eps0;
                    end

                    v = vf;
                end

                resSim.v = v;
                resSim.x = x;
                resSim.y = y;
                resSim.w = w;

                out = resSim;
            catch ME
                aaaa = 1.0;
                rethrow(ME)
            end
        end
        
        function out = RSQESimHist(obj, params, xi,paths, steps, expiry)
            try
                eta = params.eta;
                lam = params.lam;
                H = params.al - 1/2;
                rho = params.rho;
                rho2m1 = sqrt(1 - rho * rho);
                eps0 = 1e-10;
                %     W = matrix(rnorm(steps * paths), nrow = steps, ncol = paths);
                rng('default');
                rng(0);
                
                W = randn(paths/2,steps);W=[W;-W];W=W';
                Wperp = randn(paths/2,steps);Wperp=[Wperp;-Wperp];Wperp=Wperp';
                Un = randn(paths/2,steps);Un=[Un;-Un];Un=Un';
                U = normcdf(Un);
                
                G00p = @(t) (arrayfun(@(t) obj.G00(params,t),t));
    
                dt = expiry/steps;
                sqrtDt = sqrt(dt);
                tj = (1:steps) * dt;
                xij = xi(tj);
                G00del = obj.G00(params,dt);
                

                G00j = [0, G00p(tj)];

                bstar = sqrt(diff(G00j)./dt);

                bstar1 = bstar(1); % bstar is average g over an interval
                u = zeros(steps,paths);
                v = ones(1,paths).* xi(0);
                xihat = ones(1,paths).*xij(1);
                x = zeros(1,paths);
                y = zeros(1,paths);
                w = zeros(1,paths);
                vf = zeros(1,paths);
                
                % log S(t)/F(t) 's simulation step for all paths
                xhist = zeros(steps,paths);
                
                for j =1:steps 
                    varv = eta.^2 .* (xihat + 2.* H.* v)/(1+2.*H).* G00del;
                    psi =varv./xihat.^2;
                    vf = (psi < 3/2).*obj.psiM(psi, xihat,W(j,:))+(psi >= 3/2).*obj.psiP(psi, xihat, U(j,:));

                    u(j,:) = vf - xihat;
                    dw = (v + vf)./2 .* dt;
                    w  = w + dw;
                    dy = u(j,:)./(eta .* bstar1);
                    y = y + dy;
                    x = x - dw./2 + sqrt(dw) .* rho2m1 .* Wperp(j,:) + rho .* dy;

                    if (j < steps)
                        btilde = flip(bstar(2:(j+1)));

                        xihat = xij(j + 1) + (btilde * u(1:j,:))./bstar1;
                    end

                    idx11 = find(xihat <= eps0);
                    if ~isempty(idx11)
                        xihat(idx11) = eps0;
                    end

                    v = vf;
                    
                    xhist(j,:) = x;
                end

                resSim.v = v;
                resSim.x = x;
                resSim.y = y;
                resSim.w = w;
                
                resSim.xhist = xhist;
                out = resSim;
            catch ME
                aaaa = 1.0;
                rethrow(ME)
            end
        end
        
        function out = HQESim(obj, params, xi,paths, steps, expiry)
            try
                eta = params.eta;
                lam = params.lam;
                H = params.al - 1/2;
                rho = params.rho;
                rho2m1 = sqrt(1 - rho * rho);
                eps0 = 1e-10;
                %     W = matrix(rnorm(steps * paths), nrow = steps, ncol = paths);
                
                rng('default');
                rng(0);
                
                W = randn(paths/2,steps);W=[W;-W];W=W';
                Wperp = randn(paths/2,steps);Wperp=[Wperp;-Wperp];Wperp=Wperp';
                Z = randn(paths/2,steps);Z=[Z;-Z];Z=Z';
                Un = randn(paths/2,steps);Un=[Un;-Un];Un=Un';
                U = normcdf(Un);
                Uperpn = randn(paths/2,steps);Uperpn=[Uperpn;-Uperpn];Uperpn=Uperpn';
                Uperp = normcdf(Uperpn);
                
                G00p = @(t) (arrayfun(@(t) obj.G00(params,t),t));
    
                dt = expiry/steps;
                sqrtDt = sqrt(dt);
                tj = (1:steps) * dt;
                xij = xi(tj);
%                 G00del = obj.G00(params,dt);
                G0del = eta.*obj.G0(params,dt);
                G1del  = eta.*obj.G1(params,dt);
                G00del = eta.^2.*obj.G00(params,dt);
                G11del = eta.^2.*obj.G11(params,dt);
                G01del = eta.^2.*obj.G01(params,dt);
                G00j   = eta.^2.*[0, G00p(tj)];
                bstar  = sqrt(diff(G00j)./dt);
                bstar1 = bstar(1); % # bstar is average g over an interval
                rho_vchi = G0del/sqrt(G00del*dt);
                beta_vchi = G0del/dt;

                u = zeros(steps,paths);
                chi = zeros(steps,paths);
                v = ones(1,paths).* xi(0);
                xihat = ones(1,paths).*xij(1);
                
                x = zeros(1,paths);
                y = zeros(1,paths);
                w = zeros(1,paths);
                vf = zeros(1,paths);
                
                for j =1:steps 
                    xibar = (xihat + 2 .* H.* v)./(1 + 2.* H);
                    var_chi = xibar.* dt;
                    var_eps = xibar.* G00del.*(1-rho_vchi.^2);
                    psi_chi = 4.*G00del.*rho_vchi.^2./xibar;
                    psi_eps = 4.*G00del.*(1-rho_vchi.^2)./xibar;

                    z_chi = (psi_chi < 3/2).*obj.psiM(psi_chi, xihat./2,W(j,:))+(psi_chi >= 3/2).*obj.psiP(psi_chi, xihat./2, U(j,:));
                    z_eps = (psi_eps < 3/2).*obj.psiM(psi_eps, xihat./2,Wperp(j,:))+(psi_eps >= 3/2).*obj.psiP(psi_eps, xihat./2, Uperp(j,:));
                    
                    chi(j,:) = (z_chi-xihat./2)./beta_vchi;
                    eps = z_eps-xihat./2;
                    u(j,:) = beta_vchi.*chi(j,:) + eps;
                    vf = xihat + u(j,:);
                    
                    idx11 = find(vf <= eps0);
                    if ~isempty(idx11)
                        vf(idx11) = eps0;
                    end
                    
                    dw = (v + vf)/2 .* dt;
                    w  =  w + dw; 
                    y  =  y + chi(j,:);
                    x  =  x - dw./2 + sqrt(dw) .*rho2m1 .* Z(j,:) + rho.* chi(j,:);
                    
                    if (j < steps)
                        btilde = flip(bstar(2:(j+1)));

                        xihat = xij(j + 1) + (btilde * chi(1:j,:));
                    end

                    v = vf;

                end

                resSim.v = v;
                resSim.x = x;
                resSim.y = y;
                resSim.w = w;

                out = resSim;
                
            catch ME
                aaaa = 1.0;
                rethrow(ME)
            end
        end
        
        function out = HQESimHist(obj, params, xi,paths, steps, expiry)
            try
                eta = params.eta;
                lam = params.lam;
                H = params.al - 1/2;
                rho = params.rho;
                rho2m1 = sqrt(1 - rho * rho);
                eps0 = 1e-10;
                %     W = matrix(rnorm(steps * paths), nrow = steps, ncol = paths);
                
                rng('default');
                rng(0);
                
                W = randn(paths/2,steps);W=[W;-W];W=W';
                Wperp = randn(paths/2,steps);Wperp=[Wperp;-Wperp];Wperp=Wperp';
                Z = randn(paths/2,steps);Z=[Z;-Z];Z=Z';
                Un = randn(paths/2,steps);Un=[Un;-Un];Un=Un';
                U = normcdf(Un);
                Uperpn = randn(paths/2,steps);Uperpn=[Uperpn;-Uperpn];Uperpn=Uperpn';
                Uperp = normcdf(Uperpn);
                
                % log S(t)/F(t) 's simulation step for all paths
                xhist = zeros(steps,paths);
                
                G00p = @(t) (arrayfun(@(t) obj.G00(params,t),t));
    
                dt = expiry/steps;
                sqrtDt = sqrt(dt);
                tj = (1:steps) * dt;
                xij = xi(tj);
%                 G00del = obj.G00(params,dt);
                G0del = eta.*obj.G0(params,dt);
                G1del  = eta.*obj.G1(params,dt);
                G00del = eta.^2.*obj.G00(params,dt);
                G11del = eta.^2.*obj.G11(params,dt);
                G01del = eta.^2.*obj.G01(params,dt);
                G00j   = eta.^2.*[0, G00p(tj)];
                bstar  = sqrt(diff(G00j)./dt);
                bstar1 = bstar(1); % # bstar is average g over an interval
                rho_vchi = G0del/sqrt(G00del*dt);
                beta_vchi = G0del/dt;

                u = zeros(steps,paths);
                chi = zeros(steps,paths);
                v = ones(1,paths).* xi(0);
                xihat = ones(1,paths).*xij(1);
                
                x = zeros(1,paths);
                y = zeros(1,paths);
                w = zeros(1,paths);
                vf = zeros(1,paths);
                
                for j =1:steps 
                    xibar = (xihat + 2 .* H.* v)./(1 + 2.* H);
                    var_chi = xibar.* dt;
                    var_eps = xibar.* G00del.*(1-rho_vchi.^2);
                    psi_chi = 4.*G00del.*rho_vchi.^2./xibar;
                    psi_eps = 4.*G00del.*(1-rho_vchi.^2)./xibar;

                    z_chi = (psi_chi < 3/2).*obj.psiM(psi_chi, xihat./2,W(j,:))+(psi_chi >= 3/2).*obj.psiP(psi_chi, xihat./2, U(j,:));
                    z_eps = (psi_eps < 3/2).*obj.psiM(psi_eps, xihat./2,Wperp(j,:))+(psi_eps >= 3/2).*obj.psiP(psi_eps, xihat./2, Uperp(j,:));
                    
                    chi(j,:) = (z_chi-xihat./2)./beta_vchi;
                    eps = z_eps-xihat./2;
                    u(j,:) = beta_vchi.*chi(j,:) + eps;
                    vf = xihat + u(j,:);
                    
                    idx11 = find(vf <= eps0);
                    if ~isempty(idx11)
                        vf(idx11) = eps0;
                    end
                    
                    dw = (v + vf)/2 .* dt;
                    w  =  w + dw; 
                    y  =  y + chi(j,:);
                    x  =  x - dw./2 + sqrt(dw) .*rho2m1 .* Z(j,:) + rho.* chi(j,:);
                    
                    if (j < steps)
                        btilde = flip(bstar(2:(j+1)));

                        xihat = xij(j + 1) + (btilde * chi(1:j,:));
                    end

                    v = vf;
                    
                    xhist(j,:) = x;

                end

                resSim.v = v;
                resSim.x = x;
                resSim.y = y;
                resSim.w = w;
                
                resSim.xhist = xhist;

                out = resSim;
                
            catch ME
                aaaa = 1.0;
                rethrow(ME)
            end
        end
        
    end
end



In [ ]:
%
% Calculating sensitivites of local vol. surface of a EU call using adjoint
% technique
% Use Euler-Maruyama method and Pathwise Sensitivity Method to
% estimate sensitivity of call option to spline points of local vol.
% surface
% Test problem: dS = r*S dt + sig*S dW
%
function [LocalSens,LocalSensConf]=adjoint_mode

    % problem parameters
    r = 0.05;
    sig0 = 0.5;
    T = 1;
    S0 = 100;
    K = 110;
    %parameters for local vol grid
    J = 20;
    I = 20; %I < N
    % Monte Carlo simulation parameters
    M = 1e+6; % total number of Monte Carlo paths
    M2 = 1e+4; % number of paths at a time
    N = 100;
    h = T/N;
    % local vol surface grid setup
    Smin = 0.8*S0;
    Smax = 1.2*S0;
    dS = (Smax - Smin)/J;
    dT = T/I;
    %set up our testing local vol surface
    LocalVolSurf = ones(I+1,J+1)*sig0;
    
    for i=1:I+1
        LocalVolSurf(i,:) = LocalVolSurf(i,:)*((I+1)*((i^(-1)+i^(-0.5))/2)/(I+1));
    end
    
    %set up variable for storing sensitivities
    LocalSens = zeros(I+1,J+1); %sensitivities
    LocalSensConf = zeros(I+1,J+1); %confidence interval for MC sampling
    %iIndex - integer from interval [0,I]
    %beta - time interpolation parameter
    iIndex = floor([1:N]*h./dT);
    beta = ([1:N].*h-iIndex([1:N]).*dT)./dT;
    
    iIndex = [0,iIndex];
    beta = [0,beta];
    %setting up interpolation over time
    sigJ = zeros(N+1,J+1);
    for n=0:N
        sigJ(n+1,:) = LocalVolSurf(1+iIndex(1+n),:) + beta(1+n) * (...
        LocalVolSurf(min(1+iIndex(1+n)+1,I+1),:) - LocalVolSurf(1+iIndex(1+n),:));
    end
    
    %main MC sampling loop
    for m = 1:M2:M
        m2 = min(M2,M-m+1);
        S = S0*ones(1,m2);
        D = zeros(N,m2);
        B = zeros(N,m2);
        alpha = zeros(N+1,m2);
        jIndex = zeros(N+1,m2);
        A = zeros(N+1,m2);
        for n = 1:N
            dW = sqrt(h)*randn(1,m2);
            %jIndex - integer from interval [0,J] - indicates which block S is in
            jIndex(n,:) = max( min(J,floor(real((S-Smin)./dS))) ,0);
            %alpha - stock price interpolation parameter
            alpha(n,:) = (S - (Smin + jIndex(n,:).*dS) )./dS;
            %linear interpolation in stock price
            sig = sigJ(n,1+jIndex(n,:)) + ...
            alpha(n,:).*(sigJ(n,min(J+1,1+jIndex(n,:)+1)) - sigJ(n,1+jIndex(n,:)) );
            %storing necessary variables
            D(n,:) = 1+r*h+sig.*dW;
            A(n,:) = (1/dS).*(sigJ(n,min(J+1,1+jIndex(n,:)+1)) - ...
            sigJ(n,1+jIndex(n,:)) );
            B(n,:) = S.*dW;
            %timestep
            S = S.*D(n,:);
        end

        jIndex(N+1,:) = max( min(J,floor(real((S-Smin)./dS))) ,0);
        alpha(N+1,:) = (S - (Smin + jIndex(N+1,:).*dS) )./dS;

        %--------------------------ADJOINT METHOD-----------------------------%
        %setting Sbar(N) - recursion for calculating dP/dS0
        Sbar = zeros(1,m2);
        for i = 1:m2
            if(S(i)>K)
                Sbar(i) = exp(-r*T);
            else
                Sbar(i) = 0;
            end
        end

        sigbarJ = zeros(N+1,J+1,m2);
        sigbarIJ = zeros(I+1,J+1,m2);
        for n=0:(N-1)
            %recurrence for sigbar, Sbar
            sigbar = Sbar.*B(N-n,:);
            Sbar = (D(N-n,:)+B(N-n,:).*A(N-n,:)).*Sbar;
            %sigbarJs
            for k=1:m2
                j = 1 + jIndex(N-n,k);%in [1,J+1]
                if(j<J+1)
                    sigbarJ(N-n,j,k)= (1-alpha(N-n,k))*sigbar(k);
                    sigbarJ(N-n,j+1,k) = alpha(N-n,k)*sigbar(k);
                else
                    sigbarJ(N-n,j,k) = sigbar(k);
                end
            end

            %sigbarIJs
            for k=1:m2
                i = 1 + iIndex(N-n);
                j = 1+jIndex(N-n,k);
                sigbarIJ(i,j,k) = sigbarIJ(i,j,k) + (1-beta(N-n)).*sigbarJ(N-n,j,k);
                if(j<J+1)
                    sigbarIJ(i,j+1,k) = ...
                    sigbarIJ(i,j+1,k) + (1-beta(N-n)).*sigbarJ(N-n,j+1,k);
                    if(i<I+1)
                        sigbarIJ(i+1,j+1,k) = ...
                        sigbarIJ(i+1,j+1,k) + beta(N-n).*sigbarJ(N-n,j+1,k);
                    end
                end

                if(i<I+1)
                    sigbarIJ(i+1,j,k) = ...
                    sigbarIJ(i+1,j,k) + beta(N-n).*sigbarJ(N-n,j,k);
                end
            end
        end
        
        LocalSens = LocalSens + sum(sigbarIJ,3);
        LocalSensConf = LocalSensConf + sum(sigbarIJ.^2,3);
        
    end
    
    LocalSens = LocalSens./M;
    LocalSensConf = (LocalSensConf./M - LocalSens.^2);
    LocalSensConf = 3*sqrt((M/(M-1)) .* LocalSensConf)./sqrt(M);

end


In [ ]:
import pandas as pd
import requests
from lxml import html
from tqdm import tqdm

In [ ]:
# 삼성전자
sample_code = '005930'

In [ ]:
# parsing URL
# 우리 컴퓨터 -> [접속] -> 에프앤가이드(Data Source) -> [크롤링/웹 스크래핑] -> 우리 컴퓨터
# client -> request -> [Server] -> response -> client

SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'


In [ ]:
# object -> 데이터 덩어리(추상화)

snap_url = SNAP_URL.format(sample_code)
snap_content = requests.get(snap_url).content # 문자열 binary
snap_tree = html.fromstring(snap_content) # 객체(object)
per = snap_tree.xpath('//*[@id="corp_group2"]/dl[1]/dd')[0].text
per = float(per)

In [ ]:
per

8.15

In [ ]:
ratio_url = RATIO_URL.format(sample_code)
ratio_content = requests.get(ratio_url).content
ratio_tree = html.fromstring(ratio_content)
debt_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
debt_ratio = float(debt_ratio)

In [ ]:
# stockMkt => KOSPI
# kosdaqMkt => KOSDAQ
# konexMkt => KONEX

In [ ]:
def get_stock_list(market):
    market_code = ''
    if market == 'kospi':
        market_code = 'stockMkt'
    elif market == 'kosdaq':
        market_code = 'kosdaqMkt'
    elif market == 'konex':
        market_code = 'konexMkt'
    kind_url = 'https://kind.krx.co.kr/corpgeneral/corpList.do?method=download&pageIndex=1&currentPageSize=3000&comAbbrv=&beginIndex=&orderMode=3&orderStat=D&isurCd=&repIsuSrtCd=&searchCodeType=&marketType={}&searchType=13&industry=&fiscalYearEnd=all&comAbbrvTmp=&location=all'.format(market_code)                                                                 

    return pd.read_html(kind_url, converters={'종목코드':lambda x: str(x)})[0]


In [ ]:
def converter(x):
    return str(x)

# 람다 함수 == 무명(anonymous) 함수 == 일회용 함수
lambda x: str(x)

<function __main__.<lambda>(x)>

In [ ]:
kospi_df = get_stock_list('kospi')
print(kospi_df.shape)

(829, 9)


In [ ]:
kosdaq_df = get_stock_list('kosdaq')
print(kosdaq_df.shape)

(1632, 9)


In [ ]:
# merge -> SQL Join
# append | kospi_df.append([kosdaq_df])
# concatenate(합치다)
stock_list_df = pd.concat([kospi_df, kosdaq_df] )

In [ ]:
print(stock_list_df.shape)

(2461, 9)


In [ ]:
# stock_list_df['종목코드'].dropna()
stock_list_df = stock_list_df[stock_list_df['종목코드'].notnull()]

In [ ]:
stock_list_df = stock_list_df[~stock_list_df['회사명'].str.contains('스팩|리츠')]
print(stock_list_df.shape)

(2364, 9)


In [ ]:
# list comprehension
stock_list_df.index = [x for x in range(len(stock_list_df))]

In [ ]:
stock_list_df.to_csv('kospi_kosdaq_stock_list.csv', encoding='utf-8', index=True)

In [ ]:
code_list = stock_list_df['종목코드']
code_list

0       100090
1       453340
2       452260
3       450140
4       377740
         ...  
2359    013030
2360    019550
2361    019570
2362    019590
2363    006920
Name: 종목코드, Length: 2364, dtype: object

In [ ]:
sample_df = pd.DataFrame(
    {'005930':['삼성전자', 1, 2], '035720':['카카오', 1, 2], '015720':['카카오', 1, 2], '025720':['카카오', 1, 2]}
).transpose()



In [ ]:
sample_df.columns = ['name', 'PER', 'Debt_ratio']

In [ ]:
sample_df

,name,PER,Debt_ratio
005930,삼성전자,1,2
035720,카카오,1,2
015720,카카오,1,2
025720,카카오,1,2


In [ ]:
def FinanceInfoCrawler(li, df):
    result_dict = {}
    error_codes = []
    
    for code in tqdm(li):
        try:
            # Parsing URL setting
            SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
            RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'

            # company name
            company_name = df[df['종목코드'] == code]['회사명'].values[0]

            # Get PER
            snap_url = SNAP_URL.format(code)
            snap_content = requests.get(snap_url).content
            snap_tree = html.fromstring(snap_content)
            per = snap_tree.xpath('//*[@id="corp_group2"]/dl[1]/dd')[0].text
            per = float(per)

            # Get Debt ratio
            ratio_url = RATIO_URL.format(code)
            ratio_content = requests.get(ratio_url).content
            ratio_tree = html.fromstring(ratio_content)
            debt_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
            debt_ratio = float(debt_ratio)
            
            result_dict[code] = [company_name, per, debt_ratio]
            
        except (TypeError, IndexError, AttributeError, ValueError):
            pass
#             print(code)
            error_codes.append(code)
    
    # convert dict to DataFrame
    result_df = pd.DataFrame(result_dict)
    
    # transpose DataFrame
    result_df = result_df.transpose()
    
    # Setting column names
    result_df.columns = ['Name', 'PER', 'Debt_ratio']
    
    return result_df, error_codes

In [ ]:
crawling_result_df = FinanceInfoCrawler(code_list[:50],stock_list_df)

100%|██████████| 50/50 [00:46<00:00,  1.07it/s]


In [ ]:
print(crawling_result_df[0].shape)
crawling_result_df[0].head()

(39, 3)


,Name,PER,Debt_ratio
100090,SK오션플랜트,38.11,132.0
377740,바이오노트,1.89,9.8
446070,유니드비티플러스,104.18,11.2
108320,LX세미콘,7.5,35.7
126720,수산인더스트리,6.3,24.7


In [ ]:
# original data
# crawling_result_df

# copy data
copy_df = crawling_result_df[0].copy()

In [ ]:
# PER 10 이하
# 부채비율 50 이하
# (상위) 20개 종목

final_result_df = copy_df[
    (copy_df['PER'] <= 10)&(copy_df['Debt_ratio'] <= 50)&(copy_df['PER'] > 0)
].sort_values(
    by='PER', ascending=True
).iloc[:20]


In [ ]:
import datetime

# 시간까지 포함한 날짜
now = datetime.datetime.now()

final_result_df.to_csv('LowPER_LowDR_{}.csv'.format(now.strftime('%Y%m%d')))

In [ ]:
pd.read_csv('LowPER_LowDR_{}.csv'.format(now.strftime('%Y%m%d')))

,Unnamed: 0,Name,PER,Debt_ratio
0,377740,바이오노트,1.89,9.8
1,137310,에스디바이오센서,2.39,10.7
2,383800,LX홀딩스,3.90,1.5
3,353200,대덕전자,6.16,39.4
4,363280,티와이홀딩스,6.17,47.6
5,126720,수산인더스트리,6.30,24.7
6,108320,LX세미콘,7.50,35.7
